In [7]:
import pandas as pd
import numpy as np
from langdetect import detect
import re

from openpyxl import Workbook

import os
import time
import datetime

In [8]:
from bs4 import BeautifulSoup as bs
import json

In [9]:
import glob

In [10]:
# Python3 program to Grouping list 
# elements based on frequency
from collections import Counter
import itertools

In [11]:
from googletrans import Translator, constants

In [28]:
# streamlit app modules
import streamlit as st 
import sklearn
import plotly
import matplotlib
import streamlit_option_menu
import streamlit_pills

In [30]:
print('--Python module versions')

print(f'pandas=={pd.__version__}')
print(f'numpy=={np.__version__}')
print(f're=={re.__version__}')
print(f'json=={json.__version__}')

print('--Streamlit app module versions')

print(f'streamlit=={st.__version__}')
print(f'sklearn=={sklearn.__version__}')
print(f'plotly=={plotly.__version__}')
print(f'matplotlib=={matplotlib.__version__}')

--Python module versions
pandas==2.1.2
numpy==1.26.2
re==2.2.1
json==2.0.9
--Streamlit app module versions
streamlit==1.31.0
sklearn==1.3.2
plotly==5.16.1
matplotlib==3.8.2


In [6]:
# file = "data/data_00.txt"

# os.path.getctime("data/data_00.txt")
# # datetime.date(time.ctime(os.path.getctime(file)))
# # time.ctime(os.path.getctime(file))
# datetime.datetime.fromtimestamp(os.path.getctime("data/data_00.txt")).strftime('%Y-%m-%d')

'2023-11-29'

In [7]:
def diveDictValues(keyword = None, dict_to_search = {}):
    if keyword is None:
        return None
    if dict_to_search is False:
        return None
    
    for key in dict_to_search.keys():
        try:
            if str(dict_to_search[key]).__contains__(keyword):
                return key
        except KeyError:
            continue

    return None

def diveListValues(keyword = None, list_to_search = []):
    if keyword is None:
        return None
    if list_to_search is False:
        return None
    
    matched_items = []
    for i in range(len(list_to_search)):
        if type(list_to_search[i]) == dict:
            if keyword in list(list_to_search[i].keys()):
                matched_items.append(i)
        elif str(list_to_search[i]).__contains__(keyword):
            matched_items.append(i)

    return matched_items

def ProcessHtmlText(dict_to_search = None, dict_labels = []):
    if dict_to_search is None:
        return None
    
    curr_dict_level = dict_to_search
    result = None

    try:
        for i in range(len(dict_labels)):
            label = dict_labels[i]
            if label.__contains__('list'):
                if type(curr_dict_level) == list:
                    dive_result = diveListValues(label.split('-')[1], curr_dict_level)
                    if dive_result is None:
                        continue
                    elif len(dive_result) <= 0:
                        continue
                    elif len(dive_result) > 1:
                        possible_nests_to_check = {}
                        for j in dive_result:
                            possible_nests_to_check[j] = curr_dict_level[j]
                        accepted_nests = AttemptPossibleNest(possible_nests_to_check, dict_labels[i+1:])
                        dive_result = accepted_nests[-1]
                    elif len(dive_result) == 1:
                        dive_result = dive_result[0]
 
                    curr_dict_level = curr_dict_level[dive_result]
            else:
                if type(curr_dict_level) == dict:
                    try:
                        # dive_result = diveDictValues(label, curr_dict_level)
                        # if dive_result is None:
                        #     continue
                        
                        # print(f"label: {label}; result: {dive_result}")
                        curr_dict_level = curr_dict_level[label]
                    except KeyError:
                        continue

            if label == dict_labels[-1]:
                result = curr_dict_level
    except ValueError:
        return None

    return result

def AttemptPossibleNest(possible_nest_dict = {}, next_labels = []):
    accepted_nests = []
    for key in possible_nest_dict.keys():
        nest_to_dive = possible_nest_dict[key]
        
        # print('nesting', key, next_labels, nest_to_dive)
        result_check = ProcessHtmlText(nest_to_dive, dict_labels=next_labels)
        if result_check is not None:
            accepted_nests.append(key)
    
    return accepted_nests

def diveHtmlTags(html_body, toplevel_tag = None, dict_labels = [], multi = False, find_keyword = None):
    soup = bs(html_body, 'lxml')
    if toplevel_tag is None:
        return None
    if dict_labels is False:
        return None
    
    ids = [ids['id'] for ids in soup.find_all(toplevel_tag, id=True)]
    
    result = None
    if multi == False:
        for id_key in ids:
            long_text = soup.find(id = id_key).text
            try:
                json_dict = json.loads(long_text, strict = False)

                if json_dict is None:
                    continue
                
                result_check = ProcessHtmlText(json_dict, dict_labels=dict_labels)
                if result_check is not None:
                    if find_keyword is None:
                        result = result_check
                    elif result_check.__contains__(find_keyword):
                        # print('Found "employees"')
                        # print(result_check)
                        # print('--')
                        result = result_check
                    else:
                        result = result_check     


            except ValueError:  # includes simplejson.decoder.JSONDecodeError
                # print(f'Decoding JSON has failed at {id_key}')
                continue
    else:
        result = []
        for id_key in ids:
            long_text = soup.find(id = id_key).text
            try:
                json_dict = json.loads(long_text, strict = False)

                if json_dict is None:
                    continue
                
                result_check = ProcessHtmlText(json_dict, dict_labels=dict_labels)
                if result_check is not None:
                    print('rc')
                    print(f"id: {id_key}")
                    print(result_check)
                    print('--')
                    result.append(result_check)
            except ValueError:  # includes simplejson.decoder.JSONDecodeError
                print(f'Decoding JSON has failed at {id_key}')
                continue

    return result


In [8]:
def GetJobPostingID(html_body):
    try:
        # soup = bs(html_body)
        # # id: bpr-guid-15912216
        # id_key = 'bpr-guid-15912216'
        # long_text = soup.find(id = id_key).text

        # json_dict = json.loads(long_text, strict = False)
        # return json_dict['data']['jobPostingId']
        return diveHtmlTags(html_body, toplevel_tag='code', dict_labels = ['data', 'jobPostingId'])
    except:
        return None

def GetPostStatus(html_body):
    try:
        # soup = bs(html_body)
        # # id: bpr-guid-15912216
        # id_key = 'bpr-guid-15912216'
        # long_text = soup.find(id = id_key).text

        # json_dict = json.loads(long_text, strict = False)
        # return json_dict['data']['jobState']
        return diveHtmlTags(html_body, toplevel_tag='code', dict_labels = ['data', 'jobState'])
    except:
        return None

def GetPostLink(url_link = None, job_id = None):
    try:
        if url_link is not None:
            return url_link
        if job_id is not None:
            return f'https://www.linkedin.com/jobs/view/{job_id}/'
    except:
        return None
    
    return None

def GetJobTitle(html_body):
    try:
        # soup = bs(html_body)
        # # id: bpr-guid-15912216
        # id_key = 'bpr-guid-15912216'
        # long_text = soup.find(id = id_key).text

        # json_dict = json.loads(long_text, strict = False)
        # return json_dict['data']['title']
        return diveHtmlTags(html_body, toplevel_tag='code', dict_labels = ['data', 'title'])
    except:
        return None

def GetJobDescription(html_body):
    try:
        # soup = bs(html_body)
        # # id: bpr-guid-15912216
        # id_key = 'bpr-guid-15912216'
        # long_text = soup.find(id = id_key).text

        # json_dict = json.loads(long_text, strict = False)
        # return json_dict['data']['description']['text']
        return diveHtmlTags(html_body, toplevel_tag='code', dict_labels = ['data', 'description', 'text'])
    except:
        return None

def GetDescriptionLanguage(jd_text):
    try:
        return detect(jd_text)
    except:
        return ''

def GetRemoteWorkStatus(html_body):
    try:
        # soup = bs(html_body)
        # # id: bpr-guid-15912216
        # id_key = 'bpr-guid-15912216'
        # long_text = soup.find(id = id_key).text

        # json_dict = json.loads(long_text, strict = False)
        # return json_dict['data']['workRemoteAllowed']
        return diveHtmlTags(html_body, toplevel_tag='code', dict_labels = ['data', 'workRemoteAllowed'])
    except:
        return None

def GetRepostStatus(html_body):
    try:
        # soup = bs(html_body)
        # # id: bpr-guid-15912216
        # id_key = 'bpr-guid-15912216'
        # long_text = soup.find(id = id_key).text

        # json_dict = json.loads(long_text, strict = False)
        # return json_dict['data']['repostedJobPosting']
        return diveHtmlTags(html_body, toplevel_tag='code', dict_labels = ['data', 'repostedJobPosting'])
    except:
        return None

def GetJobCountry(html_body):
    try:
        # soup = bs(html_body)
        # # id: bpr-guid-15912216
        # id_key = 'bpr-guid-15912216'
        # long_text = soup.find(id = id_key).text

        # json_dict = json.loads(long_text, strict = False)
        # return json_dict['data']['formattedLocation']
        return diveHtmlTags(html_body, toplevel_tag='code', dict_labels = ['data', 'formattedLocation'])
    except:
        return None

def GetJobTopCardDetails(html_body):
    try:
        # soup = bs(html_body)
        # # id: bpr-guid-15912217
        # id_key = 'bpr-guid-15912217'
        # long_text = soup.find(id = id_key)
        # json_dict = json.loads(long_text.text, strict = False)
        # return [t['text']['text'] for t in json_dict['included'][4]['jobInsightsV2ResolutionResults'][0]['jobInsightViewModel']['description']]
        dive_result = diveHtmlTags(html_body, toplevel_tag='code', dict_labels = ['included', 'list-jobInsightsV2ResolutionResults', 'jobInsightsV2ResolutionResults','list-jobInsightViewModel', 'jobInsightViewModel', 'description'])
        # print('dr type', type(dive_result))
        # print(dive_result.keys())
        # print(dive_result)
        # print([t.keys() for t in dive_result])
        return [t['text']['text'] for t in dive_result]
    except:
        return None

def GetJobExperienceLevel(html_body, details = []):
    try:
        if details is True:
            for keyword in ['Internship', 'Entry', 'Associate', 'Mid-Senior', 'Director', 'Executive']:
                for detail in details:
                    if keyword in detail:
                        return detail
                    
        details = GetJobTopCardDetails(html_body)
        for keyword in ['Internship', 'Entry', 'Associate', 'Mid-Senior', 'Director', 'Executive']:
            for detail in details:
                if keyword in detail:
                    return detail
    except:
        return None
    
    return None

def GetJobWorkSetting(html_body, details = []):
    try:
        if details is True:
            for keyword in ['Hybrid', 'Remote', 'On-site']:
                for detail in details:
                    if keyword in detail:
                        return detail
                    
        details = GetJobTopCardDetails(html_body)
        for keyword in ['Hybrid', 'Remote', 'On-site']:
            for detail in details:
                if keyword in detail:
                    return detail
    except:
        return None
    
    return None

def GetJobType(html_body, details = []):
    try:
        if details is True:
            for keyword in ['Full-time', 'Part-time', 'Contract', 'Temporary', 'Volunteer', 'Internship', 'Other']:
                for detail in details:
                    if keyword in detail:
                        return detail
                    
        details = GetJobTopCardDetails(html_body)
        for keyword in ['Full-time', 'Part-time', 'Contract', 'Temporary', 'Volunteer', 'Internship', 'Other']:
            for detail in details:
                if keyword in detail:
                    return detail
    except:
        return None
    
    return None

def GetJobSkills(html_body):
    try:
        soup = bs(html_body)

        return [li.find('div').get_text().strip() for li in soup.find('ul', {'class': 'job-details-skill-match-status-list'}).find_all('li')]
    except:
        return None

def GetJobYearsExperience(html_body, job_des = None, des_lan = 'en'):
    try:
        def GetYears_RegEx(des):
            years_text = re.findall(r"([\d+-]+)\s+(years?)", des, re.IGNORECASE)
            splitted_num = []
            for clipped in years_text:
                char_to_remove = ['+', '-', ' ']
                text_to_clean = clipped[0]

                for ch in char_to_remove:
                    for el in text_to_clean.split(ch):
                        if el.isalnum() is True:
                            splitted_num = splitted_num + [int(el)]

            return np.min(list(set(splitted_num)))

        if des_lan != 'en':
            translator = Translator()
            translation_dest = 'en'
            translation = translator.translate(job_des, dest=translation_dest)
            
            job_des = translation.text

        if job_des is True:
            return GetYears_RegEx(job_des)
        
        job_des = GetJobDescription(html_body)
        if job_des is True:
            return GetYears_RegEx(job_des)
    except:
        return None

    return None

def GetCompanyID(html_body):
    try:
        # soup = bs(html_body)
        # # id: bpr-guid-15912216
        # id_key = 'bpr-guid-15912216'
        # long_text = soup.find(id = id_key).text

        # json_dict = json.loads(long_text, strict = False)
        # return json_dict['data']['companyDetails']['company']
        company_text = diveHtmlTags(html_body, toplevel_tag='code', dict_labels = ['data', 'companyDetails', 'company'])
        return company_text.split('urn:li:fs_normalized_company:')[1]
    except:
        return None

def GetCompanyName(html_body):
    try:
        # soup = bs(html_body)
        # # id: bpr-guid-15912216
        # id_key = 'bpr-guid-15912216'
        # long_text = soup.find(id = id_key).text

        # json_dict = json.loads(long_text, strict = False)
        # return json_dict['included'][1]['name']
        company_name = diveHtmlTags(html_body, toplevel_tag='code', dict_labels = ['included', 'list-name', 'name'])
        if type(company_name) == str:
            return company_name
        
        return None
    except:
        return None

def GetCompanyLink(html_body):
    try:
        # soup = bs(html_body)
        # # id: bpr-guid-15912216
        # id_key = 'bpr-guid-15912216'
        # long_text = soup.find(id = id_key).text

        # json_dict = json.loads(long_text, strict = False)
        # return json_dict['included'][1]['url']
        company_link = diveHtmlTags(html_body, toplevel_tag='code', dict_labels = ['included', 'list-url', 'url', 'list-primaryDescription', 'primaryDescription', 'attributesV2', 'list-detailData', 'detailData', 'hyperlink'])
        if type(company_link) == str:
            return company_link
        
        return None
    except:
        return None
    
def GetCompanyTopCardDetails(html_body):
    soup = bs(html_body, 'lxml')
    # id: bpr-guid-15912217
    # id_key = 'bpr-guid-15912217'
    # long_text = soup.find(id = id_key).text

    # json_dict = json.loads(long_text, strict = False)
    # return json_dict['included'][4]['jobInsightsV2ResolutionResults'][1]['insightViewModel']['text']['text'].split(' · ')
    # dive_result = diveHtmlTags(html_body, toplevel_tag='code', dict_labels = ['included', 'list-jobInsightsV2ResolutionResults', 'jobInsightsV2ResolutionResults','list-insightViewModel', 'insightViewModel', 'text', 'text'], multi = True)
    # return dive_result
    try:
        dive_result = diveHtmlTags(html_body, toplevel_tag='code', dict_labels = ['included', 'list-jobInsightsV2ResolutionResults', 'jobInsightsV2ResolutionResults', 'list-insightViewModel', 'insightViewModel', 'text', 'text'], find_keyword = 'employees')
        if type(dive_result) == list:
            if dive_result[0].__contains__('works here') or dive_result[0].__contains__('work here'):
                return None
            elif dive_result[0].__contains__('hiring for this job'):
                return None
            elif dive_result[0].__contains__('Actively recruiting'):
                return None
        elif type(dive_result) == str:
            if dive_result.__contains__('works here') or dive_result.__contains__('work here'):
                return None
            elif dive_result.__contains__('hiring for this job'):
                return None
            elif dive_result.__contains__('Actively recruiting'):
                return None
            
        if type(dive_result) == str:
            if dive_result.__contains__('employees') == False:
                return None
        
        return dive_result.split(' · ')
    except:
        return None
    # for result in dive_result:
    #     print('r')
    #     print(result)
    #     print('--')
    #     if str.lower(result).__contains__('employee'):
    #         dive_result = result

    # return dive_result.split(' · ')

def GetCompanySize(html_body, details = []):
    try:
        if details is True:
            for detail in details:
                if 'employees' in detail:
                    return detail
                    
        details = GetCompanyTopCardDetails(html_body)
        for detail in details:
            if 'employees' in detail:
                return detail
    except:
        return None
    
    return None

def GetCompanyIndustry(html_body, details = []):
    try:
        if details is True:
            for detail in details:
                if 'employees' not in detail:
                    return detail
                    
        details = GetCompanyTopCardDetails(html_body)
        for detail in details:
            if 'employees' not in detail:
                return detail
    except:
        return None
    
    return None

def GetCompanyDescription(html_body):
    try:
        soup = bs(html_body)

        long_text = soup.find('div', {'class': 'jobs-company__box'}).find('div', {'class': 'inline-show-more-text inline-show-more-text--is-collapsed inline-show-more-text--is-collapsed-with-line-clamp'}).text.strip()
        return long_text
    except:
        return None

In [9]:
def get_html_from_file(file_path):
    try:
        f = open(file_path, "r")
        file_text = f.read()

        return file_text
    except:
        print(f'invalid file path: {file_path}')

def read_files(files_df = pd.DataFrame([], columns = ['Job ID', 'page_path', 'skills_path', 'page_file_date', 'skills_file_date'])):
    job_files = []
    for file in glob.glob("data/*.txt"):
        job_files.append(file)

        job_id = 0
        f = open(file, "r")
        file_text = f.read()
        job_id = GetJobPostingID(file_text)
        if job_id == None:
            print(f'id not found in {file}')
            continue

        if job_id not in files_df['Job ID'].values:
            file_entry = {'Job ID':job_id, 'page_path':'', 'skills_path':''}
            files_df = pd.concat([files_df, pd.DataFrame([file_entry])], ignore_index=True)
            files_df.reset_index()

        if file.__contains__('skills.txt'):
            file_row = files_df[files_df['Job ID'] == job_id]
            file_row['skills_path'] = file
            # datetime.datetime.fromtimestamp(os.path.getctime("data/data_00.txt")).strftime('%Y-%m-%d')
            file_row['skills_file_date'] = os.path.getctime(file)

            files_df[files_df['Job ID'] == job_id] = file_row
        else:
            file_row = files_df[files_df['Job ID'] == job_id]
            file_row['page_path'] = file
            file_row['page_file_date'] = os.path.getctime(file)

            files_df[files_df['Job ID'] == job_id] = file_row
    
    return files_df

In [10]:
# diveHtmlTags(html_body_test, toplevel_tag='code', dict_labels = ['included', 'list-jobInsightsV2ResolutionResults', 'jobInsightsV2ResolutionResults', 'list-insightViewModel', 'insightViewModel', 'text', 'text'], find_keyword = 'employees')

In [11]:
files_df_columns = ['Job ID', 'page_path', 'skills_path', 'page_file_date', 'skills_file_date']
files_df = pd.DataFrame([], columns = files_df_columns)

files_df = read_files(files_df)

C:\Users\naimi\AppData\Local\Temp\ipykernel_26372\1813570446.py:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  file_row['page_path'] = file
C:\Users\naimi\AppData\Local\Temp\ipykernel_26372\1813570446.py:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  file_row['page_file_date'] = os.path.getctime(file)
C:\Users\naimi\AppData\Local\Temp\ipykernel_26372\1813570446.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = v

id not found in data\data_157.txt
id not found in data\data_157_skills.txt
id not found in data\data_158.txt
id not found in data\data_158_skills.txt


C:\Users\naimi\AppData\Local\Temp\ipykernel_26372\1813570446.py:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  file_row['page_path'] = file
C:\Users\naimi\AppData\Local\Temp\ipykernel_26372\1813570446.py:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  file_row['page_file_date'] = os.path.getctime(file)
C:\Users\naimi\AppData\Local\Temp\ipykernel_26372\1813570446.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = v

In [12]:
# body_text = html_body_test
# html_skills_matched_html = ''

def GetJobDetails(html_body, html_skills_body):
    posting_id = GetJobPostingID(html_body)
    post_status = GetPostStatus(html_body)
    job_title = GetJobTitle(html_body)
    job_description = GetJobDescription(html_body)
    desc_lan = GetDescriptionLanguage(GetJobDescription(html_body))
    remote_status = GetRemoteWorkStatus(html_body)
    repost_status = GetRepostStatus(html_body)
    job_country = GetJobCountry(html_body)
    company_id = GetCompanyID(html_body)
    job_topcard_details = GetJobTopCardDetails(html_body)
    company_topcard_details = GetCompanyTopCardDetails(html_body)
    # company_topcard_details = []
    job_skills = GetJobSkills(html_skills_body)

    dict_job = {'Job ID': GetJobPostingID(html_body),
                'Post Date': '',
                'Status': GetPostStatus(html_body),
                'Job Link': GetPostLink(url_link=None, job_id=GetJobPostingID(html_body)),
                'Job Title': GetJobTitle(html_body),
                'Job Description': GetJobDescription(html_body),
                'Post Language': GetDescriptionLanguage(GetJobDescription(html_body)),
                'Remote': GetRemoteWorkStatus(html_body),
                'Reposted': GetRepostStatus(html_body),
                'Top Card Details': GetJobTopCardDetails(html_body),
                'Experience Level': GetJobExperienceLevel(html_body, details = job_topcard_details),
                'Experience in Years': GetJobYearsExperience(html_body, job_description, desc_lan),
                'Work Setting': GetJobWorkSetting(html_body, details = job_topcard_details),
                'Job Type': GetJobType(html_body, details = job_topcard_details),
                'Job Country': GetJobCountry(html_body),
                'Job Skills': GetJobSkills(html_skills_body),
                'Company ID': GetCompanyID(html_body),
                'Company Name': GetCompanyName(html_body),
                'Company Link': GetCompanyLink(html_body),
                'Company Card Details': GetCompanyTopCardDetails(html_body),
                'Company Size': GetCompanySize(html_body, details = company_topcard_details),
                'Industry': GetCompanyIndustry(html_body, details = company_topcard_details),
                'Company Description': GetCompanyDescription(html_body)
                }
    return dict_job

df_columns = ['Job ID', 'Post Date', 'Status', 'Job Link', 'Job Title', 'Job Description', 'Post Language', 'Remote', 'Reposted', 'Top Card Details', 'Experience Level', 'Experience in Years', 'Work Setting', 'Job Type', 'Job Country', 'Job Skills', 'Company ID', 'Company Name', 'Company Link', 'Company Card Details', 'Company Size', 'Industry', 'Company Description']

df_jobs = pd.DataFrame([], columns = df_columns)

for index in files_df.index:
    html_body = get_html_from_file(files_df.loc[index]['page_path'])
    html_skills = get_html_from_file(files_df.loc[index]['skills_path'])

    job_entry = GetJobDetails(html_body, html_skills)
    job_entry['Post Date'] = list(files_df[files_df['Job ID'] == job_entry['Job ID']]['page_file_date'])[0]
    df_jobs = pd.concat([df_jobs, pd.DataFrame([job_entry])], ignore_index=True)
    df_jobs.reset_index()


Found "employees"
501-1,000 employees · Software Development
--
Found "employees"
501-1,000 employees · Software Development
--
Found "employees"
501-1,000 employees · Software Development
--
Found "employees"
501-1,000 employees · Software Development
--


C:\Users\naimi\AppData\Local\Temp\ipykernel_26372\4113962444.py:55: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_jobs = pd.concat([df_jobs, pd.DataFrame([job_entry])], ignore_index=True)


Found "employees"
51-200 employees · Climate Data and Analytics
--
Found "employees"
51-200 employees · Climate Data and Analytics
--
Found "employees"
51-200 employees · Climate Data and Analytics
--
Found "employees"
51-200 employees · Climate Data and Analytics
--
Found "employees"
1-10 employees · Data Infrastructure and Analytics
--
Found "employees"
1-10 employees · Data Infrastructure and Analytics
--
Found "employees"
1-10 employees · Data Infrastructure and Analytics
--
Found "employees"
1-10 employees · Data Infrastructure and Analytics
--
Found "employees"
11-50 employees · Staffing and Recruiting
--
Found "employees"
11-50 employees · Staffing and Recruiting
--
Found "employees"
11-50 employees · Staffing and Recruiting
--
Found "employees"
11-50 employees · Staffing and Recruiting
--
Found "employees"
1,001-5,000 employees · IT Services and IT Consulting
--
Found "employees"
1,001-5,000 employees · IT Services and IT Consulting
--
Found "employees"
1,001-5,000 employees · 

In [13]:
df_jobs

,Job ID,Post Date,Status,Job Link,Job Title,Job Description,Post Language,Remote,Reposted,Top Card Details,...,Job Type,Job Country,Job Skills,Company ID,Company Name,Company Link,Company Card Details,Company Size,Industry,Company Description
0,3772602742,1.701295e+09,LISTED,https://www.linkedin.com/jobs/view/3772602742/,Financial Data Analyst,"About Team\n\nWe are Quadcode, a fintech compa...",en,True,None,"[Remote, Full-time, Mid-Senior level]",...,Full-time,EMEA,"[Analytical Skills, Communication, Attention t...",42345997,Quadcode,https://www.linkedin.com/company/quadcodecaree...,"[501-1,000 employees, Software Development]","501-1,000 employees",Software Development,Quadcode is a fintech company specializing in ...
1,3768710281,1.701296e+09,LISTED,https://www.linkedin.com/jobs/view/3768710281/,Consultant in Data Analysis and Scientific Pro...,Your Responsibilities\n\nWe are currently look...,en,False,None,"[On-site, Full-time, Mid-Senior level]",...,Full-time,"Berlin, Berlin, Germany","[Communication, Data Analysis, Data Analytics,...",10198832,Climate Analytics,https://www.linkedin.com/company/climate-analy...,"[51-200 employees, Climate Data and Analytics]",51-200 employees,Climate Data and Analytics,Climate Analytics is a global climate science ...
2,3773349346,1.701296e+09,LISTED,https://www.linkedin.com/jobs/view/3773349346/,Data-Analyst (m/w/d) in SaaS-Startup,"Wir sind cinify, ein junges Start-up, dass den...",de,True,None,"[Remote, Full-time, Entry level]",...,Full-time,Germany,"[Business Analysis, Dashboards, Data Analytics...",88277276,cinify,https://www.linkedin.com/company/cinify-io/life,"[1-10 employees, Data Infrastructure and Analy...",1-10 employees,Data Infrastructure and Analytics,cinify wurde Anfang des Jahres 2023 gegründet ...
3,3773351712,1.701296e+09,LISTED,https://www.linkedin.com/jobs/view/3773351712/,"Data Engineer/Analyst - DBT, AWS, Redshift, Py...",Orcan Intelligence are currently partnered wit...,en,False,None,"[On-site, Contract]",...,Contract,"Berlin, Berlin, Germany","[Cloud Infrastructure, Data Engineering, Data ...",9387927,CareerAddict,https://www.linkedin.com/company/career-addict...,"[11-50 employees, Staffing and Recruiting]",11-50 employees,Staffing and Recruiting,CareerAddict is a leading career advice platfo...
4,3775291259,1.701296e+09,LISTED,https://www.linkedin.com/jobs/view/3775291259/,Junior Business Intelligence Consultant (M/W/D),Wir sind die Positive Thinking Company. Wir su...,de,False,None,"[On-site, Full-time, Mid-Senior level]",...,Full-time,"Berlin, Berlin, Germany","[Analytics, Business Intelligence (BI), Data A...",11444799,Positive Thinking Company by CBTW,https://www.linkedin.com/company/the-positive-...,"[1,001-5,000 employees, IT Services and IT Con...","1,001-5,000 employees",IT Services and IT Consulting,𝐀𝐛𝐨𝐮𝐭 𝐏𝐨𝐬𝐢𝐭𝐢𝐯𝐞 𝐓𝐡𝐢𝐧𝐤𝐢𝐧𝐠 𝐂𝐨𝐦𝐩𝐚𝐧𝐲As the technolo...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
384,3610427886,1.701465e+09,LISTED,https://www.linkedin.com/jobs/view/3610427886/,(Senior) Consultant Advanced Analytics (f/m/d),Strategy to Results.\n\nCAMELOT Management Con...,en,True,None,[],...,None,Germany,"[Analytics, Data Analytics, Data Mining, Data ...",89553,None,None,None,None,None,CAMELOT Management Consultants is the globally...
385,3765456123,1.701465e+09,LISTED,https://www.linkedin.com/jobs/view/3765456123/,Biostatistician - 100% remote - 12+ Months,Biostatistician - 100% remote - 12+ Months\nWe...,en,True,None,None,...,None,Germany,"[Biostatistics, Construction, EPCM, Health Tec...",100128089,None,None,None,None,None,Your Partner in Life Sciences Recruitment Acro...
386,3749323014,1.701465e+09,LISTED,https://www.linkedin.com/jobs/view/3749323014/,Quantitative Developer,Imagine a future where you employ your Modern ...,en,False,None,"[On-site, Full-time]",...,Full-time,"Gentofte, Capital Region, Denmark","[Data Science, Programming, Computer Science, ...",18324710,Ørsted,https://www.linkedin.com/company/orsted

In [14]:
for index in df_jobs.index[:5]:
    row = df_jobs.loc[index]

    if row['Company Card Details'] is not None:
        print(row['Company Card Details'])

['501-1,000 employees', 'Software Development']
['51-200 employees', 'Climate Data and Analytics']
['1-10 employees', 'Data Infrastructure and Analytics']
['11-50 employees', 'Staffing and Recruiting']
['1,001-5,000 employees', 'IT Services and IT Consulting']


In [15]:
for index in df_jobs.index:
    row = df_jobs.loc[index]

    if row['Company Card Details'] is not None:
        continue

    company_id = row['Company ID']
    company_details_df = df_jobs[(df_jobs['Company ID'] == company_id) & (df_jobs['Company Name'] is not None)]

    info_index = None
    for cd_index in company_details_df.index:
        if (company_details_df.loc[cd_index]['Company Name'] is not None) & (company_details_df.loc[cd_index]['Company Card Details'] is not None):
            info_index = cd_index
            break
    
    if info_index is None:
        continue

    if row['Company Name'] is None:
        row['Company Name'] = company_details_df.loc[info_index]['Company Name']
    if row['Company Card Details'] is None:
        row['Company Card Details'] = company_details_df.loc[info_index]['Company Card Details']
    if row['Company Size'] is None:
        row['Company Size'] = company_details_df.loc[info_index]['Company Size']
    if row['Industry'] is None:
        row['Industry'] = company_details_df.loc[info_index]['Industry']
    if row['Company Description'] is None:
        row['Company Description'] = company_details_df.loc[info_index]['Company Description']
    
    df_jobs.loc[index] = row

C:\Users\naimi\AppData\Local\Temp\ipykernel_26372\4076228659.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  row['Company Name'] = company_details_df.loc[info_index]['Company Name']
C:\Users\naimi\AppData\Local\Temp\ipykernel_26372\4076228659.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  row['Company Card Details'] = company_details_df.loc[info_index]['Company Card Details']
C:\Users\naimi\AppData\Local\Temp\ipykernel_26372\4076228659.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/

In [16]:
# with pd.ExcelWriter('linkedin_job_details.xlsx') as writer:
#     df_jobs.to_excel(writer)  

df_jobs.to_excel('linkedin_job_details.xlsx')  

In [17]:
top_skills_dict = {}

for keyword_skill in list(set().union(*df_jobs[pd.isna(df_jobs['Job Skills']) == False]['Job Skills'].values)):
    skills = []
    
    if keyword_skill.__contains__('('):
        skills = keyword_skill.split('(')

        skills = [skill.rstrip() for skill in skills]
        skills = [skill.replace(')', '') for skill in skills]
    else:
        skills = [keyword_skill]
    
    for skill in skills:
        if keyword_skill not in top_skills_dict.keys():
            top_skills_dict[keyword_skill] = skills
        else:
            top_skills_dict[keyword_skill] = list(set().union(top_skills_dict[keyword_skill], skills))
    
# top_skills_dict
skills_columns = ['skill name', 'common phrases']

df_top_skills = pd.DataFrame([], columns=skills_columns)

for key in top_skills_dict.keys():
    dict_entry = {}
    dict_entry['skill name'] = key
    dict_entry['common phrases'] = top_skills_dict[key]
    df_top_skills = pd.concat([df_top_skills, pd.DataFrame([dict_entry])], ignore_index=True)
    df_top_skills.reset_index()

df_top_skills

,skill name,common phrases
0,Analytical Skills,[Analytical Skills]
1,Scanning Electron Microscopy (SEM),"[Scanning Electron Microscopy, SEM]"
2,SQL Server Analysis Services (SSAS),"[SSAS, SQL Server Analysis Services]"
3,Algorithms,[Algorithms]
4,Data Analysis,[Data Analysis]
...,...,...
793,Bank Confirmation,[Bank Confirmation]
794,Financial Modeling,[Financial Modeling]
795,Fraud Investigations,[Fraud Investigations]
796,Client-focused,[Client-focused]


In [18]:
# print(df_jobs.columns)
# matched_skills_df = df_jobs.copy()

# matched_skills_df = matched_skills_df[['Job ID', 'Status', 'Job Title', 'Job Skills']]
df_jobs['Associated Skill Phrases'] = ''
df_jobs['Unlisted Skill'] = ''
df_jobs['Unlisted Skill Phrases'] = ''

def get_listed_skill_phrases(row, skills_df):
    associated_skills = []

    if row['Job Skills'] is None:
        return row

    for skill in row['Job Skills']:
        # print(skill)
        # print(df_top_skills[df_top_skills['skill name'] == skill]['common phrases'].values)
        # print(associated_skills)
        associated_skills.append(df_top_skills[df_top_skills['skill name'] == skill]['common phrases'].tolist()[0])
        # associated_skills.append(df_top_skills[df_top_skills['skill name'] == skill]['common phrases'].values[0])
        # associated_skills = associated_skills + df_top_skills[df_top_skills['skill name'] == skill]['common phrases'].values
    row['Associated Skill Phrases'] = list(set().union(*associated_skills))
   
    return row
    
def get_unlisted_skill_phrases(row, skills_df):
    unlisted_skills = []
    unlisted_skill_phrases = []

    for index in skills_df.index:
        if row['Job Skills'] is not None:
            if skills_df.loc[index]['skill name'] in row['Job Skills']:
                continue
        
        for phrase in skills_df.loc[index]['common phrases']:
            if phrase in row['Job Description'].split(' '):
                unlisted_skills.append(skills_df.loc[index]['skill name'])
                unlisted_skill_phrases.append(phrase)
    
    row['Unlisted Skill'] = unlisted_skills
    row['Unlisted Skill Phrases'] = unlisted_skill_phrases

    return row

for index in df_jobs.index:
    skill_row = df_jobs.loc[index]
    df_jobs.loc[index] = get_listed_skill_phrases(skill_row, df_top_skills)
    df_jobs.loc[index] = get_unlisted_skill_phrases(skill_row, df_top_skills)


df_jobs.head(5)

C:\Users\naimi\AppData\Local\Temp\ipykernel_26372\2329831534.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  row['Associated Skill Phrases'] = list(set().union(*associated_skills))
C:\Users\naimi\AppData\Local\Temp\ipykernel_26372\2329831534.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  row['Unlisted Skill'] = unlisted_skills
C:\Users\naimi\AppData\Local\Temp\ipykernel_26372\2329831534.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-v

,Job ID,Post Date,Status,Job Link,Job Title,Job Description,Post Language,Remote,Reposted,Top Card Details,...,Company ID,Company Name,Company Link,Company Card Details,Company Size,Industry,Company Description,Associated Skill Phrases,Unlisted Skill,Unlisted Skill Phrases
0,3772602742,1.701295e+09,LISTED,https://www.linkedin.com/jobs/view/3772602742/,Financial Data Analyst,"About Team\n\nWe are Quadcode, a fintech compa...",en,True,None,"[Remote, Full-time, Mid-Senior level]",...,42345997,Quadcode,https://www.linkedin.com/company/quadcodecaree...,"[501-1,000 employees, Software Development]","501-1,000 employees",Software Development,Quadcode is a fintech company specializing in ...,"[Analytical Skills, English, B2, Quantitative ...","[SQL, Python (Programming Language), Software ...","[SQL, Python, SaaS]"
1,3768710281,1.701296e+09,LISTED,https://www.linkedin.com/jobs/view/3768710281/,Consultant in Data Analysis and Scientific Pro...,Your Responsibilities\n\nWe are currently look...,en,False,None,"[On-site, Full-time, Mid-Senior level]",...,10198832,Climate Analytics,https://www.linkedin.com/company/climate-analy...,"[51-200 employees, Climate Data and Analytics]",51-200 employees,Climate Data and Analytics,Climate Analytics is a global climate science ...,"[Data Visualization, Data Analysis, Data Analy...","[Java, Analytics]","[Java, Analytics]"
2,3773349346,1.701296e+09,LISTED,https://www.linkedin.com/jobs/view/3773349346/,Data-Analyst (m/w/d) in SaaS-Startup,"Wir sind cinify, ein junges Start-up, dass den...",de,True,None,"[Remote, Full-time, Entry level]",...,88277276,cinify,https://www.linkedin.com/company/cinify-io/life,"[1-10 employees, Data Infrastructure and Analy...",1-10 employees,Data Infrastructure and Analytics,cinify wurde Anfang des Jahres 2023 gegründet ...,"[Extract, Transform, Load, Data Visualization,...","[Business Intelligence (BI), SAS (Software), H...","[BI, Software, Software, Software, Software]"
3,3773351712,1.701296e+09,LISTED,https://www.linkedin.com/jobs/view/3773351712/,"Data Engineer/Analyst - DBT, AWS, Redshift, Py...",Orcan Intelligence are currently partnered wit...,en,False,None,"[On-site, Contract]",...,9387927,CareerAddict,https://www.linkedin.com/company/career-addict...,"[11-50 employees, Staffing and Recruiting]",11-50 employees,Staffing and Recruiting,CareerAddict is a leading career advice platfo...,"[Extract, Transform, Load, Data Science, Data ...",[],[]
4,3775291259,1.701296e+09,LISTED,https://www.linkedin.com/jobs/view/3775291259/,Junior Business Intelligence Consultant (M/W/D),Wir sind die Positive Thinking Company. Wir su...,de,False,None,"[On-site, Full-time, Mid-Senior level]",...,11444799,Positive Thinking Company by CBTW,https://www.linkedin.com/company/the-positive-...,"[1,001-5,000 employees, IT Services and IT Con...","1,001-5,000 employees",IT Services and IT Consulting,𝐀𝐛𝐨𝐮𝐭 𝐏𝐨𝐬𝐢𝐭𝐢𝐯𝐞 𝐓𝐡𝐢𝐧𝐤𝐢𝐧𝐠 𝐂𝐨𝐦𝐩𝐚𝐧𝐲As the technolo...,"[SAP, BPCS, Data Analytics, SAP BPC, IBM Plann...",[],[]


In [19]:
df_jobs.to_excel('linkedin_job_details.xlsx')  

In [20]:
df_top_skills['Listed frequency'] = 0
df_top_skills['Unlisted frequency'] = 0

def group_list(lst):
     
    return list(zip(Counter(lst).keys(), Counter(lst).values()))
     
combined_listed_skill_lists = list(itertools.chain.from_iterable(df_jobs[pd.isna(df_jobs['Job Skills']) == False]['Job Skills'].values))
combined_unlisted_skill_lists = list(itertools.chain.from_iterable(df_jobs['Unlisted Skill'].values))

listed_skills_grouped = group_list(combined_listed_skill_lists)
unlisted_skills_grouped = group_list(combined_unlisted_skill_lists)
# print(listed_skills_grouped)
# print(unlisted_skills_grouped)

for skill_tuple in listed_skills_grouped:
    skill_row = df_top_skills[df_top_skills['skill name'] == skill_tuple[0]]

    skill_row['Listed frequency'] = skill_tuple[1]
    df_top_skills.loc[skill_row.index] = skill_row


for skill_tuple in unlisted_skills_grouped:
    skill_row = df_top_skills[df_top_skills['skill name'] == skill_tuple[0]]

    skill_row['Unlisted frequency'] = skill_tuple[1]
    df_top_skills.loc[skill_row.index] = skill_row

df_top_skills

C:\Users\naimi\AppData\Local\Temp\ipykernel_26372\2873718512.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  skill_row['Listed frequency'] = skill_tuple[1]
C:\Users\naimi\AppData\Local\Temp\ipykernel_26372\2873718512.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  skill_row['Unlisted frequency'] = skill_tuple[1]


,skill name,common phrases,Listed frequency,Unlisted frequency
0,Analytical Skills,[Analytical Skills],73,0
1,Scanning Electron Microscopy (SEM),"[Scanning Electron Microscopy, SEM]",1,0
2,SQL Server Analysis Services (SSAS),"[SSAS, SQL Server Analysis Services]",1,0
3,Algorithms,[Algorithms],6,0
4,Data Analysis,[Data Analysis],55,0
...,...,...,...,...
793,Bank Confirmation,[Bank Confirmation],1,0
794,Financial Modeling,[Financial Modeling],3,0
795,Fraud Investigations,[Fraud Investigations],2,0
796,Client-focused,[Client-focused],1,0


In [21]:
# init the Google API translator
translator = Translator()
translation_dest = 'en'

In [22]:
df_translated = df_jobs.copy()
df_translated = df_translated[['Job ID', 'Job Title', 'Job Description', 'Post Language', 'Remote', 'Top Card Details', 'Job Country', 'Job Skills', 'Company Name', 'Company Card Details', 'Industry', 'Company Description']]

def translate_phrases(phrase_to_translate, translate_dest = 'en'):
    if type(phrase_to_translate) != str:
        return phrase_to_translate
    
    translation = translator.translate(phrase_to_translate, dest=translate_dest)
    return translation.text

df_translated['final'] = ''
for index in df_translated.index:
    row = df_translated.loc[index]
    if row['Post Language'] == translation_dest:
        row['final'] = ', '.join([str(el) for el in df_translated.loc[index].tolist()])
        if row['Remote'] == True:
            row['final'] = row['final'] + ', Remote'
    else:
        row['final'] = ', '.join([str(translate_phrases(el, translation_dest)) for el in df_translated.loc[index].tolist()])
        if row['Remote'] == True:
            row['final'] = row['final'] + ', Remote'
    
    df_translated.loc[index] = row

df_jobs['translated_en'] = df_translated['final']

C:\Users\naimi\AppData\Local\Temp\ipykernel_26372\358001166.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  row['final'] = ', '.join([str(el) for el in df_translated.loc[index].tolist()])
C:\Users\naimi\AppData\Local\Temp\ipykernel_26372\358001166.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  row['final'] = row['final'] + ', Remote'
C:\Users\naimi\AppData\Local\Temp\ipykernel_26372\358001166.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-

In [24]:
df_jobs.to_excel('linkedin_job_details.xlsx') 
df_top_skills.to_excel('linkedin_job_skills.xlsx') 

In [1]:
html_body_test = """
<body dir="ltr" class="render-mode-BIGPIPE nav-v2 ember-application icons-loaded boot-complete" data-t-link-to-event-attached="true">
  

  
    <!-- HUED-11420 -->
    <div id="artdeco-toasts__wormhole" role="region">    <section id="artdeco-toasts" class="artdeco-toasts" aria-label="Toast message">
      <header class="artdeco-toasts__header">
        <h2 class="artdeco-toasts__title">
          0 notifications total
        </h2>
      </header>

<!---->
      <div class="artdeco-toasts_toasts">
<!---->      </div>
    </section>
</div>
    <!-- EMBER_CLI_FASTBOOT_BODY -->
<div id="app-boot-bg-loader" class="app-boot-bg-skeleton">
  <div class="top-bar"></div>
  <div class="content">
    <div class="initial-load-animation fade-load">
      <div class="linkedin-image display-flex justify-center">
        <svg xmlns="http://www.w3.org/2000/svg" class="loader__linkedin-logo" width="190" height="48" viewBox="0 0 190 48">
          <g>
            <g>
              <path d="M58,27.22V41H51V28.88c0-3.7-2.07-5.24-4-5.24a5,5,0,0,0-5.14,4.85,4.34,4.34,0,0,0,0,.51V41H35V18h6.6v3.25h.09c.69-1.41,3.64-3.75,7.66-3.75S58,20.22,58,27.22ZM24,41h7V18H24ZM27.5,6.45a4.05,4.05,0,1,0,4.1,4.05,4,4,0,0,0-4-4ZM190,3.5v41a3.5,3.5,0,0,1-3.5,3.5h-41a3.5,3.5,0,0,1-3.5-3.5V3.5A3.5,3.5,0,0,1,145.5,0h41A3.5,3.5,0,0,1,190,3.5ZM156,18h-7V41h7Zm.6-7.5a4.1,4.1,0,1,0-4.15,4.05h.05a4,4,0,0,0,4.1-3.9ZM183,27.22c0-7-4.63-9.72-8.65-9.72s-7,2.34-7.66,3.75h-.09V18H160V41h7V29c0-3.69,2.51-5.33,4.95-5.33,2,0,4.05,1.54,4.05,5.24V41h7ZM8,7H0V41H21V34H8ZM108,29.77v2H91a1.33,1.33,0,0,0,.11.43c.58,1.93,2.67,3.56,5.75,3.56A6.3,6.3,0,0,0,102,33.52l5.1,3.18a12.72,12.72,0,0,1-10.45,4.8C89.94,41.5,84,37.42,84,29.59S90,17.5,96.44,17.5,108,21.81,108,29.77ZM101,27c0-2.4-1.56-4.38-4.75-4.38-3,0-5.09,2-5.25,4.38ZM85.26,18H76.68l-7.54,9.37H69V7H62V41h7V30h.14l7.72,11h8.77L76.2,28.43ZM128,7h7V41h-6.6V38h-.09c-.88,1.52-3.24,3.49-7.4,3.49-5,0-10.91-3.63-10.91-12,0-7.53,5.1-11.95,10.82-11.95a9.55,9.55,0,0,1,7.09,3H128Zm.3,22.49a5.74,5.74,0,0,0-5.59-5.89h-.15A5.54,5.54,0,0,0,116.89,29c0,.15,0,.29,0,.44a5.61,5.61,0,0,0,5.26,5.94h.4A5.83,5.83,0,0,0,128.3,29.49Z" fill="currentColor"></path>
            </g>
          </g>
        </svg>
      </div>
      <div class="loading-bar">
        <div class="blue-bar"></div>
      </div>
    </div>
  </div>
</div>

    <script src="https://static.licdn.com/aero-v1/sc/h/9vghoawmkb5fvzav7oabel24l" data-fastboot-src="/assets/vendor-static.js" crossorigin=""></script>
    <script src="https://static.licdn.com/aero-v1/sc/h/11eolzz5b283nc4tnagm3alq7" data-fastboot-src="/assets/vendor.js" crossorigin=""></script>
    <!-- graphql-script-injection -->
    <script src="https://static.licdn.com/aero-v1/sc/h/a8i9w57wqoyi8fjxs9ulzk14e" data-fastboot-src="/assets/pdsc/register-metadata.js" crossorigin=""></script>
    <script src="https://static.licdn.com/aero-v1/sc/h/22fggrlii7wsltz6fl4hii414" data-fastboot-src="/assets/chunk.328.8d0e86876347a9c2aa5b.js" crossorigin=""></script>
<script src="https://static.licdn.com/aero-v1/sc/h/d0ctz04bmij7oz7fegwxrtnad" data-fastboot-src="/assets/chunk.143.b8974102cf3edaf96109.js" crossorigin=""></script>
    <script data-embroider-ignore="" src="https://static.licdn.com/aero-v1/sc/h/cc8v3w7w7p9hjm24481ikut6n" data-fastboot-src="/assets/i18n/support_en_US.js" crossorigin=""></script>
    <script src="https://static.licdn.com/aero-v1/sc/h/1w15yji05xoobybogjdkfz0px"></script><script src="https://static.licdn.com/aero-v1/sc/h/3td7ryy98rs4l2sup9qw4jytz"></script>
    <script src="https://static.licdn.com/aero-v1/sc/h/e4srt2ytpjcnxrhvg4000s7g3" data-fastboot-src="/assets/voyager-web.js" crossorigin=""></script>
    <div id="artdeco-modal-outlet"></div>

<code style="display: none" id="bpr-guid-15912206">
  {"data":{"entityUrn":"urn:li:collectionResponse:4HlijJjSiEXmgxSNTaZbyKnZ88db1eTefgOnimBx6wM=","elements":[{"lixTracking":{"urn":"urn:li:member:780222017","segmentIndex":5,"experimentId":4753206,"treatmentIndex":0,"$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigLixTrackingInfo"},"chameleonConfigTrackingItem":{"configLixTrackingInfoListV2":[{"lixTracking":{"urn":"urn:li:member:780222017","segmentIndex":5,"experimentId":4753206,"treatmentIndex":0,"$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigLixTrackingInfo"},"lixTreatment":"control","lixKey":"chameleon.GROWTH_global.copy-definition.51323.child.51327","$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigLixTrackingInfoWrapper"},{"lixTracking":{"urn":"urn:li:member:780222017","segmentIndex":5,"experimentId":4753207,"treatmentIndex":0,"$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigLixTrackingInfo"},"lixTreatment":"control","lixKey":"chameleon.GROWTH_global.copy-definition.51323.child.51327","$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigLixTrackingInfoWrapper"}],"$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigTrackingItem"},"data":{"namespace":"onboarding/components/widgets/job-seeker-intent","message":"","locale":"en_US","key":"i18n_onboarding_job_seeker_intent_option_none","$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigDataI18n"},"parentLixKey":"chameleon.GROWTH_global.copy-definition.51323.parent","lixTreatment":"control","parentLixTracking":{"urn":"urn:li:member:780222017","segmentIndex":5,"experimentId":4753207,"treatmentIndex":0,"$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigLixTrackingInfo"},"lixKey":"chameleon.GROWTH_global.copy-definition.51323.child.51327","$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigItem"},{"lixTracking":{"urn":"urn:li:member:780222017","segmentIndex":6,"experimentId":4741090,"treatmentIndex":1,"$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigLixTrackingInfo"},"chameleonConfigTrackingItem":{"configLixTrackingInfoListV2":[{"lixTracking":{"urn":"urn:li:member:780222017","segmentIndex":6,"experimentId":4741090,"treatmentIndex":1,"$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigLixTrackingInfo"},"lixTreatment":"VAR_t50329_PR_1","lixKey":"chameleon.ACQUIRE_Global.web-copy-definition.50327.child.50329","$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigLixTrackingInfoWrapper"},{"lixTracking":{"urn":"urn:li:member:780222017","segmentIndex":6,"experimentId":4741091,"treatmentIndex":1,"$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigLixTrackingInfo"},"lixTreatment":"VAR_t50327_PR_1","lixKey":"chameleon.ACQUIRE_Global.web-copy-definition.50327.child.50329","$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigLixTrackingInfoWrapper"}],"$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigTrackingItem"},"data":{"namespace":"support-chat/components/virtual-chat-assistant","message":"Chat with an expert now","locale":"en_US","key":"i18n_chat_online_text","$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigDataI18n"},"parentLixKey":"chameleon.ACQUIRE_Global.web-copy-definition.50327.parent","lixTreatment":"VAR_t50329_PR_1","parentLixTracking":{"urn":"urn:li:member:780222017","segmentIndex":6,"experimentId":4741091,"treatmentIndex":1,"$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigLixTrackingInfo"},"lixKey":"chameleon.ACQUIRE_Global.web-copy-definition.50327.child.50329","$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigItem"},{"lixTracking":{"urn":"urn:li:member:780222017","segmentIndex":5,"experimentId":4753204,"treatmentIndex":0,"$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigLixTrackingInfo"},"chameleonConfigTrackingItem":{"configLixTrackingInfoListV2":[{"lixTracking":{"urn":"urn:li:member:780222017","segmentIndex":5,"experimentId":4753204,"treatmentIndex":0,"$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigLixTrackingInfo"},"lixTreatment":"control","lixKey":"chameleon.GROWTH_global.copy-definition.51323.child.51325","$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigLixTrackingInfoWrapper"},{"lixTracking":{"urn":"urn:li:member:780222017","segmentIndex":5,"experimentId":4753207,"treatmentIndex":0,"$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigLixTrackingInfo"},"lixTreatment":"control","lixKey":"chameleon.GROWTH_global.copy-definition.51323.child.51325","$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigLixTrackingInfoWrapper"}],"$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigTrackingItem"},"data":{"namespace":"onboarding/components/widgets/job-seeker-intent","message":"","locale":"en_US","key":"i18n_onboarding_job_seeker_intent_option_active","$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigDataI18n"},"parentLixKey":"chameleon.GROWTH_global.copy-definition.51323.parent","lixTreatment":"control","parentLixTracking":{"urn":"urn:li:member:780222017","segmentIndex":5,"experimentId":4753207,"treatmentIndex":0,"$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigLixTrackingInfo"},"lixKey":"chameleon.GROWTH_global.copy-definition.51323.child.51325","$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigItem"},{"lixTracking":{"urn":"urn:li:member:780222017","segmentIndex":2,"experimentId":4560784,"treatmentIndex":1,"$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigLixTrackingInfo"},"chameleonConfigTrackingItem":{"configLixTrackingInfoListV2":[{"lixTracking":{"urn":"urn:li:member:780222017","segmentIndex":2,"experimentId":4560784,"treatmentIndex":1,"$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigLixTrackingInfo"},"lixTreatment":"VAR_t29537_PR_1","lixKey":"chameleon.PREMIUM:us.copy-definition.27071.child.29537","$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigLixTrackingInfoWrapper"},{"lixTracking":{"urn":"urn:li:member:780222017","segmentIndex":2,"experimentId":4560787,"treatmentIndex":1,"$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigLixTrackingInfo"},"lixTreatment":"VAR_t29516_PR_1","lixKey":"chameleon.PREMIUM:us.copy-definition.27071.child.29537","$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigLixTrackingInfoWrapper"}],"$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigTrackingItem"},"data":{"namespace":"feed/components/identity-module","message":"See your Premium features","locale":"en_US","key":"premium_access_headline","$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigDataI18n"},"parentLixKey":"chameleon.PREMIUM:us.copy-definition.27071.parent","lixTreatment":"VAR_t29537_PR_1","parentLixTracking":{"urn":"urn:li:member:780222017","segmentIndex":2,"experimentId":4560787,"treatmentIndex":1,"$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigLixTrackingInfo"},"lixKey":"chameleon.PREMIUM:us.copy-definition.27071.child.29537","$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigItem"},{"lixTracking":{"urn":"urn:li:member:780222017","segmentIndex":5,"experimentId":4753205,"treatmentIndex":0,"$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigLixTrackingInfo"},"chameleonConfigTrackingItem":{"configLixTrackingInfoListV2":[{"lixTracking":{"urn":"urn:li:member:780222017","segmentIndex":5,"experimentId":4753205,"treatmentIndex":0,"$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigLixTrackingInfo"},"lixTreatment":"control","lixKey":"chameleon.GROWTH_global.copy-definition.51323.child.51326","$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigLixTrackingInfoWrapper"},{"lixTracking":{"urn":"urn:li:member:780222017","segmentIndex":5,"experimentId":4753207,"treatmentIndex":0,"$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigLixTrackingInfo"},"lixTreatment":"control","lixKey":"chameleon.GROWTH_global.copy-definition.51323.child.51326","$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigLixTrackingInfoWrapper"}],"$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigTrackingItem"},"data":{"namespace":"onboarding/components/widgets/job-seeker-intent","message":"","locale":"en_US","key":"i18n_onboarding_job_seeker_intent_option_passive","$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigDataI18n"},"parentLixKey":"chameleon.GROWTH_global.copy-definition.51323.parent","lixTreatment":"control","parentLixTracking":{"urn":"urn:li:member:780222017","segmentIndex":5,"experimentId":4753207,"treatmentIndex":0,"$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigLixTrackingInfo"},"lixKey":"chameleon.GROWTH_global.copy-definition.51323.child.51326","$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigItem"},{"lixTracking":{"urn":"urn:li:member:780222017","segmentIndex":5,"experimentId":4727865,"treatmentIndex":2,"$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigLixTrackingInfo"},"chameleonConfigTrackingItem":{"configLixTrackingInfoListV2":[{"lixTracking":{"urn":"urn:li:member:780222017","segmentIndex":5,"experimentId":4727865,"treatmentIndex":2,"$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigLixTrackingInfo"},"lixTreatment":"VAR_t48324_PR_2","lixKey":"chameleon.GROWTH_global.web-copy-definition.31854","$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigLixTrackingInfoWrapper"}],"$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigTrackingItem"},"data":{"namespace":"mynetwork/templates/network-manager/people-follow/following","message":"Follow experts in your field to see their industry-specific content in your feed.","locale":"en_US","key":"i18n_following_empty_state_description","$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigDataI18n"},"lixTreatment":"VAR_t48324_PR_2","lixKey":"chameleon.GROWTH_global.web-copy-definition.31854","$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigItem"},{"lixTracking":{"urn":"urn:li:member:780222017","segmentIndex":2,"experimentId":4562276,"treatmentIndex":1,"$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigLixTrackingInfo"},"chameleonConfigTrackingItem":{"configLixTrackingInfoListV2":[{"lixTracking":{"urn":"urn:li:member:780222017","segmentIndex":2,"experimentId":4562276,"treatmentIndex":1,"$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigLixTrackingInfo"},"lixTreatment":"VAR_t33653_PR_1","lixKey":"chameleon.CAREER-SEARCH-EVAL:GLOBAL.web-copy-definition.27904","$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigLixTrackingInfoWrapper"}],"$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigTrackingItem"},"data":{"namespace":"job-details-engine/components/detail/job-view-layout","message":"More jobs","locale":"en_US","key":"i18n_simjob_header","$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigDataI18n"},"lixTreatment":"VAR_t33653_PR_1","lixKey":"chameleon.CAREER-SEARCH-EVAL:GLOBAL.web-copy-definition.27904","$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigItem"},{"lixTracking":{"urn":"urn:li:member:780222017","segmentIndex":3,"experimentId":4704568,"treatmentIndex":0,"$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigLixTrackingInfo"},"chameleonConfigTrackingItem":{"configLixTrackingInfoListV2":[{"lixTracking":{"urn":"urn:li:member:780222017","segmentIndex":3,"experimentId":4704568,"treatmentIndex":0,"$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigLixTrackingInfo"},"lixTreatment":"control","lixKey":"chameleon.PREMIUM_us.web-copy-definition.31708","$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigLixTrackingInfoWrapper"}],"$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigTrackingItem"},"data":{"namespace":"hiring/components/jobs/applicants/top-choice/top-choice-card","message":"","locale":"en_US","key":"i18n_top_choice_survey_prompt","$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigDataI18n"},"lixTreatment":"control","lixKey":"chameleon.PREMIUM_us.web-copy-definition.31708","$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigItem"},{"lixTracking":{"urn":"urn:li:member:780222017","segmentIndex":5,"experimentId":4749819,"treatmentIndex":0,"$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigLixTrackingInfo"},"chameleonConfigTrackingItem":{"configLixTrackingInfoListV2":[{"lixTracking":{"urn":"urn:li:member:780222017","segmentIndex":5,"experimentId":4749819,"treatmentIndex":0,"$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigLixTrackingInfo"},"lixTreatment":"control","lixKey":"chameleon.OJ_GLOBAL.web-copy-definition.50683","$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigLixTrackingInfoWrapper"}],"$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigTrackingItem"},"data":{"namespace":"job-posting/templates/index","message":"","locale":"en_US","key":"i18n_form_subheader","$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigDataI18n"},"lixTreatment":"control","lixKey":"chameleon.OJ_GLOBAL.web-copy-definition.50683","$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigItem"},{"lixTracking":{"urn":"urn:li:member:780222017","segmentIndex":3,"experimentId":4729785,"treatmentIndex":1,"$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigLixTrackingInfo"},"chameleonConfigTrackingItem":{"configLixTrackingInfoListV2":[{"lixTracking":{"urn":"urn:li:member:780222017","segmentIndex":3,"experimentId":4729785,"treatmentIndex":1,"$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigLixTrackingInfo"},"lixTreatment":"VAR_t48704_PR_3","lixKey":"chameleon.ONBOARDING_global.web-copy-definition.32052","$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigLixTrackingInfoWrapper"}],"$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigTrackingItem"},"data":{"namespace":"onboarding/components/widgets/get-the-app","message":"See {:firstName}{:firstName,possessive} full profile on the app","locale":"en_US","key":"i18n_onboarding_get_the_app_page_header_profile","$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigDataI18n"},"lixTreatment":"VAR_t48704_PR_3","lixKey":"chameleon.ONBOARDING_global.web-copy-definition.32052","$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigItem"},{"lixTracking":{"urn":"urn:li:member:780222017","segmentIndex":5,"experimentId":4721390,"treatmentIndex":1,"$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigLixTrackingInfo"},"chameleonConfigTrackingItem":{"configLixTrackingInfoListV2":[{"lixTracking":{"urn":"urn:li:member:780222017","segmentIndex":5,"experimentId":4721390,"treatmentIndex":1,"$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigLixTrackingInfo"},"lixTreatment":"VAR_t48262_PR_2","lixKey":"chameleon.OJ_GLOBAL.web-copy-definition.31824","$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigLixTrackingInfoWrapper"}],"$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigTrackingItem"},"data":{"namespace":"job-posting-shared/components/budget/budget-setter","message":"{:amount} daily max","locale":"en_US","key":"i18n_budget_recommended_daily","$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigDataI18n"},"lixTreatment":"VAR_t48262_PR_2","lixKey":"chameleon.OJ_GLOBAL.web-copy-definition.31824","$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigItem"},{"lixTracking":{"urn":"urn:li:member:780222017","segmentIndex":2,"experimentId":4697659,"treatmentIndex":1,"$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigLixTrackingInfo"},"chameleonConfigTrackingItem":{"configLixTrackingInfoListV2":[{"lixTracking":{"urn":"urn:li:member:780222017","segmentIndex":2,"experimentId":4697659,"treatmentIndex":1,"$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigLixTrackingInfo"},"lixTreatment":"VAR_t47452_PR_1","lixKey":"chameleon.PROFILE_GLOBAL.web-copy-definition.31506","$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigLixTrackingInfoWrapper"}],"$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigTrackingItem"},"data":{"namespace":"profile-creator-mode-shared/components/creator-tools-list/item","message":"Learn more","locale":"en_US","key":"i18n_available","$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigDataI18n"},"lixTreatment":"VAR_t47452_PR_1","lixKey":"chameleon.PROFILE_GLOBAL.web-copy-definition.31506","$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigItem"},{"lixTracking":{"urn":"urn:li:member:780222017","segmentIndex":4,"experimentId":4626639,"treatmentIndex":1,"$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigLixTrackingInfo"},"chameleonConfigTrackingItem":{"configLixTrackingInfoListV2":[{"lixTracking":{"urn":"urn:li:member:780222017","segmentIndex":4,"experimentId":4626639,"treatmentIndex":1,"$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigLixTrackingInfo"},"lixTreatment":"VAR_t42421_PR_1","lixKey":"chameleon.PROJECTX:Global.web-copy-definition.29643.child.42421","$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigLixTrackingInfoWrapper"},{"lixTracking":{"urn":"urn:li:member:780222017","segmentIndex":4,"experimentId":4626644,"treatmentIndex":1,"$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigLixTrackingInfo"},"lixTreatment":"VAR_t42433_PR_1","lixKey":"chameleon.PROJECTX:Global.web-copy-definition.29643.child.42421","$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigLixTrackingInfoWrapper"}],"$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigTrackingItem"},"data":{"namespace":"social-details/components/x-article-insightful-button","message":"Mark as insightful","locale":"en_US","key":"i18n_mark_as_insightful","$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigDataI18n"},"parentLixKey":"chameleon.PROJECTX:Global.ssr-copy-definition.29653.parent","lixTreatment":"VAR_t42421_PR_1","parentLixTracking":{"urn":"urn:li:member:780222017","segmentIndex":4,"experimentId":4626644,"treatmentIndex":1,"$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigLixTrackingInfo"},"lixKey":"chameleon.PROJECTX:Global.web-copy-definition.29643.child.42421","$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigItem"},{"lixTracking":{"urn":"urn:li:member:780222017","segmentIndex":3,"experimentId":4510388,"treatmentIndex":0,"$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigLixTrackingInfo"},"chameleonConfigTrackingItem":{"configLixTrackingInfoListV2":[{"lixTracking":{"urn":"urn:li:member:780222017","segmentIndex":3,"experimentId":4510388,"treatmentIndex":0,"$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigLixTrackingInfo"},"lixTreatment":"control","lixKey":"chameleon.PAGES:Global.web-copy-definition.26519","$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigLixTrackingInfoWrapper"}],"$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigTrackingItem"},"data":{"namespace":"organization-admin-pages-entrypoint/components/card","message":"","locale":"en_US","key":"i18n_org_page_notifications","$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigDataI18n"},"lixTreatment":"control","lixKey":"chameleon.PAGES:Global.web-copy-definition.26519","$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigItem"},{"lixTracking":{"urn":"urn:li:member:780222017","segmentIndex":4,"experimentId":4626638,"treatmentIndex":1,"$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigLixTrackingInfo"},"chameleonConfigTrackingItem":{"configLixTrackingInfoListV2":[{"lixTracking":{"urn":"urn:li:member:780222017","segmentIndex":4,"experimentId":4626638,"treatmentIndex":1,"$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigLixTrackingInfo"},"lixTreatment":"VAR_t42420_PR_1","lixKey":"chameleon.PROJECTX:Global.web-copy-definition.29643.child.42420","$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigLixTrackingInfoWrapper"},{"lixTracking":{"urn":"urn:li:member:780222017","segmentIndex":4,"experimentId":4626644,"treatmentIndex":1,"$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigLixTrackingInfo"},"lixTreatment":"VAR_t42433_PR_1","lixKey":"chameleon.PROJECTX:Global.web-copy-definition.29643.child.42420","$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigLixTrackingInfoWrapper"}],"$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigTrackingItem"},"data":{"namespace":"social-details/components/x-article-insightful-button","message":"{:value,number,integer} {:value,choice,singular#reaction|plural#reactions}","locale":"en_US","key":"i18n_insightful_count","$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigDataI18n"},"parentLixKey":"chameleon.PROJECTX:Global.ssr-copy-definition.29653.parent","lixTreatment":"VAR_t42420_PR_1","parentLixTracking":{"urn":"urn:li:member:780222017","segmentIndex":4,"experimentId":4626644,"treatmentIndex":1,"$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigLixTrackingInfo"},"lixKey":"chameleon.PROJECTX:Global.web-copy-definition.29643.child.42420","$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigItem"},{"lixTracking":{"urn":"urn:li:member:780222017","segmentIndex":5,"experimentId":4741617,"treatmentIndex":0,"$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigLixTrackingInfo"},"chameleonConfigTrackingItem":{"configLixTrackingInfoListV2":[{"lixTracking":{"urn":"urn:li:member:780222017","segmentIndex":5,"experimentId":4741617,"treatmentIndex":0,"$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigLixTrackingInfo"},"lixTreatment":"control","lixKey":"chameleon.OJ_GLOBAL.web-copy-definition.50312.child.50314","$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigLixTrackingInfoWrapper"},{"lixTracking":{"urn":"urn:li:member:780222017","segmentIndex":5,"experimentId":4741618,"treatmentIndex":0,"$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigLixTrackingInfo"},"lixTreatment":"control","lixKey":"chameleon.OJ_GLOBAL.web-copy-definition.50312.child.50314","$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigLixTrackingInfoWrapper"}],"$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigTrackingItem"},"data":{"namespace":"job-posting/templates/index","message":"","locale":"en_US","key":"i18n_form_draft_job_on_my_own","$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigDataI18n"},"parentLixKey":"chameleon.OJ_GLOBAL.web-copy-definition.50312.parent","lixTreatment":"control","parentLixTracking":{"urn":"urn:li:member:780222017","segmentIndex":5,"experimentId":4741618,"treatmentIndex":0,"$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigLixTrackingInfo"},"lixKey":"chameleon.OJ_GLOBAL.web-copy-definition.50312.child.50314","$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigItem"},{"lixTracking":{"urn":"urn:li:member:780222017","segmentIndex":6,"experimentId":4749852,"treatmentIndex":0,"$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigLixTrackingInfo"},"chameleonConfigTrackingItem":{"configLixTrackingInfoListV2":[{"lixTracking":{"urn":"urn:li:member:780222017","segmentIndex":6,"experimentId":4749852,"treatmentIndex":0,"$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigLixTrackingInfo"},"lixTreatment":"control","lixKey":"chameleon.GROWTH_global.web-copy-definition.51057","$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigLixTrackingInfoWrapper"}],"$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigTrackingItem"},"data":{"namespace":"onboarding/components/widgets/follow-recommendations-common","message":"","locale":"en_US","key":"i18n_onboarding_follow_recommendations_footer_no_follows","$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigDataI18n"},"lixTreatment":"control","lixKey":"chameleon.GROWTH_global.web-copy-definition.51057","$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigItem"},{"lixTracking":{"urn":"urn:li:member:780222017","segmentIndex":5,"experimentId":4741616,"treatmentIndex":0,"$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigLixTrackingInfo"},"chameleonConfigTrackingItem":{"configLixTrackingInfoListV2":[{"lixTracking":{"urn":"urn:li:member:780222017","segmentIndex":5,"experimentId":4741616,"treatmentIndex":0,"$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigLixTrackingInfo"},"lixTreatment":"control","lixKey":"chameleon.OJ_GLOBAL.web-copy-definition.50312.child.50313","$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigLixTrackingInfoWrapper"},{"lixTracking":{"urn":"urn:li:member:780222017","segmentIndex":5,"experimentId":4741618,"treatmentIndex":0,"$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigLixTrackingInfo"},"lixTreatment":"control","lixKey":"chameleon.OJ_GLOBAL.web-copy-definition.50312.child.50313","$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigLixTrackingInfoWrapper"}],"$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigTrackingItem"},"data":{"namespace":"job-posting/templates/index","message":"","locale":"en_US","key":"i18n_form_draft_job_with_ai_cta","$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigDataI18n"},"parentLixKey":"chameleon.OJ_GLOBAL.web-copy-definition.50312.parent","lixTreatment":"control","parentLixTracking":{"urn":"urn:li:member:780222017","segmentIndex":5,"experimentId":4741618,"treatmentIndex":0,"$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigLixTrackingInfo"},"lixKey":"chameleon.OJ_GLOBAL.web-copy-definition.50312.child.50313","$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigItem"},{"lixTracking":{"urn":"urn:li:member:780222017","segmentIndex":6,"experimentId":4741089,"treatmentIndex":1,"$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigLixTrackingInfo"},"chameleonConfigTrackingItem":{"configLixTrackingInfoListV2":[{"lixTracking":{"urn":"urn:li:member:780222017","segmentIndex":6,"experimentId":4741089,"treatmentIndex":1,"$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigLixTrackingInfo"},"lixTreatment":"VAR_t50328_PR_1","lixKey":"chameleon.ACQUIRE_Global.web-copy-definition.50327.child.50328","$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigLixTrackingInfoWrapper"},{"lixTracking":{"urn":"urn:li:member:780222017","segmentIndex":6,"experimentId":4741091,"treatmentIndex":1,"$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigLixTrackingInfo"},"lixTreatment":"VAR_t50327_PR_1","lixKey":"chameleon.ACQUIRE_Global.web-copy-definition.50327.child.50328","$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigLixTrackingInfoWrapper"}],"$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigTrackingItem"},"data":{"namespace":"support-chat/components/virtual-chat-assistant","message":"Ready to decide?","locale":"en_US","key":"i18n_tooltip_title","$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigDataI18n"},"parentLixKey":"chameleon.ACQUIRE_Global.web-copy-definition.50327.parent","lixTreatment":"VAR_t50328_PR_1","parentLixTracking":{"urn":"urn:li:member:780222017","segmentIndex":6,"experimentId":4741091,"treatmentIndex":1,"$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigLixTrackingInfo"},"lixKey":"chameleon.ACQUIRE_Global.web-copy-definition.50327.child.50328","$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigItem"},{"lixTracking":{"urn":"urn:li:member:780222017","segmentIndex":5,"experimentId":4510389,"treatmentIndex":0,"$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigLixTrackingInfo"},"chameleonConfigTrackingItem":{"configLixTrackingInfoListV2":[{"lixTracking":{"urn":"urn:li:member:780222017","segmentIndex":5,"experimentId":4510389,"treatmentIndex":0,"$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigLixTrackingInfo"},"lixTreatment":"control","lixKey":"chameleon.PAGES:Global.web-copy-definition.26520","$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigLixTrackingInfoWrapper"}],"$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigTrackingItem"},"data":{"namespace":"organization-admin/components/admin/admin-nav","message":"","locale":"en_US","key":"i18n_org_activity_tab","$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigDataI18n"},"lixTreatment":"control","lixKey":"chameleon.PAGES:Global.web-copy-definition.26520","$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigItem"},{"lixTracking":{"urn":"urn:li:member:780222017","segmentIndex":2,"experimentId":4656757,"treatmentIndex":1,"$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigLixTrackingInfo"},"chameleonConfigTrackingItem":{"configLixTrackingInfoListV2":[{"lixTracking":{"urn":"urn:li:member:780222017","segmentIndex":2,"experimentId":4656757,"treatmentIndex":1,"$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigLixTrackingInfo"},"lixTreatment":"VAR_t42423_PR_1","lixKey":"chameleon.PROJECTX:Global.web-copy-definition.29644.child.42423","$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigLixTrackingInfoWrapper"},{"lixTracking":{"urn":"urn:li:member:780222017","segmentIndex":2,"experimentId":4656763,"treatmentIndex":1,"$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigLixTrackingInfo"},"lixTreatment":"VAR_t42434_PR_1","lixKey":"chameleon.PROJECTX:Global.web-copy-definition.29644.child.42423","$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigLixTrackingInfoWrapper"}],"$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigTrackingItem"},"data":{"namespace":"social-details/components/x-article-insightful-button","message":"Upvote as insightful","locale":"en_US","key":"i18n_upvote_as_insightful","$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigDataI18n"},"parentLixKey":"chameleon.PROJECTX:Global.ssr-copy-definition.29654.parent","lixTreatment":"VAR_t42423_PR_1","parentLixTracking":{"urn":"urn:li:member:780222017","segmentIndex":2,"experimentId":4656763,"treatmentIndex":1,"$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigLixTrackingInfo"},"lixKey":"chameleon.PROJECTX:Global.web-copy-definition.29644.child.42423","$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigItem"},{"lixTracking":{"urn":"urn:li:member:780222017","segmentIndex":2,"experimentId":4656758,"treatmentIndex":1,"$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigLixTrackingInfo"},"chameleonConfigTrackingItem":{"configLixTrackingInfoListV2":[{"lixTracking":{"urn":"urn:li:member:780222017","segmentIndex":2,"experimentId":4656758,"treatmentIndex":1,"$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigLixTrackingInfo"},"lixTreatment":"VAR_t42424_PR_1","lixKey":"chameleon.PROJECTX:Global.web-copy-definition.29644.child.42424","$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigLixTrackingInfoWrapper"},{"lixTracking":{"urn":"urn:li:member:780222017","segmentIndex":2,"experimentId":4656763,"treatmentIndex":1,"$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigLixTrackingInfo"},"lixTreatment":"VAR_t42434_PR_1","lixKey":"chameleon.PROJECTX:Global.web-copy-definition.29644.child.42424","$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigLixTrackingInfoWrapper"}],"$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigTrackingItem"},"data":{"namespace":"social-details/components/x-article-insightful-button","message":"{:value,number,integer} {:value,choice,singular#insightful upvote|plural#insightful upvotes}","locale":"en_US","key":"i18n_reaction_count","$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigDataI18n"},"parentLixKey":"chameleon.PROJECTX:Global.ssr-copy-definition.29654.parent","lixTreatment":"VAR_t42424_PR_1","parentLixTracking":{"urn":"urn:li:member:780222017","segmentIndex":2,"experimentId":4656763,"treatmentIndex":1,"$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigLixTrackingInfo"},"lixKey":"chameleon.PROJECTX:Global.web-copy-definition.29644.child.42424","$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigItem"}],"paging":{"count":10,"start":0,"links":[]},"$type":"com.linkedin.restli.common.CollectionResponse"},"included":[]}
</code>
<code style="display: none" id="datalet-bpr-guid-15912206">
  {"request":"/voyager/api/voyagerSegmentsDashChameleonConfig","status":200,"body":"bpr-guid-15912206","method":"GET","headers":{"x-li-uuid":"AAYLE+vtCEZ6Amp7P3S1tA\u003D\u003D"}}
</code>
<img src="data:image/gif;base64,R0lGODlhAQABAIAAAAAAAP///yH5BAEAAAAALAAAAAABAAEAAAIBRAA7" style="display: none" class="datalet-bpr-guid-15912206"><code style="display: none" id="bpr-guid-15912207">
  {"data":{"data":{"$recipeTypes":["com.linkedin.8e0c7a8828e4c16fcdc1196207bbd156"],"contactsDashMemberEmailAddressByFindEmailForVerification":{"$recipeTypes":["com.linkedin.ed8fec8807266362228e5d3cd36838b1"],"elements":[],"$type":"com.linkedin.restli.common.CollectionResponse"},"$type":"com.linkedin.8e0c7a8828e4c16fcdc1196207bbd156"},"extensions":{"webMetadata":{}}},"meta":{"microSchema":{"isGraphQL":true,"version":"2.1","types":{"com.linkedin.f5137496a80e6bbe4ca1511182d896f0":{"fields":{"emailAddress":{"type":"com.linkedin.common.EmailAddress"}},"baseType":"com.linkedin.voyager.dash.contacts.MemberEmailAddress"},"com.linkedin.ed8fec8807266362228e5d3cd36838b1":{"fields":{"elements":{"type":{"array":"com.linkedin.f5137496a80e6bbe4ca1511182d896f0"}}},"baseType":"com.linkedin.restli.common.CollectionResponse"},"com.linkedin.8e0c7a8828e4c16fcdc1196207bbd156":{"fields":{"contactsDashMemberEmailAddressByFindEmailForVerification":{"type":"com.linkedin.ed8fec8807266362228e5d3cd36838b1"}},"baseType":"com.linkedin.graphql.Query"}}}},"included":[]}
</code>
<code style="display: none" id="datalet-bpr-guid-15912207">
  {"request":"/voyager/api/graphql?includeWebMetadata\u003Dtrue\u0026queryId\u003DvoyagerContactsDashMemberEmailAddress.dd06cc4aca95a6afd4b45e17a18a4007","status":200,"body":"bpr-guid-15912207","method":"GET","headers":{"x-li-uuid":"AAYLE+vtCEZ6Amp7P3S1tA\u003D\u003D"}}
</code>
<img src="data:image/gif;base64,R0lGODlhAQABAIAAAAAAAP///yH5BAEAAAAALAAAAAABAAEAAAIBRAA7" style="display: none" class="datalet-bpr-guid-15912207"><code style="display: none" id="bpr-guid-15912208">
  {"data":{"canBrowseProfiles":false,"reactivationFeaturesEligible":false,"canViewJobAnalytics":false,"canViewWVMP":false,"premiumFreeTrialEligible":true,"canAccessLearningVideos":true,"premiumLearningUpgradeEligible":false,"canViewCompanyInsights":false,"$type":"com.linkedin.voyager.premium.FeatureAccess"},"included":[]}
</code>
<code style="display: none" id="datalet-bpr-guid-15912208">
  {"request":"/voyager/api/premium/featureAccess?name\u003DreactivationFeaturesEligible","status":200,"body":"bpr-guid-15912208","method":"GET","headers":{"x-li-uuid":"AAYLE+vtCEZ6Amp7P3S1tA\u003D\u003D"}}
</code>
<img src="data:image/gif;base64,R0lGODlhAQABAIAAAAAAAP///yH5BAEAAAAALAAAAAABAAEAAAIBRAA7" style="display: none" class="datalet-bpr-guid-15912208"><code style="display: none" id="bpr-guid-15912209">
  {"data":{"plainId":780222017,"publicContactInfo":{"$type":"com.linkedin.voyager.identity.shared.PublicContactInfo"},"premiumSubscriber":false,"*miniProfile":"urn:li:fs_miniProfile:ACoAAC6BPkEBwSD4NTQ7khSJbuAxrRMYhVSwmBo","$type":"com.linkedin.voyager.common.Me"},"included":[{"customPronoun":null,"memorialized":false,"lastName":"Zahari","dashEntityUrn":"urn:li:fsd_profile:ACoAAC6BPkEBwSD4NTQ7khSJbuAxrRMYhVSwmBo","standardizedPronoun":null,"occupation":"Performance Analyst | Aspiring Data Scientist | Data Enthusiast | Game Programmer","objectUrn":"urn:li:member:780222017","backgroundImage":null,"picture":{"artifacts":[{"width":99,"fileIdentifyingUrlPathSegment":"100_100/0/1605279012830?e=1706745600&amp;v=beta&amp;t=GoGDD2DE3nGXji1ktdhEWgWX0uNRNd93omIm9L4PTBM","expiresAt":1706745600000,"height":100,"$type":"com.linkedin.common.VectorArtifact"},{"width":199,"fileIdentifyingUrlPathSegment":"200_200/0/1605279012830?e=1706745600&amp;v=beta&amp;t=KxJr8NDL0QX2hH6Hgu-7kOudbuCrhYafGAmjrDIUX6g","expiresAt":1706745600000,"height":200,"$type":"com.linkedin.common.VectorArtifact"},{"width":399,"fileIdentifyingUrlPathSegment":"400_400/0/1605279012830?e=1706745600&amp;v=beta&amp;t=MEX6KTkEsBN1Iwy4RYn8IoyQikzi3wK-ws0Ij5Qr478","expiresAt":1706745600000,"height":400,"$type":"com.linkedin.common.VectorArtifact"},{"width":670,"fileIdentifyingUrlPathSegment":"800_800/0/1605279012830?e=1706745600&amp;v=beta&amp;t=72OsosG2vCVJHRjJ9BEqeLcfgJkzqOn-4Eq5tGu4964","expiresAt":1706745600000,"height":670,"$type":"com.linkedin.common.VectorArtifact"}],"rootUrl":"https://media.licdn.com/dms/image/C5603AQEkRciOp6B6dA/profile-displayphoto-shrink_","$type":"com.linkedin.common.VectorImage"},"$type":"com.linkedin.voyager.identity.shared.MiniProfile","firstName":"Naim Iskandar","entityUrn":"urn:li:fs_miniProfile:ACoAAC6BPkEBwSD4NTQ7khSJbuAxrRMYhVSwmBo","publicIdentifier":"naimiskandar22","trackingId":"a8kNoZxCT6CMzZRO1s6H+w=="}]}
</code>
<code style="display: none" id="datalet-bpr-guid-15912209">
  {"request":"/voyager/api/me","status":200,"body":"bpr-guid-15912209","method":"GET","headers":{"x-li-uuid":"AAYLE+vtCEZ6Amp7P3S1tA\u003D\u003D"}}
</code>
<img src="data:image/gif;base64,R0lGODlhAQABAIAAAAAAAP///yH5BAEAAAAALAAAAAABAAEAAAIBRAA7" style="display: none" class="datalet-bpr-guid-15912209"><code style="display: none" id="bpr-guid-15912210">
  {"data":{"entityUrn":"urn:li:collectionResponse:71q+z9OBlTRLck+JrP7RA9AfJZYM19vZlhsXU1XxaK0=","elements":[],"paging":{"count":10,"start":0,"links":[]},"$type":"com.linkedin.restli.common.CollectionResponse"},"meta":{"microSchema":{"version":"2","types":{"com.linkedin.voyager.dash.deco.common.forms.DropdownFormComponent":{"fields":{"placeHolderText":{"type":"com.linkedin.voyager.dash.deco.common.text.TextViewModelV2"},"controlName":{"type":"string"},"textSelectableOptions":{"type":{"array":"com.linkedin.voyager.dash.deco.common.forms.TextSelectableOption"}}},"baseType":"com.linkedin.voyager.dash.common.forms.DropdownFormComponent"},"com.linkedin.deco.recipe.anonymous.Anon1080314395":{"fields":{"ringContentType":{"type":"string"},"actionTarget":{"type":"string"},"preDashEntityUrn":{"type":"string"},"entityUrn":{"type":"string"},"emphasized":{"type":"boolean"}},"baseType":"com.linkedin.voyager.dash.common.image.RingStatus"},"com.linkedin.deco.recipe.anonymous.Anon1117966179":{"fields":{"clusterType":{"type":"string"},"selectionCountRangeValidation":{"type":"com.linkedin.voyager.dash.deco.common.forms.SelectionCountRangeValidation"}},"baseType":"com.linkedin.voyager.dash.common.forms.ContextualRangeValidation"},"com.linkedin.deco.recipe.anonymous.Anon2065966944":{"fields":{"endpoint":{"type":"string"},"contextualSuggestionDependent":{"type":{"map":"com.linkedin.deco.recipe.anonymous.Anon109927483"},"resolvedFrom":"contextualSuggestionDependentUrns"},"contextualSuggestionQueryParameterUrns":{"type":{"array":"string"}},"contextualSuggestionDependentUrns":{"type":{"array":"string"}},"contextualSuggestionQueryParameter":{"type":{"map":"com.linkedin.deco.recipe.anonymous.Anon109927483"},"resolvedFrom":"contextualSuggestionQueryParameterUrns"},"contextualSuggestionType":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.common.forms.ContextualSuggestionsTriggers"},"com.linkedin.deco.recipe.anonymous.Anon489758292":{"fields":{"viewer":{"type":"com.linkedin.deco.recipe.anonymous.Anon430310092","resolvedFrom":"viewerUrn"},"contentSeriesUrn":{"type":"string"},"contentSeries":{"type":"com.linkedin.deco.recipe.anonymous.Anon1789236903","resolvedFrom":"contentSeriesUrn"},"viewerUrn":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.relationships.MemberToContentSeriesSubscribeMetadata"},"com.linkedin.voyager.dash.deco.common.forms.EntityInput":{"fields":{"inputEntityName":{"type":"string"},"inputEntityUrn":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.common.forms.EntityInputValue"},"com.linkedin.deco.recipe.anonymous.Anon377984030":{"fields":{"professionalEvent":{"type":"com.linkedin.deco.recipe.anonymous.Anon1213723597","resolvedFrom":"professionalEventUrn"},"vectorImage":{"type":"com.linkedin.voyager.dash.deco.common.VectorImageOnlyRootUrlAndAttribution"},"professionalEventUrn":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.common.image.NonEntityProfessionalEventLogo"},"com.linkedin.voyager.dash.deco.common.image.Company":{"fields":{"logo":{"type":{"union":{"url":"string","vectorImage":"com.linkedin.voyager.dash.deco.common.VectorImageOnlyRootUrlAndAttribution"}}},"entityUrn":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.organization.Company"},"com.linkedin.voyager.dash.deco.feed.publishing.SubscribeAction":{"fields":{"subscribed":{"type":"boolean"},"preDashEntityUrn":{"type":"string"},"entityUrn":{"type":"string"},"subscriberCount":{"type":"int"}},"baseType":"com.linkedin.voyager.dash.publishing.SubscribeAction"},"com.linkedin.voyager.dash.deco.launchpad.EdgeBuildingCohortReason":{"fields":{"sourceType":{"type":"string"},"reasonObjects":{"type":{"array":"string"}},"reasonContext":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.launchpad.attribute.EdgeBuildingCohortReason"},"com.linkedin.deco.recipe.anonymous.Anon1444411496":{"fields":{"countryUrn":{"type":"string"},"country":{"type":"com.linkedin.deco.recipe.anonymous.Anon1918694303","resolvedFrom":"countryUrn"},"defaultLocalizedNameWithoutCountryName":{"type":"string"},"entityUrn":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.common.Geo"},"com.linkedin.deco.recipe.anonymous.Anon1226238669":{"fields":{"name":{"type":"string"},"logo":{"type":{"union":{"url":"string","vectorImage":"com.linkedin.voyager.dash.deco.common.VectorImageOnlyRootUrlAndAttribution"}}},"universalName":{"type":"string"},"entityUrn":{"type":"string"},"url":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.organization.Company"},"com.linkedin.deco.recipe.anonymous.Anon798864187":{"fields":{"influencer":{"type":"boolean"},"entityUrn":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.identity.profile.Profile"},"com.linkedin.voyager.dash.deco.common.ux.StatefulButtonModel":{"fields":{"actionDataModel":{"type":{"union":{"follow":"com.linkedin.voyager.dash.deco.feed.FollowingState","connectionOrInvitation":"com.linkedin.voyager.dash.deco.relationships.MemberRelationshipV2","memberToEntityRelationship":"com.linkedin.voyager.dash.deco.relationships.DirectionalEntityRelationship"}},"resolvedFrom":"actionDataModelUnion"},"actionDataModelUnion":{"type":{"union":{"connectionOrInvitation":"string","relationshipActionData":"com.linkedin.voyager.dash.deco.common.ux.RelationshipActionData","memberToEntityRelationship":"string","follow":"string"}}},"emphasisStyle":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.common.ux.button.StatefulButtonModel"},"com.linkedin.deco.recipe.anonymous.Anon1636615691":{"fields":{"transcripts":{"type":{"array":"string"}},"pagesPerResolution":{"type":{"array":"com.linkedin.deco.recipe.anonymous.Anon1525160909"}}},"baseType":"com.linkedin.documentcontent.DocumentPages"},"com.linkedin.deco.recipe.anonymous.Anon934606613":{"fields":{"type":{"type":"string"},"name":{"type":"string"},"controlName":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.relationships.invitation.InvitationRelationshipOption"},"com.linkedin.voyager.dash.deco.common.TapTargetWithoutEntity":{"fields":{"stickerLinkViewUnion":{"type":{"union":{"mediumTemplate":"com.linkedin.deco.recipe.anonymous.Anon739258202","largeTemplate":"com.linkedin.deco.recipe.anonymous.Anon2003298771","smallTemplate":"com.linkedin.deco.recipe.anonymous.Anon548743587"}}},"stickerLinkTemplateSize":{"type":"string"},"firstCornerXOffsetPercentage":{"type":"float"},"type":{"type":"string"},"thirdCornerYOffsetPercentage":{"type":"float"},"url":{"type":"string"},"urn":{"type":"string"},"thirdCornerXOffsetPercentage":{"type":"float"},"secondCornerYOffsetPercentage":{"type":"float"},"fourthCornerXOffsetPercentage":{"type":"float"},"firstCornerYOffsetPercentage":{"type":"float"},"untaggable":{"type":"boolean"},"rank":{"type":"int"},"text":{"type":"string"},"fourthCornerYOffsetPercentage":{"type":"float"},"secondCornerXOffsetPercentage":{"type":"float"}},"baseType":"com.linkedin.voyager.dash.common.TapTarget"},"com.linkedin.deco.recipe.anonymous.Anon1956213100":{"fields":{"productCategories":{"type":{"array":"string"}},"signatureProduct":{"type":"boolean"},"localizedName":{"type":"string"},"organizationsUsingProduct":{"type":{"map":"com.linkedin.deco.recipe.anonymous.Anon570745354"},"resolvedFrom":"organizationsUsingProductUrns"},"entityUrn":{"type":"string"},"logo":{"type":"com.linkedin.voyager.dash.deco.common.image.ImageViewModel"},"connectionsUsingProductProfiles":{"type":"com.linkedin.deco.recipe.anonymous.Anon1122581856","isInjection":true},"company":{"type":"com.linkedin.deco.recipe.anonymous.Anon1927941263","resolvedFrom":"companyUrn"},"universalName":{"type":"string"},"organizationsUsingProductUrns":{"type":{"array":"string"}},"productCategoriesResolutionResults":{"type":{"map":"com.linkedin.voyager.dash.deco.organization.FullProductCategory"},"resolvedFrom":"productCategories"},"companyUrn":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.organization.OrganizationProduct"},"com.linkedin.voyager.dash.deco.premium.FullPremiumButton":{"fields":{"controlName":{"type":"string"},"openExternally":{"type":"boolean"},"appearance":{"type":"com.linkedin.voyager.dash.deco.common.ux.ButtonAppearance"},"accessibilityText":{"type":"string"},"url":{"type":"string"},"disable":{"type":"boolean"}},"baseType":"com.linkedin.voyager.dash.premium.PremiumButton"},"com.linkedin.deco.recipe.anonymous.Anon1312950162":{"fields":{"entityUrn":{"type":"string"},"memberRelationshipUnion":{"type":{"union":{"self":"com.linkedin.restli.common.EmptyRecord","connection":"com.linkedin.deco.recipe.anonymous.Anon2134644859","noConnection":"com.linkedin.deco.recipe.anonymous.Anon1792705474"}}}},"baseType":"com.linkedin.voyager.dash.relationships.MemberRelationship"},"com.linkedin.voyager.dash.deco.organization.OrganizationPermissions":{"fields":{"canCreateBroadcast":{"type":"boolean"},"canReadContentSuggestions":{"type":"boolean"},"canSeeProducts":{"type":"boolean"},"canUpdateOrganizationProfile":{"type":"boolean"},"canSponsorShare":{"type":"boolean"},"canEditPendingDirectSponsoredContentPosters":{"type":"boolean"},"canReadAdminFeedFollowingPill":{"type":"boolean"},"canUntagFromMention":{"type":"boolean"},"canReadOrganizationActivity":{"type":"boolean"},"canManageCareerPages":{"type":"boolean"},"canAssociateHashtag":{"type":"boolean"},"canDeleteShare":{"type":"boolean"},"canInviteMemberToFollow":{"type":"boolean"},"canEditPipelineBuilderAdministrators":{"type":"boolean"},"canReadBroadcastAnalytics":{"type":"boolean"},"canManageMessagingAccess":{"type":"boolean"},"canMembersInviteToFollow":{"type":"boolean"},"canReadPipelineBuilderAdminPage":{"type":"boolean"},"canDeactivateOrganization":{"type":"boolean"},"canCreateOrganicShare":{"type":"boolean"},"canNotifyEmployeesOfShare":{"type":"boolean"},"canEditDarkShare":{"type":"boolean"},"canSeePostJobButton":{"type":"boolean"},"canEnableCommentsShare":{"type":"boolean"},"canReadOrganizationLeadsAnalytics":{"type":"boolean"},"canDeleteDarkShare":{"type":"boolean"},"canCreateJobPosting":{"type":"boolean"},"canEditLeadGenerationFormManagers":{"type":"boolean"},"canReadMessages":{"type":"boolean"},"canReadOrganizationTalentBrandAnalytics":{"type":"boolean"},"canEditDirectSponsoredContentPosters":{"type":"boolean"},"canReadOrganizationNewsletterAnalytics":{"type":"boolean"},"canReadAdminDashboard":{"type":"boolean"},"canPinShare":{"type":"boolean"},"canReadPipelineBuilderAdministrators":{"type":"boolean"},"canCreateLinkedInPagesProductFeedBack":{"type":"boolean"},"canReadEvents":{"type":"boolean"},"organizationRoles":{"type":{"array":"com.linkedin.voyager.dash.deco.organization.OrganizationRoleType"}},"canEditEvents":{"type":"boolean"},"canManageOrganizationalPageFollow":{"type":"boolean"},"canReadOrganizationUpdateAnalytics":{"type":"boolean"},"canCreateComment":{"type":"boolean"},"canRequestAdminAccess":{"type":"boolean"},"canEditAdministrators":{"type":"boolean"},"canCreateDarkShare":{"type":"boolean"},"canEmployeesInviteToFollow":{"type":"boolean"},"canSeeEmployeeExperienceAsMember":{"type":"boolean"},"canReadAdministrators":{"type":"boolean"},"canExportLeads":{"type":"boolean"},"canReadDirectSponsoredContentPosters":{"type":"boolean"},"canCreateShowcase":{"type":"boolean"},"canManageEmployeeExperienceSettings":{"type":"boolean"},"canReadLeadGenerationFormManagers":{"type":"boolean"},"canReadOrganizationVisitorAnalytics":{"type":"boolean"},"canEditCurators":{"type":"boolean"},"canEditProducts":{"type":"boolean"},"canManageVerifiedEmailDomains":{"type":"boolean"},"canReadPendingAdministrators":{"type":"boolean"},"canReadPendingDirectSponsoredContentPosters":{"type":"boolean"},"canEditPipelineBuilderAdminPage":{"type":"boolean"},"canReadTermsAndAgreements":{"type":"boolean"},"canReadOrganizationFollowerAnalytics":{"type":"boolean"},"canManageOrganizationCompetitors":{"type":"boolean"},"canSendMessages":{"type":"boolean"},"canCreateReaction":{"type":"boolean"},"canDisableCommentsShare":{"type":"boolean"},"canEditPendingAdministrators":{"type":"boolean"},"canNotifyEmployees":{"type":"boolean"},"canReadOrganizationPipelineBuilderAnalytics":{"type":"boolean"},"canSeeOrganizationAdministrativePage":{"type":"boolean"}},"baseType":"com.linkedin.voyager.dash.organization.OrganizationPermissions"},"com.linkedin.voyager.dash.deco.common.video.Transcript":{"fields":{"captionFormat":{"type":"string"},"locale":{"type":"com.linkedin.voyager.dash.deco.common.Locale"},"lines":{"type":{"array":"com.linkedin.voyager.dash.deco.common.video.TranscriptLine"}},"isAutogenerated":{"type":"boolean"},"captionFile":{"type":"string"}},"baseType":"com.linkedin.videocontent.Transcript"},"com.linkedin.deco.recipe.anonymous.Anon1286825962":{"fields":{"sharedSecret":{"type":"string"},"entityUrn":{"type":"string"},"invitationId":{"type":"long"}},"baseType":"com.linkedin.voyager.dash.relationships.invitation.Invitation"},"com.linkedin.voyager.dash.deco.common.image.OrganizationalPage":{"fields":{"entityUrn":{"type":"string"},"pageType":{"type":"string"},"logoV2":{"type":"com.linkedin.voyager.dash.deco.common.VectorImageOnlyRootUrlAndAttribution"}},"baseType":"com.linkedin.voyager.dash.organization.OrganizationalPage"},"com.linkedin.voyager.dash.deco.premium.PremiumUpsellCard":{"fields":{"image":{"type":"com.linkedin.voyager.dash.deco.common.image.ImageViewModel"},"largeTitle":{"type":"com.linkedin.voyager.dash.deco.common.text.TextViewModelV2"},"socialProofInsight":{"type":"com.linkedin.voyager.dash.deco.common.ux.baseInsightViewModel"},"subtext":{"type":"com.linkedin.voyager.dash.deco.common.text.TextViewModelV2"},"upsellOrderOrigin":{"type":"string"},"ctaText":{"type":"com.linkedin.voyager.dash.deco.common.text.TextViewModelV2"},"controlName":{"type":"string"},"actionUrl":{"type":"string"},"legoTrackingToken":{"type":"string"},"title":{"type":"com.linkedin.voyager.dash.deco.common.text.TextViewModelV2"},"globalLegoTrackingToken":{"type":"string"},"cancelCta":{"type":"com.linkedin.voyager.dash.deco.premium.FullPremiumButton"},"highlightedValues":{"type":{"array":"com.linkedin.voyager.dash.deco.premium.plans.FullPremiumFeature"}},"promotionLegoTrackingToken":{"type":"string"},"funnelCommonHeader":{"type":"com.linkedin.voyager.dash.deco.premium.FullPremiumFunnelCommonHeader"},"fireDiscoveryImpressionEvent":{"type":"boolean"},"subtitle":{"type":"com.linkedin.voyager.dash.deco.common.text.TextViewModelV2"},"dismissible":{"type":"boolean"},"layoutStyle":{"type":"string"},"ctaButtonSecondaryEmphasizedTheme":{"type":"boolean"},"colorScheme":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.premium.PremiumUpsellCard"},"com.linkedin.deco.recipe.anonymous.Anon1358424483":{"fields":{"text":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.sponsoredcontent.leadgen.TextFieldComponent"},"com.linkedin.deco.recipe.anonymous.Anon632521550":{"fields":{"entityUrn":{"type":"string"},"joinedAt":{"type":"long"},"profile":{"type":"com.linkedin.deco.recipe.anonymous.Anon437558093","resolvedFrom":"profileUrn"},"groupUrn":{"type":"string"},"preDashEntityUrn":{"type":"string"},"profileUrn":{"type":"string"},"status":{"type":"string"},"group":{"type":"com.linkedin.deco.recipe.anonymous.Anon1869367056","resolvedFrom":"groupUrn"}},"baseType":"com.linkedin.voyager.dash.groups.GroupMembership"},"com.linkedin.deco.recipe.anonymous.Anon523036483":{"fields":{"title":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.organization.FollowersMetadata"},"com.linkedin.deco.recipe.anonymous.Anon1949606970":{"fields":{"followerResolutionResult":{"type":"com.linkedin.deco.recipe.anonymous.Anon1313960689","resolvedFrom":"follower"},"followedAt":{"type":"com.linkedin.voyager.dash.deco.common.text.TextViewModelV2"},"follower":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.organization.OrganizationFollower"},"com.linkedin.deco.recipe.anonymous.Anon407602795":{"fields":{"requiredFieldMissingErrorText":{"type":"string"},"formComponentUnion":{"type":{"union":{"textFieldComponent":"com.linkedin.deco.recipe.anonymous.Anon1358424483","dropdownSelectComponent":"com.linkedin.deco.recipe.anonymous.Anon8381687","textInputComponent":"com.linkedin.deco.recipe.anonymous.Anon1619791307","checkboxComponent":"com.linkedin.deco.recipe.anonymous.Anon554507693"}}},"question":{"type":"com.linkedin.voyager.dash.deco.common.text.TextViewModelV2"},"required":{"type":"boolean"},"editable":{"type":"boolean"},"leadGenFormQuestionUrn":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.sponsoredcontent.leadgen.LeadGenFormQuestion"},"com.linkedin.voyager.dash.deco.common.forms.DateFormComponent":{"fields":{"dateText":{"type":"com.linkedin.voyager.dash.deco.common.text.TextViewModelV2"},"controlName":{"type":"string"},"dateValidation":{"type":"com.linkedin.voyager.dash.deco.common.forms.DateValidation"},"dateInputType":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.common.forms.DateFormComponent"},"com.linkedin.voyager.dash.deco.common.image.Group":{"fields":{"logo":{"type":{"union":{"url":"string","vectorImage":"com.linkedin.voyager.dash.deco.common.VectorImageOnlyRootUrlAndAttribution"}}},"entityUrn":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.groups.Group"},"com.linkedin.deco.recipe.anonymous.Anon1918694303":{"fields":{"entityUrn":{"type":"string"},"defaultLocalizedName":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.common.Geo"},"com.linkedin.voyager.dash.deco.launchpad.OrganizationTopCardForEntityFollow":{"fields":{"socialProofInsight":{"type":{"union":{"standard":"com.linkedin.deco.recipe.anonymous.Anon1350307109","compact":"com.linkedin.deco.recipe.anonymous.Anon853977300"}},"resolvedFrom":"socialProofInsightUnion"},"viewerPermissions":{"type":"com.linkedin.voyager.dash.deco.launchpad.OrganizationPermissions"},"salesNavigatorCompanyUrl":{"type":"string"},"industryV2Taxonomy":{"type":{"map":"com.linkedin.voyager.dash.deco.common.IndustryV2"},"resolvedFrom":"industryV2TaxonomyUrns"},"industry":{"type":{"map":"com.linkedin.voyager.dash.deco.common.Industry"},"resolvedFrom":"industryUrns"},"industryUrns":{"type":{"array":"string"}},"industryV2Urns":{"type":{"array":"string"}},"socialProofInsightUnion":{"type":{"union":{"standard":"string","compact":"string"}}},"employeeCount":{"type":"long"},"associatedSignatureProductUrn":{"type":"string"},"url":{"type":"string"},"callToAction":{"type":"com.linkedin.voyager.dash.deco.launchpad.CallToAction"},"relevanceReason":{"type":"com.linkedin.voyager.dash.deco.common.ux.baseInsightViewModel"},"associatedSignatureProduct":{"type":"com.linkedin.voyager.dash.deco.organization.FullMemberProduct","resolvedFrom":"associatedSignatureProductUrn"},"industryV2":{"type":{"map":"com.linkedin.voyager.dash.deco.common.Industry"},"resolvedFrom":"industryV2Urns"},"industryV2TaxonomyUrns":{"type":{"array":"string"}},"followingStateUrn":{"type":"string"},"entityUrn":{"type":"string"},"followingState":{"type":"com.linkedin.voyager.dash.deco.feed.FollowingState","resolvedFrom":"followingStateUrn"},"name":{"type":"string"},"logo":{"type":{"union":{"url":"string","vectorImage":"com.linkedin.voyager.dash.deco.common.VectorImageOnlyRootUrlAndAttribution"}}},"universalName":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.organization.Company"},"com.linkedin.deco.recipe.anonymous.Anon1213723597":{"fields":{"entityUrn":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.events.ProfessionalEvent"},"com.linkedin.voyager.dash.deco.premium.FullPremiumFunnelCommonHeader":{"fields":{"utype":{"type":"string"},"referenceId":{"type":"bytes"}},"baseType":"com.linkedin.voyager.dash.premium.PremiumFunnelCommonHeader"},"com.linkedin.deco.recipe.anonymous.Anon184130218":{"fields":{"name":{"type":"string"},"logo":{"type":{"union":{"url":"string","vectorImage":"com.linkedin.voyager.dash.deco.common.VectorImageOnlyRootUrlAndAttribution"}}},"entityUrn":{"type":"string"},"url":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.organization.Company"},"com.linkedin.deco.recipe.anonymous.Anon487759616":{"fields":{"company":{"type":"com.linkedin.deco.recipe.anonymous.Anon1226238669","resolvedFrom":"companyUrn"},"schoolName":{"type":"string"},"school":{"type":"com.linkedin.deco.recipe.anonymous.Anon1621960601","resolvedFrom":"schoolUrn"},"entityUrn":{"type":"string"},"companyUrn":{"type":"string"},"schoolUrn":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.identity.profile.Education"},"com.linkedin.deco.recipe.anonymous.Anon1251747613":{"fields":{"urn":{"type":"string"},"modelName":{"type":"string"},"fieldName":{"type":"string"},"value":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.common.StringFieldReference"},"com.linkedin.voyager.dash.deco.organization.FullMemberProduct":{"fields":{"organizationsUsingProduct":{"type":{"map":"com.linkedin.deco.recipe.anonymous.Anon1432799903"},"resolvedFrom":"organizationsUsingProductUrns"},"productUserTitle":{"type":{"map":"com.linkedin.deco.recipe.anonymous.Anon1369245809"},"resolvedFrom":"productUserTitleUrns"},"companyUrn":{"type":"string"},"productUserTitleUrns":{"type":{"array":"string"}},"similarProducts":{"type":"com.linkedin.deco.recipe.anonymous.Anon975174582","isInjection":true},"entityUrn":{"type":"string"},"organizationalPage":{"type":"com.linkedin.deco.recipe.anonymous.Anon1196575529","resolvedFrom":"organizationalPageUrn"},"logo":{"type":"com.linkedin.voyager.dash.deco.common.image.ImageViewModel"},"similarProductsFromSameOrganization":{"type":"com.linkedin.deco.recipe.anonymous.Anon1914510571","isInjection":true},"company":{"type":"com.linkedin.deco.recipe.anonymous.Anon995204871","resolvedFrom":"companyUrn"},"productCardCallToActionUnion":{"type":{"union":{"organizationProductLeadGenFormCallToAction":"com.linkedin.deco.recipe.anonymous.Anon346909806","organizationProductCallToAction":"com.linkedin.deco.recipe.anonymous.Anon625674184","organizationProductAddSkillCallToAction":"com.linkedin.deco.recipe.anonymous.Anon2115336743"}}},"state":{"type":"string"},"universalName":{"type":"string"},"groupUrn":{"type":"string"},"localizedDescription":{"type":"string"},"localizedWebsite":{"type":"string"},"customizableCallToActionUnion":{"type":{"union":{"organizationProductLeadGenFormCallToAction":"com.linkedin.deco.recipe.anonymous.Anon346909806","organizationProductCallToAction":"com.linkedin.deco.recipe.anonymous.Anon625674184","organizationProductAddSkillCallToAction":"com.linkedin.deco.recipe.anonymous.Anon2115336743"}}},"group":{"type":"com.linkedin.deco.recipe.anonymous.Anon690575082","resolvedFrom":"groupUrn"},"hashtag":{"type":{"map":"com.linkedin.deco.recipe.anonymous.Anon2070591469"},"resolvedFrom":"hashtagUrns"},"localizedName":{"type":"string"},"connectionsUsingProductProfiles":{"type":"com.linkedin.deco.recipe.anonymous.Anon1517647575","isInjection":true},"mediaSections":{"type":{"array":"com.linkedin.deco.recipe.anonymous.Anon805140200"}},"productCategoriesResolutionResults":{"type":{"map":"com.linkedin.voyager.dash.deco.organization.FullProductCategory"},"resolvedFrom":"productCategories"},"hashtagUrns":{"type":{"array":"string"}},"productCategories":{"type":{"array":"string"}},"memberFacingCallToActionUnion":{"type":{"union":{"organizationProductLeadGenFormCallToAction":"com.linkedin.deco.recipe.anonymous.Anon346909806","organizationProductCallToAction":"com.linkedin.deco.recipe.anonymous.Anon625674184","organizationProductAddSkillCallToAction":"com.linkedin.deco.recipe.anonymous.Anon2115336743"}}},"standardizedSkillUrns":{"type":{"array":"string"}},"organizationalPageUrn":{"type":"string"},"autoPublished":{"type":"boolean"},"organizationsUsingProductUrns":{"type":{"array":"string"}},"standardizedSkill":{"type":{"map":"com.linkedin.deco.recipe.anonymous.Anon503562015"},"resolvedFrom":"standardizedSkillUrns"}},"baseType":"com.linkedin.voyager.dash.organization.OrganizationProduct"},"com.linkedin.voyager.dash.deco.common.forms.PageKey":{"fields":{"pageKey":{"type":"string"},"anchorPage":{"type":"boolean"}},"baseType":"com.linkedin.voyager.dash.common.tracking.PageKey"},"com.linkedin.voyager.dash.deco.launchpad.OrganizationPermissions":{"fields":{"canReadOrganizationVisitorAnalytics":{"type":"boolean"},"canCreateBroadcast":{"type":"boolean"},"canMembersInviteToFollow":{"type":"boolean"},"canSeeOrganizationAdministrativePage":{"type":"boolean"}},"baseType":"com.linkedin.voyager.dash.organization.OrganizationPermissions"},"com.linkedin.deco.recipe.anonymous.Anon1869367056":{"fields":{"entityUrn":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.groups.Group"},"com.linkedin.voyager.dash.deco.identity.profile.ProfileActionsInjection":{"fields":{"secondaryActionResolutionResult":{"type":{"union":{"personalizedConnect":"com.linkedin.deco.recipe.anonymous.Anon4539795","saveToPdf":"com.linkedin.deco.recipe.anonymous.Anon587401631","followingState":"com.linkedin.deco.recipe.anonymous.Anon1760283115","ignore":"com.linkedin.deco.recipe.anonymous.Anon1312950162","composeOption":"com.linkedin.deco.recipe.anonymous.Anon1575907873","withdraw":"com.linkedin.deco.recipe.anonymous.Anon1213132949","saveInSalesNavigatorState":"com.linkedin.deco.recipe.anonymous.Anon1975396536"}},"resolvedFrom":"secondaryAction"},"primaryActionResolutionResult":{"type":{"union":{"personalizedConnect":"com.linkedin.deco.recipe.anonymous.Anon4539795","saveToPdf":"com.linkedin.deco.recipe.anonymous.Anon587401631","followingState":"com.linkedin.deco.recipe.anonymous.Anon1760283115","ignore":"com.linkedin.deco.recipe.anonymous.Anon1312950162","composeOption":"com.linkedin.deco.recipe.anonymous.Anon1575907873","withdraw":"com.linkedin.deco.recipe.anonymous.Anon1213132949","saveInSalesNavigatorState":"com.linkedin.deco.recipe.anonymous.Anon1975396536"}},"resolvedFrom":"primaryAction"},"trackingIdentifier":{"type":"bytes"},"primaryAction":{"type":{"union":{"personalizedConnect":"string","saveToPdf":"string","shareProfileUrlViaMessage":"string","saveToPdfUrl":"string","composeOption":"string","saveInSalesNavigatorState":"string","statefulAction":"com.linkedin.voyager.dash.deco.common.ux.StatefulButtonModel","viewProfileInExternalContext":"com.linkedin.voyager.dash.deco.identity.profile.ViewProfileInExternalContextAction","statelessAction":"string","followingState":"string","report":"com.linkedin.deco.recipe.anonymous.Anon743120773","ignore":"string","shareProfileUrl":"string","connection":"com.linkedin.voyager.dash.deco.identity.profile.MemberRelationshipWrapper","withdraw":"string"}}},"secondaryAction":{"type":{"union":{"personalizedConnect":"string","saveToPdf":"string","shareProfileUrlViaMessage":"string","saveToPdfUrl":"string","composeOption":"string","saveInSalesNavigatorState":"string","statefulAction":"com.linkedin.voyager.dash.deco.common.ux.StatefulButtonModel","viewProfileInExternalContext":"com.linkedin.voyager.dash.deco.identity.profile.ViewProfileInExternalContextAction","statelessAction":"string","followingState":"string","report":"com.linkedin.deco.recipe.anonymous.Anon743120773","ignore":"string","shareProfileUrl":"string","connection":"com.linkedin.voyager.dash.deco.identity.profile.MemberRelationshipWrapper","withdraw":"string"}}},"overflowActionsV2":{"type":{"array":"com.linkedin.deco.recipe.anonymous.Anon956797500"}},"overflowActionsResolutionResults":{"type":{"map":{"union":{"personalizedConnect":"com.linkedin.deco.recipe.anonymous.Anon4539795","saveToPdf":"com.linkedin.deco.recipe.anonymous.Anon587401631","followingState":"com.linkedin.deco.recipe.anonymous.Anon1760283115","ignore":"com.linkedin.deco.recipe.anonymous.Anon1312950162","composeOption":"com.linkedin.deco.recipe.anonymous.Anon1575907873","withdraw":"com.linkedin.deco.recipe.anonymous.Anon1213132949","saveInSalesNavigatorState":"com.linkedin.deco.recipe.anonymous.Anon1975396536"}}},"resolvedFrom":"overflowActions"},"overflowActions":{"type":{"array":{"union":{"personalizedConnect":"string","saveToPdf":"string","shareProfileUrlViaMessage":"string","saveToPdfUrl":"string","composeOption":"string","saveInSalesNavigatorState":"string","statefulAction":"com.linkedin.voyager.dash.deco.common.ux.StatefulButtonModel","viewProfileInExternalContext":"com.linkedin.voyager.dash.deco.identity.profile.ViewProfileInExternalContextAction","statelessAction":"string","followingState":"string","report":"com.linkedin.deco.recipe.anonymous.Anon743120773","ignore":"string","shareProfileUrl":"string","connection":"com.linkedin.voyager.dash.deco.identity.profile.MemberRelationshipWrapper","withdraw":"string"}}}},"secondaryActionV2":{"type":"com.linkedin.deco.recipe.anonymous.Anon956797500"},"trackingId":{"type":"string"},"primaryActionV2":{"type":"com.linkedin.deco.recipe.anonymous.Anon956797500"}},"baseType":"com.linkedin.voyager.dash.identity.profile.actions.ProfileActions"},"com.linkedin.deco.recipe.anonymous.Anon2127563472":{"fields":{"followerCount":{"type":"long"},"entityUrn":{"type":"string"},"preDashFollowingInfoUrn":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.feed.FollowingState"},"com.linkedin.deco.recipe.anonymous.Anon514683669":{"fields":{"currentProgress":{"type":"int"},"threshold":{"type":"int"},"progressText":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.launchpad.LaunchpadProgressMeter"},"com.linkedin.voyager.dash.deco.premium.plans.FullPremiumFeature":{"fields":{"excluded":{"type":"boolean"},"topFeature":{"type":"boolean"},"tooltip":{"type":"com.linkedin.voyager.dash.deco.common.text.TextViewModelV2"},"icon":{"type":"com.linkedin.voyager.dash.deco.common.image.ImageViewModel"},"description":{"type":"string"},"illustration":{"type":"com.linkedin.voyager.dash.deco.common.image.ImageViewModel"},"value":{"type":"int"},"headlineV2":{"type":"com.linkedin.voyager.dash.deco.common.text.TextViewModelV2"},"headline":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.premium.plans.PremiumFeature"},"com.linkedin.voyager.dash.deco.common.sponsored.SponsoredUrlAttributes":{"fields":{"impressionId":{"type":"string"},"unwrappedUrlDomain":{"type":"string"},"creativeId":{"type":"string"},"campaignId":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.common.sponsored.SponsoredUrlAttributes"},"com.linkedin.deco.recipe.anonymous.Anon2086023182":{"fields":{"geo":{"type":"com.linkedin.deco.recipe.anonymous.Anon1444411496","resolvedFrom":"geoUrn"},"geoUrn":{"type":"string"},"postalCode":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.common.forms.LocationInputValue"},"com.linkedin.voyager.dash.deco.relationships.MemberRelationshipV2ForInjection":{"fields":{"memberRelationshipData":{"type":{"union":{"noInvitation":"com.linkedin.deco.recipe.anonymous.Anon1031101697","invitation":"com.linkedin.voyager.dash.deco.relationships.Invitation","connection":"com.linkedin.voyager.dash.deco.relationships.Connection"}}},"entityUrn":{"type":"string"},"memberRelationshipUnion":{"type":{"union":{"self":"com.linkedin.restli.common.EmptyRecord","connection":"com.linkedin.voyager.dash.deco.relationships.Connection","noConnection":"com.linkedin.voyager.dash.deco.relationships.NoConnection"}}}},"baseType":"com.linkedin.voyager.dash.relationships.MemberRelationship"},"com.linkedin.voyager.dash.deco.organization.FullProductCategory":{"fields":{"name":{"type":"com.linkedin.voyager.dash.deco.common.text.TextViewModelV2"},"entityUrn":{"type":"string"},"productCategoryPageUrl":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.organization.ProductCategory"},"com.linkedin.deco.recipe.anonymous.Anon1428941918":{"fields":{"paging":{"type":"com.linkedin.voyager.dash.deco.common.FullPaging"},"elements":{"type":{"array":"com.linkedin.deco.recipe.anonymous.Anon2116040434"}}},"baseType":"com.linkedin.restli.common.CollectionResponse"},"com.linkedin.deco.recipe.anonymous.Anon4539795":{"fields":{"entityUrn":{"type":"string"},"memberRelationshipUnion":{"type":{"union":{"self":"com.linkedin.restli.common.EmptyRecord","connection":"com.linkedin.deco.recipe.anonymous.Anon2134644859","noConnection":"com.linkedin.deco.recipe.anonymous.Anon2058972220"}}}},"baseType":"com.linkedin.voyager.dash.relationships.MemberRelationship"},"com.linkedin.voyager.dash.deco.common.text.ProfileForMention":{"fields":{"entityUrn":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.identity.profile.Profile"},"com.linkedin.deco.recipe.anonymous.Anon870521084":{"fields":{"formElementInputs":{"type":{"array":"com.linkedin.voyager.dash.deco.common.forms.FormElementInput"}}},"baseType":"com.linkedin.voyager.dash.common.forms.PrerequisiteFormInputs"},"com.linkedin.voyager.dash.deco.common.VectorImageOnlyRootUrlAndAttribution":{"fields":{"attribution":{"type":"string"},"rootUrl":{"type":"string"},"artifacts":{"type":{"array":"com.linkedin.voyager.dash.deco.common.VectorArtifact"}}},"baseType":"com.linkedin.common.VectorImage"},"com.linkedin.deco.recipe.anonymous.Anon831664802":{"fields":{"entityUrn":{"type":"string"},"invitationId":{"type":"long"}},"baseType":"com.linkedin.voyager.dash.relationships.invitation.Invitation"},"com.linkedin.deco.recipe.anonymous.Anon2070591469":{"fields":{"actionTarget":{"type":"string"},"entityUrn":{"type":"string"},"followingState":{"type":"com.linkedin.deco.recipe.anonymous.Anon2127563472","isInjection":true},"displayName":{"type":"string"},"trackingUrn":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.feed.Hashtag"},"com.linkedin.deco.recipe.anonymous.Anon2105959233":{"fields":{"maximumFileSizeSupported":{"type":"long"},"mimeTypes":{"type":{"array":"string"}},"uploadFileControlName":{"type":"string"},"uploadFileCtaText":{"type":"string"},"deleteControlName":{"type":"string"},"mediaUploadType":{"type":"string"},"previewText":{"type":"com.linkedin.voyager.dash.deco.common.text.TextViewModelV2"},"helperText":{"type":"com.linkedin.voyager.dash.deco.common.text.TextViewModelV2"}},"baseType":"com.linkedin.voyager.dash.common.forms.MediaUploadFormComponent"},"com.linkedin.voyager.dash.deco.common.Industry":{"fields":{"name":{"type":"string"},"entityUrn":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.common.Industry"},"com.linkedin.deco.recipe.anonymous.Anon1202743511":{"fields":{"entityUrn":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.jobs.StandardizedSkill"},"com.linkedin.deco.recipe.anonymous.Anon912817537":{"fields":{"memberDistance":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.relationships.NoConnection"},"com.linkedin.deco.recipe.anonymous.Anon8381687":{"fields":{"multipleChoiceQuestionOptions":{"type":{"array":"com.linkedin.deco.recipe.anonymous.Anon1507465610"}},"placeholderText":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.sponsoredcontent.leadgen.DropdownSelectComponent"},"com.linkedin.deco.recipe.anonymous.Anon1689150826":{"fields":{"typeaheadFilterQuery":{"type":"com.linkedin.deco.recipe.anonymous.Anon1435489553"},"showFullLastNameForConnections":{"type":"boolean"},"typeaheadUseCase":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.search.TypeaheadQuery"},"com.linkedin.voyager.dash.deco.relationships.InvitationRaw":{"fields":{"genericInvitationType":{"type":"string"},"invitationId":{"type":"long"},"sharedSecret":{"type":"string"},"entityUrn":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.relationships.invitation.Invitation"},"com.linkedin.deco.recipe.anonymous.Anon739258202":{"fields":{"name":{"type":"com.linkedin.voyager.dash.deco.common.text.TextViewModelV2"},"nameSupplementaryInfo":{"type":"com.linkedin.voyager.dash.deco.common.text.TextViewModelV2"},"imageUnion":{"type":{"union":{"url":"string","vectorImage":"com.linkedin.voyager.dash.deco.common.VectorImageOnlyRootUrlAndAttribution"}}},"headline":{"type":"com.linkedin.voyager.dash.deco.common.text.TextViewModelV2"},"subHeadline":{"type":"com.linkedin.voyager.dash.deco.common.text.TextViewModelV2"}},"baseType":"com.linkedin.voyager.dash.common.media.StickerLinkMediumTemplateView"},"com.linkedin.deco.recipe.anonymous.Anon902172343":{"fields":{"topLevelOption":{"type":"com.linkedin.voyager.dash.deco.common.forms.TextSelectableOption"},"subOptions":{"type":{"array":"com.linkedin.voyager.dash.deco.common.forms.TextSelectableOption"}}},"baseType":"com.linkedin.voyager.dash.common.forms.NestedTextSelectableOption"},"com.linkedin.voyager.dash.deco.common.forms.FormSection":{"fields":{"title":{"type":"com.linkedin.voyager.dash.deco.common.text.TextViewModelV2"},"footerText":{"type":"com.linkedin.voyager.dash.deco.common.text.TextViewModelV2"},"collapsedState":{"type":"string"},"formElementGroups":{"type":{"array":"com.linkedin.voyager.dash.deco.common.forms.FormElementGroup"}},"footerTextTitle":{"type":"com.linkedin.voyager.dash.deco.common.text.TextViewModelV2"},"subtitle":{"type":"com.linkedin.voyager.dash.deco.common.text.TextViewModelV2"}},"baseType":"com.linkedin.voyager.dash.common.forms.FormSection"},"com.linkedin.deco.recipe.anonymous.Anon1266426900":{"fields":{"controlName":{"type":"string"},"text":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.common.forms.FormButton"},"com.linkedin.voyager.dash.deco.common.video.AdaptiveStream":{"fields":{"initialBitRate":{"type":"int"},"protocol":{"type":"string"},"mediaType":{"type":"string"},"masterPlaylists":{"type":{"array":"com.linkedin.voyager.dash.deco.common.video.StreamingLocation"}}},"baseType":"com.linkedin.videocontent.AdaptiveStream"},"com.linkedin.deco.recipe.anonymous.Anon1529112441":{"fields":{"originalImageUrl":{"type":"string"},"originalHeight":{"type":"int"},"url":{"type":"string"},"originalWidth":{"type":"int"}},"baseType":"com.linkedin.voyager.dash.common.ImageUrl"},"com.linkedin.voyager.dash.deco.common.video.StreamingLocation":{"fields":{"url":{"type":"string"},"expiresAt":{"type":"long"}},"baseType":"com.linkedin.videocontent.StreamingLocation"},"com.linkedin.voyager.dash.deco.launchpad.LaunchpadView":{"fields":{"launchpadCards":{"type":{"array":"com.linkedin.deco.recipe.anonymous.Anon1353776944"}},"dismissDialog":{"type":"com.linkedin.deco.recipe.anonymous.Anon1670704461"},"headerImage":{"type":"com.linkedin.voyager.dash.deco.common.image.ImageViewModel"},"focusedCardIndex":{"type":"int"},"progressMeter":{"type":"com.linkedin.deco.recipe.anonymous.Anon514683669"},"branding":{"type":"com.linkedin.deco.recipe.anonymous.Anon1963883771"},"dismissible":{"type":"boolean"},"pageKey":{"type":"com.linkedin.voyager.dash.deco.launchpad.PageKey"},"theme":{"type":"string"},"legoTrackingToken":{"type":"string"},"title":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.launchpad.LaunchpadView"},"com.linkedin.voyager.dash.deco.common.video.VideoPlayMetadata":{"fields":{"thumbnail":{"type":"com.linkedin.voyager.dash.deco.common.VectorImageOnlyRootUrl"},"progressiveStreams":{"type":{"array":"com.linkedin.voyager.dash.deco.common.video.ProgressiveDownloadMetadata"}},"liveStreamCreatedAt":{"type":"long"},"prevMedia":{"type":"string"},"transcripts":{"type":{"array":"com.linkedin.voyager.dash.deco.common.video.Transcript"}},"aspectRatio":{"type":"float"},"media":{"type":"string"},"adaptiveStreams":{"type":{"array":"com.linkedin.voyager.dash.deco.common.video.AdaptiveStream"}},"liveStreamEndedAt":{"type":"long"},"duration":{"type":"long"},"provider":{"type":"string"},"entityUrn":{"type":"string"},"nextMedia":{"type":"string"},"thumbnails":{"type":{"array":"com.linkedin.voyager.dash.deco.common.video.Thumbnail"}},"trackingId":{"type":"string"}},"baseType":"com.linkedin.videocontent.VideoPlayMetadata"},"com.linkedin.voyager.dash.deco.identity.profile.ProfilePhotoDecoSpecReadOnlyBackground":{"fields":{"displayImageUrn":{"type":"string"},"displayImageReference":{"type":{"union":{"url":"string","vectorImage":"com.linkedin.voyager.dash.deco.common.VectorImageOnlyRootUrlAndAttribution"}}}},"baseType":"com.linkedin.voyager.dash.identity.profile.PhotoFilterPicture"},"com.linkedin.deco.recipe.anonymous.Anon2058972220":{"fields":{"invitationUnion":{"type":{"union":{"noInvitation":"com.linkedin.deco.recipe.anonymous.Anon412783727","invitation":"com.linkedin.deco.recipe.anonymous.Anon831664802"}}}},"baseType":"com.linkedin.voyager.dash.relationships.NoConnection"},"com.linkedin.deco.recipe.anonymous.Anon561948584":{"fields":{"ctaText":{"type":"string"},"landingPage":{"type":"com.linkedin.deco.recipe.anonymous.Anon1586568086"},"document":{"type":"com.linkedin.deco.recipe.anonymous.Anon856218111"},"confirmationTitle":{"type":"com.linkedin.voyager.dash.deco.common.text.TextViewModelV2"},"urlViewingBehavior":{"type":"string"},"sponsoredUrlAttributes":{"type":"com.linkedin.voyager.dash.deco.common.sponsored.SponsoredUrlAttributes"},"confirmationDescription":{"type":"com.linkedin.voyager.dash.deco.common.text.TextViewModelV2"}},"baseType":"com.linkedin.voyager.dash.sponsoredcontent.leadgen.LeadGenGatedContent"},"com.linkedin.voyager.dash.deco.premium.PremiumUpsellSlotContent":{"fields":{"entityUrn":{"type":"string"},"upsellCard":{"type":"com.linkedin.voyager.dash.deco.premium.PremiumUpsellCard"}},"baseType":"com.linkedin.voyager.dash.premium.PremiumUpsellSlotContent"},"com.linkedin.voyager.dash.deco.relationships.Invitation":{"fields":{"inviteeMemberResolutionResult":{"type":"com.linkedin.deco.recipe.anonymous.Anon336652209","resolvedFrom":"inviteeMember"},"invitationType":{"type":"string"},"inviteeMember":{"type":"string"},"entityUrn":{"type":"string"},"invitationState":{"type":"string"},"invitationId":{"type":"long"},"sharedSecret":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.relationships.invitation.Invitation"},"com.linkedin.voyager.dash.deco.common.IndustryV2":{"fields":{"name":{"type":"string"},"entityUrn":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.identity.profile.IndustryV2"},"com.linkedin.voyager.dash.deco.launchpad.ProfileTopCardForEntityFollow":{"fields":{"lastName":{"type":"string"},"profileTopEducation":{"type":"com.linkedin.deco.recipe.anonymous.Anon964860152","isInjection":true},"profilePositions":{"type":"com.linkedin.deco.recipe.anonymous.Anon1861880920","isInjection":true},"educationOnProfileTopCardShown":{"type":"boolean"},"profileTopPosition":{"type":"com.linkedin.deco.recipe.anonymous.Anon1449981882","isInjection":true},"associatedHashtagsCopy":{"type":"com.linkedin.voyager.dash.deco.common.text.TextViewModelV2"},"profilePicture":{"type":"com.linkedin.voyager.dash.deco.identity.profile.ProfilePhotoDecoSpecReadOnlyProfilePicture"},"firstName":{"type":"string"},"profileInsight":{"type":"com.linkedin.deco.recipe.anonymous.Anon1439603308","isInjection":true},"geoLocation":{"type":"com.linkedin.deco.recipe.anonymous.Anon708807917"},"entityUrn":{"type":"string"},"followingState":{"type":"com.linkedin.deco.recipe.anonymous.Anon89639392","isInjection":true},"memberRelationship":{"type":"com.linkedin.voyager.dash.deco.relationships.MemberRelationshipV2ForInjection","isInjection":true},"companyNameOnProfileTopCardShown":{"type":"boolean"},"headline":{"type":"string"},"publicIdentifier":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.identity.profile.Profile"},"com.linkedin.deco.recipe.anonymous.Anon1662119579":{"fields":{"singleQuestionSubForm":{"type":"com.linkedin.deco.recipe.anonymous.Anon795093289"},"icon":{"type":"com.linkedin.voyager.dash.deco.common.image.ImageViewModel"},"controlName":{"type":"string"},"iconAfterText":{"type":"boolean"},"placeHolderText":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.common.forms.VisibilitySettingButton"},"com.linkedin.deco.recipe.anonymous.Anon1069349607":{"fields":{"entityUrn":{"type":"string"},"memberRelationshipUnion":{"type":{"union":{"self":"com.linkedin.restli.common.EmptyRecord","connection":"com.linkedin.deco.recipe.anonymous.Anon2134644859","noConnection":"com.linkedin.deco.recipe.anonymous.Anon912817537"}}}},"baseType":"com.linkedin.voyager.dash.relationships.MemberRelationship"},"com.linkedin.deco.recipe.anonymous.Anon554507693":{"fields":{"checked":{"type":"boolean"},"text":{"type":"com.linkedin.voyager.dash.deco.common.text.TextViewModelV2"}},"baseType":"com.linkedin.voyager.dash.sponsoredcontent.leadgen.CheckboxComponent"},"com.linkedin.deco.recipe.anonymous.Anon1128782547":{"fields":{"entityUrn":{"type":"string"},"joinedAt":{"type":"long"},"profile":{"type":"com.linkedin.deco.recipe.anonymous.Anon437558093","resolvedFrom":"profileUrn"},"groupUrn":{"type":"string"},"preDashEntityUrn":{"type":"string"},"profileUrn":{"type":"string"},"status":{"type":"string"},"group":{"type":"com.linkedin.deco.recipe.anonymous.Anon1869367056","resolvedFrom":"groupUrn"}},"baseType":"com.linkedin.voyager.dash.groups.GroupMembership"},"com.linkedin.deco.recipe.anonymous.Anon1509374374":{"fields":{"displayText":{"type":"string"},"url":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.organization.GenericCallToAction"},"com.linkedin.deco.recipe.anonymous.Anon503562015":{"fields":{"name":{"type":"string"},"entityUrn":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.jobs.StandardizedSkill"},"com.linkedin.deco.recipe.anonymous.Anon109927483":{"fields":{"entityUrn":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.common.forms.FormElement"},"com.linkedin.deco.recipe.anonymous.Anon86528645":{"fields":{"infoText":{"type":"string"},"hintText":{"type":"string"},"typeaheadQuery":{"type":"com.linkedin.deco.recipe.anonymous.Anon1689150826"},"typeaheadQueryContext":{"type":{"array":"string"}},"freeFormTextNotAllowedErrorText":{"type":"string"},"validationMetadataUnion":{"type":{"union":{"pronounsV2":"com.linkedin.voyager.dash.deco.common.forms.StringValidationMetadata","string":"com.linkedin.voyager.dash.deco.common.forms.StringValidationMetadata","textFormInputType":"string","postalCode":"com.linkedin.voyager.dash.deco.common.forms.PostalCodeValidationMetadata","name":"com.linkedin.voyager.dash.deco.common.forms.StringValidationMetadata","pronouns":"string","integer":"com.linkedin.voyager.dash.deco.common.forms.NumericValidationMetadata","decimal":"com.linkedin.voyager.dash.deco.common.forms.NumericValidationMetadata","headline":"com.linkedin.voyager.dash.deco.common.forms.StringValidationMetadata","url":"com.linkedin.deco.recipe.anonymous.Anon906563573"}}},"typeaheadType":{"type":"string"},"freeFormTextAllowed":{"type":"boolean"},"selectionLimitExceededErrorText":{"type":"string"},"clusters":{"type":{"array":"string"}},"navTitle":{"type":"string"},"clustersAtMax":{"type":{"array":"string"}}},"baseType":"com.linkedin.voyager.dash.common.forms.TypeaheadMetadata"},"com.linkedin.deco.recipe.anonymous.Anon708807917":{"fields":{"geo":{"type":"com.linkedin.deco.recipe.anonymous.Anon1444411496","resolvedFrom":"geoUrn"},"geoUrn":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.identity.profile.ProfileGeoLocation"},"com.linkedin.voyager.dash.deco.identity.profile.ProfilePhotoDecoSpecReadOnlyProfilePicture":{"fields":{"displayImageWithFrameReferenceUnion":{"type":{"union":{"url":"string","vectorImage":"com.linkedin.voyager.dash.deco.common.VectorImageOnlyRootUrlAndAttribution"}}},"frameType":{"type":"string"},"a11yText":{"type":"string"},"displayImageUrn":{"type":"string"},"displayImageReference":{"type":{"union":{"url":"string","vectorImage":"com.linkedin.voyager.dash.deco.common.VectorImageOnlyRootUrlAndAttribution"}}}},"baseType":"com.linkedin.voyager.dash.identity.profile.PhotoFilterPicture"},"com.linkedin.deco.recipe.anonymous.Anon858976209":{"fields":{"type":{"type":"string"},"epochAt":{"type":"long"}},"baseType":"com.linkedin.voyager.dash.common.text.EpochTime"},"com.linkedin.deco.recipe.anonymous.Anon1435489553":{"fields":{"geoSearchTypes":{"type":{"array":"string"}},"excludedSkillUrns":{"type":{"array":"string"}},"companyUrns":{"type":{"array":"string"}},"segmentAttribute":{"type":"com.linkedin.deco.recipe.anonymous.Anon462114681","resolvedFrom":"segmentAttributeUrn"},"countryCodes":{"type":{"array":"string"}},"segmentAttributeUrn":{"type":"string"},"excludedSkill":{"type":{"map":"com.linkedin.deco.recipe.anonymous.Anon1202743511"},"resolvedFrom":"excludedSkillUrns"},"countryCodesResolutionResults":{"type":{"map":"com.linkedin.deco.recipe.anonymous.Anon1271139837"},"resolvedFrom":"countryCodes"},"company":{"type":{"map":"com.linkedin.deco.recipe.anonymous.Anon1927941263"},"resolvedFrom":"companyUrns"},"standardizationEntityType":{"type":"string"},"groupUrn":{"type":"string"},"segmentProductSurface":{"type":"string"},"group":{"type":"com.linkedin.deco.recipe.anonymous.Anon1869367056","resolvedFrom":"groupUrn"}},"baseType":"com.linkedin.voyager.dash.search.TypeaheadFilterQuery"},"com.linkedin.deco.recipe.anonymous.Anon1517647575":{"fields":{"paging":{"type":"com.linkedin.voyager.dash.deco.common.FullPaging"},"metadata":{"type":"com.linkedin.deco.recipe.anonymous.Anon1585981083"},"elements":{"type":{"array":"com.linkedin.deco.recipe.anonymous.Anon1703899306"}}},"baseType":"com.linkedin.restli.common.CollectionResponse"},"com.linkedin.deco.recipe.anonymous.Anon462114681":{"fields":{"entityUrn":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.hiring.SegmentAttribute"},"com.linkedin.voyager.dash.deco.identity.profile.ViewProfileInExternalContextAction":{"fields":{"deeplinkUrl":{"type":"string"},"recruiterContext":{"type":"boolean"}},"baseType":"com.linkedin.voyager.dash.identity.profile.actions.ViewProfileInExternalContextAction"},"com.linkedin.voyager.dash.deco.common.ux.ButtonAppearance":{"fields":{"premiumStyle":{"type":"boolean"},"size":{"type":"string"},"icon":{"type":"string"},"text":{"type":"string"},"iconAfterText":{"type":"boolean"},"emphasize":{"type":"boolean"},"category":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.common.ux.button.ButtonAppearance"},"com.linkedin.voyager.dash.deco.common.forms.ChoicePill":{"fields":{"textSelectableOption":{"type":"com.linkedin.voyager.dash.deco.common.forms.TextSelectableOption"}},"baseType":"com.linkedin.voyager.dash.common.forms.ChoicePill"},"com.linkedin.deco.recipe.anonymous.Anon1843097075":{"fields":{"invitationUnion":{"type":{"union":{"noInvitation":"com.linkedin.deco.recipe.anonymous.Anon881636240","invitation":"com.linkedin.deco.recipe.anonymous.Anon1286825962"}}}},"baseType":"com.linkedin.voyager.dash.relationships.NoConnection"},"com.linkedin.voyager.dash.deco.common.forms.RadioButtonFormComponent":{"fields":{"selectableOptions":{"type":{"array":{"union":{"textSelectableOption":"com.linkedin.voyager.dash.deco.common.forms.TextSelectableOption","entitySelectableOption":"com.linkedin.voyager.dash.deco.common.forms.EntitySelectableOption"}}}},"textSelectableOptions":{"type":{"array":"com.linkedin.voyager.dash.deco.common.forms.TextSelectableOption"}}},"baseType":"com.linkedin.voyager.dash.common.forms.RadioButtonFormComponent"},"com.linkedin.deco.recipe.anonymous.Anon89639392":{"fields":{"followerCount":{"type":"long"},"entityUrn":{"type":"string"},"preDashFollowingInfoUrn":{"type":"string"},"following":{"type":"boolean"}},"baseType":"com.linkedin.voyager.dash.feed.FollowingState"},"com.linkedin.voyager.dash.deco.common.forms.DateRangeFormComponent":{"fields":{"startDateText":{"type":"com.linkedin.voyager.dash.deco.common.text.TextViewModelV2"},"startDateControlName":{"type":"string"},"endDateControlName":{"type":"string"},"ongoingDateRangeFormElementUrn":{"type":"string"},"endDateValidation":{"type":"com.linkedin.voyager.dash.deco.common.forms.DateValidation"},"endDateText":{"type":"com.linkedin.voyager.dash.deco.common.text.TextViewModelV2"},"startDateValidation":{"type":"com.linkedin.voyager.dash.deco.common.forms.DateValidation"},"ongoingDateText":{"type":"com.linkedin.voyager.dash.deco.common.text.TextViewModelV2"},"dateInputType":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.common.forms.DateRangeFormComponent"},"com.linkedin.voyager.dash.deco.common.forms.SingleLineTextFormComponent":{"fields":{"controlName":{"type":"string"},"hintText":{"type":"com.linkedin.voyager.dash.deco.common.text.TextViewModelV2"},"helperText":{"type":"com.linkedin.voyager.dash.deco.common.text.TextViewModelV2"},"validationMetadata":{"type":{"union":{"pronounsV2":"com.linkedin.voyager.dash.deco.common.forms.StringValidationMetadata","string":"com.linkedin.voyager.dash.deco.common.forms.StringValidationMetadata","textFormInputType":"string","postalCode":"com.linkedin.voyager.dash.deco.common.forms.PostalCodeValidationMetadata","name":"com.linkedin.voyager.dash.deco.common.forms.StringValidationMetadata","pronouns":"string","integer":"com.linkedin.voyager.dash.deco.common.forms.NumericValidationMetadata","decimal":"com.linkedin.voyager.dash.deco.common.forms.NumericValidationMetadata","headline":"com.linkedin.voyager.dash.deco.common.forms.StringValidationMetadata","url":"com.linkedin.deco.recipe.anonymous.Anon906563573"}}}},"baseType":"com.linkedin.voyager.dash.common.forms.SingleLineTextFormComponent"},"com.linkedin.voyager.dash.deco.common.video.TranscriptLine":{"fields":{"lineStartAt":{"type":"long"},"caption":{"type":"string"},"lineEndAt":{"type":"long"}},"baseType":"com.linkedin.videocontent.TranscriptLine"},"com.linkedin.voyager.dash.deco.launchpad.CallToAction":{"fields":{"displayText":{"type":"string"},"visible":{"type":"boolean"},"type":{"type":"string"},"url":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.organization.CallToAction"},"com.linkedin.deco.recipe.anonymous.Anon625674184":{"fields":{"type":{"type":"string"},"callToAction":{"type":"com.linkedin.deco.recipe.anonymous.Anon1509374374"}},"baseType":"com.linkedin.voyager.dash.organization.OrganizationProductCallToAction"},"com.linkedin.voyager.dash.deco.common.forms.MultilineTextFormComponent":{"fields":{"controlName":{"type":"string"},"hintText":{"type":"com.linkedin.voyager.dash.deco.common.text.TextViewModelV2"},"numVisibleLines":{"type":"int"},"helperText":{"type":"com.linkedin.voyager.dash.deco.common.text.TextViewModelV2"},"validationMetadata":{"type":{"union":{"pronounsV2":"com.linkedin.voyager.dash.deco.common.forms.StringValidationMetadata","string":"com.linkedin.voyager.dash.deco.common.forms.StringValidationMetadata","textFormInputType":"string","postalCode":"com.linkedin.voyager.dash.deco.common.forms.PostalCodeValidationMetadata","name":"com.linkedin.voyager.dash.deco.common.forms.StringValidationMetadata","pronouns":"string","integer":"com.linkedin.voyager.dash.deco.common.forms.NumericValidationMetadata","decimal":"com.linkedin.voyager.dash.deco.common.forms.NumericValidationMetadata","headline":"com.linkedin.voyager.dash.deco.common.forms.StringValidationMetadata","url":"com.linkedin.deco.recipe.anonymous.Anon906563573"}}}},"baseType":"com.linkedin.voyager.dash.common.forms.MultilineTextFormComponent"},"com.linkedin.voyager.dash.deco.common.forms.FormElementGroup":{"fields":{"subtitle":{"type":"com.linkedin.voyager.dash.deco.common.text.TextViewModelV2"},"spacingType":{"type":"string"},"title":{"type":"string"},"navigationButton":{"type":"com.linkedin.deco.recipe.anonymous.Anon1957375702"},"horizontalOrientation":{"type":"boolean"},"visibilitySettingButton":{"type":"com.linkedin.deco.recipe.anonymous.Anon1662119579"},"formElements":{"type":{"array":"com.linkedin.voyager.dash.deco.common.forms.FormElement"}}},"baseType":"com.linkedin.voyager.dash.common.forms.FormElementGroup"},"com.linkedin.voyager.dash.deco.common.text.OrganizationalPage":{"fields":{"entityUrn":{"type":"string"},"localizedName":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.organization.OrganizationalPage"},"com.linkedin.deco.recipe.anonymous.Anon600254320":{"fields":{"errorText":{"type":"com.linkedin.voyager.dash.deco.common.text.TextViewModelV2"},"validRange":{"type":"com.linkedin.voyager.dash.deco.common.FloatRange"}},"baseType":"com.linkedin.voyager.dash.common.forms.validation.NumericValueRangeValidation"},"com.linkedin.deco.recipe.anonymous.Anon1432799903":{"fields":{"industryV2Taxonomy":{"type":{"map":"com.linkedin.voyager.dash.deco.common.IndustryV2"},"resolvedFrom":"industryV2TaxonomyUrns"},"industry":{"type":{"map":"com.linkedin.voyager.dash.deco.common.Industry"},"resolvedFrom":"industryUrns"},"industryV2Urns":{"type":{"array":"string"}},"industryUrns":{"type":{"array":"string"}},"url":{"type":"string"},"relevanceReason":{"type":"com.linkedin.voyager.dash.deco.common.ux.baseInsightViewModel"},"industryV2":{"type":{"map":"com.linkedin.voyager.dash.deco.common.Industry"},"resolvedFrom":"industryV2Urns"},"industryV2TaxonomyUrns":{"type":{"array":"string"}},"followingStateUrn":{"type":"string"},"entityUrn":{"type":"string"},"followingState":{"type":"com.linkedin.voyager.dash.deco.feed.FollowingState","resolvedFrom":"followingStateUrn"},"name":{"type":"string"},"logo":{"type":{"union":{"url":"string","vectorImage":"com.linkedin.voyager.dash.deco.common.VectorImage"}}},"universalName":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.organization.Company"},"com.linkedin.deco.recipe.anonymous.Anon1031101697":{"fields":{"targetInviteeResolutionResult":{"type":"com.linkedin.voyager.dash.deco.relationships.ProfileWithEmailRequired","resolvedFrom":"targetInvitee"},"inviter":{"type":"string"},"targetInvitee":{"type":"string"},"invitationRelationshipForm":{"type":"com.linkedin.deco.recipe.anonymous.Anon918376989"},"inviterResolutionResult":{"type":"com.linkedin.voyager.dash.deco.relationships.ProfileWithIweWarned","resolvedFrom":"inviter"}},"baseType":"com.linkedin.voyager.dash.relationships.invitation.NoInvitation"},"com.linkedin.deco.recipe.anonymous.Anon1271139837":{"fields":{"entityUrn":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.common.Geo"},"com.linkedin.deco.recipe.anonymous.Anon1313960689":{"fields":{"profilePicture":{"type":"com.linkedin.voyager.dash.deco.identity.profile.ProfilePhotoDecoSpecReadOnlyProfilePicture"},"lastName":{"type":"string"},"insight":{"type":"com.linkedin.deco.recipe.anonymous.Anon137428918","isInjection":true},"firstName":{"type":"string"},"profileProfileActions":{"type":"com.linkedin.voyager.dash.deco.identity.profile.ProfileActionsInjection","isInjection":true},"entityUrn":{"type":"string"},"memberRelationship":{"type":"com.linkedin.voyager.dash.deco.relationships.MemberRelationshipV2ForInjection","isInjection":true},"profileTopPosition":{"type":"com.linkedin.deco.recipe.anonymous.Anon1428941918","isInjection":true},"publicIdentifier":{"type":"string"},"headline":{"type":"string"},"backgroundPicture":{"type":"com.linkedin.voyager.dash.deco.identity.profile.ProfilePhotoDecoSpecReadOnlyBackground"}},"baseType":"com.linkedin.voyager.dash.identity.profile.Profile"},"com.linkedin.deco.recipe.anonymous.Anon163061530":{"fields":{"name":{"type":"string"},"entityUrn":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.groups.Group"},"com.linkedin.voyager.dash.deco.common.forms.CharacterCountRangeValidation":{"fields":{"showCharacterCount":{"type":"boolean"},"errorText":{"type":"com.linkedin.voyager.dash.deco.common.text.TextViewModelV2"},"validRange":{"type":"com.linkedin.voyager.dash.deco.common.IntegerRange"}},"baseType":"com.linkedin.voyager.dash.common.forms.validation.CharacterCountRangeValidation"},"com.linkedin.voyager.dash.deco.relationships.profile":{"fields":{"lastName":{"type":"string"},"firstName":{"type":"string"},"creator":{"type":"boolean"},"entityUrn":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.identity.profile.Profile"},"com.linkedin.voyager.dash.deco.common.text.ProfileForFullName":{"fields":{"lastName":{"type":"string"},"firstName":{"type":"string"},"entityUrn":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.identity.profile.Profile"},"com.linkedin.voyager.dash.deco.common.forms.DismissPill":{"fields":{"textSelectableOption":{"type":"com.linkedin.voyager.dash.deco.common.forms.TextSelectableOption"}},"baseType":"com.linkedin.voyager.dash.common.forms.DismissPill"},"com.linkedin.voyager.dash.deco.common.forms.MultiSelectTypeaheadEntityFormComponent":{"fields":{"typeaheadCta":{"type":"com.linkedin.voyager.dash.deco.common.forms.TypeaheadCta"},"description":{"type":"com.linkedin.voyager.dash.deco.common.text.TextViewModelV2"},"selectedEntities":{"type":{"array":"com.linkedin.voyager.dash.deco.common.forms.EntitySelectableOption"}},"selectionCountRangeValidation":{"type":"com.linkedin.voyager.dash.deco.common.forms.SelectionCountRangeValidation"}},"baseType":"com.linkedin.voyager.dash.common.forms.MultiSelectTypeaheadEntityFormComponent"},"com.linkedin.deco.recipe.anonymous.Anon378816379":{"fields":{"viewer":{"type":"com.linkedin.deco.recipe.anonymous.Anon430310092","resolvedFrom":"viewerUrn"},"company":{"type":"com.linkedin.deco.recipe.anonymous.Anon660720507","resolvedFrom":"companyUrn"},"companyUrn":{"type":"string"},"viewerUrn":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.relationships.MemberToCompanyFollowMetadata"},"com.linkedin.voyager.dash.deco.common.forms.StringValidationMetadata":{"fields":{"characterCountRangeValidation":{"type":"com.linkedin.voyager.dash.deco.common.forms.CharacterCountRangeValidation"},"errorText":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.common.forms.validation.StringValidationMetadata"},"com.linkedin.deco.recipe.anonymous.Anon743120773":{"fields":{"authorProfileId":{"type":"string"},"targetUrn":{"type":"string"},"contentSource":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.common.semaphore.SemaphoreContext"},"com.linkedin.deco.recipe.anonymous.Anon805140200":{"fields":{"mediaContentUnion":{"type":{"union":{"embeddedVideo":"com.linkedin.voyager.dash.deco.organization.EmbeddedVideoDecoSpec","url":"string","videoPlayMetadata":"com.linkedin.voyager.dash.deco.common.video.VideoPlayMetadata","vectorImage":"com.linkedin.voyager.dash.deco.common.VectorImage"}}},"description":{"type":"string"},"mediaEntityUnion":{"type":{"union":{"digitalMediaAssetUrn":"string","articleUrn":"string"}}},"title":{"type":"string"},"mediaSource":{"type":"string"},"primary":{"type":"boolean"}},"baseType":"com.linkedin.voyager.dash.organization.MediaSection"},"com.linkedin.deco.recipe.anonymous.Anon93322792":{"fields":{"entityUrn":{"type":"string"},"organizationalPageUrn":{"type":"string"},"organizationalPage":{"type":"com.linkedin.deco.recipe.anonymous.Anon1196575529","resolvedFrom":"organizationalPageUrn"}},"baseType":"com.linkedin.voyager.dash.organization.Company"},"com.linkedin.deco.recipe.anonymous.Anon964860152":{"fields":{"paging":{"type":"com.linkedin.voyager.dash.deco.common.FullPaging"},"elements":{"type":{"array":"com.linkedin.deco.recipe.anonymous.Anon487759616"}}},"baseType":"com.linkedin.restli.common.CollectionResponse"},"com.linkedin.voyager.dash.deco.relationships.NoConnection":{"fields":{"memberDistance":{"type":"string"},"invitationUnion":{"type":{"union":{"noInvitation":"com.linkedin.deco.recipe.anonymous.Anon1031101697","invitation":"com.linkedin.voyager.dash.deco.relationships.Invitation"}}}},"baseType":"com.linkedin.voyager.dash.relationships.NoConnection"},"com.linkedin.voyager.dash.deco.common.forms.TextSelectableOption":{"fields":{"optionUnion":{"type":{"union":{"optionUrn":"string","optionEnumString":"string"}}},"controlName":{"type":"string"},"optionUrn":{"type":"string"},"optionEnumString":{"type":"string"},"optionText":{"type":"com.linkedin.voyager.dash.deco.common.text.TextViewModelV2"}},"baseType":"com.linkedin.voyager.dash.common.forms.TextSelectableOption"},"com.linkedin.deco.recipe.anonymous.Anon1575907873":{"fields":{"icon":{"type":"com.linkedin.voyager.dash.deco.common.image.ImageViewModel"},"composeNavigationContext":{"type":"com.linkedin.deco.recipe.anonymous.Anon199392175"},"displayText":{"type":"com.linkedin.voyager.dash.deco.common.text.TextViewModelV2"},"textStartIcon":{"type":"com.linkedin.voyager.dash.deco.common.image.ImageViewModel"},"composeOptionType":{"type":"string"},"entityUrn":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.messaging.compose.ComposeOption"},"com.linkedin.deco.recipe.anonymous.Anon40833127":{"fields":{"metadataUnion":{"type":{"union":{"memberToContentSeriesSubscribe":"com.linkedin.deco.recipe.anonymous.Anon489758292"}}},"subscribeActionUrn":{"type":"string"},"subscribeAction":{"type":"com.linkedin.voyager.dash.deco.feed.publishing.SubscribeAction","resolvedFrom":"subscribeActionUrn"}},"baseType":"com.linkedin.voyager.dash.relationships.SubscribeRelationship"},"com.linkedin.deco.recipe.anonymous.Anon2115336743":{"fields":{"displayText":{"type":"string"},"toggledDisplayText":{"type":"string"},"enabled":{"type":"boolean"}},"baseType":"com.linkedin.voyager.dash.organization.OrganizationProductAddSkillCallToAction"},"com.linkedin.deco.recipe.anonymous.Anon336652209":{"fields":{"profilePicture":{"type":"com.linkedin.voyager.dash.deco.identity.profile.ProfilePhotoDecoSpecReadOnlyProfilePicture"},"memorialized":{"type":"boolean"},"lastName":{"type":"string"},"firstName":{"type":"string"},"tempStatus":{"type":"string"},"entityUrn":{"type":"string"},"tempStatusEmoji":{"type":"string"},"publicIdentifier":{"type":"string"},"headline":{"type":"string"},"tempStatusExpiredAtUnion":{"type":{"union":{"customizedExpiredAt":"long","standardizedExpiration":"string"}}}},"baseType":"com.linkedin.voyager.dash.identity.profile.Profile"},"com.linkedin.voyager.dash.deco.common.VectorArtifact":{"fields":{"width":{"type":"int"},"fileIdentifyingUrlPathSegment":{"type":"string"},"expiresAt":{"type":"long"},"height":{"type":"int"}},"baseType":"com.linkedin.common.VectorArtifact"},"com.linkedin.voyager.dash.deco.common.text.School":{"fields":{"name":{"type":"string"},"entityUrn":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.organization.School"},"com.linkedin.deco.recipe.anonymous.Anon2066892616":{"fields":{"invitationType":{"type":"string"},"invitationId":{"type":"long"},"sharedSecret":{"type":"string"},"entityUrn":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.relationships.invitation.Invitation"},"com.linkedin.deco.recipe.anonymous.Anon1350307109":{"fields":{"insightViewee":{"type":"com.linkedin.deco.recipe.anonymous.Anon798864187","resolvedFrom":"insightVieweeUrn"},"insightVieweeUrn":{"type":"string"},"entityUrn":{"type":"string"},"sharedConnectionDetailTargetResolutionResult":{"type":"com.linkedin.deco.recipe.anonymous.Anon798864187","resolvedFrom":"sharedConnectionDetailTarget"},"insightImage":{"type":"com.linkedin.voyager.dash.deco.common.image.ImageViewModel"},"sharedConnectionDetailTarget":{"type":"string"},"text":{"type":"com.linkedin.voyager.dash.deco.common.text.TextViewModelV2"},"navigationUrl":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.relationships.Insight"},"com.linkedin.voyager.dash.deco.common.forms.NumericValidationMetadata":{"fields":{"numericValueRangeValidation":{"type":"com.linkedin.deco.recipe.anonymous.Anon600254320"}},"baseType":"com.linkedin.voyager.dash.common.forms.validation.NumericValidationMetadata"},"com.linkedin.voyager.dash.deco.launchpad.PageKey":{"fields":{"pageKey":{"type":"string"},"anchorPage":{"type":"boolean"}},"baseType":"com.linkedin.voyager.dash.common.tracking.PageKey"},"com.linkedin.voyager.dash.deco.common.forms.ReorderableListFormComponent":{"fields":{"contextualSuggestionsTriggers":{"type":"com.linkedin.deco.recipe.anonymous.Anon2065966944"},"contextualRangeValidations":{"type":{"array":"com.linkedin.deco.recipe.anonymous.Anon1117966179"}},"suggestionUseCase":{"type":"string"},"contextualSuggestionViewModel":{"type":"com.linkedin.deco.recipe.anonymous.Anon1976515688"},"selectionCountRangeValidation":{"type":"com.linkedin.voyager.dash.deco.common.forms.SelectionCountRangeValidation"},"typeaheadCta":{"type":"com.linkedin.voyager.dash.deco.common.forms.TypeaheadCta"},"reorderableItemListUnion":{"type":{"union":{"textItems":{"array":"com.linkedin.voyager.dash.deco.common.forms.TextSelectableOption"}}}},"description":{"type":"com.linkedin.voyager.dash.deco.common.text.TextViewModelV2"},"numberOfInitialItems":{"type":"int"},"trackingId":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.common.forms.ReorderableListFormComponent"},"com.linkedin.deco.recipe.anonymous.Anon430310092":{"fields":{"entityUrn":{"type":"string"},"creator":{"type":"boolean"}},"baseType":"com.linkedin.voyager.dash.identity.profile.Profile"},"com.linkedin.deco.recipe.anonymous.Anon1586568086":{"fields":{"url":{"type":"string"},"text":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.common.Link"},"com.linkedin.deco.recipe.anonymous.Anon1976702492":{"fields":{"iweWarned":{"type":"boolean"},"entityUrn":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.identity.profile.Profile"},"com.linkedin.voyager.dash.deco.launchpad.ShareBoxAction":{"fields":{"shareMessage":{"type":"com.linkedin.voyager.dash.deco.common.text.TextViewModelV2"}},"baseType":"com.linkedin.voyager.dash.launchpad.actionrecommendation.ShareBoxAction"},"com.linkedin.deco.recipe.anonymous.Anon1439603308":{"fields":{"elements":{"type":{"array":"com.linkedin.deco.recipe.anonymous.Anon1122468175"}}},"baseType":"com.linkedin.restli.common.CollectionResponse"},"com.linkedin.deco.recipe.anonymous.Anon1898728769":{"fields":{"countryUrnRequired":{"type":"boolean"},"locationField":{"type":"com.linkedin.voyager.dash.deco.common.forms.SingleTypeaheadEntityFormComponent"},"requiredFieldMissingErrorText":{"type":"com.linkedin.voyager.dash.deco.common.text.TextViewModelV2"},"responseRequired":{"type":"boolean"},"labelText":{"type":"com.linkedin.voyager.dash.deco.common.text.TextViewModelV2"}},"baseType":"com.linkedin.voyager.dash.common.forms.LocationTypeaheadEntityField"},"com.linkedin.deco.recipe.anonymous.Anon1735278252":{"fields":{"controlName":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.common.forms.StarRatingFormComponent"},"com.linkedin.deco.recipe.anonymous.Anon437558093":{"fields":{"profilePicture":{"type":"com.linkedin.voyager.dash.deco.identity.profile.ProfilePhotoDecoSpecReadOnlyProfilePicture"},"memorialized":{"type":"boolean"},"lastName":{"type":"string"},"firstName":{"type":"string"},"tempStatus":{"type":"string"},"entityUrn":{"type":"string"},"memberRelationship":{"type":"com.linkedin.deco.recipe.anonymous.Anon1069349607","isInjection":true},"tempStatusEmoji":{"type":"string"},"publicIdentifier":{"type":"string"},"headline":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.identity.profile.Profile"},"com.linkedin.deco.recipe.anonymous.Anon1507465610":{"fields":{"type":{"type":"string"},"textOption":{"type":"string"},"selected":{"type":"boolean"}},"baseType":"com.linkedin.voyager.dash.sponsoredcontent.leadgen.MultipleChoiceQuestionOptions"},"com.linkedin.deco.recipe.anonymous.Anon409680083":{"fields":{"type":{"type":"string"},"index":{"type":"int"}},"baseType":"com.linkedin.voyager.dash.common.text.ListItemStyle"},"com.linkedin.deco.recipe.anonymous.Anon1792705474":{"fields":{"invitationUnion":{"type":{"union":{"noInvitation":"com.linkedin.deco.recipe.anonymous.Anon881636240","invitation":"com.linkedin.deco.recipe.anonymous.Anon2066892616"}}}},"baseType":"com.linkedin.voyager.dash.relationships.NoConnection"},"com.linkedin.deco.recipe.anonymous.Anon1369245809":{"fields":{"name":{"type":"string"},"entityUrn":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.identity.profile.StandardizedTitle"},"com.linkedin.deco.recipe.anonymous.Anon412783727":{"fields":{"targetInviteeResolutionResult":{"type":"com.linkedin.deco.recipe.anonymous.Anon352881287","resolvedFrom":"targetInvitee"},"inviter":{"type":"string"},"targetInvitee":{"type":"string"},"inviterResolutionResult":{"type":"com.linkedin.deco.recipe.anonymous.Anon1976702492","resolvedFrom":"inviter"}},"baseType":"com.linkedin.voyager.dash.relationships.invitation.NoInvitation"},"com.linkedin.deco.recipe.anonymous.Anon533924488":{"fields":{"insightImage":{"type":"com.linkedin.voyager.dash.deco.common.image.ImageViewModel"},"sharedConnectionDetailTarget":{"type":"string"},"text":{"type":"com.linkedin.voyager.dash.deco.common.text.TextViewModelV2"},"navigationUrl":{"type":"string"},"entityUrn":{"type":"string"},"sharedConnectionDetailTargetResolutionResult":{"type":"com.linkedin.deco.recipe.anonymous.Anon587401631","resolvedFrom":"sharedConnectionDetailTarget"}},"baseType":"com.linkedin.voyager.dash.relationships.Insight"},"com.linkedin.deco.recipe.anonymous.Anon918376989":{"fields":{"title":{"type":"string"},"invitationRelationshipOptions":{"type":{"array":"com.linkedin.deco.recipe.anonymous.Anon934606613"}},"subtitle":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.relationships.invitation.InvitationRelationshipForm"},"com.linkedin.deco.recipe.anonymous.Anon1308136603":{"fields":{"questions":{"type":{"array":"com.linkedin.deco.recipe.anonymous.Anon407602795"}}},"baseType":"com.linkedin.voyager.dash.sponsoredcontent.leadgen.LeadGenFormSection"},"com.linkedin.voyager.dash.deco.common.Locale":{"fields":{"variant":{"type":"string"},"country":{"type":"string"},"language":{"type":"string"}},"baseType":"com.linkedin.common.Locale"},"com.linkedin.voyager.dash.deco.common.forms.ActionableSuggestedEntity":{"fields":{"suggestedEntity":{"type":"com.linkedin.voyager.dash.deco.common.forms.TextSelectableOption"},"navigationUrl":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.common.forms.ActionableSuggestedEntity"},"com.linkedin.voyager.dash.deco.common.VectorImageOnlyRootUrl":{"fields":{"rootUrl":{"type":"string"},"artifacts":{"type":{"array":"com.linkedin.voyager.dash.deco.common.VectorArtifact"}}},"baseType":"com.linkedin.common.VectorImage"},"com.linkedin.voyager.dash.deco.common.video.Resolution":{"fields":{"width":{"type":"int"},"height":{"type":"int"}},"baseType":"com.linkedin.videocontent.Resolution"},"com.linkedin.deco.recipe.anonymous.Anon853977300":{"fields":{"entityUrn":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.relationships.Insight"},"com.linkedin.voyager.dash.deco.common.text.ProfileForFamiliarName":{"fields":{"lastName":{"type":"string"},"firstName":{"type":"string"},"entityUrn":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.identity.profile.Profile"},"com.linkedin.deco.recipe.anonymous.Anon1976515688":{"fields":{"suggestedEntities":{"type":{"array":"com.linkedin.voyager.dash.deco.common.forms.TextSelectableOption"}},"subtitle":{"type":"com.linkedin.voyager.dash.deco.common.text.TextViewModelV2"},"dismissSelectableOption":{"type":"com.linkedin.voyager.dash.deco.common.forms.TextSelectableOption"},"dismissControlName":{"type":"string"},"actionableSuggestedEntities":{"type":{"array":"com.linkedin.voyager.dash.deco.common.forms.ActionableSuggestedEntity"}},"title":{"type":"com.linkedin.voyager.dash.deco.common.text.TextViewModelV2"},"typeaheadFormSuggestionUseCase":{"type":"string"},"navigationButton":{"type":"com.linkedin.deco.recipe.anonymous.Anon1957375702"},"trackingId":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.common.forms.TypeaheadFormSuggestionViewModel"},"com.linkedin.voyager.dash.deco.organization.Profile":{"fields":{"profilePicture":{"type":"com.linkedin.voyager.dash.deco.identity.profile.ProfilePhotoDecoSpecReadOnlyProfilePicture"},"lastName":{"type":"string"},"firstName":{"type":"string"},"publicIdentifier":{"type":"string"},"headline":{"type":"string"},"entityUrn":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.identity.profile.Profile"},"com.linkedin.deco.recipe.anonymous.Anon1525160909":{"fields":{"width":{"type":"int"},"imageUrls":{"type":{"array":"string"}},"height":{"type":"int"}},"baseType":"com.linkedin.documentcontent.DocumentResolutionPages"},"com.linkedin.voyager.dash.deco.common.Link":{"fields":{"type":{"type":"string"},"rel":{"type":"string"},"href":{"type":"string"}},"baseType":"com.linkedin.restli.common.Link"},"com.linkedin.voyager.dash.deco.common.ux.baseInsightViewModel":{"fields":{"image":{"type":"com.linkedin.voyager.dash.deco.common.image.ImageViewModel"},"text":{"type":"com.linkedin.voyager.dash.deco.common.text.TextViewModelV2"}},"baseType":"com.linkedin.voyager.dash.common.ux.InsightViewModel"},"com.linkedin.voyager.dash.deco.common.image.School":{"fields":{"logo":{"type":{"union":{"url":"string","vectorImage":"com.linkedin.voyager.dash.deco.common.VectorImageOnlyRootUrlAndAttribution"}}},"entityUrn":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.organization.School"},"com.linkedin.voyager.dash.deco.common.VectorImage":{"fields":{"attribution":{"type":"string"},"digitalmediaAsset":{"type":"string"},"rootUrl":{"type":"string"},"artifacts":{"type":{"array":"com.linkedin.voyager.dash.deco.common.VectorArtifact"}}},"baseType":"com.linkedin.common.VectorImage"},"com.linkedin.deco.recipe.anonymous.Anon881636240":{"fields":{"inviter":{"type":"string"},"inviterResolutionResult":{"type":"com.linkedin.deco.recipe.anonymous.Anon587401631","resolvedFrom":"inviter"}},"baseType":"com.linkedin.voyager.dash.relationships.invitation.NoInvitation"},"com.linkedin.voyager.dash.deco.relationships.Connection":{"fields":{"createdAt":{"type":"long"},"connectedMemberResolutionResult":{"type":"com.linkedin.deco.recipe.anonymous.Anon336652209","resolvedFrom":"connectedMember"},"connectedMember":{"type":"string"},"entityUrn":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.relationships.Connection"},"com.linkedin.voyager.dash.deco.organization.MiniCompany":{"fields":{"name":{"type":"string"},"logo":{"type":{"union":{"url":"string","vectorImage":"com.linkedin.voyager.dash.deco.common.VectorImage"}}},"entityUrn":{"type":"string"},"url":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.organization.Company"},"com.linkedin.deco.recipe.anonymous.Anon198476569":{"fields":{"viewer":{"type":"com.linkedin.deco.recipe.anonymous.Anon430310092","resolvedFrom":"viewerUrn"},"vieweeFollowingViewer":{"type":"boolean"},"viewerUrn":{"type":"string"},"viewee":{"type":"com.linkedin.voyager.dash.deco.relationships.profile","resolvedFrom":"vieweeUrn"},"vieweeUrn":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.relationships.MemberToMemberFollowMetadata"},"com.linkedin.deco.recipe.anonymous.Anon1703899306":{"fields":{"profilePicture":{"type":"com.linkedin.voyager.dash.deco.identity.profile.ProfilePhotoDecoSpecReadOnlyProfilePicture"},"lastName":{"type":"string"},"firstName":{"type":"string"},"profileProfileActions":{"type":"com.linkedin.voyager.dash.deco.identity.profile.ProfileActionsInjection","isInjection":true},"entityUrn":{"type":"string"},"memberRelationship":{"type":"com.linkedin.voyager.dash.deco.relationships.MemberRelationshipV2ForInjection","isInjection":true},"publicIdentifier":{"type":"string"},"headline":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.identity.profile.Profile"},"com.linkedin.deco.recipe.anonymous.Anon2003298771":{"fields":{"nameSupplementaryInfo":{"type":"com.linkedin.voyager.dash.deco.common.text.TextViewModelV2"},"footerText":{"type":"com.linkedin.voyager.dash.deco.common.text.TextViewModelV2"},"imageUnion":{"type":{"union":{"url":"string","vectorImage":"com.linkedin.voyager.dash.deco.common.VectorImageOnlyRootUrlAndAttribution"}}},"subHeadline":{"type":"com.linkedin.voyager.dash.deco.common.text.TextViewModelV2"},"name":{"type":"com.linkedin.voyager.dash.deco.common.text.TextViewModelV2"},"insightText":{"type":"com.linkedin.voyager.dash.deco.common.text.TextViewModelV2"},"headline":{"type":"com.linkedin.voyager.dash.deco.common.text.TextViewModelV2"},"backgroundImageUnion":{"type":{"union":{"url":"string","vectorImage":"com.linkedin.voyager.dash.deco.common.VectorImageOnlyRootUrlAndAttribution"}}}},"baseType":"com.linkedin.voyager.dash.common.media.StickerLinkLargeTemplateView"},"com.linkedin.voyager.dash.deco.feed.FollowingState":{"fields":{"followeeCount":{"type":"long"},"entityUrn":{"type":"string"},"trackingUrn":{"type":"string"},"preDashFollowingInfoUrn":{"type":"string"},"following":{"type":"boolean"},"followingType":{"type":"string"},"followerCount":{"type":"long"}},"baseType":"com.linkedin.voyager.dash.feed.FollowingState"},"com.linkedin.deco.recipe.anonymous.Anon1914510571":{"fields":{"paging":{"type":"com.linkedin.voyager.dash.deco.common.FullPaging"},"elements":{"type":{"array":"com.linkedin.deco.recipe.anonymous.Anon1956213100"}}},"baseType":"com.linkedin.restli.common.CollectionResponse"},"com.linkedin.voyager.dash.deco.relationships.DirectionalEntityRelationship":{"fields":{"invitationUrn":{"type":"string"},"entityUrn":{"type":"string"},"invitation":{"type":"com.linkedin.voyager.dash.deco.relationships.InvitationRaw","resolvedFrom":"invitationUrn"},"relationshipDataUnion":{"type":{"union":{"follow":"com.linkedin.deco.recipe.anonymous.Anon191738356","subscribe":"com.linkedin.deco.recipe.anonymous.Anon40833127"}}}},"baseType":"com.linkedin.voyager.dash.relationships.DirectionalEntityRelationship"},"com.linkedin.voyager.dash.deco.common.forms.EntitySelectableOption":{"fields":{"optionUrn":{"type":"string"},"entityImage":{"type":"com.linkedin.voyager.dash.deco.common.image.ImageViewModel"},"dismissable":{"type":"boolean"},"shouldRenderTextBeforeImage":{"type":"boolean"},"subtitle":{"type":"com.linkedin.voyager.dash.deco.common.text.TextViewModelV2"},"optionUnion":{"type":{"union":{"optionUrn":"string","optionEnumString":"string"}}},"controlName":{"type":"string"},"title":{"type":"com.linkedin.voyager.dash.deco.common.text.TextViewModelV2"}},"baseType":"com.linkedin.voyager.dash.common.forms.EntitySelectableOption"},"com.linkedin.deco.recipe.anonymous.Anon1568806612":{"fields":{"school":{"type":"com.linkedin.deco.recipe.anonymous.Anon929195650","resolvedFrom":"schoolUrn"},"vectorImage":{"type":"com.linkedin.voyager.dash.deco.common.VectorImageOnlyRootUrlAndAttribution"},"schoolUrn":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.common.image.NonEntitySchoolLogo"},"com.linkedin.deco.recipe.anonymous.Anon1449981882":{"fields":{"paging":{"type":"com.linkedin.voyager.dash.deco.common.FullPaging"},"elements":{"type":{"array":"com.linkedin.deco.recipe.anonymous.Anon1566936928"}}},"baseType":"com.linkedin.restli.common.CollectionResponse"},"com.linkedin.deco.recipe.anonymous.Anon660720507":{"fields":{"name":{"type":"string"},"entityUrn":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.organization.Company"},"com.linkedin.voyager.dash.deco.common.forms.TextEntityListFormComponent":{"fields":{"controlName":{"type":"string"},"pageKey":{"type":"com.linkedin.voyager.dash.deco.common.forms.PageKey"},"placeHolderText":{"type":"com.linkedin.voyager.dash.deco.common.text.TextViewModelV2"},"textSelectableOptions":{"type":{"array":"com.linkedin.voyager.dash.deco.common.forms.TextSelectableOption"}},"helperText":{"type":"com.linkedin.voyager.dash.deco.common.text.TextViewModelV2"},"preferredRenderingStyle":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.common.forms.TextEntityListFormComponent"},"com.linkedin.voyager.dash.deco.relationships.ProfileWithIweWarned":{"fields":{"memorialized":{"type":"boolean"},"lastName":{"type":"string"},"firstName":{"type":"string"},"tempStatus":{"type":"string"},"entityUrn":{"type":"string"},"tempStatusEmoji":{"type":"string"},"iweWarned":{"type":"boolean"},"publicIdentifier":{"type":"string"},"headline":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.identity.profile.Profile"},"com.linkedin.voyager.dash.deco.common.image.ImageViewModel":{"fields":{"attributes":{"type":{"array":"com.linkedin.voyager.dash.deco.common.image.ImageAttribute"}},"actionTarget":{"type":"string"},"totalCount":{"type":"int"},"accessibilityTextAttributes":{"type":{"array":"com.linkedin.voyager.dash.deco.common.text.TextAttributeV2"}},"accessibilityText":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.common.image.ImageViewModel"},"com.linkedin.deco.recipe.anonymous.Anon1957375702":{"fields":{"icon":{"type":"string"},"controlName":{"type":"string"},"text":{"type":"string"},"openExternally":{"type":"boolean"},"navigationUrl":{"type":"string"},"appearance":{"type":"com.linkedin.voyager.dash.deco.common.ux.ButtonAppearance"}},"baseType":"com.linkedin.voyager.dash.common.forms.NavigationButton"},"com.linkedin.deco.recipe.anonymous.Anon648914460":{"fields":{"companyUrn":{"type":"string"},"company":{"type":"com.linkedin.deco.recipe.anonymous.Anon1927941263","resolvedFrom":"companyUrn"},"vectorImage":{"type":"com.linkedin.voyager.dash.deco.common.VectorImageOnlyRootUrlAndAttribution"}},"baseType":"com.linkedin.voyager.dash.common.image.NonEntityCompanyLogo"},"com.linkedin.deco.recipe.anonymous.Anon1042323916":{"fields":{"entityUrn":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.jobs.JobPosting"},"com.linkedin.voyager.dash.deco.common.text.Hashtag":{"fields":{"entityUrn":{"type":"string"},"trackingUrn":{"type":"string"},"actionTarget":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.feed.Hashtag"},"com.linkedin.voyager.dash.deco.common.FullPaging":{"fields":{"start":{"type":"int"},"count":{"type":"int"},"total":{"type":"int"},"links":{"type":{"array":"com.linkedin.voyager.dash.deco.common.Link"}}},"baseType":"com.linkedin.restli.common.CollectionMetadata"},"com.linkedin.deco.recipe.anonymous.Anon1122468175":{"fields":{"insightImage":{"type":"com.linkedin.voyager.dash.deco.common.image.ImageViewModel"},"text":{"type":"com.linkedin.voyager.dash.deco.common.text.TextViewModelV2"},"navigationUrl":{"type":"string"},"entityUrn":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.relationships.Insight"},"com.linkedin.voyager.dash.deco.launchpad.JobSearchQuery":{"fields":{"geo":{"type":"com.linkedin.deco.recipe.anonymous.Anon1271139837","resolvedFrom":"geoUrn"},"keywords":{"type":"string"},"geoUrn":{"type":"string"},"savedSearchId":{"type":"long"}},"baseType":"com.linkedin.voyager.dash.launchpad.attribute.JobSearchQuery"},"com.linkedin.voyager.dash.deco.common.image.ProfessionalEvent":{"fields":{"logoImage":{"type":{"union":{"url":"string","vectorImage":"com.linkedin.voyager.dash.deco.common.VectorImageOnlyRootUrlAndAttribution"}}},"entityUrn":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.events.ProfessionalEvent"},"com.linkedin.voyager.dash.deco.common.video.Thumbnail":{"fields":{"resolution":{"type":"com.linkedin.voyager.dash.deco.common.video.Resolution"},"url":{"type":"string"}},"baseType":"com.linkedin.videocontent.Thumbnail"},"com.linkedin.voyager.dash.deco.common.DateRange":{"fields":{"start":{"type":"com.linkedin.voyager.dash.deco.common.Date"},"end":{"type":"com.linkedin.voyager.dash.deco.common.Date"}},"baseType":"com.linkedin.common.DateRange"},"com.linkedin.deco.recipe.anonymous.Anon301818032":{"fields":{"entityImage":{"type":"com.linkedin.voyager.dash.deco.common.image.ImageViewModel"},"backgroundImage":{"type":"com.linkedin.voyager.dash.deco.common.image.ImageViewModel"},"subtitle":{"type":"com.linkedin.voyager.dash.deco.common.text.TextViewModelV2"},"subDescription":{"type":"com.linkedin.voyager.dash.deco.common.text.TextViewModelV2"},"description":{"type":"com.linkedin.voyager.dash.deco.common.text.TextViewModelV2"},"contentHorizontallyCentered":{"type":"boolean"},"title":{"type":"com.linkedin.voyager.dash.deco.common.text.TextViewModelV2"}},"baseType":"com.linkedin.voyager.dash.sponsoredcontent.leadgen.LeadGenBannerComponent"},"com.linkedin.voyager.dash.deco.organization.EmbeddedVideoDecoSpec":{"fields":{"url":{"type":"string"},"thumbnail":{"type":"com.linkedin.voyager.dash.deco.common.VectorImage"}},"baseType":"com.linkedin.voyager.dash.organization.EmbeddedVideo"},"com.linkedin.deco.recipe.anonymous.Anon1670704461":{"fields":{"title":{"type":"string"},"subTitle":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.launchpad.LaunchpadDismissDialog"},"com.linkedin.voyager.dash.deco.common.Date":{"fields":{"month":{"type":"int"},"year":{"type":"int"},"day":{"type":"int"}},"baseType":"com.linkedin.common.Date"},"com.linkedin.voyager.dash.deco.common.video.ProgressiveDownloadMetadata":{"fields":{"streamingLocations":{"type":{"array":"com.linkedin.voyager.dash.deco.common.video.StreamingLocation"}},"size":{"type":"long"},"bitRate":{"type":"int"},"width":{"type":"int"},"mediaType":{"type":"string"},"mimeType":{"type":"string"},"height":{"type":"int"}},"baseType":"com.linkedin.videocontent.ProgressiveDownloadMetadata"},"com.linkedin.deco.recipe.anonymous.Anon1196575529":{"fields":{"entityUrn":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.organization.OrganizationalPage"},"com.linkedin.voyager.dash.deco.organization.OrganizationRoleType":{"fields":{"localizedName":{"type":"string"},"role":{"type":"string"},"paidMediaRole":{"type":"boolean"},"localizedDescription":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.organization.OrganizationRoleType"},"com.linkedin.deco.recipe.anonymous.Anon906563573":{"fields":{"characterCountRangeValidation":{"type":"com.linkedin.voyager.dash.deco.common.forms.CharacterCountRangeValidation"},"invalidUrlErrorText":{"type":"com.linkedin.voyager.dash.deco.common.text.TextViewModelV2"}},"baseType":"com.linkedin.voyager.dash.common.forms.validation.UrlValidationMetadata"},"com.linkedin.voyager.dash.deco.common.IntegerRange":{"fields":{"start":{"type":"int"},"end":{"type":"int"}},"baseType":"com.linkedin.common.IntegerRange"},"com.linkedin.voyager.dash.deco.common.text.TextViewModelV2":{"fields":{"textDirection":{"type":"string"},"text":{"type":"string"},"attributesV2":{"type":{"array":"com.linkedin.voyager.dash.deco.common.text.TextAttributeV2"}},"accessibilityTextAttributesV2":{"type":{"array":"com.linkedin.voyager.dash.deco.common.text.TextAttributeV2"}},"accessibilityText":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.common.text.TextViewModel"},"com.linkedin.deco.recipe.anonymous.Anon1789236903":{"fields":{"title":{"type":"string"},"entityUrn":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.publishing.ContentSeries"},"com.linkedin.voyager.dash.deco.relationships.MemberRelationshipV2":{"fields":{"memberRelationshipData":{"type":{"union":{"noInvitation":"com.linkedin.deco.recipe.anonymous.Anon1031101697","invitation":"com.linkedin.voyager.dash.deco.relationships.Invitation","connection":"com.linkedin.voyager.dash.deco.relationships.Connection"}}},"entityUrn":{"type":"string"},"memberRelationshipUnion":{"type":{"union":{"self":"com.linkedin.restli.common.EmptyRecord","connection":"com.linkedin.voyager.dash.deco.relationships.Connection","noConnection":"com.linkedin.voyager.dash.deco.relationships.NoConnection"}}}},"baseType":"com.linkedin.voyager.dash.relationships.MemberRelationship"},"com.linkedin.voyager.dash.deco.sponsoredcontent.LeadGenForm":{"fields":{"formSections":{"type":{"array":"com.linkedin.deco.recipe.anonymous.Anon1308136603"}},"submitted":{"type":"boolean"},"entityUrn":{"type":"string"},"privacyPolicy":{"type":"com.linkedin.voyager.dash.deco.common.text.TextViewModelV2"},"postSubmissionContent":{"type":"com.linkedin.deco.recipe.anonymous.Anon561948584"},"lastSubmittedAt":{"type":"long"},"trackingMetadata":{"type":"string"},"header":{"type":"com.linkedin.voyager.dash.deco.common.text.TextViewModelV2"},"banner":{"type":"com.linkedin.deco.recipe.anonymous.Anon301818032"},"submitButtonText":{"type":"string"},"customPrivacyPolicy":{"type":"com.linkedin.voyager.dash.deco.common.text.TextViewModelV2"},"consentSection":{"type":"com.linkedin.deco.recipe.anonymous.Anon1308136603"}},"baseType":"com.linkedin.voyager.dash.sponsoredcontent.leadgen.LeadGenForm"},"com.linkedin.voyager.dash.deco.relationships.ProfileWithEmailRequired":{"fields":{"memorialized":{"type":"boolean"},"lastName":{"type":"string"},"firstName":{"type":"string"},"tempStatus":{"type":"string"},"entityUrn":{"type":"string"},"tempStatusEmoji":{"type":"string"},"publicIdentifier":{"type":"string"},"headline":{"type":"string"},"emailRequired":{"type":"boolean"}},"baseType":"com.linkedin.voyager.dash.identity.profile.Profile"},"com.linkedin.voyager.dash.deco.common.forms.FormElementInput":{"fields":{"evaluationType":{"type":"string"},"formElementInputValues":{"type":{"array":{"union":{"entityInputValue":"com.linkedin.voyager.dash.deco.common.forms.EntityInput","locationInputValue":"com.linkedin.deco.recipe.anonymous.Anon2086023182","integerInputValue":"int","urnInputValue":"string","dateRangeInputValue":"com.linkedin.voyager.dash.deco.common.DateRange","textInputValue":"string","floatInputValue":"float","booleanInputValue":"boolean"}}}},"formElementUrn":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.common.forms.FormElementInput"},"com.linkedin.deco.recipe.anonymous.Anon1975396536":{"fields":{"saved":{"type":"boolean"},"salesListNavigationUrl":{"type":"string"},"entityUrn":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.identity.profile.actions.SalesNavigatorSaveState"},"com.linkedin.deco.recipe.anonymous.Anon929195650":{"fields":{"entityUrn":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.organization.School"},"com.linkedin.voyager.dash.deco.common.forms.CheckboxFormComponent":{"fields":{"textSelectableOptions":{"type":{"array":"com.linkedin.voyager.dash.deco.common.forms.TextSelectableOption"}},"selectionCountRangeValidation":{"type":"com.linkedin.voyager.dash.deco.common.forms.SelectionCountRangeValidation"},"exclusiveSelectableOption":{"type":"com.linkedin.voyager.dash.deco.common.forms.TextSelectableOption"},"subtitle":{"type":"com.linkedin.voyager.dash.deco.common.text.TextViewModelV2"}},"baseType":"com.linkedin.voyager.dash.common.forms.CheckboxFormComponent"},"com.linkedin.voyager.dash.deco.common.forms.PillFormComponent":{"fields":{"cta":{"type":{"union":{"typeaheadCta":"com.linkedin.voyager.dash.deco.common.forms.TypeaheadCta","navigationCta":"com.linkedin.deco.recipe.anonymous.Anon1957375702"}}},"contextualSuggestionsTriggers":{"type":"com.linkedin.deco.recipe.anonymous.Anon2065966944"},"contextualRangeValidations":{"type":{"array":"com.linkedin.deco.recipe.anonymous.Anon1117966179"}},"pillsUnion":{"type":{"union":{"choicePills":{"array":"com.linkedin.voyager.dash.deco.common.forms.ChoicePill"},"dismissPills":{"array":"com.linkedin.voyager.dash.deco.common.forms.DismissPill"},"togglePills":{"array":"com.linkedin.voyager.dash.deco.common.forms.TogglePill"}}}},"contextualSuggestionViewModel":{"type":"com.linkedin.deco.recipe.anonymous.Anon1976515688"},"selectionCountRangeValidation":{"type":"com.linkedin.voyager.dash.deco.common.forms.SelectionCountRangeValidation"},"typeaheadCta":{"type":"com.linkedin.voyager.dash.deco.common.forms.TypeaheadCta"},"numberOfInitialPills":{"type":"int"},"description":{"type":"com.linkedin.voyager.dash.deco.common.text.TextViewModelV2"},"togglePills":{"type":{"array":"com.linkedin.voyager.dash.deco.common.forms.TogglePill"}},"pillSuggestionUseCase":{"type":"string"},"trackingId":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.common.forms.PillFormComponent"},"com.linkedin.deco.recipe.anonymous.Anon1299693827":{"fields":{"paging":{"type":"com.linkedin.voyager.dash.deco.common.FullPaging"},"elements":{"type":{"array":"com.linkedin.deco.recipe.anonymous.Anon1128782547"}}},"baseType":"com.linkedin.restli.common.CollectionResponse"},"com.linkedin.voyager.dash.deco.common.forms.SelectionCountRangeValidation":{"fields":{"lessThanMinErrorText":{"type":"string"},"errorText":{"type":"com.linkedin.voyager.dash.deco.common.text.TextViewModelV2"},"maxReachedInfoText":{"type":"string"},"maxReachedWithExclusionListInfoText":{"type":"string"},"validRange":{"type":"com.linkedin.voyager.dash.deco.common.IntegerRange"},"maxExceededErrorText":{"type":"string"},"exclusionListUnions":{"type":{"array":{"union":{"entityInputValue":"com.linkedin.voyager.dash.deco.common.forms.EntityInput","locationInputValue":"com.linkedin.deco.recipe.anonymous.Anon2086023182","integerInputValue":"int","urnInputValue":"string","dateRangeInputValue":"com.linkedin.voyager.dash.deco.common.DateRange","textInputValue":"string","floatInputValue":"float","booleanInputValue":"boolean"}}}}},"baseType":"com.linkedin.voyager.dash.common.forms.validation.SelectionCountRangeValidation"},"com.linkedin.deco.recipe.anonymous.Anon352881287":{"fields":{"entityUrn":{"type":"string"},"emailRequired":{"type":"boolean"}},"baseType":"com.linkedin.voyager.dash.identity.profile.Profile"},"com.linkedin.deco.recipe.anonymous.Anon1760283115":{"fields":{"entityUrn":{"type":"string"},"preDashFollowingInfoUrn":{"type":"string"},"following":{"type":"boolean"}},"baseType":"com.linkedin.voyager.dash.feed.FollowingState"},"com.linkedin.deco.recipe.anonymous.Anon690575082":{"fields":{"viewerGroupMembership":{"type":"com.linkedin.deco.recipe.anonymous.Anon632521550","resolvedFrom":"viewerGroupMembershipUrn"},"displayNotificationSubscriptionLevelOptions":{"type":"boolean"},"groupPostNotificationsEdgeSetting":{"type":"com.linkedin.deco.recipe.anonymous.Anon735489647","resolvedFrom":"groupPostNotificationsEdgeSettingUrn"},"groupPostNotificationsEdgeSettingUrn":{"type":"string"},"postApprovalEnabled":{"type":"boolean"},"backendGroupUrn":{"type":"string"},"memberCount":{"type":"int"},"owners":{"type":"com.linkedin.deco.recipe.anonymous.Anon1299693827","isInjection":true},"memberConnectionsFacePile":{"type":"com.linkedin.voyager.dash.deco.common.image.ImageViewModel"},"memberConnections":{"type":"com.linkedin.deco.recipe.anonymous.Anon721197842","isInjection":true},"membersFacePile":{"type":"com.linkedin.voyager.dash.deco.common.image.ImageViewModel"},"directJoinEnabled":{"type":"boolean"},"entityUrn":{"type":"string"},"logo":{"type":{"union":{"url":"string","vectorImage":"com.linkedin.voyager.dash.deco.common.VectorImage"}}},"globalNewPostNotificationSettingOn":{"type":"boolean"},"viewerGroupMembershipUrn":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.groups.Group"},"com.linkedin.voyager.dash.deco.common.text.LearningCourse":{"fields":{"title":{"type":"string"},"entityUrn":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.learning.LearningCourse"},"com.linkedin.deco.recipe.anonymous.Anon50101142":{"fields":{"ringStatus":{"type":"com.linkedin.deco.recipe.anonymous.Anon1320789737"},"profileUrn":{"type":"string"},"vectorImage":{"type":"com.linkedin.voyager.dash.deco.common.VectorImageOnlyRootUrlAndAttribution"},"profile":{"type":"com.linkedin.deco.recipe.anonymous.Anon587401631","resolvedFrom":"profileUrn"}},"baseType":"com.linkedin.voyager.dash.common.image.NonEntityProfilePicture"},"com.linkedin.deco.recipe.anonymous.Anon1566936928":{"fields":{"company":{"type":"com.linkedin.deco.recipe.anonymous.Anon1226238669","resolvedFrom":"companyUrn"},"multiLocaleCompanyName":{"type":{"map":"string"}},"entityUrn":{"type":"string"},"dateRange":{"type":"com.linkedin.voyager.dash.deco.common.DateRange"},"companyUrn":{"type":"string"},"companyName":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.identity.profile.Position"},"com.linkedin.deco.recipe.anonymous.Anon1834093919":{"fields":{"locationList":{"type":"com.linkedin.voyager.dash.deco.common.forms.TextEntityListFormComponent"},"requiredFieldMissingErrorText":{"type":"com.linkedin.voyager.dash.deco.common.text.TextViewModelV2"},"responseRequired":{"type":"boolean"},"labelText":{"type":"com.linkedin.voyager.dash.deco.common.text.TextViewModelV2"}},"baseType":"com.linkedin.voyager.dash.common.forms.LocationTextEntityListField"},"com.linkedin.deco.recipe.anonymous.Anon1122581856":{"fields":{"paging":{"type":"com.linkedin.voyager.dash.deco.common.FullPaging"},"metadata":{"type":"com.linkedin.deco.recipe.anonymous.Anon1585981083"},"elements":{"type":{"array":"com.linkedin.voyager.dash.deco.organization.Profile"}}},"baseType":"com.linkedin.restli.common.CollectionResponse"},"com.linkedin.voyager.dash.deco.common.forms.DateValidation":{"fields":{"errorText":{"type":"com.linkedin.voyager.dash.deco.common.text.TextViewModelV2"},"validDateRange":{"type":"com.linkedin.voyager.dash.deco.common.DateRange"}},"baseType":"com.linkedin.voyager.dash.common.forms.validation.DateValidation"},"com.linkedin.deco.recipe.anonymous.Anon2136767122":{"fields":{"requiredFieldMissingErrorText":{"type":"com.linkedin.voyager.dash.deco.common.text.TextViewModelV2"},"responseRequired":{"type":"boolean"},"textField":{"type":"com.linkedin.voyager.dash.deco.common.forms.SingleLineTextFormComponent"},"labelText":{"type":"com.linkedin.voyager.dash.deco.common.text.TextViewModelV2"}},"baseType":"com.linkedin.voyager.dash.common.forms.SingleLineTextField"},"com.linkedin.voyager.dash.deco.common.image.ImageAttribute":{"fields":{"detailData":{"type":{"union":{"profilePictureWithoutFrame":"com.linkedin.voyager.dash.deco.common.image.ProfileWithoutFrame","profilePicture":"com.linkedin.voyager.dash.deco.common.image.Profile","profilePictureWithRingStatus":"com.linkedin.voyager.dash.deco.common.image.ProfileWithRingStatus","companyLogo":"com.linkedin.voyager.dash.deco.common.image.Company","professionalEventLogo":"com.linkedin.voyager.dash.deco.common.image.ProfessionalEvent","organizationalPageLogo":"com.linkedin.voyager.dash.deco.common.image.OrganizationalPage","schoolLogo":"com.linkedin.voyager.dash.deco.common.image.School","groupLogo":"com.linkedin.voyager.dash.deco.common.image.Group"}},"resolvedFrom":"detailDataUnion"},"tintColor":{"type":"string"},"detailDataUnion":{"type":{"union":{"profilePictureWithoutFrame":"string","profilePictureWithRingStatus":"string","companyLogo":"string","icon":"string","systemImage":"string","nonEntityGroupLogo":"com.linkedin.deco.recipe.anonymous.Anon1436383648","organizationalPageLogo":"string","vectorImage":"com.linkedin.voyager.dash.deco.common.VectorImageOnlyRootUrlAndAttribution","nonEntityProfessionalEventLogo":"com.linkedin.deco.recipe.anonymous.Anon377984030","profilePicture":"string","imageUrl":"com.linkedin.deco.recipe.anonymous.Anon1529112441","professionalEventLogo":"string","nonEntitySchoolLogo":"com.linkedin.deco.recipe.anonymous.Anon1568806612","nonEntityCompanyLogo":"com.linkedin.deco.recipe.anonymous.Anon648914460","schoolLogo":"string","groupLogo":"string","ghostImage":"string","nonEntityProfilePicture":"com.linkedin.deco.recipe.anonymous.Anon50101142"}}},"tapTargets":{"type":{"array":"com.linkedin.voyager.dash.deco.common.TapTargetWithoutEntity"}},"scalingType":{"type":"string"},"displayAspectRatio":{"type":"double"}},"baseType":"com.linkedin.voyager.dash.common.image.ImageAttribute"},"com.linkedin.voyager.dash.deco.common.forms.TypeaheadCta":{"fields":{"controlName":{"type":"string"},"renderingStyle":{"type":"string"},"typeaheadType":{"type":"string"},"typeaheadQueryContext":{"type":{"array":"string"}},"typeaheadMetadata":{"type":"com.linkedin.deco.recipe.anonymous.Anon86528645"},"labelText":{"type":"com.linkedin.voyager.dash.deco.common.text.TextViewModelV2"}},"baseType":"com.linkedin.voyager.dash.common.forms.TypeaheadCta"},"com.linkedin.deco.recipe.anonymous.Anon2053503342":{"fields":{"paging":{"type":"com.linkedin.voyager.dash.deco.common.FullPaging"},"metadata":{"type":"com.linkedin.deco.recipe.anonymous.Anon523036483"},"elements":{"type":{"array":"com.linkedin.deco.recipe.anonymous.Anon1949606970"}}},"baseType":"com.linkedin.restli.common.CollectionResponse"},"com.linkedin.voyager.dash.deco.common.image.ProfileWithRingStatus":{"fields":{"ringStatus":{"type":"com.linkedin.deco.recipe.anonymous.Anon1080314395","isInjection":true},"profilePicture":{"type":"com.linkedin.voyager.dash.deco.common.image.PhotoFilterPicture"},"entityUrn":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.identity.profile.Profile"},"com.linkedin.deco.recipe.anonymous.Anon1149738921":{"fields":{"useCase":{"type":"string"},"premiumUpsellSlotUrn":{"type":"string"},"premiumUpsellSlot":{"type":"com.linkedin.voyager.dash.deco.premium.PremiumUpsellSlotContent","resolvedFrom":"premiumUpsellSlotUrn"}},"baseType":"com.linkedin.voyager.dash.common.forms.AiSuggestionBar"},"com.linkedin.voyager.dash.deco.common.image.PhotoFilterPicture":{"fields":{"displayImageWithFrameReferenceUnion":{"type":{"union":{"url":"string","vectorImage":"com.linkedin.voyager.dash.deco.common.VectorImageOnlyRootUrlAndAttribution"}}},"a11yText":{"type":"string"},"displayImageReference":{"type":{"union":{"url":"string","vectorImage":"com.linkedin.voyager.dash.deco.common.VectorImageOnlyRootUrlAndAttribution"}}}},"baseType":"com.linkedin.voyager.dash.identity.profile.PhotoFilterPicture"},"com.linkedin.voyager.dash.deco.common.text.Company":{"fields":{"name":{"type":"string"},"entityUrn":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.organization.Company"},"com.linkedin.deco.recipe.anonymous.Anon462345189":{"fields":{"controlName":{"type":"string"},"viewingBehavior":{"type":"string"},"url":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.common.text.TextLink"},"com.linkedin.voyager.dash.deco.common.forms.TogglePill":{"fields":{"textSelectableOption":{"type":"com.linkedin.voyager.dash.deco.common.forms.TextSelectableOption"},"positiveToggleIcon":{"type":"com.linkedin.voyager.dash.deco.common.image.ImageViewModel"},"negativeToggleIcon":{"type":"com.linkedin.voyager.dash.deco.common.image.ImageViewModel"}},"baseType":"com.linkedin.voyager.dash.common.forms.TogglePill"},"com.linkedin.deco.recipe.anonymous.Anon795093289":{"fields":{"dismissControlName":{"type":"string"},"title":{"type":"string"},"formElement":{"type":"com.linkedin.deco.recipe.anonymous.Anon517727117"},"subtitle":{"type":"com.linkedin.voyager.dash.deco.common.text.TextViewModelV2"}},"baseType":"com.linkedin.voyager.dash.common.forms.SingleQuestionSubForm"},"com.linkedin.deco.recipe.anonymous.Anon956797500":{"fields":{"profileActionUnion":{"type":{"union":{"personalizedConnect":"string","saveToPdf":"string","shareProfileUrlViaMessage":"string","saveToPdfUrl":"string","composeOption":"string","saveInSalesNavigatorState":"string","statefulAction":"com.linkedin.voyager.dash.deco.common.ux.StatefulButtonModel","viewProfileInExternalContext":"com.linkedin.voyager.dash.deco.identity.profile.ViewProfileInExternalContextAction","statelessAction":"string","followingState":"string","report":"com.linkedin.deco.recipe.anonymous.Anon743120773","ignore":"string","shareProfileUrl":"string","connection":"com.linkedin.voyager.dash.deco.identity.profile.MemberRelationshipWrapper","withdraw":"string"}}},"profileAction":{"type":{"union":{"personalizedConnect":"com.linkedin.deco.recipe.anonymous.Anon4539795","saveToPdf":"com.linkedin.deco.recipe.anonymous.Anon587401631","followingState":"com.linkedin.deco.recipe.anonymous.Anon1760283115","ignore":"com.linkedin.deco.recipe.anonymous.Anon1312950162","composeOption":"com.linkedin.deco.recipe.anonymous.Anon1575907873","withdraw":"com.linkedin.deco.recipe.anonymous.Anon1213132949","saveInSalesNavigatorState":"com.linkedin.deco.recipe.anonymous.Anon1975396536"}},"resolvedFrom":"profileActionUnion"},"trackingId":{"type":"bytes"}},"baseType":"com.linkedin.voyager.dash.identity.profile.actions.ProfileActionInfo"},"com.linkedin.deco.recipe.anonymous.Anon995204871":{"fields":{"viewerPermissions":{"type":"com.linkedin.voyager.dash.deco.organization.OrganizationPermissions"},"industryV2Taxonomy":{"type":{"map":"com.linkedin.voyager.dash.deco.common.IndustryV2"},"resolvedFrom":"industryV2TaxonomyUrns"},"productsAccessible":{"type":"boolean"},"croppedCoverImageUnion":{"type":{"union":{"url":"string","vectorImage":"com.linkedin.voyager.dash.deco.common.VectorImage"}}},"industry":{"type":{"map":"com.linkedin.voyager.dash.deco.common.Industry"},"resolvedFrom":"industryUrns"},"industryV2Urns":{"type":{"array":"string"}},"lcpStaffingOrganization":{"type":"boolean"},"industryUrns":{"type":{"array":"string"}},"url":{"type":"string"},"lcpTreatment":{"type":"boolean"},"industryV2":{"type":{"map":"com.linkedin.voyager.dash.deco.common.Industry"},"resolvedFrom":"industryV2Urns"},"relevantFollowersForViewerAndOrganization":{"type":"com.linkedin.deco.recipe.anonymous.Anon2053503342","isInjection":true},"industryV2TaxonomyUrns":{"type":{"array":"string"}},"followingStateUrn":{"type":"string"},"entityUrn":{"type":"string"},"followingState":{"type":"com.linkedin.voyager.dash.deco.feed.FollowingState","resolvedFrom":"followingStateUrn"},"originalCoverImageUnion":{"type":{"union":{"url":"string","vectorImage":"com.linkedin.voyager.dash.deco.common.VectorImage"}}},"name":{"type":"string"},"lcpCustomer":{"type":"boolean"},"logo":{"type":{"union":{"url":"string","vectorImage":"com.linkedin.voyager.dash.deco.common.VectorImage"}}},"universalName":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.organization.Company"},"com.linkedin.deco.recipe.anonymous.Anon2006189319":{"fields":{"postalCodeTextField":{"type":"com.linkedin.deco.recipe.anonymous.Anon2136767122"},"cityTypeaheadField":{"type":"com.linkedin.deco.recipe.anonymous.Anon1898728769"},"useCurrentLocation":{"type":"com.linkedin.deco.recipe.anonymous.Anon1266426900"},"countryField":{"type":"com.linkedin.deco.recipe.anonymous.Anon1898728769"},"cityTextEntityListField":{"type":"com.linkedin.deco.recipe.anonymous.Anon1834093919"}},"baseType":"com.linkedin.voyager.dash.common.forms.LocationFormComponent"},"com.linkedin.deco.recipe.anonymous.Anon735489647":{"fields":{"entityUrn":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.identity.notifications.edgesetting.EdgeSetting"},"com.linkedin.deco.recipe.anonymous.Anon1927941263":{"fields":{"entityUrn":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.organization.Company"},"com.linkedin.voyager.dash.deco.common.image.ProfileWithoutFrame":{"fields":{"profilePicture":{"type":"com.linkedin.voyager.dash.deco.common.image.PhotoFilterPicture"},"entityUrn":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.identity.profile.Profile"},"com.linkedin.deco.recipe.anonymous.Anon587401631":{"fields":{"entityUrn":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.identity.profile.Profile"},"com.linkedin.voyager.dash.deco.common.FloatRange":{"fields":{"start":{"type":"float"},"end":{"type":"float"}},"baseType":"com.linkedin.common.FloatRange"},"com.linkedin.voyager.dash.deco.common.forms.PostalCodeValidationMetadata":{"fields":{"errorText":{"type":"com.linkedin.voyager.dash.deco.common.text.TextViewModelV2"},"countryFormElementUrn":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.common.forms.validation.PostalCodeValidationMetadata"},"com.linkedin.deco.recipe.anonymous.Anon1619791307":{"fields":{"saveResponseAsWorkEmail":{"type":"boolean"},"maxResponseLength":{"type":"int"},"response":{"type":"string"},"validationType":{"type":"string"},"minResponseLength":{"type":"int"}},"baseType":"com.linkedin.voyager.dash.sponsoredcontent.leadgen.TextInputComponent"},"com.linkedin.deco.recipe.anonymous.Anon1621960601":{"fields":{"name":{"type":"string"},"logo":{"type":{"union":{"url":"string","vectorImage":"com.linkedin.voyager.dash.deco.common.VectorImageOnlyRootUrlAndAttribution"}}},"entityUrn":{"type":"string"},"url":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.organization.School"},"com.linkedin.deco.recipe.anonymous.Anon199392175":{"fields":{"paidInMail":{"type":"boolean"},"recipient":{"type":{"map":"com.linkedin.deco.recipe.anonymous.Anon587401631"},"resolvedFrom":"recipientUrns"},"messageRequestContextUrn":{"type":"string"},"recipientUrns":{"type":{"array":"string"}},"targetUrl":{"type":"string"},"existingConversation":{"type":"com.linkedin.deco.recipe.anonymous.Anon807183080","resolvedFrom":"existingConversationUrn"},"existingConversationUrn":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.messaging.compose.ComposeNavigationContext"},"com.linkedin.deco.recipe.anonymous.Anon1320789737":{"fields":{"ringContentType":{"type":"string"},"actionTarget":{"type":"string"},"preDashEntityUrn":{"type":"string"},"entityUrn":{"type":"string"},"emphasized":{"type":"boolean"}},"baseType":"com.linkedin.voyager.dash.common.image.RingStatus"},"com.linkedin.deco.recipe.anonymous.Anon721197842":{"fields":{"paging":{"type":"com.linkedin.voyager.dash.deco.common.FullPaging"},"elements":{"type":{"array":"com.linkedin.deco.recipe.anonymous.Anon1128782547"}}},"baseType":"com.linkedin.restli.common.CollectionResponse"},"com.linkedin.deco.recipe.anonymous.Anon517727117":{"fields":{"urn":{"type":"string"},"prerequisiteInputEvaluationStrategyUnion":{"type":{"union":{"answered":"string","matchAnyPrerequisiteInput":"com.linkedin.deco.recipe.anonymous.Anon870521084","notMatchAnyPrequisiteInput":"com.linkedin.deco.recipe.anonymous.Anon870521084"}}},"formComponent":{"type":{"union":{"gaiTextFormComponent":"com.linkedin.deco.recipe.anonymous.Anon146637213","multilineTextFormComponent":"com.linkedin.voyager.dash.deco.common.forms.MultilineTextFormComponent","textEntityListFormComponent":"com.linkedin.voyager.dash.deco.common.forms.TextEntityListFormComponent","toggleFormComponent":"com.linkedin.voyager.dash.deco.common.forms.ToggleFormComponent","starRatingFormComponent":"com.linkedin.deco.recipe.anonymous.Anon1735278252","nestedCheckboxFormComponent":"com.linkedin.deco.recipe.anonymous.Anon1048186976","mediaUploadFormComponent":"com.linkedin.deco.recipe.anonymous.Anon2105959233","checkboxFormComponent":"com.linkedin.voyager.dash.deco.common.forms.CheckboxFormComponent","dropdownFormComponent":"com.linkedin.voyager.dash.deco.common.forms.DropdownFormComponent","numberInputFormComponent":"com.linkedin.deco.recipe.anonymous.Anon2083470902","reorderableListFormComponent":"com.linkedin.voyager.dash.deco.common.forms.ReorderableListFormComponent","singleTypeaheadEntityFormComponent":"com.linkedin.voyager.dash.deco.common.forms.SingleTypeaheadEntityFormComponent","dateRangeFormComponent":"com.linkedin.voyager.dash.deco.common.forms.DateRangeFormComponent","dateFormComponent":"com.linkedin.voyager.dash.deco.common.forms.DateFormComponent","multiSelectTypeaheadEntityFormComponent":"com.linkedin.voyager.dash.deco.common.forms.MultiSelectTypeaheadEntityFormComponent","locationFormComponent":"com.linkedin.deco.recipe.anonymous.Anon2006189319","singleLineTextFormComponent":"com.linkedin.voyager.dash.deco.common.forms.SingleLineTextFormComponent","radioButtonFormComponent":"com.linkedin.voyager.dash.deco.common.forms.RadioButtonFormComponent","pillFormComponent":"com.linkedin.voyager.dash.deco.common.forms.PillFormComponent"}}},"input":{"type":"com.linkedin.voyager.dash.deco.common.forms.FormElementInput"},"requiredFieldMissingErrorText":{"type":"com.linkedin.voyager.dash.deco.common.text.TextViewModelV2"},"entityUrn":{"type":"string"},"helperLink":{"type":"com.linkedin.voyager.dash.deco.common.text.TextViewModelV2"},"required":{"type":"boolean"},"shouldAlwaysSendBackFormElementInput":{"type":"boolean"}},"baseType":"com.linkedin.voyager.dash.common.forms.FormElement"},"com.linkedin.deco.recipe.anonymous.Anon783667843":{"fields":{"ctaColor":{"type":"string"},"ctaType":{"type":"string"},"ctaStyle":{"type":"string"},"presentationStyle":{"type":"string"},"shareMessage":{"type":"com.linkedin.voyager.dash.deco.common.text.TextViewModelV2"},"ctaTitle":{"type":"com.linkedin.voyager.dash.deco.common.text.TextViewModelV2"},"deeplinkUrl":{"type":"string"},"ctaActionUnion":{"type":{"union":{"redirect":"com.linkedin.voyager.dash.deco.launchpad.RedirectAction","launchpadApiRequest":"com.linkedin.restli.common.EmptyRecord","launchpadApiRequestAndRedirect":"com.linkedin.voyager.dash.deco.launchpad.RedirectAction","shareBox":"com.linkedin.voyager.dash.deco.launchpad.ShareBoxAction","launchpadFormSubmission":"com.linkedin.restli.common.EmptyRecord"}}},"legoActionCategory":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.launchpad.LaunchpadCta"},"com.linkedin.deco.recipe.anonymous.Anon1085906135":{"fields":{"multiLocaleCompanyName":{"type":{"map":"string"}},"entityUrn":{"type":"string"},"dateRange":{"type":"com.linkedin.voyager.dash.deco.common.DateRange"},"companyName":{"type":"string"},"company":{"type":"com.linkedin.deco.recipe.anonymous.Anon184130218","resolvedFrom":"companyUrn"},"title":{"type":"string"},"companyUrn":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.identity.profile.Position"},"com.linkedin.deco.recipe.anonymous.Anon1963883771":{"fields":{"accentType":{"type":"string"},"image":{"type":"com.linkedin.voyager.dash.deco.common.image.ImageViewModel"}},"baseType":"com.linkedin.voyager.dash.launchpad.LaunchpadBranding"},"com.linkedin.deco.recipe.anonymous.Anon807183080":{"fields":{"entityUrn":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.messaging.Conversation"},"com.linkedin.deco.recipe.anonymous.Anon1353776944":{"fields":{"backgroundImage":{"type":"com.linkedin.voyager.dash.deco.common.image.ImageViewModel"},"cardType":{"type":"string"},"icon":{"type":"com.linkedin.voyager.dash.deco.common.image.ImageViewModel"},"pageKey":{"type":"com.linkedin.voyager.dash.deco.launchpad.PageKey"},"completed":{"type":"boolean"},"legoTrackingToken":{"type":"string"},"cardTitle":{"type":"com.linkedin.voyager.dash.deco.common.text.TextViewModelV2"},"ctas":{"type":{"array":"com.linkedin.deco.recipe.anonymous.Anon783667843"}},"animate":{"type":"boolean"},"videoMetadata":{"type":"com.linkedin.voyager.dash.deco.common.video.VideoPlayMetadata"},"cardLabel":{"type":"com.linkedin.voyager.dash.deco.common.text.TextViewModelV2"},"attributeUnion":{"type":{"union":{"followOrganization":"string","launchpad":"string","jobSearchQuery":"com.linkedin.voyager.dash.deco.launchpad.JobSearchQuery","companyJYMBII":"string","generalJYMBII":{"array":"string"},"viewExperienceEntity":"string","followPostCreator":"string","fifWidgetGroupId":"string","connectWithEntity":"string","entityActivityFeed":"string","edgeBuilding":{"array":"com.linkedin.voyager.dash.deco.launchpad.EdgeBuildingCohortReason"},"jobAlertJYMBII":{"array":"string"},"organizationActivityFeed":"string"}}},"insightImage":{"type":"com.linkedin.voyager.dash.deco.common.image.ImageViewModel"},"formSection":{"type":"com.linkedin.voyager.dash.deco.common.forms.FormSection"},"overlayImage":{"type":"com.linkedin.voyager.dash.deco.common.image.ImageViewModel"},"attribute":{"type":{"union":{"followOrganization":"com.linkedin.voyager.dash.deco.launchpad.OrganizationTopCardForEntityFollow","companyJYMBII":"com.linkedin.deco.recipe.anonymous.Anon184130218","generalJYMBII":{"map":"com.linkedin.deco.recipe.anonymous.Anon1042323916"},"connectWithEntity":"com.linkedin.voyager.dash.deco.launchpad.ProfileTopCardForEntityFollow","viewExperienceEntity":"com.linkedin.voyager.dash.deco.launchpad.ProfileTopCardForEntityFollow","entityActivityFeed":"com.linkedin.deco.recipe.anonymous.Anon587401631","followPostCreator":"com.linkedin.voyager.dash.deco.launchpad.ProfileTopCardForEntityFollow","jobAlertJYMBII":{"map":"com.linkedin.deco.recipe.anonymous.Anon1042323916"},"organizationActivityFeed":"com.linkedin.deco.recipe.anonymous.Anon93322792"}},"resolvedFrom":"attributeUnion"},"cardSubtitle":{"type":"com.linkedin.voyager.dash.deco.common.text.TextViewModelV2"},"cardStyle":{"type":"string"},"entitySize":{"type":"int"}},"baseType":"com.linkedin.voyager.dash.launchpad.LaunchpadCard"},"com.linkedin.deco.recipe.anonymous.Anon570745354":{"fields":{"name":{"type":"string"},"logo":{"type":{"union":{"url":"string","vectorImage":"com.linkedin.voyager.dash.deco.common.VectorImage"}}},"entityUrn":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.organization.Company"},"com.linkedin.deco.recipe.anonymous.Anon346909806":{"fields":{"dashLeadGenFormUrn":{"type":"string"},"privacyPolicyUrl":{"type":"string"},"dashLeadGenForm":{"type":"com.linkedin.voyager.dash.deco.sponsoredcontent.LeadGenForm","resolvedFrom":"dashLeadGenFormUrn"},"organizationProductCallToAction":{"type":"com.linkedin.deco.recipe.anonymous.Anon625674184"},"leadGenFormUrn":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.organization.OrganizationProductLeadGenFormCallToAction"},"com.linkedin.voyager.dash.deco.common.image.Profile":{"fields":{"profilePicture":{"type":"com.linkedin.voyager.dash.deco.common.image.PhotoFilterPicture"},"entityUrn":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.identity.profile.Profile"},"com.linkedin.voyager.dash.deco.common.text.JobPosting":{"fields":{"title":{"type":"string"},"entityUrn":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.jobs.JobPosting"},"com.linkedin.voyager.dash.deco.common.text.TextAttributeV2":{"fields":{"start":{"type":"int"},"length":{"type":"int"},"detailData":{"type":{"union":{"jobPostingName":"com.linkedin.voyager.dash.deco.common.text.JobPosting","profileFamiliarName":"com.linkedin.voyager.dash.deco.common.text.ProfileForFamiliarName","groupName":"com.linkedin.deco.recipe.anonymous.Anon163061530","profileFullName":"com.linkedin.voyager.dash.deco.common.text.ProfileForFullName","learningCourseName":"com.linkedin.voyager.dash.deco.common.text.LearningCourse","companyName":"com.linkedin.voyager.dash.deco.common.text.Company","profileMention":"com.linkedin.voyager.dash.deco.common.text.ProfileForMention","schoolName":"com.linkedin.voyager.dash.deco.common.text.School","organizationalPageName":"com.linkedin.voyager.dash.deco.common.text.OrganizationalPage","hashtag":"com.linkedin.voyager.dash.deco.common.text.Hashtag"}},"resolvedFrom":"detailDataUnion"},"detailDataUnion":{"type":{"union":{"jobPostingName":"string","profileFamiliarName":"string","hyperlink":"string","color":"string","companyName":"string","icon":"string","systemImage":"string","epoch":"com.linkedin.deco.recipe.anonymous.Anon858976209","organizationalPageName":"string","textLink":"com.linkedin.deco.recipe.anonymous.Anon462345189","listItemStyle":"com.linkedin.deco.recipe.anonymous.Anon409680083","hyperlinkOpenExternally":"string","listStyle":"string","groupName":"string","profileFullName":"string","stringFieldReference":"com.linkedin.deco.recipe.anonymous.Anon1251747613","learningCourseName":"string","profileMention":"string","style":"string","schoolName":"string","hashtag":"string"}}}},"baseType":"com.linkedin.voyager.dash.common.text.TextAttribute"},"com.linkedin.deco.recipe.anonymous.Anon146637213":{"fields":{"controlName":{"type":"string"},"hintText":{"type":"string"},"aiSuggestionBar":{"type":"com.linkedin.deco.recipe.anonymous.Anon1149738921"},"numVisibleLines":{"type":"int"},"helperText":{"type":"string"},"validationMetadata":{"type":{"union":{"pronounsV2":"com.linkedin.voyager.dash.deco.common.forms.StringValidationMetadata","string":"com.linkedin.voyager.dash.deco.common.forms.StringValidationMetadata","textFormInputType":"string","postalCode":"com.linkedin.voyager.dash.deco.common.forms.PostalCodeValidationMetadata","name":"com.linkedin.voyager.dash.deco.common.forms.StringValidationMetadata","pronouns":"string","integer":"com.linkedin.voyager.dash.deco.common.forms.NumericValidationMetadata","decimal":"com.linkedin.voyager.dash.deco.common.forms.NumericValidationMetadata","headline":"com.linkedin.voyager.dash.deco.common.forms.StringValidationMetadata","url":"com.linkedin.deco.recipe.anonymous.Anon906563573"}}}},"baseType":"com.linkedin.voyager.dash.common.forms.GaiTextFormComponent"},"com.linkedin.deco.recipe.anonymous.Anon975174582":{"fields":{"paging":{"type":"com.linkedin.voyager.dash.deco.common.FullPaging"},"elements":{"type":{"array":"com.linkedin.deco.recipe.anonymous.Anon1956213100"}}},"baseType":"com.linkedin.restli.common.CollectionResponse"},"com.linkedin.deco.recipe.anonymous.Anon1213132949":{"fields":{"entityUrn":{"type":"string"},"memberRelationshipUnion":{"type":{"union":{"self":"com.linkedin.restli.common.EmptyRecord","connection":"com.linkedin.deco.recipe.anonymous.Anon2134644859","noConnection":"com.linkedin.deco.recipe.anonymous.Anon1843097075"}}}},"baseType":"com.linkedin.voyager.dash.relationships.MemberRelationship"},"com.linkedin.voyager.dash.deco.common.ux.RelationshipActionData":{"fields":{"relationshipsTrackingId":{"type":"bytes"},"relationshipData":{"type":{"union":{"connectionOrInvitation":"com.linkedin.voyager.dash.deco.relationships.MemberRelationshipV2","memberToEntityRelationship":"com.linkedin.voyager.dash.deco.relationships.DirectionalEntityRelationship"}},"resolvedFrom":"relationshipDataUnion"},"relationshipDataUnion":{"type":{"union":{"connectionOrInvitation":"string","memberToEntityRelationship":"string"}}},"moduleKey":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.common.relationships.RelationshipActionData"},"com.linkedin.voyager.dash.deco.identity.profile.MemberRelationshipWrapper":{"fields":{"memberRelationshipUrn":{"type":"string"},"memberRelationship":{"type":"com.linkedin.voyager.dash.deco.relationships.MemberRelationshipV2","resolvedFrom":"memberRelationshipUrn"}},"baseType":"com.linkedin.voyager.dash.relationships.MemberRelationshipWrapper"},"com.linkedin.deco.recipe.anonymous.Anon137428918":{"fields":{"elements":{"type":{"array":"com.linkedin.deco.recipe.anonymous.Anon533924488"}}},"baseType":"com.linkedin.restli.common.CollectionResponse"},"com.linkedin.voyager.dash.deco.common.forms.SingleTypeaheadEntityFormComponent":{"fields":{"typeaheadFormSuggestionViewModel":{"type":"com.linkedin.deco.recipe.anonymous.Anon1976515688"},"controlName":{"type":"string"},"typeaheadType":{"type":"string"},"contextualSuggestionQueryParameterUrns":{"type":{"array":"string"}},"helperText":{"type":"com.linkedin.voyager.dash.deco.common.text.TextViewModelV2"},"hintText":{"type":"com.linkedin.voyager.dash.deco.common.text.TextViewModelV2"},"entityImage":{"type":"com.linkedin.voyager.dash.deco.common.image.ImageViewModel"},"typeaheadQueryContext":{"type":{"array":"string"}},"freeFormTextNotAllowedErrorText":{"type":"com.linkedin.voyager.dash.deco.common.text.TextViewModelV2"},"entityName":{"type":"com.linkedin.voyager.dash.deco.common.text.TextViewModelV2"},"validationMetadataUnion":{"type":{"union":{"pronounsV2":"com.linkedin.voyager.dash.deco.common.forms.StringValidationMetadata","string":"com.linkedin.voyager.dash.deco.common.forms.StringValidationMetadata","textFormInputType":"string","postalCode":"com.linkedin.voyager.dash.deco.common.forms.PostalCodeValidationMetadata","name":"com.linkedin.voyager.dash.deco.common.forms.StringValidationMetadata","pronouns":"string","integer":"com.linkedin.voyager.dash.deco.common.forms.NumericValidationMetadata","decimal":"com.linkedin.voyager.dash.deco.common.forms.NumericValidationMetadata","headline":"com.linkedin.voyager.dash.deco.common.forms.StringValidationMetadata","url":"com.linkedin.deco.recipe.anonymous.Anon906563573"}}},"freeFormTextAllowed":{"type":"boolean"},"typeaheadMetadata":{"type":"com.linkedin.deco.recipe.anonymous.Anon86528645"},"ghostImage":{"type":"com.linkedin.voyager.dash.deco.common.image.ImageViewModel"}},"baseType":"com.linkedin.voyager.dash.common.forms.SingleTypeaheadEntityFormComponent"},"com.linkedin.deco.recipe.anonymous.Anon191738356":{"fields":{"metadataUnion":{"type":{"union":{"memberToMemberFollow":"com.linkedin.deco.recipe.anonymous.Anon198476569","memberToCompanyFollow":"com.linkedin.deco.recipe.anonymous.Anon378816379"}}},"followingStateUrn":{"type":"string"},"followingState":{"type":"com.linkedin.voyager.dash.deco.feed.FollowingState","resolvedFrom":"followingStateUrn"}},"baseType":"com.linkedin.voyager.dash.relationships.FollowRelationship"},"com.linkedin.voyager.dash.deco.launchpad.RedirectAction":{"fields":{"presentationStyle":{"type":"string"},"url":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.launchpad.actionrecommendation.RedirectAction"},"com.linkedin.deco.recipe.anonymous.Anon2116040434":{"fields":{"company":{"type":"com.linkedin.voyager.dash.deco.organization.MiniCompany","resolvedFrom":"companyUrn"},"title":{"type":"string"},"multiLocaleTitle":{"type":{"map":"string"}},"entityUrn":{"type":"string"},"companyUrn":{"type":"string"},"companyName":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.identity.profile.Position"},"com.linkedin.deco.recipe.anonymous.Anon856218111":{"fields":{"transcribedDocumentUrl":{"type":"string"},"urn":{"type":"string"},"manifestUrl":{"type":"string"},"scanRequiredForDownload":{"type":"boolean"},"totalPageCount":{"type":"int"},"manifestUrlExpiresAt":{"type":"long"},"title":{"type":"string"},"coverPages":{"type":"com.linkedin.deco.recipe.anonymous.Anon1636615691"},"transcribedDocumentUrlExpiresAt":{"type":"long"}},"baseType":"com.linkedin.documentcontent.Document"},"com.linkedin.deco.recipe.anonymous.Anon1436383648":{"fields":{"groupUrn":{"type":"string"},"vectorImage":{"type":"com.linkedin.voyager.dash.deco.common.VectorImageOnlyRootUrlAndAttribution"},"group":{"type":"com.linkedin.deco.recipe.anonymous.Anon1869367056","resolvedFrom":"groupUrn"}},"baseType":"com.linkedin.voyager.dash.common.image.NonEntityGroupLogo"},"com.linkedin.deco.recipe.anonymous.Anon548743587":{"fields":{"name":{"type":"com.linkedin.voyager.dash.deco.common.text.TextViewModelV2"},"imageUnion":{"type":{"union":{"url":"string","vectorImage":"com.linkedin.voyager.dash.deco.common.VectorImageOnlyRootUrlAndAttribution"}}}},"baseType":"com.linkedin.voyager.dash.common.media.StickerLinkSmallTemplateView"},"com.linkedin.deco.recipe.anonymous.Anon2083470902":{"fields":{"controlName":{"type":"string"},"numFractionalDigits":{"type":"int"},"hintText":{"type":"string"},"numericRangeValidation":{"type":"com.linkedin.deco.recipe.anonymous.Anon600254320"}},"baseType":"com.linkedin.voyager.dash.common.forms.NumberInputFormComponent"},"com.linkedin.voyager.dash.deco.common.forms.FormElement":{"fields":{"urn":{"type":"string"},"prerequisiteInputEvaluationStrategyUnion":{"type":{"union":{"answered":"string","matchAnyPrerequisiteInput":"com.linkedin.deco.recipe.anonymous.Anon870521084","notMatchAnyPrequisiteInput":"com.linkedin.deco.recipe.anonymous.Anon870521084"}}},"formComponent":{"type":{"union":{"gaiTextFormComponent":"com.linkedin.deco.recipe.anonymous.Anon146637213","multilineTextFormComponent":"com.linkedin.voyager.dash.deco.common.forms.MultilineTextFormComponent","textEntityListFormComponent":"com.linkedin.voyager.dash.deco.common.forms.TextEntityListFormComponent","toggleFormComponent":"com.linkedin.voyager.dash.deco.common.forms.ToggleFormComponent","starRatingFormComponent":"com.linkedin.deco.recipe.anonymous.Anon1735278252","nestedCheckboxFormComponent":"com.linkedin.deco.recipe.anonymous.Anon1048186976","mediaUploadFormComponent":"com.linkedin.deco.recipe.anonymous.Anon2105959233","checkboxFormComponent":"com.linkedin.voyager.dash.deco.common.forms.CheckboxFormComponent","dropdownFormComponent":"com.linkedin.voyager.dash.deco.common.forms.DropdownFormComponent","numberInputFormComponent":"com.linkedin.deco.recipe.anonymous.Anon2083470902","reorderableListFormComponent":"com.linkedin.voyager.dash.deco.common.forms.ReorderableListFormComponent","singleTypeaheadEntityFormComponent":"com.linkedin.voyager.dash.deco.common.forms.SingleTypeaheadEntityFormComponent","dateRangeFormComponent":"com.linkedin.voyager.dash.deco.common.forms.DateRangeFormComponent","dateFormComponent":"com.linkedin.voyager.dash.deco.common.forms.DateFormComponent","multiSelectTypeaheadEntityFormComponent":"com.linkedin.voyager.dash.deco.common.forms.MultiSelectTypeaheadEntityFormComponent","locationFormComponent":"com.linkedin.deco.recipe.anonymous.Anon2006189319","singleLineTextFormComponent":"com.linkedin.voyager.dash.deco.common.forms.SingleLineTextFormComponent","radioButtonFormComponent":"com.linkedin.voyager.dash.deco.common.forms.RadioButtonFormComponent","pillFormComponent":"com.linkedin.voyager.dash.deco.common.forms.PillFormComponent"}}},"input":{"type":"com.linkedin.voyager.dash.deco.common.forms.FormElementInput"},"prerequisiteFormElementInputs":{"type":{"array":"com.linkedin.voyager.dash.deco.common.forms.FormElementInput"}},"requiredFieldMissingErrorText":{"type":"com.linkedin.voyager.dash.deco.common.text.TextViewModelV2"},"entityUrn":{"type":"string"},"weight":{"type":"float"},"helperLink":{"type":"com.linkedin.voyager.dash.deco.common.text.TextViewModelV2"},"title":{"type":"com.linkedin.voyager.dash.deco.common.text.TextViewModelV2"},"required":{"type":"boolean"},"shouldAlwaysSendBackFormElementInput":{"type":"boolean"}},"baseType":"com.linkedin.voyager.dash.common.forms.FormElement"},"com.linkedin.deco.recipe.anonymous.Anon1861880920":{"fields":{"paging":{"type":"com.linkedin.voyager.dash.deco.common.FullPaging"},"elements":{"type":{"array":"com.linkedin.deco.recipe.anonymous.Anon1085906135"}}},"baseType":"com.linkedin.restli.common.CollectionResponse"},"com.linkedin.voyager.dash.deco.common.forms.ToggleFormComponent":{"fields":{"icon":{"type":"string"},"controlName":{"type":"string"},"positiveToggleLabel":{"type":"com.linkedin.voyager.dash.deco.common.text.TextViewModelV2"},"negativeToggleLabel":{"type":"com.linkedin.voyager.dash.deco.common.text.TextViewModelV2"}},"baseType":"com.linkedin.voyager.dash.common.forms.ToggleFormComponent"},"com.linkedin.deco.recipe.anonymous.Anon1585981083":{"fields":{"title":{"type":"string"},"description":{"type":"string"},"connectionsUsingProductTypeUrn":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.identity.profile.ConnectionsUsingProductMetadata"},"com.linkedin.deco.recipe.anonymous.Anon2134644859":{"fields":{"entityUrn":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.relationships.Connection"},"com.linkedin.deco.recipe.anonymous.Anon1048186976":{"fields":{"nestedTextSelectableOptions":{"type":{"array":"com.linkedin.deco.recipe.anonymous.Anon902172343"}},"selectionCountRangeValidation":{"type":"com.linkedin.voyager.dash.deco.common.forms.SelectionCountRangeValidation"},"exclusiveSelectableOption":{"type":"com.linkedin.voyager.dash.deco.common.forms.TextSelectableOption"},"subtitle":{"type":"com.linkedin.voyager.dash.deco.common.text.TextViewModelV2"}},"baseType":"com.linkedin.voyager.dash.common.forms.NestedCheckboxFormComponent"}}}},"included":[]}
</code>
<code style="display: none" id="datalet-bpr-guid-15912210">
  {"request":"/voyager/api/voyagerLaunchpadDashLaunchpadViews?decorationId\u003Dcom.linkedin.voyager.dash.deco.launchpad.LaunchpadView-76\u0026launchpadContext\u003DTAKEOVER\u0026q\u003Dcontext","status":200,"body":"bpr-guid-15912210","method":"GET","headers":{"x-li-uuid":"AAYLE+vtCEZ6Amp7P3S1tA\u003D\u003D"}}
</code>
<img src="data:image/gif;base64,R0lGODlhAQABAIAAAAAAAP///yH5BAEAAAAALAAAAAABAAEAAAIBRAA7" style="display: none" class="datalet-bpr-guid-15912210"><code style="display: none" id="bpr-guid-15912213">
  {"data":{"data":{"feedDashGlobalNavs":{"logo":{"controlName":"nav_inbug","logo":{"attributes":[{"scalingType":null,"detailData":{"profilePictureWithoutFrame":null,"profilePictureWithRingStatus":null,"companyLogo":null,"icon":"IC_APP_LINKEDIN_BUG_COLOR_24DP","systemImage":null,"nonEntityGroupLogo":null,"vectorImage":null,"nonEntityProfessionalEventLogo":null,"profilePicture":null,"imageUrl":null,"professionalEventLogo":null,"nonEntityCompanyLogo":null,"nonEntitySchoolLogo":null,"groupLogo":null,"schoolLogo":null,"ghostImage":null,"nonEntityProfilePicture":null},"tintColor":null,"$recipeTypes":["com.linkedin.e6506ba5f5cc1a9b5124781a3661a7db"],"tapTargets":[],"displayAspectRatio":null,"$type":"com.linkedin.voyager.dash.common.image.ImageAttribute"}],"editableAccessibilityText":false,"actionTarget":"https://www.linkedin.com/feed/?doFeedRefresh=true&amp;nis=true","accessibilityTextAttributes":[],"totalCount":null,"accessibilityText":"LinkedIn","$recipeTypes":["com.linkedin.f89edd0cdc11632a12f86e550f2f5e46"],"$type":"com.linkedin.voyager.dash.common.image.ImageViewModel"},"$recipeTypes":["com.linkedin.4973670c8e07d9bf8572c4c7f8658e2b"],"hoverText":null,"$type":"com.linkedin.voyager.dash.feed.nav.BrandingLogo"},"$recipeTypes":["com.linkedin.eae2ed6e463973c8e0d55baecd814363"],"spotlight":{"navContent":{"navElement":{"staticIcon":{"attributes":[{"scalingType":null,"detailData":{"profilePictureWithoutFrame":null,"profilePictureWithRingStatus":null,"companyLogo":null,"icon":"IC_NAV_SMALL_LEARNING_24DP","systemImage":null,"nonEntityGroupLogo":null,"vectorImage":null,"nonEntityProfessionalEventLogo":null,"profilePicture":null,"imageUrl":null,"professionalEventLogo":null,"nonEntityCompanyLogo":null,"nonEntitySchoolLogo":null,"groupLogo":null,"schoolLogo":null,"ghostImage":null,"nonEntityProfilePicture":null},"tintColor":null,"$recipeTypes":["com.linkedin.e6506ba5f5cc1a9b5124781a3661a7db"],"tapTargets":[],"displayAspectRatio":null,"$type":"com.linkedin.voyager.dash.common.image.ImageAttribute"}],"editableAccessibilityText":false,"actionTarget":null,"accessibilityTextAttributes":[],"totalCount":null,"accessibilityText":null,"$recipeTypes":["com.linkedin.f89edd0cdc11632a12f86e550f2f5e46"],"$type":"com.linkedin.voyager.dash.common.image.ImageViewModel"},"activeIcon":null,"hasCoachmark":false,"controlName":"nav_learning","showIconBorder":false,"actionTarget":"https://www.linkedin.com/learning/?trk=nav_neptune_learning","text":"Learning","$recipeTypes":["com.linkedin.b2c03744140a6cd6a2e693c32790988c"],"hasPaid":true,"$type":"com.linkedin.voyager.dash.feed.nav.NavElement"},"premiumUpsellSlotUrn":null},"type":"LEARNING","$recipeTypes":["com.linkedin.c7f876e8e9f8a631b2896f62a656ce64"],"$type":"com.linkedin.voyager.dash.feed.nav.NavItem"},"primaryItems":[{"appLauncher":null,"navItem":{"navContent":{"navElement":{"activeIcon":{"attributes":[{"scalingType":null,"detailData":{"profilePictureWithoutFrame":null,"profilePictureWithRingStatus":null,"companyLogo":null,"icon":"IC_NAV_SMALL_HOME_ACTIVE_24DP","systemImage":null,"nonEntityGroupLogo":null,"vectorImage":null,"nonEntityProfessionalEventLogo":null,"profilePicture":null,"imageUrl":null,"professionalEventLogo":null,"nonEntityCompanyLogo":null,"nonEntitySchoolLogo":null,"groupLogo":null,"schoolLogo":null,"ghostImage":null,"nonEntityProfilePicture":null},"tintColor":null,"$recipeTypes":["com.linkedin.e6506ba5f5cc1a9b5124781a3661a7db"],"tapTargets":[],"displayAspectRatio":null,"$type":"com.linkedin.voyager.dash.common.image.ImageAttribute"}],"editableAccessibilityText":false,"actionTarget":null,"accessibilityTextAttributes":[],"totalCount":null,"accessibilityText":null,"$recipeTypes":["com.linkedin.f89edd0cdc11632a12f86e550f2f5e46"],"$type":"com.linkedin.voyager.dash.common.image.ImageViewModel"},"staticIcon":{"attributes":[{"scalingType":null,"detailData":{"profilePictureWithoutFrame":null,"profilePictureWithRingStatus":null,"companyLogo":null,"icon":"IC_NAV_SMALL_HOME_24DP","systemImage":null,"nonEntityGroupLogo":null,"vectorImage":null,"nonEntityProfessionalEventLogo":null,"profilePicture":null,"imageUrl":null,"professionalEventLogo":null,"nonEntityCompanyLogo":null,"nonEntitySchoolLogo":null,"groupLogo":null,"schoolLogo":null,"ghostImage":null,"nonEntityProfilePicture":null},"tintColor":null,"$recipeTypes":["com.linkedin.e6506ba5f5cc1a9b5124781a3661a7db"],"tapTargets":[],"displayAspectRatio":null,"$type":"com.linkedin.voyager.dash.common.image.ImageAttribute"}],"editableAccessibilityText":false,"actionTarget":null,"accessibilityTextAttributes":[],"totalCount":null,"accessibilityText":null,"$recipeTypes":["com.linkedin.f89edd0cdc11632a12f86e550f2f5e46"],"$type":"com.linkedin.voyager.dash.common.image.ImageViewModel"},"hasCoachmark":false,"controlName":"nav_homepage","showIconBorder":false,"actionTarget":"https://www.linkedin.com/feed/?doFeedRefresh=true&amp;nis=true","text":"Home","$recipeTypes":["com.linkedin.b2c03744140a6cd6a2e693c32790988c"],"hasPaid":false,"$type":"com.linkedin.voyager.dash.feed.nav.NavElement"},"premiumUpsellSlotUrn":null},"type":"HOME","$recipeTypes":["com.linkedin.c7f876e8e9f8a631b2896f62a656ce64"],"$type":"com.linkedin.voyager.dash.feed.nav.NavItem"},"meMenu":null},{"appLauncher":null,"navItem":{"navContent":{"navElement":{"activeIcon":{"attributes":[{"scalingType":null,"detailData":{"profilePictureWithoutFrame":null,"profilePictureWithRingStatus":null,"companyLogo":null,"icon":"IC_NAV_SMALL_PEOPLE_ACTIVE_24DP","systemImage":null,"nonEntityGroupLogo":null,"vectorImage":null,"nonEntityProfessionalEventLogo":null,"profilePicture":null,"imageUrl":null,"professionalEventLogo":null,"nonEntityCompanyLogo":null,"nonEntitySchoolLogo":null,"groupLogo":null,"schoolLogo":null,"ghostImage":null,"nonEntityProfilePicture":null},"tintColor":null,"$recipeTypes":["com.linkedin.e6506ba5f5cc1a9b5124781a3661a7db"],"tapTargets":[],"displayAspectRatio":null,"$type":"com.linkedin.voyager.dash.common.image.ImageAttribute"}],"editableAccessibilityText":false,"actionTarget":null,"accessibilityTextAttributes":[],"totalCount":null,"accessibilityText":null,"$recipeTypes":["com.linkedin.f89edd0cdc11632a12f86e550f2f5e46"],"$type":"com.linkedin.voyager.dash.common.image.ImageViewModel"},"staticIcon":{"attributes":[{"scalingType":null,"detailData":{"profilePictureWithoutFrame":null,"profilePictureWithRingStatus":null,"companyLogo":null,"icon":"IC_NAV_SMALL_PEOPLE_24DP","systemImage":null,"nonEntityGroupLogo":null,"vectorImage":null,"nonEntityProfessionalEventLogo":null,"profilePicture":null,"imageUrl":null,"professionalEventLogo":null,"nonEntityCompanyLogo":null,"nonEntitySchoolLogo":null,"groupLogo":null,"schoolLogo":null,"ghostImage":null,"nonEntityProfilePicture":null},"tintColor":null,"$recipeTypes":["com.linkedin.e6506ba5f5cc1a9b5124781a3661a7db"],"tapTargets":[],"displayAspectRatio":null,"$type":"com.linkedin.voyager.dash.common.image.ImageAttribute"}],"editableAccessibilityText":false,"actionTarget":null,"accessibilityTextAttributes":[],"totalCount":null,"accessibilityText":null,"$recipeTypes":["com.linkedin.f89edd0cdc11632a12f86e550f2f5e46"],"$type":"com.linkedin.voyager.dash.common.image.ImageViewModel"},"hasCoachmark":false,"controlName":"nav_mynetwork","showIconBorder":false,"actionTarget":"https://www.linkedin.com/mynetwork/","text":"My Network","$recipeTypes":["com.linkedin.b2c03744140a6cd6a2e693c32790988c"],"hasPaid":false,"$type":"com.linkedin.voyager.dash.feed.nav.NavElement"},"premiumUpsellSlotUrn":null},"type":"MY_NETWORK","$recipeTypes":["com.linkedin.c7f876e8e9f8a631b2896f62a656ce64"],"$type":"com.linkedin.voyager.dash.feed.nav.NavItem"},"meMenu":null},{"appLauncher":null,"navItem":{"navContent":{"navElement":{"activeIcon":{"attributes":[{"scalingType":null,"detailData":{"profilePictureWithoutFrame":null,"profilePictureWithRingStatus":null,"companyLogo":null,"icon":"IC_NAV_SMALL_JOBS_ACTIVE_24DP","systemImage":null,"nonEntityGroupLogo":null,"vectorImage":null,"nonEntityProfessionalEventLogo":null,"profilePicture":null,"imageUrl":null,"professionalEventLogo":null,"nonEntityCompanyLogo":null,"nonEntitySchoolLogo":null,"groupLogo":null,"schoolLogo":null,"ghostImage":null,"nonEntityProfilePicture":null},"tintColor":null,"$recipeTypes":["com.linkedin.e6506ba5f5cc1a9b5124781a3661a7db"],"tapTargets":[],"displayAspectRatio":null,"$type":"com.linkedin.voyager.dash.common.image.ImageAttribute"}],"editableAccessibilityText":false,"actionTarget":null,"accessibilityTextAttributes":[],"totalCount":null,"accessibilityText":null,"$recipeTypes":["com.linkedin.f89edd0cdc11632a12f86e550f2f5e46"],"$type":"com.linkedin.voyager.dash.common.image.ImageViewModel"},"staticIcon":{"attributes":[{"scalingType":null,"detailData":{"profilePictureWithoutFrame":null,"profilePictureWithRingStatus":null,"companyLogo":null,"icon":"IC_NAV_SMALL_JOBS_24DP","systemImage":null,"nonEntityGroupLogo":null,"vectorImage":null,"nonEntityProfessionalEventLogo":null,"profilePicture":null,"imageUrl":null,"professionalEventLogo":null,"nonEntityCompanyLogo":null,"nonEntitySchoolLogo":null,"groupLogo":null,"schoolLogo":null,"ghostImage":null,"nonEntityProfilePicture":null},"tintColor":null,"$recipeTypes":["com.linkedin.e6506ba5f5cc1a9b5124781a3661a7db"],"tapTargets":[],"displayAspectRatio":null,"$type":"com.linkedin.voyager.dash.common.image.ImageAttribute"}],"editableAccessibilityText":false,"actionTarget":null,"accessibilityTextAttributes":[],"totalCount":null,"accessibilityText":null,"$recipeTypes":["com.linkedin.f89edd0cdc11632a12f86e550f2f5e46"],"$type":"com.linkedin.voyager.dash.common.image.ImageViewModel"},"hasCoachmark":false,"controlName":"nav_jobs","showIconBorder":false,"actionTarget":"https://www.linkedin.com/jobs/","text":"Jobs","$recipeTypes":["com.linkedin.b2c03744140a6cd6a2e693c32790988c"],"hasPaid":false,"$type":"com.linkedin.voyager.dash.feed.nav.NavElement"},"premiumUpsellSlotUrn":null},"type":"JOBS","$recipeTypes":["com.linkedin.c7f876e8e9f8a631b2896f62a656ce64"],"$type":"com.linkedin.voyager.dash.feed.nav.NavItem"},"meMenu":null},{"appLauncher":null,"navItem":{"navContent":{"navElement":{"activeIcon":{"attributes":[{"scalingType":null,"detailData":{"profilePictureWithoutFrame":null,"profilePictureWithRingStatus":null,"companyLogo":null,"icon":"IC_NAV_SMALL_MESSAGING_ACTIVE_24DP","systemImage":null,"nonEntityGroupLogo":null,"vectorImage":null,"nonEntityProfessionalEventLogo":null,"profilePicture":null,"imageUrl":null,"professionalEventLogo":null,"nonEntityCompanyLogo":null,"nonEntitySchoolLogo":null,"groupLogo":null,"schoolLogo":null,"ghostImage":null,"nonEntityProfilePicture":null},"tintColor":null,"$recipeTypes":["com.linkedin.e6506ba5f5cc1a9b5124781a3661a7db"],"tapTargets":[],"displayAspectRatio":null,"$type":"com.linkedin.voyager.dash.common.image.ImageAttribute"}],"editableAccessibilityText":false,"actionTarget":null,"accessibilityTextAttributes":[],"totalCount":null,"accessibilityText":null,"$recipeTypes":["com.linkedin.f89edd0cdc11632a12f86e550f2f5e46"],"$type":"com.linkedin.voyager.dash.common.image.ImageViewModel"},"staticIcon":{"attributes":[{"scalingType":null,"detailData":{"profilePictureWithoutFrame":null,"profilePictureWithRingStatus":null,"companyLogo":null,"icon":"IC_NAV_SMALL_MESSAGING_24DP","systemImage":null,"nonEntityGroupLogo":null,"vectorImage":null,"nonEntityProfessionalEventLogo":null,"profilePicture":null,"imageUrl":null,"professionalEventLogo":null,"nonEntityCompanyLogo":null,"nonEntitySchoolLogo":null,"groupLogo":null,"schoolLogo":null,"ghostImage":null,"nonEntityProfilePicture":null},"tintColor":null,"$recipeTypes":["com.linkedin.e6506ba5f5cc1a9b5124781a3661a7db"],"tapTargets":[],"displayAspectRatio":null,"$type":"com.linkedin.voyager.dash.common.image.ImageAttribute"}],"editableAccessibilityText":false,"actionTarget":null,"accessibilityTextAttributes":[],"totalCount":null,"accessibilityText":null,"$recipeTypes":["com.linkedin.f89edd0cdc11632a12f86e550f2f5e46"],"$type":"com.linkedin.voyager.dash.common.image.ImageViewModel"},"hasCoachmark":false,"controlName":"nav_messaging","showIconBorder":false,"actionTarget":"https://www.linkedin.com/messaging/","text":"Messaging","$recipeTypes":["com.linkedin.b2c03744140a6cd6a2e693c32790988c"],"hasPaid":false,"$type":"com.linkedin.voyager.dash.feed.nav.NavElement"},"premiumUpsellSlotUrn":null},"type":"MESSAGING","$recipeTypes":["com.linkedin.c7f876e8e9f8a631b2896f62a656ce64"],"$type":"com.linkedin.voyager.dash.feed.nav.NavItem"},"meMenu":null},{"appLauncher":null,"navItem":{"navContent":{"navElement":{"activeIcon":{"attributes":[{"scalingType":null,"detailData":{"profilePictureWithoutFrame":null,"profilePictureWithRingStatus":null,"companyLogo":null,"icon":"IC_NAV_SMALL_NOTIFICATIONS_ACTIVE_24DP","systemImage":null,"nonEntityGroupLogo":null,"vectorImage":null,"nonEntityProfessionalEventLogo":null,"profilePicture":null,"imageUrl":null,"professionalEventLogo":null,"nonEntityCompanyLogo":null,"nonEntitySchoolLogo":null,"groupLogo":null,"schoolLogo":null,"ghostImage":null,"nonEntityProfilePicture":null},"tintColor":null,"$recipeTypes":["com.linkedin.e6506ba5f5cc1a9b5124781a3661a7db"],"tapTargets":[],"displayAspectRatio":null,"$type":"com.linkedin.voyager.dash.common.image.ImageAttribute"}],"editableAccessibilityText":false,"actionTarget":null,"accessibilityTextAttributes":[],"totalCount":null,"accessibilityText":null,"$recipeTypes":["com.linkedin.f89edd0cdc11632a12f86e550f2f5e46"],"$type":"com.linkedin.voyager.dash.common.image.ImageViewModel"},"staticIcon":{"attributes":[{"scalingType":null,"detailData":{"profilePictureWithoutFrame":null,"profilePictureWithRingStatus":null,"companyLogo":null,"icon":"IC_NAV_SMALL_NOTIFICATIONS_24DP","systemImage":null,"nonEntityGroupLogo":null,"vectorImage":null,"nonEntityProfessionalEventLogo":null,"profilePicture":null,"imageUrl":null,"professionalEventLogo":null,"nonEntityCompanyLogo":null,"nonEntitySchoolLogo":null,"groupLogo":null,"schoolLogo":null,"ghostImage":null,"nonEntityProfilePicture":null},"tintColor":null,"$recipeTypes":["com.linkedin.e6506ba5f5cc1a9b5124781a3661a7db"],"tapTargets":[],"displayAspectRatio":null,"$type":"com.linkedin.voyager.dash.common.image.ImageAttribute"}],"editableAccessibilityText":false,"actionTarget":null,"accessibilityTextAttributes":[],"totalCount":null,"accessibilityText":null,"$recipeTypes":["com.linkedin.f89edd0cdc11632a12f86e550f2f5e46"],"$type":"com.linkedin.voyager.dash.common.image.ImageViewModel"},"hasCoachmark":false,"controlName":"nav_notifications","showIconBorder":false,"actionTarget":"https://www.linkedin.com/notifications/","text":"Notifications","$recipeTypes":["com.linkedin.b2c03744140a6cd6a2e693c32790988c"],"hasPaid":false,"$type":"com.linkedin.voyager.dash.feed.nav.NavElement"},"premiumUpsellSlotUrn":null},"type":"NOTIFICATIONS","$recipeTypes":["com.linkedin.c7f876e8e9f8a631b2896f62a656ce64"],"$type":"com.linkedin.voyager.dash.feed.nav.NavItem"},"meMenu":null},{"appLauncher":null,"meMenu":{"viewProfileText":"View Profile","*profile":"urn:li:fsd_profile:ACoAAC6BPkEBwSD4NTQ7khSJbuAxrRMYhVSwmBo","meGroups":[{"meItems":[{"meItem":{"controlName":"nav_settings_account_manage_account","actionTarget":"https://www.linkedin.com/mypreferences/d/","text":{"textDirection":"USER_LOCALE","text":"Settings &amp; Privacy","attributesV2":[],"accessibilityTextAttributesV2":[],"accessibilityText":null,"$recipeTypes":["com.linkedin.566376561ef86249fc89f5596eaf6fe1"],"$type":"com.linkedin.voyager.dash.common.text.TextViewModel"},"quickHelp":false,"$recipeTypes":["com.linkedin.fb4511fb6c2df4a805ef662367e15026"],"$type":"com.linkedin.voyager.dash.feed.nav.MeItem"},"premiumUpsellSlotUrn":null},{"meItem":{"controlName":"nav_settings_account_quick_help","actionTarget":"https://www.linkedin.com/help/linkedin/","text":{"textDirection":"USER_LOCALE","text":"Help","attributesV2":[],"accessibilityTextAttributesV2":[],"accessibilityText":null,"$recipeTypes":["com.linkedin.566376561ef86249fc89f5596eaf6fe1"],"$type":"com.linkedin.voyager.dash.common.text.TextViewModel"},"quickHelp":true,"$recipeTypes":["com.linkedin.fb4511fb6c2df4a805ef662367e15026"],"$type":"com.linkedin.voyager.dash.feed.nav.MeItem"},"premiumUpsellSlotUrn":null},{"meItem":{"controlName":"nav_settings_account_language","actionTarget":"https://www.linkedin.com/mypreferences/d/language/","text":{"textDirection":"USER_LOCALE","text":"Language","attributesV2":[],"accessibilityTextAttributesV2":[],"accessibilityText":null,"$recipeTypes":["com.linkedin.566376561ef86249fc89f5596eaf6fe1"],"$type":"com.linkedin.voyager.dash.common.text.TextViewModel"},"quickHelp":false,"$recipeTypes":["com.linkedin.fb4511fb6c2df4a805ef662367e15026"],"$type":"com.linkedin.voyager.dash.feed.nav.MeItem"},"premiumUpsellSlotUrn":null}],"title":"Account","$recipeTypes":["com.linkedin.bb35e914e880d9f9ad79d52bf644c44c"],"$type":"com.linkedin.voyager.dash.feed.nav.MeGroup"},{"meItems":[{"meItem":{"controlName":"recent_activity_nav_all","actionTarget":"https://www.linkedin.com/in/naimiskandar22/recent-activity/","text":{"textDirection":"USER_LOCALE","text":"Posts &amp; Activity","attributesV2":[],"accessibilityTextAttributesV2":[],"accessibilityText":null,"$recipeTypes":["com.linkedin.566376561ef86249fc89f5596eaf6fe1"],"$type":"com.linkedin.voyager.dash.common.text.TextViewModel"},"quickHelp":false,"$recipeTypes":["com.linkedin.fb4511fb6c2df4a805ef662367e15026"],"$type":"com.linkedin.voyager.dash.feed.nav.MeItem"},"premiumUpsellSlotUrn":null},{"meItem":{"controlName":"nav_settings_manage_job_posting_account","actionTarget":"https://www.linkedin.com/talent/job-management-redirect?trk=nav_app_launcher_manage_job_post_nept","text":{"textDirection":"USER_LOCALE","text":"Job Posting Account","attributesV2":[],"accessibilityTextAttributesV2":[],"accessibilityText":null,"$recipeTypes":["com.linkedin.566376561ef86249fc89f5596eaf6fe1"],"$type":"com.linkedin.voyager.dash.common.text.TextViewModel"},"quickHelp":false,"$recipeTypes":["com.linkedin.fb4511fb6c2df4a805ef662367e15026"],"$type":"com.linkedin.voyager.dash.feed.nav.MeItem"},"premiumUpsellSlotUrn":null}],"title":"Manage","$recipeTypes":["com.linkedin.bb35e914e880d9f9ad79d52bf644c44c"],"$type":"com.linkedin.voyager.dash.feed.nav.MeGroup"}],"controlName":"nav_settings","text":"Me","$recipeTypes":["com.linkedin.73113dd58d976ff22a5e4aa2ca35d21d"],"$type":"com.linkedin.voyager.dash.feed.nav.MeMenu"},"navItem":null},{"appLauncher":{"showDivider":true,"productItems":[{"navContent":{"navElement":{"staticIcon":{"attributes":[{"scalingType":null,"detailData":{"profilePictureWithoutFrame":null,"profilePictureWithRingStatus":null,"companyLogo":null,"icon":"IC_APP_TALENT_INSIGHTS_24DP","systemImage":null,"nonEntityGroupLogo":null,"vectorImage":null,"nonEntityProfessionalEventLogo":null,"profilePicture":null,"imageUrl":null,"professionalEventLogo":null,"nonEntityCompanyLogo":null,"nonEntitySchoolLogo":null,"groupLogo":null,"schoolLogo":null,"ghostImage":null,"nonEntityProfilePicture":null},"tintColor":null,"$recipeTypes":["com.linkedin.e6506ba5f5cc1a9b5124781a3661a7db"],"tapTargets":[],"displayAspectRatio":null,"$type":"com.linkedin.voyager.dash.common.image.ImageAttribute"}],"editableAccessibilityText":false,"actionTarget":null,"accessibilityTextAttributes":[],"totalCount":null,"accessibilityText":null,"$recipeTypes":["com.linkedin.f89edd0cdc11632a12f86e550f2f5e46"],"$type":"com.linkedin.voyager.dash.common.image.ImageViewModel"},"activeIcon":null,"hasCoachmark":false,"controlName":"nav_launcher_insights","showIconBorder":true,"actionTarget":"https://www.linkedin.com/insights?trk=nav_app_launcher_insights_nept&amp;src=li-nav","text":"Talent Insights","$recipeTypes":["com.linkedin.b2c03744140a6cd6a2e693c32790988c"],"hasPaid":false,"$type":"com.linkedin.voyager.dash.feed.nav.NavElement"},"premiumUpsellSlotUrn":null},"type":"INSIGHTS","$recipeTypes":["com.linkedin.c7f876e8e9f8a631b2896f62a656ce64"],"$type":"com.linkedin.voyager.dash.feed.nav.NavItem"},{"navContent":{"navElement":{"staticIcon":{"attributes":[{"scalingType":null,"detailData":{"profilePictureWithoutFrame":null,"profilePictureWithRingStatus":null,"companyLogo":null,"icon":"IC_APP_JOBS_POSTING_24DP","systemImage":null,"nonEntityGroupLogo":null,"vectorImage":null,"nonEntityProfessionalEventLogo":null,"profilePicture":null,"imageUrl":null,"professionalEventLogo":null,"nonEntityCompanyLogo":null,"nonEntitySchoolLogo":null,"groupLogo":null,"schoolLogo":null,"ghostImage":null,"nonEntityProfilePicture":null},"tintColor":null,"$recipeTypes":["com.linkedin.e6506ba5f5cc1a9b5124781a3661a7db"],"tapTargets":[],"displayAspectRatio":null,"$type":"com.linkedin.voyager.dash.common.image.ImageAttribute"}],"editableAccessibilityText":false,"actionTarget":null,"accessibilityTextAttributes":[],"totalCount":null,"accessibilityText":null,"$recipeTypes":["com.linkedin.f89edd0cdc11632a12f86e550f2f5e46"],"$type":"com.linkedin.voyager.dash.common.image.ImageViewModel"},"activeIcon":null,"hasCoachmark":false,"controlName":"nav_launcher_job_postings","showIconBorder":true,"actionTarget":"https://www.linkedin.com/talent/job-posting-redirect?trk=nav_app_launcher_job_post_nept","text":"Post a job","$recipeTypes":["com.linkedin.b2c03744140a6cd6a2e693c32790988c"],"hasPaid":true,"$type":"com.linkedin.voyager.dash.feed.nav.NavElement"},"premiumUpsellSlotUrn":null},"type":"JOB_POSTINGS","$recipeTypes":["com.linkedin.c7f876e8e9f8a631b2896f62a656ce64"],"$type":"com.linkedin.voyager.dash.feed.nav.NavItem"},{"navContent":{"navElement":{"staticIcon":{"attributes":[{"scalingType":null,"detailData":{"profilePictureWithoutFrame":null,"profilePictureWithRingStatus":null,"companyLogo":null,"icon":"IC_APP_ADS_24DP","systemImage":null,"nonEntityGroupLogo":null,"vectorImage":null,"nonEntityProfessionalEventLogo":null,"profilePicture":null,"imageUrl":null,"professionalEventLogo":null,"nonEntityCompanyLogo":null,"nonEntitySchoolLogo":null,"groupLogo":null,"schoolLogo":null,"ghostImage":null,"nonEntityProfilePicture":null},"tintColor":null,"$recipeTypes":["com.linkedin.e6506ba5f5cc1a9b5124781a3661a7db"],"tapTargets":[],"displayAspectRatio":null,"$type":"com.linkedin.voyager.dash.common.image.ImageAttribute"}],"editableAccessibilityText":false,"actionTarget":null,"accessibilityTextAttributes":[],"totalCount":null,"accessibilityText":null,"$recipeTypes":["com.linkedin.f89edd0cdc11632a12f86e550f2f5e46"],"$type":"com.linkedin.voyager.dash.common.image.ImageViewModel"},"activeIcon":null,"hasCoachmark":false,"controlName":"nav_launcher_ads","showIconBorder":true,"actionTarget":"https://www.linkedin.com/campaignmanager/accounts","text":"Advertise","$recipeTypes":["com.linkedin.b2c03744140a6cd6a2e693c32790988c"],"hasPaid":true,"$type":"com.linkedin.voyager.dash.feed.nav.NavElement"},"premiumUpsellSlotUrn":null},"type":"ADVERTISE","$recipeTypes":["com.linkedin.c7f876e8e9f8a631b2896f62a656ce64"],"$type":"com.linkedin.voyager.dash.feed.nav.NavItem"},{"navContent":{"navElement":{"staticIcon":{"attributes":[{"scalingType":null,"detailData":{"profilePictureWithoutFrame":null,"profilePictureWithRingStatus":null,"companyLogo":null,"icon":"IC_APP_SALES_NAVIGATOR_24DP","systemImage":null,"nonEntityGroupLogo":null,"vectorImage":null,"nonEntityProfessionalEventLogo":null,"profilePicture":null,"imageUrl":null,"professionalEventLogo":null,"nonEntityCompanyLogo":null,"nonEntitySchoolLogo":null,"groupLogo":null,"schoolLogo":null,"ghostImage":null,"nonEntityProfilePicture":null},"tintColor":null,"$recipeTypes":["com.linkedin.e6506ba5f5cc1a9b5124781a3661a7db"],"tapTargets":[],"displayAspectRatio":null,"$type":"com.linkedin.voyager.dash.common.image.ImageAttribute"}],"editableAccessibilityText":false,"actionTarget":null,"accessibilityTextAttributes":[],"totalCount":null,"accessibilityText":null,"$recipeTypes":["com.linkedin.f89edd0cdc11632a12f86e550f2f5e46"],"$type":"com.linkedin.voyager.dash.common.image.ImageViewModel"},"activeIcon":null,"hasCoachmark":false,"controlName":"nav_launcher_find_leads","showIconBorder":true,"actionTarget":"https://www.linkedin.com/premium/products/?intentType=FIND_LEADS&amp;upsellOrderOrigin=premium_nav_more_products_panel&amp;utype=sales","text":"Sell","$recipeTypes":["com.linkedin.b2c03744140a6cd6a2e693c32790988c"],"hasPaid":false,"$type":"com.linkedin.voyager.dash.feed.nav.NavElement"},"premiumUpsellSlotUrn":null},"type":"FIND_LEADS","$recipeTypes":["com.linkedin.c7f876e8e9f8a631b2896f62a656ce64"],"$type":"com.linkedin.voyager.dash.feed.nav.NavItem"},{"navContent":{"navElement":{"staticIcon":{"attributes":[{"scalingType":null,"detailData":{"profilePictureWithoutFrame":null,"profilePictureWithRingStatus":null,"companyLogo":null,"icon":"IC_APP_GROUPS_24DP","systemImage":null,"nonEntityGroupLogo":null,"vectorImage":null,"nonEntityProfessionalEventLogo":null,"profilePicture":null,"imageUrl":null,"professionalEventLogo":null,"nonEntityCompanyLogo":null,"nonEntitySchoolLogo":null,"groupLogo":null,"schoolLogo":null,"ghostImage":null,"nonEntityProfilePicture":null},"tintColor":null,"$recipeTypes":["com.linkedin.e6506ba5f5cc1a9b5124781a3661a7db"],"tapTargets":[],"displayAspectRatio":null,"$type":"com.linkedin.voyager.dash.common.image.ImageAttribute"}],"editableAccessibilityText":false,"actionTarget":null,"accessibilityTextAttributes":[],"totalCount":null,"accessibilityText":null,"$recipeTypes":["com.linkedin.f89edd0cdc11632a12f86e550f2f5e46"],"$type":"com.linkedin.voyager.dash.common.image.ImageViewModel"},"activeIcon":null,"hasCoachmark":false,"controlName":"nav_launcher_groups","showIconBorder":true,"actionTarget":"https://www.linkedin.com/groups","text":"Groups","$recipeTypes":["com.linkedin.b2c03744140a6cd6a2e693c32790988c"],"hasPaid":false,"$type":"com.linkedin.voyager.dash.feed.nav.NavElement"},"premiumUpsellSlotUrn":null},"type":"GROUPS","$recipeTypes":["com.linkedin.c7f876e8e9f8a631b2896f62a656ce64"],"$type":"com.linkedin.voyager.dash.feed.nav.NavItem"},{"navContent":{"navElement":{"staticIcon":{"attributes":[{"scalingType":null,"detailData":{"profilePictureWithoutFrame":null,"profilePictureWithRingStatus":null,"companyLogo":null,"icon":"IC_APP_PROFINDER_24DP","systemImage":null,"nonEntityGroupLogo":null,"vectorImage":null,"nonEntityProfessionalEventLogo":null,"profilePicture":null,"imageUrl":null,"professionalEventLogo":null,"nonEntityCompanyLogo":null,"nonEntitySchoolLogo":null,"groupLogo":null,"schoolLogo":null,"ghostImage":null,"nonEntityProfilePicture":null},"tintColor":null,"$recipeTypes":["com.linkedin.e6506ba5f5cc1a9b5124781a3661a7db"],"tapTargets":[],"displayAspectRatio":null,"$type":"com.linkedin.voyager.dash.common.image.ImageAttribute"}],"editableAccessibilityText":false,"actionTarget":null,"accessibilityTextAttributes":[],"totalCount":null,"accessibilityText":null,"$recipeTypes":["com.linkedin.f89edd0cdc11632a12f86e550f2f5e46"],"$type":"com.linkedin.voyager.dash.common.image.ImageViewModel"},"activeIcon":null,"hasCoachmark":false,"controlName":"nav_launcher_services_marketplace","showIconBorder":true,"actionTarget":"https://www.linkedin.com/services?trk=d_flagship3_nav","text":"Services Marketplace","$recipeTypes":["com.linkedin.b2c03744140a6cd6a2e693c32790988c"],"hasPaid":false,"$type":"com.linkedin.voyager.dash.feed.nav.NavElement"},"premiumUpsellSlotUrn":null},"type":"PROFINDER","$recipeTypes":["com.linkedin.c7f876e8e9f8a631b2896f62a656ce64"],"$type":"com.linkedin.voyager.dash.feed.nav.NavItem"}],"businessServices":[{"businessServiceAction":{"controlName":"nav_business_talent_solutions","actionTarget":"https://business.linkedin.com/talent-solutions?trk=flagship_nav&amp;veh=li-header-dropdown-lts-control&amp;src=li-nav","title":{"textDirection":"USER_LOCALE","text":"Hire on LinkedIn","attributesV2":[],"accessibilityTextAttributesV2":[],"accessibilityText":null,"$recipeTypes":["com.linkedin.566376561ef86249fc89f5596eaf6fe1"],"$type":"com.linkedin.voyager.dash.common.text.TextViewModel"},"$recipeTypes":["com.linkedin.188858304ae14097e59518d245c7cc85"],"$type":"com.linkedin.voyager.dash.feed.nav.BusinessServiceAction"},"description":"Find, attract and recruit talent","$recipeTypes":["com.linkedin.a94ff0919cddaa7056025e9491dc3794"],"$type":"com.linkedin.voyager.dash.feed.nav.BusinessService"},{"businessServiceAction":{"controlName":"nav_business_sales_solutions","actionTarget":"https://business.linkedin.com/sales-solutions?trk=flagship_nav&amp;veh=li-header-dropdown-lss-control&amp;src=li-nav","title":{"textDirection":"USER_LOCALE","text":"Sell with LinkedIn","attributesV2":[],"accessibilityTextAttributesV2":[],"accessibilityText":null,"$recipeTypes":["com.linkedin.566376561ef86249fc89f5596eaf6fe1"],"$type":"com.linkedin.voyager.dash.common.text.TextViewModel"},"$recipeTypes":["com.linkedin.188858304ae14097e59518d245c7cc85"],"$type":"com.linkedin.voyager.dash.feed.nav.BusinessServiceAction"},"description":"Build relationships with buyers","$recipeTypes":["com.linkedin.a94ff0919cddaa7056025e9491dc3794"],"$type":"com.linkedin.voyager.dash.feed.nav.BusinessService"},{"businessServiceAction":{"controlName":"nav_business_post_a_job","actionTarget":"https://www.linkedin.com/talent/job-posting-redirect?trk=nav_biz_serv_job_post_nept","title":{"textDirection":"USER_LOCALE","text":"Post a job for free","attributesV2":[],"accessibilityTextAttributesV2":[],"accessibilityText":null,"$recipeTypes":["com.linkedin.566376561ef86249fc89f5596eaf6fe1"],"$type":"com.linkedin.voyager.dash.common.text.TextViewModel"},"$recipeTypes":["com.linkedin.188858304ae14097e59518d245c7cc85"],"$type":"com.linkedin.voyager.dash.feed.nav.BusinessServiceAction"},"description":"Find quality candidates","$recipeTypes":["com.linkedin.a94ff0919cddaa7056025e9491dc3794"],"$type":"com.linkedin.voyager.dash.feed.nav.BusinessService"},{"businessServiceAction":{"controlName":"nav_business_advertise","actionTarget":"https://business.linkedin.com/marketing-solutions/ads?trk=n_nav_ads_rr_b&amp;src=li-nav","title":{"textDirection":"USER_LOCALE","text":"Advertise on LinkedIn","attributesV2":[],"accessibilityTextAttributesV2":[],"accessibilityText":null,"$recipeTypes":["com.linkedin.566376561ef86249fc89f5596eaf6fe1"],"$type":"com.linkedin.voyager.dash.common.text.TextViewModel"},"$recipeTypes":["com.linkedin.188858304ae14097e59518d245c7cc85"],"$type":"com.linkedin.voyager.dash.feed.nav.BusinessServiceAction"},"description":"Acquire customers and grow your business","$recipeTypes":["com.linkedin.a94ff0919cddaa7056025e9491dc3794"],"$type":"com.linkedin.voyager.dash.feed.nav.BusinessService"},{"businessServiceAction":{"controlName":"nav_business_learning_solutions","actionTarget":"https://learning.linkedin.com/?trk=d_flagship3_nav&amp;veh=learning_solutions&amp;src=li-nav","title":{"textDirection":"USER_LOCALE","text":"Learn with LinkedIn","attributesV2":[],"accessibilityTextAttributesV2":[],"accessibilityText":null,"$recipeTypes":["com.linkedin.566376561ef86249fc89f5596eaf6fe1"],"$type":"com.linkedin.voyager.dash.common.text.TextViewModel"},"$recipeTypes":["com.linkedin.188858304ae14097e59518d245c7cc85"],"$type":"com.linkedin.voyager.dash.feed.nav.BusinessServiceAction"},"description":"Courses to develop your employees","$recipeTypes":["com.linkedin.a94ff0919cddaa7056025e9491dc3794"],"$type":"com.linkedin.voyager.dash.feed.nav.BusinessService"},{"businessServiceAction":{"controlName":"nav_business_admin_center","actionTarget":"https://business.linkedin.com/talent-solutions/customer/admin-center","title":{"textDirection":"USER_LOCALE","text":"Admin Center","attributesV2":[],"accessibilityTextAttributesV2":[],"accessibilityText":null,"$recipeTypes":["com.linkedin.566376561ef86249fc89f5596eaf6fe1"],"$type":"com.linkedin.voyager.dash.common.text.TextViewModel"},"$recipeTypes":["com.linkedin.188858304ae14097e59518d245c7cc85"],"$type":"com.linkedin.voyager.dash.feed.nav.BusinessServiceAction"},"description":"Manage billing and account details","$recipeTypes":["com.linkedin.a94ff0919cddaa7056025e9491dc3794"],"$type":"com.linkedin.voyager.dash.feed.nav.BusinessService"}],"productItemsTitle":{"textDirection":"USER_LOCALE","text":"Visit More LinkedIn Products","attributesV2":[],"accessibilityTextAttributesV2":[],"accessibilityText":null,"$recipeTypes":["com.linkedin.566376561ef86249fc89f5596eaf6fe1"],"$type":"com.linkedin.voyager.dash.common.text.TextViewModel"},"businessServicesTitle":"Explore more for business","businessServiceActions":[{"controlName":"nav_business_create_company","actionTarget":"https://www.linkedin.com/company/setup/new/","title":{"textDirection":"USER_LOCALE","text":"Create a Company Page  ","attributesV2":[{"start":22,"length":1,"detailData":{"jobPostingName":null,"hyperlink":null,"profileFamiliarName":null,"color":null,"companyName":null,"icon":"IC_PLUS_16DP","epoch":null,"systemImage":null,"textLink":null,"listItemStyle":null,"groupName":null,"hyperlinkOpenExternally":null,"listStyle":null,"profileFullName":null,"stringFieldReference":null,"learningCourseName":null,"profileMention":null,"style":null,"schoolName":null,"hashtag":null},"$recipeTypes":["com.linkedin.9d1c9f8db73cc268b525043481708082"],"$type":"com.linkedin.voyager.dash.common.text.TextAttribute"}],"accessibilityTextAttributesV2":[],"accessibilityText":null,"$recipeTypes":["com.linkedin.566376561ef86249fc89f5596eaf6fe1"],"$type":"com.linkedin.voyager.dash.common.text.TextViewModel"},"$recipeTypes":["com.linkedin.188858304ae14097e59518d245c7cc85"],"$type":"com.linkedin.voyager.dash.feed.nav.BusinessServiceAction"}],"controlName":"nav_launcher","icon":{"attributes":[{"scalingType":null,"detailData":{"profilePictureWithoutFrame":null,"profilePictureWithRingStatus":null,"companyLogo":null,"icon":"IC_NAV_SMALL_APP_SWITCHER_24DP","systemImage":null,"nonEntityGroupLogo":null,"vectorImage":null,"nonEntityProfessionalEventLogo":null,"profilePicture":null,"imageUrl":null,"professionalEventLogo":null,"nonEntityCompanyLogo":null,"nonEntitySchoolLogo":null,"groupLogo":null,"schoolLogo":null,"ghostImage":null,"nonEntityProfilePicture":null},"tintColor":null,"$recipeTypes":["com.linkedin.e6506ba5f5cc1a9b5124781a3661a7db"],"tapTargets":[],"displayAspectRatio":null,"$type":"com.linkedin.voyager.dash.common.image.ImageAttribute"}],"editableAccessibilityText":false,"actionTarget":null,"accessibilityTextAttributes":[],"totalCount":null,"accessibilityText":null,"$recipeTypes":["com.linkedin.f89edd0cdc11632a12f86e550f2f5e46"],"$type":"com.linkedin.voyager.dash.common.image.ImageViewModel"},"text":"For Business","$recipeTypes":["com.linkedin.f94aca2770a997f8a58dfb7d6154954a"],"$type":"com.linkedin.voyager.dash.feed.nav.AppLauncher"},"meMenu":null,"navItem":null}],"$type":"com.linkedin.voyager.dash.feed.nav.GlobalNav"},"$recipeTypes":["com.linkedin.1f89c60d56117f36dc02eb3d95a6504d"],"$type":"com.linkedin.1f89c60d56117f36dc02eb3d95a6504d"},"extensions":{"webMetadata":{}}},"meta":{"microSchema":{"isGraphQL":true,"version":"2.1","types":{"com.linkedin.dcb85e1ca353361cb2b5f6fa24a1c9af":{"fields":{"groupUrn":{"type":"string"},"vectorImage":{"type":"com.linkedin.ca116024396bd29dd1345ad0998a57d5"},"group":{"resolvedFrom":"groupUrn","type":"com.linkedin.6c29a299f640b6e802d15141c355bd92"}},"baseType":"com.linkedin.voyager.dash.common.image.NonEntityGroupLogo"},"com.linkedin.1e797f68b521546886989869c5b44a2d":{"fields":{"name":{"type":"string"},"logo":{"type":{"union":{"url":"com.linkedin.common.Url","vectorImage":"com.linkedin.ca116024396bd29dd1345ad0998a57d5"}}},"logoResolutionResult":{"derivedFrom":"logo","type":{"union":{"url":"com.linkedin.common.Url","vectorImage":"com.linkedin.ca116024396bd29dd1345ad0998a57d5"}}},"entityUrn":{"type":"com.linkedin.voyager.dash.common.SchoolUrn"}},"baseType":"com.linkedin.voyager.dash.organization.School"},"com.linkedin.fb4511fb6c2df4a805ef662367e15026":{"fields":{"controlName":{"type":"com.linkedin.voyager.dash.common.tracking.ControlName"},"actionTarget":{"type":"com.linkedin.common.Url"},"text":{"type":"com.linkedin.566376561ef86249fc89f5596eaf6fe1"},"quickHelp":{"type":"boolean"}},"baseType":"com.linkedin.voyager.dash.feed.nav.MeItem"},"com.linkedin.f1332bd87cf4dccab8aa4b4583ad44be":{"fields":{"width":{"type":"int"},"fileIdentifyingUrlPathSegment":{"type":"string"},"expiresAt":{"type":"com.linkedin.common.Time"},"height":{"type":"int"}},"baseType":"com.linkedin.common.VectorArtifact"},"com.linkedin.61dcef34ee68ce7a34919062297c038b":{"fields":{"ringContentType":{"type":"com.linkedin.voyager.dash.common.image.RingContentType"},"actionTarget":{"type":"com.linkedin.common.Url"},"preDashEntityUrn":{"type":"com.linkedin.voyager.dash.common.PreDashUrn"},"entityUrn":{"type":"com.linkedin.voyager.dash.common.RingStatusUrn"},"emphasized":{"type":"boolean"}},"baseType":"com.linkedin.voyager.dash.common.image.RingStatus"},"com.linkedin.6c29a299f640b6e802d15141c355bd92":{"fields":{"name":{"type":"string"},"logo":{"type":{"union":{"url":"com.linkedin.common.Url","vectorImage":"com.linkedin.ca116024396bd29dd1345ad0998a57d5"}}},"logoResolutionResult":{"derivedFrom":"logo","type":{"union":{"url":"com.linkedin.common.Url","vectorImage":"com.linkedin.ca116024396bd29dd1345ad0998a57d5"}}},"entityUrn":{"type":"com.linkedin.voyager.dash.common.GroupUrn"}},"baseType":"com.linkedin.voyager.dash.groups.Group"},"com.linkedin.1f89c60d56117f36dc02eb3d95a6504d":{"fields":{"feedDashGlobalNavs":{"type":"com.linkedin.eae2ed6e463973c8e0d55baecd814363"}},"baseType":"com.linkedin.graphql.Query"},"com.linkedin.bb35e914e880d9f9ad79d52bf644c44c":{"fields":{"title":{"type":"string"},"meItemsUnions":{"type":{"array":{"union":{"meItem":"com.linkedin.fb4511fb6c2df4a805ef662367e15026","premiumUpsellSlotUrn":"string"}}}},"meItems":{"resolvedFrom":"meItemsUnions","type":{"array":{"union":{"meItem":"com.linkedin.fb4511fb6c2df4a805ef662367e15026","premiumUpsellSlotUrn":"com.linkedin.6cf7727c816c79567a0564982ee35ef4"}}}}},"baseType":"com.linkedin.voyager.dash.feed.nav.MeGroup"},"com.linkedin.1112232e110fbaf9f6866335ae420a21":{"fields":{"type":{"type":"com.linkedin.voyager.dash.common.text.ListStyleType"},"index":{"type":"int"}},"baseType":"com.linkedin.voyager.dash.common.text.ListItemStyle"},"com.linkedin.bda725ec0f4ddc7380fb44de84e29723":{"fields":{"displayImageWithFrameReferenceUnion":{"type":{"union":{"url":"com.linkedin.common.Url","vectorImage":"com.linkedin.ca116024396bd29dd1345ad0998a57d5"}}},"displayImageReferenceResolutionResult":{"derivedFrom":"displayImageReference","type":{"union":{"url":"com.linkedin.common.Url","vectorImage":"com.linkedin.ca116024396bd29dd1345ad0998a57d5"}}},"displayImageWithFrameReference":{"derivedFrom":"displayImageWithFrameReferenceUnion","type":{"union":{"url":"com.linkedin.common.Url","vectorImage":"com.linkedin.ca116024396bd29dd1345ad0998a57d5"}}},"displayImageReference":{"type":{"union":{"url":"com.linkedin.common.Url","vectorImage":"com.linkedin.ca116024396bd29dd1345ad0998a57d5"}}},"displayImageUrn":{"type":"com.linkedin.common.Urn"}},"baseType":"com.linkedin.voyager.dash.identity.profile.PhotoFilterPicture"},"com.linkedin.b2c03744140a6cd6a2e693c32790988c":{"fields":{"staticIcon":{"type":"com.linkedin.f89edd0cdc11632a12f86e550f2f5e46"},"activeIcon":{"type":"com.linkedin.f89edd0cdc11632a12f86e550f2f5e46"},"hasCoachmark":{"type":"boolean"},"controlName":{"type":"com.linkedin.voyager.dash.common.tracking.ControlName"},"showIconBorder":{"type":"boolean"},"text":{"type":"string"},"actionTarget":{"type":"com.linkedin.common.Url"},"hasPaid":{"type":"boolean"}},"baseType":"com.linkedin.voyager.dash.feed.nav.NavElement"},"com.linkedin.371c4584dc45b7919c4eb6ed6697e8cf":{"fields":{"professionalEvent":{"resolvedFrom":"professionalEventUrn","type":"com.linkedin.0d2a5c07095246fc3e756f3e922d8aa0"},"vectorImage":{"type":"com.linkedin.ca116024396bd29dd1345ad0998a57d5"},"professionalEventUrn":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.common.image.NonEntityProfessionalEventLogo"},"com.linkedin.1c710707068d7b33bcbee05036904a3b":{"fields":{"ringStatus":{"type":"com.linkedin.61dcef34ee68ce7a34919062297c038b"},"profileUrn":{"type":"string"},"vectorImage":{"type":"com.linkedin.ca116024396bd29dd1345ad0998a57d5"},"profile":{"resolvedFrom":"profileUrn","type":"com.linkedin.c8b3cc209fc06180b17a4ffb72120344"}},"baseType":"com.linkedin.voyager.dash.common.image.NonEntityProfilePicture"},"com.linkedin.8b2657e6d402c000382bd65eb94ba628":{"fields":{"image":{"type":"com.linkedin.f89edd0cdc11632a12f86e550f2f5e46"},"socialProofInsight":{"type":"com.linkedin.ae72a2e8f27d029e18d8e6d4350b0273"},"subtext":{"type":"com.linkedin.566376561ef86249fc89f5596eaf6fe1"},"ctaText":{"type":"com.linkedin.566376561ef86249fc89f5596eaf6fe1"},"upsellOrderOrigin":{"type":"string"},"controlName":{"type":"com.linkedin.voyager.dash.common.tracking.ControlName"},"actionUrl":{"type":"com.linkedin.common.Url"},"legoTrackingToken":{"type":"string"},"title":{"type":"com.linkedin.566376561ef86249fc89f5596eaf6fe1"},"cancelCta":{"type":"com.linkedin.513969f6493543c4ad7f04b46c3b67f6"},"globalLegoTrackingToken":{"type":"string"},"promotionLegoTrackingToken":{"type":"string"},"funnelCommonHeader":{"type":"com.linkedin.f58e91c23e63f52ceb12bd0fab3eb07b"},"fireDiscoveryImpressionEvent":{"type":"boolean"},"subtitle":{"type":"com.linkedin.566376561ef86249fc89f5596eaf6fe1"},"dismissible":{"type":"boolean"},"layoutStyle":{"type":"com.linkedin.voyager.dash.premium.PremiumUpsellLayoutStyle"},"ctaButtonSecondaryEmphasizedTheme":{"type":"boolean"}},"baseType":"com.linkedin.voyager.dash.premium.PremiumUpsellCard"},"com.linkedin.ae72a2e8f27d029e18d8e6d4350b0273":{"fields":{"image":{"type":"com.linkedin.f89edd0cdc11632a12f86e550f2f5e46"},"text":{"type":"com.linkedin.566376561ef86249fc89f5596eaf6fe1"}},"baseType":"com.linkedin.voyager.dash.common.ux.InsightViewModel"},"com.linkedin.d66f0cb7963e0345d3ff893927e60c88":{"fields":{"tapTargetEntityUnion":{"type":{"union":{"companyUrn":"string","profileUrn":"string"}}},"stickerLinkViewUnion":{"type":{"union":{"mediumTemplate":"com.linkedin.637f09655a21b1df5a03c68636fd2d18","largeTemplate":"com.linkedin.1bcf1aba50ed23a46d11fe327f319b18","smallTemplate":"com.linkedin.f223abd9a48757f8395ac497a91593f3"}}},"firstCornerXOffsetPercentage":{"type":"float"},"stickerLinkTemplateSize":{"type":"com.linkedin.voyager.dash.common.media.StickerLinkTemplateSize"},"type":{"type":"com.linkedin.voyager.dash.common.TapTargetAttributeType"},"thirdCornerYOffsetPercentage":{"type":"float"},"url":{"type":"com.linkedin.common.Url"},"stickerLinkView":{"derivedFrom":"stickerLinkViewUnion","type":{"union":{"mediumTemplate":"com.linkedin.637f09655a21b1df5a03c68636fd2d18","largeTemplate":"com.linkedin.1bcf1aba50ed23a46d11fe327f319b18","smallTemplate":"com.linkedin.f223abd9a48757f8395ac497a91593f3"}}},"urn":{"type":"com.linkedin.common.Urn"},"thirdCornerXOffsetPercentage":{"type":"float"},"secondCornerYOffsetPercentage":{"type":"float"},"fourthCornerXOffsetPercentage":{"type":"float"},"tapTargetEntity":{"resolvedFrom":"tapTargetEntityUnion","type":{"union":{"companyUrn":"com.linkedin.2b5530b88c12ba36048beee3aca27737","profileUrn":"com.linkedin.c8b3cc209fc06180b17a4ffb72120344"}}},"firstCornerYOffsetPercentage":{"type":"float"},"rank":{"type":"int"},"untaggable":{"type":"boolean"},"fourthCornerYOffsetPercentage":{"type":"float"},"text":{"type":"string"},"secondCornerXOffsetPercentage":{"type":"float"}},"baseType":"com.linkedin.voyager.dash.common.TapTarget"},"com.linkedin.1bcf1aba50ed23a46d11fe327f319b18":{"fields":{"image":{"derivedFrom":"imageUnion","type":{"union":{"url":"com.linkedin.common.Url","vectorImage":"com.linkedin.ca116024396bd29dd1345ad0998a57d5"}}},"nameSupplementaryInfo":{"type":"com.linkedin.566376561ef86249fc89f5596eaf6fe1"},"footerText":{"type":"com.linkedin.566376561ef86249fc89f5596eaf6fe1"},"imageUnion":{"type":{"union":{"url":"com.linkedin.common.Url","vectorImage":"com.linkedin.ca116024396bd29dd1345ad0998a57d5"}}},"backgroundImage":{"derivedFrom":"backgroundImageUnion","type":{"union":{"url":"com.linkedin.common.Url","vectorImage":"com.linkedin.ca116024396bd29dd1345ad0998a57d5"}}},"subHeadline":{"type":"com.linkedin.566376561ef86249fc89f5596eaf6fe1"},"name":{"type":"com.linkedin.566376561ef86249fc89f5596eaf6fe1"},"insightText":{"type":"com.linkedin.566376561ef86249fc89f5596eaf6fe1"},"headline":{"type":"com.linkedin.566376561ef86249fc89f5596eaf6fe1"},"backgroundImageUnion":{"type":{"union":{"url":"com.linkedin.common.Url","vectorImage":"com.linkedin.ca116024396bd29dd1345ad0998a57d5"}}}},"baseType":"com.linkedin.voyager.dash.common.media.StickerLinkLargeTemplateView"},"com.linkedin.eae2ed6e463973c8e0d55baecd814363":{"fields":{"logo":{"type":"com.linkedin.4973670c8e07d9bf8572c4c7f8658e2b"},"primaryItemsUnions":{"type":{"array":{"union":{"appLauncher":"com.linkedin.f94aca2770a997f8a58dfb7d6154954a","navItem":"com.linkedin.c7f876e8e9f8a631b2896f62a656ce64","meMenu":"com.linkedin.73113dd58d976ff22a5e4aa2ca35d21d"}}}},"spotlight":{"type":"com.linkedin.c7f876e8e9f8a631b2896f62a656ce64"},"primaryItems":{"derivedFrom":"primaryItemsUnions","type":{"array":{"union":{"appLauncher":"com.linkedin.f94aca2770a997f8a58dfb7d6154954a","navItem":"com.linkedin.c7f876e8e9f8a631b2896f62a656ce64","meMenu":"com.linkedin.73113dd58d976ff22a5e4aa2ca35d21d"}}}}},"baseType":"com.linkedin.voyager.dash.feed.nav.GlobalNav"},"com.linkedin.9d1c9f8db73cc268b525043481708082":{"fields":{"detailData":{"resolvedFrom":"detailDataUnion","type":{"union":{"jobPostingName":"com.linkedin.1febda95b00bfc8c540a7af0dc75a88a","profileFamiliarName":"com.linkedin.c8b3cc209fc06180b17a4ffb72120344","hyperlink":"com.linkedin.common.Url","color":"com.linkedin.voyager.dash.common.text.TextColor","companyName":"com.linkedin.2b5530b88c12ba36048beee3aca27737","icon":"com.linkedin.voyager.dash.common.ArtDecoIconName","epoch":"com.linkedin.feccefa74b57bf711c74d0b09805c9f6","systemImage":"com.linkedin.voyager.dash.common.SystemImageName","textLink":"com.linkedin.72c1c1091a60c5a77aaadac057a84b54","listItemStyle":"com.linkedin.1112232e110fbaf9f6866335ae420a21","listStyle":"com.linkedin.voyager.dash.common.text.ListStyleType","groupName":"com.linkedin.6c29a299f640b6e802d15141c355bd92","hyperlinkOpenExternally":"com.linkedin.common.Url","stringFieldReference":"com.linkedin.65deee184b56ce21c8d7d30fec6acfe4","profileFullName":"com.linkedin.c8b3cc209fc06180b17a4ffb72120344","learningCourseName":"com.linkedin.44f2985bd25b663c32c3c4d4ec070248","profileMention":"com.linkedin.c8b3cc209fc06180b17a4ffb72120344","style":"com.linkedin.voyager.dash.common.text.TextStyle","schoolName":"com.linkedin.1e797f68b521546886989869c5b44a2d","hashtag":"com.linkedin.10f415928dc4647be1fe1eebc72fe277"}}},"length":{"type":"int"},"start":{"type":"int"},"detailDataUnion":{"type":{"union":{"jobPostingName":"string","profileFamiliarName":"string","hyperlink":"com.linkedin.common.Url","color":"com.linkedin.voyager.dash.common.text.TextColor","companyName":"string","icon":"com.linkedin.voyager.dash.common.ArtDecoIconName","epoch":"com.linkedin.feccefa74b57bf711c74d0b09805c9f6","systemImage":"com.linkedin.voyager.dash.common.SystemImageName","textLink":"com.linkedin.72c1c1091a60c5a77aaadac057a84b54","listItemStyle":"com.linkedin.1112232e110fbaf9f6866335ae420a21","listStyle":"com.linkedin.voyager.dash.common.text.ListStyleType","groupName":"string","hyperlinkOpenExternally":"com.linkedin.common.Url","stringFieldReference":"com.linkedin.65deee184b56ce21c8d7d30fec6acfe4","profileFullName":"string","learningCourseName":"string","profileMention":"string","style":"com.linkedin.voyager.dash.common.text.TextStyle","schoolName":"string","hashtag":"string"}}}},"baseType":"com.linkedin.voyager.dash.common.text.TextAttribute"},"com.linkedin.513969f6493543c4ad7f04b46c3b67f6":{"fields":{"controlName":{"type":"com.linkedin.voyager.dash.common.tracking.ControlName"},"appearance":{"type":"com.linkedin.4ef51bc3dc729207a4f87aa1ac1b37cf"},"openExternally":{"type":"boolean"},"accessibilityText":{"type":"string"},"disable":{"type":"boolean"},"url":{"type":"com.linkedin.common.Url"}},"baseType":"com.linkedin.voyager.dash.premium.PremiumButton"},"com.linkedin.73113dd58d976ff22a5e4aa2ca35d21d":{"fields":{"viewProfileText":{"type":"string"},"controlName":{"type":"com.linkedin.voyager.dash.common.tracking.ControlName"},"text":{"type":"string"},"meGroups":{"type":{"array":"com.linkedin.bb35e914e880d9f9ad79d52bf644c44c"}},"profileUrn":{"type":"string"},"profile":{"resolvedFrom":"profileUrn","type":"com.linkedin.c8b3cc209fc06180b17a4ffb72120344"}},"baseType":"com.linkedin.voyager.dash.feed.nav.MeMenu"},"com.linkedin.d6038751fe26f2a90ed06dcfd5acb15e":{"fields":{"memberDistance":{"type":"com.linkedin.voyager.dash.relationships.NoConnectionMemberDistance"}},"baseType":"com.linkedin.voyager.dash.relationships.NoConnection"},"com.linkedin.68d330957951bfe0ba509507962b88b5":{"fields":{"entityUrn":{"type":"com.linkedin.voyager.dash.common.ConnectionUrn"}},"baseType":"com.linkedin.voyager.dash.relationships.Connection"},"com.linkedin.f223abd9a48757f8395ac497a91593f3":{"fields":{"name":{"type":"com.linkedin.566376561ef86249fc89f5596eaf6fe1"},"imageUnion":{"type":{"union":{"url":"com.linkedin.common.Url","vectorImage":"com.linkedin.ca116024396bd29dd1345ad0998a57d5"}}},"image":{"derivedFrom":"imageUnion","type":{"union":{"url":"com.linkedin.common.Url","vectorImage":"com.linkedin.ca116024396bd29dd1345ad0998a57d5"}}}},"baseType":"com.linkedin.voyager.dash.common.media.StickerLinkSmallTemplateView"},"com.linkedin.72c1c1091a60c5a77aaadac057a84b54":{"fields":{"controlName":{"type":"com.linkedin.voyager.dash.common.tracking.ControlName"},"viewingBehavior":{"type":"com.linkedin.voyager.dash.common.ux.UrlViewingBehavior"},"sponsoredUrlAttributes":{"type":"com.linkedin.8fbc11c3b77aca2f1a34ab7a34600da7"},"url":{"type":"com.linkedin.common.Url"}},"baseType":"com.linkedin.voyager.dash.common.text.TextLink"},"com.linkedin.72e4cd594b8572e0ec9ddb034eafc088":{"fields":{"companyUrn":{"type":"string"},"company":{"resolvedFrom":"companyUrn","type":"com.linkedin.2b5530b88c12ba36048beee3aca27737"},"vectorImage":{"type":"com.linkedin.ca116024396bd29dd1345ad0998a57d5"}},"baseType":"com.linkedin.voyager.dash.common.image.NonEntityCompanyLogo"},"com.linkedin.f58e91c23e63f52ceb12bd0fab3eb07b":{"fields":{"utype":{"type":"string"},"referenceId":{"type":"com.linkedin.common.TrackingId"}},"baseType":"com.linkedin.voyager.dash.premium.PremiumFunnelCommonHeader"},"com.linkedin.4973670c8e07d9bf8572c4c7f8658e2b":{"fields":{"controlName":{"type":"com.linkedin.voyager.dash.common.tracking.ControlName"},"logo":{"type":"com.linkedin.f89edd0cdc11632a12f86e550f2f5e46"},"hoverText":{"type":"com.linkedin.566376561ef86249fc89f5596eaf6fe1"}},"baseType":"com.linkedin.voyager.dash.feed.nav.BrandingLogo"},"com.linkedin.8fbc11c3b77aca2f1a34ab7a34600da7":{"fields":{"impressionId":{"type":"string"},"unwrappedUrlDomain":{"type":"string"},"creativeId":{"type":"string"},"campaignId":{"type":"string"},"memberId":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.common.sponsored.SponsoredUrlAttributes"},"com.linkedin.72ff698dfdea20e2538931532a528e81":{"fields":{"entityUrn":{"type":"com.linkedin.voyager.dash.common.MemberRelationshipUrn"},"memberRelationshipUnion":{"type":{"union":{"self":"com.linkedin.797f28906de48c13543c3ff29a412882","connection":"com.linkedin.68d330957951bfe0ba509507962b88b5","noConnection":"com.linkedin.d6038751fe26f2a90ed06dcfd5acb15e"}}},"memberRelationship":{"derivedFrom":"memberRelationshipUnion","type":{"union":{"self":"com.linkedin.797f28906de48c13543c3ff29a412882","connection":"com.linkedin.68d330957951bfe0ba509507962b88b5","noConnection":"com.linkedin.d6038751fe26f2a90ed06dcfd5acb15e"}}}},"baseType":"com.linkedin.voyager.dash.relationships.MemberRelationship"},"com.linkedin.6cf7727c816c79567a0564982ee35ef4":{"fields":{"entityUrn":{"type":"com.linkedin.voyager.dash.common.PremiumUpsellSlotUrn"},"upsellCard":{"type":"com.linkedin.8b2657e6d402c000382bd65eb94ba628"}},"baseType":"com.linkedin.voyager.dash.premium.PremiumUpsellSlotContent"},"com.linkedin.566376561ef86249fc89f5596eaf6fe1":{"fields":{"textDirection":{"type":"com.linkedin.voyager.dash.common.text.TextDirection"},"attributesV2":{"type":{"array":"com.linkedin.9d1c9f8db73cc268b525043481708082"}},"text":{"type":"string"},"accessibilityTextAttributesV2":{"type":{"array":"com.linkedin.9d1c9f8db73cc268b525043481708082"}},"accessibilityText":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.common.text.TextViewModel"},"com.linkedin.f89edd0cdc11632a12f86e550f2f5e46":{"fields":{"editableAccessibilityText":{"type":"boolean"},"attributes":{"type":{"array":"com.linkedin.e6506ba5f5cc1a9b5124781a3661a7db"}},"actionTarget":{"type":"com.linkedin.common.Url"},"accessibilityTextAttributes":{"type":{"array":"com.linkedin.9d1c9f8db73cc268b525043481708082"}},"totalCount":{"type":"int"},"accessibilityText":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.common.image.ImageViewModel"},"com.linkedin.797f28906de48c13543c3ff29a412882":{"fields":{},"baseType":"com.linkedin.restli.common.EmptyRecord"},"com.linkedin.0d2a5c07095246fc3e756f3e922d8aa0":{"fields":{"logoImage":{"type":{"union":{"url":"com.linkedin.common.Url","vectorImage":"com.linkedin.ca116024396bd29dd1345ad0998a57d5"}}},"logoImageResolutionResult":{"derivedFrom":"logoImage","type":{"union":{"url":"com.linkedin.common.Url","vectorImage":"com.linkedin.ca116024396bd29dd1345ad0998a57d5"}}},"entityUrn":{"type":"com.linkedin.voyager.dash.common.ProfessionalEventUrn"}},"baseType":"com.linkedin.voyager.dash.events.ProfessionalEvent"},"com.linkedin.2b5530b88c12ba36048beee3aca27737":{"fields":{"followingStateUrn":{"type":"string"},"entityUrn":{"type":"com.linkedin.voyager.dash.common.CompanyUrn"},"followingState":{"resolvedFrom":"followingStateUrn","type":"com.linkedin.98f5f0b76629959525795b08bdfbcc26"},"name":{"type":"string"},"logo":{"type":{"union":{"url":"com.linkedin.common.Url","vectorImage":"com.linkedin.ca116024396bd29dd1345ad0998a57d5"}}},"logoResolutionResult":{"derivedFrom":"logo","type":{"union":{"url":"com.linkedin.common.Url","vectorImage":"com.linkedin.ca116024396bd29dd1345ad0998a57d5"}}},"universalName":{"type":"string"},"url":{"type":"com.linkedin.common.Url"}},"baseType":"com.linkedin.voyager.dash.organization.Company"},"com.linkedin.feccefa74b57bf711c74d0b09805c9f6":{"fields":{"type":{"type":"com.linkedin.voyager.dash.common.text.EpochFormat"},"epochAt":{"type":"com.linkedin.common.Time"}},"baseType":"com.linkedin.voyager.dash.common.text.EpochTime"},"com.linkedin.1febda95b00bfc8c540a7af0dc75a88a":{"fields":{"title":{"type":"string"},"entityUrn":{"type":"com.linkedin.voyager.dash.common.JobPostingUrn"}},"baseType":"com.linkedin.voyager.dash.jobs.JobPosting"},"com.linkedin.65deee184b56ce21c8d7d30fec6acfe4":{"fields":{"urn":{"type":"com.linkedin.common.Urn"},"modelName":{"type":"string"},"fieldName":{"type":"string"},"value":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.common.StringFieldReference"},"com.linkedin.98f5f0b76629959525795b08bdfbcc26":{"fields":{"followerCount":{"type":"long"},"entityUrn":{"type":"com.linkedin.voyager.dash.common.FollowingStateUrn"}},"baseType":"com.linkedin.voyager.dash.feed.FollowingState"},"com.linkedin.188858304ae14097e59518d245c7cc85":{"fields":{"title":{"type":"com.linkedin.566376561ef86249fc89f5596eaf6fe1"},"controlName":{"type":"com.linkedin.voyager.dash.common.tracking.ControlName"},"actionTarget":{"type":"com.linkedin.common.Url"}},"baseType":"com.linkedin.voyager.dash.feed.nav.BusinessServiceAction"},"com.linkedin.550a9af095fffdd7fc0f6ceb8743a141":{"fields":{"originalImageUrl":{"type":"com.linkedin.common.Url"},"originalHeight":{"type":"int"},"originalWidth":{"type":"int"},"url":{"type":"com.linkedin.common.Url"}},"baseType":"com.linkedin.voyager.dash.common.ImageUrl"},"com.linkedin.44f2985bd25b663c32c3c4d4ec070248":{"fields":{"title":{"type":"string"},"entityUrn":{"type":"com.linkedin.voyager.dash.common.LearningCourseUrn"}},"baseType":"com.linkedin.voyager.dash.learning.LearningCourse"},"com.linkedin.10f415928dc4647be1fe1eebc72fe277":{"fields":{"entityUrn":{"type":"com.linkedin.voyager.dash.common.HashtagUrn"},"trackingUrn":{"type":"com.linkedin.common.Urn"},"actionTarget":{"type":"com.linkedin.common.Url"}},"baseType":"com.linkedin.voyager.dash.feed.Hashtag"},"com.linkedin.c7f876e8e9f8a631b2896f62a656ce64":{"fields":{"type":{"type":"com.linkedin.voyager.dash.feed.nav.NavType"},"navContent":{"resolvedFrom":"navContentUnion","type":{"union":{"premiumUpsellSlotUrn":"com.linkedin.6cf7727c816c79567a0564982ee35ef4","navElement":"com.linkedin.b2c03744140a6cd6a2e693c32790988c"}}},"navContentUnion":{"type":{"union":{"premiumUpsellSlotUrn":"string","navElement":"com.linkedin.b2c03744140a6cd6a2e693c32790988c"}}}},"baseType":"com.linkedin.voyager.dash.feed.nav.NavItem"},"com.linkedin.96db794fa6a744844b487cfa3e195632":{"fields":{"school":{"resolvedFrom":"schoolUrn","type":"com.linkedin.1e797f68b521546886989869c5b44a2d"},"vectorImage":{"type":"com.linkedin.ca116024396bd29dd1345ad0998a57d5"},"schoolUrn":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.common.image.NonEntitySchoolLogo"},"com.linkedin.f94aca2770a997f8a58dfb7d6154954a":{"fields":{"showDivider":{"type":"boolean"},"productItems":{"type":{"array":"com.linkedin.c7f876e8e9f8a631b2896f62a656ce64"}},"productItemsTitle":{"type":"com.linkedin.566376561ef86249fc89f5596eaf6fe1"},"businessServices":{"type":{"array":"com.linkedin.a94ff0919cddaa7056025e9491dc3794"}},"businessServicesTitle":{"type":"string"},"icon":{"type":"com.linkedin.f89edd0cdc11632a12f86e550f2f5e46"},"controlName":{"type":"com.linkedin.voyager.dash.common.tracking.ControlName"},"businessServiceActions":{"type":{"array":"com.linkedin.188858304ae14097e59518d245c7cc85"}},"text":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.feed.nav.AppLauncher"},"com.linkedin.a94ff0919cddaa7056025e9491dc3794":{"fields":{"businessServiceAction":{"type":"com.linkedin.188858304ae14097e59518d245c7cc85"},"description":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.feed.nav.BusinessService"},"com.linkedin.637f09655a21b1df5a03c68636fd2d18":{"fields":{"name":{"type":"com.linkedin.566376561ef86249fc89f5596eaf6fe1"},"image":{"derivedFrom":"imageUnion","type":{"union":{"url":"com.linkedin.common.Url","vectorImage":"com.linkedin.ca116024396bd29dd1345ad0998a57d5"}}},"nameSupplementaryInfo":{"type":"com.linkedin.566376561ef86249fc89f5596eaf6fe1"},"headline":{"type":"com.linkedin.566376561ef86249fc89f5596eaf6fe1"},"imageUnion":{"type":{"union":{"url":"com.linkedin.common.Url","vectorImage":"com.linkedin.ca116024396bd29dd1345ad0998a57d5"}}},"subHeadline":{"type":"com.linkedin.566376561ef86249fc89f5596eaf6fe1"}},"baseType":"com.linkedin.voyager.dash.common.media.StickerLinkMediumTemplateView"},"com.linkedin.c8b3cc209fc06180b17a4ffb72120344":{"fields":{"profilePicture":{"type":"com.linkedin.bda725ec0f4ddc7380fb44de84e29723"},"lastName":{"type":"string"},"firstName":{"type":"string"},"entityUrn":{"type":"com.linkedin.voyager.dash.common.ProfileUrn"},"memberRelationship":{"type":"com.linkedin.72ff698dfdea20e2538931532a528e81","isInjection":true},"ringStatus":{"type":"com.linkedin.61dcef34ee68ce7a34919062297c038b","isInjection":true},"headline":{"type":"string"},"publicIdentifier":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.identity.profile.Profile"},"com.linkedin.4ef51bc3dc729207a4f87aa1ac1b37cf":{"fields":{"premiumStyle":{"type":"boolean"},"size":{"type":"com.linkedin.voyager.dash.common.ux.button.ButtonSize"},"icon":{"type":"com.linkedin.voyager.dash.common.SystemImageName"},"text":{"type":"string"},"iconAfterText":{"type":"boolean"},"emphasize":{"type":"boolean"},"category":{"type":"com.linkedin.voyager.dash.common.ux.button.ButtonCategory"}},"baseType":"com.linkedin.voyager.dash.common.ux.button.ButtonAppearance"},"com.linkedin.e6506ba5f5cc1a9b5124781a3661a7db":{"fields":{"detailData":{"resolvedFrom":"detailDataUnion","type":{"union":{"profilePictureWithoutFrame":"com.linkedin.c8b3cc209fc06180b17a4ffb72120344","profilePictureWithRingStatus":"com.linkedin.c8b3cc209fc06180b17a4ffb72120344","companyLogo":"com.linkedin.2b5530b88c12ba36048beee3aca27737","icon":"com.linkedin.voyager.dash.common.ArtDecoIconName","systemImage":"com.linkedin.voyager.dash.common.SystemImageName","nonEntityGroupLogo":"com.linkedin.dcb85e1ca353361cb2b5f6fa24a1c9af","vectorImage":"com.linkedin.ca116024396bd29dd1345ad0998a57d5","nonEntityProfessionalEventLogo":"com.linkedin.371c4584dc45b7919c4eb6ed6697e8cf","profilePicture":"com.linkedin.c8b3cc209fc06180b17a4ffb72120344","imageUrl":"com.linkedin.550a9af095fffdd7fc0f6ceb8743a141","professionalEventLogo":"com.linkedin.0d2a5c07095246fc3e756f3e922d8aa0","nonEntitySchoolLogo":"com.linkedin.96db794fa6a744844b487cfa3e195632","nonEntityCompanyLogo":"com.linkedin.72e4cd594b8572e0ec9ddb034eafc088","schoolLogo":"com.linkedin.1e797f68b521546886989869c5b44a2d","groupLogo":"com.linkedin.6c29a299f640b6e802d15141c355bd92","ghostImage":"com.linkedin.voyager.dash.common.image.GhostImageType","nonEntityProfilePicture":"com.linkedin.1c710707068d7b33bcbee05036904a3b"}}},"tintColor":{"type":"com.linkedin.voyager.dash.common.SystemImageTintColor"},"detailDataUnion":{"type":{"union":{"profilePictureWithoutFrame":"string","profilePictureWithRingStatus":"string","companyLogo":"string","icon":"com.linkedin.voyager.dash.common.ArtDecoIconName","systemImage":"com.linkedin.voyager.dash.common.SystemImageName","nonEntityGroupLogo":"com.linkedin.dcb85e1ca353361cb2b5f6fa24a1c9af","vectorImage":"com.linkedin.ca116024396bd29dd1345ad0998a57d5","nonEntityProfessionalEventLogo":"com.linkedin.371c4584dc45b7919c4eb6ed6697e8cf","profilePicture":"string","imageUrl":"com.linkedin.550a9af095fffdd7fc0f6ceb8743a141","professionalEventLogo":"string","nonEntitySchoolLogo":"com.linkedin.96db794fa6a744844b487cfa3e195632","nonEntityCompanyLogo":"com.linkedin.72e4cd594b8572e0ec9ddb034eafc088","schoolLogo":"string","groupLogo":"string","ghostImage":"com.linkedin.voyager.dash.common.image.GhostImageType","nonEntityProfilePicture":"com.linkedin.1c710707068d7b33bcbee05036904a3b"}}},"tapTargets":{"type":{"array":"com.linkedin.d66f0cb7963e0345d3ff893927e60c88"}},"scalingType":{"type":"com.linkedin.voyager.dash.common.image.ScalingType"},"displayAspectRatio":{"type":"double"}},"baseType":"com.linkedin.voyager.dash.common.image.ImageAttribute"},"com.linkedin.ca116024396bd29dd1345ad0998a57d5":{"fields":{"attribution":{"type":"string"},"digitalmediaAsset":{"type":"com.linkedin.common.DigitalmediaAssetUrn"},"rootUrl":{"type":"string"},"artifacts":{"type":{"array":"com.linkedin.f1332bd87cf4dccab8aa4b4583ad44be"}}},"baseType":"com.linkedin.common.VectorImage"}}}},"included":[{"firstName":"Naim Iskandar","lastName":"Zahari","profilePicture":{"displayImageReferenceResolutionResult":{"url":null,"vectorImage":{"digitalmediaAsset":null,"attribution":null,"$recipeTypes":["com.linkedin.ca116024396bd29dd1345ad0998a57d5"],"artifacts":[{"width":99,"$recipeTypes":["com.linkedin.f1332bd87cf4dccab8aa4b4583ad44be"],"fileIdentifyingUrlPathSegment":"100_100/0/1605279012830?e=1706745600&amp;v=beta&amp;t=GoGDD2DE3nGXji1ktdhEWgWX0uNRNd93omIm9L4PTBM","expiresAt":1706745600000,"height":100,"$type":"com.linkedin.common.VectorArtifact"},{"width":199,"$recipeTypes":["com.linkedin.f1332bd87cf4dccab8aa4b4583ad44be"],"fileIdentifyingUrlPathSegment":"200_200/0/1605279012830?e=1706745600&amp;v=beta&amp;t=KxJr8NDL0QX2hH6Hgu-7kOudbuCrhYafGAmjrDIUX6g","expiresAt":1706745600000,"height":200,"$type":"com.linkedin.common.VectorArtifact"},{"width":399,"$recipeTypes":["com.linkedin.f1332bd87cf4dccab8aa4b4583ad44be"],"fileIdentifyingUrlPathSegment":"400_400/0/1605279012830?e=1706745600&amp;v=beta&amp;t=MEX6KTkEsBN1Iwy4RYn8IoyQikzi3wK-ws0Ij5Qr478","expiresAt":1706745600000,"height":400,"$type":"com.linkedin.common.VectorArtifact"},{"width":670,"$recipeTypes":["com.linkedin.f1332bd87cf4dccab8aa4b4583ad44be"],"fileIdentifyingUrlPathSegment":"800_800/0/1605279012830?e=1706745600&amp;v=beta&amp;t=72OsosG2vCVJHRjJ9BEqeLcfgJkzqOn-4Eq5tGu4964","expiresAt":1706745600000,"height":670,"$type":"com.linkedin.common.VectorArtifact"}],"rootUrl":"https://media.licdn.com/dms/image/C5603AQEkRciOp6B6dA/profile-displayphoto-shrink_","$type":"com.linkedin.common.VectorImage"}},"displayImageWithFrameReference":null,"$recipeTypes":["com.linkedin.bda725ec0f4ddc7380fb44de84e29723"],"$type":"com.linkedin.voyager.dash.identity.profile.PhotoFilterPicture"},"entityUrn":"urn:li:fsd_profile:ACoAAC6BPkEBwSD4NTQ7khSJbuAxrRMYhVSwmBo","headline":"Performance Analyst | Aspiring Data Scientist | Data Enthusiast | Game Programmer","publicIdentifier":"naimiskandar22","$recipeTypes":["com.linkedin.c8b3cc209fc06180b17a4ffb72120344"],"$type":"com.linkedin.voyager.dash.identity.profile.Profile"}]}
</code>
<code style="display: none" id="datalet-bpr-guid-15912213">
  {"request":"/voyager/api/graphql?includeWebMetadata\u003Dtrue\u0026variables\u003D()\u0026queryId\u003DvoyagerFeedDashGlobalNavs.604cc295a50aa27ae69147afac5ff0e5","status":200,"body":"bpr-guid-15912213","method":"GET","headers":{"x-li-uuid":"AAYLE+vtCEZ6Amp7P3S1tA\u003D\u003D"}}
</code>
<img src="data:image/gif;base64,R0lGODlhAQABAIAAAAAAAP///yH5BAEAAAAALAAAAAABAAEAAAIBRAA7" style="display: none" class="datalet-bpr-guid-15912213"><code style="display: none" id="bpr-guid-15912214">
  {"data":{"data":{"premiumDashFeatureAccessByFeatureAccessTypes":{"*elements":["urn:li:fsd_featureAccess:CAN_ACCESS_SALES_NAV_BADGE","urn:li:fsd_featureAccess:CAN_ACCESS_ADVERTISE_BADGE"],"$recipeTypes":["com.linkedin.3fad2247165582aa95a2690797d7e0ab"],"$type":"com.linkedin.restli.common.CollectionResponse"},"$recipeTypes":["com.linkedin.6488012243094b4ef82d764d10e79d45"],"$type":"com.linkedin.6488012243094b4ef82d764d10e79d45"},"extensions":{"webMetadata":{}}},"meta":{"microSchema":{"isGraphQL":true,"version":"2.1","types":{"com.linkedin.3fad2247165582aa95a2690797d7e0ab":{"fields":{"elements":{"type":{"array":"com.linkedin.9e452672002943dc9a1521bcc7ad23cb"}}},"baseType":"com.linkedin.restli.common.CollectionResponse"},"com.linkedin.9e452672002943dc9a1521bcc7ad23cb":{"fields":{"entityUrn":{"type":"com.linkedin.voyager.dash.common.FeatureAccessUrn"},"hasAccess":{"type":"boolean"},"featureAccessType":{"type":"com.linkedin.voyager.dash.premium.FeatureAccessType"}},"baseType":"com.linkedin.voyager.dash.premium.FeatureAccess"},"com.linkedin.6488012243094b4ef82d764d10e79d45":{"fields":{"premiumDashFeatureAccessByFeatureAccessTypes":{"type":"com.linkedin.3fad2247165582aa95a2690797d7e0ab"}},"baseType":"com.linkedin.graphql.Query"}}}},"included":[{"featureAccessType":"CAN_ACCESS_SALES_NAV_BADGE","entityUrn":"urn:li:fsd_featureAccess:CAN_ACCESS_SALES_NAV_BADGE","hasAccess":false,"$recipeTypes":["com.linkedin.9e452672002943dc9a1521bcc7ad23cb"],"$type":"com.linkedin.voyager.dash.premium.FeatureAccess"},{"featureAccessType":"CAN_ACCESS_ADVERTISE_BADGE","entityUrn":"urn:li:fsd_featureAccess:CAN_ACCESS_ADVERTISE_BADGE","hasAccess":false,"$recipeTypes":["com.linkedin.9e452672002943dc9a1521bcc7ad23cb"],"$type":"com.linkedin.voyager.dash.premium.FeatureAccess"}]}
</code>
<code style="display: none" id="datalet-bpr-guid-15912214">
  {"request":"/voyager/api/graphql?includeWebMetadata\u003Dtrue\u0026variables\u003D(featureAccessTypes:List(CAN_ACCESS_SALES_NAV_BADGE,CAN_ACCESS_ADVERTISE_BADGE))\u0026queryId\u003DvoyagerPremiumDashFeatureAccess.c87b20dac35795f9920f2a8072fd7af5","status":200,"body":"bpr-guid-15912214","method":"GET","headers":{"x-li-uuid":"AAYLE+vtCEZ6Amp7P3S1tA\u003D\u003D"}}
</code>
<img src="data:image/gif;base64,R0lGODlhAQABAIAAAAAAAP///yH5BAEAAAAALAAAAAABAAEAAAIBRAA7" style="display: none" class="datalet-bpr-guid-15912214"><code style="display: none" id="bpr-guid-15912215">
  {"data":{"entityUrn":"urn:li:collectionResponse:7ZvAwvPwYldk7Tvl3jXgC4ymrtxthgc+3EdE7ncMd98=","elements":[{"count":0,"tab":"MY_NETWORK","$type":"com.linkedin.voyager.common.communications.TabBadge"},{"count":0,"tab":"MESSAGING","$type":"com.linkedin.voyager.common.communications.TabBadge"},{"count":0,"tab":"NOTIFICATIONS","$type":"com.linkedin.voyager.common.communications.TabBadge"}],"paging":{"count":10,"start":0,"links":[]},"$type":"com.linkedin.restli.common.CollectionResponse"},"included":[]}
</code>
<code style="display: none" id="datalet-bpr-guid-15912215">
  {"request":"/voyager/api/voyagerCommunicationsTabBadges?q\u003DtabBadges\u0026countFrom\u003D0","status":200,"body":"bpr-guid-15912215","method":"GET","headers":{"x-li-uuid":"AAYLE+vtCEZ6Amp7P3S1tA\u003D\u003D"}}
</code>
<img src="data:image/gif;base64,R0lGODlhAQABAIAAAAAAAP///yH5BAEAAAAALAAAAAABAAEAAAIBRAA7" style="display: none" class="datalet-bpr-guid-15912215"><code style="display: none" id="bpr-guid-15912216">
  {"data":{"dashEntityUrn":"urn:li:fsd_jobPosting:3768592532","companyDetails":{"company":"urn:li:fs_normalized_company:43338549","*companyResolutionResult":"urn:li:fs_normalized_company:43338549","$recipeTypes":["com.linkedin.voyager.deco.jobs.web.shared.WebCompactJobPostingCompany"],"$type":"com.linkedin.voyager.jobs.JobPostingCompany"},"jobState":"LISTED","description":{"attributes":[{"start":75,"length":1,"type":{"$type":"com.linkedin.pemberly.text.LineBreak"},"attributeKindUnion":{"lineBreak":{"$type":"com.linkedin.pemberly.text.LineBreak"}},"$type":"com.linkedin.pemberly.text.Attribute"},{"start":76,"length":1,"type":{"$type":"com.linkedin.pemberly.text.LineBreak"},"attributeKindUnion":{"lineBreak":{"$type":"com.linkedin.pemberly.text.LineBreak"}},"$type":"com.linkedin.pemberly.text.Attribute"},{"start":0,"length":77,"type":{"$type":"com.linkedin.pemberly.text.Bold"},"attributeKindUnion":{"bold":{"$type":"com.linkedin.pemberly.text.Bold"}},"$type":"com.linkedin.pemberly.text.Attribute"},{"start":365,"length":1,"type":{"$type":"com.linkedin.pemberly.text.LineBreak"},"attributeKindUnion":{"lineBreak":{"$type":"com.linkedin.pemberly.text.LineBreak"}},"$type":"com.linkedin.pemberly.text.Attribute"},{"start":366,"length":1,"type":{"$type":"com.linkedin.pemberly.text.LineBreak"},"attributeKindUnion":{"lineBreak":{"$type":"com.linkedin.pemberly.text.LineBreak"}},"$type":"com.linkedin.pemberly.text.Attribute"},{"start":577,"length":1,"type":{"$type":"com.linkedin.pemberly.text.LineBreak"},"attributeKindUnion":{"lineBreak":{"$type":"com.linkedin.pemberly.text.LineBreak"}},"$type":"com.linkedin.pemberly.text.Attribute"},{"start":578,"length":1,"type":{"$type":"com.linkedin.pemberly.text.LineBreak"},"attributeKindUnion":{"lineBreak":{"$type":"com.linkedin.pemberly.text.LineBreak"}},"$type":"com.linkedin.pemberly.text.Attribute"},{"start":791,"length":1,"type":{"$type":"com.linkedin.pemberly.text.LineBreak"},"attributeKindUnion":{"lineBreak":{"$type":"com.linkedin.pemberly.text.LineBreak"}},"$type":"com.linkedin.pemberly.text.Attribute"},{"start":792,"length":1,"type":{"$type":"com.linkedin.pemberly.text.LineBreak"},"attributeKindUnion":{"lineBreak":{"$type":"com.linkedin.pemberly.text.LineBreak"}},"$type":"com.linkedin.pemberly.text.Attribute"},{"start":883,"length":1,"type":{"$type":"com.linkedin.pemberly.text.LineBreak"},"attributeKindUnion":{"lineBreak":{"$type":"com.linkedin.pemberly.text.LineBreak"}},"$type":"com.linkedin.pemberly.text.Attribute"},{"start":884,"length":1,"type":{"$type":"com.linkedin.pemberly.text.LineBreak"},"attributeKindUnion":{"lineBreak":{"$type":"com.linkedin.pemberly.text.LineBreak"}},"$type":"com.linkedin.pemberly.text.Attribute"},{"start":890,"length":1,"type":{"$type":"com.linkedin.pemberly.text.LineBreak"},"attributeKindUnion":{"lineBreak":{"$type":"com.linkedin.pemberly.text.LineBreak"}},"$type":"com.linkedin.pemberly.text.Attribute"},{"start":891,"length":1,"type":{"$type":"com.linkedin.pemberly.text.LineBreak"},"attributeKindUnion":{"lineBreak":{"$type":"com.linkedin.pemberly.text.LineBreak"}},"$type":"com.linkedin.pemberly.text.Attribute"},{"start":885,"length":7,"type":{"$type":"com.linkedin.pemberly.text.Bold"},"attributeKindUnion":{"bold":{"$type":"com.linkedin.pemberly.text.Bold"}},"$type":"com.linkedin.pemberly.text.Attribute"},{"start":907,"length":1,"type":{"$type":"com.linkedin.pemberly.text.LineBreak"},"attributeKindUnion":{"lineBreak":{"$type":"com.linkedin.pemberly.text.LineBreak"}},"$type":"com.linkedin.pemberly.text.Attribute"},{"start":908,"length":1,"type":{"$type":"com.linkedin.pemberly.text.LineBreak"},"attributeKindUnion":{"lineBreak":{"$type":"com.linkedin.pemberly.text.LineBreak"}},"$type":"com.linkedin.pemberly.text.Attribute"},{"start":892,"length":17,"type":{"$type":"com.linkedin.pemberly.text.Bold"},"attributeKindUnion":{"bold":{"$type":"com.linkedin.pemberly.text.Bold"}},"$type":"com.linkedin.pemberly.text.Attribute"},{"start":909,"length":118,"type":{"$type":"com.linkedin.pemberly.text.ListItem"},"attributeKindUnion":{"listItem":{"$type":"com.linkedin.pemberly.text.ListItem"}},"$type":"com.linkedin.pemberly.text.Attribute"},{"start":1027,"length":156,"type":{"$type":"com.linkedin.pemberly.text.ListItem"},"attributeKindUnion":{"listItem":{"$type":"com.linkedin.pemberly.text.ListItem"}},"$type":"com.linkedin.pemberly.text.Attribute"},{"start":1183,"length":116,"type":{"$type":"com.linkedin.pemberly.text.ListItem"},"attributeKindUnion":{"listItem":{"$type":"com.linkedin.pemberly.text.ListItem"}},"$type":"com.linkedin.pemberly.text.Attribute"},{"start":1299,"length":144,"type":{"$type":"com.linkedin.pemberly.text.ListItem"},"attributeKindUnion":{"listItem":{"$type":"com.linkedin.pemberly.text.ListItem"}},"$type":"com.linkedin.pemberly.text.Attribute"},{"start":1443,"length":99,"type":{"$type":"com.linkedin.pemberly.text.ListItem"},"attributeKindUnion":{"listItem":{"$type":"com.linkedin.pemberly.text.ListItem"}},"$type":"com.linkedin.pemberly.text.Attribute"},{"start":1665,"length":1,"type":{"$type":"com.linkedin.pemberly.text.LineBreak"},"attributeKindUnion":{"lineBreak":{"$type":"com.linkedin.pemberly.text.LineBreak"}},"$type":"com.linkedin.pemberly.text.Attribute"},{"start":1666,"length":1,"type":{"$type":"com.linkedin.pemberly.text.LineBreak"},"attributeKindUnion":{"lineBreak":{"$type":"com.linkedin.pemberly.text.LineBreak"}},"$type":"com.linkedin.pemberly.text.Attribute"},{"start":1542,"length":125,"type":{"$type":"com.linkedin.pemberly.text.ListItem"},"attributeKindUnion":{"listItem":{"$type":"com.linkedin.pemberly.text.ListItem"}},"$type":"com.linkedin.pemberly.text.Attribute"},{"start":909,"length":758,"type":{"ordered":false,"$type":"com.linkedin.pemberly.text.List"},"attributeKindUnion":{"list":{"ordered":false,"$type":"com.linkedin.pemberly.text.List"}},"$type":"com.linkedin.pemberly.text.Attribute"},{"start":1679,"length":1,"type":{"$type":"com.linkedin.pemberly.text.LineBreak"},"attributeKindUnion":{"lineBreak":{"$type":"com.linkedin.pemberly.text.LineBreak"}},"$type":"com.linkedin.pemberly.text.Attribute"},{"start":1680,"length":1,"type":{"$type":"com.linkedin.pemberly.text.LineBreak"},"attributeKindUnion":{"lineBreak":{"$type":"com.linkedin.pemberly.text.LineBreak"}},"$type":"com.linkedin.pemberly.text.Attribute"},{"start":1667,"length":14,"type":{"$type":"com.linkedin.pemberly.text.Bold"},"attributeKindUnion":{"bold":{"$type":"com.linkedin.pemberly.text.Bold"}},"$type":"com.linkedin.pemberly.text.Attribute"},{"start":1693,"length":1,"type":{"$type":"com.linkedin.pemberly.text.LineBreak"},"attributeKindUnion":{"lineBreak":{"$type":"com.linkedin.pemberly.text.LineBreak"}},"$type":"com.linkedin.pemberly.text.Attribute"},{"start":1694,"length":1,"type":{"$type":"com.linkedin.pemberly.text.LineBreak"},"attributeKindUnion":{"lineBreak":{"$type":"com.linkedin.pemberly.text.LineBreak"}},"$type":"com.linkedin.pemberly.text.Attribute"},{"start":1681,"length":14,"type":{"$type":"com.linkedin.pemberly.text.Bold"},"attributeKindUnion":{"bold":{"$type":"com.linkedin.pemberly.text.Bold"}},"$type":"com.linkedin.pemberly.text.Attribute"},{"start":1695,"length":121,"type":{"$type":"com.linkedin.pemberly.text.ListItem"},"attributeKindUnion":{"listItem":{"$type":"com.linkedin.pemberly.text.ListItem"}},"$type":"com.linkedin.pemberly.text.Attribute"},{"start":1816,"length":159,"type":{"$type":"com.linkedin.pemberly.text.ListItem"},"attributeKindUnion":{"listItem":{"$type":"com.linkedin.pemberly.text.ListItem"}},"$type":"com.linkedin.pemberly.text.Attribute"},{"start":2126,"length":1,"type":{"$type":"com.linkedin.pemberly.text.LineBreak"},"attributeKindUnion":{"lineBreak":{"$type":"com.linkedin.pemberly.text.LineBreak"}},"$type":"com.linkedin.pemberly.text.Attribute"},{"start":2127,"length":1,"type":{"$type":"com.linkedin.pemberly.text.LineBreak"},"attributeKindUnion":{"lineBreak":{"$type":"com.linkedin.pemberly.text.LineBreak"}},"$type":"com.linkedin.pemberly.text.Attribute"},{"start":1975,"length":153,"type":{"$type":"com.linkedin.pemberly.text.ListItem"},"attributeKindUnion":{"listItem":{"$type":"com.linkedin.pemberly.text.ListItem"}},"$type":"com.linkedin.pemberly.text.Attribute"},{"start":1695,"length":433,"type":{"ordered":false,"$type":"com.linkedin.pemberly.text.List"},"attributeKindUnion":{"list":{"ordered":false,"$type":"com.linkedin.pemberly.text.List"}},"$type":"com.linkedin.pemberly.text.Attribute"},{"start":2151,"length":1,"type":{"$type":"com.linkedin.pemberly.text.LineBreak"},"attributeKindUnion":{"lineBreak":{"$type":"com.linkedin.pemberly.text.LineBreak"}},"$type":"com.linkedin.pemberly.text.Attribute"},{"start":2152,"length":1,"type":{"$type":"com.linkedin.pemberly.text.LineBreak"},"attributeKindUnion":{"lineBreak":{"$type":"com.linkedin.pemberly.text.LineBreak"}},"$type":"com.linkedin.pemberly.text.Attribute"},{"start":2128,"length":25,"type":{"$type":"com.linkedin.pemberly.text.Bold"},"attributeKindUnion":{"bold":{"$type":"com.linkedin.pemberly.text.Bold"}},"$type":"com.linkedin.pemberly.text.Attribute"},{"start":2153,"length":148,"type":{"$type":"com.linkedin.pemberly.text.ListItem"},"attributeKindUnion":{"listItem":{"$type":"com.linkedin.pemberly.text.ListItem"}},"$type":"com.linkedin.pemberly.text.Attribute"},{"start":2301,"length":152,"type":{"$type":"com.linkedin.pemberly.text.ListItem"},"attributeKindUnion":{"listItem":{"$type":"com.linkedin.pemberly.text.ListItem"}},"$type":"com.linkedin.pemberly.text.Attribute"},{"start":2517,"length":1,"type":{"$type":"com.linkedin.pemberly.text.LineBreak"},"attributeKindUnion":{"lineBreak":{"$type":"com.linkedin.pemberly.text.LineBreak"}},"$type":"com.linkedin.pemberly.text.Attribute"},{"start":2518,"length":1,"type":{"$type":"com.linkedin.pemberly.text.LineBreak"},"attributeKindUnion":{"lineBreak":{"$type":"com.linkedin.pemberly.text.LineBreak"}},"$type":"com.linkedin.pemberly.text.Attribute"},{"start":2453,"length":66,"type":{"$type":"com.linkedin.pemberly.text.ListItem"},"attributeKindUnion":{"listItem":{"$type":"com.linkedin.pemberly.text.ListItem"}},"$type":"com.linkedin.pemberly.text.Attribute"},{"start":2153,"length":366,"type":{"ordered":false,"$type":"com.linkedin.pemberly.text.List"},"attributeKindUnion":{"list":{"ordered":false,"$type":"com.linkedin.pemberly.text.List"}},"$type":"com.linkedin.pemberly.text.Attribute"},{"start":2569,"length":1,"type":{"$type":"com.linkedin.pemberly.text.LineBreak"},"attributeKindUnion":{"lineBreak":{"$type":"com.linkedin.pemberly.text.LineBreak"}},"$type":"com.linkedin.pemberly.text.Attribute"},{"start":2570,"length":1,"type":{"$type":"com.linkedin.pemberly.text.LineBreak"},"attributeKindUnion":{"lineBreak":{"$type":"com.linkedin.pemberly.text.LineBreak"}},"$type":"com.linkedin.pemberly.text.Attribute"},{"start":2519,"length":52,"type":{"$type":"com.linkedin.pemberly.text.Bold"},"attributeKindUnion":{"bold":{"$type":"com.linkedin.pemberly.text.Bold"}},"$type":"com.linkedin.pemberly.text.Attribute"},{"start":2571,"length":90,"type":{"$type":"com.linkedin.pemberly.text.ListItem"},"attributeKindUnion":{"listItem":{"$type":"com.linkedin.pemberly.text.ListItem"}},"$type":"com.linkedin.pemberly.text.Attribute"},{"start":2661,"length":101,"type":{"$type":"com.linkedin.pemberly.text.ListItem"},"attributeKindUnion":{"listItem":{"$type":"com.linkedin.pemberly.text.ListItem"}},"$type":"com.linkedin.pemberly.text.Attribute"},{"start":2823,"length":1,"type":{"$type":"com.linkedin.pemberly.text.LineBreak"},"attributeKindUnion":{"lineBreak":{"$type":"com.linkedin.pemberly.text.LineBreak"}},"$type":"com.linkedin.pemberly.text.Attribute"},{"start":2824,"length":1,"type":{"$type":"com.linkedin.pemberly.text.LineBreak"},"attributeKindUnion":{"lineBreak":{"$type":"com.linkedin.pemberly.text.LineBreak"}},"$type":"com.linkedin.pemberly.text.Attribute"},{"start":2762,"length":63,"type":{"$type":"com.linkedin.pemberly.text.ListItem"},"attributeKindUnion":{"listItem":{"$type":"com.linkedin.pemberly.text.ListItem"}},"$type":"com.linkedin.pemberly.text.Attribute"},{"start":2571,"length":254,"type":{"ordered":false,"$type":"com.linkedin.pemberly.text.List"},"attributeKindUnion":{"list":{"ordered":false,"$type":"com.linkedin.pemberly.text.List"}},"$type":"com.linkedin.pemberly.text.Attribute"},{"start":2833,"length":1,"type":{"$type":"com.linkedin.pemberly.text.LineBreak"},"attributeKindUnion":{"lineBreak":{"$type":"com.linkedin.pemberly.text.LineBreak"}},"$type":"com.linkedin.pemberly.text.Attribute"},{"start":2834,"length":1,"type":{"$type":"com.linkedin.pemberly.text.LineBreak"},"attributeKindUnion":{"lineBreak":{"$type":"com.linkedin.pemberly.text.LineBreak"}},"$type":"com.linkedin.pemberly.text.Attribute"},{"start":2825,"length":10,"type":{"$type":"com.linkedin.pemberly.text.Bold"},"attributeKindUnion":{"bold":{"$type":"com.linkedin.pemberly.text.Bold"}},"$type":"com.linkedin.pemberly.text.Attribute"},{"start":2849,"length":1,"type":{"$type":"com.linkedin.pemberly.text.LineBreak"},"attributeKindUnion":{"lineBreak":{"$type":"com.linkedin.pemberly.text.LineBreak"}},"$type":"com.linkedin.pemberly.text.Attribute"},{"start":2850,"length":1,"type":{"$type":"com.linkedin.pemberly.text.LineBreak"},"attributeKindUnion":{"lineBreak":{"$type":"com.linkedin.pemberly.text.LineBreak"}},"$type":"com.linkedin.pemberly.text.Attribute"},{"start":2835,"length":16,"type":{"$type":"com.linkedin.pemberly.text.Bold"},"attributeKindUnion":{"bold":{"$type":"com.linkedin.pemberly.text.Bold"}},"$type":"com.linkedin.pemberly.text.Attribute"},{"start":2851,"length":172,"type":{"$type":"com.linkedin.pemberly.text.ListItem"},"attributeKindUnion":{"listItem":{"$type":"com.linkedin.pemberly.text.ListItem"}},"$type":"com.linkedin.pemberly.text.Attribute"},{"start":3023,"length":40,"type":{"$type":"com.linkedin.pemberly.text.ListItem"},"attributeKindUnion":{"listItem":{"$type":"com.linkedin.pemberly.text.ListItem"}},"$type":"com.linkedin.pemberly.text.Attribute"},{"start":3063,"length":52,"type":{"$type":"com.linkedin.pemberly.text.ListItem"},"attributeKindUnion":{"listItem":{"$type":"com.linkedin.pemberly.text.ListItem"}},"$type":"com.linkedin.pemberly.text.Attribute"},{"start":3115,"length":40,"type":{"$type":"com.linkedin.pemberly.text.ListItem"},"attributeKindUnion":{"listItem":{"$type":"com.linkedin.pemberly.text.ListItem"}},"$type":"com.linkedin.pemberly.text.Attribute"},{"start":3155,"length":124,"type":{"$type":"com.linkedin.pemberly.text.ListItem"},"attributeKindUnion":{"listItem":{"$type":"com.linkedin.pemberly.text.ListItem"}},"$type":"com.linkedin.pemberly.text.Attribute"},{"start":3279,"length":85,"type":{"$type":"com.linkedin.pemberly.text.ListItem"},"attributeKindUnion":{"listItem":{"$type":"com.linkedin.pemberly.text.ListItem"}},"$type":"com.linkedin.pemberly.text.Attribute"},{"start":3432,"length":1,"type":{"$type":"com.linkedin.pemberly.text.LineBreak"},"attributeKindUnion":{"lineBreak":{"$type":"com.linkedin.pemberly.text.LineBreak"}},"$type":"com.linkedin.pemberly.text.Attribute"},{"start":3433,"length":1,"type":{"$type":"com.linkedin.pemberly.text.LineBreak"},"attributeKindUnion":{"lineBreak":{"$type":"com.linkedin.pemberly.text.LineBreak"}},"$type":"com.linkedin.pemberly.text.Attribute"},{"start":3364,"length":70,"type":{"$type":"com.linkedin.pemberly.text.ListItem"},"attributeKindUnion":{"listItem":{"$type":"com.linkedin.pemberly.text.ListItem"}},"$type":"com.linkedin.pemberly.text.Attribute"},{"start":2851,"length":583,"type":{"ordered":false,"$type":"com.linkedin.pemberly.text.List"},"attributeKindUnion":{"list":{"ordered":false,"$type":"com.linkedin.pemberly.text.List"}},"$type":"com.linkedin.pemberly.text.Attribute"},{"start":3704,"length":1,"type":{"$type":"com.linkedin.pemberly.text.LineBreak"},"attributeKindUnion":{"lineBreak":{"$type":"com.linkedin.pemberly.text.LineBreak"}},"$type":"com.linkedin.pemberly.text.Attribute"},{"start":3705,"length":1,"type":{"$type":"com.linkedin.pemberly.text.LineBreak"},"attributeKindUnion":{"lineBreak":{"$type":"com.linkedin.pemberly.text.LineBreak"}},"$type":"com.linkedin.pemberly.text.Attribute"}],"text":"Join Our Mission to Fight Climate Change as our Sustainability Data Analyst\n\nWant to be part of one of Europe's fastest growing climate tech startups? We are looking for a passionate Sustainability Data Analyst to help us transform every company into a sustainable business, with critical responsibility and influence on our product and overall venture development.\n\nAs a Sustainability Data Analyst your role will have a key responsibility of applying data-driven approaches to managing our carbon footprint calculations and ensuring our customers understand how we got there.\n\nIf you are excited about building an economy where every company creates an impact for a sustainable world and empowering companies to decarbonize and boost their business performance, we’d love to hear from you!\n\nIf you want to find out more about Cozero and our approach to hiring, check out this page.\n\nTasks\n\nWhat you’ll do:\n\nYou will be developing, maintaining and improving our methodologies and models for calculating companies GHG emissionsYou will develop and oversee processes to ensure our clients receive accurate CCF and PCF results (eg. Quality assurance, validating user carbon footprints)You will transform unstructured data sets so they are ready to be ingested into our calculation databases and modelsYou will work across Cozero’s climate, data science and engineering teams, to deliver on high quality data outputs for internal and external useYou will work closely with our Sales and Customer Success teams to define and prioritize data needsYou will proactively develop ideas on how we can create more value out of our data as well as external data for our clients\n\nRequirements\n\nWho you are:\n\nYou care deeply about our planet and staying informed on key environmental trends like CO2 emissions and climate change. Your meticulous eye for detail ensures thorough, systematic and accurate data analysis even when working with complex, unstructured and inconsistent datasets. A fast-evolving environment excites you, and you love contributing your skills to a diverse, collaborative team making a real difference in the world. \n\nWhat we’re looking for:\n\nExperience working in an analyst role or similar (ideally working with climate data like Corporate GHG emissions, Emission factor and LCA databases)You have programming skills in Python and SQL and a strong understanding of data gathering, inspecting, cleansing, transforming, and modeling techniquesYou have strong verbal and written English communication skills \n\nExtra bonus points if you also have the following:\n\nMaster's degree in Environmental Science, Engineering, Computer Science or a related fieldKnowledge of different carbon accounting standards and databases (e.g. GHG Protocol, GLEC, Ecoinvent)Experience in carbon accounting and/ or life cycle assessment\n\nBenefits\n\nWhat we offer:\n\nOpportunity to work remotely or from our great co-working space in Maschinenraum Berlin with access to innovation events of the German Mittelstand and the startup ecosystemVirtual meditation and wellness sessionsRegular team events and at least 2 offsites per yearIndividual learning &amp; development budgetThe option to participate in our Virtual Stock Option Plan (VSOP), to have direct involvement and impact on Cozero’s successA strong company culture focused on growth, collaboration and fighting climate changeParticipation opportunities in accelerator, startup &amp; event programs\n\nWe strive to build a diverse, inclusive and collaborative workplace and we follow a fair, objective, and non-discriminatory recruitment process. All applications will be considered for employment regardless of nationality, culture, or any other personal characteristics.\n\nThe glue that binds us together despite our differences is a common set of values, our purpose to transform every company into a low-emission business, and our culture that has a growth mindset at its heart. If you’re excited about this role we encourage you to apply, even if your past experience doesn’t perfectly match every requirement in the job description.","$type":"com.linkedin.pemberly.text.AttributedText"},"repostedJobPosting":null,"title":"Sustainability Data Analyst (f/m/x)","entityUrn":"urn:li:fs_normalized_jobPosting:3768592532","workRemoteAllowed":true,"applyMethod":{"companyApplyUrl":"https://join.com/companies/cozero/9872937-sustainability-data-analyst-f-m-x?pid=d7cb2e8e859c62cd7f75","applyStartersPreferenceVoid":false,"inPageOffsiteApply":false,"$type":"com.linkedin.voyager.jobs.OffsiteApply"},"talentHubJob":false,"formattedLocation":"Germany","workplaceTypes":["urn:li:fs_workplaceType:2"],"listedAt":1700948520000,"$recipeTypes":["com.linkedin.voyager.deco.jobs.web.shared.WebLightJobPosting"],"$type":"com.linkedin.voyager.jobs.JobPosting","standardizedAddresses":null,"jobPostingId":3768592532,"workplaceTypesResolutionResults":{"*urn:li:fs_workplaceType:2":"urn:li:fs_workplaceType:2"}},"meta":{"microSchema":{"version":"2","types":{"com.linkedin.voyager.deco.jobs.web.shared.WebLightJobPosting":{"fields":{"dashEntityUrn":{"type":"string"},"companyDetails":{"type":{"union":["com.linkedin.voyager.jobs.JobPostingCompanyName","com.linkedin.voyager.deco.jobs.web.shared.WebCompactJobPostingCompany"]}},"talentHubJob":{"type":"boolean"},"jobState":{"type":"string"},"formattedLocation":{"type":"string"},"description":{"type":"com.linkedin.pemberly.text.AttributedText"},"repostedJobPosting":{"type":"string"},"workplaceTypes":{"type":{"array":"string"}},"listedAt":{"type":"long"},"title":{"type":"string"},"standardizedAddresses":{"type":"com.linkedin.voyager.jobs.JobPostingAddresses"},"jobPostingId":{"type":"long"},"workRemoteAllowed":{"type":"boolean"},"entityUrn":{"type":"string"},"applyMethod":{"type":{"union":["com.linkedin.voyager.jobs.UnknownApply","com.linkedin.voyager.jobs.ComplexOnsiteApply","com.linkedin.voyager.jobs.SimpleOnsiteApply","com.linkedin.voyager.jobs.OffsiteApply"]}},"workplaceTypesResolutionResults":{"type":{"map":"com.linkedin.voyager.deco.jobs.shared.WebJobDetailsWorkplaceType"},"resolvedFrom":"workplaceTypes"}},"baseType":"com.linkedin.voyager.jobs.JobPosting"},"com.linkedin.voyager.deco.jobs.web.shared.WebCompactJobPostingCompany":{"fields":{"companyResolutionResult":{"type":"com.linkedin.voyager.deco.organization.shared.WebCompactCompany","resolvedFrom":"company"},"company":{"type":"string"}},"baseType":"com.linkedin.voyager.jobs.JobPostingCompany"},"com.linkedin.voyager.deco.jobs.shared.WebJobDetailsWorkplaceType":{"fields":{"entityUrn":{"type":"string"},"localizedName":{"type":"string"}},"baseType":"com.linkedin.voyager.jobs.shared.WorkplaceType"},"com.linkedin.voyager.deco.organization.shared.WebCompactCompany":{"fields":{"name":{"type":"string"},"logo":{"type":"com.linkedin.voyager.organization.CompanyLogoImage"},"universalName":{"type":"string"},"entityUrn":{"type":"string"},"url":{"type":"string"}},"baseType":"com.linkedin.voyager.organization.Company"}}}},"included":[{"localizedName":"Remote","entityUrn":"urn:li:fs_workplaceType:2","$recipeTypes":["com.linkedin.voyager.deco.jobs.shared.WebJobDetailsWorkplaceType"],"$type":"com.linkedin.voyager.jobs.shared.WorkplaceType"},{"$recipeTypes":["com.linkedin.voyager.deco.organization.shared.WebCompactCompany"],"url":"https://www.linkedin.com/company/cozero-io","$type":"com.linkedin.voyager.organization.Company","entityUrn":"urn:li:fs_normalized_company:43338549","name":"Cozero","logo":{"image":{"$type":"com.linkedin.common.VectorImage","rootUrl":"https://media.licdn.com/dms/image/C4D0BAQGkpvwIXWu1-g/company-logo_","artifacts":[{"width":200,"expiresAt":1709164800000,"fileIdentifyingUrlPathSegment":"200_200/0/1638439865071/cozero_io_logo?e=1709164800&amp;v=beta&amp;t=SkxNXYSV-cQiUJvKiBgKXlXVlM4i0GmvhuX8g6hGKZY","$type":"com.linkedin.common.VectorArtifact","height":200},{"width":100,"expiresAt":1709164800000,"fileIdentifyingUrlPathSegment":"100_100/0/1638439865071/cozero_io_logo?e=1709164800&amp;v=beta&amp;t=sVPH2ggrA8QHrpoTseVt0_N71x9Hg_xcONvZl48_Ogk","$type":"com.linkedin.common.VectorArtifact","height":100},{"width":400,"expiresAt":1709164800000,"fileIdentifyingUrlPathSegment":"400_400/0/1638439865071/cozero_io_logo?e=1709164800&amp;v=beta&amp;t=G2AlGC86FzMcCKbWIW1B4lPKCq9A1cBdlHhCGtKap9c","$type":"com.linkedin.common.VectorArtifact","height":400}]},"type":"SQUARE_LOGO","$type":"com.linkedin.voyager.organization.CompanyLogoImage"},"universalName":"cozero-io"}]}
</code>
<code style="display: none" id="datalet-bpr-guid-15912216">
  {"request":"/voyager/api/jobs/jobPostings/3768592532?decorationId\u003Dcom.linkedin.voyager.deco.jobs.web.shared.WebLightJobPosting-23\u0026","status":200,"body":"bpr-guid-15912216","method":"GET","headers":{"x-li-uuid":"AAYLE+vtCEZ6Amp7P3S1tA\u003D\u003D"}}
</code>
<img src="data:image/gif;base64,R0lGODlhAQABAIAAAAAAAP///yH5BAEAAAAALAAAAAABAAEAAAIBRAA7" style="display: none" class="datalet-bpr-guid-15912216"><code style="display: none" id="bpr-guid-15912217">
  {"data":{"data":{"$recipeTypes":["com.linkedin.8fbc3627c8305a33e3e6840ec7688312"],"jobsDashJobPostingDetailSectionsByCardSectionTypes":{"$recipeTypes":["com.linkedin.f1d8ff8a5c9d0a2c73d8c4418e3a4526"],"elements":[{"$recipeTypes":["com.linkedin.180bad7e2024ccff1ee0157be65c0dd1"],"jobPostingDetailSection":[{"similarJobsCard":null,"interviewPrepCard":null,"salaryCard":null,"companyInsightsCard":null,"*topCard":"urn:li:fsd_jobPostingCard:(3768592532,JOB_DETAILS)","bannerCard":null,"jobSeekerActionCard":null,"learningRecommendationCard":null,"hiringTeamCard":null,"similarJobsAtCompanyCard":null,"jobSegmentAttributesCard":null,"howYouMatchCard":null,"jobApplicantInsightsUrn":null,"jobActivityCard":null,"postApplyPromo":null,"jobDescription":null,"benefitsCard":null,"jobAlertCard":null,"marketplacePromoCard":null,"companyCardV2":null,"recommendedActionsCard":null}],"$type":"com.linkedin.voyager.dash.jobs.JobPostingDetailSection"}],"$type":"com.linkedin.restli.common.CollectionResponse"},"$type":"com.linkedin.8fbc3627c8305a33e3e6840ec7688312"},"extensions":{"webMetadata":{}}},"meta":{"microSchema":{"isGraphQL":true,"version":"2.1","types":{"com.linkedin.b1148482c3f256b956d443d3fbab2904":{"fields":{"nestedTextSelectableOptions":{"type":{"array":"com.linkedin.7bd2de9a7b9fadfe651a8dd1c13db7ad"}},"exclusiveSelectableOption":{"type":"com.linkedin.98d89bae4aa4ef8f3ffc1018ca93419c"},"selectionCountRangeValidation":{"type":"com.linkedin.e18c41f5008d005150ad33f812748af9"},"subtitle":{"type":"com.linkedin.390ac1c1431d5b4e340a65af24ed68f2"}},"baseType":"com.linkedin.voyager.dash.common.forms.NestedCheckboxFormComponent"},"com.linkedin.c8842299f35a2aecf1baf1f52de866bf":{"fields":{"textSelectableOption":{"type":"com.linkedin.98d89bae4aa4ef8f3ffc1018ca93419c"},"positiveToggleIcon":{"type":"com.linkedin.30e1ec6a491035bd7a31cc9de8a12c18"},"negativeToggleIcon":{"type":"com.linkedin.30e1ec6a491035bd7a31cc9de8a12c18"}},"baseType":"com.linkedin.voyager.dash.common.forms.TogglePill"},"com.linkedin.45e3a7da0d85a4cd9d5aaa94b34dd17d":{"fields":{"suggestedEntities":{"type":{"array":"com.linkedin.98d89bae4aa4ef8f3ffc1018ca93419c"}},"subtitle":{"type":"com.linkedin.390ac1c1431d5b4e340a65af24ed68f2"},"dismissSelectableOption":{"type":"com.linkedin.98d89bae4aa4ef8f3ffc1018ca93419c"},"dismissControlName":{"type":"com.linkedin.voyager.dash.common.tracking.ControlName"},"actionableSuggestedEntities":{"type":{"array":"com.linkedin.65862eebdeb12a456370f32142eebd8c"}},"navigationButton":{"type":"com.linkedin.72471cc7e55a55b0fd549127127633f7"},"title":{"type":"com.linkedin.390ac1c1431d5b4e340a65af24ed68f2"},"typeaheadFormSuggestionUseCase":{"type":"com.linkedin.voyager.dash.common.forms.TypeaheadFormSuggestionUseCase"},"trackingId":{"type":"com.linkedin.voyager.dash.common.Base64String"}},"baseType":"com.linkedin.voyager.dash.common.forms.TypeaheadFormSuggestionViewModel"},"com.linkedin.d0f6c4718d68ff125ca6f7e0b91f510c":{"fields":{},"baseType":"com.linkedin.voyager.dash.jobs.JobSearchHistory"},"com.linkedin.deaeaf135b51841f133d0ceae7eca621":{"fields":{"premiumStyle":{"type":"boolean"},"size":{"type":"com.linkedin.voyager.dash.common.ux.button.ButtonSize"},"icon":{"type":"com.linkedin.voyager.dash.common.SystemImageName"},"text":{"type":"string"},"iconAfterText":{"type":"boolean"},"emphasize":{"type":"boolean"},"category":{"type":"com.linkedin.voyager.dash.common.ux.button.ButtonCategory"}},"baseType":"com.linkedin.voyager.dash.common.ux.button.ButtonAppearance"},"com.linkedin.427074b29d6e63261a7a2174b0df610b":{"fields":{"groups":{"type":{"array":"com.linkedin.30951a1ee13ae9dec8fa329260e12fc2"}},"tip":{"type":"com.linkedin.00ac75ef8dae327ab29a045438e7d186"},"viewImpressionTrackingKey":{"type":"string"},"buttonCTA":{"type":"com.linkedin.5de46613f3027e5a41b356871519a78b"}},"baseType":"com.linkedin.voyager.dash.jobs.ItemsMatchSection"},"com.linkedin.b429b28d37054e5232ecd5ecffd09315":{"fields":{"geo":{"resolvedFrom":"geoUrn","type":"com.linkedin.00c4f529ff2fa302d89eaea66344f268"},"geoUrn":{"type":"string"},"postalCode":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.common.forms.LocationInputValue"},"com.linkedin.79cb580711dcd3cf7d83e724b3d9e260":{"fields":{"localizedName":{"type":"string"},"localizedDescription":{"type":"string"},"workplaceTypeEnum":{"type":"com.linkedin.voyager.dash.jobs.WorkplaceTypeEnum"},"entityUrn":{"type":"com.linkedin.voyager.dash.common.WorkplaceTypeUrn"},"preDashWorkplaceTypeUrn":{"type":"com.linkedin.voyager.dash.common.PreDashUrn"}},"baseType":"com.linkedin.voyager.dash.jobs.WorkplaceType"},"com.linkedin.1b210594d26c621d240d5cf0136a8bdd":{"fields":{"count":{"type":"int"},"seniority":{"resolvedFrom":"seniorityUrn","type":"com.linkedin.0f63072b23eab5f48ddb389e6f328afc"},"seniorityUrn":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.premium.insights.JobApplicantInsightsSeniorityDetail"},"com.linkedin.9e0491106da7f36c0672ee51875a86fa":{"fields":{"displayImageWithFrameReferenceUnion":{"type":{"union":{"url":"com.linkedin.common.Url","vectorImage":"com.linkedin.b8949adeaaa915212d0136c4b9a1952b"}}},"displayImageReferenceResolutionResult":{"derivedFrom":"displayImageReference","type":{"union":{"url":"com.linkedin.common.Url","vectorImage":"com.linkedin.b8949adeaaa915212d0136c4b9a1952b"}}},"displayImageWithFrameReference":{"derivedFrom":"displayImageWithFrameReferenceUnion","type":{"union":{"url":"com.linkedin.common.Url","vectorImage":"com.linkedin.b8949adeaaa915212d0136c4b9a1952b"}}},"displayImageReference":{"type":{"union":{"url":"com.linkedin.common.Url","vectorImage":"com.linkedin.b8949adeaaa915212d0136c4b9a1952b"}}},"displayImageUrn":{"type":"com.linkedin.common.Urn"}},"baseType":"com.linkedin.voyager.dash.identity.profile.PhotoFilterPicture"},"com.linkedin.653bbb9565e4a43fc63d44ebc167fab1":{"fields":{},"baseType":"com.linkedin.voyager.dash.jobs.SeeAllCard"},"com.linkedin.3551030d48734b4fd9cb3238efc77ece":{"fields":{"image":{"type":"com.linkedin.30e1ec6a491035bd7a31cc9de8a12c18"},"text":{"type":"com.linkedin.390ac1c1431d5b4e340a65af24ed68f2"}},"baseType":"com.linkedin.voyager.dash.common.ux.InsightViewModel"},"com.linkedin.aa4a2fb807b219a33654acc9643ff224":{"fields":{},"baseType":"com.linkedin.voyager.dash.jobs.home.RefreshStateCard"},"com.linkedin.9774b5e7450ba6db34185afe39f9e516":{"fields":{"pills":{"derivedFrom":"pillsUnion","type":{"union":{"dismissPills":{"array":"com.linkedin.c8f6142f1647b4b53caab19e4fe964ef"},"togglePills":{"array":"com.linkedin.c8842299f35a2aecf1baf1f52de866bf"}}}},"contextualSuggestionsTriggers":{"type":"com.linkedin.5481c9cf9c53bc965a2e49fdf5e17760"},"pillsUnion":{"type":{"union":{"dismissPills":{"array":"com.linkedin.c8f6142f1647b4b53caab19e4fe964ef"},"togglePills":{"array":"com.linkedin.c8842299f35a2aecf1baf1f52de866bf"}}}},"contextualSuggestionViewModel":{"type":"com.linkedin.45e3a7da0d85a4cd9d5aaa94b34dd17d"},"selectionCountRangeValidation":{"type":"com.linkedin.e18c41f5008d005150ad33f812748af9"},"typeaheadCta":{"type":"com.linkedin.c754100718c581ece9a715a9165db201"},"description":{"type":"com.linkedin.390ac1c1431d5b4e340a65af24ed68f2"},"togglePills":{"type":{"array":"com.linkedin.c8842299f35a2aecf1baf1f52de866bf"}},"pillSuggestionUseCase":{"type":"com.linkedin.voyager.dash.common.forms.TypeaheadFormSuggestionUseCase"},"trackingId":{"type":"com.linkedin.voyager.dash.common.Base64String"}},"baseType":"com.linkedin.voyager.dash.common.forms.PillFormComponent"},"com.linkedin.2f3c656d762dff10246b267bedddfcfd":{"fields":{"requiredFieldMissingErrorText":{"type":"com.linkedin.390ac1c1431d5b4e340a65af24ed68f2"},"responseRequired":{"type":"boolean"},"textField":{"type":"com.linkedin.5f82b47e5488aec975bf096d809fb051"},"labelText":{"type":"com.linkedin.390ac1c1431d5b4e340a65af24ed68f2"}},"baseType":"com.linkedin.voyager.dash.common.forms.SingleLineTextField"},"com.linkedin.d77e6926ccb788017c5e7ffe0261e4d7":{"fields":{"icon":{"type":"com.linkedin.voyager.dash.common.SystemImageName"},"questionPageUrl":{"type":"com.linkedin.common.Url"},"title":{"type":"string"},"questionText":{"type":"string"},"subtitle":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.jobs.InterviewPrepQuestionRecommendation"},"com.linkedin.b2b7b2441fdd2b88fd82124f6d8c4a1c":{"fields":{"alumni":{"type":{"array":"com.linkedin.3df29ea52bb4c2126ad5c1daa0ab5b01"}},"totalNumberOfAlumni":{"type":"int"}},"baseType":"com.linkedin.voyager.dash.premium.companyinsights.AlumniInsights"},"com.linkedin.45d6d21a26aa62742acf906ac42d8306":{"fields":{"jobsOpeningsGrowthUnselectedFunctions":{"type":{"array":"com.linkedin.f6e5193777aa8860860ecb48d0212cce"}},"jobOpeningsGrowthByFunction":{"type":{"array":"com.linkedin.cca42244a99771eb4b1bfffed219f853"}},"jobOpeningsGrowthAllFunctions":{"type":{"array":"com.linkedin.f6e5193777aa8860860ecb48d0212cce"}},"jobOpeningsByFunction":{"type":{"array":"com.linkedin.acd5ebb13f6e0c9da292f5d03c57eac4"}}},"baseType":"com.linkedin.voyager.dash.premium.companyinsights.JobOpeningsInsights"},"com.linkedin.b97672681af5b861a5ec5c53b8fcbc96":{"fields":{"name":{"type":"string"},"jobCompanyUnion":{"type":{"union":{"company":"string","rawCompanyName":"string"}}},"jobCompany":{"resolvedFrom":"jobCompanyUnion","type":{"union":{"company":"com.linkedin.a2ef5c1c8d1bd7b83de7009b834a38c6","rawCompanyName":"string"}}}},"baseType":"com.linkedin.voyager.dash.jobs.JobPostingCompany"},"com.linkedin.9196412bf6c5c0443810d690321ab8f0":{"fields":{"title":{"type":"string"},"description":{"type":"com.linkedin.390ac1c1431d5b4e340a65af24ed68f2"},"image":{"type":"com.linkedin.30e1ec6a491035bd7a31cc9de8a12c18"}},"baseType":"com.linkedin.voyager.dash.jobs.SafetyTipViewModel"},"com.linkedin.57812d345916f0da7785a8887e8fe73d":{"fields":{},"baseType":"com.linkedin.voyager.dash.jobs.CarouselAdvantageCard"},"com.linkedin.e53bf26a30b7d5901401fa6088b455dc":{"fields":{"encryptedBiddingParameters":{"type":"string"},"trackingCode":{"type":"string"},"navigationAction":{"type":"com.linkedin.5de46613f3027e5a41b356871519a78b"},"referenceId":{"type":"com.linkedin.voyager.dash.common.Base64String"},"trackingId":{"type":"com.linkedin.voyager.dash.common.Base64String"}},"baseType":"com.linkedin.voyager.dash.jobs.JobTrackingData"},"com.linkedin.efd2ef2c46b7959f85a3d6461edc3e69":{"fields":{"image":{"type":"com.linkedin.30e1ec6a491035bd7a31cc9de8a12c18"},"descriptionUnions":{"type":{"array":{"union":{"text":"com.linkedin.390ac1c1431d5b4e340a65af24ed68f2","label":"com.linkedin.4a3588cb3f298939a6ef5b2e61cadc18"}}}},"description":{"derivedFrom":"descriptionUnions","type":{"array":{"union":{"text":"com.linkedin.390ac1c1431d5b4e340a65af24ed68f2","label":"com.linkedin.4a3588cb3f298939a6ef5b2e61cadc18"}}}}},"baseType":"com.linkedin.voyager.dash.jobs.JobInsightViewModel"},"com.linkedin.00ac75ef8dae327ab29a045438e7d186":{"fields":{"navigationAction":{"type":"com.linkedin.5de46613f3027e5a41b356871519a78b"},"image":{"type":"com.linkedin.voyager.dash.common.SystemImageName"},"content":{"type":"com.linkedin.390ac1c1431d5b4e340a65af24ed68f2"}},"baseType":"com.linkedin.voyager.dash.jobs.HowYouMatchTip"},"com.linkedin.91ef6d45f2b779a5fad235f8f9dccdb0":{"fields":{"composeNavigationContext":{"type":"com.linkedin.b69dade258a78e45f8205cb49b6a98b9"},"icon":{"type":"com.linkedin.30e1ec6a491035bd7a31cc9de8a12c18"},"displayText":{"type":"com.linkedin.390ac1c1431d5b4e340a65af24ed68f2"},"textStartIcon":{"type":"com.linkedin.30e1ec6a491035bd7a31cc9de8a12c18"},"composeOptionType":{"type":"com.linkedin.voyager.dash.messaging.compose.ComposeOptionType"},"entityUrn":{"type":"com.linkedin.voyager.dash.common.ComposeOptionUrn"}},"baseType":"com.linkedin.voyager.dash.messaging.compose.ComposeOption"},"com.linkedin.00c4f529ff2fa302d89eaea66344f268":{"fields":{"countryUrn":{"type":"string"},"country":{"resolvedFrom":"countryUrn","type":"com.linkedin.00c4f529ff2fa302d89eaea66344f268"},"localizedName":{"type":"string"},"defaultLocalizedNameWithoutCountryName":{"type":"string"},"entityUrn":{"type":"com.linkedin.voyager.dash.common.GeoUrn"},"latitude":{"type":"float"},"countryISOCode":{"type":"string"},"abbreviatedLocalizedName":{"type":"string"},"defaultLocalizedName":{"type":"string"},"longitude":{"type":"float"}},"baseType":"com.linkedin.voyager.dash.common.Geo"},"com.linkedin.72617837b227d77ce48feb965f1ecc1b":{"fields":{"activityType":{"type":"com.linkedin.voyager.dash.jobs.JobActivityType"},"description":{"type":"string"},"formattedTimeAgo":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.jobs.JobActivity"},"com.linkedin.6143491c72aa6f2fce542169c8baa1d6":{"fields":{"cta":{"type":"string"},"formType":{"type":"com.linkedin.voyager.dash.jobs.JobSeekerFormActionCardType"},"ctaControlName":{"type":"com.linkedin.voyager.dash.common.tracking.ControlName"},"subtitle":{"type":"com.linkedin.390ac1c1431d5b4e340a65af24ed68f2"},"formSection":{"type":"com.linkedin.55c2879f1a800fb9b17e54740f965515"},"legoTrackingToken":{"type":"string"},"title":{"type":"com.linkedin.390ac1c1431d5b4e340a65af24ed68f2"}},"baseType":"com.linkedin.voyager.dash.jobs.JobSeekerFormActionCard"},"com.linkedin.c2408eaefddaf66bea26cb69c95979d7":{"fields":{"utype":{"type":"string"},"referenceId":{"type":"com.linkedin.common.TrackingId"}},"baseType":"com.linkedin.voyager.dash.premium.PremiumFunnelCommonHeader"},"com.linkedin.90840606293e76f91e739cc9fbd0c304":{"fields":{"textSelectableOptions":{"type":{"array":"com.linkedin.98d89bae4aa4ef8f3ffc1018ca93419c"}},"exclusiveSelectableOption":{"type":"com.linkedin.98d89bae4aa4ef8f3ffc1018ca93419c"},"selectionCountRangeValidation":{"type":"com.linkedin.e18c41f5008d005150ad33f812748af9"},"subtitle":{"type":"com.linkedin.390ac1c1431d5b4e340a65af24ed68f2"}},"baseType":"com.linkedin.voyager.dash.common.forms.CheckboxFormComponent"},"com.linkedin.594a52f4e72625c921bc156c950ef9e3":{"fields":{"formElementInputs":{"type":{"array":"com.linkedin.81235c3224517a02609a065763927f01"}}},"baseType":"com.linkedin.voyager.dash.common.forms.PrerequisiteFormInputs"},"com.linkedin.181ad18bcf938cd6016ba9ffd2137150":{"fields":{"addButtonText":{"type":"string"},"removeButtonText":{"type":"string"},"selectionCountRangeValidation":{"type":"com.linkedin.e18c41f5008d005150ad33f812748af9"},"repeatableFormElementGroups":{"type":{"array":"com.linkedin.6a325d9bf5f44baf8fede1db5a7ce66f"}},"addButtonControlName":{"type":"com.linkedin.voyager.dash.common.tracking.ControlName"},"title":{"type":"string"},"removeButtonControlName":{"type":"com.linkedin.voyager.dash.common.tracking.ControlName"}},"baseType":"com.linkedin.voyager.dash.common.forms.RepeatableSectionData"},"com.linkedin.a2ef5c1c8d1bd7b83de7009b834a38c6":{"fields":{"industryV2Taxonomy":{"resolvedFrom":"industryV2TaxonomyUrns","type":{"array":"com.linkedin.7a7f0ab9f40a57ecd2a27e26059721fe"}},"description":{"type":"string"},"employeeCount":{"type":"long"},"url":{"type":"com.linkedin.common.Url"},"employeeCountRange":{"type":"com.linkedin.6148d310783eaffead8570485d05a156"},"industryV2TaxonomyUrns":{"type":{"array":"string"}},"followingStateUrn":{"type":"string"},"entityUrn":{"type":"com.linkedin.voyager.dash.common.CompanyUrn"},"followingState":{"resolvedFrom":"followingStateUrn","type":"com.linkedin.756c19defef3189024bfe8e3e74d83ab"},"name":{"type":"string"},"logo":{"type":{"union":{"url":"com.linkedin.common.Url","vectorImage":"com.linkedin.b8949adeaaa915212d0136c4b9a1952b"}}},"logoResolutionResult":{"derivedFrom":"logo","type":{"union":{"url":"com.linkedin.common.Url","vectorImage":"com.linkedin.b8949adeaaa915212d0136c4b9a1952b"}}},"universalName":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.organization.Company"},"com.linkedin.c4e907da1ff053ca48ebe32c32f78805":{"fields":{"entityUrn":{"type":"com.linkedin.voyager.dash.common.PromotionPlacementUrn"}},"baseType":"com.linkedin.voyager.dash.jobs.PromotionalCard"},"com.linkedin.7f23d6d0556f01f090158424e75ff2ab":{"fields":{"totalHires":{"type":"int"},"seniorHires":{"type":{"array":"com.linkedin.d09aeb85fdf4991b6b063ae75ef6ef54"}},"hireCounts":{"type":{"array":"com.linkedin.2815a36a9c66a3c907e5474fae6a1903"}},"totalNumberOfSeniorHires":{"type":"int"}},"baseType":"com.linkedin.voyager.dash.premium.companyinsights.HiresInsights"},"com.linkedin.94721090e597af7c97503cb8b1a7e50f":{"fields":{"type":{"type":"com.linkedin.voyager.dash.common.text.EpochFormat"},"epochAt":{"type":"com.linkedin.common.Time"}},"baseType":"com.linkedin.voyager.dash.common.text.EpochTime"},"com.linkedin.48407601ffa8bd83b8651487ea606df9":{"fields":{"errorText":{"type":"com.linkedin.390ac1c1431d5b4e340a65af24ed68f2"},"validDateRange":{"type":"com.linkedin.8fafa573919c2bc96c29a2be79ce3fb3"}},"baseType":"com.linkedin.voyager.dash.common.forms.validation.DateValidation"},"com.linkedin.0a38c5f17cb738fcfee67c772442d536":{"fields":{"screeningQuestions":{"type":{"array":"com.linkedin.55c2879f1a800fb9b17e54740f965515"}},"entityUrn":{"type":"com.linkedin.voyager.dash.common.ScreeningSurveyFormUrn"},"markedAsSeen":{"type":"boolean"}},"baseType":"com.linkedin.voyager.dash.jobs.assessments.ScreeningSurveyForm"},"com.linkedin.e5b59282f574d2493aeb20a65a9a1f94":{"fields":{"startedOn":{"type":"com.linkedin.fefbb3ce7d8629ad1cfd497514eb598e"},"employeeCount":{"type":"int"}},"baseType":"com.linkedin.voyager.dash.premium.companyinsights.CompanyInsightsEmployeeGrowthDetail"},"com.linkedin.b7cc21947d4a1ba974040324a832ba64":{"fields":{"employeeCount":{"type":"int"},"organizationUrn":{"type":"string"},"viewersOrganization":{"type":"boolean"},"organization":{"resolvedFrom":"organizationUrn","type":"com.linkedin.a2ef5c1c8d1bd7b83de7009b834a38c6"}},"baseType":"com.linkedin.voyager.dash.premium.companyinsights.CompanyInsightsOrganizationRankingDetail"},"com.linkedin.e94906839b912dcb4829422361b5028c":{"fields":{"ringContentType":{"type":"com.linkedin.voyager.dash.common.image.RingContentType"},"actionTarget":{"type":"com.linkedin.common.Url"},"preDashEntityUrn":{"type":"com.linkedin.voyager.dash.common.PreDashUrn"},"entityUrn":{"type":"com.linkedin.voyager.dash.common.RingStatusUrn"},"emphasized":{"type":"boolean"}},"baseType":"com.linkedin.voyager.dash.common.image.RingStatus"},"com.linkedin.2791a0d398e611fad8c820c60183e2bb":{"fields":{"recommendations":{"type":{"array":"com.linkedin.939138a9c6796d21af12eaf891a20e80"}},"branding":{"type":"com.linkedin.voyager.dash.learning.jobDetails.Branding"}},"baseType":"com.linkedin.voyager.dash.learning.LearningRecommendationCard"},"com.linkedin.ce28f52e436b3cdd8100675b5288a600":{"fields":{"controlName":{"type":"com.linkedin.voyager.dash.common.tracking.ControlName"},"viewingBehavior":{"type":"com.linkedin.voyager.dash.common.ux.UrlViewingBehavior"},"sponsoredUrlAttributes":{"type":"com.linkedin.7210b8f140dcf756fa9cea58d6413a4b"},"url":{"type":"com.linkedin.common.Url"}},"baseType":"com.linkedin.voyager.dash.common.text.TextLink"},"com.linkedin.0da643628377886b9fb1a0bb0cc0b73c":{"fields":{"hasConfirmedEmailAddress":{"type":"boolean"},"applyStartersPreferenceVoid":{"type":"boolean"},"applied":{"type":"boolean"},"jobDraftApplicationInfo":{"type":"com.linkedin.ff0a7ae4688d8d97ea2151c00e6e8392"},"formattedApplyDate":{"type":"string"},"onsiteApply":{"type":"boolean"},"applyCtaText":{"type":"com.linkedin.390ac1c1431d5b4e340a65af24ed68f2"},"entityUrn":{"type":"com.linkedin.voyager.dash.common.JobSeekerApplicationDetailUrn"},"inPageOffsiteApply":{"type":"boolean"},"applicantTrackingSystemName":{"type":"string"},"preApplySafetyTips":{"type":{"array":"com.linkedin.9196412bf6c5c0443810d690321ab8f0"}},"ambryResumeContainer":{"type":"string"},"companyApplyUrl":{"type":"com.linkedin.common.Url"},"appliedAt":{"type":"com.linkedin.common.Time"}},"baseType":"com.linkedin.voyager.dash.jobs.JobSeekerApplicationDetail"},"com.linkedin.c734e1c934a3cbf9562953df54d4e771":{"fields":{"startDateText":{"type":"com.linkedin.390ac1c1431d5b4e340a65af24ed68f2"},"startDateControlName":{"type":"com.linkedin.voyager.dash.common.tracking.ControlName"},"endDateControlName":{"type":"com.linkedin.voyager.dash.common.tracking.ControlName"},"ongoingDateRangeFormElementUrn":{"type":"com.linkedin.common.Urn"},"endDateValidation":{"type":"com.linkedin.48407601ffa8bd83b8651487ea606df9"},"endDateText":{"type":"com.linkedin.390ac1c1431d5b4e340a65af24ed68f2"},"startDateValidation":{"type":"com.linkedin.48407601ffa8bd83b8651487ea606df9"},"ongoingDateText":{"type":"com.linkedin.390ac1c1431d5b4e340a65af24ed68f2"},"dateInputType":{"type":"com.linkedin.voyager.dash.common.forms.DateInputType"}},"baseType":"com.linkedin.voyager.dash.common.forms.DateRangeFormComponent"},"com.linkedin.274132557f0c6d4e240f7372d489e1cc":{"fields":{"controlName":{"type":"string"},"numFractionalDigits":{"type":"int"},"hintText":{"type":"string"},"helperText":{"type":"string"},"numericRangeValidation":{"type":"com.linkedin.9ddc780810d0c93fa698a26faab52f10"}},"baseType":"com.linkedin.voyager.dash.common.forms.NumberInputFormComponent"},"com.linkedin.1b53ff96dc1e6cc3644919ffccaddf68":{"fields":{"applicantCount":{"type":"long"},"skillDetails":{"type":{"array":"com.linkedin.f3f2514a55c0f5153f3bd99b35ea8e2b"}},"degreeDetails":{"type":{"array":"com.linkedin.eef1cf048228a183463b81d9c6790e5f"}},"seniorityDetails":{"type":{"array":"com.linkedin.1b210594d26c621d240d5cf0136a8bdd"}},"applicantsInPastDay":{"type":"long"},"entityUrn":{"type":"com.linkedin.voyager.dash.common.JobApplicantInsightsUrn"},"applicantRankPercentile":{"type":"int"},"locationDetails":{"type":{"array":"com.linkedin.2c280c86f261412efd5c2e0d60bd88d4"}},"applicantThresholdMet":{"type":"boolean"}},"baseType":"com.linkedin.voyager.dash.premium.insights.JobApplicantInsights"},"com.linkedin.c4a89ad017a52048790e89ac5ba1ab35":{"fields":{"viewImpressionTrackingKey":{"type":"string"},"title":{"type":"string"},"sections":{"type":{"array":"com.linkedin.2d03db73fb8c8c66e4245b79a3d25e52"}},"subtitle":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.jobs.JobSegmentAttributesCard"},"com.linkedin.b49a045df66f28b2a65f3df75d119eae":{"fields":{"icon":{"type":"com.linkedin.voyager.dash.common.ArtDecoIconName"},"iconBeforeCtaText":{"type":"boolean"}},"baseType":"com.linkedin.voyager.dash.jobs.CtaIcon"},"com.linkedin.4fb8cecee08f0c588138694ff045dfd9":{"fields":{"latestHeadcountByFunction":{"type":"com.linkedin.acd5ebb13f6e0c9da292f5d03c57eac4"},"headcountGrowthByFunction":{"type":{"array":"com.linkedin.cca42244a99771eb4b1bfffed219f853"}}},"baseType":"com.linkedin.voyager.dash.premium.companyinsights.FunctionHeadcountInsights"},"com.linkedin.32bab5d7af29a8bdf4fe920d7d2155b1":{"fields":{"name":{"type":"string"},"logo":{"type":{"union":{"url":"com.linkedin.common.Url","vectorImage":"com.linkedin.b8949adeaaa915212d0136c4b9a1952b"}}},"logoResolutionResult":{"derivedFrom":"logo","type":{"union":{"url":"com.linkedin.common.Url","vectorImage":"com.linkedin.b8949adeaaa915212d0136c4b9a1952b"}}},"entityUrn":{"type":"com.linkedin.voyager.dash.common.GroupUrn"}},"baseType":"com.linkedin.voyager.dash.groups.Group"},"com.linkedin.c754100718c581ece9a715a9165db201":{"fields":{"controlName":{"type":"com.linkedin.voyager.dash.common.tracking.ControlName"},"renderingStyle":{"type":"com.linkedin.voyager.dash.common.forms.TypeaheadCtaRenderingStyle"},"typeaheadType":{"type":"com.linkedin.voyager.dash.search.TypeaheadType"},"typeaheadQueryContext":{"type":{"array":"string"}},"typeaheadMetadata":{"type":"com.linkedin.2dea04d4082c6960578ac30a3ace5ee0"},"labelText":{"type":"com.linkedin.390ac1c1431d5b4e340a65af24ed68f2"}},"baseType":"com.linkedin.voyager.dash.common.forms.TypeaheadCta"},"com.linkedin.8409c9296c5bda7be8aee44feff66dd3":{"fields":{"numericValueRangeValidation":{"type":"com.linkedin.9ddc780810d0c93fa698a26faab52f10"}},"baseType":"com.linkedin.voyager.dash.common.forms.validation.NumericValidationMetadata"},"com.linkedin.43d5fcbafc4556d0c474dabd56f017a2":{"fields":{"controlName":{"type":"com.linkedin.voyager.dash.common.tracking.ControlName"},"appearance":{"type":"com.linkedin.deaeaf135b51841f133d0ceae7eca621"},"openExternally":{"type":"boolean"},"accessibilityText":{"type":"string"},"disable":{"type":"boolean"},"url":{"type":"com.linkedin.common.Url"}},"baseType":"com.linkedin.voyager.dash.premium.PremiumButton"},"com.linkedin.8d64ead1eeea51a297dc70da47337de1":{"fields":{"width":{"type":"int"},"fileIdentifyingUrlPathSegment":{"type":"string"},"expiresAt":{"type":"com.linkedin.common.Time"},"height":{"type":"int"}},"baseType":"com.linkedin.common.VectorArtifact"},"com.linkedin.7210b8f140dcf756fa9cea58d6413a4b":{"fields":{"impressionId":{"type":"string"},"unwrappedUrlDomain":{"type":"string"},"creativeId":{"type":"string"},"campaignId":{"type":"string"},"memberId":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.common.sponsored.SponsoredUrlAttributes"},"com.linkedin.b0ac2d5e2d9d20cb81b800352f82dc8e":{"fields":{"icon":{"type":"com.linkedin.voyager.dash.common.ArtDecoIconName"},"controlName":{"type":"com.linkedin.voyager.dash.common.tracking.ControlName"},"positiveToggleLabel":{"type":"com.linkedin.390ac1c1431d5b4e340a65af24ed68f2"},"negativeToggleLabel":{"type":"com.linkedin.390ac1c1431d5b4e340a65af24ed68f2"}},"baseType":"com.linkedin.voyager.dash.common.forms.ToggleFormComponent"},"com.linkedin.64aff242d842d2f5c0d6ea1432ceb79a":{"fields":{"applyJobActionResolutionResult":{"resolvedFrom":"applyJobAction","type":"com.linkedin.0da643628377886b9fb1a0bb0cc0b73c"},"applyControlName":{"type":"com.linkedin.voyager.dash.common.tracking.ControlName"},"applyJobAction":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.jobs.ApplyJobAction"},"com.linkedin.f3f2514a55c0f5153f3bd99b35ea8e2b":{"fields":{"count":{"type":"int"},"skillUrn":{"type":"string"},"skill":{"resolvedFrom":"skillUrn","type":"com.linkedin.7a3dfe06b5ebab0041d8cea76d58565a"},"viewersHasSkill":{"type":"boolean"}},"baseType":"com.linkedin.voyager.dash.premium.insights.JobApplicantInsightsSkillDetail"},"com.linkedin.6e88a2e94acf8ea5aa8c9f7051705709":{"fields":{},"baseType":"com.linkedin.voyager.dash.jobs.search.NewCollectionHeaderCard"},"com.linkedin.c9156c9beebda6b437868e48c22a8f02":{"fields":{"ringStatus":{"type":"com.linkedin.e94906839b912dcb4829422361b5028c"},"profileUrn":{"type":"string"},"vectorImage":{"type":"com.linkedin.b8949adeaaa915212d0136c4b9a1952b"},"profile":{"resolvedFrom":"profileUrn","type":"com.linkedin.826b8c485ed1073773697502413b9597"}},"baseType":"com.linkedin.voyager.dash.common.image.NonEntityProfilePicture"},"com.linkedin.036f42f6cc87d5881cf2351020672e97":{"fields":{"insightViewModel":{"type":"com.linkedin.3551030d48734b4fd9cb3238efc77ece"},"responseGroup":{"type":"com.linkedin.voyager.dash.jobs.ResponsivenessActionGroup"},"learnMoreControlName":{"type":"com.linkedin.voyager.dash.common.tracking.ControlName"}},"baseType":"com.linkedin.voyager.dash.jobs.ResponsivenessInsightViewModel"},"com.linkedin.8fafa573919c2bc96c29a2be79ce3fb3":{"fields":{"start":{"type":"com.linkedin.fefbb3ce7d8629ad1cfd497514eb598e"},"end":{"type":"com.linkedin.fefbb3ce7d8629ad1cfd497514eb598e"}},"baseType":"com.linkedin.common.DateRange"},"com.linkedin.7a3dfe06b5ebab0041d8cea76d58565a":{"fields":{"name":{"type":"string"},"entityUrn":{"type":"com.linkedin.voyager.dash.common.StandardizedSkillUrn"}},"baseType":"com.linkedin.voyager.dash.jobs.StandardizedSkill"},"com.linkedin.194482b00bd3b5ae56a4d81712d0525c":{"fields":{"inputEntityName":{"type":"string"},"inputEntityUrn":{"type":"com.linkedin.common.Urn"}},"baseType":"com.linkedin.voyager.dash.common.forms.EntityInputValue"},"com.linkedin.3df29ea52bb4c2126ad5c1daa0ab5b01":{"fields":{"entityLockup":{"type":"com.linkedin.bd66bae736eabe6505d1b80c182e31af"},"exitedPosition":{"type":"com.linkedin.390ac1c1431d5b4e340a65af24ed68f2"},"exitYearMonthOn":{"type":"com.linkedin.fefbb3ce7d8629ad1cfd497514eb598e"}},"baseType":"com.linkedin.voyager.dash.premium.companyinsights.AlumniItem"},"com.linkedin.e1e02f8665f14f794053a412332f146a":{"fields":{"action":{"type":"com.linkedin.5de46613f3027e5a41b356871519a78b"},"description":{"type":"com.linkedin.390ac1c1431d5b4e340a65af24ed68f2"}},"baseType":"com.linkedin.voyager.dash.jobs.JobQualificationDescription"},"com.linkedin.ca1fb70631b137b6d459efdae8a18bc0":{"fields":{"image":{"derivedFrom":"imageUnion","type":{"union":{"url":"com.linkedin.common.Url","vectorImage":"com.linkedin.b8949adeaaa915212d0136c4b9a1952b"}}},"nameSupplementaryInfo":{"type":"com.linkedin.390ac1c1431d5b4e340a65af24ed68f2"},"footerText":{"type":"com.linkedin.390ac1c1431d5b4e340a65af24ed68f2"},"imageUnion":{"type":{"union":{"url":"com.linkedin.common.Url","vectorImage":"com.linkedin.b8949adeaaa915212d0136c4b9a1952b"}}},"backgroundImage":{"derivedFrom":"backgroundImageUnion","type":{"union":{"url":"com.linkedin.common.Url","vectorImage":"com.linkedin.b8949adeaaa915212d0136c4b9a1952b"}}},"subHeadline":{"type":"com.linkedin.390ac1c1431d5b4e340a65af24ed68f2"},"name":{"type":"com.linkedin.390ac1c1431d5b4e340a65af24ed68f2"},"insightText":{"type":"com.linkedin.390ac1c1431d5b4e340a65af24ed68f2"},"headline":{"type":"com.linkedin.390ac1c1431d5b4e340a65af24ed68f2"},"backgroundImageUnion":{"type":{"union":{"url":"com.linkedin.common.Url","vectorImage":"com.linkedin.b8949adeaaa915212d0136c4b9a1952b"}}}},"baseType":"com.linkedin.voyager.dash.common.media.StickerLinkLargeTemplateView"},"com.linkedin.7948e5031442b35b939b7672eac8bad0":{"fields":{"entityUrn":{"type":"com.linkedin.voyager.dash.common.HashtagUrn"},"trackingUrn":{"type":"com.linkedin.common.Urn"},"actionTarget":{"type":"com.linkedin.common.Url"}},"baseType":"com.linkedin.voyager.dash.feed.Hashtag"},"com.linkedin.cca42244a99771eb4b1bfffed219f853":{"fields":{"functionUrn":{"type":"string"},"growthPeriods":{"type":{"array":"com.linkedin.f6e5193777aa8860860ecb48d0212cce"}},"function":{"resolvedFrom":"functionUrn","type":"com.linkedin.39c50e73d5d652d9f5f7194f5023e967"}},"baseType":"com.linkedin.voyager.dash.premium.companyinsights.GrowthByFunction"},"com.linkedin.5481c9cf9c53bc965a2e49fdf5e17760":{"fields":{"contextualSuggestionDependent":{"resolvedFrom":"contextualSuggestionDependentUrns","type":{"array":"com.linkedin.258b7479f7c0999032ce22032fa02cdf"}},"contextualSuggestionQueryParameterUrns":{"type":{"array":"string"}},"contextualSuggestionDependentUrns":{"type":{"array":"string"}},"contextualSuggestionQueryParameter":{"resolvedFrom":"contextualSuggestionQueryParameterUrns","type":{"array":"com.linkedin.258b7479f7c0999032ce22032fa02cdf"}},"contextualSuggestionType":{"type":"com.linkedin.voyager.dash.search.TypeaheadType"}},"baseType":"com.linkedin.voyager.dash.common.forms.ContextualSuggestionsTriggers"},"com.linkedin.2c280c86f261412efd5c2e0d60bd88d4":{"fields":{"geo":{"resolvedFrom":"geoUrn","type":"com.linkedin.00c4f529ff2fa302d89eaea66344f268"},"numApplicants":{"type":"string"},"geoUrn":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.premium.insights.JobApplicantInsightsLocationDetail"},"com.linkedin.22e0be7fe535e4176ecde39607391326":{"fields":{"controlName":{"type":"com.linkedin.voyager.dash.common.tracking.ControlName"},"hintText":{"type":"com.linkedin.390ac1c1431d5b4e340a65af24ed68f2"},"numVisibleLines":{"type":"int"},"helperText":{"type":"com.linkedin.390ac1c1431d5b4e340a65af24ed68f2"},"validationMetadataResolutionResult":{"derivedFrom":"validationMetadata","type":{"union":{"pronounsV2":"com.linkedin.bfd20979d76e9c8ce942bc03238c6b02","string":"com.linkedin.bfd20979d76e9c8ce942bc03238c6b02","textFormInputType":"com.linkedin.voyager.dash.common.forms.validation.TextFormInputType","postalCode":"com.linkedin.2b6a63a1bb3f7b0bf5c50d8530db9f78","name":"com.linkedin.bfd20979d76e9c8ce942bc03238c6b02","pronouns":"string","integer":"com.linkedin.8409c9296c5bda7be8aee44feff66dd3","decimal":"com.linkedin.8409c9296c5bda7be8aee44feff66dd3","headline":"com.linkedin.bfd20979d76e9c8ce942bc03238c6b02","url":"com.linkedin.48be3e5c4d0dfb61ba0a4a296e719909"}}},"validationMetadata":{"type":{"union":{"pronounsV2":"com.linkedin.bfd20979d76e9c8ce942bc03238c6b02","string":"com.linkedin.bfd20979d76e9c8ce942bc03238c6b02","textFormInputType":"com.linkedin.voyager.dash.common.forms.validation.TextFormInputType","postalCode":"com.linkedin.2b6a63a1bb3f7b0bf5c50d8530db9f78","name":"com.linkedin.bfd20979d76e9c8ce942bc03238c6b02","pronouns":"string","integer":"com.linkedin.8409c9296c5bda7be8aee44feff66dd3","decimal":"com.linkedin.8409c9296c5bda7be8aee44feff66dd3","headline":"com.linkedin.bfd20979d76e9c8ce942bc03238c6b02","url":"com.linkedin.48be3e5c4d0dfb61ba0a4a296e719909"}}}},"baseType":"com.linkedin.voyager.dash.common.forms.MultilineTextFormComponent"},"com.linkedin.8b1684dd351b279ed40c6661257d0e63":{"fields":{},"baseType":"com.linkedin.restli.common.EmptyRecord"},"com.linkedin.23ea16078c6de6222c73b55c40e8e8ab":{"fields":{"linkUrl":{"type":"com.linkedin.common.Url"},"controlName":{"type":"com.linkedin.voyager.dash.common.tracking.ControlName"},"linkText":{"type":"string"},"text":{"type":"string"},"type":{"type":"com.linkedin.voyager.dash.common.ux.InlineFeedbackType"}},"baseType":"com.linkedin.voyager.dash.common.ux.InlineFeedbackViewModel"},"com.linkedin.dd03bc4de44734367e478c7c8eed60a0":{"fields":{"title":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.jobs.JobsFeedModuleHeader"},"com.linkedin.b7e88af0a8aa84677bc8effdf13ab5d9":{"fields":{"countryUrnRequired":{"type":"boolean"},"locationField":{"type":"com.linkedin.fe3639bb2accd8987c7e8ed7912c8eff"},"requiredFieldMissingErrorText":{"type":"com.linkedin.390ac1c1431d5b4e340a65af24ed68f2"},"responseRequired":{"type":"boolean"},"labelText":{"type":"com.linkedin.390ac1c1431d5b4e340a65af24ed68f2"}},"baseType":"com.linkedin.voyager.dash.common.forms.LocationTypeaheadEntityField"},"com.linkedin.bfb56f758ba39329fbb4081cfe291d5e":{"fields":{"controlName":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.common.forms.StarRatingFormComponent"},"com.linkedin.98d89bae4aa4ef8f3ffc1018ca93419c":{"fields":{"optionUnion":{"type":{"union":{"optionUrn":"com.linkedin.common.Urn","optionEnumString":"string"}}},"controlName":{"type":"com.linkedin.voyager.dash.common.tracking.ControlName"},"optionUrn":{"type":"com.linkedin.common.Urn"},"optionEnumString":{"type":"string"},"optionText":{"type":"com.linkedin.390ac1c1431d5b4e340a65af24ed68f2"},"option":{"derivedFrom":"optionUnion","type":{"union":{"optionUrn":"com.linkedin.common.Urn","optionEnumString":"string"}}}},"baseType":"com.linkedin.voyager.dash.common.forms.TextSelectableOption"},"com.linkedin.f6e5193777aa8860860ecb48d0212cce":{"fields":{"changePercentage":{"type":"int"},"monthDifference":{"type":"int"}},"baseType":"com.linkedin.voyager.dash.premium.companyinsights.GrowthPeriod"},"com.linkedin.d2acc160e7db812cd2fc11b5eaac7b0f":{"fields":{"qualification":{"derivedFrom":"qualificationUnion","type":{"union":{"qualificationDescription":"com.linkedin.e1e02f8665f14f794053a412332f146a","qualificationList":"com.linkedin.f26ce46d6c12df14239f36d844033955"}}},"header":{"type":"string"},"qualificationUnion":{"type":{"union":{"qualificationDescription":"com.linkedin.e1e02f8665f14f794053a412332f146a","qualificationList":"com.linkedin.f26ce46d6c12df14239f36d844033955"}}},"subheader":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.jobs.JobQualificationGroup"},"com.linkedin.7269fdb3a0f2f75ea98f76d3d3c9dfc3":{"fields":{"jobPostingRelevanceReasons":{"type":{"array":"com.linkedin.f4f99771435399029308495de4e742b6"}},"employeeGrowthForFunction":{"type":"com.linkedin.e276a8a343fc735c387871abc9190c7d"},"formattedJobFunction":{"type":"string"},"granularity":{"type":"com.linkedin.voyager.dash.premium.companyinsights.CompanyInsightsGranularity"},"employeeGrowth":{"type":"com.linkedin.e276a8a343fc735c387871abc9190c7d"},"company":{"resolvedFrom":"companyUrn","type":"com.linkedin.a2ef5c1c8d1bd7b83de7009b834a38c6"},"companyRanking":{"type":"com.linkedin.34436f930667efbac1e7e0a600c8c653"},"companyUrn":{"type":"string"},"schoolRanking":{"type":"com.linkedin.34436f930667efbac1e7e0a600c8c653"},"talentSourcesTitle":{"type":"com.linkedin.390ac1c1431d5b4e340a65af24ed68f2"}},"baseType":"com.linkedin.voyager.dash.premium.companyinsights.JobPostingCompanyInsights"},"com.linkedin.d85acf0e41970312e8912b9455cefa24":{"fields":{"dismissed":{"type":"boolean"},"entityUrn":{"type":"com.linkedin.voyager.dash.common.JobPostingRelevanceFeedbackUrn"},"dismissedEntityRelevanceFeedbackUrn":{"type":"string"},"dismissedEntityRelevanceFeedback":{"resolvedFrom":"dismissedEntityRelevanceFeedbackUrn","type":"com.linkedin.d8e085065c27e5a4b4a3b47e39d34eae"}},"baseType":"com.linkedin.voyager.dash.jobs.feedback.JobPostingRelevanceFeedback"},"com.linkedin.a7d235a42f3653f9539026976306f11b":{"fields":{"school":{"resolvedFrom":"schoolUrn","type":"com.linkedin.d4dcbabbf47d9675b0597099d9cd7d69"},"vectorImage":{"type":"com.linkedin.b8949adeaaa915212d0136c4b9a1952b"},"schoolUrn":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.common.image.NonEntitySchoolLogo"},"com.linkedin.bb47d92b56e85e0625cd6b7f5d3d0384":{"fields":{"entityUrn":{"type":"com.linkedin.voyager.dash.common.ConversationUrn"}},"baseType":"com.linkedin.voyager.dash.messaging.Conversation"},"com.linkedin.6c333b78b6aeac833ef88e8174932712":{"fields":{"type":{"type":"com.linkedin.voyager.dash.jobs.JobPostingCardFooterItemType"},"timeAt":{"type":"com.linkedin.common.Time"},"text":{"type":"com.linkedin.390ac1c1431d5b4e340a65af24ed68f2"}},"baseType":"com.linkedin.voyager.dash.jobs.JobPostingCardFooterItem"},"com.linkedin.f4f99771435399029308495de4e742b6":{"fields":{"relevanceReason":{"type":"com.linkedin.390ac1c1431d5b4e340a65af24ed68f2"},"currentCompanyUrn":{"type":"string"},"currentCompany":{"resolvedFrom":"currentCompanyUrn","type":"com.linkedin.a2ef5c1c8d1bd7b83de7009b834a38c6"},"relevanceReasonFlavor":{"type":"com.linkedin.voyager.dash.premium.companyinsights.JobPostingRelevanceReasonFlavor"}},"baseType":"com.linkedin.voyager.dash.premium.companyinsights.JobPostingRelevanceReason"},"com.linkedin.f0aa7f6258b1a3e754f7d6bbfc9af84b":{"fields":{"postApplyPromoType":{"resolvedFrom":"postApplyPromoTypeUnion","type":{"union":{"interviewPrep":{"array":"com.linkedin.85d03babfc1dc1ea8537cef9b2cc1bf5"},"eeocConsent":"com.linkedin.8b1684dd351b279ed40c6661257d0e63","premiumUpsell":"com.linkedin.dc4c6e3940e502e42eb82438140331c7","makeMeMove":"com.linkedin.8b1684dd351b279ed40c6661257d0e63","skillAssessments":{"array":"com.linkedin.4f63adbedb57590490ae8c38e3318a15"},"addSkill":"com.linkedin.8b1684dd351b279ed40c6661257d0e63","paidCompany":"com.linkedin.8b1684dd351b279ed40c6661257d0e63","similarJobs":"com.linkedin.8b1684dd351b279ed40c6661257d0e63","screeningSurvey":"com.linkedin.0a38c5f17cb738fcfee67c772442d536","verification":"com.linkedin.8b1684dd351b279ed40c6661257d0e63","demographics":"com.linkedin.8b1684dd351b279ed40c6661257d0e63"}}},"summary":{"type":"com.linkedin.390ac1c1431d5b4e340a65af24ed68f2"},"cta":{"type":"com.linkedin.390ac1c1431d5b4e340a65af24ed68f2"},"description":{"type":"com.linkedin.390ac1c1431d5b4e340a65af24ed68f2"},"postApplyPromoTypeUnion":{"type":{"union":{"interviewPrep":{"array":"com.linkedin.85d03babfc1dc1ea8537cef9b2cc1bf5"},"eeocConsent":"com.linkedin.8b1684dd351b279ed40c6661257d0e63","premiumUpsell":"string","makeMeMove":"com.linkedin.8b1684dd351b279ed40c6661257d0e63","skillAssessments":"string","addSkill":"com.linkedin.8b1684dd351b279ed40c6661257d0e63","paidCompany":"com.linkedin.8b1684dd351b279ed40c6661257d0e63","similarJobs":"com.linkedin.8b1684dd351b279ed40c6661257d0e63","screeningSurvey":"string","verification":"com.linkedin.8b1684dd351b279ed40c6661257d0e63","demographics":"com.linkedin.8b1684dd351b279ed40c6661257d0e63"}}},"legoTrackingToken":{"type":"string"},"renderType":{"type":"com.linkedin.voyager.dash.jobs.postapply.PostApplyPromoCardRenderType"}},"baseType":"com.linkedin.voyager.dash.jobs.postapply.PostApplyPromo"},"com.linkedin.7bd2de9a7b9fadfe651a8dd1c13db7ad":{"fields":{"subOptions":{"type":{"array":"com.linkedin.98d89bae4aa4ef8f3ffc1018ca93419c"}},"topLevelOption":{"type":"com.linkedin.98d89bae4aa4ef8f3ffc1018ca93419c"}},"baseType":"com.linkedin.voyager.dash.common.forms.NestedTextSelectableOption"},"com.linkedin.dbb12a0149e541da7067f13d22b542eb":{"fields":{"start":{"type":"float"},"end":{"type":"float"}},"baseType":"com.linkedin.common.FloatRange"},"com.linkedin.65862eebdeb12a456370f32142eebd8c":{"fields":{"suggestedEntity":{"type":"com.linkedin.98d89bae4aa4ef8f3ffc1018ca93419c"},"navigationUrl":{"type":"com.linkedin.common.Url"}},"baseType":"com.linkedin.voyager.dash.common.forms.ActionableSuggestedEntity"},"com.linkedin.81315c1c0ea01a8f7161ae78bbbd0ff1":{"fields":{"jobTrackingData":{"type":"com.linkedin.e53bf26a30b7d5901401fa6088b455dc"},"jobPostingCardUrn":{"type":"string"},"jobPostingCard":{"resolvedFrom":"jobPostingCardUrn","type":"com.linkedin.f42293b24e5d60e317d2d68ab7597f64"}},"baseType":"com.linkedin.voyager.dash.jobs.JobPostingCardWrapper"},"com.linkedin.5d8878fd7b4508fcbef4f2d67c765db3":{"fields":{"memberDistance":{"type":"com.linkedin.voyager.dash.relationships.NoConnectionMemberDistance"}},"baseType":"com.linkedin.voyager.dash.relationships.NoConnection"},"com.linkedin.8e19fba94618da7ceb9cca894420a5da":{"fields":{"entityUrn":{"type":"com.linkedin.voyager.dash.common.CoachJobsKickoffPromptsUrn"},"kickoffPrompts":{"type":{"array":"com.linkedin.70094a2b1194a3dfad82481287a40a81"}}},"baseType":"com.linkedin.voyager.dash.jobs.gai.CoachJobsKickoffPrompts"},"com.linkedin.847ffe38969d62d2505e06cf8cbfaa3d":{"fields":{"functionPercentage":{"type":"int"},"functionUrn":{"type":"string"},"functionCount":{"type":"int"},"function":{"resolvedFrom":"functionUrn","type":"com.linkedin.39c50e73d5d652d9f5f7194f5023e967"}},"baseType":"com.linkedin.voyager.dash.premium.companyinsights.FunctionCount"},"com.linkedin.59b66a5c22b53ee350982aba9da43296":{"fields":{"formattedBaseSalary":{"type":"string"},"geoLocationUrn":{"type":"string"},"geoLocation":{"resolvedFrom":"geoLocationUrn","type":"com.linkedin.00c4f529ff2fa302d89eaea66344f268"},"entityUrn":{"type":"com.linkedin.voyager.dash.common.SalaryInsightsUrn"},"formattedAdditionalCompensation":{"type":"string"},"collectAccuracyFeedback":{"type":"boolean"},"compensationSource":{"type":"com.linkedin.voyager.dash.salary.CompensationSource"},"additionalCompensationTypes":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.salary.SalaryInsights"},"com.linkedin.70094a2b1194a3dfad82481287a40a81":{"fields":{"queryText":{"type":"string"},"controlName":{"type":"com.linkedin.voyager.dash.common.tracking.ControlName"},"coachSparkleIcon":{"type":"com.linkedin.30e1ec6a491035bd7a31cc9de8a12c18"},"isPremium":{"type":"boolean"},"title":{"type":"com.linkedin.390ac1c1431d5b4e340a65af24ed68f2"},"toolTipText":{"type":"string"},"coachQueryContextV2":{"type":"com.linkedin.8864b68d440629f1899c8ff463b7f71d"},"coachUseCaseType":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.jobs.gai.CoachJobsKickoffPrompt"},"com.linkedin.be7717be8d269f7c62e51d9156652c9b":{"fields":{"navigationAction":{"type":"com.linkedin.5de46613f3027e5a41b356871519a78b"},"text":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.jobs.ItemsMatchDescription"},"com.linkedin.b4d79116ccce85e0d4ef36bf211da0a3":{"fields":{"singleQuestionSubForm":{"type":"com.linkedin.554b975e2cabf1c2139af63808fd028a"},"icon":{"type":"com.linkedin.30e1ec6a491035bd7a31cc9de8a12c18"},"controlName":{"type":"com.linkedin.voyager.dash.common.tracking.ControlName"},"iconAfterText":{"type":"boolean"},"placeHolderText":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.common.forms.VisibilitySettingButton"},"com.linkedin.bb168eba9e814ac2c87fa5288ea74e6f":{"fields":{"image":{"type":"com.linkedin.voyager.dash.common.SystemImageName"},"title":{"type":"string"},"imageColor":{"type":"com.linkedin.voyager.dash.common.SystemImageTintColor"},"navigationAction":{"type":"com.linkedin.5de46613f3027e5a41b356871519a78b"},"subtitle":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.jobs.ItemsMatchItem"},"com.linkedin.60257368060941d7900dbbf2748f711c":{"fields":{"postalCodeTextField":{"type":"com.linkedin.2f3c656d762dff10246b267bedddfcfd"},"cityTypeaheadField":{"type":"com.linkedin.b7e88af0a8aa84677bc8effdf13ab5d9"},"useCurrentLocation":{"type":"com.linkedin.0608c02e08bf95759764fc707e772cb9"},"countryField":{"type":"com.linkedin.b7e88af0a8aa84677bc8effdf13ab5d9"},"cityTextEntityListField":{"type":"com.linkedin.84ab2fe95ad74fa18ab07857b1c58889"}},"baseType":"com.linkedin.voyager.dash.common.forms.LocationFormComponent"},"com.linkedin.a1cf47049f3ce4950c977d8052ae04d3":{"fields":{"viewImpressionTrackingKey":{"type":"string"},"legoTrackingToken":{"type":"string"},"text":{"type":"com.linkedin.390ac1c1431d5b4e340a65af24ed68f2"},"type":{"type":"com.linkedin.voyager.dash.jobs.JobPostingTipType"}},"baseType":"com.linkedin.voyager.dash.jobs.JobPostingTip"},"com.linkedin.4c9ae0bb91144f3556c7bcc9bd1b33bd":{"fields":{"entityUrn":{"type":"com.linkedin.voyager.dash.common.SkillAssessmentAttemptReportUrn"}},"baseType":"com.linkedin.voyager.dash.assessments.SkillAssessmentAttemptReport"},"com.linkedin.2d03db73fb8c8c66e4245b79a3d25e52":{"fields":{"title":{"type":"string"},"subtitle":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.jobs.JobSegmentAttributesCardSection"},"com.linkedin.296fd4ebc3f7a46ccca548d534f656e4":{"fields":{"detailData":{"resolvedFrom":"detailDataUnion","type":{"union":{"profilePictureWithoutFrame":"com.linkedin.826b8c485ed1073773697502413b9597","profilePictureWithRingStatus":"com.linkedin.826b8c485ed1073773697502413b9597","companyLogo":"com.linkedin.a2ef5c1c8d1bd7b83de7009b834a38c6","icon":"com.linkedin.voyager.dash.common.ArtDecoIconName","systemImage":"com.linkedin.voyager.dash.common.SystemImageName","nonEntityGroupLogo":"com.linkedin.cdabf7a3337e639c42189d7fe44cdc92","vectorImage":"com.linkedin.b8949adeaaa915212d0136c4b9a1952b","nonEntityProfessionalEventLogo":"com.linkedin.6e3e751d09550f922978dedd7f52eea5","profilePicture":"com.linkedin.826b8c485ed1073773697502413b9597","imageUrl":"com.linkedin.2ef5b25d61e5db956d269b94a55f4763","professionalEventLogo":"com.linkedin.3900e4e094a1b889ef9285a430d6f2d5","nonEntitySchoolLogo":"com.linkedin.a7d235a42f3653f9539026976306f11b","nonEntityCompanyLogo":"com.linkedin.5fe8101cb3a914392f7301964e56573c","schoolLogo":"com.linkedin.d4dcbabbf47d9675b0597099d9cd7d69","groupLogo":"com.linkedin.32bab5d7af29a8bdf4fe920d7d2155b1","ghostImage":"com.linkedin.voyager.dash.common.image.GhostImageType","nonEntityProfilePicture":"com.linkedin.c9156c9beebda6b437868e48c22a8f02"}}},"tintColor":{"type":"com.linkedin.voyager.dash.common.SystemImageTintColor"},"detailDataUnion":{"type":{"union":{"profilePictureWithoutFrame":"string","profilePictureWithRingStatus":"string","companyLogo":"string","icon":"com.linkedin.voyager.dash.common.ArtDecoIconName","systemImage":"com.linkedin.voyager.dash.common.SystemImageName","nonEntityGroupLogo":"com.linkedin.cdabf7a3337e639c42189d7fe44cdc92","vectorImage":"com.linkedin.b8949adeaaa915212d0136c4b9a1952b","nonEntityProfessionalEventLogo":"com.linkedin.6e3e751d09550f922978dedd7f52eea5","profilePicture":"string","imageUrl":"com.linkedin.2ef5b25d61e5db956d269b94a55f4763","professionalEventLogo":"string","nonEntitySchoolLogo":"com.linkedin.a7d235a42f3653f9539026976306f11b","nonEntityCompanyLogo":"com.linkedin.5fe8101cb3a914392f7301964e56573c","schoolLogo":"string","groupLogo":"string","ghostImage":"com.linkedin.voyager.dash.common.image.GhostImageType","nonEntityProfilePicture":"com.linkedin.c9156c9beebda6b437868e48c22a8f02"}}},"tapTargets":{"type":{"array":"com.linkedin.409356f1c0f808083c5e0174516009dd"}},"scalingType":{"type":"com.linkedin.voyager.dash.common.image.ScalingType"},"displayAspectRatio":{"type":"double"}},"baseType":"com.linkedin.voyager.dash.common.image.ImageAttribute"},"com.linkedin.acd5ebb13f6e0c9da292f5d03c57eac4":{"fields":{"countByFunction":{"type":{"array":"com.linkedin.847ffe38969d62d2505e06cf8cbfaa3d"}},"totalCount":{"type":"long"},"yearMonthOn":{"type":"com.linkedin.fefbb3ce7d8629ad1cfd497514eb598e"}},"baseType":"com.linkedin.voyager.dash.premium.companyinsights.CountByFunction"},"com.linkedin.72471cc7e55a55b0fd549127127633f7":{"fields":{"appearance":{"type":"com.linkedin.deaeaf135b51841f133d0ceae7eca621"},"controlName":{"type":"com.linkedin.voyager.dash.common.tracking.ControlName"},"icon":{"type":"com.linkedin.voyager.dash.common.ArtDecoIconName"},"openExternally":{"type":"boolean"},"navigationUrl":{"type":"com.linkedin.common.Url"},"text":{"type":"string"},"accessibilityText":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.common.forms.NavigationButton"},"com.linkedin.985210b44c74343f912e35f90d721799":{"fields":{"typeaheadCta":{"type":"com.linkedin.c754100718c581ece9a715a9165db201"},"description":{"type":"com.linkedin.390ac1c1431d5b4e340a65af24ed68f2"},"selectedEntities":{"type":{"array":"com.linkedin.9013fa15f12f02c028d5b08f542f32d5"}},"selectionCountRangeValidation":{"type":"com.linkedin.e18c41f5008d005150ad33f812748af9"}},"baseType":"com.linkedin.voyager.dash.common.forms.MultiSelectTypeaheadEntityFormComponent"},"com.linkedin.be19a4b9083f30fb8796eb20ea30c1d0":{"fields":{"maximumFileSizeSupported":{"type":"long"},"mimeTypes":{"type":{"array":"com.linkedin.voyager.dash.common.MimeType"}},"uploadFileControlName":{"type":"com.linkedin.voyager.dash.common.tracking.ControlName"},"uploadFileCtaText":{"type":"string"},"deleteControlName":{"type":"com.linkedin.voyager.dash.common.tracking.ControlName"},"mediaUploadType":{"type":"com.linkedin.voyager.dash.video.MediaUploadType"},"previewText":{"type":"com.linkedin.390ac1c1431d5b4e340a65af24ed68f2"},"helperText":{"type":"com.linkedin.390ac1c1431d5b4e340a65af24ed68f2"}},"baseType":"com.linkedin.voyager.dash.common.forms.MediaUploadFormComponent"},"com.linkedin.39c50e73d5d652d9f5f7194f5023e967":{"fields":{"entityUrn":{"type":"com.linkedin.voyager.dash.common.FunctionUrn"},"localizedName":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.premium.companyinsights.Function"},"com.linkedin.03248b99e950a8be3201e56283063ed8":{"fields":{"primaryCtaText":{"type":"string"},"image":{"type":"com.linkedin.30e1ec6a491035bd7a31cc9de8a12c18"},"entityUrn":{"type":"com.linkedin.voyager.dash.common.MarketplacePromoCardUrn"},"subtitle":{"type":"com.linkedin.390ac1c1431d5b4e340a65af24ed68f2"},"primaryCtaNavigationUrl":{"type":"com.linkedin.common.Url"},"title":{"type":"com.linkedin.390ac1c1431d5b4e340a65af24ed68f2"},"trackingId":{"type":"com.linkedin.common.TrackingId"}},"baseType":"com.linkedin.voyager.dash.marketplaces.PromoCard"},"com.linkedin.cdabf7a3337e639c42189d7fe44cdc92":{"fields":{"groupUrn":{"type":"string"},"vectorImage":{"type":"com.linkedin.b8949adeaaa915212d0136c4b9a1952b"},"group":{"resolvedFrom":"groupUrn","type":"com.linkedin.32bab5d7af29a8bdf4fe920d7d2155b1"}},"baseType":"com.linkedin.voyager.dash.common.image.NonEntityGroupLogo"},"com.linkedin.b2a671b0acb9520f18deade4e955c616":{"fields":{"name":{"type":"com.linkedin.390ac1c1431d5b4e340a65af24ed68f2"},"image":{"derivedFrom":"imageUnion","type":{"union":{"url":"com.linkedin.common.Url","vectorImage":"com.linkedin.b8949adeaaa915212d0136c4b9a1952b"}}},"nameSupplementaryInfo":{"type":"com.linkedin.390ac1c1431d5b4e340a65af24ed68f2"},"headline":{"type":"com.linkedin.390ac1c1431d5b4e340a65af24ed68f2"},"imageUnion":{"type":{"union":{"url":"com.linkedin.common.Url","vectorImage":"com.linkedin.b8949adeaaa915212d0136c4b9a1952b"}}},"subHeadline":{"type":"com.linkedin.390ac1c1431d5b4e340a65af24ed68f2"}},"baseType":"com.linkedin.voyager.dash.common.media.StickerLinkMediumTemplateView"},"com.linkedin.55c2879f1a800fb9b17e54740f965515":{"fields":{"footerText":{"type":"com.linkedin.390ac1c1431d5b4e340a65af24ed68f2"},"collapsedState":{"type":"com.linkedin.voyager.dash.common.forms.FormSectionCollapsedState"},"formElementGroups":{"type":{"array":"com.linkedin.6a325d9bf5f44baf8fede1db5a7ce66f"}},"subtitle":{"type":"com.linkedin.390ac1c1431d5b4e340a65af24ed68f2"},"repeatableSectionData":{"type":"com.linkedin.181ad18bcf938cd6016ba9ffd2137150"},"title":{"type":"com.linkedin.390ac1c1431d5b4e340a65af24ed68f2"},"footerTextTitle":{"type":"com.linkedin.390ac1c1431d5b4e340a65af24ed68f2"}},"baseType":"com.linkedin.voyager.dash.common.forms.FormSection"},"com.linkedin.7a7f0ab9f40a57ecd2a27e26059721fe":{"fields":{"name":{"type":"string"},"entityUrn":{"type":"com.linkedin.voyager.dash.common.IndustryV2Urn"}},"baseType":"com.linkedin.voyager.dash.identity.profile.IndustryV2"},"com.linkedin.9ddc780810d0c93fa698a26faab52f10":{"fields":{"errorText":{"type":"com.linkedin.390ac1c1431d5b4e340a65af24ed68f2"},"validRange":{"type":"com.linkedin.dbb12a0149e541da7067f13d22b542eb"}},"baseType":"com.linkedin.voyager.dash.common.forms.validation.NumericValueRangeValidation"},"com.linkedin.6ba053f5dc7e14e40216570d97a71e52":{"fields":{"placeHolderText":{"type":"com.linkedin.390ac1c1431d5b4e340a65af24ed68f2"},"controlName":{"type":"com.linkedin.voyager.dash.common.tracking.ControlName"},"textSelectableOptions":{"type":{"array":"com.linkedin.98d89bae4aa4ef8f3ffc1018ca93419c"}}},"baseType":"com.linkedin.voyager.dash.common.forms.DropdownFormComponent"},"com.linkedin.1ed8d64e266be8d739bffff5d4200fb4":{"fields":{},"baseType":"com.linkedin.voyager.dash.jobs.NoResultsCard"},"com.linkedin.81235c3224517a02609a065763927f01":{"fields":{"formElementInputValues":{"type":{"array":{"union":{"entityInputValue":"com.linkedin.194482b00bd3b5ae56a4d81712d0525c","locationInputValue":"com.linkedin.b429b28d37054e5232ecd5ecffd09315","integerInputValue":"int","dateRangeInputValue":"com.linkedin.8fafa573919c2bc96c29a2be79ce3fb3","urnInputValue":"com.linkedin.common.Urn","floatInputValue":"float","textInputValue":"string","booleanInputValue":"boolean"}}}},"formElementUrn":{"type":"com.linkedin.common.Urn"},"formElementInputValuesResolutionResults":{"derivedFrom":"formElementInputValues","type":{"array":{"union":{"entityInputValue":"com.linkedin.194482b00bd3b5ae56a4d81712d0525c","locationInputValue":"com.linkedin.b429b28d37054e5232ecd5ecffd09315","integerInputValue":"int","dateRangeInputValue":"com.linkedin.8fafa573919c2bc96c29a2be79ce3fb3","urnInputValue":"com.linkedin.common.Urn","floatInputValue":"float","textInputValue":"string","booleanInputValue":"boolean"}}}}},"baseType":"com.linkedin.voyager.dash.common.forms.FormElementInput"},"com.linkedin.c138fd2c1944eec6c90de7592dd91018":{"fields":{"groups":{"type":{"array":"com.linkedin.d2acc160e7db812cd2fc11b5eaac7b0f"}},"tip":{"type":"com.linkedin.00ac75ef8dae327ab29a045438e7d186"},"viewImpressionTrackingKey":{"type":"string"},"buttonCTA":{"type":"com.linkedin.5de46613f3027e5a41b356871519a78b"}},"baseType":"com.linkedin.voyager.dash.jobs.JobQualificationSection"},"com.linkedin.29a8a179800ba03932dae2884cf40cff":{"fields":{"insightViewModel":{"type":"com.linkedin.3551030d48734b4fd9cb3238efc77ece"},"popoverTriggerIcon":{"type":"com.linkedin.voyager.dash.common.SystemImageName"},"popoverControlName":{"type":"com.linkedin.voyager.dash.common.tracking.ControlName"},"popoverContent":{"type":"com.linkedin.390ac1c1431d5b4e340a65af24ed68f2"}},"baseType":"com.linkedin.voyager.dash.jobs.TrustInsightViewModel"},"com.linkedin.c99e83ae287631a7e9fc1ff222ab73bb":{"fields":{},"baseType":"com.linkedin.voyager.dash.jobs.search.EndOfResultsCard"},"com.linkedin.fc957b3fdfa381bf007a3b1c29d6fe4c":{"fields":{"pageKey":{"type":"string"},"anchorPage":{"type":"boolean"}},"baseType":"com.linkedin.voyager.dash.common.tracking.PageKey"},"com.linkedin.21a4612bc8ff64d01e0062f74b272040":{"fields":{},"baseType":"com.linkedin.voyager.dash.jobs.CarouselCollectionCard"},"com.linkedin.85d03babfc1dc1ea8537cef9b2cc1bf5":{"fields":{"thumbnail":{"type":"com.linkedin.30e1ec6a491035bd7a31cc9de8a12c18"},"questionText":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.jobs.postapply.InterviewPrepQuestion"},"com.linkedin.84ab2fe95ad74fa18ab07857b1c58889":{"fields":{"locationList":{"type":"com.linkedin.310d48bff502026d1fdc80b24448401d"},"requiredFieldMissingErrorText":{"type":"com.linkedin.390ac1c1431d5b4e340a65af24ed68f2"},"responseRequired":{"type":"boolean"},"labelText":{"type":"com.linkedin.390ac1c1431d5b4e340a65af24ed68f2"}},"baseType":"com.linkedin.voyager.dash.common.forms.LocationTextEntityListField"},"com.linkedin.180bad7e2024ccff1ee0157be65c0dd1":{"fields":{"jobPostingDetailSectionUnions":{"type":{"array":{"union":{"similarJobsCard":"com.linkedin.be72f0bd614be87c519a3b06d3042062","interviewPrepCard":"com.linkedin.87b6c1610a369328d34d660f0b651ea4","salaryCard":"string","companyInsightsCard":"string","topCard":"string","bannerCard":"com.linkedin.19edabee847f40f1158364b31cb6309d","jobSeekerActionCard":"com.linkedin.44e816344c2289e8a62e33486601f239","learningRecommendationCard":"string","hiringTeamCard":"com.linkedin.392fa039af72d21e485244574f20ca3b","similarJobsAtCompanyCard":"com.linkedin.be72f0bd614be87c519a3b06d3042062","jobSegmentAttributesCard":"com.linkedin.c4a89ad017a52048790e89ac5ba1ab35","howYouMatchCard":"string","jobApplicantInsightsUrn":"string","jobActivityCard":"com.linkedin.863cd9abd8abab984b4347da5429d447","postApplyPromo":{"array":"com.linkedin.f0aa7f6258b1a3e754f7d6bbfc9af84b"},"jobDescription":"com.linkedin.f1007edfc8733ef870d8dc290d661d6a","benefitsCard":"com.linkedin.8e2783ee98d67c681658fdc2f0519697","jobAlertCard":"com.linkedin.2db94c9a357c51aebecf0054e59f9bd7","marketplacePromoCard":"string","companyCardV2":"string","recommendedActionsCard":"string"}}}},"jobPostingDetailSection":{"resolvedFrom":"jobPostingDetailSectionUnions","type":{"array":{"union":{"similarJobsCard":"com.linkedin.be72f0bd614be87c519a3b06d3042062","interviewPrepCard":"com.linkedin.87b6c1610a369328d34d660f0b651ea4","salaryCard":"com.linkedin.59b66a5c22b53ee350982aba9da43296","companyInsightsCard":"com.linkedin.f8ea4e1538764dc27e403a1d55047e51","topCard":"com.linkedin.f42293b24e5d60e317d2d68ab7597f64","bannerCard":"com.linkedin.19edabee847f40f1158364b31cb6309d","jobSeekerActionCard":"com.linkedin.44e816344c2289e8a62e33486601f239","learningRecommendationCard":"com.linkedin.2791a0d398e611fad8c820c60183e2bb","hiringTeamCard":"com.linkedin.392fa039af72d21e485244574f20ca3b","similarJobsAtCompanyCard":"com.linkedin.be72f0bd614be87c519a3b06d3042062","jobSegmentAttributesCard":"com.linkedin.c4a89ad017a52048790e89ac5ba1ab35","howYouMatchCard":"com.linkedin.6966477af81c9237d13230a0dced17c7","jobApplicantInsightsUrn":"com.linkedin.1b53ff96dc1e6cc3644919ffccaddf68","jobActivityCard":"com.linkedin.863cd9abd8abab984b4347da5429d447","postApplyPromo":{"array":"com.linkedin.f0aa7f6258b1a3e754f7d6bbfc9af84b"},"jobDescription":"com.linkedin.f1007edfc8733ef870d8dc290d661d6a","benefitsCard":"com.linkedin.8e2783ee98d67c681658fdc2f0519697","jobAlertCard":"com.linkedin.2db94c9a357c51aebecf0054e59f9bd7","marketplacePromoCard":"com.linkedin.03248b99e950a8be3201e56283063ed8","companyCardV2":"com.linkedin.ca8249c8427e4d60dbeb41d6b9cc8bbd","recommendedActionsCard":"com.linkedin.236a047ab00edc2a263bcd4777b91ac1"}}}}},"baseType":"com.linkedin.voyager.dash.jobs.JobPostingDetailSection"},"com.linkedin.3c0ef9a4c6857bf56baeb410170785ee":{"fields":{"duration":{"type":"string"},"thumbnail":{"type":"com.linkedin.30e1ec6a491035bd7a31cc9de8a12c18"},"navigationUrl":{"type":"com.linkedin.common.Url"},"chapterTitle":{"type":"string"},"entityUrn":{"type":"com.linkedin.voyager.dash.common.LearningVideoUrn"}},"baseType":"com.linkedin.voyager.dash.learning.LearningVideo"},"com.linkedin.30951a1ee13ae9dec8fa329260e12fc2":{"fields":{"header":{"type":"string"},"itemsDescription":{"type":"com.linkedin.be7717be8d269f7c62e51d9156652c9b"},"subheader":{"type":"string"},"items":{"type":{"array":"com.linkedin.bb168eba9e814ac2c87fa5288ea74e6f"}}},"baseType":"com.linkedin.voyager.dash.jobs.ItemsMatchGroup"},"com.linkedin.f8ea4e1538764dc27e403a1d55047e51":{"fields":{"jobWorkplaceTypes":{"resolvedFrom":"jobWorkplaceTypesUrns","type":{"array":"com.linkedin.79cb580711dcd3cf7d83e724b3d9e260"}},"companyInsightsCard":{"type":"com.linkedin.0165d1fc4938f280476b4c9f9ffd3f3f","isInjection":true},"companyDetails":{"type":"com.linkedin.b97672681af5b861a5ec5c53b8fcbc96"},"jobState":{"type":"com.linkedin.voyager.dash.jobs.JobState"},"trackingUrn":{"type":"com.linkedin.common.JobPostingUrn"},"description":{"type":"com.linkedin.390ac1c1431d5b4e340a65af24ed68f2"},"trustReviewDecision":{"type":"com.linkedin.voyager.dash.jobs.JobPostingTrustClassification"},"workplaceTypes":{"type":{"array":"string"}},"repostedJob":{"type":"boolean"},"title":{"type":"string"},"posterId":{"type":"string"},"createdAt":{"type":"com.linkedin.common.Time"},"contentSource":{"type":"com.linkedin.voyager.dash.common.semaphore.ContentSource"},"premiumUpsellCard":{"type":"com.linkedin.86ff9fd16c75bc659bfb6287b8ec1647","isInjection":true},"entityUrn":{"type":"com.linkedin.voyager.dash.common.JobPostingUrn"},"locationUrn":{"type":"string"},"location":{"resolvedFrom":"locationUrn","type":"com.linkedin.00c4f529ff2fa302d89eaea66344f268"},"workplaceTypesResolutionResults":{"resolvedFrom":"workplaceTypes","type":{"array":"com.linkedin.79cb580711dcd3cf7d83e724b3d9e260"}},"jobWorkplaceTypesUrns":{"type":{"array":"string"}},"jobDetailsSummaryCard":{"type":"com.linkedin.513522081428feabe67539f69de00fc6","isInjection":true}},"baseType":"com.linkedin.voyager.dash.jobs.JobPosting"},"com.linkedin.ff0a7ae4688d8d97ea2151c00e6e8392":{"fields":{"lastPageLeftOff":{"type":"int"},"entityUrn":{"type":"com.linkedin.voyager.dash.common.JobDraftApplicationInfoUrn"},"saveAsDraftAt":{"type":"com.linkedin.common.Time"}},"baseType":"com.linkedin.voyager.dash.jobs.JobDraftApplicationInfo"},"com.linkedin.e276a8a343fc735c387871abc9190c7d":{"fields":{"headcountGrowth":{"type":{"array":"com.linkedin.e5b59282f574d2493aeb20a65a9a1f94"}},"medianTenureYears":{"type":"com.linkedin.390ac1c1431d5b4e340a65af24ed68f2"}},"baseType":"com.linkedin.voyager.dash.premium.companyinsights.CompanyInsightsEmployeeGrowth"},"com.linkedin.f1007edfc8733ef870d8dc290d661d6a":{"fields":{"thirdPartyLabel":{"type":"com.linkedin.23ea16078c6de6222c73b55c40e8e8ab"},"jobPostingUrn":{"type":"string"},"entityUrn":{"type":"com.linkedin.voyager.dash.common.JobDescriptionUrn"},"postedOnText":{"type":"string"},"header":{"type":"com.linkedin.390ac1c1431d5b4e340a65af24ed68f2"},"jobPosting":{"resolvedFrom":"jobPostingUrn","type":"com.linkedin.f8ea4e1538764dc27e403a1d55047e51"},"descriptionText":{"type":"com.linkedin.390ac1c1431d5b4e340a65af24ed68f2"}},"baseType":"com.linkedin.voyager.dash.jobs.JobDescription"},"com.linkedin.b8949adeaaa915212d0136c4b9a1952b":{"fields":{"attribution":{"type":"string"},"digitalmediaAsset":{"type":"com.linkedin.common.DigitalmediaAssetUrn"},"rootUrl":{"type":"string"},"artifacts":{"type":{"array":"com.linkedin.8d64ead1eeea51a297dc70da47337de1"}}},"baseType":"com.linkedin.common.VectorImage"},"com.linkedin.236a047ab00edc2a263bcd4777b91ac1":{"fields":{"postApplyPromos":{"type":{"array":"com.linkedin.f0aa7f6258b1a3e754f7d6bbfc9af84b"}},"entityUrn":{"type":"com.linkedin.voyager.dash.common.RecommendedActionsCardUrn"}},"baseType":"com.linkedin.voyager.dash.jobs.postapply.RecommendedActionsCard"},"com.linkedin.cb6a34e10a21dc3fd25d32ef8d5002f5":{"fields":{"jobPostingRelevanceFeedback":{"resolvedFrom":"jobPostingRelevanceFeedbackUrn","type":"com.linkedin.d85acf0e41970312e8912b9455cefa24"},"dismissControlName":{"type":"com.linkedin.voyager.dash.common.tracking.ControlName"},"channel":{"type":"com.linkedin.voyager.dash.jobs.feedback.JobPostingRelevanceFeedbackChannel"},"followUpFeedbackReasons":{"type":{"array":"com.linkedin.d29a7d8226ce963ff5fc2dcd8f1376f6"}},"jobPostingRelevanceFeedbackUrn":{"type":"string"},"dismissUndoControlName":{"type":"com.linkedin.voyager.dash.common.tracking.ControlName"}},"baseType":"com.linkedin.voyager.dash.jobs.DismissJobAction"},"com.linkedin.2dea04d4082c6960578ac30a3ace5ee0":{"fields":{"typeaheadQueryContext":{"type":{"array":"string"}},"typeaheadQuery":{"type":"com.linkedin.d1ea2131164c2418a986748ae0e5000a"},"freeFormTextNotAllowedErrorText":{"type":"string"},"validationMetadata":{"derivedFrom":"validationMetadataUnion","type":{"union":{"pronounsV2":"com.linkedin.bfd20979d76e9c8ce942bc03238c6b02","string":"com.linkedin.bfd20979d76e9c8ce942bc03238c6b02","textFormInputType":"com.linkedin.voyager.dash.common.forms.validation.TextFormInputType","postalCode":"com.linkedin.2b6a63a1bb3f7b0bf5c50d8530db9f78","name":"com.linkedin.bfd20979d76e9c8ce942bc03238c6b02","pronouns":"string","integer":"com.linkedin.8409c9296c5bda7be8aee44feff66dd3","decimal":"com.linkedin.8409c9296c5bda7be8aee44feff66dd3","headline":"com.linkedin.bfd20979d76e9c8ce942bc03238c6b02","url":"com.linkedin.48be3e5c4d0dfb61ba0a4a296e719909"}}},"validationMetadataUnion":{"type":{"union":{"pronounsV2":"com.linkedin.bfd20979d76e9c8ce942bc03238c6b02","string":"com.linkedin.bfd20979d76e9c8ce942bc03238c6b02","textFormInputType":"com.linkedin.voyager.dash.common.forms.validation.TextFormInputType","postalCode":"com.linkedin.2b6a63a1bb3f7b0bf5c50d8530db9f78","name":"com.linkedin.bfd20979d76e9c8ce942bc03238c6b02","pronouns":"string","integer":"com.linkedin.8409c9296c5bda7be8aee44feff66dd3","decimal":"com.linkedin.8409c9296c5bda7be8aee44feff66dd3","headline":"com.linkedin.bfd20979d76e9c8ce942bc03238c6b02","url":"com.linkedin.48be3e5c4d0dfb61ba0a4a296e719909"}}},"typeaheadType":{"type":"com.linkedin.voyager.dash.search.TypeaheadType"},"freeFormTextAllowed":{"type":"boolean"}},"baseType":"com.linkedin.voyager.dash.common.forms.TypeaheadMetadata"},"com.linkedin.d9052d9e22b039f19b4d6484e2ebec27":{"fields":{"image":{"type":"com.linkedin.30e1ec6a491035bd7a31cc9de8a12c18"},"socialProofInsight":{"type":"com.linkedin.3551030d48734b4fd9cb3238efc77ece"},"subtext":{"type":"com.linkedin.390ac1c1431d5b4e340a65af24ed68f2"},"ctaText":{"type":"com.linkedin.390ac1c1431d5b4e340a65af24ed68f2"},"upsellOrderOrigin":{"type":"string"},"controlName":{"type":"com.linkedin.voyager.dash.common.tracking.ControlName"},"actionUrl":{"type":"com.linkedin.common.Url"},"legoTrackingToken":{"type":"string"},"title":{"type":"com.linkedin.390ac1c1431d5b4e340a65af24ed68f2"},"cancelCta":{"type":"com.linkedin.43d5fcbafc4556d0c474dabd56f017a2"},"globalLegoTrackingToken":{"type":"string"},"promotionLegoTrackingToken":{"type":"string"},"funnelCommonHeader":{"type":"com.linkedin.c2408eaefddaf66bea26cb69c95979d7"},"fireDiscoveryImpressionEvent":{"type":"boolean"},"subtitle":{"type":"com.linkedin.390ac1c1431d5b4e340a65af24ed68f2"},"dismissible":{"type":"boolean"},"layoutStyle":{"type":"com.linkedin.voyager.dash.premium.PremiumUpsellLayoutStyle"},"ctaButtonSecondaryEmphasizedTheme":{"type":"boolean"}},"baseType":"com.linkedin.voyager.dash.premium.PremiumUpsellCard"},"com.linkedin.310d48bff502026d1fdc80b24448401d":{"fields":{"controlName":{"type":"com.linkedin.voyager.dash.common.tracking.ControlName"},"pageKey":{"type":"com.linkedin.fc957b3fdfa381bf007a3b1c29d6fe4c"},"placeHolderText":{"type":"com.linkedin.390ac1c1431d5b4e340a65af24ed68f2"},"textSelectableOptions":{"type":{"array":"com.linkedin.98d89bae4aa4ef8f3ffc1018ca93419c"}},"helperText":{"type":"com.linkedin.390ac1c1431d5b4e340a65af24ed68f2"},"preferredRenderingStyle":{"type":"com.linkedin.voyager.dash.common.forms.TextEntityListPreferredRenderingStyle"}},"baseType":"com.linkedin.voyager.dash.common.forms.TextEntityListFormComponent"},"com.linkedin.c175e2014cb329dc190cefda1f8b1191":{"fields":{"saveControlName":{"type":"com.linkedin.voyager.dash.common.tracking.ControlName"},"saveState":{"type":"string"},"unsaveControlName":{"type":"com.linkedin.voyager.dash.common.tracking.ControlName"},"saveStateResolutionResult":{"resolvedFrom":"saveState","type":"com.linkedin.9731877e56bf195f5714e9f273590184"}},"baseType":"com.linkedin.voyager.dash.jobs.SaveJobAction"},"com.linkedin.2c2e713c6b2c9f6c424799fa2ababafd":{"fields":{"viewerCount":{"type":"int"},"thumbnail":{"type":"com.linkedin.30e1ec6a491035bd7a31cc9de8a12c18"},"title":{"type":"string"},"entityUrn":{"type":"com.linkedin.voyager.dash.common.LearningCourseUrn"},"learningNavigationUrl":{"type":"com.linkedin.common.Url"}},"baseType":"com.linkedin.voyager.dash.learning.LearningCourse"},"com.linkedin.c8f6142f1647b4b53caab19e4fe964ef":{"fields":{"textSelectableOption":{"type":"com.linkedin.98d89bae4aa4ef8f3ffc1018ca93419c"}},"baseType":"com.linkedin.voyager.dash.common.forms.DismissPill"},"com.linkedin.d29a7d8226ce963ff5fc2dcd8f1376f6":{"fields":{"type":{"type":"com.linkedin.voyager.dash.jobs.search.DismissedFeedbackReasonType"},"text":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.jobs.search.DismissedFeedbackReason"},"com.linkedin.0608c02e08bf95759764fc707e772cb9":{"fields":{"controlName":{"type":"com.linkedin.voyager.dash.common.tracking.ControlName"},"text":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.common.forms.FormButton"},"com.linkedin.48be3e5c4d0dfb61ba0a4a296e719909":{"fields":{"characterCountRangeValidation":{"type":"com.linkedin.0bec4a68c3bab987feaffd1b11fa4cf5"},"invalidUrlErrorText":{"type":"com.linkedin.390ac1c1431d5b4e340a65af24ed68f2"}},"baseType":"com.linkedin.voyager.dash.common.forms.validation.UrlValidationMetadata"},"com.linkedin.2b6a63a1bb3f7b0bf5c50d8530db9f78":{"fields":{"errorText":{"type":"com.linkedin.390ac1c1431d5b4e340a65af24ed68f2"},"countryFormElementUrn":{"type":"com.linkedin.common.Urn"}},"baseType":"com.linkedin.voyager.dash.common.forms.validation.PostalCodeValidationMetadata"},"com.linkedin.eef1cf048228a183463b81d9c6790e5f":{"fields":{"degree":{"resolvedFrom":"degreeUrn","type":"com.linkedin.b0ab2e740be6873e75f413db4e533bb9"},"viewersHighestDegree":{"type":"boolean"},"degreeUrn":{"type":"string"},"percentage":{"type":"int"}},"baseType":"com.linkedin.voyager.dash.premium.insights.JobApplicantInsightsDegreeDetail"},"com.linkedin.c88fbcf94b11ad7020d237307a79e8d5":{"fields":{"name":{"type":"com.linkedin.390ac1c1431d5b4e340a65af24ed68f2"},"imageUnion":{"type":{"union":{"url":"com.linkedin.common.Url","vectorImage":"com.linkedin.b8949adeaaa915212d0136c4b9a1952b"}}},"image":{"derivedFrom":"imageUnion","type":{"union":{"url":"com.linkedin.common.Url","vectorImage":"com.linkedin.b8949adeaaa915212d0136c4b9a1952b"}}}},"baseType":"com.linkedin.voyager.dash.common.media.StickerLinkSmallTemplateView"},"com.linkedin.9a2250b4fbfd26ffbc3d92812eb2880d":{"fields":{"entityUrn":{"type":"com.linkedin.voyager.dash.common.MemberRelationshipUrn"},"memberRelationshipUnion":{"type":{"union":{"self":"com.linkedin.8b1684dd351b279ed40c6661257d0e63","connection":"com.linkedin.0030135583a12944453eaad4d3729402","noConnection":"com.linkedin.5d8878fd7b4508fcbef4f2d67c765db3"}}},"memberRelationship":{"derivedFrom":"memberRelationshipUnion","type":{"union":{"self":"com.linkedin.8b1684dd351b279ed40c6661257d0e63","connection":"com.linkedin.0030135583a12944453eaad4d3729402","noConnection":"com.linkedin.5d8878fd7b4508fcbef4f2d67c765db3"}}}},"baseType":"com.linkedin.voyager.dash.relationships.MemberRelationship"},"com.linkedin.8864b68d440629f1899c8ff463b7f71d":{"fields":{"serverIntent":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.common.coach.CoachQueryContextV2"},"com.linkedin.0165d1fc4938f280476b4c9f9ffd3f3f":{"fields":{"elements":{"type":{"array":"com.linkedin.04a6d570213522d9270f88b0374505b6"}}},"baseType":"com.linkedin.restli.common.CollectionResponse"},"com.linkedin.a29bf6b9168c31d9f760e514428df1c5":{"fields":{"totalEmployees":{"type":"long"},"headcounts":{"type":"com.linkedin.e276a8a343fc735c387871abc9190c7d"},"growthPeriods":{"type":{"array":"com.linkedin.f6e5193777aa8860860ecb48d0212cce"}}},"baseType":"com.linkedin.voyager.dash.premium.companyinsights.HeadcountInsights"},"com.linkedin.8fbc3627c8305a33e3e6840ec7688312":{"fields":{"jobsDashJobPostingDetailSectionsByCardSectionTypes":{"type":"com.linkedin.f1d8ff8a5c9d0a2c73d8c4418e3a4526"}},"baseType":"com.linkedin.graphql.Query"},"com.linkedin.d8e085065c27e5a4b4a3b47e39d34eae":{"fields":{"entityUrn":{"type":"com.linkedin.voyager.dash.common.EntityRelevanceFeedbackUrn"}},"baseType":"com.linkedin.voyager.dash.jobs.feedback.EntityRelevanceFeedback"},"com.linkedin.2815a36a9c66a3c907e5474fae6a1903":{"fields":{"seniorHireCount":{"type":"long"},"allEmployeeHireCount":{"type":"long"},"yearMonthOn":{"type":"com.linkedin.fefbb3ce7d8629ad1cfd497514eb598e"}},"baseType":"com.linkedin.voyager.dash.premium.companyinsights.HireCountsItem"},"com.linkedin.272e46084795068d9243a5e3978fc2e2":{"fields":{"entityUrn":{"type":"com.linkedin.voyager.dash.common.JobAlertUrn"}},"baseType":"com.linkedin.voyager.dash.jobs.JobAlert"},"com.linkedin.e18c41f5008d005150ad33f812748af9":{"fields":{"maxReachedWithExclusionListInfoText":{"type":"string"},"lessThanMinErrorText":{"type":"string"},"errorText":{"type":"com.linkedin.390ac1c1431d5b4e340a65af24ed68f2"},"validRange":{"type":"com.linkedin.6148d310783eaffead8570485d05a156"},"maxExceededErrorText":{"type":"string"},"maxReachedInfoText":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.common.forms.validation.SelectionCountRangeValidation"},"com.linkedin.3900e4e094a1b889ef9285a430d6f2d5":{"fields":{"logoImage":{"type":{"union":{"url":"com.linkedin.common.Url","vectorImage":"com.linkedin.b8949adeaaa915212d0136c4b9a1952b"}}},"logoImageResolutionResult":{"derivedFrom":"logoImage","type":{"union":{"url":"com.linkedin.common.Url","vectorImage":"com.linkedin.b8949adeaaa915212d0136c4b9a1952b"}}},"entityUrn":{"type":"com.linkedin.voyager.dash.common.ProfessionalEventUrn"}},"baseType":"com.linkedin.voyager.dash.events.ProfessionalEvent"},"com.linkedin.5d5d123ba49a49b959a39ab7ab5ea1c4":{"fields":{"popoverTriggerIcon":{"type":"com.linkedin.voyager.dash.common.SystemImageName"},"title":{"type":"com.linkedin.390ac1c1431d5b4e340a65af24ed68f2"},"popoverContent":{"type":"com.linkedin.390ac1c1431d5b4e340a65af24ed68f2"},"subtitle":{"type":"com.linkedin.390ac1c1431d5b4e340a65af24ed68f2"}},"baseType":"com.linkedin.voyager.dash.premium.companyinsights.Header"},"com.linkedin.f26ce46d6c12df14239f36d844033955":{"fields":{"action":{"type":"com.linkedin.5de46613f3027e5a41b356871519a78b"},"qualifications":{"type":{"array":"string"}}},"baseType":"com.linkedin.voyager.dash.jobs.JobQualificationList"},"com.linkedin.939138a9c6796d21af12eaf891a20e80":{"fields":{"pivotUnion":{"type":{"union":{"skill":"string"}}},"pivot":{"resolvedFrom":"pivotUnion","type":{"union":{"skill":"com.linkedin.7a3dfe06b5ebab0041d8cea76d58565a"}}},"contentUnion":{"type":{"union":{"learningCourse":"com.linkedin.2c2e713c6b2c9f6c424799fa2ababafd"}}},"content":{"derivedFrom":"contentUnion","type":{"union":{"learningCourse":"com.linkedin.2c2e713c6b2c9f6c424799fa2ababafd"}}}},"baseType":"com.linkedin.voyager.dash.learning.recommendations.RecommendationContent"},"com.linkedin.aa89cadbee726b8d3c62475faddcf670":{"fields":{"dateText":{"type":"com.linkedin.390ac1c1431d5b4e340a65af24ed68f2"},"controlName":{"type":"com.linkedin.voyager.dash.common.tracking.ControlName"},"dateValidation":{"type":"com.linkedin.48407601ffa8bd83b8651487ea606df9"},"dateInputType":{"type":"com.linkedin.voyager.dash.common.forms.DateInputType"}},"baseType":"com.linkedin.voyager.dash.common.forms.DateFormComponent"},"com.linkedin.59e2a69f3c2d74e9d4b5b8530e816743":{"fields":{"learningVideoUrns":{"type":{"array":"string"}},"learningCourseUrn":{"type":"string"},"learningVideo":{"resolvedFrom":"learningVideoUrns","type":{"array":"com.linkedin.3c0ef9a4c6857bf56baeb410170785ee"}},"learningCourse":{"resolvedFrom":"learningCourseUrn","type":"com.linkedin.2c2e713c6b2c9f6c424799fa2ababafd"}},"baseType":"com.linkedin.voyager.dash.jobs.InterviewPrepLearningRecommendation"},"com.linkedin.30e1ec6a491035bd7a31cc9de8a12c18":{"fields":{"editableAccessibilityText":{"type":"boolean"},"attributes":{"type":{"array":"com.linkedin.296fd4ebc3f7a46ccca548d534f656e4"}},"actionTarget":{"type":"com.linkedin.common.Url"},"accessibilityTextAttributes":{"type":{"array":"com.linkedin.b0a683a611d43de435ed53c6232d8d44"}},"totalCount":{"type":"int"},"accessibilityText":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.common.image.ImageViewModel"},"com.linkedin.4ece7b9b6c81c2cdb35c6bf9c8e01cb7":{"fields":{},"baseType":"com.linkedin.voyager.dash.jobs.feedback.PageFeedbackWidget"},"com.linkedin.bfd20979d76e9c8ce942bc03238c6b02":{"fields":{"characterCountRangeValidation":{"type":"com.linkedin.0bec4a68c3bab987feaffd1b11fa4cf5"},"errorText":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.common.forms.validation.StringValidationMetadata"},"com.linkedin.65d9b685ce8d8355ba025fb1ae135598":{"fields":{"entityUrn":{"type":"com.linkedin.voyager.dash.common.JobSkillMatchInsightUrn"},"insightSummary":{"type":"com.linkedin.3551030d48734b4fd9cb3238efc77ece"}},"baseType":"com.linkedin.voyager.dash.assessments.JobSkillMatchInsight"},"com.linkedin.d09aeb85fdf4991b6b063ae75ef6ef54":{"fields":{"entityLockup":{"type":"com.linkedin.bd66bae736eabe6505d1b80c182e31af"},"hiredPosition":{"type":"com.linkedin.390ac1c1431d5b4e340a65af24ed68f2"},"hireYearMonthOn":{"type":"com.linkedin.fefbb3ce7d8629ad1cfd497514eb598e"}},"baseType":"com.linkedin.voyager.dash.premium.companyinsights.SeniorHiresItem"},"com.linkedin.554b975e2cabf1c2139af63808fd028a":{"fields":{"dismissControlName":{"type":"com.linkedin.voyager.dash.common.tracking.ControlName"},"formElement":{"type":"com.linkedin.258b7479f7c0999032ce22032fa02cdf"},"title":{"type":"string"},"subtitle":{"type":"com.linkedin.390ac1c1431d5b4e340a65af24ed68f2"}},"baseType":"com.linkedin.voyager.dash.common.forms.SingleQuestionSubForm"},"com.linkedin.f42293b24e5d60e317d2d68ab7597f64":{"fields":{"preDashNormalizedJobPostingUrn":{"type":"com.linkedin.voyager.dash.common.PreDashUrn"},"jobInsightsV2":{"type":{"array":{"union":{"premiumUpsellCard":"string","jobInsightViewModel":"com.linkedin.efd2ef2c46b7959f85a3d6461edc3e69","premiumUpsell":"string","insightViewModel":"com.linkedin.3551030d48734b4fd9cb3238efc77ece","verifiedHiringInsight":"string","skillMatchInsight":"string","responsivenessInsightViewModel":"com.linkedin.036f42f6cc87d5881cf2351020672e97","trustInsightViewModel":"com.linkedin.29a8a179800ba03932dae2884cf40cff"}}}},"footerItems":{"type":{"array":"com.linkedin.6c333b78b6aeac833ef88e8174932712"}},"jobPostingTitle":{"type":"string"},"jobPosting":{"resolvedFrom":"jobPostingUrn","type":"com.linkedin.f8ea4e1538764dc27e403a1d55047e51"},"primaryDescription":{"type":"com.linkedin.390ac1c1431d5b4e340a65af24ed68f2"},"navigationBarSubtitle":{"type":"string"},"jobInsightsV2ResolutionResults":{"resolvedFrom":"jobInsightsV2","type":{"array":{"union":{"premiumUpsellCard":"com.linkedin.f8ea4e1538764dc27e403a1d55047e51","jobInsightViewModel":"com.linkedin.efd2ef2c46b7959f85a3d6461edc3e69","premiumUpsell":"com.linkedin.f8ea4e1538764dc27e403a1d55047e51","insightViewModel":"com.linkedin.3551030d48734b4fd9cb3238efc77ece","verifiedHiringInsight":"com.linkedin.ac21e10e9207a4a6523aafa0c51066d4","skillMatchInsight":"com.linkedin.65d9b685ce8d8355ba025fb1ae135598","responsivenessInsightViewModel":"com.linkedin.036f42f6cc87d5881cf2351020672e97","trustInsightViewModel":"com.linkedin.29a8a179800ba03932dae2884cf40cff"}}}},"primaryActionV2Union":{"type":{"union":{"dismissJobAction":"com.linkedin.cb6a34e10a21dc3fd25d32ef8d5002f5","applyJobAction":"com.linkedin.64aff242d842d2f5c0d6ea1432ceb79a","saveJobAction":"com.linkedin.c175e2014cb329dc190cefda1f8b1191"}}},"secondaryActionsV2ResolutionResults":{"derivedFrom":"secondaryActionsV2","type":{"array":{"union":{"dismissJobAction":"com.linkedin.cb6a34e10a21dc3fd25d32ef8d5002f5","applyJobAction":"com.linkedin.64aff242d842d2f5c0d6ea1432ceb79a","saveJobAction":"com.linkedin.c175e2014cb329dc190cefda1f8b1191"}}}},"primaryActionV2":{"derivedFrom":"primaryActionV2Union","type":{"union":{"dismissJobAction":"com.linkedin.cb6a34e10a21dc3fd25d32ef8d5002f5","applyJobAction":"com.linkedin.64aff242d842d2f5c0d6ea1432ceb79a","saveJobAction":"com.linkedin.c175e2014cb329dc190cefda1f8b1191"}}},"coachJobsKickoffPrompts":{"resolvedFrom":"coachJobsKickoffPromptsUrn","type":"com.linkedin.8e19fba94618da7ceb9cca894420a5da"},"secondaryDescription":{"type":"com.linkedin.390ac1c1431d5b4e340a65af24ed68f2"},"secondaryActionsV2":{"type":{"array":{"union":{"dismissJobAction":"com.linkedin.cb6a34e10a21dc3fd25d32ef8d5002f5","applyJobAction":"com.linkedin.64aff242d842d2f5c0d6ea1432ceb79a","saveJobAction":"com.linkedin.c175e2014cb329dc190cefda1f8b1191"}}}},"jobPostingUrn":{"type":"string"},"entityUrn":{"type":"com.linkedin.voyager.dash.common.JobPostingCardUrn"},"logo":{"type":"com.linkedin.30e1ec6a491035bd7a31cc9de8a12c18"},"tip":{"type":"com.linkedin.a1cf47049f3ce4950c977d8052ae04d3"},"tertiaryDescription":{"type":"com.linkedin.390ac1c1431d5b4e340a65af24ed68f2"},"coachJobsKickoffPromptsUrn":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.jobs.JobPostingCard"},"com.linkedin.5de46613f3027e5a41b356871519a78b":{"fields":{"controlName":{"type":"com.linkedin.voyager.dash.common.tracking.ControlName"},"displayText":{"type":"string"},"systemImageNameIcon":{"type":"com.linkedin.voyager.dash.common.SystemImageName"},"actionTarget":{"type":"com.linkedin.common.Url"},"ctaIcon":{"type":"com.linkedin.b49a045df66f28b2a65f3df75d119eae"}},"baseType":"com.linkedin.voyager.dash.jobs.NavigationAction"},"com.linkedin.b69dade258a78e45f8205cb49b6a98b9":{"fields":{"paidInMail":{"type":"boolean"},"extensionContentContextUrn":{"type":"com.linkedin.common.Urn"},"recipient":{"resolvedFrom":"recipientUrns","type":{"array":"com.linkedin.826b8c485ed1073773697502413b9597"}},"messageRequestContextUrn":{"type":"com.linkedin.common.Urn"},"recipientUrns":{"type":{"array":"string"}},"targetUrl":{"type":"com.linkedin.common.Url"},"existingConversation":{"resolvedFrom":"existingConversationUrn","type":"com.linkedin.bb47d92b56e85e0625cd6b7f5d3d0384"},"existingConversationUrn":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.messaging.compose.ComposeNavigationContext"},"com.linkedin.3a31504449fad28c205c97d4985ea256":{"fields":{"upsellSlot":{"resolvedFrom":"upsellSlotUrn","type":"com.linkedin.dc4c6e3940e502e42eb82438140331c7"},"upsellSlotUrn":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.jobs.PremiumUpsellSection"},"com.linkedin.390ac1c1431d5b4e340a65af24ed68f2":{"fields":{"textDirection":{"type":"com.linkedin.voyager.dash.common.text.TextDirection"},"attributesV2":{"type":{"array":"com.linkedin.b0a683a611d43de435ed53c6232d8d44"}},"text":{"type":"string"},"accessibilityTextAttributesV2":{"type":{"array":"com.linkedin.b0a683a611d43de435ed53c6232d8d44"}},"accessibilityText":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.common.text.TextViewModel"},"com.linkedin.0bec4a68c3bab987feaffd1b11fa4cf5":{"fields":{"showCharacterCount":{"type":"boolean"},"errorText":{"type":"com.linkedin.390ac1c1431d5b4e340a65af24ed68f2"},"validRange":{"type":"com.linkedin.6148d310783eaffead8570485d05a156"}},"baseType":"com.linkedin.voyager.dash.common.forms.validation.CharacterCountRangeValidation"},"com.linkedin.1581fb7fcc9bc37fc013106a0e87c905":{"fields":{},"baseType":"com.linkedin.voyager.dash.jobs.search.JobSearchSuggestionComponent"},"com.linkedin.fefbb3ce7d8629ad1cfd497514eb598e":{"fields":{"month":{"type":"int"},"year":{"type":"int"},"day":{"type":"int"}},"baseType":"com.linkedin.common.Date"},"com.linkedin.0f63072b23eab5f48ddb389e6f328afc":{"fields":{"name":{"type":"string"},"entityUrn":{"type":"com.linkedin.voyager.dash.common.SeniorityUrn"}},"baseType":"com.linkedin.voyager.dash.premium.insights.StandardizedSeniority"},"com.linkedin.ac21e10e9207a4a6523aafa0c51066d4":{"fields":{"controlName":{"type":"com.linkedin.voyager.dash.common.tracking.ControlName"},"ctaText":{"type":"string"},"entityUrn":{"type":"com.linkedin.voyager.dash.common.JobVerifiedHiringInsightUrn"},"insightSummary":{"type":"com.linkedin.3551030d48734b4fd9cb3238efc77ece"}},"baseType":"com.linkedin.voyager.dash.jobs.JobVerifiedHiringInsight"},"com.linkedin.04a6d570213522d9270f88b0374505b6":{"fields":{"companyInsightsUnion":{"type":{"union":{"alumniInsights":"com.linkedin.b2b7b2441fdd2b88fd82124f6d8c4a1c","jobPostingCompanyInsights":"com.linkedin.7269fdb3a0f2f75ea98f76d3d3c9dfc3","jobOpeningsInsights":"com.linkedin.45d6d21a26aa62742acf906ac42d8306","functionHeadcountInsights":"com.linkedin.4fb8cecee08f0c588138694ff045dfd9","headcountInsights":"com.linkedin.a29bf6b9168c31d9f760e514428df1c5","hiresInsights":"com.linkedin.7f23d6d0556f01f090158424e75ff2ab"}}},"header":{"type":"com.linkedin.5d5d123ba49a49b959a39ab7ab5ea1c4"},"companyInsights":{"derivedFrom":"companyInsightsUnion","type":{"union":{"alumniInsights":"com.linkedin.b2b7b2441fdd2b88fd82124f6d8c4a1c","jobPostingCompanyInsights":"com.linkedin.7269fdb3a0f2f75ea98f76d3d3c9dfc3","jobOpeningsInsights":"com.linkedin.45d6d21a26aa62742acf906ac42d8306","functionHeadcountInsights":"com.linkedin.4fb8cecee08f0c588138694ff045dfd9","headcountInsights":"com.linkedin.a29bf6b9168c31d9f760e514428df1c5","hiresInsights":"com.linkedin.7f23d6d0556f01f090158424e75ff2ab"}}}},"baseType":"com.linkedin.voyager.dash.premium.companyinsights.CompanyInsightsCard"},"com.linkedin.b26a8e722557e805b373f37f9d0ef00a":{"fields":{},"baseType":"com.linkedin.voyager.dash.jobs.CarouselEntityHighlightCard"},"com.linkedin.b0a683a611d43de435ed53c6232d8d44":{"fields":{"detailData":{"resolvedFrom":"detailDataUnion","type":{"union":{"jobPostingName":"com.linkedin.f8ea4e1538764dc27e403a1d55047e51","profileFamiliarName":"com.linkedin.826b8c485ed1073773697502413b9597","hyperlink":"com.linkedin.common.Url","color":"com.linkedin.voyager.dash.common.text.TextColor","companyName":"com.linkedin.a2ef5c1c8d1bd7b83de7009b834a38c6","icon":"com.linkedin.voyager.dash.common.ArtDecoIconName","epoch":"com.linkedin.94721090e597af7c97503cb8b1a7e50f","systemImage":"com.linkedin.voyager.dash.common.SystemImageName","textLink":"com.linkedin.ce28f52e436b3cdd8100675b5288a600","listItemStyle":"com.linkedin.984a5b2fd75109bc59c5241e98cb46ac","listStyle":"com.linkedin.voyager.dash.common.text.ListStyleType","groupName":"com.linkedin.32bab5d7af29a8bdf4fe920d7d2155b1","hyperlinkOpenExternally":"com.linkedin.common.Url","stringFieldReference":"com.linkedin.146c92f10032ac18e61e96a3f143b9f1","profileFullName":"com.linkedin.826b8c485ed1073773697502413b9597","learningCourseName":"com.linkedin.2c2e713c6b2c9f6c424799fa2ababafd","profileMention":"com.linkedin.826b8c485ed1073773697502413b9597","style":"com.linkedin.voyager.dash.common.text.TextStyle","schoolName":"com.linkedin.d4dcbabbf47d9675b0597099d9cd7d69","hashtag":"com.linkedin.7948e5031442b35b939b7672eac8bad0"}}},"length":{"type":"int"},"start":{"type":"int"},"detailDataUnion":{"type":{"union":{"jobPostingName":"string","profileFamiliarName":"string","hyperlink":"com.linkedin.common.Url","color":"com.linkedin.voyager.dash.common.text.TextColor","companyName":"string","icon":"com.linkedin.voyager.dash.common.ArtDecoIconName","epoch":"com.linkedin.94721090e597af7c97503cb8b1a7e50f","systemImage":"com.linkedin.voyager.dash.common.SystemImageName","textLink":"com.linkedin.ce28f52e436b3cdd8100675b5288a600","listItemStyle":"com.linkedin.984a5b2fd75109bc59c5241e98cb46ac","listStyle":"com.linkedin.voyager.dash.common.text.ListStyleType","groupName":"string","hyperlinkOpenExternally":"com.linkedin.common.Url","stringFieldReference":"com.linkedin.146c92f10032ac18e61e96a3f143b9f1","profileFullName":"string","learningCourseName":"string","profileMention":"string","style":"com.linkedin.voyager.dash.common.text.TextStyle","schoolName":"string","hashtag":"string"}}}},"baseType":"com.linkedin.voyager.dash.common.text.TextAttribute"},"com.linkedin.6e3e751d09550f922978dedd7f52eea5":{"fields":{"professionalEvent":{"resolvedFrom":"professionalEventUrn","type":"com.linkedin.3900e4e094a1b889ef9285a430d6f2d5"},"vectorImage":{"type":"com.linkedin.b8949adeaaa915212d0136c4b9a1952b"},"professionalEventUrn":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.common.image.NonEntityProfessionalEventLogo"},"com.linkedin.409356f1c0f808083c5e0174516009dd":{"fields":{"tapTargetEntityUnion":{"type":{"union":{"companyUrn":"string","profileUrn":"string"}}},"stickerLinkViewUnion":{"type":{"union":{"mediumTemplate":"com.linkedin.b2a671b0acb9520f18deade4e955c616","largeTemplate":"com.linkedin.ca1fb70631b137b6d459efdae8a18bc0","smallTemplate":"com.linkedin.c88fbcf94b11ad7020d237307a79e8d5"}}},"firstCornerXOffsetPercentage":{"type":"float"},"stickerLinkTemplateSize":{"type":"com.linkedin.voyager.dash.common.media.StickerLinkTemplateSize"},"type":{"type":"com.linkedin.voyager.dash.common.TapTargetAttributeType"},"thirdCornerYOffsetPercentage":{"type":"float"},"url":{"type":"com.linkedin.common.Url"},"stickerLinkView":{"derivedFrom":"stickerLinkViewUnion","type":{"union":{"mediumTemplate":"com.linkedin.b2a671b0acb9520f18deade4e955c616","largeTemplate":"com.linkedin.ca1fb70631b137b6d459efdae8a18bc0","smallTemplate":"com.linkedin.c88fbcf94b11ad7020d237307a79e8d5"}}},"urn":{"type":"com.linkedin.common.Urn"},"thirdCornerXOffsetPercentage":{"type":"float"},"secondCornerYOffsetPercentage":{"type":"float"},"fourthCornerXOffsetPercentage":{"type":"float"},"tapTargetEntity":{"resolvedFrom":"tapTargetEntityUnion","type":{"union":{"companyUrn":"com.linkedin.a2ef5c1c8d1bd7b83de7009b834a38c6","profileUrn":"com.linkedin.826b8c485ed1073773697502413b9597"}}},"firstCornerYOffsetPercentage":{"type":"float"},"rank":{"type":"int"},"untaggable":{"type":"boolean"},"fourthCornerYOffsetPercentage":{"type":"float"},"text":{"type":"string"},"secondCornerXOffsetPercentage":{"type":"float"}},"baseType":"com.linkedin.voyager.dash.common.TapTarget"},"com.linkedin.826b8c485ed1073773697502413b9597":{"fields":{"profilePicture":{"type":"com.linkedin.9e0491106da7f36c0672ee51875a86fa"},"lastName":{"type":"string"},"firstName":{"type":"string"},"entityUrn":{"type":"com.linkedin.voyager.dash.common.ProfileUrn"},"memberRelationship":{"type":"com.linkedin.9a2250b4fbfd26ffbc3d92812eb2880d","isInjection":true},"ringStatus":{"type":"com.linkedin.e94906839b912dcb4829422361b5028c","isInjection":true},"headline":{"type":"string"},"publicIdentifier":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.identity.profile.Profile"},"com.linkedin.9731877e56bf195f5714e9f273590184":{"fields":{"preDashEntityUrn":{"type":"com.linkedin.voyager.dash.common.PreDashUrn"},"saved":{"type":"boolean"},"entityUrn":{"type":"com.linkedin.voyager.dash.common.SaveStateUrn"}},"baseType":"com.linkedin.voyager.dash.feed.SaveState"},"com.linkedin.19edabee847f40f1158364b31cb6309d":{"fields":{"jobPostingBanner":{"derivedFrom":"jobPostingBannerUnion","type":{"union":{"jobDetailsClaimJobBanner":"com.linkedin.b97672681af5b861a5ec5c53b8fcbc96","trustBanner":"com.linkedin.0054dfa46558041e61c38c995b4809c8","viewAsHirerBanner":"com.linkedin.common.Time","jobTrustBanner":"string"}}},"navigationActionUnion":{"type":{"union":{"navigateToReviewLearnMoreModal":"com.linkedin.8b1684dd351b279ed40c6661257d0e63","ctaAction":"com.linkedin.5de46613f3027e5a41b356871519a78b"}}},"navigationAction":{"derivedFrom":"navigationActionUnion","type":{"union":{"navigateToReviewLearnMoreModal":"com.linkedin.8b1684dd351b279ed40c6661257d0e63","ctaAction":"com.linkedin.5de46613f3027e5a41b356871519a78b"}}},"jobPostingBannerUnion":{"type":{"union":{"jobDetailsClaimJobBanner":"com.linkedin.b97672681af5b861a5ec5c53b8fcbc96","trustBanner":"com.linkedin.0054dfa46558041e61c38c995b4809c8","viewAsHirerBanner":"com.linkedin.common.Time","jobTrustBanner":"string"}}}},"baseType":"com.linkedin.voyager.dash.jobs.JobPostingBannerCard"},"com.linkedin.ca8249c8427e4d60dbeb41d6b9cc8bbd":{"fields":{"company":{"resolvedFrom":"companyUrn","type":"com.linkedin.a2ef5c1c8d1bd7b83de7009b834a38c6"},"commitments":{"type":{"array":"com.linkedin.b2744b01d95785f8c9ab5f466c1674d5"}},"published":{"type":"boolean"},"companyUrn":{"type":"string"},"entityUrn":{"type":"com.linkedin.voyager.dash.common.CompanyCardUrn"}},"baseType":"com.linkedin.voyager.dash.talentbrand.CompanyCard"},"com.linkedin.be72f0bd614be87c519a3b06d3042062":{"fields":{"header":{"type":"com.linkedin.dd03bc4de44734367e478c7c8eed60a0"},"footer":{"derivedFrom":"footerUnion","type":{"union":{"navigationFooter":"com.linkedin.5de46613f3027e5a41b356871519a78b"}}},"entities":{"type":{"array":{"union":{"promotionalCard":"com.linkedin.c4e907da1ff053ca48ebe32c32f78805","endOfResultsCard":"com.linkedin.c99e83ae287631a7e9fc1ff222ab73bb","jobPostingCardWrapper":"com.linkedin.81315c1c0ea01a8f7161ae78bbbd0ff1","jobSearchHistoryCard":"com.linkedin.d0f6c4718d68ff125ca6f7e0b91f510c","premiumUpsellSlot":"string","jobSearchSuggestion":"com.linkedin.1581fb7fcc9bc37fc013106a0e87c905","newCollectionHeaderCard":"com.linkedin.6e88a2e94acf8ea5aa8c9f7051705709","carouselEntityHighlightCard":"com.linkedin.b26a8e722557e805b373f37f9d0ef00a","feedbackCard":"com.linkedin.4ece7b9b6c81c2cdb35c6bf9c8e01cb7","carouselCollectionCard":"com.linkedin.21a4612bc8ff64d01e0062f74b272040","carouselAdvantageCard":"com.linkedin.57812d345916f0da7785a8887e8fe73d","careerEnrichmentCard":"com.linkedin.b2c46ce630f9fc1eb9ed163a1a7b2616","noResultsCard":"com.linkedin.1ed8d64e266be8d739bffff5d4200fb4","seeAllCard":"com.linkedin.653bbb9565e4a43fc63d44ebc167fab1","refreshStateCard":"com.linkedin.aa4a2fb807b219a33654acc9643ff224","jobPostingCard":"com.linkedin.f42293b24e5d60e317d2d68ab7597f64"}}}},"entityUrn":{"type":"com.linkedin.voyager.dash.common.JobsFeedCardModuleUrn"},"footerUnion":{"type":{"union":{"navigationFooter":"com.linkedin.5de46613f3027e5a41b356871519a78b"}}},"entitiesResolutionResults":{"resolvedFrom":"entities","type":{"array":{"union":{"promotionalCard":"com.linkedin.c4e907da1ff053ca48ebe32c32f78805","endOfResultsCard":"com.linkedin.c99e83ae287631a7e9fc1ff222ab73bb","jobPostingCardWrapper":"com.linkedin.81315c1c0ea01a8f7161ae78bbbd0ff1","jobSearchHistoryCard":"com.linkedin.d0f6c4718d68ff125ca6f7e0b91f510c","premiumUpsellSlot":"com.linkedin.dc4c6e3940e502e42eb82438140331c7","jobSearchSuggestion":"com.linkedin.1581fb7fcc9bc37fc013106a0e87c905","newCollectionHeaderCard":"com.linkedin.6e88a2e94acf8ea5aa8c9f7051705709","carouselEntityHighlightCard":"com.linkedin.b26a8e722557e805b373f37f9d0ef00a","feedbackCard":"com.linkedin.4ece7b9b6c81c2cdb35c6bf9c8e01cb7","carouselCollectionCard":"com.linkedin.21a4612bc8ff64d01e0062f74b272040","carouselAdvantageCard":"com.linkedin.57812d345916f0da7785a8887e8fe73d","careerEnrichmentCard":"com.linkedin.b2c46ce630f9fc1eb9ed163a1a7b2616","noResultsCard":"com.linkedin.1ed8d64e266be8d739bffff5d4200fb4","seeAllCard":"com.linkedin.653bbb9565e4a43fc63d44ebc167fab1","refreshStateCard":"com.linkedin.aa4a2fb807b219a33654acc9643ff224","jobPostingCard":"com.linkedin.f42293b24e5d60e317d2d68ab7597f64"}}}}},"baseType":"com.linkedin.voyager.dash.jobs.JobsFeedCardModule"},"com.linkedin.b2744b01d95785f8c9ab5f466c1674d5":{"fields":{"commitmentConfig":{"type":"com.linkedin.ef6520077f06821116709dada74ee910"},"commitmentType":{"type":"com.linkedin.voyager.dash.talentbrand.CommitmentType"},"description":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.talentbrand.Commitment"},"com.linkedin.87b6c1610a369328d34d660f0b651ea4":{"fields":{"contentUnions":{"type":{"array":{"union":{"questionRecommendation":"com.linkedin.d77e6926ccb788017c5e7ffe0261e4d7","learningRecommendation":"com.linkedin.59e2a69f3c2d74e9d4b5b8530e816743"}}}},"viewImpressionTrackingKey":{"type":"string"},"entityUrn":{"type":"com.linkedin.voyager.dash.common.InterviewPrepCardUrn"},"description":{"type":"string"},"title":{"type":"string"},"ctaNavigationAction":{"type":"com.linkedin.5de46613f3027e5a41b356871519a78b"},"content":{"derivedFrom":"contentUnions","type":{"array":{"union":{"questionRecommendation":"com.linkedin.d77e6926ccb788017c5e7ffe0261e4d7","learningRecommendation":"com.linkedin.59e2a69f3c2d74e9d4b5b8530e816743"}}}}},"baseType":"com.linkedin.voyager.dash.jobs.InterviewPrepCard"},"com.linkedin.8e2783ee98d67c681658fdc2f0519697":{"fields":{"benefits":{"type":{"array":"string"}},"header":{"type":"string"},"descriptionText":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.jobs.JobPostingBenefitsCard"},"com.linkedin.5f82b47e5488aec975bf096d809fb051":{"fields":{"controlName":{"type":"com.linkedin.voyager.dash.common.tracking.ControlName"},"hintText":{"type":"com.linkedin.390ac1c1431d5b4e340a65af24ed68f2"},"helperText":{"type":"com.linkedin.390ac1c1431d5b4e340a65af24ed68f2"},"validationMetadataResolutionResult":{"derivedFrom":"validationMetadata","type":{"union":{"pronounsV2":"com.linkedin.bfd20979d76e9c8ce942bc03238c6b02","string":"com.linkedin.bfd20979d76e9c8ce942bc03238c6b02","textFormInputType":"com.linkedin.voyager.dash.common.forms.validation.TextFormInputType","postalCode":"com.linkedin.2b6a63a1bb3f7b0bf5c50d8530db9f78","name":"com.linkedin.bfd20979d76e9c8ce942bc03238c6b02","pronouns":"string","integer":"com.linkedin.8409c9296c5bda7be8aee44feff66dd3","decimal":"com.linkedin.8409c9296c5bda7be8aee44feff66dd3","headline":"com.linkedin.bfd20979d76e9c8ce942bc03238c6b02","url":"com.linkedin.48be3e5c4d0dfb61ba0a4a296e719909"}}},"validationMetadata":{"type":{"union":{"pronounsV2":"com.linkedin.bfd20979d76e9c8ce942bc03238c6b02","string":"com.linkedin.bfd20979d76e9c8ce942bc03238c6b02","textFormInputType":"com.linkedin.voyager.dash.common.forms.validation.TextFormInputType","postalCode":"com.linkedin.2b6a63a1bb3f7b0bf5c50d8530db9f78","name":"com.linkedin.bfd20979d76e9c8ce942bc03238c6b02","pronouns":"string","integer":"com.linkedin.8409c9296c5bda7be8aee44feff66dd3","decimal":"com.linkedin.8409c9296c5bda7be8aee44feff66dd3","headline":"com.linkedin.bfd20979d76e9c8ce942bc03238c6b02","url":"com.linkedin.48be3e5c4d0dfb61ba0a4a296e719909"}}}},"baseType":"com.linkedin.voyager.dash.common.forms.SingleLineTextFormComponent"},"com.linkedin.5fe8101cb3a914392f7301964e56573c":{"fields":{"companyUrn":{"type":"string"},"company":{"resolvedFrom":"companyUrn","type":"com.linkedin.a2ef5c1c8d1bd7b83de7009b834a38c6"},"vectorImage":{"type":"com.linkedin.b8949adeaaa915212d0136c4b9a1952b"}},"baseType":"com.linkedin.voyager.dash.common.image.NonEntityCompanyLogo"},"com.linkedin.984a5b2fd75109bc59c5241e98cb46ac":{"fields":{"type":{"type":"com.linkedin.voyager.dash.common.text.ListStyleType"},"index":{"type":"int"}},"baseType":"com.linkedin.voyager.dash.common.text.ListItemStyle"},"com.linkedin.6966477af81c9237d13230a0dced17c7":{"fields":{"showPremiumBranding":{"type":"boolean"},"headerContent":{"type":"string"},"howYouMatchSectionUnions":{"type":{"array":{"union":{"qualificationSection":"com.linkedin.c138fd2c1944eec6c90de7592dd91018","premiumUpsellSection":"com.linkedin.3a31504449fad28c205c97d4985ea256","itemsMatchSection":"com.linkedin.427074b29d6e63261a7a2174b0df610b"}}}},"headerImage":{"type":"com.linkedin.voyager.dash.common.SystemImageName"},"entityUrn":{"type":"com.linkedin.voyager.dash.common.HowYouMatchCardUrn"},"headerTip":{"type":"com.linkedin.00ac75ef8dae327ab29a045438e7d186"},"howYouMatchSection":{"derivedFrom":"howYouMatchSectionUnions","type":{"array":{"union":{"qualificationSection":"com.linkedin.c138fd2c1944eec6c90de7592dd91018","premiumUpsellSection":"com.linkedin.3a31504449fad28c205c97d4985ea256","itemsMatchSection":"com.linkedin.427074b29d6e63261a7a2174b0df610b"}}}},"headerTitle":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.jobs.HowYouMatchCard"},"com.linkedin.4f63adbedb57590490ae8c38e3318a15":{"fields":{"memberStatusDescription":{"type":"com.linkedin.390ac1c1431d5b4e340a65af24ed68f2"},"standardizedSkillUrn":{"type":"string"},"entityUrn":{"type":"com.linkedin.voyager.dash.common.SkillAssessmentCardUrn"},"memberStatus":{"type":"com.linkedin.voyager.dash.jobs.SkillAssessmentMemberStatus"},"attemptReport":{"resolvedFrom":"attemptReportUrn","type":"com.linkedin.4c9ae0bb91144f3556c7bcc9bd1b33bd"},"attemptReportUrn":{"type":"string"},"standardizedSkill":{"resolvedFrom":"standardizedSkillUrn","type":"com.linkedin.7a3dfe06b5ebab0041d8cea76d58565a"}},"baseType":"com.linkedin.voyager.dash.jobs.SkillAssessmentCard"},"com.linkedin.82a950002c26f79d2a63a8f699ce13ca":{"fields":{"selectableOptions":{"type":{"array":{"union":{"textSelectableOption":"com.linkedin.98d89bae4aa4ef8f3ffc1018ca93419c","entitySelectableOption":"com.linkedin.9013fa15f12f02c028d5b08f542f32d5"}}}},"textSelectableOptions":{"type":{"array":"com.linkedin.98d89bae4aa4ef8f3ffc1018ca93419c"}},"selectableOptionsResolutionResults":{"derivedFrom":"selectableOptions","type":{"array":{"union":{"textSelectableOption":"com.linkedin.98d89bae4aa4ef8f3ffc1018ca93419c","entitySelectableOption":"com.linkedin.9013fa15f12f02c028d5b08f542f32d5"}}}}},"baseType":"com.linkedin.voyager.dash.common.forms.RadioButtonFormComponent"},"com.linkedin.d1ea2131164c2418a986748ae0e5000a":{"fields":{"typeaheadFilterQuery":{"type":"com.linkedin.21042ae01d5b4a25985f74505d1df54c"},"showFullLastNameForConnections":{"type":"boolean"},"typeaheadUseCase":{"type":"com.linkedin.voyager.dash.search.TypeaheadUseCase"}},"baseType":"com.linkedin.voyager.dash.search.TypeaheadQuery"},"com.linkedin.fe3639bb2accd8987c7e8ed7912c8eff":{"fields":{"typeaheadFormSuggestionViewModel":{"type":"com.linkedin.45e3a7da0d85a4cd9d5aaa94b34dd17d"},"controlName":{"type":"com.linkedin.voyager.dash.common.tracking.ControlName"},"typeaheadType":{"type":"com.linkedin.voyager.dash.search.TypeaheadType"},"contextualSuggestionQueryParameterUrns":{"type":{"array":"com.linkedin.common.Urn"}},"helperText":{"type":"com.linkedin.390ac1c1431d5b4e340a65af24ed68f2"},"entityImage":{"type":"com.linkedin.30e1ec6a491035bd7a31cc9de8a12c18"},"hintText":{"type":"com.linkedin.390ac1c1431d5b4e340a65af24ed68f2"},"typeaheadQueryContext":{"type":{"array":"string"}},"freeFormTextNotAllowedErrorText":{"type":"com.linkedin.390ac1c1431d5b4e340a65af24ed68f2"},"entityName":{"type":"com.linkedin.390ac1c1431d5b4e340a65af24ed68f2"},"validationMetadata":{"derivedFrom":"validationMetadataUnion","type":{"union":{"pronounsV2":"com.linkedin.bfd20979d76e9c8ce942bc03238c6b02","string":"com.linkedin.bfd20979d76e9c8ce942bc03238c6b02","textFormInputType":"com.linkedin.voyager.dash.common.forms.validation.TextFormInputType","postalCode":"com.linkedin.2b6a63a1bb3f7b0bf5c50d8530db9f78","name":"com.linkedin.bfd20979d76e9c8ce942bc03238c6b02","pronouns":"string","integer":"com.linkedin.8409c9296c5bda7be8aee44feff66dd3","decimal":"com.linkedin.8409c9296c5bda7be8aee44feff66dd3","headline":"com.linkedin.bfd20979d76e9c8ce942bc03238c6b02","url":"com.linkedin.48be3e5c4d0dfb61ba0a4a296e719909"}}},"validationMetadataUnion":{"type":{"union":{"pronounsV2":"com.linkedin.bfd20979d76e9c8ce942bc03238c6b02","string":"com.linkedin.bfd20979d76e9c8ce942bc03238c6b02","textFormInputType":"com.linkedin.voyager.dash.common.forms.validation.TextFormInputType","postalCode":"com.linkedin.2b6a63a1bb3f7b0bf5c50d8530db9f78","name":"com.linkedin.bfd20979d76e9c8ce942bc03238c6b02","pronouns":"string","integer":"com.linkedin.8409c9296c5bda7be8aee44feff66dd3","decimal":"com.linkedin.8409c9296c5bda7be8aee44feff66dd3","headline":"com.linkedin.bfd20979d76e9c8ce942bc03238c6b02","url":"com.linkedin.48be3e5c4d0dfb61ba0a4a296e719909"}}},"freeFormTextAllowed":{"type":"boolean"},"typeaheadMetadata":{"type":"com.linkedin.2dea04d4082c6960578ac30a3ace5ee0"},"ghostImage":{"type":"com.linkedin.30e1ec6a491035bd7a31cc9de8a12c18"}},"baseType":"com.linkedin.voyager.dash.common.forms.SingleTypeaheadEntityFormComponent"},"com.linkedin.756c19defef3189024bfe8e3e74d83ab":{"fields":{"followeeCount":{"type":"long"},"entityUrn":{"type":"com.linkedin.voyager.dash.common.FollowingStateUrn"},"preDashFollowingInfoUrn":{"type":"com.linkedin.voyager.dash.common.PreDashUrn"},"trackingUrn":{"type":"com.linkedin.common.Urn"},"following":{"type":"boolean"},"followingType":{"type":"com.linkedin.voyager.dash.feed.FollowingType"},"followerCount":{"type":"long"}},"baseType":"com.linkedin.voyager.dash.feed.FollowingState"},"com.linkedin.0030135583a12944453eaad4d3729402":{"fields":{"entityUrn":{"type":"com.linkedin.voyager.dash.common.ConnectionUrn"}},"baseType":"com.linkedin.voyager.dash.relationships.Connection"},"com.linkedin.2db94c9a357c51aebecf0054e59f9bd7":{"fields":{"recommendedWorkplaceType":{"resolvedFrom":"recommendedWorkplaceTypeUrns","type":{"array":"com.linkedin.79cb580711dcd3cf7d83e724b3d9e260"}},"existingJobAlert":{"resolvedFrom":"existingJobAlertUrn","type":"com.linkedin.272e46084795068d9243a5e3978fc2e2"},"recommendedGeoUrn":{"type":"string"},"existingJobAlertUrn":{"type":"string"},"recommendedGeo":{"resolvedFrom":"recommendedGeoUrn","type":"com.linkedin.00c4f529ff2fa302d89eaea66344f268"},"recommendedKeywords":{"type":"string"},"recommendedWorkplaceTypeUrns":{"type":{"array":"string"}},"newSearchResultsCount":{"type":"int"},"eligibleToCreate":{"type":"boolean"}},"baseType":"com.linkedin.voyager.dash.jobs.JobAlertCreateEligibility"},"com.linkedin.86ff9fd16c75bc659bfb6287b8ec1647":{"fields":{"elements":{"type":{"array":"com.linkedin.d9052d9e22b039f19b4d6484e2ebec27"}}},"baseType":"com.linkedin.restli.common.CollectionResponse"},"com.linkedin.0054dfa46558041e61c38c995b4809c8":{"fields":{"navigateToReviewLearnMoreModal":{"type":"boolean"},"bannerText":{"type":"string"},"reviewSla":{"type":"int"}},"baseType":"com.linkedin.voyager.dash.jobs.JobPostingTrustDetail"},"com.linkedin.6a325d9bf5f44baf8fede1db5a7ce66f":{"fields":{"subtitle":{"type":"com.linkedin.390ac1c1431d5b4e340a65af24ed68f2"},"spacingType":{"type":"com.linkedin.voyager.dash.common.ux.SpacingType"},"navigationButton":{"type":"com.linkedin.72471cc7e55a55b0fd549127127633f7"},"title":{"type":"string"},"horizontalOrientation":{"type":"boolean"},"formElements":{"type":{"array":"com.linkedin.258b7479f7c0999032ce22032fa02cdf"}},"visibilitySettingButton":{"type":"com.linkedin.b4d79116ccce85e0d4ef36bf211da0a3"}},"baseType":"com.linkedin.voyager.dash.common.forms.FormElementGroup"},"com.linkedin.2ef5b25d61e5db956d269b94a55f4763":{"fields":{"originalImageUrl":{"type":"com.linkedin.common.Url"},"originalHeight":{"type":"int"},"originalWidth":{"type":"int"},"url":{"type":"com.linkedin.common.Url"}},"baseType":"com.linkedin.voyager.dash.common.ImageUrl"},"com.linkedin.b0ab2e740be6873e75f413db4e533bb9":{"fields":{"name":{"type":"string"},"entityUrn":{"type":"com.linkedin.voyager.dash.common.DegreeUrn"}},"baseType":"com.linkedin.voyager.dash.identity.profile.StandardizedDegree"},"com.linkedin.513522081428feabe67539f69de00fc6":{"fields":{"elements":{"type":{"array":"com.linkedin.dc4c6e3940e502e42eb82438140331c7"}}},"baseType":"com.linkedin.restli.common.CollectionResponse"},"com.linkedin.4a3588cb3f298939a6ef5b2e61cadc18":{"fields":{"colorStyle":{"type":"com.linkedin.voyager.dash.common.ux.label.LabelColorStyle"},"text":{"type":"com.linkedin.390ac1c1431d5b4e340a65af24ed68f2"}},"baseType":"com.linkedin.voyager.dash.common.ux.label.LabelViewModel"},"com.linkedin.d4dcbabbf47d9675b0597099d9cd7d69":{"fields":{"name":{"type":"string"},"logo":{"type":{"union":{"url":"com.linkedin.common.Url","vectorImage":"com.linkedin.b8949adeaaa915212d0136c4b9a1952b"}}},"logoResolutionResult":{"derivedFrom":"logo","type":{"union":{"url":"com.linkedin.common.Url","vectorImage":"com.linkedin.b8949adeaaa915212d0136c4b9a1952b"}}},"entityUrn":{"type":"com.linkedin.voyager.dash.common.SchoolUrn"}},"baseType":"com.linkedin.voyager.dash.organization.School"},"com.linkedin.863cd9abd8abab984b4347da5429d447":{"fields":{"cardType":{"type":"com.linkedin.voyager.dash.jobs.JobActivityCardType"},"companyApplyUrl":{"type":"com.linkedin.common.Url"},"entityUrn":{"type":"com.linkedin.voyager.dash.common.JobActivityCardUrn"},"activities":{"type":{"array":"com.linkedin.72617837b227d77ce48feb965f1ecc1b"}}},"baseType":"com.linkedin.voyager.dash.jobs.JobActivityCard"},"com.linkedin.258b7479f7c0999032ce22032fa02cdf":{"fields":{"prerequisiteInputEvaluationStrategyUnion":{"type":{"union":{"answered":"com.linkedin.common.Urn","matchAnyPrerequisiteInput":"com.linkedin.594a52f4e72625c921bc156c950ef9e3","notMatchAnyPrequisiteInput":"com.linkedin.594a52f4e72625c921bc156c950ef9e3"}}},"formComponent":{"type":{"union":{"multilineTextFormComponent":"com.linkedin.22e0be7fe535e4176ecde39607391326","textEntityListFormComponent":"com.linkedin.310d48bff502026d1fdc80b24448401d","toggleFormComponent":"com.linkedin.b0ac2d5e2d9d20cb81b800352f82dc8e","starRatingFormComponent":"com.linkedin.bfb56f758ba39329fbb4081cfe291d5e","nestedCheckboxFormComponent":"com.linkedin.b1148482c3f256b956d443d3fbab2904","mediaUploadFormComponent":"com.linkedin.be19a4b9083f30fb8796eb20ea30c1d0","checkboxFormComponent":"com.linkedin.90840606293e76f91e739cc9fbd0c304","dropdownFormComponent":"com.linkedin.6ba053f5dc7e14e40216570d97a71e52","singleTypeaheadEntityFormComponent":"com.linkedin.fe3639bb2accd8987c7e8ed7912c8eff","numberInputFormComponent":"com.linkedin.274132557f0c6d4e240f7372d489e1cc","dateFormComponent":"com.linkedin.aa89cadbee726b8d3c62475faddcf670","dateRangeFormComponent":"com.linkedin.c734e1c934a3cbf9562953df54d4e771","multiSelectTypeaheadEntityFormComponent":"com.linkedin.985210b44c74343f912e35f90d721799","locationFormComponent":"com.linkedin.60257368060941d7900dbbf2748f711c","singleLineTextFormComponent":"com.linkedin.5f82b47e5488aec975bf096d809fb051","radioButtonFormComponent":"com.linkedin.82a950002c26f79d2a63a8f699ce13ca","pillFormComponent":"com.linkedin.9774b5e7450ba6db34185afe39f9e516"}}},"prerequisiteFormElementInputs":{"type":{"array":"com.linkedin.81235c3224517a02609a065763927f01"}},"requiredFieldMissingErrorText":{"type":"com.linkedin.390ac1c1431d5b4e340a65af24ed68f2"},"weight":{"type":"float"},"title":{"type":"com.linkedin.390ac1c1431d5b4e340a65af24ed68f2"},"prerequisiteInputEvaluationStrategy":{"derivedFrom":"prerequisiteInputEvaluationStrategyUnion","type":{"union":{"answered":"com.linkedin.common.Urn","matchAnyPrerequisiteInput":"com.linkedin.594a52f4e72625c921bc156c950ef9e3","notMatchAnyPrequisiteInput":"com.linkedin.594a52f4e72625c921bc156c950ef9e3"}}},"required":{"type":"boolean"},"shouldAlwaysSendBackFormElementInput":{"type":"boolean"},"urn":{"type":"com.linkedin.common.Urn"},"input":{"type":"com.linkedin.81235c3224517a02609a065763927f01"},"entityUrn":{"type":"com.linkedin.voyager.dash.common.FormElementUrn"},"formComponentResolutionResult":{"derivedFrom":"formComponent","type":{"union":{"multilineTextFormComponent":"com.linkedin.22e0be7fe535e4176ecde39607391326","textEntityListFormComponent":"com.linkedin.310d48bff502026d1fdc80b24448401d","toggleFormComponent":"com.linkedin.b0ac2d5e2d9d20cb81b800352f82dc8e","starRatingFormComponent":"com.linkedin.bfb56f758ba39329fbb4081cfe291d5e","nestedCheckboxFormComponent":"com.linkedin.b1148482c3f256b956d443d3fbab2904","mediaUploadFormComponent":"com.linkedin.be19a4b9083f30fb8796eb20ea30c1d0","checkboxFormComponent":"com.linkedin.90840606293e76f91e739cc9fbd0c304","dropdownFormComponent":"com.linkedin.6ba053f5dc7e14e40216570d97a71e52","singleTypeaheadEntityFormComponent":"com.linkedin.fe3639bb2accd8987c7e8ed7912c8eff","numberInputFormComponent":"com.linkedin.274132557f0c6d4e240f7372d489e1cc","dateFormComponent":"com.linkedin.aa89cadbee726b8d3c62475faddcf670","dateRangeFormComponent":"com.linkedin.c734e1c934a3cbf9562953df54d4e771","multiSelectTypeaheadEntityFormComponent":"com.linkedin.985210b44c74343f912e35f90d721799","locationFormComponent":"com.linkedin.60257368060941d7900dbbf2748f711c","singleLineTextFormComponent":"com.linkedin.5f82b47e5488aec975bf096d809fb051","radioButtonFormComponent":"com.linkedin.82a950002c26f79d2a63a8f699ce13ca","pillFormComponent":"com.linkedin.9774b5e7450ba6db34185afe39f9e516"}}},"helperLink":{"type":"com.linkedin.390ac1c1431d5b4e340a65af24ed68f2"}},"baseType":"com.linkedin.voyager.dash.common.forms.FormElement"},"com.linkedin.f1d8ff8a5c9d0a2c73d8c4418e3a4526":{"fields":{"elements":{"type":{"array":"com.linkedin.180bad7e2024ccff1ee0157be65c0dd1"}}},"baseType":"com.linkedin.restli.common.CollectionResponse"},"com.linkedin.ef6520077f06821116709dada74ee910":{"fields":{"commitmentDisplayText":{"type":"com.linkedin.390ac1c1431d5b4e340a65af24ed68f2"}},"baseType":"com.linkedin.voyager.dash.talentbrand.CommitmentConfig"},"com.linkedin.44e816344c2289e8a62e33486601f239":{"fields":{"dismissControlName":{"type":"com.linkedin.voyager.dash.common.tracking.ControlName"},"viewImpressionTrackingKey":{"type":"string"},"jobSeekerActionCardUnion":{"type":{"union":{"jobSeekerFormActionCard":"com.linkedin.6143491c72aa6f2fce542169c8baa1d6"}}},"jobSeekerActionCard":{"derivedFrom":"jobSeekerActionCardUnion","type":{"union":{"jobSeekerFormActionCard":"com.linkedin.6143491c72aa6f2fce542169c8baa1d6"}}}},"baseType":"com.linkedin.voyager.dash.jobs.JobSeekerActionCard"},"com.linkedin.dc4c6e3940e502e42eb82438140331c7":{"fields":{"entityUrn":{"type":"com.linkedin.voyager.dash.common.PremiumUpsellSlotUrn"},"upsellCard":{"type":"com.linkedin.d9052d9e22b039f19b4d6484e2ebec27"}},"baseType":"com.linkedin.voyager.dash.premium.PremiumUpsellSlotContent"},"com.linkedin.146c92f10032ac18e61e96a3f143b9f1":{"fields":{"urn":{"type":"com.linkedin.common.Urn"},"modelName":{"type":"string"},"fieldName":{"type":"string"},"value":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.common.StringFieldReference"},"com.linkedin.6148d310783eaffead8570485d05a156":{"fields":{"start":{"type":"int"},"end":{"type":"int"}},"baseType":"com.linkedin.common.IntegerRange"},"com.linkedin.34436f930667efbac1e7e0a600c8c653":{"fields":{"title":{"type":"com.linkedin.390ac1c1431d5b4e340a65af24ed68f2"},"organizationRankingDetails":{"type":{"array":"com.linkedin.b7cc21947d4a1ba974040324a832ba64"}}},"baseType":"com.linkedin.voyager.dash.premium.companyinsights.CompanyInsightsOrganizationRanking"},"com.linkedin.9013fa15f12f02c028d5b08f542f32d5":{"fields":{"optionUrn":{"type":"com.linkedin.common.Urn"},"entityImage":{"type":"com.linkedin.30e1ec6a491035bd7a31cc9de8a12c18"},"dismissable":{"type":"boolean"},"shouldRenderTextBeforeImage":{"type":"boolean"},"subtitle":{"type":"com.linkedin.390ac1c1431d5b4e340a65af24ed68f2"},"optionUnion":{"type":{"union":{"optionUrn":"com.linkedin.common.Urn","optionEnumString":"string"}}},"controlName":{"type":"com.linkedin.voyager.dash.common.tracking.ControlName"},"title":{"type":"com.linkedin.390ac1c1431d5b4e340a65af24ed68f2"},"option":{"derivedFrom":"optionUnion","type":{"union":{"optionUrn":"com.linkedin.common.Urn","optionEnumString":"string"}}}},"baseType":"com.linkedin.voyager.dash.common.forms.EntitySelectableOption"},"com.linkedin.b2c46ce630f9fc1eb9ed163a1a7b2616":{"fields":{},"baseType":"com.linkedin.voyager.dash.jobs.CareerEnrichmentCard"},"com.linkedin.392fa039af72d21e485244574f20ca3b":{"fields":{"image":{"type":"com.linkedin.30e1ec6a491035bd7a31cc9de8a12c18"},"titleInsight":{"type":"com.linkedin.390ac1c1431d5b4e340a65af24ed68f2"},"composeOptionControlName":{"type":"com.linkedin.voyager.dash.common.tracking.ControlName"},"navigationUrl":{"type":"com.linkedin.common.Url"},"title":{"type":"com.linkedin.390ac1c1431d5b4e340a65af24ed68f2"},"primaryCTA":{"type":"com.linkedin.5de46613f3027e5a41b356871519a78b"},"composeOption":{"resolvedFrom":"composeOptionUrn","type":"com.linkedin.91ef6d45f2b779a5fad235f8f9dccdb0"},"primaryInsight":{"type":"com.linkedin.390ac1c1431d5b4e340a65af24ed68f2"},"navigationAction":{"type":"com.linkedin.5de46613f3027e5a41b356871519a78b"},"accessibilityText":{"type":"string"},"localizedPrefilledMessageSubject":{"type":"string"},"secondaryInsight":{"type":"com.linkedin.390ac1c1431d5b4e340a65af24ed68f2"},"composeOptionUrn":{"type":"string"},"showPremiumBar":{"type":"boolean"},"subtitle":{"type":"com.linkedin.390ac1c1431d5b4e340a65af24ed68f2"},"localizedPrefilledMessageBody":{"type":"string"},"headline":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.jobs.HiringTeamCard"},"com.linkedin.bd66bae736eabe6505d1b80c182e31af":{"fields":{"image":{"type":"com.linkedin.30e1ec6a491035bd7a31cc9de8a12c18"},"trackingUrn":{"type":"com.linkedin.common.Urn"},"subtitle":{"type":"com.linkedin.390ac1c1431d5b4e340a65af24ed68f2"},"caption":{"type":"com.linkedin.390ac1c1431d5b4e340a65af24ed68f2"},"navigationUrl":{"type":"com.linkedin.common.Url"},"label":{"type":"com.linkedin.390ac1c1431d5b4e340a65af24ed68f2"},"title":{"type":"com.linkedin.390ac1c1431d5b4e340a65af24ed68f2"},"trackingId":{"type":"com.linkedin.voyager.dash.common.Base64String"}},"baseType":"com.linkedin.voyager.dash.common.ux.EntityLockupViewModel"},"com.linkedin.21042ae01d5b4a25985f74505d1df54c":{"fields":{"geoSearchTypes":{"type":{"array":"com.linkedin.voyager.dash.search.GeoSearchType"}},"countryCodes":{"type":{"array":"string"}},"excludedSkill":{"resolvedFrom":"excludedSkillUrns","type":{"array":"com.linkedin.7a3dfe06b5ebab0041d8cea76d58565a"}},"countryCodesResolutionResults":{"resolvedFrom":"countryCodes","type":{"array":"com.linkedin.00c4f529ff2fa302d89eaea66344f268"}},"company":{"resolvedFrom":"companyUrns","type":{"array":"com.linkedin.a2ef5c1c8d1bd7b83de7009b834a38c6"}},"excludedSkillUrns":{"type":{"array":"string"}},"standardizationEntityType":{"type":"string"},"groupUrn":{"type":"string"},"companyUrns":{"type":{"array":"string"}},"group":{"resolvedFrom":"groupUrn","type":"com.linkedin.32bab5d7af29a8bdf4fe920d7d2155b1"}},"baseType":"com.linkedin.voyager.dash.search.TypeaheadFilterQuery"}}}},"included":[{"entityUrn":"urn:li:fsd_jobSkillMatchInsight:3768592532","$recipeTypes":["com.linkedin.65d9b685ce8d8355ba025fb1ae135598"],"insightSummary":{"image":{"attributes":[{"scalingType":null,"detailData":{"profilePictureWithoutFrame":null,"profilePictureWithRingStatus":null,"companyLogo":null,"icon":null,"systemImage":"SYS_ICN_CHECKLIST_MEDIUM","nonEntityGroupLogo":null,"vectorImage":null,"nonEntityProfessionalEventLogo":null,"profilePicture":null,"imageUrl":null,"professionalEventLogo":null,"nonEntityCompanyLogo":null,"nonEntitySchoolLogo":null,"groupLogo":null,"schoolLogo":null,"ghostImage":null,"nonEntityProfilePicture":null},"tintColor":null,"$recipeTypes":["com.linkedin.296fd4ebc3f7a46ccca548d534f656e4"],"tapTargets":[],"displayAspectRatio":null,"$type":"com.linkedin.voyager.dash.common.image.ImageAttribute"}],"editableAccessibilityText":false,"actionTarget":null,"accessibilityTextAttributes":[],"totalCount":null,"accessibilityText":null,"$recipeTypes":["com.linkedin.30e1ec6a491035bd7a31cc9de8a12c18"],"$type":"com.linkedin.voyager.dash.common.image.ImageViewModel"},"text":{"textDirection":"USER_LOCALE","text":"Skills: Data Analytics, Data Analysis, +8 more","attributesV2":[{"start":0,"length":46,"detailData":{"jobPostingName":null,"hyperlink":"#HYM","profileFamiliarName":null,"color":null,"companyName":null,"icon":null,"epoch":null,"systemImage":null,"textLink":null,"listItemStyle":null,"groupName":null,"hyperlinkOpenExternally":null,"listStyle":null,"profileFullName":null,"stringFieldReference":null,"learningCourseName":null,"profileMention":null,"style":null,"schoolName":null,"hashtag":null},"$recipeTypes":["com.linkedin.b0a683a611d43de435ed53c6232d8d44"],"$type":"com.linkedin.voyager.dash.common.text.TextAttribute"}],"accessibilityTextAttributesV2":[],"accessibilityText":null,"$recipeTypes":["com.linkedin.390ac1c1431d5b4e340a65af24ed68f2"],"$type":"com.linkedin.voyager.dash.common.text.TextViewModel"},"$recipeTypes":["com.linkedin.3551030d48734b4fd9cb3238efc77ece"],"$type":"com.linkedin.voyager.dash.common.ux.InsightViewModel"},"$type":"com.linkedin.voyager.dash.assessments.JobSkillMatchInsight"},{"entityUrn":"urn:li:fsd_geo:101282230","countryISOCode":"DE","abbreviatedLocalizedName":"Germany","$recipeTypes":["com.linkedin.00c4f529ff2fa302d89eaea66344f268"],"defaultLocalizedName":"Germany","$type":"com.linkedin.voyager.dash.common.Geo"},{"saved":false,"entityUrn":"urn:li:fsd_saveState:(SAVE,urn:li:fsd_jobPosting:3768592532)","preDashEntityUrn":"urn:li:fs_saveAction:(SAVE,urn:li:fsd_jobPosting:3768592532)","$recipeTypes":["com.linkedin.9731877e56bf195f5714e9f273590184"],"$type":"com.linkedin.voyager.dash.feed.SaveState"},{"companyDetails":{"$recipeTypes":["com.linkedin.b97672681af5b861a5ec5c53b8fcbc96"],"jobCompany":{"rawCompanyName":null,"*company":"urn:li:fsd_company:43338549"},"$type":"com.linkedin.voyager.dash.jobs.JobPostingCompany"},"jobState":"LISTED","trackingUrn":"urn:li:jobPosting:3768592532","*jobWorkplaceTypes":["urn:li:fsd_workplaceType:2"],"repostedJob":false,"trustReviewDecision":null,"title":"Sustainability Data Analyst (f/m/x)","$recipeTypes":["com.linkedin.f8ea4e1538764dc27e403a1d55047e51"],"posterId":null,"$type":"com.linkedin.voyager.dash.jobs.JobPosting","createdAt":1700948520000,"contentSource":"JOBS_CREATE","entityUrn":"urn:li:fsd_jobPosting:3768592532","*location":"urn:li:fsd_geo:101282230","*workplaceTypesResolutionResults":["urn:li:fsd_workplaceType:2"],"jobDetailsSummaryCard":{"*elements":["urn:li:fsd_premiumUpsellSlot:JOB_DETAILS_SUMMARY_CARD"],"$recipeTypes":["com.linkedin.513522081428feabe67539f69de00fc6"],"$type":"com.linkedin.restli.common.CollectionResponse"}},{"preDashNormalizedJobPostingUrn":"urn:li:fs_normalized_jobPosting:3768592532","jobPostingTitle":"Sustainability Data Analyst (f/m/x)","*coachJobsKickoffPrompts":"urn:li:fsd_coachJobsKickoffPrompts:(3768592532,JOB_DETAILS)","navigationBarSubtitle":"Cozero · Germany (Remote)","primaryDescription":{"textDirection":"USER_LOCALE","text":"Cozero · Germany   22 hours ago · 71 applicants","attributesV2":[{"start":0,"length":6,"detailData":{"jobPostingName":null,"hyperlink":"https://www.linkedin.com/company/cozero-io/life","profileFamiliarName":null,"color":null,"companyName":null,"icon":null,"epoch":null,"systemImage":null,"textLink":null,"listItemStyle":null,"groupName":null,"hyperlinkOpenExternally":null,"listStyle":null,"profileFullName":null,"stringFieldReference":null,"learningCourseName":null,"profileMention":null,"style":null,"schoolName":null,"hashtag":null},"$recipeTypes":["com.linkedin.b0a683a611d43de435ed53c6232d8d44"],"$type":"com.linkedin.voyager.dash.common.text.TextAttribute"},{"start":19,"length":12,"detailData":{"jobPostingName":null,"hyperlink":null,"profileFamiliarName":null,"color":"POSITIVE","companyName":null,"icon":null,"epoch":null,"systemImage":null,"textLink":null,"listItemStyle":null,"groupName":null,"hyperlinkOpenExternally":null,"listStyle":null,"profileFullName":null,"stringFieldReference":null,"learningCourseName":null,"profileMention":null,"style":null,"schoolName":null,"hashtag":null},"$recipeTypes":["com.linkedin.b0a683a611d43de435ed53c6232d8d44"],"$type":"com.linkedin.voyager.dash.common.text.TextAttribute"},{"start":19,"length":12,"detailData":{"jobPostingName":null,"hyperlink":null,"profileFamiliarName":null,"color":null,"companyName":null,"icon":null,"epoch":null,"systemImage":null,"textLink":null,"listItemStyle":null,"groupName":null,"hyperlinkOpenExternally":null,"listStyle":null,"profileFullName":null,"stringFieldReference":null,"learningCourseName":null,"profileMention":null,"style":"BOLD","schoolName":null,"hashtag":null},"$recipeTypes":["com.linkedin.b0a683a611d43de435ed53c6232d8d44"],"$type":"com.linkedin.voyager.dash.common.text.TextAttribute"},{"start":19,"length":12,"detailData":{"jobPostingName":null,"hyperlink":null,"profileFamiliarName":null,"color":null,"companyName":null,"icon":null,"epoch":{"epochAt":1700948520000,"type":"TIME_AGO","$recipeTypes":["com.linkedin.94721090e597af7c97503cb8b1a7e50f"],"$type":"com.linkedin.voyager.dash.common.text.EpochTime"},"systemImage":null,"textLink":null,"listItemStyle":null,"groupName":null,"hyperlinkOpenExternally":null,"listStyle":null,"profileFullName":null,"stringFieldReference":null,"learningCourseName":null,"profileMention":null,"style":null,"schoolName":null,"hashtag":null},"$recipeTypes":["com.linkedin.b0a683a611d43de435ed53c6232d8d44"],"$type":"com.linkedin.voyager.dash.common.text.TextAttribute"},{"start":31,"length":3,"detailData":{"jobPostingName":null,"hyperlink":null,"profileFamiliarName":null,"color":"NEUTRAL","companyName":null,"icon":null,"epoch":null,"systemImage":null,"textLink":null,"listItemStyle":null,"groupName":null,"hyperlinkOpenExternally":null,"listStyle":null,"profileFullName":null,"stringFieldReference":null,"learningCourseName":null,"profileMention":null,"style":null,"schoolName":null,"hashtag":null},"$recipeTypes":["com.linkedin.b0a683a611d43de435ed53c6232d8d44"],"$type":"com.linkedin.voyager.dash.common.text.TextAttribute"},{"start":34,"length":13,"detailData":{"jobPostingName":null,"hyperlink":null,"profileFamiliarName":null,"color":"NEUTRAL","companyName":null,"icon":null,"epoch":null,"systemImage":null,"textLink":null,"listItemStyle":null,"groupName":null,"hyperlinkOpenExternally":null,"listStyle":null,"profileFullName":null,"stringFieldReference":null,"learningCourseName":null,"profileMention":null,"style":null,"schoolName":null,"hashtag":null},"$recipeTypes":["com.linkedin.b0a683a611d43de435ed53c6232d8d44"],"$type":"com.linkedin.voyager.dash.common.text.TextAttribute"}],"accessibilityTextAttributesV2":[],"accessibilityText":null,"$recipeTypes":["com.linkedin.390ac1c1431d5b4e340a65af24ed68f2"],"$type":"com.linkedin.voyager.dash.common.text.TextViewModel"},"jobInsightsV2ResolutionResults":[{"premiumUpsellCard":null,"jobInsightViewModel":{"image":{"attributes":[{"scalingType":null,"detailData":{"profilePictureWithoutFrame":null,"profilePictureWithRingStatus":null,"companyLogo":null,"icon":"IC_BRIEFCASE_24DP","systemImage":null,"nonEntityGroupLogo":null,"vectorImage":null,"nonEntityProfessionalEventLogo":null,"profilePicture":null,"imageUrl":null,"professionalEventLogo":null,"nonEntityCompanyLogo":null,"nonEntitySchoolLogo":null,"groupLogo":null,"schoolLogo":null,"ghostImage":null,"nonEntityProfilePicture":null},"tintColor":null,"$recipeTypes":["com.linkedin.296fd4ebc3f7a46ccca548d534f656e4"],"tapTargets":[],"displayAspectRatio":null,"$type":"com.linkedin.voyager.dash.common.image.ImageAttribute"}],"editableAccessibilityText":false,"actionTarget":null,"accessibilityTextAttributes":[],"totalCount":null,"accessibilityText":null,"$recipeTypes":["com.linkedin.30e1ec6a491035bd7a31cc9de8a12c18"],"$type":"com.linkedin.voyager.dash.common.image.ImageViewModel"},"description":[{"text":{"textDirection":"USER_LOCALE","text":"Remote","attributesV2":[],"accessibilityTextAttributesV2":[],"accessibilityText":null,"$recipeTypes":["com.linkedin.390ac1c1431d5b4e340a65af24ed68f2"],"$type":"com.linkedin.voyager.dash.common.text.TextViewModel"},"label":null},{"text":{"textDirection":"USER_LOCALE","text":"Full-time","attributesV2":[],"accessibilityTextAttributesV2":[],"accessibilityText":null,"$recipeTypes":["com.linkedin.390ac1c1431d5b4e340a65af24ed68f2"],"$type":"com.linkedin.voyager.dash.common.text.TextViewModel"},"label":null},{"text":{"textDirection":"USER_LOCALE","text":"Entry level","attributesV2":[],"accessibilityTextAttributesV2":[],"accessibilityText":null,"$recipeTypes":["com.linkedin.390ac1c1431d5b4e340a65af24ed68f2"],"$type":"com.linkedin.voyager.dash.common.text.TextViewModel"},"label":null}],"$recipeTypes":["com.linkedin.efd2ef2c46b7959f85a3d6461edc3e69"],"$type":"com.linkedin.voyager.dash.jobs.JobInsightViewModel"},"insightViewModel":null,"premiumUpsell":null,"skillMatchInsight":null,"verifiedHiringInsight":null,"responsivenessInsightViewModel":null,"trustInsightViewModel":null},{"premiumUpsellCard":null,"jobInsightViewModel":null,"insightViewModel":{"image":{"attributes":[{"scalingType":null,"detailData":{"profilePictureWithoutFrame":null,"profilePictureWithRingStatus":null,"companyLogo":null,"icon":"IC_COMPANY_24DP","systemImage":null,"nonEntityGroupLogo":null,"vectorImage":null,"nonEntityProfessionalEventLogo":null,"profilePicture":null,"imageUrl":null,"professionalEventLogo":null,"nonEntityCompanyLogo":null,"nonEntitySchoolLogo":null,"groupLogo":null,"schoolLogo":null,"ghostImage":null,"nonEntityProfilePicture":null},"tintColor":null,"$recipeTypes":["com.linkedin.296fd4ebc3f7a46ccca548d534f656e4"],"tapTargets":[],"displayAspectRatio":null,"$type":"com.linkedin.voyager.dash.common.image.ImageAttribute"}],"editableAccessibilityText":false,"actionTarget":null,"accessibilityTextAttributes":[],"totalCount":null,"accessibilityText":null,"$recipeTypes":["com.linkedin.30e1ec6a491035bd7a31cc9de8a12c18"],"$type":"com.linkedin.voyager.dash.common.image.ImageViewModel"},"text":{"textDirection":"USER_LOCALE","text":"11-50 employees · Software Development","attributesV2":[],"accessibilityTextAttributesV2":[],"accessibilityText":null,"$recipeTypes":["com.linkedin.390ac1c1431d5b4e340a65af24ed68f2"],"$type":"com.linkedin.voyager.dash.common.text.TextViewModel"},"$recipeTypes":["com.linkedin.3551030d48734b4fd9cb3238efc77ece"],"$type":"com.linkedin.voyager.dash.common.ux.InsightViewModel"},"premiumUpsell":null,"skillMatchInsight":null,"verifiedHiringInsight":null,"responsivenessInsightViewModel":null,"trustInsightViewModel":null},{"premiumUpsellCard":null,"jobInsightViewModel":null,"*premiumUpsell":"urn:li:fsd_jobPosting:3768592532","insightViewModel":null,"skillMatchInsight":null,"verifiedHiringInsight":null,"responsivenessInsightViewModel":null,"trustInsightViewModel":null},{"premiumUpsellCard":null,"jobInsightViewModel":null,"*skillMatchInsight":"urn:li:fsd_jobSkillMatchInsight:3768592532","insightViewModel":null,"premiumUpsell":null,"verifiedHiringInsight":null,"responsivenessInsightViewModel":null,"trustInsightViewModel":null},{"*verifiedHiringInsight":"urn:li:fsd_jobVerifiedHiringInsight:3768592532","premiumUpsellCard":null,"jobInsightViewModel":null,"insightViewModel":null,"premiumUpsell":null,"skillMatchInsight":null,"responsivenessInsightViewModel":null,"trustInsightViewModel":null}],"$recipeTypes":["com.linkedin.f42293b24e5d60e317d2d68ab7597f64"],"primaryActionV2":{"dismissJobAction":null,"applyJobAction":{"applyControlName":"job_details_job_action_apply","$recipeTypes":["com.linkedin.64aff242d842d2f5c0d6ea1432ceb79a"],"$type":"com.linkedin.voyager.dash.jobs.ApplyJobAction","*applyJobActionResolutionResult":"urn:li:fsd_jobSeekerApplicationDetail:3768592532"},"saveJobAction":null},"secondaryActionsV2ResolutionResults":[{"dismissJobAction":null,"applyJobAction":null,"saveJobAction":{"*saveStateResolutionResult":"urn:li:fsd_saveState:(SAVE,urn:li:fsd_jobPosting:3768592532)","saveControlName":"job_details_job_action_save","unsaveControlName":"job_details_job_action_unsave","$recipeTypes":["com.linkedin.c175e2014cb329dc190cefda1f8b1191"],"$type":"com.linkedin.voyager.dash.jobs.SaveJobAction"}}],"$type":"com.linkedin.voyager.dash.jobs.JobPostingCard","secondaryDescription":null,"entityUrn":"urn:li:fsd_jobPostingCard:(3768592532,JOB_DETAILS)","logo":{"attributes":[{"scalingType":null,"detailData":{"profilePictureWithoutFrame":null,"profilePictureWithRingStatus":null,"icon":null,"systemImage":null,"*companyLogo":"urn:li:fsd_company:43338549","nonEntityGroupLogo":null,"vectorImage":null,"nonEntityProfessionalEventLogo":null,"profilePicture":null,"imageUrl":null,"professionalEventLogo":null,"nonEntityCompanyLogo":null,"nonEntitySchoolLogo":null,"groupLogo":null,"schoolLogo":null,"ghostImage":null,"nonEntityProfilePicture":null},"tintColor":null,"$recipeTypes":["com.linkedin.296fd4ebc3f7a46ccca548d534f656e4"],"tapTargets":[],"displayAspectRatio":null,"$type":"com.linkedin.voyager.dash.common.image.ImageAttribute"}],"editableAccessibilityText":false,"actionTarget":"https://www.linkedin.com/company/cozero-io/life","accessibilityTextAttributes":[],"totalCount":null,"accessibilityText":"Cozero logo","$recipeTypes":["com.linkedin.30e1ec6a491035bd7a31cc9de8a12c18"],"$type":"com.linkedin.voyager.dash.common.image.ImageViewModel"},"tip":null,"*jobPosting":"urn:li:fsd_jobPosting:3768592532","tertiaryDescription":null},{"hasConfirmedEmailAddress":true,"applied":false,"applyStartersPreferenceVoid":false,"jobDraftApplicationInfo":null,"formattedApplyDate":null,"$recipeTypes":["com.linkedin.0da643628377886b9fb1a0bb0cc0b73c"],"onsiteApply":false,"$type":"com.linkedin.voyager.dash.jobs.JobSeekerApplicationDetail","applyCtaText":{"textDirection":"USER_LOCALE","text":"Apply","attributesV2":[],"accessibilityTextAttributesV2":[],"accessibilityText":"Apply on company website","$recipeTypes":["com.linkedin.390ac1c1431d5b4e340a65af24ed68f2"],"$type":"com.linkedin.voyager.dash.common.text.TextViewModel"},"entityUrn":"urn:li:fsd_jobSeekerApplicationDetail:3768592532","inPageOffsiteApply":false,"applicantTrackingSystemName":null,"ambryResumeContainer":null,"companyApplyUrl":"https://join.com/companies/cozero/9872937-sustainability-data-analyst-f-m-x?pid=d7cb2e8e859c62cd7f75","appliedAt":null},{"ctaText":null,"entityUrn":"urn:li:fsd_jobVerifiedHiringInsight:3768592532","controlName":"","$recipeTypes":["com.linkedin.ac21e10e9207a4a6523aafa0c51066d4"],"insightSummary":{"image":null,"text":{"textDirection":"USER_LOCALE","attributesV2":[],"text":null,"accessibilityTextAttributesV2":[],"accessibilityText":null,"$recipeTypes":["com.linkedin.390ac1c1431d5b4e340a65af24ed68f2"],"$type":"com.linkedin.voyager.dash.common.text.TextViewModel"},"$recipeTypes":["com.linkedin.3551030d48734b4fd9cb3238efc77ece"],"$type":"com.linkedin.voyager.dash.common.ux.InsightViewModel"},"$type":"com.linkedin.voyager.dash.jobs.JobVerifiedHiringInsight"},{"localizedName":"Remote","entityUrn":"urn:li:fsd_workplaceType:2","workplaceTypeEnum":"REMOTE","localizedDescription":"Employees work off-site.","$recipeTypes":["com.linkedin.79cb580711dcd3cf7d83e724b3d9e260"],"$type":"com.linkedin.voyager.dash.jobs.WorkplaceType"},{"entityUrn":"urn:li:fsd_coachJobsKickoffPrompts:(3768592532,JOB_DETAILS)","$recipeTypes":["com.linkedin.8e19fba94618da7ceb9cca894420a5da"],"kickoffPrompts":[{"queryText":"","controlName":"view_coach","coachSparkleIcon":{"attributes":[{"scalingType":null,"detailData":{"profilePictureWithoutFrame":null,"profilePictureWithRingStatus":null,"companyLogo":null,"icon":null,"systemImage":"SYS_ICN_SIGNAL_AI_SMALL","nonEntityGroupLogo":null,"vectorImage":null,"nonEntityProfessionalEventLogo":null,"profilePicture":null,"imageUrl":null,"professionalEventLogo":null,"nonEntityCompanyLogo":null,"nonEntitySchoolLogo":null,"groupLogo":null,"schoolLogo":null,"ghostImage":null,"nonEntityProfilePicture":null},"tintColor":"PREMIUM_INBUG","$recipeTypes":["com.linkedin.296fd4ebc3f7a46ccca548d534f656e4"],"tapTargets":[],"displayAspectRatio":null,"$type":"com.linkedin.voyager.dash.common.image.ImageAttribute"}],"editableAccessibilityText":false,"actionTarget":null,"accessibilityTextAttributes":[],"totalCount":null,"accessibilityText":null,"$recipeTypes":["com.linkedin.30e1ec6a491035bd7a31cc9de8a12c18"],"$type":"com.linkedin.voyager.dash.common.image.ImageViewModel"},"isPremium":true,"title":{"textDirection":"USER_LOCALE","text":"","attributesV2":[],"accessibilityTextAttributesV2":[],"accessibilityText":null,"$recipeTypes":["com.linkedin.390ac1c1431d5b4e340a65af24ed68f2"],"$type":"com.linkedin.voyager.dash.common.text.TextViewModel"},"$recipeTypes":["com.linkedin.70094a2b1194a3dfad82481287a40a81"],"toolTipText":"Get advice on the right jobs for you","coachQueryContextV2":{"serverIntent":"CoachClientIntent_JOB_ASSESSMENT","$recipeTypes":["com.linkedin.8864b68d440629f1899c8ff463b7f71d"],"$type":"com.linkedin.voyager.dash.common.coach.CoachQueryContextV2"},"coachUseCaseType":"JOB_SEEKER","$type":"com.linkedin.voyager.dash.jobs.gai.CoachJobsKickoffPrompt"},{"queryText":"Am I a good fit for this job?","controlName":"view_coach","coachSparkleIcon":{"attributes":[{"scalingType":null,"detailData":{"profilePictureWithoutFrame":null,"profilePictureWithRingStatus":null,"companyLogo":null,"icon":null,"systemImage":"SYS_ICN_SIGNAL_AI_SMALL","nonEntityGroupLogo":null,"vectorImage":null,"nonEntityProfessionalEventLogo":null,"profilePicture":null,"imageUrl":null,"professionalEventLogo":null,"nonEntityCompanyLogo":null,"nonEntitySchoolLogo":null,"groupLogo":null,"schoolLogo":null,"ghostImage":null,"nonEntityProfilePicture":null},"tintColor":"PREMIUM_INBUG","$recipeTypes":["com.linkedin.296fd4ebc3f7a46ccca548d534f656e4"],"tapTargets":[],"displayAspectRatio":null,"$type":"com.linkedin.voyager.dash.common.image.ImageAttribute"}],"editableAccessibilityText":false,"actionTarget":null,"accessibilityTextAttributes":[],"totalCount":null,"accessibilityText":null,"$recipeTypes":["com.linkedin.30e1ec6a491035bd7a31cc9de8a12c18"],"$type":"com.linkedin.voyager.dash.common.image.ImageViewModel"},"isPremium":true,"title":{"textDirection":"USER_LOCALE","text":"Am I a good fit for this job?","attributesV2":[],"accessibilityTextAttributesV2":[],"accessibilityText":null,"$recipeTypes":["com.linkedin.390ac1c1431d5b4e340a65af24ed68f2"],"$type":"com.linkedin.voyager.dash.common.text.TextViewModel"},"$recipeTypes":["com.linkedin.70094a2b1194a3dfad82481287a40a81"],"toolTipText":"Get advice on the right jobs for you","coachQueryContextV2":{"serverIntent":"CoachClientIntent_JOB_ASSESSMENT","$recipeTypes":["com.linkedin.8864b68d440629f1899c8ff463b7f71d"],"$type":"com.linkedin.voyager.dash.common.coach.CoachQueryContextV2"},"coachUseCaseType":"JOB_SEEKER","$type":"com.linkedin.voyager.dash.jobs.gai.CoachJobsKickoffPrompt"},{"queryText":"How can I best position myself for this job?","controlName":"view_coach","coachSparkleIcon":{"attributes":[{"scalingType":null,"detailData":{"profilePictureWithoutFrame":null,"profilePictureWithRingStatus":null,"companyLogo":null,"icon":null,"systemImage":"SYS_ICN_SIGNAL_AI_SMALL","nonEntityGroupLogo":null,"vectorImage":null,"nonEntityProfessionalEventLogo":null,"profilePicture":null,"imageUrl":null,"professionalEventLogo":null,"nonEntityCompanyLogo":null,"nonEntitySchoolLogo":null,"groupLogo":null,"schoolLogo":null,"ghostImage":null,"nonEntityProfilePicture":null},"tintColor":"PREMIUM_INBUG","$recipeTypes":["com.linkedin.296fd4ebc3f7a46ccca548d534f656e4"],"tapTargets":[],"displayAspectRatio":null,"$type":"com.linkedin.voyager.dash.common.image.ImageAttribute"}],"editableAccessibilityText":false,"actionTarget":null,"accessibilityTextAttributes":[],"totalCount":null,"accessibilityText":null,"$recipeTypes":["com.linkedin.30e1ec6a491035bd7a31cc9de8a12c18"],"$type":"com.linkedin.voyager.dash.common.image.ImageViewModel"},"isPremium":true,"title":{"textDirection":"USER_LOCALE","text":"How can I best position myself for this job?","attributesV2":[],"accessibilityTextAttributesV2":[],"accessibilityText":null,"$recipeTypes":["com.linkedin.390ac1c1431d5b4e340a65af24ed68f2"],"$type":"com.linkedin.voyager.dash.common.text.TextViewModel"},"$recipeTypes":["com.linkedin.70094a2b1194a3dfad82481287a40a81"],"toolTipText":"Get advice on the right jobs for you","coachQueryContextV2":{"serverIntent":"CoachClientIntent_JOB_RECOMMENDED_ACTIONS","$recipeTypes":["com.linkedin.8864b68d440629f1899c8ff463b7f71d"],"$type":"com.linkedin.voyager.dash.common.coach.CoachQueryContextV2"},"coachUseCaseType":"JOB_SEEKER","$type":"com.linkedin.voyager.dash.jobs.gai.CoachJobsKickoffPrompt"},{"queryText":"Tell me more about Cozero","controlName":"view_coach","coachSparkleIcon":{"attributes":[{"scalingType":null,"detailData":{"profilePictureWithoutFrame":null,"profilePictureWithRingStatus":null,"companyLogo":null,"icon":null,"systemImage":"SYS_ICN_SIGNAL_AI_SMALL","nonEntityGroupLogo":null,"vectorImage":null,"nonEntityProfessionalEventLogo":null,"profilePicture":null,"imageUrl":null,"professionalEventLogo":null,"nonEntityCompanyLogo":null,"nonEntitySchoolLogo":null,"groupLogo":null,"schoolLogo":null,"ghostImage":null,"nonEntityProfilePicture":null},"tintColor":"PREMIUM_INBUG","$recipeTypes":["com.linkedin.296fd4ebc3f7a46ccca548d534f656e4"],"tapTargets":[],"displayAspectRatio":null,"$type":"com.linkedin.voyager.dash.common.image.ImageAttribute"}],"editableAccessibilityText":false,"actionTarget":null,"accessibilityTextAttributes":[],"totalCount":null,"accessibilityText":null,"$recipeTypes":["com.linkedin.30e1ec6a491035bd7a31cc9de8a12c18"],"$type":"com.linkedin.voyager.dash.common.image.ImageViewModel"},"isPremium":true,"title":{"textDirection":"USER_LOCALE","text":"Tell me more about Cozero","attributesV2":[],"accessibilityTextAttributesV2":[],"accessibilityText":null,"$recipeTypes":["com.linkedin.390ac1c1431d5b4e340a65af24ed68f2"],"$type":"com.linkedin.voyager.dash.common.text.TextViewModel"},"$recipeTypes":["com.linkedin.70094a2b1194a3dfad82481287a40a81"],"toolTipText":"Get advice on the right jobs for you","coachQueryContextV2":{"serverIntent":"CoachClientIntent_COMPANY_INFO","$recipeTypes":["com.linkedin.8864b68d440629f1899c8ff463b7f71d"],"$type":"com.linkedin.voyager.dash.common.coach.CoachQueryContextV2"},"coachUseCaseType":"JOB_SEEKER","$type":"com.linkedin.voyager.dash.jobs.gai.CoachJobsKickoffPrompt"}],"$type":"com.linkedin.voyager.dash.jobs.gai.CoachJobsKickoffPrompts"},{"name":"Cozero","logoResolutionResult":{"url":null,"vectorImage":{"digitalmediaAsset":"urn:li:digitalmediaAsset:C4D0BAQGkpvwIXWu1-g","attribution":null,"$recipeTypes":["com.linkedin.b8949adeaaa915212d0136c4b9a1952b"],"artifacts":[{"width":200,"$recipeTypes":["com.linkedin.8d64ead1eeea51a297dc70da47337de1"],"fileIdentifyingUrlPathSegment":"200_200/0/1638439865071/cozero_io_logo?e=1709164800&amp;v=beta&amp;t=SkxNXYSV-cQiUJvKiBgKXlXVlM4i0GmvhuX8g6hGKZY","expiresAt":1709164800000,"height":200,"$type":"com.linkedin.common.VectorArtifact"},{"width":100,"$recipeTypes":["com.linkedin.8d64ead1eeea51a297dc70da47337de1"],"fileIdentifyingUrlPathSegment":"100_100/0/1638439865071/cozero_io_logo?e=1709164800&amp;v=beta&amp;t=sVPH2ggrA8QHrpoTseVt0_N71x9Hg_xcONvZl48_Ogk","expiresAt":1709164800000,"height":100,"$type":"com.linkedin.common.VectorArtifact"},{"width":400,"$recipeTypes":["com.linkedin.8d64ead1eeea51a297dc70da47337de1"],"fileIdentifyingUrlPathSegment":"400_400/0/1638439865071/cozero_io_logo?e=1709164800&amp;v=beta&amp;t=G2AlGC86FzMcCKbWIW1B4lPKCq9A1cBdlHhCGtKap9c","expiresAt":1709164800000,"height":400,"$type":"com.linkedin.common.VectorArtifact"}],"rootUrl":"https://media.licdn.com/dms/image/C4D0BAQGkpvwIXWu1-g/company-logo_","$type":"com.linkedin.common.VectorImage"}},"entityUrn":"urn:li:fsd_company:43338549","$recipeTypes":["com.linkedin.a2ef5c1c8d1bd7b83de7009b834a38c6"],"$type":"com.linkedin.voyager.dash.organization.Company"},{"entityUrn":"urn:li:fsd_premiumUpsellSlot:JOB_DETAILS_SUMMARY_CARD","$recipeTypes":["com.linkedin.dc4c6e3940e502e42eb82438140331c7"],"upsellCard":{"image":{"attributes":[{"scalingType":null,"detailData":{"profilePictureWithoutFrame":null,"profilePictureWithRingStatus":null,"companyLogo":null,"icon":null,"systemImage":"SYS_ICN_LIGHTBULB_MEDIUM","nonEntityGroupLogo":null,"vectorImage":null,"nonEntityProfessionalEventLogo":null,"profilePicture":null,"imageUrl":null,"professionalEventLogo":null,"nonEntityCompanyLogo":null,"nonEntitySchoolLogo":null,"groupLogo":null,"schoolLogo":null,"ghostImage":null,"nonEntityProfilePicture":null},"tintColor":null,"$recipeTypes":["com.linkedin.296fd4ebc3f7a46ccca548d534f656e4"],"tapTargets":[],"displayAspectRatio":null,"$type":"com.linkedin.voyager.dash.common.image.ImageAttribute"}],"editableAccessibilityText":false,"actionTarget":null,"accessibilityTextAttributes":[],"totalCount":null,"accessibilityText":null,"$recipeTypes":["com.linkedin.30e1ec6a491035bd7a31cc9de8a12c18"],"$type":"com.linkedin.voyager.dash.common.image.ImageViewModel"},"socialProofInsight":null,"subtext":null,"upsellOrderOrigin":"premium_job_details_summary_card","ctaText":{"textDirection":"USER_LOCALE","text":"See how you compare to 71 applicants. Try Premium for free","attributesV2":[{"start":38,"length":20,"detailData":{"jobPostingName":null,"hyperlink":"https://www.linkedin.com/premium/products/?family=JSS&amp;upsellOrderOrigin=premium_job_details_summary_card&amp;utype=job&amp;referenceId=35%2FiR8HFQmqrpPg04moZPA%3D%3D","profileFamiliarName":null,"color":null,"companyName":null,"icon":null,"epoch":null,"systemImage":null,"textLink":null,"listItemStyle":null,"groupName":null,"hyperlinkOpenExternally":null,"listStyle":null,"profileFullName":null,"stringFieldReference":null,"learningCourseName":null,"profileMention":null,"style":null,"schoolName":null,"hashtag":null},"$recipeTypes":["com.linkedin.b0a683a611d43de435ed53c6232d8d44"],"$type":"com.linkedin.voyager.dash.common.text.TextAttribute"}],"accessibilityTextAttributesV2":[],"accessibilityText":null,"$recipeTypes":["com.linkedin.390ac1c1431d5b4e340a65af24ed68f2"],"$type":"com.linkedin.voyager.dash.common.text.TextViewModel"},"actionUrl":"https://www.linkedin.com/premium/products/?family=JSS&amp;upsellOrderOrigin=premium_job_details_summary_card&amp;utype=job&amp;referenceId=35%2FiR8HFQmqrpPg04moZPA%3D%3D","controlName":"premium_job_details_summary_card","legoTrackingToken":null,"title":{"textDirection":"USER_LOCALE","text":"See how you compare to 71 applicants.","attributesV2":[],"accessibilityTextAttributesV2":[],"accessibilityText":null,"$recipeTypes":["com.linkedin.390ac1c1431d5b4e340a65af24ed68f2"],"$type":"com.linkedin.voyager.dash.common.text.TextViewModel"},"$recipeTypes":["com.linkedin.d9052d9e22b039f19b4d6484e2ebec27"],"$type":"com.linkedin.voyager.dash.premium.PremiumUpsellCard","cancelCta":null,"globalLegoTrackingToken":null,"promotionLegoTrackingToken":null,"funnelCommonHeader":{"utype":"job","$recipeTypes":["com.linkedin.c2408eaefddaf66bea26cb69c95979d7"],"referenceId":"ßâGÁÅBj«¤ø4âj\u0019&lt;","$type":"com.linkedin.voyager.dash.premium.PremiumFunnelCommonHeader"},"fireDiscoveryImpressionEvent":false,"subtitle":null,"dismissible":false,"layoutStyle":"CUSTOM","ctaButtonSecondaryEmphasizedTheme":false},"$type":"com.linkedin.voyager.dash.premium.PremiumUpsellSlotContent"}]}
</code>
<code style="display: none" id="datalet-bpr-guid-15912217">
  {"request":"/voyager/api/graphql?includeWebMetadata\u003Dtrue\u0026variables\u003D(cardSectionTypes:List(TOP_CARD),jobPostingUrn:urn%3Ali%3Afsd_jobPosting%3A3768592532)\u0026queryId\u003DvoyagerJobsDashJobPostingDetailSections.05d61c74bf304974dfa84c4ec2016541","status":200,"body":"bpr-guid-15912217","method":"GET","headers":{"x-li-uuid":"AAYLE+vtCEZ6Amp7P3S1tA\u003D\u003D"}}
</code>
<img src="data:image/gif;base64,R0lGODlhAQABAIAAAAAAAP///yH5BAEAAAAALAAAAAABAAEAAAIBRAA7" style="display: none" class="datalet-bpr-guid-15912217">



  <svg role="none" xmlns="http://www.w3.org/2000/svg" id="hue-web-icons-sprite" style="display: none;">
    <svg xmlns="http://www.w3.org/2000/svg" viewBox="0 0 24 24" id="close-medium" aria-hidden="true" role="none" data-supported-dps="24x24" fill="currentColor">
  <path d="M13.42 12L20 18.58 18.58 20 12 13.42 5.42 20 4 18.58 10.58 12 4 5.42 5.42 4 12 10.58 18.58 4 20 5.42z"></path>
</svg>

    <svg xmlns="http://www.w3.org/2000/svg" viewBox="0 0 16 16" id="search-small" aria-hidden="true" role="none" data-supported-dps="16x16" fill="currentColor">
  <path d="M14.56 12.44L11.3 9.18a5.51 5.51 0 10-2.12 2.12l3.26 3.26a1.5 1.5 0 102.12-2.12zM3 6.5A3.5 3.5 0 116.5 10 3.5 3.5 0 013 6.5z"></path>
</svg>

    <svg xmlns="http://www.w3.org/2000/svg" viewBox="0 0 24 24" id="search-medium" aria-hidden="true" role="none" data-supported-dps="24x24" fill="currentColor">
  <path d="M21.41 18.59l-5.27-5.28A6.83 6.83 0 0017 10a7 7 0 10-7 7 6.83 6.83 0 003.31-.86l5.28 5.27a2 2 0 002.82-2.82zM5 10a5 5 0 115 5 5 5 0 01-5-5z"></path>
</svg>

    <svg xmlns="http://www.w3.org/2000/svg" viewBox="0 0 16 16" id="caret-small" aria-hidden="true" role="none" data-supported-dps="16x16" fill="currentColor">
  <path d="M8 11L3 6h10z" fill-rule="evenodd"></path>
</svg>

    <svg xmlns="http://www.w3.org/2000/svg" viewBox="0 0 24 24" id="overflow-web-ios-medium" aria-hidden="true" role="none" data-supported-dps="24x24" fill="currentColor">
  <path d="M14 12a2 2 0 11-2-2 2 2 0 012 2zM4 10a2 2 0 102 2 2 2 0 00-2-2zm16 0a2 2 0 102 2 2 2 0 00-2-2z"></path>
</svg>

    <svg xmlns="http://www.w3.org/2000/svg" viewBox="0 0 16 16" id="link-external-small" aria-hidden="true" role="none" data-supported-dps="16x16" fill="currentColor">
  <path d="M15 1v6h-2V4.41L7.41 10 6 8.59 11.59 3H9V1zm-4 10a1 1 0 01-1 1H5a1 1 0 01-1-1V6a1 1 0 011-1h2V3H5a3 3 0 00-3 3v5a3 3 0 003 3h5a3 3 0 003-3V9h-2z"></path>
</svg>

    <svg xmlns="http://www.w3.org/2000/svg" viewBox="0 0 24 24" class="rtl-flip" id="share-linkedin-medium" aria-hidden="true" role="none" data-supported-dps="24x24" fill="currentColor">
  <path d="M23 12l-4.61 7H16l4-6H8a3.92 3.92 0 00-4 3.84V17a4 4 0 00.19 1.24L5.12 21H3l-.73-2.22A6.4 6.4 0 012 16.94 6 6 0 018 11h12l-4-6h2.39z"></path>
</svg>

    <svg xmlns="http://www.w3.org/2000/svg" viewBox="0 0 16 16" id="chevron-down-small" aria-hidden="true" role="none" data-supported-dps="16x16" fill="currentColor">
  <path d="M1 5l7 4.61L15 5v2.39L8 12 1 7.39z"></path>
</svg>

    <svg xmlns="http://www.w3.org/2000/svg" viewBox="0 0 16 16" id="bell-outline-small" aria-hidden="true" role="none" data-supported-dps="16x16" fill="currentColor">
  <path d="M15 11.84a2.56 2.56 0 00-.5-1.51L12.9 8.2l-.52-3.41a4.43 4.43 0 00-8.76 0L3.1 8.2l-1.6 2.13a2.56 2.56 0 00-.5 1.51V13h5.59a1.64 1.64 0 00-.09.5 1.5 1.5 0 003 0 1.64 1.64 0 00-.09-.5H15zM8 3a2.43 2.43 0 012.41 2.1l.45 2.9H5.14l.45-2.91A2.43 2.43 0 018 3zm-4.52 8l1.21-1.61L5 9h6l.29.39L12.52 11z"></path>
</svg>

    <svg xmlns="http://www.w3.org/2000/svg" viewBox="0 0 16 16" id="overflow-web-ios-small" aria-hidden="true" role="none" data-supported-dps="16x16" fill="currentColor">
  <path d="M3 9.5A1.5 1.5 0 114.5 8 1.5 1.5 0 013 9.5zM11.5 8A1.5 1.5 0 1013 6.5 1.5 1.5 0 0011.5 8zm-5 0A1.5 1.5 0 108 6.5 1.5 1.5 0 006.5 8z"></path>
</svg>

    <svg xmlns="http://www.w3.org/2000/svg" viewBox="0 0 16 16" id="compose-small" aria-hidden="true" role="none" data-supported-dps="16x16" fill="currentColor">
  <path d="M15 2.53a1.51 1.51 0 01-.44 1L9.15 9 6 10l1-3.12 5.44-5.44A1.5 1.5 0 0115 2.53zM12 11a1 1 0 01-1 1H5a1 1 0 01-1-1V5a1 1 0 011-1h3V2H5a3 3 0 00-3 3v6a3 3 0 003 3h6a3 3 0 003-3V8h-2z"></path>
</svg>

    <svg xmlns="http://www.w3.org/2000/svg" viewBox="0 0 16 16" id="chevron-up-small" aria-hidden="true" role="none" data-supported-dps="16x16" fill="currentColor">
  <path d="M15 11L8 6.39 1 11V8.61L8 4l7 4.61z"></path>
</svg>

    <svg xmlns="http://www.w3.org/2000/svg" viewBox="0 0 16 16" id="check-small" aria-hidden="true" role="none" data-supported-dps="16x16" fill="currentColor">
  <path d="M12.57 2H15L6 15l-5-5 1.41-1.41 3.31 3.3z"></path>
</svg>

    <svg xmlns="http://www.w3.org/2000/svg" viewBox="0 0 16 16" id="add-small" aria-hidden="true" role="none" data-supported-dps="16x16" fill="currentColor">
  <path d="M14 9H9v5H7V9H2V7h5V2h2v5h5z"></path>
</svg>

    <svg xmlns="http://www.w3.org/2000/svg" viewBox="0 0 16 16" id="signal-success-small" aria-hidden="true" role="none" data-supported-dps="16x16" fill="currentColor">
  <path d="M8 1a7 7 0 107 7 7 7 0 00-7-7zm-.6 11L4.25 8.85 5.6 7.51 7.1 9l2.63-4H12z"></path>
</svg>

    <svg xmlns="http://www.w3.org/2000/svg" viewBox="0 0 16 16" id="signal-caution-small" aria-hidden="true" role="none" data-supported-dps="16x16" fill="currentColor">
  <path d="M14.42 6.59l-5-5a2 2 0 00-2.82 0l-5 5a2 2 0 000 2.82l5 5a2 2 0 002.82 0l5-5a2 2 0 000-2.82zM7 4h2v5H7zm1 8.25A1.25 1.25 0 119.25 11 1.25 1.25 0 018 12.25z"></path>
</svg>

    <svg xmlns="http://www.w3.org/2000/svg" viewBox="0 0 24 24" id="lightbulb-medium" aria-hidden="true" role="none" data-supported-dps="24x24" fill="currentColor">
  <path d="M10 20h4v.1a1.9 1.9 0 01-1.9 1.9h-.2a1.9 1.9 0 01-1.9-1.9zm4.68-14.47A1 1 0 0016 5l.76-1.85a1 1 0 00-.54-1.3 1 1 0 00-1.31.54l-.76 1.85a1 1 0 00.53 1.29zm5.09 4.33l1.85-.76a1 1 0 00.54-1.31 1 1 0 00-1.3-.54L19 8a1 1 0 00-.54 1.31 1 1 0 001.3.55zM4.23 14.14l-1.85.76a1 1 0 00-.54 1.31 1 1 0 001.3.54L5 16a1 1 0 00.54-1.31 1 1 0 00-1.3-.55zm17.39.76l-1.85-.76A1 1 0 0019 16l1.85.76a1 1 0 00.78-1.86zM5 8l-1.86-.75a1 1 0 00-1.3.54 1 1 0 00.54 1.31l1.85.76A1 1 0 005 8zm3-3a1 1 0 001.31.54 1 1 0 00.54-1.3L9.1 2.38a1 1 0 00-1.31-.54 1 1 0 00-.54 1.3zm9 6.91V12a5.48 5.48 0 01-1.61 3.88l-.07.08a3.94 3.94 0 00-1.08 1.94L14 19h-4l-.24-1.08A4 4 0 008.68 16l-.07-.07A5.51 5.51 0 017 12v-.1A4.9 4.9 0 0111.9 7h.2a4.9 4.9 0 014.9 4.9zm-2 0A2.9 2.9 0 0012.11 9h-.21A2.9 2.9 0 009 11.9v.1a3.47 3.47 0 001 2.47l.08.08a5.92 5.92 0 011.5 2.45h.84a6 6 0 011.48-2.44l.08-.07A3.48 3.48 0 0015 12z"></path>
</svg>

    <svg xmlns="http://www.w3.org/2000/svg" viewBox="0 0 16 16" class="rtl-flip" id="arrow-right-small" aria-hidden="true" role="none" data-supported-dps="16x16" fill="currentColor">
  <path d="M11.45 3L15 8l-3.55 5H9l2.84-4H2V7h9.84L9 3z"></path>
</svg>

</svg>

  <!---->

  
<!---->  

<!---->
<div class="application-outlet">
<!---->
        
    <div class="global-nav__a11y-menu
        ">
      <div class="global-nav__a11y-menu-container">
          <button id="ember7" class="mr4 artdeco-button artdeco-button--muted artdeco-button--2 artdeco-button--secondary ember-view"><!---->
<span class="artdeco-button__text">
    Skip to search
</span></button>

        <button id="ember8" class="artdeco-button artdeco-button--muted artdeco-button--2 artdeco-button--secondary ember-view"><!---->
<span class="artdeco-button__text">
    Skip to main content
</span></button>

<!---->
        <button id="ember9" class="global-nav__a11y-menu-close artdeco-button artdeco-button--circle artdeco-button--muted artdeco-button--2 artdeco-button--tertiary ember-view">        <svg role="none" aria-hidden="true" class="artdeco-button__icon " xmlns="http://www.w3.org/2000/svg" width="24" height="24" viewBox="0 0 24 24" data-supported-dps="24x24" data-test-icon="close-medium">
<!---->    

    <use href="#close-medium" width="24" height="24"></use>
</svg>


<span class="artdeco-button__text">
    Close jump menu
</span></button>
      </div>
    </div>
  
      
    <header id="global-nav" class="global-nav global-alert-offset-top
        
        global-nav--visible">
      <div class="global-nav__content">
          
    <a class="app-aware-link " href="https://www.linkedin.com/feed/?nis=true" data-test-app-aware-link="">
      
    <div class="ivm-image-view-model    global-nav__branding-logo">
        
    <div class="ivm-view-attr__img-wrapper display-flex">
        <li-icon type="app-linkedin-bug-color-icon" class=" " size="large" role="img" aria-label="LinkedIn"><svg xmlns="http://www.w3.org/2000/svg" viewBox="0 0 24 24" data-supported-dps="24x24" fill="currentColor" class="mercado-match" width="24" height="24" focusable="false">
      <path d="M20.5 2h-17A1.5 1.5 0 002 3.5v17A1.5 1.5 0 003.5 22h17a1.5 1.5 0 001.5-1.5v-17A1.5 1.5 0 0020.5 2zM8 19H5v-9h3zM6.5 8.25A1.75 1.75 0 118.3 6.5a1.78 1.78 0 01-1.8 1.75zM19 19h-3v-4.74c0-1.42-.6-1.93-1.38-1.93A1.74 1.74 0 0013 14.19a.66.66 0 000 .14V19h-3v-9h2.9v1.3a3.11 3.11 0 012.7-1.4c1.55 0 3.36.86 3.36 3.66z"></path>
    </svg></li-icon>
    </div>
  
          </div>
  
    </a>
  

        <div id="global-nav-search" class="global-nav__search
            ">
          
          
    <div class="search-global-typeahead
        
         global-nav__search-typeahead">
        
    <div id="global-nav-typeahead" class="search-global-typeahead__typeahead">
<!---->        
          
    <input class="search-global-typeahead__input" placeholder="Search" role="combobox" aria-autocomplete="list" aria-label="Search" aria-activedescendant="" aria-expanded="false" type="text">
  
          <div aria-hidden="true" class="search-global-typeahead__search-icon-container">
            <svg role="none" aria-hidden="true" class="search-global-typeahead__search-icon" xmlns="http://www.w3.org/2000/svg" width="16" height="16" viewBox="0 0 16 16" data-supported-dps="16x16" data-test-icon="search-small">
<!---->    

    <use href="#search-small" width="16" height="16"></use>
</svg>

          </div>
<!---->          <div class="search-global-typeahead__overlay global-alert-offset-top
              "></div>
<!---->        
    </div>
  
      <button class="search-global-typeahead__collapsed-search-button" aria-label="Click to start a search" type="button">
        <span class="search-global-typeahead__collapsed-search-button-icon t-black--light">
          <svg role="none" aria-hidden="true" xmlns="http://www.w3.org/2000/svg" width="24" height="24" viewBox="0 0 24 24" data-supported-dps="24x24" data-test-icon="search-medium">
<!---->    

    <use href="#search-medium" width="24" height="24"></use>
</svg>

        </span>
        <div class="search-global-typeahead__collapsed-search-button-text t-black--light t-12 t-normal">
          Search
        </div>
      </button>
      
  <div id="ember11" class="ember-view"><!----></div>

    </div>
  
      
        </div>

        <nav class="global-nav__nav" aria-label="Primary Navigation">
          <ul class="global-nav__primary-items
              ">
              <li class="global-nav__primary-item
                  ">
                  
      
    <a class="app-aware-link  global-nav__primary-link" target="_self" href="https://www.linkedin.com/feed/?nis=true&amp;" data-test-app-aware-link="">
      

      <div id="ember12" class="global-nav__primary-link-notif artdeco-notification-badge ember-view">  <span class="notification-badge notification-badge--show ">
        <span aria-hidden="true" class="notification-badge__no-count"></span>
        <span class="a11y-text" data-test-notification-a11y="">new feed updates notifications</span>
  </span>
  
        
    <div class="ivm-image-view-model    global-nav__icon-ivm
            ">
        
    <div class="ivm-view-attr__img-wrapper display-flex">
        <li-icon aria-hidden="true" type="home" class=" " size="large"><svg xmlns="http://www.w3.org/2000/svg" viewBox="0 0 24 24" data-supported-dps="24x24" fill="currentColor" class="mercado-match" width="24" height="24" focusable="false">
      <path d="M23 9v2h-2v7a3 3 0 01-3 3h-4v-6h-4v6H6a3 3 0 01-3-3v-7H1V9l11-7z"></path>
    </svg></li-icon>
    </div>
  
          </div>
  
      
</div>
  

      <span class="t-12 break-words block t-black--light t-normal
          global-nav__primary-link-text" title="Home">
        Home
      </span>
    </a>
  

<!---->  
<!----><!----><!---->              </li>
              <li class="global-nav__primary-item
                  ">
                  
      
    <a class="app-aware-link  global-nav__primary-link" target="_self" href="https://www.linkedin.com/mynetwork/?" data-test-app-aware-link="">
      

        
    <div class="ivm-image-view-model    global-nav__icon-ivm
            ">
        
    <div class="ivm-view-attr__img-wrapper display-flex">
        <li-icon aria-hidden="true" type="people" class=" " size="large"><svg xmlns="http://www.w3.org/2000/svg" viewBox="0 0 24 24" data-supported-dps="24x24" fill="currentColor" class="mercado-match" width="24" height="24" focusable="false">
      <path d="M12 16v6H3v-6a3 3 0 013-3h3a3 3 0 013 3zm5.5-3A3.5 3.5 0 1014 9.5a3.5 3.5 0 003.5 3.5zm1 2h-2a2.5 2.5 0 00-2.5 2.5V22h7v-4.5a2.5 2.5 0 00-2.5-2.5zM7.5 2A4.5 4.5 0 1012 6.5 4.49 4.49 0 007.5 2z"></path>
    </svg></li-icon>
    </div>
  
          </div>
  
  

      <span class="t-12 break-words block t-black--light t-normal
          global-nav__primary-link-text" title="My Network">
        My Network
      </span>
    </a>
  

<!---->  
<!----><!----><!---->              </li>
              <li class="global-nav__primary-item
                  ">
                  
      
    <a class="app-aware-link  global-nav__primary-link" target="_self" href="https://www.linkedin.com/jobs/?" data-test-app-aware-link="">
      

        
    <div class="ivm-image-view-model    global-nav__icon-ivm
            ">
        
    <div class="ivm-view-attr__img-wrapper display-flex">
        <li-icon aria-hidden="true" type="job" class=" " size="large"><svg xmlns="http://www.w3.org/2000/svg" viewBox="0 0 24 24" data-supported-dps="24x24" fill="currentColor" class="mercado-match" width="24" height="24" focusable="false">
      <path d="M17 6V5a3 3 0 00-3-3h-4a3 3 0 00-3 3v1H2v4a3 3 0 003 3h14a3 3 0 003-3V6zM9 5a1 1 0 011-1h4a1 1 0 011 1v1H9zm10 9a4 4 0 003-1.38V17a3 3 0 01-3 3H5a3 3 0 01-3-3v-4.38A4 4 0 005 14z"></path>
    </svg></li-icon>
    </div>
  
          </div>
  
  

      <span class="t-12 break-words block t-black--light t-normal
          global-nav__primary-link-text" title="Jobs">
        Jobs
      </span>
    </a>
  

<!---->  
<!----><!----><!---->              </li>
              <li class="global-nav__primary-item
                  ">
                  
      
    <a class="app-aware-link  global-nav__primary-link" target="_self" href="https://www.linkedin.com/messaging/?" data-test-app-aware-link="">
      

        
    <div class="ivm-image-view-model    global-nav__icon-ivm
            ">
        
    <div class="ivm-view-attr__img-wrapper display-flex">
        <li-icon aria-hidden="true" type="nav-small-messaging-icon" class=" " size="large"><svg xmlns="http://www.w3.org/2000/svg" viewBox="0 0 24 24" data-supported-dps="24x24" fill="currentColor" class="mercado-match" width="24" height="24" focusable="false">
      <path d="M16 4H8a7 7 0 000 14h4v4l8.16-5.39A6.78 6.78 0 0023 11a7 7 0 00-7-7zm-8 8.25A1.25 1.25 0 119.25 11 1.25 1.25 0 018 12.25zm4 0A1.25 1.25 0 1113.25 11 1.25 1.25 0 0112 12.25zm4 0A1.25 1.25 0 1117.25 11 1.25 1.25 0 0116 12.25z"></path>
    </svg></li-icon>
    </div>
  
          </div>
  
  

      <span class="t-12 break-words block t-black--light t-normal
          global-nav__primary-link-text" title="Messaging">
        Messaging
      </span>
    </a>
  

<!---->  
<!----><!----><!---->              </li>
              <li class="global-nav__primary-item
                  ">
                  
      
    <a class="app-aware-link  global-nav__primary-link" target="_self" href="https://www.linkedin.com/notifications/?" data-test-app-aware-link="">
      

        
    <div class="ivm-image-view-model    global-nav__icon-ivm
            ">
        
    <div class="ivm-view-attr__img-wrapper display-flex">
        <li-icon aria-hidden="true" type="bell-fill" class=" " size="large"><svg xmlns="http://www.w3.org/2000/svg" viewBox="0 0 24 24" data-supported-dps="24x24" fill="currentColor" class="mercado-match" width="24" height="24" focusable="false">
      <path d="M22 19h-8.28a2 2 0 11-3.44 0H2v-1a4.52 4.52 0 011.17-2.83l1-1.17h15.7l1 1.17A4.42 4.42 0 0122 18zM18.21 7.44A6.27 6.27 0 0012 2a6.27 6.27 0 00-6.21 5.44L5 13h14z"></path>
    </svg></li-icon>
    </div>
  
          </div>
  
  

      <span class="t-12 break-words block t-black--light t-normal
          global-nav__primary-link-text" title="Notifications">
        Notifications
      </span>
    </a>
  

<!---->  
<!----><!----><!---->              </li>
              <li class="global-nav__primary-item
                  ">
<!---->                  
    <div id="ember13" class="global-nav__me artdeco-dropdown artdeco-dropdown--placement-bottom artdeco-dropdown--justification-left ember-view">
      <button aria-expanded="false" id="ember14" class="global-nav__primary-link global-nav__primary-link-me-menu-trigger artdeco-dropdown__trigger artdeco-dropdown__trigger--placement-bottom ember-view" type="button" tabindex="0">
        <img width="24" src="https://media.licdn.com/dms/image/C5603AQEkRciOp6B6dA/profile-displayphoto-shrink_100_100/0/1605279012830?e=1706745600&amp;v=beta&amp;t=GoGDD2DE3nGXji1ktdhEWgWX0uNRNd93omIm9L4PTBM" height="24" alt="Naim Iskandar Zahari" id="ember15" class="global-nav__me-photo evi-image ember-view">

        <span class="global-nav__primary-link-text">
          Me
          <svg role="none" aria-hidden="true" class="global-nav__icon global-nav__icon--small" xmlns="http://www.w3.org/2000/svg" width="16" height="16" viewBox="0 0 16 16" data-supported-dps="16x16" data-test-icon="caret-small">
<!---->    

    <use href="#caret-small" width="16" height="16"></use>
</svg>

        </span>
      
<!----></button>

      <div tabindex="-1" aria-hidden="true" id="ember16" class="global-nav__me-content artdeco-dropdown__content artdeco-dropdown--is-dropdown-element artdeco-dropdown__content--justification-right artdeco-dropdown__content--placement-bottom ember-view"><!----></div>
    </div>

<!---->  
<!----><!---->              </li>
              <li class="global-nav__primary-item
                  ">
<!----><!---->                  
    
    
      
  <button aria-expanded="false" class="global-nav__primary-link
          global-nav__primary-item--divider pl3
          global-nav__app-launcher-trigger pr3" type="button">
    
          
    <div class="ivm-image-view-model    global-nav__icon-ivm">
        
    <div class="ivm-view-attr__img-wrapper display-flex">
        <li-icon aria-hidden="true" type="grid" class=" " size="large"><svg xmlns="http://www.w3.org/2000/svg" viewBox="0 0 24 24" data-supported-dps="24x24" fill="currentColor" class="mercado-match" width="24" height="24" focusable="false">
      <path d="M3 3h4v4H3zm7 4h4V3h-4zm7-4v4h4V3zM3 14h4v-4H3zm7 0h4v-4h-4zm7 0h4v-4h-4zM3 21h4v-4H3zm7 0h4v-4h-4zm7 0h4v-4h-4z"></path>
    </svg></li-icon>
    </div>
  
          </div>
  

        <span class="global-nav__primary-link-text" title="For Business">
          For Business
          <svg role="none" aria-hidden="true" xmlns="http://www.w3.org/2000/svg" width="16" height="16" viewBox="0 0 16 16" data-supported-dps="16x16" data-test-icon="caret-small">
<!---->    

    <use href="#caret-small" width="16" height="16"></use>
</svg>

        </span>
      
  </button>


      <!---->
    

    <div id="ember17" class="ember-view"><!----></div>
  
  
<!---->              </li>
              <li class="global-nav__primary-item
                  ">
<!----><!----><!---->                  
      
    <a class="app-aware-link  global-nav__primary-link" target="_blank" rel="noopener noreferrer" href="https://www.linkedin.com/learning/?trk=nav_neptune_learning&amp;" data-test-app-aware-link="">
      

        
    <div class="ivm-image-view-model    global-nav__icon-ivm
            ">
        
    <div class="ivm-view-attr__img-wrapper display-flex">
        <li-icon aria-hidden="true" type="nav-small-learning-icon" active="true" class=" " size="large"><svg xmlns="http://www.w3.org/2000/svg" viewBox="0 0 24 24" data-supported-dps="24x24" fill="currentColor" width="24" height="24" focusable="false">
      <path d="M22 5v14H2V5h20m1-2H1a1 1 0 00-1 1v16a1 1 0 001 1h22a1 1 0 001-1V4a1 1 0 00-1-1z"></path>
      <path d="M2 5v14h10V5H2zm8 12H4v-2h6v2zm0-4H4v-2h6v2zm0-4H4V7h6v2z" style="isolation:isolate" opacity=".25"></path>
      <path style="isolation:isolate" opacity=".6" d="M14 7h6v2h-6zM14 11h6v2h-6zM14 15h6v2h-6z"></path>
      <path d="M10 7.53v8.93a.28.28 0 00.44.23l6.43-4.44a.33.33 0 00.06-.46l-.06-.06-6.43-4.43a.28.28 0 00-.44.23z"></path>
    </svg></li-icon>
    </div>
  
          </div>
  
  

      <span class="t-12 break-words block t-black--light t-normal
          global-nav__primary-link-text" title="Learning">
        Learning
      </span>
    </a>
  

<!---->  
              </li>

<!---->          </ul>
        </nav>
      </div>
    </header>
  

    
    <div class="videoinappalert-inapp-alerts-manager
        hidden">
      
<!---->    
    </div>
  

<!---->
<div class="authentication-outlet">
<!---->  <!---->



  
        <div class="scaffold-layout__tracking-element"></div>

<div class="scaffold-layout
    scaffold-layout--breakpoint-none
    scaffold-layout--main-aside
    scaffold-layout--single-column
    scaffold-layout--reflow
    
     job-view-layout jobs-details">
    <section class="scaffold-layout-toolbar
    scaffold-layout-toolbar--is-fixed
    
    ">
  <div class="scaffold-layout-toolbar__content scaffold-layout-container
      scaffold-layout-container--reflow
      ">
    
      
          
    <div class="display-flex flex-row justify-space-between">
      <div>
        <h2 class="t-16 t-black t-bold truncate">
          Sustainability Data Analyst (f/m/x)
        </h2>
        <span>Cozero</span>
          <span class="jobs-unified-top-card__bullet">Germany</span>
          <span class="jobs-unified-top-card__workplace-type">Remote</span>
      </div>
      <div class="display-flex align-items-center">
        
    <div id="ember23" class="artdeco-dropdown jobs-options artdeco-dropdown--placement-bottom artdeco-dropdown--justification-right ember-view inline-flex
            visually-hidden">
      
          <button aria-expanded="false" id="ember24" class="artdeco-button artdeco-button--2 artdeco-button--tertiary artdeco-button--muted artdeco-button--circle artdeco-dropdown__trigger artdeco-dropdown__trigger--placement-bottom ember-view" type="button" tabindex="0">
            <svg role="none" aria-hidden="true" class="artdeco-button__icon" xmlns="http://www.w3.org/2000/svg" width="24" height="24" viewBox="0 0 24 24" data-supported-dps="24x24" data-test-icon="overflow-web-ios-medium">
<!---->    

    <use href="#overflow-web-ios-medium" width="24" height="24"></use>
</svg>

            <span class="artdeco-button__text">
              Show more options
            </span>
          
<!----></button>
        
      <div tabindex="-1" aria-hidden="true" id="ember25" class="artdeco-dropdown__content artdeco-dropdown--is-dropdown-element artdeco-dropdown__content--has-arrow artdeco-dropdown__content--arrow-right artdeco-dropdown__content--justification-right artdeco-dropdown__content--placement-bottom ember-view"><!----></div>
    </div>
  
            
    <button class="jobs-save-button artdeco-button artdeco-button--3 artdeco-button--secondary ml2" type="button">
<!---->          <span aria-hidden="true">
            Save
          </span>
        <span class="a11y-text">
          Save Sustainability Data Analyst (f/m/x) at Cozero
        </span>
    </button>
  
        
    <div class="jobs-s-apply jobs-s-apply--fadein inline-flex ml2">
        
    <div>
      <button role="link" aria-label="Apply to Sustainability Data Analyst (f/m/x) on company website" id="ember26" class="jobs-apply-button artdeco-button artdeco-button--icon-right artdeco-button--3 artdeco-button--primary ember-view">        <svg role="none" aria-hidden="true" class="artdeco-button__icon " xmlns="http://www.w3.org/2000/svg" width="16" height="16" viewBox="0 0 16 16" data-supported-dps="16x16" data-test-icon="link-external-small">
<!---->    

    <use href="#link-external-small" width="16" height="16"></use>
</svg>


<span class="artdeco-button__text">
    Apply
</span></button>
    </div>
  
    </div>
  
      </div>
    </div>
  
        
    
  </div>
</section>

  <div class="scaffold-layout__inner scaffold-layout-container
      scaffold-layout-container--reflow
      ">
<!---->
      <div class="scaffold-layout__row scaffold-layout__content
          scaffold-layout__content--main-aside
          
          
          scaffold-layout__content--has-aside
          
          ">
<!---->
          <main class="scaffold-layout__main">
  
            
          
    <div class="job-view-layout jobs-details">
      <div>
        <div class="
            mt4
            " role="main">
            

<!---->  
          
          <div>
            
    <div class="jobs-unified-top-card t-14 artdeco-card mb4">
<!---->      <div class="relative
          ">

        <div class="p5">
              <a class="app-aware-link " aria-label="Cozero logo" href="https://www.linkedin.com/company/cozero-io/life" data-test-app-aware-link="">
                
    <div class="ivm-image-view-model    mb3">
        
    <div class="ivm-view-attr__img-wrapper display-flex">
<!---->          <img width="48" src="https://media.licdn.com/dms/image/C4D0BAQGkpvwIXWu1-g/company-logo_100_100/0/1638439865071/cozero_io_logo?e=1709164800&amp;v=beta&amp;t=sVPH2ggrA8QHrpoTseVt0_N71x9Hg_xcONvZl48_Ogk" loading="lazy" height="48" alt="Cozero logo" id="ember46" class="ivm-view-attr__img--centered EntityPhoto-square-3   evi-image lazy-image ember-view">
    </div>
  
          </div>
  
              </a>

          <div class="display-flex justify-space-between flex-wrap">
              <h1 class="t-24 t-bold job-details-jobs-unified-top-card__job-title">
                  Sustainability Data Analyst (f/m/x)
              </h1>

<!---->          </div>

          <div class="job-details-jobs-unified-top-card__primary-description">
              <div>
                <a class="app-aware-link " target="_self" href="https://www.linkedin.com/company/cozero-io/life" data-test-app-aware-link=""><!---->Cozero<!----></a><span class="white-space-pre"> </span>· Germany<span class="white-space-pre"> </span><span class="tvm__text tvm__text--positive"><strong>      <span>
        <!---->22 hours ago<!---->
      </span>
</strong></span><span class="tvm__text tvm__text--neutral"><span class="white-space-pre"> </span>·<span class="white-space-pre"> </span></span><span class="tvm__text tvm__text--neutral"><!---->71 applicants<!----></span>
              </div>
          </div>

<!---->
          <div class="mt5 mb2">
              <ul>
                    <li class="job-details-jobs-unified-top-card__job-insight">
                      <div class="flex-shrink-zero mr2 t-black--light">
                        
    <div class="ivm-image-view-model   ">
        
    <div class="ivm-view-attr__img-wrapper display-flex">
        <li-icon aria-hidden="true" type="job" class=" " size="large"><svg xmlns="http://www.w3.org/2000/svg" viewBox="0 0 24 24" data-supported-dps="24x24" fill="currentColor" class="mercado-match" width="24" height="24" focusable="false">
      <path d="M17 6V5a3 3 0 00-3-3h-4a3 3 0 00-3 3v1H2v4a3 3 0 003 3h14a3 3 0 003-3V6zM9 5a1 1 0 011-1h4a1 1 0 011 1v1H9zm10 9a4 4 0 003-1.38V17a3 3 0 01-3 3H5a3 3 0 01-3-3v-4.38A4 4 0 005 14z"></path>
    </svg></li-icon>
    </div>
  
          </div>
  
                      </div>
                      <span>
                            <span>
                              <!---->Remote<!---->
                            </span>

<!---->                            <span class="job-details-jobs-unified-top-card__job-insight-view-model-secondary">
                              <!---->Full-time<!---->
                            </span>

<!---->                            <span class="job-details-jobs-unified-top-card__job-insight-view-model-secondary">
                              <!---->Entry level<!---->
                            </span>

<!---->                      </span>
                    </li>
                                      <li class="job-details-jobs-unified-top-card__job-insight">
                      <div class="flex-shrink-zero mr2 t-black--light">
                        
    <div class="ivm-image-view-model   ">
        
    <div class="ivm-view-attr__img-wrapper display-flex">
        <li-icon aria-hidden="true" type="company" class=" " size="large"><svg xmlns="http://www.w3.org/2000/svg" viewBox="0 0 24 24" data-supported-dps="24x24" fill="currentColor" class="mercado-match" width="24" height="24" focusable="false">
      <path d="M4 2v20h16V2zm14 18h-4v-2h-4v2H6V4h12zm-7-8H8v-2h3zm0 4H8v-2h3zm5-4h-3v-2h3zm-5-4H8V6h3zm5 0h-3V6h3zm0 8h-3v-2h3z"></path>
    </svg></li-icon>
    </div>
  
          </div>
  
                      </div>
                      <span>
                        <!---->11-50 employees · Software Development<!---->
                      </span>
                    </li>
                        <li class="job-details-jobs-unified-top-card__job-insight job-details-jobs-unified-top-card__job-insight--highlight">
                          <div class="flex-shrink-zero mr2 t-black--light">
                            
    <div class="ivm-image-view-model   ">
        
    <div class="ivm-view-attr__img-wrapper display-flex">
        <li-icon aria-hidden="true" type="lightbulb" class=" " size="medium"><svg xmlns="http://www.w3.org/2000/svg" viewBox="0 0 24 24" data-supported-dps="24x24" fill="currentColor" class="mercado-match" width="24" height="24" focusable="false">
      <path d="M10 20h4v.1a1.9 1.9 0 01-1.9 1.9h-.2a1.9 1.9 0 01-1.9-1.9zm4.68-14.47A1 1 0 0016 5l.76-1.85a1 1 0 00-.54-1.3 1 1 0 00-1.31.54l-.76 1.85a1 1 0 00.53 1.29zm5.09 4.33l1.85-.76a1 1 0 00.54-1.31 1 1 0 00-1.3-.54L19 8a1 1 0 00-.54 1.31 1 1 0 001.3.55zM4.23 14.14l-1.85.76a1 1 0 00-.54 1.31 1 1 0 001.3.54L5 16a1 1 0 00.54-1.31 1 1 0 00-1.3-.55zm17.39.76l-1.85-.76A1 1 0 0019 16l1.85.76a1 1 0 00.78-1.86zM5 8l-1.86-.75a1 1 0 00-1.3.54 1 1 0 00.54 1.31l1.85.76A1 1 0 005 8zm3-3a1 1 0 001.31.54 1 1 0 00.54-1.3L9.1 2.38a1 1 0 00-1.31-.54 1 1 0 00-.54 1.3zm9 6.91V12a5.48 5.48 0 01-1.61 3.88l-.07.08a3.94 3.94 0 00-1.08 1.94L14 19h-4l-.24-1.08A4 4 0 008.68 16l-.07-.07A5.51 5.51 0 017 12v-.1A4.9 4.9 0 0111.9 7h.2a4.9 4.9 0 014.9 4.9zm-2 0A2.9 2.9 0 0012.11 9h-.21A2.9 2.9 0 009 11.9v.1a3.47 3.47 0 001 2.47l.08.08a5.92 5.92 0 011.5 2.45h.84a6 6 0 011.48-2.44l.08-.07A3.48 3.48 0 0015 12z"></path>
    </svg></li-icon>
    </div>
  
          </div>
  
                          </div>
                          <span>
                            <!---->See how you compare to 71 applicants.<span class="white-space-pre"> </span><a class="app-aware-link " target="_self" href="https://www.linkedin.com/premium/products/?family=JSS&amp;upsellOrderOrigin=premium_job_details_summary_card&amp;utype=job&amp;referenceId=35%2FiR8HFQmqrpPg04moZPA%3D%3D" data-test-app-aware-link=""><!---->Try Premium for free<!----></a>
                          </span>
                        </li>
                        <li class="job-details-jobs-unified-top-card__job-insight">
                          <div class="flex-shrink-zero mr2 t-black--light">
                            
    <div class="ivm-image-view-model   ">
        
    <div class="ivm-view-attr__img-wrapper display-flex">
        <li-icon aria-hidden="true" type="checklist" class=" " size="medium"><svg xmlns="http://www.w3.org/2000/svg" viewBox="0 0 24 24" data-supported-dps="24x24" fill="currentColor" class="mercado-match" width="24" height="24" focusable="false">
      <path d="M22 3v2H11V3zM11 13h11v-2H11zm0 8h11v-2H11zM4.85 4L3.34 2.51 2 3.85 5.15 7l4.6-7H7.48zm0 8l-1.51-1.49L2 11.85 5.15 15l4.6-7H7.48zm0 8l-1.51-1.49L2 19.85 5.15 23l4.6-7H7.48z"></path>
    </svg></li-icon>
    </div>
  
          </div>
  
                          </div>
                          <button class="job-details-jobs-unified-top-card__job-insight-text-button" aria-label="View strong skill match modal" tabindex="-1" type="button">
                            <a class="app-aware-link " target="_self" href="#HYM" data-test-app-aware-link=""><!---->Skills: Data Analytics, Data Analysis, +8 more<!----></a>
                          </button>
                        </li>
<!---->              </ul>
          </div>

<!---->
          <div class="mt5">
              <div class="display-flex">
                  
    <div class="jobs-s-apply jobs-s-apply--fadein inline-flex mr2">
        
    <div class="jobs-apply-button--top-card">
      <button role="link" aria-label="Apply to Sustainability Data Analyst (f/m/x) on company website" id="ember47" class="jobs-apply-button artdeco-button artdeco-button--icon-right artdeco-button--3 artdeco-button--primary ember-view">        <svg role="none" aria-hidden="true" class="artdeco-button__icon " xmlns="http://www.w3.org/2000/svg" width="16" height="16" viewBox="0 0 16 16" data-supported-dps="16x16" data-test-icon="link-external-small">
<!---->    

    <use href="#link-external-small" width="16" height="16"></use>
</svg>


<span class="artdeco-button__text">
    Apply
</span></button>
    </div>
  
    </div>
  

                  <span class="visibility-hidden"></span>

                      
    <button class="jobs-save-button artdeco-button artdeco-button--3 artdeco-button--secondary" type="button">
<!---->          <span aria-hidden="true">
            Save
          </span>
        <span class="a11y-text">
          Save Sustainability Data Analyst (f/m/x) at Cozero
        </span>
    </button>
  
              </div>

<!---->
<!---->
<!----><!---->          </div>

<!---->
<!---->
<!----><!---->
<!---->        </div>

<!---->
<!---->
        <div class="job-details-jobs-unified-top-card__buttons-container absolute">
          <div class="display-flex flex-column">
            <div class="display-flex justify-flex-end">
                
    <div id="ember30" class="artdeco-dropdown artdeco-dropdown--placement-bottom artdeco-dropdown--justification-right ember-view">
        <button aria-expanded="false" aria-label="Share" id="ember31" class="social-share__dropdown-trigger artdeco-button artdeco-button--3 artdeco-button--tertiary artdeco-button--circle artdeco-button--muted artdeco-dropdown__trigger artdeco-dropdown__trigger--placement-bottom ember-view" type="button" tabindex="0">
          <svg role="none" aria-hidden="true" class="artdeco-button__icon rtl-flip" xmlns="http://www.w3.org/2000/svg" width="24" height="24" viewBox="0 0 24 24" data-supported-dps="24x24" data-test-icon="share-linkedin-medium">
<!---->    

    <use href="#share-linkedin-medium" width="24" height="24"></use>
</svg>

          <span class="artdeco-button__text">Share</span>
        
<!----></button>

      <div tabindex="-1" aria-hidden="true" id="ember32" class="social-share__content text-align-left artdeco-dropdown__content artdeco-dropdown--is-dropdown-element artdeco-dropdown__content--has-arrow artdeco-dropdown__content--arrow-right artdeco-dropdown__content--justification-right artdeco-dropdown__content--placement-bottom ember-view"><!----></div>
    </div>

    <div>
  
      
<!---->  
  
</div>

<!---->

<!---->  
                
    <div id="ember33" class="artdeco-dropdown jobs-options artdeco-dropdown--placement-bottom artdeco-dropdown--justification-right ember-view">
      
                  <button aria-expanded="false" id="ember34" class="artdeco-button artdeco-button--3 artdeco-button--tertiary artdeco-button--muted artdeco-button--circle artdeco-dropdown__trigger artdeco-dropdown__trigger--placement-bottom ember-view" type="button" tabindex="0">
                    <svg role="none" aria-hidden="true" class="artdeco-button__icon" xmlns="http://www.w3.org/2000/svg" width="24" height="24" viewBox="0 0 24 24" data-supported-dps="24x24" data-test-icon="overflow-web-ios-medium">
<!---->    

    <use href="#overflow-web-ios-medium" width="24" height="24"></use>
</svg>

                    <span class="artdeco-button__text">
                      Show more options
                    </span>
                  
<!----></button>
                
      <div tabindex="-1" aria-hidden="true" id="ember35" class="artdeco-dropdown__content artdeco-dropdown--is-dropdown-element artdeco-dropdown__content--has-arrow artdeco-dropdown__content--arrow-right artdeco-dropdown__content--justification-right artdeco-dropdown__content--placement-bottom ember-view"><!----></div>
    </div>
  
            </div>
          </div>
        </div>
      </div>
<!---->    </div>
  
<!---->
<!---->
<!---->
<!---->          </div>

<!---->
            
    <div class="jobs-box--fadein jobs-box--full-width jobs-box--with-cta-large jobs-description
         jobs-description--is-truncated jobs-description--is-truncated-poster
        
        
         artdeco-card mb4
        
        ">

<!---->
      <article class="jobs-description__container
          jobs-description__container--condensed">
        <div class="jobs-description__content jobs-description-content
            jobs-description__content--condensed">
          <div class="jobs-box__html-content jobs-description-content__text t-14 t-normal
              jobs-description-content__text--stretch" id="job-details" tabindex="-1">
            <h2 class="text-heading-large
                mb4">
              About the job
            </h2>

<!---->            <span>
                <strong><!---->Join Our Mission to Fight Climate Change as our Sustainability Data Analyst<!----><br><br></strong><!---->Want to be part of one of Europe's fastest growing climate tech startups? We are looking for a passionate Sustainability Data Analyst to help us transform every company into a sustainable business, with critical responsibility and influence on our product and overall venture development.<!----><br><br><!---->As a Sustainability Data Analyst your role will have a key responsibility of applying data-driven approaches to managing our carbon footprint calculations and ensuring our customers understand how we got there.<!----><br><br><!---->If you are excited about building an economy where every company creates an impact for a sustainable world and empowering companies to decarbonize and boost their business performance, we’d love to hear from you!<!----><br><br><!---->If you want to find out more about Cozero and our approach to hiring, check out this page.<!----><br><br><strong><!---->Tasks<!----><br><br></strong><strong><!---->What you’ll do:<!----><br><br></strong>      <ul><li><!---->You will be developing, maintaining and improving our methodologies and models for calculating companies GHG emissions<!----></li><li><!---->You will develop and oversee processes to ensure our clients receive accurate CCF and PCF results (eg. Quality assurance, validating user carbon footprints)<!----></li><li><!---->You will transform unstructured data sets so they are ready to be ingested into our calculation databases and models<!----></li><li><!---->You will work across Cozero’s climate, data science and engineering teams, to deliver on high quality data outputs for internal and external use<!----></li><li><!---->You will work closely with our Sales and Customer Success teams to define and prioritize data needs<!----></li><li><!---->You will proactively develop ideas on how we can create more value out of our data as well as external data for our clients<!----><br><br></li></ul>
<strong><!---->Requirements<!----><br><br></strong><strong><!---->Who you are:<!----><br><br></strong>      <ul><li><!---->You care deeply about our planet and staying informed on key environmental trends like CO2 emissions and climate change.<span class="white-space-pre"> </span></li><li><!---->Your meticulous eye for detail ensures thorough, systematic and accurate data analysis even when working with complex, unstructured and inconsistent datasets.<span class="white-space-pre"> </span></li><li><!---->A fast-evolving environment excites you, and you love contributing your skills to a diverse, collaborative team making a real difference in the world.<span class="white-space-pre"> </span><br><br></li></ul>
<strong><!---->What we’re looking for:<!----><br><br></strong>      <ul><li><!---->Experience working in an analyst role or similar (ideally working with climate data like Corporate GHG emissions, Emission factor and LCA databases)<!----></li><li><!---->You have programming skills in Python and SQL and a strong understanding of data gathering, inspecting, cleansing, transforming, and modeling techniques<!----></li><li><!---->You have strong verbal and written English communication skills<span class="white-space-pre"> </span><br><br></li></ul>
<strong><!---->Extra bonus points if you also have the following:<!----><br><br></strong>      <ul><li><!---->Master's degree in Environmental Science, Engineering, Computer Science or a related field<!----></li><li><!---->Knowledge of different carbon accounting standards and databases (e.g. GHG Protocol, GLEC, Ecoinvent)<!----></li><li><!---->Experience in carbon accounting and/ or life cycle assessment<!----><br><br></li></ul>
<strong><!---->Benefits<!----><br><br></strong><strong><!---->What we offer:<!----><br><br></strong>      <ul><li><!---->Opportunity to work remotely or from our great co-working space in Maschinenraum Berlin with access to innovation events of the German Mittelstand and the startup ecosystem<!----></li><li><!---->Virtual meditation and wellness sessions<!----></li><li><!---->Regular team events and at least 2 offsites per year<!----></li><li><!---->Individual learning &amp; development budget<!----></li><li><!---->The option to participate in our Virtual Stock Option Plan (VSOP), to have direct involvement and impact on Cozero’s success<!----></li><li><!---->A strong company culture focused on growth, collaboration and fighting climate change<!----></li><li><!---->Participation opportunities in accelerator, startup &amp; event programs<!----><br><br></li></ul>
<!---->We strive to build a diverse, inclusive and collaborative workplace and we follow a fair, objective, and non-discriminatory recruitment process. All applications will be considered for employment regardless of nationality, culture, or any other personal characteristics.<!----><br><br><!---->The glue that binds us together despite our differences is a common set of values, our purpose to transform every company into a low-emission business, and our culture that has a growth mindset at its heart. If you’re excited about this role we encourage you to apply, even if your past experience doesn’t perfectly match every requirement in the job description.<!---->
<!---->            </span>
          </div>
<!---->          <div class="jobs-description__details">
<!---->          </div>
        </div>
      </article>
        <footer>
          <button aria-label="Click to see more description" aria-expanded="false" id="ember36" class="jobs-description__footer-button t-14 t-black--light t-bold artdeco-card__action artdeco-button artdeco-button--icon-right artdeco-button--3 artdeco-button--fluid artdeco-button--tertiary ember-view">        <svg role="none" aria-hidden="true" class="artdeco-button__icon " xmlns="http://www.w3.org/2000/svg" width="16" height="16" viewBox="0 0 16 16" data-supported-dps="16x16" data-test-icon="chevron-down-small">
<!---->    

    <use href="#chevron-down-small" width="16" height="16"></use>
</svg>


<span class="artdeco-button__text">
    See more
</span></button>
        </footer>
    </div>
  
            
    <div class="artdeco-card pv5 mb4 full-width ph5">
      <div class="display-flex justify-space-between align-items-center">
        <div>
          <h2 class="t-sans t-black job-details-job-alert__header text-heading-large">
            Set alert for similar jobs
          </h2>
            <div class="job-details-job-alert__description text-body-small">
              <span>
                Data Analyst, Germany
              </span>
            </div>
        </div>
        <div>
          <button disabled="" id="ember37" class="artdeco-button artdeco-button--3 artdeco-button--secondary artdeco-button--disabled ember-view">        <svg role="none" aria-hidden="true" class="artdeco-button__icon " xmlns="http://www.w3.org/2000/svg" width="16" height="16" viewBox="0 0 16 16" data-supported-dps="16x16" data-test-icon="check-small">
<!---->    

    <use href="#check-small" width="16" height="16"></use>
</svg>


<span class="artdeco-button__text">
    Alert on
</span></button>
        </div>
      </div>
    </div>
  

<!---->
              <div id="SALARY" class="jobs-details__salary-main-rail-card">
<!----><!---->              </div>

                  
    <div class="job-details-how-you-match-card__container mb4 artdeco-card
        " id="how-you-match-card-container">
      <section class="job-details-how-you-match-card__header pt5 ph5 pb4">
<!---->        <h2 id="how-you-match-card-title" tabindex="-1" class="t-20 t-bold">
          How you match
        </h2>
          <div class="display-flex flex-row align-items-center mt4">
              <figure class="ivm-view-attr__system-image ivm-view-attr__system-image--small ivm-view-attr__system-image--illustration-microspot ivm-view-attr__system-image--pencil-ruler-small"></figure>
            <h2 class="job-details-how-you-match-card__header-content fit-content-width text-body-medium">
              3 skills match your profile. Stand out by adding other skills you have.
            </h2>
          </div>
<!---->      </section>

<!---->
          <section class="ph5">
            
    <div>
        <div class="pt5">
          <h3 class="t-16 t-bold">Skills associated with the job post</h3>
          <p class="job-details-how-you-match__skills-section-subheader t-12">Identified by LinkedIn</p>
            
    <div class="job-details-how-you-match__skills-item-wrapper display-flex flex-row pt4">
        <span class="ivm-view-attr__icon--signal-positive job-details-how-you-match__skills-item-icon">
          <svg role="none" aria-hidden="true" xmlns="http://www.w3.org/2000/svg" width="16" height="16" viewBox="0 0 16 16" data-supported-dps="16x16" data-test-icon="signal-success-small">
<!---->    

    <use href="#signal-success-small" width="16" height="16"></use>
</svg>

        </span>
      <div class="display-flex flex-column overflow-hidden">
          <h3 class="t-14 t-bold">3 skills on your profile</h3>
          <a class="app-aware-link  job-details-how-you-match__skills-item-subtitle t-14 overflow-hidden" href="#" data-test-app-aware-link="">Data Analysis, Data Analytics, and Unstructured Data</a>
      </div>
    </div>
  
            
    <div class="job-details-how-you-match__skills-item-wrapper display-flex flex-row pt4">
        <span class="ivm-view-attr__icon--signal-neutral job-details-how-you-match__skills-item-icon">
          <svg role="none" aria-hidden="true" xmlns="http://www.w3.org/2000/svg" width="16" height="16" viewBox="0 0 16 16" data-supported-dps="16x16" data-test-icon="signal-caution-small">
<!---->    

    <use href="#signal-caution-small" width="16" height="16"></use>
</svg>

        </span>
      <div class="display-flex flex-column overflow-hidden">
          <h3 class="t-14 t-bold">7 skills missing on your profile</h3>
          <a class="app-aware-link  job-details-how-you-match__skills-item-subtitle t-14 overflow-hidden" href="#" data-test-app-aware-link="">Carbon, Carbon Accounting, Data Collection, Databases, Datasets, Life Cycle Assessment, and Offsites</a>
      </div>
    </div>
  
<!---->        </div>
        
    <div class="ph4 pv3 mt4 display-flex align-items-center job-details-how-you-match__tip-wrapper--freemium">
<!---->        <div class="display-flex">
          <svg role="none" aria-hidden="true" class="mr2 job-details-how-you-match__tip-icon--freemium" xmlns="http://www.w3.org/2000/svg" width="24" height="24" viewBox="0 0 24 24" data-supported-dps="24x24" data-test-icon="lightbulb-medium">
<!---->    

    <use href="#lightbulb-medium" width="24" height="24"></use>
</svg>

        </div>
        <div class="text-body-small">
          <!---->Add skills you have to your profile to stand out to the employer.<span class="white-space-pre"> </span><a class="app-aware-link " target="_self" href="https://www.linkedin.com/jobs/skill-match?jobId=3768592532" data-test-app-aware-link=""><span><strong><!---->Add skills<!----></strong></span></a>
        </div>
<!---->    </div>
  
        <button id="ember86" class="mv5 t-16 pt1 pb1 artdeco-button artdeco-button--muted artdeco-button--icon-right artdeco-button--1 artdeco-button--secondary ember-view">        <svg role="none" aria-hidden="true" class="artdeco-button__icon  rtl-flip" xmlns="http://www.w3.org/2000/svg" width="16" height="16" viewBox="0 0 16 16" data-supported-dps="16x16" data-test-icon="arrow-right-small">
<!---->    

    <use href="#arrow-right-small" width="16" height="16"></use>
</svg>


<span class="artdeco-button__text">
    Show all skills
</span></button>
    </div>
  
          </section>

<!----><!---->
<!---->
          <section class="job-details-how-you-match-card__upsell pv4 ph5">
              
          

    <div class="align-items-flex-start pb3
              display-flex align-items-center
              flex-column
              ">
      <div class="display-flex flex-column">
<!---->          
<!---->          
        <div>
            <h3 class="t-16 t-bold">
              Achieve your goals faster with Premium
            </h3>

            <p class="t-14 t-black--light mb2">
              Get exclusive access to applicant insights, see jobs where you’d be a top applicant and more
            </p>

            <div class="display-flex align-items-flex-start mb3">
              
    <div class="ivm-image-view-model   ">
        <ul class="ivm-image-view-model__img-list--stacked list-style-none display-flex justify-center">
            <li class="ivm-image-view-model__img-list-item--stacked">
              
    <div class="ivm-view-attr__img-wrapper display-flex">
<!---->          <img width="32" src="https://media.licdn.com/dms/image/C5603AQE4h5-0UwsrOw/profile-displayphoto-shrink_100_100/0/1517793443567?e=1706745600&amp;v=beta&amp;t=vL5XEiLfXpHDRePgYWgkkstxh1ljlMpZs0vCtEGDNPw" loading="lazy" height="32" alt="" id="ember87" class="ivm-view-attr__img--centered ivm-view-attr__img--stacked ivm-view-attr__img--stacked-circle-size-1 EntityPhoto-circle-1-stackedFacepile   evi-image lazy-image ember-view">
    </div>
  
            </li>
            <li class="ivm-image-view-model__img-list-item--stacked">
              
    <div class="ivm-view-attr__img-wrapper display-flex">
<!---->          <img width="32" src="https://media.licdn.com/dms/image/D5603AQGUsLQAhDzKNg/profile-displayphoto-shrink_100_100/0/1690879601692?e=1706745600&amp;v=beta&amp;t=LfZ9rwFQnFkz_2Y3w6sR7UBQU76hSKYvIglEVkxXaYM" loading="lazy" height="32" alt="" id="ember88" class="ivm-view-attr__img--centered ivm-view-attr__img--stacked ivm-view-attr__img--stacked-circle-size-1 EntityPhoto-circle-1-stackedFacepile   evi-image lazy-image ember-view">
    </div>
  
            </li>
            <li class="ivm-image-view-model__img-list-item--stacked">
              
    <div class="ivm-view-attr__img-wrapper display-flex">
<!---->          <img width="32" src="https://media.licdn.com/dms/image/C5603AQH8wcMQgoBQ1A/profile-displayphoto-shrink_100_100/0/1588736600325?e=1706745600&amp;v=beta&amp;t=Abw1ANS7DCIw25UEoBzIGt7elFmtBFsodDI8e0c7vW0" loading="lazy" height="32" alt="" id="ember89" class="ivm-view-attr__img--centered ivm-view-attr__img--stacked ivm-view-attr__img--stacked-circle-size-1 EntityPhoto-circle-1-stackedFacepile   evi-image lazy-image ember-view">
    </div>
  
            </li>
        </ul>

<!---->    </div>
  
              <p class="t-12 t-black--light ml2 align-self-center">
                <!---->Crystine PL and millions of other members use Premium<!---->
              </p>
            </div>
        </div>
      </div>
<!---->        <a class="app-aware-link  artdeco-button artdeco-button--premium artdeco-button--secondary " href="https://www.linkedin.com/premium/products/?upsellOrderOrigin=premium_job_details_how_you_match_upsell&amp;utype=job&amp;referenceId=rmtcapsfTIW2Dx4aX44fvA%3D%3D&amp;destRedirectURL=https%3A%2F%2Fwww.linkedin.com%2Fjobs%2Fview%2F3768592532%2F%3FeBP%3DJOB_SEARCH_ORGANIC%26refId%3DzJUYZKcKNK0KckLEpIRG6g%253D%253D%26trackingId%3DLwVF686xVu9u%252FR6wEjnRaQ%253D%253D%26trk%3Dflagship3_search_srp_jobs" data-test-app-aware-link="">
          Try Premium for free
        </a>
        <p class="t-12 card-upsell__subtext">
          1-month free trial. Cancel anytime.
        </p>
    </div>
  
  
          </section>
    </div>
  


                    
    <section id="ember71" class="artdeco-card ember-view jobs-box jobs-details__smp-promo artdeco-card"><!---->

        <div class="display-flex pt2">
          <div class="smp-promos-card__icon"></div>
          <div>
            <h2 class="smp-promos-card__title text-heading-large">
              
                        <span dir="ltr"><span aria-hidden="true"><!---->Put your best foot forward with your application<!----></span><span class="visually-hidden"><!---->Put your best foot forward with your application<!----></span></span>
                      
            </h2>
            <p class="smp-promos-card__subtitle text-body-small">
              
                        <span dir="ltr"><span aria-hidden="true"><!---->Hire a resume writer<!----></span><span class="visually-hidden"><!---->Hire a resume writer<!----></span></span>
                      
            </p>
          </div>
        </div>
      <div class="smp-promos-card__button--jdp">
          <div data-view-name="marketplace-jobs-distribution-platform-promo-cta">
            <a class="app-aware-link  full-width artdeco-button artdeco-button--2 artdeco-button--secondary" target="_self" href="https://www.linkedin.com/service-marketplace/create-project/1005?trk=profinder_neptune_jobdetails_rotation_with_premium" data-test-app-aware-link="">
              <span class="artdeco-button__text">
                
                        Get a resume review
                      
              </span>
            </a>
          </div>
      </div>
    </section>
  
<!---->
                <section id="ember72" class="artdeco-card ember-view"><!---->

                  
    <section class="jobs-company jobs-box--fadein mb4">
      <div class="jobs-company__box">
        <h2 class="text-heading-large">
          About the company
        </h2>

        <div class="display-flex align-items-center mt5">
          <div id="ember73" class="artdeco-entity-lockup artdeco-entity-lockup--size-5 ember-view flex-grow-1">
            <div id="ember74" class="artdeco-entity-lockup__image artdeco-entity-lockup__image--type-square ember-view" type="square">
              <a href="/company/cozero-io/life/" id="ember75" class="ember-view link-without-hover-state inline-block">
                <img title="Cozero" src="https://media.licdn.com/dms/image/C4D0BAQGkpvwIXWu1-g/company-logo_100_100/0/1638439865071/cozero_io_logo?e=1709164800&amp;v=beta&amp;t=sVPH2ggrA8QHrpoTseVt0_N71x9Hg_xcONvZl48_Ogk" alt="Cozero company logo" id="ember76" class="evi-image ember-view">
              </a>
            
</div>
            <div id="ember77" class="artdeco-entity-lockup__content ember-view flex-grow-1">
              <div id="ember78" class="artdeco-entity-lockup__title ember-view t-20">
                <a href="/company/cozero-io/life/" id="ember79" class="ember-view link-without-visited-state inline-block t-black">
                  Cozero
                </a>
              
</div>
              <div id="ember80" class="artdeco-entity-lockup__subtitle ember-view t-16">
                5,298 followers
              </div>
            </div>
          
</div>
          <button class="follow   artdeco-button artdeco-button--secondary
              
              ml5" aria-label="Follow" aria-live="polite" type="button">
      <svg role="none" aria-hidden="true" class="artdeco-button__icon" xmlns="http://www.w3.org/2000/svg" width="16" height="16" viewBox="0 0 16 16" data-supported-dps="16x16" data-test-icon="add-small">
<!---->    

    <use href="#add-small" width="16" height="16"></use>
</svg>

    <span aria-hidden="true">Follow</span>
</button>
        </div>

        <div class="t-14 mt5">
          Software Development
            <span class="jobs-company__inline-information">
              11-50 employees
            </span>
            <span class="jobs-company__inline-information">
              35 on LinkedIn
            </span>
        </div>

        <p class="jobs-company__company-description text-body-small-open">
          
    <div class="inline-show-more-text
        inline-show-more-text--is-collapsed
        inline-show-more-text--is-collapsed-with-line-clamp
        
        
        " style="-webkit-line-clamp:3;" tabindex="-1">

        Take control of your corporate emission data with Cozero's digital Climate Action Platform (CAP). Cozero tracks, optimizes, and engages on your carbon performance. Forget endless excel files &amp; project sounding boards. Cozero digitally unites all carbon management data in one end-to-end platform, so that you can easily keep track - and act. Customize your climate action platform for your industry and company. Get smart on your carbon emissions and take the right decisions. <br><br>Cozero was founded in 2020 with the purpose to enable the corporate sector in a transformational shift to a lower emission economy. We strongly believe that the private sector has the potential to make this transformation possible and to fight climate change effectively. <br><br>Whoever you are - be it a business leader, a climate action enthusiast, a potential low carbon service or technology partner or a motivated individual who wants to join our mission - please reach out and get on board: info@cozero.io
      
          <span class="inline-show-more-text__link-container-collapsed">
              <span>…</span>
            <button class="inline-show-more-text__button
                inline-show-more-text__button--light
                link" aria-expanded="false" role="button" type="button">
              show more
            </button>
          </span>

<!---->    </div>
  
        </p>

<!---->
<!---->
<!---->      </div>

<!---->
        <div class="jobs-company__footer text-align-center">
          <a href="/company/cozero-io/life/" id="ember81" class="ember-view link-without-hover-state block pv3" aria-label="Show more about the company">
            Show more
          </a>
        </div>
    </section>
  
                </section>

<!---->
<!---->
<!---->                    </div>

<!---->      </div>

<!---->
      
    <div id="ember39" class="ember-view"><!----></div>

    <div id="ember40" class="ember-view"><div id="ember41" class="ember-view"><!----></div></div>
  

      
  <div id="ember42" class="ember-view"><!----></div>


<!---->
<!---->
<!---->
      
    <div>
        
    <div id="ember43" class="ember-view"><!----></div>
  
    </div>
  

<!---->
<!---->    </div>
  
        
          
</main>

          <aside class="scaffold-layout__aside
    
    ">
  
            
          <div class="pt4">
              <div class="mb4">
                  
          

    <div class="artdeco-card justify-center p4
              align-items-left
              display-flex flex-column
              
              ">
      <div class="display-flex flex-column">
<!---->          
<!---->          
        <div class="mb3">
            <h3 class="text-heading-medium">
              Get ahead with Premium Career
            </h3>

            <p class="mt2 t-14">
              Contact recruiters directly, see who's viewed your profile, stand out as a top applicant, and more.
            </p>

            <div class="display-flex align-items-center mt2">
              
    <div class="ivm-image-view-model   ">
        <ul class="ivm-image-view-model__img-list--stacked list-style-none display-flex justify-center">
            <li class="ivm-image-view-model__img-list-item--stacked">
              
    <div class="ivm-view-attr__img-wrapper display-flex">
<!---->          <img width="32" src="https://media.licdn.com/dms/image/C5603AQE4h5-0UwsrOw/profile-displayphoto-shrink_100_100/0/1517793443567?e=1706745600&amp;v=beta&amp;t=vL5XEiLfXpHDRePgYWgkkstxh1ljlMpZs0vCtEGDNPw" loading="lazy" height="32" alt="" id="ember83" class="ivm-view-attr__img--centered ivm-view-attr__img--stacked ivm-view-attr__img--stacked-circle-size-1 EntityPhoto-circle-1-stackedFacepile   evi-image lazy-image ember-view">
    </div>
  
            </li>
            <li class="ivm-image-view-model__img-list-item--stacked">
              
    <div class="ivm-view-attr__img-wrapper display-flex">
<!---->          <img width="32" src="https://media.licdn.com/dms/image/D5603AQGUsLQAhDzKNg/profile-displayphoto-shrink_100_100/0/1690879601692?e=1706745600&amp;v=beta&amp;t=LfZ9rwFQnFkz_2Y3w6sR7UBQU76hSKYvIglEVkxXaYM" loading="lazy" height="32" alt="" id="ember84" class="ivm-view-attr__img--centered ivm-view-attr__img--stacked ivm-view-attr__img--stacked-circle-size-1 EntityPhoto-circle-1-stackedFacepile   evi-image lazy-image ember-view">
    </div>
  
            </li>
            <li class="ivm-image-view-model__img-list-item--stacked">
              
    <div class="ivm-view-attr__img-wrapper display-flex">
<!---->          <img width="32" src="https://media.licdn.com/dms/image/C5603AQH8wcMQgoBQ1A/profile-displayphoto-shrink_100_100/0/1588736600325?e=1706745600&amp;v=beta&amp;t=Abw1ANS7DCIw25UEoBzIGt7elFmtBFsodDI8e0c7vW0" loading="lazy" height="32" alt="" id="ember85" class="ivm-view-attr__img--centered ivm-view-attr__img--stacked ivm-view-attr__img--stacked-circle-size-1 EntityPhoto-circle-1-stackedFacepile   evi-image lazy-image ember-view">
    </div>
  
            </li>
        </ul>

<!---->    </div>
  
              <p class="t-12 t-black--light ml2">
                <!---->Crystine PL and millions of other members use Premium<!---->
              </p>
            </div>
        </div>
      </div>
<!---->        <a class="app-aware-link  artdeco-button artdeco-button--premium artdeco-button--secondary fit-content-width" href="https://www.linkedin.com/premium/products/?upsellOrderOrigin=job_right_rail_upsell&amp;utype=job&amp;referenceId=UZX%2BvE5RREmdRVUCFTO1Qw%3D%3D&amp;destRedirectURL=https%3A%2F%2Fwww.linkedin.com%2Fjobs%2Fview%2F3768592532%2F%3FeBP%3DJOB_SEARCH_ORGANIC%26refId%3DzJUYZKcKNK0KckLEpIRG6g%253D%253D%26trackingId%3DLwVF686xVu9u%252FR6wEjnRaQ%253D%253D%26trk%3Dflagship3_search_srp_jobs" data-test-app-aware-link="">
          Try Premium for free
        </a>
<!---->    </div>
  
  
              </div>

              
  <div class="jobs-post-job artdeco-card jobs-box--inset">
    <p class="jobs-box__body jobs-post-job__title">
      Looking for talent?
    </p>

    <a tabindex="0" rel="noopener noreferrer" target="_blank" href="https://www.linkedin.com/talent/job-posting-redirect?trk=flagship3_job_detail" id="ember57" class="jobs-box__link jobs-post-job__link artdeco-button artdeco-button--secondary ember-view">
      Post a job
    </a>
  </div>

          </div>
        
          
</aside>
      </div>

<!---->  </div>
</div>
  


        <div id="ember44" class="ember-view jobs-box--generic-occludable-area-small  ">
<!---->        </div>
  
</div>

<div class="application-outlet__overlay-container">
    <!----><aside id="msg-overlay" class="msg-overlay-container msg-overlay-container-reflow
    ">
  
    <div tabindex="-1" class="msg-overlay-list-bubble
        msg-overlay-list-bubble--is-minimized
        
        ml4">
      <header class="msg-overlay-bubble-header">
        <div class="msg-overlay-bubble-header__badge-container
            "></div>
        
<!---->  
        <div class="msg-overlay-bubble-header__details flex-row align-items-center ml1">
          
    <div class="presence-entity presence-entity--size-1">
      <img src="https://media.licdn.com/dms/image/C5603AQEkRciOp6B6dA/profile-displayphoto-shrink_100_100/0/1605279012830?e=1706745600&amp;v=beta&amp;t=GoGDD2DE3nGXji1ktdhEWgWX0uNRNd93omIm9L4PTBM" loading="lazy" alt="Naim Iskandar Zahari" id="ember51" class="presence-entity__image EntityPhoto-circle-1  evi-image lazy-image ember-view">

        

    <div class="presence-entity__indicator presence-entity__indicator--size-1
             presence-indicator
        presence-indicator--is-online
        presence-indicator--size-1">
      <span class="visually-hidden">
          Status is online
      </span>
    </div>
  
    </div>
  
          <button class="msg-overlay-bubble-header__button truncate ml2" type="button">
            <span class="truncate t-14 t-bold
                t-black">
              <span aria-hidden="true">
                Messaging
              </span>
              <span class="visually-hidden">
                You are on the messaging overlay. Press enter to open the list of conversations.
              </span>
            </span>
          </button>

<!----><!---->        </div>
        <div class="msg-overlay-bubble-header__controls display-flex">
          <div id="ember52" class="artdeco-dropdown artdeco-dropdown--placement-top artdeco-dropdown--justification-right ember-view">
            <button aria-expanded="false" id="ember53" class="msg-overlay-bubble-header__dropdown-trigger artdeco-button artdeco-button--1 artdeco-button--circle artdeco-button--muted artdeco-button--tertiary artdeco-dropdown__trigger artdeco-dropdown__trigger--placement-top ember-view" type="button" tabindex="0">
              <svg role="img" aria-hidden="false" aria-label="Open messenger dropdown menu" xmlns="http://www.w3.org/2000/svg" width="16" height="16" viewBox="0 0 16 16" data-supported-dps="16x16" data-test-icon="overflow-web-ios-small">
<!---->    

    <use href="#overflow-web-ios-small" width="16" height="16"></use>
</svg>

            
<!----></button>
            <div class="msg-overlay-bubble-header__dropdown-container">
              <div tabindex="-1" aria-hidden="true" id="ember54" class="artdeco-dropdown__content artdeco-dropdown--is-dropdown-element artdeco-dropdown__content--has-arrow artdeco-dropdown__content--arrow-right artdeco-dropdown__content--justification-right artdeco-dropdown__content--placement-top ember-view"><!----></div>
            </div>
          </div>

          <button id="ember55" class="msg-overlay-bubble-header__control msg-overlay-bubble-header__control--new-convo-btn artdeco-button artdeco-button--circle artdeco-button--muted artdeco-button--1 artdeco-button--tertiary ember-view">        <svg role="none" aria-hidden="true" class="artdeco-button__icon " xmlns="http://www.w3.org/2000/svg" width="16" height="16" viewBox="0 0 16 16" data-supported-dps="16x16" data-test-icon="compose-small">
<!---->    

    <use href="#compose-small" width="16" height="16"></use>
</svg>


<span class="artdeco-button__text">
    Compose message
</span></button>

          <button id="ember56" class="msg-overlay-bubble-header__control msg-overlay-bubble-header__control--new-convo-btn artdeco-button artdeco-button--circle artdeco-button--muted artdeco-button--1 artdeco-button--tertiary ember-view">        <svg role="none" aria-hidden="true" class="artdeco-button__icon " xmlns="http://www.w3.org/2000/svg" width="16" height="16" viewBox="0 0 16 16" data-supported-dps="16x16" data-test-icon="chevron-up-small">
<!---->    

    <use href="#chevron-up-small" width="16" height="16"></use>
</svg>


<span class="artdeco-button__text">
    You are on the messaging overlay. Press enter to open the list of conversations.
</span></button>
        </div>
      </header>

      
<!---->  

<!---->
<!----><!---->    </div>
  

<!---->
<!---->
  <div id="msg-overlay__emoji-hoverable-outlet"></div>

  <div id="msg-overlay__reactor-list-outlet"></div>
</aside>

      
    <aside class="coach-container
        coach-container--overlay
        
        coach-container--standalone-page" id="coach-container" tabindex="-1">
<!---->    </aside>
  
<!----></div>

<!---->
  
      
        
    <div id="ember48" class="ember-view"><!----></div>
  
    
  

  
      
    <img src="https://px.ads.linkedin.com/collect/?pid=6883&amp;fmt=gif&amp;_t=1701030053004" alt="" role="none" class="third-party-tracking-pixel hidden" data-test-third-party-tracking-pixel="">
  
  

</div>

<div id="artdeco-hoverable-outlet"></div>

<!---->
<div id="type-ahead-wormhole" class="type-ahead type-ahead-wormhole">
</div>

<div id="toast-wormhole" class="toast-wormhole">
</div>

<div id="profile-inline-modal-outlet">
</div>

  <iframe title="cedex beacon iframe" sandbox="allow-scripts" style="display:none;" src="//radar.cedexis.com/1/11326/radar.html">
  </iframe>


  <div><!----></div>
<code style="display: none" id="clientPageInstance">
  urn:li:page:d_flagship3_job_details;4deffb7b-0190-47ed-9b38-6a728e1de036
</code>

<img src="data:image/gif;base64,R0lGODlhAQABAIAAAAAAAP///yH5BAEAAAAALAAAAAABAAEAAAIBRAA7" style="display: none" class="terminatorlet">



    <div id="a11y-notification" class="visually-hidden" role="region" aria-live="polite"></div>
  

</body>
"""

In [6]:
# def diveDictValues(keyword = None, dict_to_search = {}):
#     if keyword is None:
#         return None
#     if dict_to_search is False:
#         return None
    
#     for key in dict_to_search.keys():
#         try:
#             if str(dict_to_search[key]).__contains__(keyword):
#                 return key
#         except KeyError:
#             continue

#     return None

# def diveListValues(keyword = None, list_to_search = []):
#     if keyword is None:
#         return None
#     if list_to_search is False:
#         return None
    
#     for i in range(len(list_to_search)):
#         if str(list_to_search[i]).__contains__(keyword):
#             return i

#     return None

# def ProcessHtmlText(dict_to_search = None, dict_labels = []):
#     if dict_to_search is None:
#         return None
    
#     curr_dict_level = dict_to_search
#     result = None

#     try:
#         for label in dict_labels:
#             if label.__contains__('list'):
#                 if type(curr_dict_level) == list:
#                     dive_result = diveListValues(label.split('-')[1], curr_dict_level)
#                     if dive_result is None:
#                         continue

#                     curr_dict_level = curr_dict_level[dive_result]
#             else:
#                 if type(curr_dict_level) == dict:
#                     try:
#                         # dive_result = diveDictValues(label, curr_dict_level)
#                         # if dive_result is None:
#                         #     continue
                        
#                         # print(f"label: {label}; result: {dive_result}")
#                         curr_dict_level = curr_dict_level[label]
#                     except KeyError:
#                         continue

#             if label == dict_labels[-1]:
#                 result = curr_dict_level
#     except ValueError:
#         return None

#     return result


# def diveHtmlTags(html_body, toplevel_tag = None, dict_labels = [], multi = False):
#     soup = bs(html_body, 'lxml')
#     if toplevel_tag is None:
#         return None
#     if dict_labels is False:
#         return None
    
#     ids = [ids['id'] for ids in soup.find_all(toplevel_tag, id=True)]
    
#     result = None
#     if multi == False:
#         for id_key in ids:
#             long_text = soup.find(id = id_key).text
#             try:
#                 json_dict = json.loads(long_text, strict = False)

#                 if json_dict is None:
#                     continue
                
#                 result_check = ProcessHtmlText(json_dict, dict_labels=dict_labels)
#                 if result_check is not None:
#                     result = result_check
#             except ValueError:  # includes simplejson.decoder.JSONDecodeError
#                 # print(f'Decoding JSON has failed at {id_key}')
#                 continue
#     else:
#         result = []
#         for id_key in ids:
#             print(id_key)
#             long_text = soup.find(id = id_key).text
#             if id_key == 'bpr-guid-15912217':
#                 print('1')
#                 print(long_text)
#                 print('2')
#                 print(json.loads(long_text, strict = False))
#                 result_check = ProcessHtmlText(json_dict, dict_labels=dict_labels)
#                 print(result_check)
#             try:
#                 json_dict = json.loads(long_text, strict = False)

#                 if json_dict is None:
#                     continue
                
#                 result_check = ProcessHtmlText(json_dict, dict_labels=dict_labels)
#                 if result_check is not None:
#                     print('rc')
#                     print(f"id: {id_key}")
#                     print(result_check)
#                     print('--')
#                     result.append(result_check)
#             except ValueError:  # includes simplejson.decoder.JSONDecodeError
#                 print(f'Decoding JSON has failed at {id_key}')
#                 continue

#     return result

def diveDictValues(keyword = None, dict_to_search = {}):
    if keyword is None:
        return None
    if dict_to_search is False:
        return None
    
    for key in dict_to_search.keys():
        try:
            if str(dict_to_search[key]).__contains__(keyword):
                return key
        except KeyError:
            continue

    return None

def diveListValues(keyword = None, list_to_search = []):
    if keyword is None:
        return None
    if list_to_search is False:
        return None
    
    matched_items = []
    for i in range(len(list_to_search)):
        if type(list_to_search[i]) == dict:
            if keyword in list(list_to_search[i].keys()):
                matched_items.append(i)
        elif str(list_to_search[i]).__contains__(keyword):
            matched_items.append(i)

    return matched_items

def ProcessHtmlText(dict_to_search = None, dict_labels = []):
    if dict_to_search is None:
        return None
    
    curr_dict_level = dict_to_search
    result = None

    try:
        for i in range(len(dict_labels)):
            label = dict_labels[i]
            if label.__contains__('list'):
                if type(curr_dict_level) == list:
                    dive_result = diveListValues(label.split('-')[1], curr_dict_level)
                    if dive_result is None:
                        continue
                    elif len(dive_result) <= 0:
                        continue
                    elif len(dive_result) > 1:
                        possible_nests_to_check = {}
                        for j in dive_result:
                            possible_nests_to_check[j] = curr_dict_level[j]
                        accepted_nests = AttemptPossibleNest(possible_nests_to_check, dict_labels[i+1:])
                        dive_result = accepted_nests[-1]
                    elif len(dive_result) == 1:
                        dive_result = dive_result[0]
 
                    curr_dict_level = curr_dict_level[dive_result]
            else:
                if type(curr_dict_level) == dict:
                    try:
                        # dive_result = diveDictValues(label, curr_dict_level)
                        # if dive_result is None:
                        #     continue
                        
                        # print(f"label: {label}; result: {dive_result}")
                        curr_dict_level = curr_dict_level[label]
                    except KeyError:
                        continue

            if label == dict_labels[-1]:
                result = curr_dict_level
    except ValueError:
        return None

    return result

def AttemptPossibleNest(possible_nest_dict = {}, next_labels = []):
    accepted_nests = []
    for key in possible_nest_dict.keys():
        nest_to_dive = possible_nest_dict[key]
        
        # print('nesting', key, next_labels, nest_to_dive)
        result_check = ProcessHtmlText(nest_to_dive, dict_labels=next_labels)
        if result_check is not None:
            accepted_nests.append(key)
    
    return accepted_nests

def diveHtmlTags(html_body, toplevel_tag = None, dict_labels = [], multi = False):
    soup = bs(html_body, 'lxml')
    if toplevel_tag is None:
        return None
    if dict_labels is False:
        return None
    
    ids = [ids['id'] for ids in soup.find_all(toplevel_tag, id=True)]
    
    result = None
    if multi == False:
        for id_key in ids:
            long_text = soup.find(id = id_key).text
            try:
                json_dict = json.loads(long_text, strict = False)

                if json_dict is None:
                    continue
                
                result_check = ProcessHtmlText(json_dict, dict_labels=dict_labels)
                if result_check is not None:
                    result = result_check
            except ValueError:  # includes simplejson.decoder.JSONDecodeError
                # print(f'Decoding JSON has failed at {id_key}')
                continue
    else:
        result = []
        for id_key in ids:
            long_text = soup.find(id = id_key).text
            try:
                json_dict = json.loads(long_text, strict = False)

                if json_dict is None:
                    continue
                
                result_check = ProcessHtmlText(json_dict, dict_labels=dict_labels)
                if result_check is not None:
                    print('rc')
                    print(f"id: {id_key}")
                    print(result_check)
                    print('--')
                    result.append(result_check)
            except ValueError:  # includes simplejson.decoder.JSONDecodeError
                print(f'Decoding JSON has failed at {id_key}')
                continue

    return result

# def diveDictValues(keyword = None, dict_to_search = {}):
#     if keyword is None:
#         return None
#     if dict_to_search is False:
#         return None
    
#     for key in dict_to_search.keys():
#         try:
#             if str(dict_to_search[key]).__contains__(keyword):
#                 return key
#         except KeyError:
#             continue

#     return None

# def diveListValues(keyword = None, list_to_search = []):
#     if keyword is None:
#         return None
#     if list_to_search is False:
#         return None
    
#     matched_items = []
#     for i in range(len(list_to_search)):
#         if type(list_to_search[i]) == dict:
#             if keyword in list(list_to_search[i].keys()):
#                 matched_items.append(i)
#         elif str(list_to_search[i]).__contains__(keyword):
#             matched_items.append(i)

#     return matched_items

# def AttemptPossibleNest(possible_nest_dict = {}, next_labels = []):
#     accepted_nests = []
#     for key in possible_nest_dict.keys():
#         nest_to_dive = possible_nest_dict[key]
        
#         # print('nesting', key, next_labels, nest_to_dive)
#         result_check = ProcessHtmlText(nest_to_dive, dict_labels=next_labels)
#         if result_check is not None:
#             accepted_nests.append(key)
    
#     return accepted_nests


# def ProcessHtmlText(dict_to_search = None, dict_labels = []):
#     if dict_to_search is None:
#         return None
    
#     curr_dict_level = dict_to_search
#     result = None

#     for i in range(len(dict_labels)):
#         label = dict_labels[i]
#         if label.__contains__('list'):
#             if type(curr_dict_level) == list:
#                 dive_result = diveListValues(label.split('-')[1], curr_dict_level)
#                 if dive_result is None:
#                     continue
#                 elif len(dive_result) <= 0:
#                     continue
#                 elif len(dive_result) > 1:
#                     possible_nests_to_check = {}
#                     for j in dive_result:
#                         possible_nests_to_check[j] = curr_dict_level[j]
#                     accepted_nests = AttemptPossibleNest(possible_nests_to_check, dict_labels[i+1:])
#                     # print('accepted nests:', accepted_nests)
#                     dive_result = accepted_nests[-1]
#                 elif len(dive_result) == 1:
#                     dive_result = dive_result[0]
#                     # print('dr:', dive_result)
#                     # print('checking\n', curr_dict_level[dive_result])

#                 # print(dive_result)
#                 # print('dr check')
#                 # print(curr_dict_level[dive_result])  
#                 # print('crr level\n', curr_dict_level)  
#                 curr_dict_level = curr_dict_level[dive_result]
#                 # print(dict_labels[i], dict_labels[i:], curr_dict_level)
#         else:
#             if type(curr_dict_level) == dict:
#                 try:
#                     curr_dict_level = curr_dict_level[label]
#                 except KeyError:
#                     continue
#                 # print(dict_labels[i], dict_labels[i:], curr_dict_level)

#     # print('final\n', curr_dict_level)
#     return curr_dict_level


# def diveHtmlTags(html_body, toplevel_tag = None, dict_labels = [], multi = False):
#     soup = bs(html_body)
#     if toplevel_tag is None:
#         return None
#     if dict_labels is False:
#         return None
    
#     ids = [ids['id'] for ids in soup.find_all(toplevel_tag, id=True)]
    
#     result = None
#     if multi == False:
#         for id_key in ids:
#             long_text = soup.find(id = id_key).text
#             try:
#                 json_dict = json.loads(long_text, strict = False)

#                 if json_dict is None:
#                     continue
                
#                 result_check = ProcessHtmlText(json_dict, dict_labels=dict_labels)
#                 if result_check is not None:
#                     result = result_check
#                     return result
#             except ValueError:  # includes simplejson.decoder.JSONDecodeError
#                 # print(f'Decoding JSON has failed at {id_key}')
#                 continue
#     else:
#         result = []
#         for id_key in ids:
#             long_text = soup.find(id = id_key).text
#             try:
#                 json_dict = json.loads(long_text, strict = False)

#                 if json_dict is None:
#                     continue
                
#                 result_check = ProcessHtmlText(json_dict, dict_labels=dict_labels)
#                 if result_check is not None:
#                     result.append(result_check)
#             except ValueError:  # includes simplejson.decoder.JSONDecodeError
#                 print(f'Decoding JSON has failed at {id_key}')
#                 continue

#     # print('result\n', result)
#     return result

def GetJobPostingID(html_body):
    # soup = bs(html_body)
    # # id: bpr-guid-15912216
    # id_key = 'bpr-guid-15912216'
    # long_text = soup.find(id = id_key).text

    # json_dict = json.loads(long_text, strict = False)
    # return json_dict['data']['jobPostingId']
    return diveHtmlTags(html_body, toplevel_tag='code', dict_labels = ['data', 'jobPostingId'])

def GetPostStatus(html_body):
    # soup = bs(html_body)
    # # id: bpr-guid-15912216
    # id_key = 'bpr-guid-15912216'
    # long_text = soup.find(id = id_key).text

    # json_dict = json.loads(long_text, strict = False)
    # return json_dict['data']['jobState']
    return diveHtmlTags(html_body, toplevel_tag='code', dict_labels = ['data', 'jobState'])

def GetPostLink(url_link = None, job_id = None):
    if url_link is not None:
        return url_link
    if job_id is not None:
        return f'https://www.linkedin.com/jobs/view/{job_id}/'
    
    return None

def GetJobTitle(html_body):
    # soup = bs(html_body)
    # # id: bpr-guid-15912216
    # id_key = 'bpr-guid-15912216'
    # long_text = soup.find(id = id_key).text

    # json_dict = json.loads(long_text, strict = False)
    # return json_dict['data']['title']
    return diveHtmlTags(html_body, toplevel_tag='code', dict_labels = ['data', 'title'])

def GetJobDescription(html_body):
    # soup = bs(html_body)
    # # id: bpr-guid-15912216
    # id_key = 'bpr-guid-15912216'
    # long_text = soup.find(id = id_key).text

    # json_dict = json.loads(long_text, strict = False)
    # return json_dict['data']['description']['text']
    return diveHtmlTags(html_body, toplevel_tag='code', dict_labels = ['data', 'description', 'text'])

def GetDescriptionLanguage(jd_text):
    try:
        return detect(jd_text)
    except:
        return ''

def GetRemoteWorkStatus(html_body):
    # soup = bs(html_body)
    # # id: bpr-guid-15912216
    # id_key = 'bpr-guid-15912216'
    # long_text = soup.find(id = id_key).text

    # json_dict = json.loads(long_text, strict = False)
    # return json_dict['data']['workRemoteAllowed']
    return diveHtmlTags(html_body, toplevel_tag='code', dict_labels = ['data', 'workRemoteAllowed'])

def GetRepostStatus(html_body):
    # soup = bs(html_body)
    # # id: bpr-guid-15912216
    # id_key = 'bpr-guid-15912216'
    # long_text = soup.find(id = id_key).text

    # json_dict = json.loads(long_text, strict = False)
    # return json_dict['data']['repostedJobPosting']
    return diveHtmlTags(html_body, toplevel_tag='code', dict_labels = ['data', 'repostedJobPosting'])

def GetJobCountry(html_body):
    # soup = bs(html_body)
    # # id: bpr-guid-15912216
    # id_key = 'bpr-guid-15912216'
    # long_text = soup.find(id = id_key).text

    # json_dict = json.loads(long_text, strict = False)
    # return json_dict['data']['formattedLocation']
    return diveHtmlTags(html_body, toplevel_tag='code', dict_labels = ['data', 'formattedLocation'])

def GetJobTopCardDetails(html_body):
    # soup = bs(html_body)
    # # id: bpr-guid-15912217
    # id_key = 'bpr-guid-15912217'
    # long_text = soup.find(id = id_key)
    # json_dict = json.loads(long_text.text, strict = False)
    # return [t['text']['text'] for t in json_dict['included'][4]['jobInsightsV2ResolutionResults'][0]['jobInsightViewModel']['description']]
    dive_result = diveHtmlTags(html_body, toplevel_tag='code', dict_labels = ['included', 'list-jobInsightsV2ResolutionResults', 'jobInsightsV2ResolutionResults','list-jobInsightViewModel', 'jobInsightViewModel', 'description'])
    # print('dr type', type(dive_result))
    # print(dive_result.keys())
    # print(dive_result)
    # print([t.keys() for t in dive_result])
    return [t['text']['text'] for t in dive_result]

def GetJobExperienceLevel(html_body, details = []):
    if details is True:
        for keyword in ['Internship', 'Entry', 'Associate', 'Mid-Senior', 'Director', 'Executive']:
            for detail in details:
                if keyword in detail:
                    return detail
                
    details = GetJobTopCardDetails(html_body)
    for keyword in ['Internship', 'Entry', 'Associate', 'Mid-Senior', 'Director', 'Executive']:
        for detail in details:
            if keyword in detail:
                return detail
    
    return None

def GetJobWorkSetting(html_body, details = []):
    if details is True:
        for keyword in ['Hybrid', 'Remote', 'On-site']:
            for detail in details:
                if keyword in detail:
                    return detail
                
    details = GetJobTopCardDetails(html_body)
    for keyword in ['Hybrid', 'Remote', 'On-site']:
        for detail in details:
            if keyword in detail:
                return detail
    
    return None

def GetJobType(html_body, details = []):
    if details is True:
        for keyword in ['Full-time', 'Part-time', 'Contract', 'Temporary', 'Volunteer', 'Internship', 'Other']:
            for detail in details:
                if keyword in detail:
                    return detail
                
    details = GetJobTopCardDetails(html_body)
    for keyword in ['Full-time', 'Part-time', 'Contract', 'Temporary', 'Volunteer', 'Internship', 'Other']:
        for detail in details:
            if keyword in detail:
                return detail
    
    return None

def GetJobSkills(html_body):
    soup = bs(html_body)

    return [li.find('div').get_text().strip() for li in soup.find('ul', {'class': 'job-details-skill-match-status-list'}).find_all('li')]

def GetJobYearsExperience(html_body, job_des = None):
    def GetYears_RegEx(des):
        years_text = re.findall(r"([\d+-]+)\s+(years?)", des, re.IGNORECASE)
        splitted_num = []
        for clipped in years_text:
            char_to_remove = ['+', '-', ' ']
            text_to_clean = clipped[0]

            for ch in char_to_remove:
                for el in text_to_clean.split(ch):
                    if el.isalnum() is True:
                        splitted_num = splitted_num + [int(el)]

        return np.min(list(set(splitted_num)))

    if job_des is True:
        return GetYears_RegEx(job_des)
    
    job_des = GetJobDescription(html_body)
    if job_des is True:
        return GetYears_RegEx(job_des)

    return None

def GetCompanyID(html_body):
    # soup = bs(html_body)
    # # id: bpr-guid-15912216
    # id_key = 'bpr-guid-15912216'
    # long_text = soup.find(id = id_key).text

    # json_dict = json.loads(long_text, strict = False)
    # return json_dict['data']['companyDetails']['company']
    return diveHtmlTags(html_body, toplevel_tag='code', dict_labels = ['data', 'companyDetails', 'company'])

def GetCompanyName(html_body):
    # soup = bs(html_body)
    # # id: bpr-guid-15912216
    # id_key = 'bpr-guid-15912216'
    # long_text = soup.find(id = id_key).text

    # json_dict = json.loads(long_text, strict = False)
    # return json_dict['included'][1]['name']
    return diveHtmlTags(html_body, toplevel_tag='code', dict_labels = ['included', 'list-name', 'name'])

def GetCompanyLink(html_body):
    # soup = bs(html_body)
    # # id: bpr-guid-15912216
    # id_key = 'bpr-guid-15912216'
    # long_text = soup.find(id = id_key).text

    # json_dict = json.loads(long_text, strict = False)
    # return json_dict['included'][1]['url']
    return diveHtmlTags(html_body, toplevel_tag='code', dict_labels = ['included', 'list-url', 'url', 'list-primaryDescription', 'primaryDescription', 'attributesV2', 'list-detailData', 'detailData', 'hyperlink'])
    
def GetCompanyTopCardDetails(html_body):
    soup = bs(html_body, 'lxml')
    # id: bpr-guid-15912217
    # id_key = 'bpr-guid-15912217'
    # long_text = soup.find(id = id_key).text

    # json_dict = json.loads(long_text, strict = False)
    # return json_dict['included'][4]['jobInsightsV2ResolutionResults'][1]['insightViewModel']['text']['text'].split(' · ')
    # dive_result = diveHtmlTags(html_body, toplevel_tag='code', dict_labels = ['included', 'list-jobInsightsV2ResolutionResults', 'jobInsightsV2ResolutionResults','list-insightViewModel', 'insightViewModel', 'text', 'text'], multi = True)
    # return dive_result
    dive_result = diveHtmlTags(html_body, toplevel_tag='code', dict_labels = ['included', 'list-jobInsightsV2ResolutionResults', 'jobInsightsV2ResolutionResults', 'list-insightViewModel', 'insightViewModel', 'text', 'text'])
    return dive_result.split(' · ')
    # for result in dive_result:
    #     print('r')
    #     print(result)
    #     print('--')
    #     if str.lower(result).__contains__('employee'):
    #         dive_result = result

    # return dive_result.split(' · ')

def GetCompanySize(html_body, details = []):
    if details is True:
        for detail in details:
            if 'employees' in detail:
                return detail
                
    details = GetCompanyTopCardDetails(html_body)
    for detail in details:
        if 'employees' in detail:
            return detail
    
    return None

def GetCompanyIndustry(html_body, details = []):
    if details is True:
        for detail in details:
            if 'employees' not in detail:
                return detail
                
    details = GetCompanyTopCardDetails(html_body)
    for detail in details:
        if 'employees' not in detail:
            return detail
    
    return None

def GetCompanyDescription(html_body):
    soup = bs(html_body)

    long_text = soup.find('div', {'class': 'jobs-company__box'}).find('div', {'class': 'inline-show-more-text inline-show-more-text--is-collapsed inline-show-more-text--is-collapsed-with-line-clamp'}).text.strip()
    return long_text

In [31]:
soup = bs(html_body_test, 'lxml')
for fa in soup.find_all('id'):
    try:
        print(f"id found at: {fa.find('div')['id']}")
    except KeyError:
        # print(f'id not found at {fa.find("div")["div"]}')
        print(f'id not found at {fa.find("div")}')
    except TypeError:
        print(f'id not found at {fa.find("div")}')
# print(soup.find('div', {"class": "jobs-company__box"}).find('a')['href'])

In [73]:
# # soup = bs(html_body_test, 'lxml')
# # soup.find_all('code', id=True)

# soup = bs(html_body_test)
# # id: bpr-guid-15912217
# id_key = 'bpr-guid-15912217'
# long_text = soup.find(id = id_key)
# # long_text
# long_text.__contains__('Software')

# # [ids['id'] for ids in soup.find_all('code', id=True)]
# # soup = bs(html_body_test, 'lxml')
# # for fa in soup.find_all('img', id=True):
# #     try:
# #         print(f"id found at: {fa.find('div')['id']}")
# #     except KeyError:
# #         # print(f'id not found at {fa.find("div")["div"]}')
# #         print(f'id not found at {fa.find("div")}')
# #     except TypeError:
# #         print(f'id not found at {fa.find("div")}')
# # # print(soup.find('div', {"class": "jobs-company__box"}).find('a')['href'])

def diveDictValues(keyword = None, dict_to_search = {}):
    if keyword is None:
        return None
    if dict_to_search is False:
        return None
    
    for key in dict_to_search.keys():
        try:
            if str(dict_to_search[key]).__contains__(keyword):
                return key
        except KeyError:
            continue

    return None

def diveListValues(keyword = None, list_to_search = []):
    if keyword is None:
        return None
    if list_to_search is False:
        return None
    
    matched_items = []
    for i in range(len(list_to_search)):
        if type(list_to_search[i]) == dict:
            if keyword in list(list_to_search[i].keys()):
                matched_items.append(i)
        elif str(list_to_search[i]).__contains__(keyword):
            matched_items.append(i)

    return matched_items

def AttemptPossibleNest(possible_nest_dict = {}, next_labels = []):
    accepted_nests = []
    for key in possible_nest_dict.keys():
        nest_to_dive = possible_nest_dict[key]
        
        result_check = ProcessHtmlText(nest_to_dive, dict_labels=next_labels)
        if result_check is not None:
            accepted_nests.append(key)
    
    return accepted_nests


def ProcessHtmlText(dict_to_search = None, dict_labels = []):
    if dict_to_search is None:
        return None
    
    curr_dict_level = dict_to_search
    result = None

    for i in range(len(dict_labels)):
        label = dict_labels[i]
        if label.__contains__('list'):
            if type(curr_dict_level) == list:
                dive_result = diveListValues(label.split('-')[1], curr_dict_level)
                if dive_result is None:
                    continue
                elif len(dive_result) <= 0:
                    continue
                elif len(dive_result) > 1:
                    possible_nests_to_check = {}
                    for j in dive_result:
                        possible_nests_to_check[j] = curr_dict_level[j]
                    accepted_nests = AttemptPossibleNest(possible_nests_to_check, dict_labels[i+1:])
                    dive_result = accepted_nests[-1]
                elif len(dive_result) == 1:
                    dive_result = dive_result[0]

                # print(dive_result)
                curr_dict_level = curr_dict_level[dive_result]
        else:
            if type(curr_dict_level) == dict:
                try:
                    curr_dict_level = curr_dict_level[label]
                except KeyError:
                    continue

    return curr_dict_level


def diveHtmlTags(html_body, toplevel_tag = None, dict_labels = [], multi = False):
    soup = bs(html_body)
    if toplevel_tag is None:
        return None
    if dict_labels is False:
        return None
    
    ids = [ids['id'] for ids in soup.find_all(toplevel_tag, id=True)]
    
    result = None
    if multi == False:
        for id_key in ids:
            long_text = soup.find(id = id_key).text
            try:
                json_dict = json.loads(long_text, strict = False)

                if json_dict is None:
                    continue
                
                result_check = ProcessHtmlText(json_dict, dict_labels=dict_labels)
                if result_check is not None:
                    result = result_check
            except ValueError:  # includes simplejson.decoder.JSONDecodeError
                # print(f'Decoding JSON has failed at {id_key}')
                continue
    else:
        result = []
        print(f'ids: {ids}')
        for id_key in ids:
            long_text = soup.find(id = id_key).text
            try:
                json_dict = json.loads(long_text, strict = False)

                if json_dict is None:
                    continue
                
                result_check = ProcessHtmlText(json_dict, dict_labels=dict_labels)
                if result_check is not None:
                    result.append(result_check)
            except ValueError:  # includes simplejson.decoder.JSONDecodeError
                print(f'Decoding JSON has failed at {id_key}')
                continue

    return result

# id: bpr-guid-15912217
# id_key = 'bpr-guid-15912217'
# long_text = soup.find(id = id_key).text

# json_dict = json.loads(long_text, strict = False)
# return json_dict['included'][4]['jobInsightsV2ResolutionResults'][1]['insightViewModel']['text']['text'].split(' · ')

# soup = bs(html_body_test, 'lxml')
# diveHtmlTags(html_body_test, toplevel_tag='code', dict_labels = ['included', 'list-jobInsightsV2ResolutionResults', 'jobInsightsV2ResolutionResults', 'list-insightViewModel', 'insightViewModel'])

soup = bs(html_body_test)
toplevel_tag='code'

ids = [ids['id'] for ids in soup.find_all(toplevel_tag, id=True)]
# [print(id) for id in ids]
# diveHtmlTags(html_body_test, toplevel_tag='code', dict_labels = ['included', 'list-jobInsightsV2ResolutionResults', 'jobInsightsV2ResolutionResults', 'list-insightViewModel', 'insightViewModel'])

id_key = 'bpr-guid-15912217'
long_text = soup.find(id = id_key).text
json_dict = json.loads(long_text, strict = False)
dict_labels = ['included', 'list-jobInsightsV2ResolutionResults', 'jobInsightsV2ResolutionResults', 'list-insightViewModel', 'insightViewModel', 'text', 'text']
result_check = ProcessHtmlText(json_dict, dict_labels=dict_labels)
print(result_check)

11-50 employees · Software Development


In [8]:
body_text = html_body_test

def GetJobDetails(html_body, html_skills_body):
    posting_id = GetJobPostingID(html_body)
    post_status = GetPostStatus(html_body)
    job_title = GetJobTitle(html_body)
    job_description = GetJobDescription(html_body)
    remote_status = GetRemoteWorkStatus(html_body)
    repost_status = GetRepostStatus(html_body)
    job_country = GetJobCountry(html_body)
    company_id = GetCompanyID(html_body)
    job_topcard_details = GetJobTopCardDetails(html_body)
    company_topcard_details = GetCompanyTopCardDetails(html_body)
    # company_topcard_details = []
    job_skills = GetJobSkills(html_skills_body)

    dict_job = {'Job ID': GetJobPostingID(html_body),
                'Status': GetPostStatus(html_body),
                'Job Link': GetPostLink(url_link=None, job_id=GetJobPostingID(html_body)),
                'Job Title': GetJobTitle(html_body),
                'Job Description': GetJobDescription(html_body),
                'Post Language': GetDescriptionLanguage(GetJobDescription(html_body)),
                'Remote': GetRemoteWorkStatus(html_body),
                'Reposted': GetRepostStatus(html_body),
                'Top Card Details': GetJobTopCardDetails(html_body),
                'Experience Level': GetJobExperienceLevel(html_body, details = job_topcard_details),
                'Experience in Years': GetJobYearsExperience(html_body, job_description),
                'Work Setting': GetJobWorkSetting(html_body, details = job_topcard_details),
                'Job Type': GetJobType(html_body, details = job_topcard_details),
                'Job Country': GetJobCountry(html_body),
                'Job Skills': GetJobSkills(html_skills_body),
                'Company ID': GetCompanyID(html_body),
                'Company Name': GetCompanyName(html_body),
                'Company Link': GetCompanyLink(html_body),
                'Company Card Details': GetCompanyTopCardDetails(html_body),
                'Company Size': GetCompanySize(html_body, details = company_topcard_details),
                'Industry': GetCompanyIndustry(html_body, details = company_topcard_details),
                'Company Description': GetCompanyDescription(html_body)
                }
    return dict_job

job_entry = GetJobDetails(body_text, html_skills_matched_html)
job_entry
# df_entry = pd.DataFrame.from_dict(job_entry, orient='index')
# df_entry = df_entry.transpose()
# df_jobs = pd.concat([df_jobs, job_entry], ignore_index=True)
# df_jobs.reset_index()
# df_jobs


# df_entry
# pd.DataFrame.from_dict(job_entry, orient = 'index', columns = job_entry.keys())

# job_id = GetJobPostingID(body_text)
# if df_jobs[df_jobs['Job ID'] == job_id].empty:
#     job_entry = pd.DataFrame(GetJobDetails(body_text, html_skills_matched_html), columns=df_columns)
#     print(len(job_entry.columns))
#     print(len(df_jobs.columns))
#     df_jobs = df_jobs.concat([df_jobs, job_entry], ignore_index=True)
#     # df_jobs.reset_index()
# df_jobs



# dict_entry = GetJobDetails(body_text, html_skills_matched_html)

# df_columns = ['Job ID', 'Status', 'Job Link', 'Job Title', 'Job Description', 'Post Language', 'Remote', 'Reposted', 'Top Card Details', 'Experience Level', 'Experience in Years', 'Work Setting', 'Job Type', 'Job Country', 'Job Skills', 'Company ID', 'Company Name', 'Company Link', 'Company Card Details', 'Company Size', 'Industry', 'Company Description']
# dict_entry.keys()

{'Job ID': 3768592532,
 'Status': 'LISTED',
 'Job Link': 'https://www.linkedin.com/jobs/view/3768592532/',
 'Job Title': 'Sustainability Data Analyst (f/m/x)',
 'Job Description': "Join Our Mission to Fight Climate Change as our Sustainability Data Analyst\n\nWant to be part of one of Europe's fastest growing climate tech startups? We are looking for a passionate Sustainability Data Analyst to help us transform every company into a sustainable business, with critical responsibility and influence on our product and overall venture development.\n\nAs a Sustainability Data Analyst your role will have a key responsibility of applying data-driven approaches to managing our carbon footprint calculations and ensuring our customers understand how we got there.\n\nIf you are excited about building an economy where every company creates an impact for a sustainable world and empowering companies to decarbonize and boost their business performance, we’d love to hear from you!\n\nIf you want to fin

In [13]:
df_jobs = pd.concat([df_jobs, pd.DataFrame([job_entry])], ignore_index=True)
df_jobs.reset_index()
df_jobs

,Job ID,Status,Job Link,Job Title,Job Description,Post Language,Remote,Reposted,Top Card Details,Experience Level,...,Job Type,Job Country,Job Skills,Company ID,Company Name,Company Link,Company Card Details,Company Size,Industry,Company Description
0,3768592532,LISTED,https://www.linkedin.com/jobs/view/3768592532/,Sustainability Data Analyst (f/m/x),Join Our Mission to Fight Climate Change as ou...,en,True,None,"[Remote, Full-time, Entry level]",Entry level,...,Full-time,Germany,"[Data Analysis, Data Analytics, Unstructured D...",urn:li:fs_normalized_company:43338549,Cozero,https://www.linkedin.com/company/cozero-io/life,"[11-50 employees, Software Development]",11-50 employees,Software Development,Take control of your corporate emission data w...


In [9]:
x ="""
123 10+ years some text
some text 99 10 years ssss
text 1 year and more text
some text 10-15 years some text
"""

# x ="""
# 10+ years
# """

years_text = re.findall(r"([\d+-]+)\s+(years?)", x, re.IGNORECASE)
splitted_num = []
# years_text
for clipped in years_text:
    char_to_remove = ['+', '-', ' ']
    text_to_clean = clipped[0]

    for ch in char_to_remove:
        for el in text_to_clean.split(ch):
            if el.isalnum() is True:
                splitted_num = splitted_num + [int(el)]
        
        # text_to_clean = text_to_clean.replace(ch, '')

print(np.min(list(set(splitted_num))))
# print(GetJobYearsExperience(x, None))

1


In [32]:
soup = bs(html_body_test, 'lxml')
# print(soup.find('div', {'class': 'jobs-company__box'}).find('div', {'class': 'inline-show-more-text inline-show-more-text--is-collapsed inline-show-more-text--is-collapsed-with-line-clamp'}).text.strip())
# print(soup.find('div', {'class': 'jobs-company__box'}).find('p', {'class': 'jobs-company__company-description text-body-small-open'}).text)
# print(soup.find('p', {'class': 'jobs-company__company-description text-body-small-open'}))
# print(soup.find('div', {"class": "jobs-company__box"}))
# print(soup.find('div', {"class": "jobs-company__box"}).find('a')['href'])
# print(soup.find('div', {"class": "jobs-company__box"}).find('a', {'id':'ember79'}).text.strip())
# print(soup.find('div', {"class": "jobs-company__box"}).find('div', {'class': 'inline-show-more-text inline-show-more-text--is-collapsed inline-show-more-text--is-collapsed-with-line-clamp'}).text)

Take control of your corporate emission data with Cozero's digital Climate Action Platform (CAP). Cozero tracks, optimizes, and engages on your carbon performance. Forget endless excel files & project sounding boards. Cozero digitally unites all carbon management data in one end-to-end platform, so that you can easily keep track - and act. Customize your climate action platform for your industry and company. Get smart on your carbon emissions and take the right decisions. Cozero was founded in 2020 with the purpose to enable the corporate sector in a transformational shift to a lower emission economy. We strongly believe that the private sector has the potential to make this transformation possible and to fight climate change effectively. Whoever you are - be it a business leader, a climate action enthusiast, a potential low carbon service or technology partner or a motivated individual who wants to join our mission - please reach out and get on board: info@cozero.io
      
          


In [7]:
import textwrap

soup = bs(html_body_test, 'lxml')

# print(textwrap.fill(soup.find(id = "bpr-guid-15912216").text[:16160], 100))
# print(soup.find(id = "bpr-guid-15912216").text[16151])
new_dict = json.loads(soup.find(id = "bpr-guid-15912216").text, strict = False)

In [54]:
soup = bs(html_body_test, 'lxml')

print('Software Development' in str(json.loads(soup.find(id = "bpr-guid-15912217").text, strict = False)['included']))
# for i in range(len(json.loads(soup.find(id = "bpr-guid-15912217").text, strict = False)['included'][4]['jobInsightsV2ResolutionResults'])):
#     el = json.loads(soup.find(id = "bpr-guid-15912217").text, strict = False)['included'][4]['jobInsightsV2ResolutionResults'][i]
#     if 'Software' in str(el):
#         print(i)
#         print(el)
# json.loads(soup.find(id = "bpr-guid-15912217").text, strict = False)['included'][4].keys()
# for key in json.loads(soup.find(id = "bpr-guid-15912217").text, strict = False)['included'][4]['jobInsightsV2ResolutionResults'][1]:
#     if 'Software Development' in str(json.loads(soup.find(id = "bpr-guid-15912217").text, strict = False)['included'][4]['jobInsightsV2ResolutionResults'][1][key]):
#         print(key)
#         print(json.loads(soup.find(id = "bpr-guid-15912217").text, strict = False)['included'][4]['jobInsightsV2ResolutionResults'][1][key])

# json.loads(soup.find(id = "bpr-guid-15912217").text, strict = False)['included'][4]['jobInsightsV2ResolutionResults']
json.loads(soup.find(id = "bpr-guid-15912217").text, strict = False)['included'][4]['jobInsightsV2ResolutionResults'][1]['insightViewModel']['text']['text']

True


'11-50 employees · Software Development'

In [66]:
soup = bs(html_body_test, 'lxml')

hint = soup.find(id = 'bpr-guid-15912217')
# print('Entry' in hint.text)
# json.loads(hint.text, strict = False)
# 'Entry' in str(json.loads(hint.text, strict = False)['included'])
print(json.loads(hint.text, strict = False)['included'][4].keys())
print(json.loads(hint.text, strict = False)['included'][4]['primaryDescription']['text'])
print('Entry' in str(json.loads(hint.text, strict = False)['included'][4]['jobInsightsV2ResolutionResults'][4]))
print(json.loads(hint.text, strict = False)['included'][4]['jobInsightsV2ResolutionResults'][1].keys())
texts_list = json.loads(hint.text, strict = False)['included'][4]['jobInsightsV2ResolutionResults'][0]['jobInsightViewModel']['description']
text = [t['text']['text'] for t in texts_list]

print(text)
# json.loads(hint.text, strict = False)['included'][4].keys()
# <img src="data:image/gif;base64,R0lGODlhAQABAIAAAAAAAP///yH5BAEAAAAALAAAAAABAAEAAAIBRAA7" style="display: none" class="datalet-bpr-guid-15912216"><code style="display: none" id="bpr-guid-15912217">
# <img src="data:image/gif;base64,R0lGODlhAQABAIAAAAAAAP///yH5BAEAAAAALAAAAAABAAEAAAIBRAA7" style="display: none" class="datalet-bpr-guid-15912216"><code style="display: none" id="bpr-guid-15912217">

# new_dict = json.loads(soup.find(id = 'datalet-bpr-guid-15912216').text, strict = False)
# new_dict

dict_keys(['preDashNormalizedJobPostingUrn', 'jobPostingTitle', '*coachJobsKickoffPrompts', 'navigationBarSubtitle', 'primaryDescription', 'jobInsightsV2ResolutionResults', '$recipeTypes', 'primaryActionV2', 'secondaryActionsV2ResolutionResults', '$type', 'secondaryDescription', 'entityUrn', 'logo', 'tip', '*jobPosting', 'tertiaryDescription'])
Cozero · Germany   22 hours ago · 71 applicants
False
dict_keys(['premiumUpsellCard', 'jobInsightViewModel', 'insightViewModel', 'premiumUpsell', 'skillMatchInsight', 'verifiedHiringInsight', 'responsivenessInsightViewModel', 'trustInsightViewModel'])
['Remote', 'Full-time', 'Entry level']


In [100]:
# soup.find(id = 'bpr-guid-15912217').text[126300:126380]
len(soup.find(id = 'bpr-guid-15912217').text)

145115

In [125]:
# hint['data']['data']['jobInsightsV2ResolutionResults']['jobInsightViewModel']
# hint.keys()
# print('Entry' in str(hint['included']))
hint['included']

[{'entityUrn': 'urn:li:fsd_jobSkillMatchInsight:3768592532',
  '$recipeTypes': ['com.linkedin.65d9b685ce8d8355ba025fb1ae135598'],
  'insightSummary': {'image': {'attributes': [{'scalingType': None,
      'detailData': {'profilePictureWithoutFrame': None,
       'profilePictureWithRingStatus': None,
       'companyLogo': None,
       'icon': None,
       'systemImage': 'SYS_ICN_CHECKLIST_MEDIUM',
       'nonEntityGroupLogo': None,
       'vectorImage': None,
       'nonEntityProfessionalEventLogo': None,
       'profilePicture': None,
       'imageUrl': None,
       'professionalEventLogo': None,
       'nonEntityCompanyLogo': None,
       'nonEntitySchoolLogo': None,
       'groupLogo': None,
       'schoolLogo': None,
       'ghostImage': None,
       'nonEntityProfilePicture': None},
      'tintColor': None,
      '$recipeTypes': ['com.linkedin.296fd4ebc3f7a46ccca548d534f656e4'],
      'tapTargets': [],
      'displayAspectRatio': None,
      '$type': 'com.linkedin.voyager.dash.commo

In [95]:
soup = bs(html_body_test, 'lxml')

hint = json.loads(soup.find(id = 'bpr-guid-15912217').text, strict = False)
hint

{'data': {'data': {'$recipeTypes': ['com.linkedin.8fbc3627c8305a33e3e6840ec7688312'],
   'jobsDashJobPostingDetailSectionsByCardSectionTypes': {'$recipeTypes': ['com.linkedin.f1d8ff8a5c9d0a2c73d8c4418e3a4526'],
    'elements': [{'$recipeTypes': ['com.linkedin.180bad7e2024ccff1ee0157be65c0dd1'],
      'jobPostingDetailSection': [{'similarJobsCard': None,
        'interviewPrepCard': None,
        'salaryCard': None,
        'companyInsightsCard': None,
        '*topCard': 'urn:li:fsd_jobPostingCard:(3768592532,JOB_DETAILS)',
        'bannerCard': None,
        'jobSeekerActionCard': None,
        'learningRecommendationCard': None,
        'hiringTeamCard': None,
        'similarJobsAtCompanyCard': None,
        'jobSegmentAttributesCard': None,
        'howYouMatchCard': None,
        'jobApplicantInsightsUrn': None,
        'jobActivityCard': None,
        'postApplyPromo': None,
        'jobDescription': None,
        'benefitsCard': None,
        'jobAlertCard': None,
        'marke

In [78]:
# new_dict.keys()
new_dict['data']

{'dashEntityUrn': 'urn:li:fsd_jobPosting:3768592532',
 'companyDetails': {'company': 'urn:li:fs_normalized_company:43338549',
  '*companyResolutionResult': 'urn:li:fs_normalized_company:43338549',
  '$recipeTypes': ['com.linkedin.voyager.deco.jobs.web.shared.WebCompactJobPostingCompany'],
  '$type': 'com.linkedin.voyager.jobs.JobPostingCompany'},
 'jobState': 'LISTED',
 'description': {'attributes': [{'start': 75,
    'length': 1,
    'type': {'$type': 'com.linkedin.pemberly.text.LineBreak'},
    'attributeKindUnion': {'lineBreak': {'$type': 'com.linkedin.pemberly.text.LineBreak'}},
    '$type': 'com.linkedin.pemberly.text.Attribute'},
   {'start': 76,
    'length': 1,
    'type': {'$type': 'com.linkedin.pemberly.text.LineBreak'},
    'attributeKindUnion': {'lineBreak': {'$type': 'com.linkedin.pemberly.text.LineBreak'}},
    '$type': 'com.linkedin.pemberly.text.Attribute'},
   {'start': 0,
    'length': 77,
    'type': {'$type': 'com.linkedin.pemberly.text.Bold'},
    'attributeKindUni

In [22]:
import textwrap

# text = soup.find(id = "bpr-guid-15912216").text[1: 20883]
text = soup.find(id = "bpr-guid-15912216").text[1: 16150]
# sign_text = text.find('\n')

# print(textwrap.fill(text, 100))


  {"data":{"dashEntityUrn":"urn:li:fsd_jobPosting:3768592532","companyDetails":{"company":"urn:li:fs
_normalized_company:43338549","*companyResolutionResult":"urn:li:fs_normalized_company:43338549","$r
ecipeTypes":["com.linkedin.voyager.deco.jobs.web.shared.WebCompactJobPostingCompany"],"$type":"com.l
inkedin.voyager.jobs.JobPostingCompany"},"jobState":"LISTED","description":{"attributes":[{"start":7
5,"length":1,"type":{"$type":"com.linkedin.pemberly.text.LineBreak"},"attributeKindUnion":{"lineBrea
k":{"$type":"com.linkedin.pemberly.text.LineBreak"}},"$type":"com.linkedin.pemberly.text.Attribute"}
,{"start":76,"length":1,"type":{"$type":"com.linkedin.pemberly.text.LineBreak"},"attributeKindUnion"
:{"lineBreak":{"$type":"com.linkedin.pemberly.text.LineBreak"}},"$type":"com.linkedin.pemberly.text.
Attribute"},{"start":0,"length":77,"type":{"$type":"com.linkedin.pemberly.text.Bold"},"attributeKind
Union":{"bold":{"$type":"com.linkedin.pemberly.text.Bold"}},"$type":"com.linkedin.pemberly.

In [15]:
keyword = 'jobPostingId'
text = soup.find(id = "bpr-guid-15912216").text

if keyword in text:
    print(text.index(keyword))
    print(text[:text.index(keyword)])

20883

  {"data":{"dashEntityUrn":"urn:li:fsd_jobPosting:3768592532","companyDetails":{"company":"urn:li:fs_normalized_company:43338549","*companyResolutionResult":"urn:li:fs_normalized_company:43338549","$recipeTypes":["com.linkedin.voyager.deco.jobs.web.shared.WebCompactJobPostingCompany"],"$type":"com.linkedin.voyager.jobs.JobPostingCompany"},"jobState":"LISTED","description":{"attributes":[{"start":75,"length":1,"type":{"$type":"com.linkedin.pemberly.text.LineBreak"},"attributeKindUnion":{"lineBreak":{"$type":"com.linkedin.pemberly.text.LineBreak"}},"$type":"com.linkedin.pemberly.text.Attribute"},{"start":76,"length":1,"type":{"$type":"com.linkedin.pemberly.text.LineBreak"},"attributeKindUnion":{"lineBreak":{"$type":"com.linkedin.pemberly.text.LineBreak"}},"$type":"com.linkedin.pemberly.text.Attribute"},{"start":0,"length":77,"type":{"$type":"com.linkedin.pemberly.text.Bold"},"attributeKindUnion":{"bold":{"$type":"com.linkedin.pemberly.text.Bold"}},"$type":"com.linkedin.pemberly.te

In [5]:
keyword = 'jobPostingId'

def find_dict_key(dict_to_check, keyword, initial_dict_key):
    for key in dict_to_check.keys():
        if key == keyword:
            return key + '\n'
        else:
            if type(dict_to_check[key]) == dict:
                find_dict_key(dict_to_check[key], keyword)
    return initial_dict_key

key_depth = find_dict_key(new_dict, keyword)
print(key_depth)

TypeError: find_dict_key() missing 1 required positional argument: 'initial_dict_key'

In [25]:
test_dict = {'a': 'lah', 'jobPostingId': '123'}

In [26]:
print(test_dict.get(keyword))

123


In [ ]:
def find(element, JSON, path, all_paths):    
  if element in JSON:
    path = path + element + ' = ' + JSON[element].encode('utf-8')
    print path
    all_paths.append(path)
  for key in JSON:
    if isinstance(JSON[key], dict):
      find(element, JSON[key],path + key + '.',all_paths)

all_paths = []
find(element_to_search,j,'',all_paths)

In [7]:
html_skills_matched_html = """
<body dir="ltr" class="render-mode-BIGPIPE nav-v2 ember-application icons-loaded boot-complete artdeco-modal-is-open" data-t-link-to-event-attached="true">
  

  
    <!-- HUED-11420 -->
    <div id="artdeco-toasts__wormhole" role="region">    <section id="artdeco-toasts" class="artdeco-toasts" aria-label="Toast message">
      <header class="artdeco-toasts__header">
        <h2 class="artdeco-toasts__title">
          0 notifications total
        </h2>
      </header>

<!---->
      <div class="artdeco-toasts_toasts">
<!---->      </div>
    </section>
</div>
    <!-- EMBER_CLI_FASTBOOT_BODY -->
<div id="app-boot-bg-loader" class="app-boot-bg-skeleton" aria-hidden="true">
  <div class="top-bar"></div>
  <div class="content">
    <div class="initial-load-animation fade-load">
      <div class="linkedin-image display-flex justify-center">
        <svg xmlns="http://www.w3.org/2000/svg" class="loader__linkedin-logo" width="190" height="48" viewBox="0 0 190 48">
          <g>
            <g>
              <path d="M58,27.22V41H51V28.88c0-3.7-2.07-5.24-4-5.24a5,5,0,0,0-5.14,4.85,4.34,4.34,0,0,0,0,.51V41H35V18h6.6v3.25h.09c.69-1.41,3.64-3.75,7.66-3.75S58,20.22,58,27.22ZM24,41h7V18H24ZM27.5,6.45a4.05,4.05,0,1,0,4.1,4.05,4,4,0,0,0-4-4ZM190,3.5v41a3.5,3.5,0,0,1-3.5,3.5h-41a3.5,3.5,0,0,1-3.5-3.5V3.5A3.5,3.5,0,0,1,145.5,0h41A3.5,3.5,0,0,1,190,3.5ZM156,18h-7V41h7Zm.6-7.5a4.1,4.1,0,1,0-4.15,4.05h.05a4,4,0,0,0,4.1-3.9ZM183,27.22c0-7-4.63-9.72-8.65-9.72s-7,2.34-7.66,3.75h-.09V18H160V41h7V29c0-3.69,2.51-5.33,4.95-5.33,2,0,4.05,1.54,4.05,5.24V41h7ZM8,7H0V41H21V34H8ZM108,29.77v2H91a1.33,1.33,0,0,0,.11.43c.58,1.93,2.67,3.56,5.75,3.56A6.3,6.3,0,0,0,102,33.52l5.1,3.18a12.72,12.72,0,0,1-10.45,4.8C89.94,41.5,84,37.42,84,29.59S90,17.5,96.44,17.5,108,21.81,108,29.77ZM101,27c0-2.4-1.56-4.38-4.75-4.38-3,0-5.09,2-5.25,4.38ZM85.26,18H76.68l-7.54,9.37H69V7H62V41h7V30h.14l7.72,11h8.77L76.2,28.43ZM128,7h7V41h-6.6V38h-.09c-.88,1.52-3.24,3.49-7.4,3.49-5,0-10.91-3.63-10.91-12,0-7.53,5.1-11.95,10.82-11.95a9.55,9.55,0,0,1,7.09,3H128Zm.3,22.49a5.74,5.74,0,0,0-5.59-5.89h-.15A5.54,5.54,0,0,0,116.89,29c0,.15,0,.29,0,.44a5.61,5.61,0,0,0,5.26,5.94h.4A5.83,5.83,0,0,0,128.3,29.49Z" fill="currentColor"></path>
            </g>
          </g>
        </svg>
      </div>
      <div class="loading-bar">
        <div class="blue-bar"></div>
      </div>
    </div>
  </div>
</div>

    <script src="https://static.licdn.com/aero-v1/sc/h/9vghoawmkb5fvzav7oabel24l" data-fastboot-src="/assets/vendor-static.js" crossorigin="" aria-hidden="true"></script>
    <script src="https://static.licdn.com/aero-v1/sc/h/11eolzz5b283nc4tnagm3alq7" data-fastboot-src="/assets/vendor.js" crossorigin="" aria-hidden="true"></script>
    <!-- graphql-script-injection -->
    <script src="https://static.licdn.com/aero-v1/sc/h/a8i9w57wqoyi8fjxs9ulzk14e" data-fastboot-src="/assets/pdsc/register-metadata.js" crossorigin="" aria-hidden="true"></script>
    <script src="https://static.licdn.com/aero-v1/sc/h/22fggrlii7wsltz6fl4hii414" data-fastboot-src="/assets/chunk.328.8d0e86876347a9c2aa5b.js" crossorigin="" aria-hidden="true"></script>
<script src="https://static.licdn.com/aero-v1/sc/h/d0ctz04bmij7oz7fegwxrtnad" data-fastboot-src="/assets/chunk.143.b8974102cf3edaf96109.js" crossorigin="" aria-hidden="true"></script>
    <script data-embroider-ignore="" src="https://static.licdn.com/aero-v1/sc/h/cc8v3w7w7p9hjm24481ikut6n" data-fastboot-src="/assets/i18n/support_en_US.js" crossorigin="" aria-hidden="true"></script>
    <script src="https://static.licdn.com/aero-v1/sc/h/1w15yji05xoobybogjdkfz0px" aria-hidden="true"></script><script src="https://static.licdn.com/aero-v1/sc/h/3td7ryy98rs4l2sup9qw4jytz" aria-hidden="true"></script>
    <script src="https://static.licdn.com/aero-v1/sc/h/e4srt2ytpjcnxrhvg4000s7g3" data-fastboot-src="/assets/voyager-web.js" crossorigin="" aria-hidden="true"></script>
    <div id="artdeco-modal-outlet">    <div data-test-modal-container="" aria-hidden="false" id="ember90" class="artdeco-modal-overlay artdeco-modal-overlay--layer-default artdeco-modal-overlay--is-top-layer  ember-view">
      <div data-test-modal="" role="dialog" tabindex="-1" class="artdeco-modal artdeco-modal--layer-default job-details-skill-match-modal" size="medium" aria-labelledby="job-details-skill-match-modal__header">
        <span class="a11y-text">Dialog content start.</span>
            <button aria-label="Dismiss" id="ember91" class="artdeco-button artdeco-button--circle artdeco-button--muted artdeco-button--2 artdeco-button--tertiary ember-view artdeco-modal__dismiss" data-test-modal-close-btn="">        <svg role="none" aria-hidden="true" class="artdeco-button__icon " xmlns="http://www.w3.org/2000/svg" width="24" height="24" viewBox="0 0 24 24" data-supported-dps="24x24" data-test-icon="close-medium">
<!---->    

    <use href="#close-medium" width="24" height="24"></use>
</svg>


<span class="artdeco-button__text">
    
</span></button>
        
      <div id="ember92" class="artdeco-modal__header ember-view">
        <h1 id="job-details-skill-match-modal__header">
            <!---->Skill details<!---->
        </h1>
      
<!----></div>
      <div id="ember93" class="artdeco-modal__content ember-view job-details-skill-match-modal__container">
<!---->
        <div class="job-details-skill-match-modal__content">

                  
    <div>
        <h2 class="t-bold t-16">
          <!---->Skills associated with the job<!---->
        </h2>
        <p class="text-body-small">
          <span aria-hidden="true"><!---->3 skills found on your profile match the skills associated with the job.<span class="white-space-pre"> </span><strong><a class="app-aware-link " target="_blank" href="https://www.linkedin.com/help/linkedin/answer/a793433?lang=en" data-test-app-aware-link=""><!---->Learn more<!----></a></strong></span><span class="visually-hidden"><!---->Learn more about how your skills are matched<!----></span>
        </p>
<!---->      <ul class="job-details-skill-match-status-list">
          <li class="job-details-skill-match-status-list__matched-skill text-body-small">
            <div class="display-flex align-items-center">
                <div class="display-flex">
                  <svg role="none" aria-hidden="true" class="job-details-skill-match-status-list__check" xmlns="http://www.w3.org/2000/svg" width="24" height="24" viewBox="0 0 24 24" data-supported-dps="24x24" data-test-icon="signal-success-medium">
<!---->    

    <use href="#signal-success-medium" width="24" height="24"></use>
</svg>

                </div>

              <div aria-label="Your profile shows you have Data Analysis as a skill. Data Analysis matches the job description.">
                Data Analysis
              </div>
            </div>
<!---->          </li>
          <li class="job-details-skill-match-status-list__matched-skill text-body-small">
            <div class="display-flex align-items-center">
                <div class="display-flex">
                  <svg role="none" aria-hidden="true" class="job-details-skill-match-status-list__check" xmlns="http://www.w3.org/2000/svg" width="24" height="24" viewBox="0 0 24 24" data-supported-dps="24x24" data-test-icon="signal-success-medium">
<!---->    

    <use href="#signal-success-medium" width="24" height="24"></use>
</svg>

                </div>

              <div aria-label="Your profile shows you have Data Analytics as a skill. Data Analytics matches the job description.">
                Data Analytics
              </div>
            </div>
<!---->          </li>
          <li class="job-details-skill-match-status-list__matched-skill text-body-small">
            <div class="display-flex align-items-center">
                <div class="display-flex">
                  <svg role="none" aria-hidden="true" class="job-details-skill-match-status-list__check" xmlns="http://www.w3.org/2000/svg" width="24" height="24" viewBox="0 0 24 24" data-supported-dps="24x24" data-test-icon="signal-success-medium">
<!---->    

    <use href="#signal-success-medium" width="24" height="24"></use>
</svg>

                </div>

              <div aria-label="Your profile shows you have Unstructured Data as a skill. Unstructured Data matches the job description.">
                Unstructured Data
              </div>
            </div>
<!---->          </li>
          <li class="job-details-skill-match-status-list__unmatched-skill text-body-small">
            <div class="display-flex align-items-center">

                <div class="display-flex">
                  <svg role="none" aria-hidden="true" class="job-details-skill-match-status-list__check-muted" xmlns="http://www.w3.org/2000/svg" width="24" height="24" viewBox="0 0 24 24" data-supported-dps="24x24" data-test-icon="signal-success-medium">
<!---->    

    <use href="#signal-success-medium" width="24" height="24"></use>
</svg>

                </div>
              <div aria-label="Your profile does not show Carbon as a skill. If you have this skill, you can add it to your profile.">
                Carbon
              </div>
            </div>
              <div>
                <button aria-label="Add Carbon as a skill to your profile" id="ember96" class="artdeco-button artdeco-button--2 artdeco-button--tertiary ember-view job-details-skill-match-status-list__add-skill">        <svg role="none" aria-hidden="true" class="artdeco-button__icon " xmlns="http://www.w3.org/2000/svg" width="16" height="16" viewBox="0 0 16 16" data-supported-dps="16x16" data-test-icon="add-small">
<!---->    

    <use href="#add-small" width="16" height="16"></use>
</svg>


<span class="artdeco-button__text">
    Add
</span></button>
              </div>
          </li>
          <li class="job-details-skill-match-status-list__unmatched-skill text-body-small">
            <div class="display-flex align-items-center">

                <div class="display-flex">
                  <svg role="none" aria-hidden="true" class="job-details-skill-match-status-list__check-muted" xmlns="http://www.w3.org/2000/svg" width="24" height="24" viewBox="0 0 24 24" data-supported-dps="24x24" data-test-icon="signal-success-medium">
<!---->    

    <use href="#signal-success-medium" width="24" height="24"></use>
</svg>

                </div>
              <div aria-label="Your profile does not show Carbon Accounting as a skill. If you have this skill, you can add it to your profile.">
                Carbon Accounting
              </div>
            </div>
              <div>
                <button aria-label="Add Carbon Accounting as a skill to your profile" id="ember97" class="artdeco-button artdeco-button--2 artdeco-button--tertiary ember-view job-details-skill-match-status-list__add-skill">        <svg role="none" aria-hidden="true" class="artdeco-button__icon " xmlns="http://www.w3.org/2000/svg" width="16" height="16" viewBox="0 0 16 16" data-supported-dps="16x16" data-test-icon="add-small">
<!---->    

    <use href="#add-small" width="16" height="16"></use>
</svg>


<span class="artdeco-button__text">
    Add
</span></button>
              </div>
          </li>
          <li class="job-details-skill-match-status-list__unmatched-skill text-body-small">
            <div class="display-flex align-items-center">

                <div class="display-flex">
                  <svg role="none" aria-hidden="true" class="job-details-skill-match-status-list__check-muted" xmlns="http://www.w3.org/2000/svg" width="24" height="24" viewBox="0 0 24 24" data-supported-dps="24x24" data-test-icon="signal-success-medium">
<!---->    

    <use href="#signal-success-medium" width="24" height="24"></use>
</svg>

                </div>
              <div aria-label="Your profile does not show Data Collection as a skill. If you have this skill, you can add it to your profile.">
                Data Collection
              </div>
            </div>
              <div>
                <button aria-label="Add Data Collection as a skill to your profile" id="ember98" class="artdeco-button artdeco-button--2 artdeco-button--tertiary ember-view job-details-skill-match-status-list__add-skill">        <svg role="none" aria-hidden="true" class="artdeco-button__icon " xmlns="http://www.w3.org/2000/svg" width="16" height="16" viewBox="0 0 16 16" data-supported-dps="16x16" data-test-icon="add-small">
<!---->    

    <use href="#add-small" width="16" height="16"></use>
</svg>


<span class="artdeco-button__text">
    Add
</span></button>
              </div>
          </li>
          <li class="job-details-skill-match-status-list__unmatched-skill text-body-small">
            <div class="display-flex align-items-center">

                <div class="display-flex">
                  <svg role="none" aria-hidden="true" class="job-details-skill-match-status-list__check-muted" xmlns="http://www.w3.org/2000/svg" width="24" height="24" viewBox="0 0 24 24" data-supported-dps="24x24" data-test-icon="signal-success-medium">
<!---->    

    <use href="#signal-success-medium" width="24" height="24"></use>
</svg>

                </div>
              <div aria-label="Your profile does not show Databases as a skill. If you have this skill, you can add it to your profile.">
                Databases
              </div>
            </div>
              <div>
                <button aria-label="Add Databases as a skill to your profile" id="ember99" class="artdeco-button artdeco-button--2 artdeco-button--tertiary ember-view job-details-skill-match-status-list__add-skill">        <svg role="none" aria-hidden="true" class="artdeco-button__icon " xmlns="http://www.w3.org/2000/svg" width="16" height="16" viewBox="0 0 16 16" data-supported-dps="16x16" data-test-icon="add-small">
<!---->    

    <use href="#add-small" width="16" height="16"></use>
</svg>


<span class="artdeco-button__text">
    Add
</span></button>
              </div>
          </li>
          <li class="job-details-skill-match-status-list__unmatched-skill text-body-small">
            <div class="display-flex align-items-center">

                <div class="display-flex">
                  <svg role="none" aria-hidden="true" class="job-details-skill-match-status-list__check-muted" xmlns="http://www.w3.org/2000/svg" width="24" height="24" viewBox="0 0 24 24" data-supported-dps="24x24" data-test-icon="signal-success-medium">
<!---->    

    <use href="#signal-success-medium" width="24" height="24"></use>
</svg>

                </div>
              <div aria-label="Your profile does not show Datasets as a skill. If you have this skill, you can add it to your profile.">
                Datasets
              </div>
            </div>
              <div>
                <button aria-label="Add Datasets as a skill to your profile" id="ember100" class="artdeco-button artdeco-button--2 artdeco-button--tertiary ember-view job-details-skill-match-status-list__add-skill">        <svg role="none" aria-hidden="true" class="artdeco-button__icon " xmlns="http://www.w3.org/2000/svg" width="16" height="16" viewBox="0 0 16 16" data-supported-dps="16x16" data-test-icon="add-small">
<!---->    

    <use href="#add-small" width="16" height="16"></use>
</svg>


<span class="artdeco-button__text">
    Add
</span></button>
              </div>
          </li>
          <li class="job-details-skill-match-status-list__unmatched-skill text-body-small">
            <div class="display-flex align-items-center">

                <div class="display-flex">
                  <svg role="none" aria-hidden="true" class="job-details-skill-match-status-list__check-muted" xmlns="http://www.w3.org/2000/svg" width="24" height="24" viewBox="0 0 24 24" data-supported-dps="24x24" data-test-icon="signal-success-medium">
<!---->    

    <use href="#signal-success-medium" width="24" height="24"></use>
</svg>

                </div>
              <div aria-label="Your profile does not show Life Cycle Assessment as a skill. If you have this skill, you can add it to your profile.">
                Life Cycle Assessment
              </div>
            </div>
              <div>
                <button aria-label="Add Life Cycle Assessment as a skill to your profile" id="ember101" class="artdeco-button artdeco-button--2 artdeco-button--tertiary ember-view job-details-skill-match-status-list__add-skill">        <svg role="none" aria-hidden="true" class="artdeco-button__icon " xmlns="http://www.w3.org/2000/svg" width="16" height="16" viewBox="0 0 16 16" data-supported-dps="16x16" data-test-icon="add-small">
<!---->    

    <use href="#add-small" width="16" height="16"></use>
</svg>


<span class="artdeco-button__text">
    Add
</span></button>
              </div>
          </li>
          <li class="job-details-skill-match-status-list__unmatched-skill text-body-small">
            <div class="display-flex align-items-center">

                <div class="display-flex">
                  <svg role="none" aria-hidden="true" class="job-details-skill-match-status-list__check-muted" xmlns="http://www.w3.org/2000/svg" width="24" height="24" viewBox="0 0 24 24" data-supported-dps="24x24" data-test-icon="signal-success-medium">
<!---->    

    <use href="#signal-success-medium" width="24" height="24"></use>
</svg>

                </div>
              <div aria-label="Your profile does not show Offsites as a skill. If you have this skill, you can add it to your profile.">
                Offsites
              </div>
            </div>
              <div>
                <button aria-label="Add Offsites as a skill to your profile" id="ember102" class="artdeco-button artdeco-button--2 artdeco-button--tertiary ember-view job-details-skill-match-status-list__add-skill">        <svg role="none" aria-hidden="true" class="artdeco-button__icon " xmlns="http://www.w3.org/2000/svg" width="16" height="16" viewBox="0 0 16 16" data-supported-dps="16x16" data-test-icon="add-small">
<!---->    

    <use href="#add-small" width="16" height="16"></use>
</svg>


<span class="artdeco-button__text">
    Add
</span></button>
              </div>
          </li>
      </ul>

    </div>
  

                  
    <div>
        <div class="job-details-skill-match-modal-member-feedback-actions">
          <span class="job-details-skill-match-modal-member-feedback-actions__feedback-cta-description">Are these skills relevant to the job?</span>
          <button aria-label="Yes, the skills are relevant" id="ember103" class="artdeco-button artdeco-button--circle artdeco-button--1 artdeco-button--secondary ember-view job-details-skill-match-modal-member-feedback-actions__feedback-button">        <svg role="none" aria-hidden="true" class="artdeco-button__icon " xmlns="http://www.w3.org/2000/svg" width="16" height="16" viewBox="0 0 16 16" data-supported-dps="16x16" data-test-icon="thumbs-up-outline-small">
<!---->    

    <use href="#thumbs-up-outline-small" width="16" height="16"></use>
</svg>


<span class="artdeco-button__text">
    
</span></button>
          <button aria-label="No, the skills are not relevant" id="ember104" class="artdeco-button artdeco-button--circle artdeco-button--1 artdeco-button--secondary ember-view job-details-skill-match-modal-member-feedback-actions__feedback-button">        <svg role="none" aria-hidden="true" class="artdeco-button__icon " xmlns="http://www.w3.org/2000/svg" width="16" height="16" viewBox="0 0 16 16" data-supported-dps="16x16" data-test-icon="thumbs-down-outline-small">
<!---->    

    <use href="#thumbs-down-outline-small" width="16" height="16"></use>
</svg>


<span class="artdeco-button__text">
    
</span></button>
        </div>
    </div>
  

<!---->
                  <div class="display-flex mt4">
                    <div class="mr1 mt1 t-black--light">
                      
    <div class="ivm-image-view-model   ">
        
    <div class="ivm-view-attr__img-wrapper display-flex">
        <li-icon aria-hidden="true" type="signal-notice" class="ivm-view-attr__icon--icon  " size="small"><svg xmlns="http://www.w3.org/2000/svg" viewBox="0 0 16 16" data-supported-dps="16x16" fill="currentColor" class="mercado-match" width="16" height="16" focusable="false">
      <path d="M12 2H4a2 2 0 00-2 2v8a2 2 0 002 2h8a2 2 0 002-2V4a2 2 0 00-2-2zm-3 8v2H7.5A1.5 1.5 0 016 10.5a1.56 1.56 0 01.1-.5l1.08-3h2.13l-1.09 3zm0-3.75A1.25 1.25 0 1110.25 5 1.25 1.25 0 019 6.25z"></path>
    </svg></li-icon>
    </div>
  
          </div>
  
                    </div>
                    <div class="t-14 t-black--light mr8">
                      <!---->You can manage all your skills on your profile.<span class="white-space-pre"> </span><strong><a class="app-aware-link " target="_blank" href="https://www.linkedin.com/in/naimiskandar22/details/skills?profileUrn=urn%3Ali%3Afsd_profile%3AACoAAC6BPkEBwSD4NTQ7khSJbuAxrRMYhVSwmBo" data-test-app-aware-link=""><!---->Edit skills<!----></a></strong>
                    </div>
                  </div>
                  </div>
      </div>

      <div id="ember94" class="artdeco-modal__actionbar ember-view job-details-skill-match-modal__footer">
        <button id="ember95" class="artdeco-button artdeco-button--2 artdeco-button--primary ember-view"><!---->
<span class="artdeco-button__text">
    Done
</span></button>
      </div>
    
        <span class="a11y-text">Dialog content end.</span>
      </div>
    </div>
</div>

<code style="display: none" id="bpr-guid-15912206" aria-hidden="true">
  {"data":{"entityUrn":"urn:li:collectionResponse:4HlijJjSiEXmgxSNTaZbyKnZ88db1eTefgOnimBx6wM=","elements":[{"lixTracking":{"urn":"urn:li:member:780222017","segmentIndex":5,"experimentId":4753206,"treatmentIndex":0,"$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigLixTrackingInfo"},"chameleonConfigTrackingItem":{"configLixTrackingInfoListV2":[{"lixTracking":{"urn":"urn:li:member:780222017","segmentIndex":5,"experimentId":4753206,"treatmentIndex":0,"$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigLixTrackingInfo"},"lixTreatment":"control","lixKey":"chameleon.GROWTH_global.copy-definition.51323.child.51327","$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigLixTrackingInfoWrapper"},{"lixTracking":{"urn":"urn:li:member:780222017","segmentIndex":5,"experimentId":4753207,"treatmentIndex":0,"$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigLixTrackingInfo"},"lixTreatment":"control","lixKey":"chameleon.GROWTH_global.copy-definition.51323.child.51327","$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigLixTrackingInfoWrapper"}],"$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigTrackingItem"},"data":{"namespace":"onboarding/components/widgets/job-seeker-intent","message":"","locale":"en_US","key":"i18n_onboarding_job_seeker_intent_option_none","$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigDataI18n"},"parentLixKey":"chameleon.GROWTH_global.copy-definition.51323.parent","lixTreatment":"control","parentLixTracking":{"urn":"urn:li:member:780222017","segmentIndex":5,"experimentId":4753207,"treatmentIndex":0,"$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigLixTrackingInfo"},"lixKey":"chameleon.GROWTH_global.copy-definition.51323.child.51327","$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigItem"},{"lixTracking":{"urn":"urn:li:member:780222017","segmentIndex":6,"experimentId":4741090,"treatmentIndex":1,"$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigLixTrackingInfo"},"chameleonConfigTrackingItem":{"configLixTrackingInfoListV2":[{"lixTracking":{"urn":"urn:li:member:780222017","segmentIndex":6,"experimentId":4741090,"treatmentIndex":1,"$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigLixTrackingInfo"},"lixTreatment":"VAR_t50329_PR_1","lixKey":"chameleon.ACQUIRE_Global.web-copy-definition.50327.child.50329","$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigLixTrackingInfoWrapper"},{"lixTracking":{"urn":"urn:li:member:780222017","segmentIndex":6,"experimentId":4741091,"treatmentIndex":1,"$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigLixTrackingInfo"},"lixTreatment":"VAR_t50327_PR_1","lixKey":"chameleon.ACQUIRE_Global.web-copy-definition.50327.child.50329","$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigLixTrackingInfoWrapper"}],"$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigTrackingItem"},"data":{"namespace":"support-chat/components/virtual-chat-assistant","message":"Chat with an expert now","locale":"en_US","key":"i18n_chat_online_text","$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigDataI18n"},"parentLixKey":"chameleon.ACQUIRE_Global.web-copy-definition.50327.parent","lixTreatment":"VAR_t50329_PR_1","parentLixTracking":{"urn":"urn:li:member:780222017","segmentIndex":6,"experimentId":4741091,"treatmentIndex":1,"$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigLixTrackingInfo"},"lixKey":"chameleon.ACQUIRE_Global.web-copy-definition.50327.child.50329","$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigItem"},{"lixTracking":{"urn":"urn:li:member:780222017","segmentIndex":5,"experimentId":4753204,"treatmentIndex":0,"$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigLixTrackingInfo"},"chameleonConfigTrackingItem":{"configLixTrackingInfoListV2":[{"lixTracking":{"urn":"urn:li:member:780222017","segmentIndex":5,"experimentId":4753204,"treatmentIndex":0,"$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigLixTrackingInfo"},"lixTreatment":"control","lixKey":"chameleon.GROWTH_global.copy-definition.51323.child.51325","$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigLixTrackingInfoWrapper"},{"lixTracking":{"urn":"urn:li:member:780222017","segmentIndex":5,"experimentId":4753207,"treatmentIndex":0,"$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigLixTrackingInfo"},"lixTreatment":"control","lixKey":"chameleon.GROWTH_global.copy-definition.51323.child.51325","$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigLixTrackingInfoWrapper"}],"$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigTrackingItem"},"data":{"namespace":"onboarding/components/widgets/job-seeker-intent","message":"","locale":"en_US","key":"i18n_onboarding_job_seeker_intent_option_active","$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigDataI18n"},"parentLixKey":"chameleon.GROWTH_global.copy-definition.51323.parent","lixTreatment":"control","parentLixTracking":{"urn":"urn:li:member:780222017","segmentIndex":5,"experimentId":4753207,"treatmentIndex":0,"$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigLixTrackingInfo"},"lixKey":"chameleon.GROWTH_global.copy-definition.51323.child.51325","$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigItem"},{"lixTracking":{"urn":"urn:li:member:780222017","segmentIndex":2,"experimentId":4560784,"treatmentIndex":1,"$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigLixTrackingInfo"},"chameleonConfigTrackingItem":{"configLixTrackingInfoListV2":[{"lixTracking":{"urn":"urn:li:member:780222017","segmentIndex":2,"experimentId":4560784,"treatmentIndex":1,"$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigLixTrackingInfo"},"lixTreatment":"VAR_t29537_PR_1","lixKey":"chameleon.PREMIUM:us.copy-definition.27071.child.29537","$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigLixTrackingInfoWrapper"},{"lixTracking":{"urn":"urn:li:member:780222017","segmentIndex":2,"experimentId":4560787,"treatmentIndex":1,"$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigLixTrackingInfo"},"lixTreatment":"VAR_t29516_PR_1","lixKey":"chameleon.PREMIUM:us.copy-definition.27071.child.29537","$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigLixTrackingInfoWrapper"}],"$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigTrackingItem"},"data":{"namespace":"feed/components/identity-module","message":"See your Premium features","locale":"en_US","key":"premium_access_headline","$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigDataI18n"},"parentLixKey":"chameleon.PREMIUM:us.copy-definition.27071.parent","lixTreatment":"VAR_t29537_PR_1","parentLixTracking":{"urn":"urn:li:member:780222017","segmentIndex":2,"experimentId":4560787,"treatmentIndex":1,"$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigLixTrackingInfo"},"lixKey":"chameleon.PREMIUM:us.copy-definition.27071.child.29537","$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigItem"},{"lixTracking":{"urn":"urn:li:member:780222017","segmentIndex":5,"experimentId":4753205,"treatmentIndex":0,"$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigLixTrackingInfo"},"chameleonConfigTrackingItem":{"configLixTrackingInfoListV2":[{"lixTracking":{"urn":"urn:li:member:780222017","segmentIndex":5,"experimentId":4753205,"treatmentIndex":0,"$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigLixTrackingInfo"},"lixTreatment":"control","lixKey":"chameleon.GROWTH_global.copy-definition.51323.child.51326","$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigLixTrackingInfoWrapper"},{"lixTracking":{"urn":"urn:li:member:780222017","segmentIndex":5,"experimentId":4753207,"treatmentIndex":0,"$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigLixTrackingInfo"},"lixTreatment":"control","lixKey":"chameleon.GROWTH_global.copy-definition.51323.child.51326","$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigLixTrackingInfoWrapper"}],"$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigTrackingItem"},"data":{"namespace":"onboarding/components/widgets/job-seeker-intent","message":"","locale":"en_US","key":"i18n_onboarding_job_seeker_intent_option_passive","$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigDataI18n"},"parentLixKey":"chameleon.GROWTH_global.copy-definition.51323.parent","lixTreatment":"control","parentLixTracking":{"urn":"urn:li:member:780222017","segmentIndex":5,"experimentId":4753207,"treatmentIndex":0,"$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigLixTrackingInfo"},"lixKey":"chameleon.GROWTH_global.copy-definition.51323.child.51326","$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigItem"},{"lixTracking":{"urn":"urn:li:member:780222017","segmentIndex":5,"experimentId":4727865,"treatmentIndex":2,"$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigLixTrackingInfo"},"chameleonConfigTrackingItem":{"configLixTrackingInfoListV2":[{"lixTracking":{"urn":"urn:li:member:780222017","segmentIndex":5,"experimentId":4727865,"treatmentIndex":2,"$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigLixTrackingInfo"},"lixTreatment":"VAR_t48324_PR_2","lixKey":"chameleon.GROWTH_global.web-copy-definition.31854","$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigLixTrackingInfoWrapper"}],"$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigTrackingItem"},"data":{"namespace":"mynetwork/templates/network-manager/people-follow/following","message":"Follow experts in your field to see their industry-specific content in your feed.","locale":"en_US","key":"i18n_following_empty_state_description","$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigDataI18n"},"lixTreatment":"VAR_t48324_PR_2","lixKey":"chameleon.GROWTH_global.web-copy-definition.31854","$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigItem"},{"lixTracking":{"urn":"urn:li:member:780222017","segmentIndex":2,"experimentId":4562276,"treatmentIndex":1,"$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigLixTrackingInfo"},"chameleonConfigTrackingItem":{"configLixTrackingInfoListV2":[{"lixTracking":{"urn":"urn:li:member:780222017","segmentIndex":2,"experimentId":4562276,"treatmentIndex":1,"$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigLixTrackingInfo"},"lixTreatment":"VAR_t33653_PR_1","lixKey":"chameleon.CAREER-SEARCH-EVAL:GLOBAL.web-copy-definition.27904","$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigLixTrackingInfoWrapper"}],"$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigTrackingItem"},"data":{"namespace":"job-details-engine/components/detail/job-view-layout","message":"More jobs","locale":"en_US","key":"i18n_simjob_header","$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigDataI18n"},"lixTreatment":"VAR_t33653_PR_1","lixKey":"chameleon.CAREER-SEARCH-EVAL:GLOBAL.web-copy-definition.27904","$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigItem"},{"lixTracking":{"urn":"urn:li:member:780222017","segmentIndex":3,"experimentId":4704568,"treatmentIndex":0,"$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigLixTrackingInfo"},"chameleonConfigTrackingItem":{"configLixTrackingInfoListV2":[{"lixTracking":{"urn":"urn:li:member:780222017","segmentIndex":3,"experimentId":4704568,"treatmentIndex":0,"$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigLixTrackingInfo"},"lixTreatment":"control","lixKey":"chameleon.PREMIUM_us.web-copy-definition.31708","$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigLixTrackingInfoWrapper"}],"$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigTrackingItem"},"data":{"namespace":"hiring/components/jobs/applicants/top-choice/top-choice-card","message":"","locale":"en_US","key":"i18n_top_choice_survey_prompt","$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigDataI18n"},"lixTreatment":"control","lixKey":"chameleon.PREMIUM_us.web-copy-definition.31708","$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigItem"},{"lixTracking":{"urn":"urn:li:member:780222017","segmentIndex":5,"experimentId":4749819,"treatmentIndex":0,"$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigLixTrackingInfo"},"chameleonConfigTrackingItem":{"configLixTrackingInfoListV2":[{"lixTracking":{"urn":"urn:li:member:780222017","segmentIndex":5,"experimentId":4749819,"treatmentIndex":0,"$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigLixTrackingInfo"},"lixTreatment":"control","lixKey":"chameleon.OJ_GLOBAL.web-copy-definition.50683","$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigLixTrackingInfoWrapper"}],"$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigTrackingItem"},"data":{"namespace":"job-posting/templates/index","message":"","locale":"en_US","key":"i18n_form_subheader","$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigDataI18n"},"lixTreatment":"control","lixKey":"chameleon.OJ_GLOBAL.web-copy-definition.50683","$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigItem"},{"lixTracking":{"urn":"urn:li:member:780222017","segmentIndex":3,"experimentId":4729785,"treatmentIndex":1,"$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigLixTrackingInfo"},"chameleonConfigTrackingItem":{"configLixTrackingInfoListV2":[{"lixTracking":{"urn":"urn:li:member:780222017","segmentIndex":3,"experimentId":4729785,"treatmentIndex":1,"$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigLixTrackingInfo"},"lixTreatment":"VAR_t48704_PR_3","lixKey":"chameleon.ONBOARDING_global.web-copy-definition.32052","$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigLixTrackingInfoWrapper"}],"$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigTrackingItem"},"data":{"namespace":"onboarding/components/widgets/get-the-app","message":"See {:firstName}{:firstName,possessive} full profile on the app","locale":"en_US","key":"i18n_onboarding_get_the_app_page_header_profile","$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigDataI18n"},"lixTreatment":"VAR_t48704_PR_3","lixKey":"chameleon.ONBOARDING_global.web-copy-definition.32052","$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigItem"},{"lixTracking":{"urn":"urn:li:member:780222017","segmentIndex":5,"experimentId":4721390,"treatmentIndex":1,"$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigLixTrackingInfo"},"chameleonConfigTrackingItem":{"configLixTrackingInfoListV2":[{"lixTracking":{"urn":"urn:li:member:780222017","segmentIndex":5,"experimentId":4721390,"treatmentIndex":1,"$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigLixTrackingInfo"},"lixTreatment":"VAR_t48262_PR_2","lixKey":"chameleon.OJ_GLOBAL.web-copy-definition.31824","$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigLixTrackingInfoWrapper"}],"$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigTrackingItem"},"data":{"namespace":"job-posting-shared/components/budget/budget-setter","message":"{:amount} daily max","locale":"en_US","key":"i18n_budget_recommended_daily","$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigDataI18n"},"lixTreatment":"VAR_t48262_PR_2","lixKey":"chameleon.OJ_GLOBAL.web-copy-definition.31824","$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigItem"},{"lixTracking":{"urn":"urn:li:member:780222017","segmentIndex":2,"experimentId":4697659,"treatmentIndex":1,"$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigLixTrackingInfo"},"chameleonConfigTrackingItem":{"configLixTrackingInfoListV2":[{"lixTracking":{"urn":"urn:li:member:780222017","segmentIndex":2,"experimentId":4697659,"treatmentIndex":1,"$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigLixTrackingInfo"},"lixTreatment":"VAR_t47452_PR_1","lixKey":"chameleon.PROFILE_GLOBAL.web-copy-definition.31506","$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigLixTrackingInfoWrapper"}],"$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigTrackingItem"},"data":{"namespace":"profile-creator-mode-shared/components/creator-tools-list/item","message":"Learn more","locale":"en_US","key":"i18n_available","$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigDataI18n"},"lixTreatment":"VAR_t47452_PR_1","lixKey":"chameleon.PROFILE_GLOBAL.web-copy-definition.31506","$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigItem"},{"lixTracking":{"urn":"urn:li:member:780222017","segmentIndex":4,"experimentId":4626639,"treatmentIndex":1,"$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigLixTrackingInfo"},"chameleonConfigTrackingItem":{"configLixTrackingInfoListV2":[{"lixTracking":{"urn":"urn:li:member:780222017","segmentIndex":4,"experimentId":4626639,"treatmentIndex":1,"$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigLixTrackingInfo"},"lixTreatment":"VAR_t42421_PR_1","lixKey":"chameleon.PROJECTX:Global.web-copy-definition.29643.child.42421","$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigLixTrackingInfoWrapper"},{"lixTracking":{"urn":"urn:li:member:780222017","segmentIndex":4,"experimentId":4626644,"treatmentIndex":1,"$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigLixTrackingInfo"},"lixTreatment":"VAR_t42433_PR_1","lixKey":"chameleon.PROJECTX:Global.web-copy-definition.29643.child.42421","$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigLixTrackingInfoWrapper"}],"$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigTrackingItem"},"data":{"namespace":"social-details/components/x-article-insightful-button","message":"Mark as insightful","locale":"en_US","key":"i18n_mark_as_insightful","$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigDataI18n"},"parentLixKey":"chameleon.PROJECTX:Global.ssr-copy-definition.29653.parent","lixTreatment":"VAR_t42421_PR_1","parentLixTracking":{"urn":"urn:li:member:780222017","segmentIndex":4,"experimentId":4626644,"treatmentIndex":1,"$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigLixTrackingInfo"},"lixKey":"chameleon.PROJECTX:Global.web-copy-definition.29643.child.42421","$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigItem"},{"lixTracking":{"urn":"urn:li:member:780222017","segmentIndex":3,"experimentId":4510388,"treatmentIndex":0,"$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigLixTrackingInfo"},"chameleonConfigTrackingItem":{"configLixTrackingInfoListV2":[{"lixTracking":{"urn":"urn:li:member:780222017","segmentIndex":3,"experimentId":4510388,"treatmentIndex":0,"$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigLixTrackingInfo"},"lixTreatment":"control","lixKey":"chameleon.PAGES:Global.web-copy-definition.26519","$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigLixTrackingInfoWrapper"}],"$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigTrackingItem"},"data":{"namespace":"organization-admin-pages-entrypoint/components/card","message":"","locale":"en_US","key":"i18n_org_page_notifications","$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigDataI18n"},"lixTreatment":"control","lixKey":"chameleon.PAGES:Global.web-copy-definition.26519","$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigItem"},{"lixTracking":{"urn":"urn:li:member:780222017","segmentIndex":4,"experimentId":4626638,"treatmentIndex":1,"$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigLixTrackingInfo"},"chameleonConfigTrackingItem":{"configLixTrackingInfoListV2":[{"lixTracking":{"urn":"urn:li:member:780222017","segmentIndex":4,"experimentId":4626638,"treatmentIndex":1,"$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigLixTrackingInfo"},"lixTreatment":"VAR_t42420_PR_1","lixKey":"chameleon.PROJECTX:Global.web-copy-definition.29643.child.42420","$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigLixTrackingInfoWrapper"},{"lixTracking":{"urn":"urn:li:member:780222017","segmentIndex":4,"experimentId":4626644,"treatmentIndex":1,"$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigLixTrackingInfo"},"lixTreatment":"VAR_t42433_PR_1","lixKey":"chameleon.PROJECTX:Global.web-copy-definition.29643.child.42420","$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigLixTrackingInfoWrapper"}],"$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigTrackingItem"},"data":{"namespace":"social-details/components/x-article-insightful-button","message":"{:value,number,integer} {:value,choice,singular#reaction|plural#reactions}","locale":"en_US","key":"i18n_insightful_count","$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigDataI18n"},"parentLixKey":"chameleon.PROJECTX:Global.ssr-copy-definition.29653.parent","lixTreatment":"VAR_t42420_PR_1","parentLixTracking":{"urn":"urn:li:member:780222017","segmentIndex":4,"experimentId":4626644,"treatmentIndex":1,"$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigLixTrackingInfo"},"lixKey":"chameleon.PROJECTX:Global.web-copy-definition.29643.child.42420","$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigItem"},{"lixTracking":{"urn":"urn:li:member:780222017","segmentIndex":5,"experimentId":4741617,"treatmentIndex":0,"$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigLixTrackingInfo"},"chameleonConfigTrackingItem":{"configLixTrackingInfoListV2":[{"lixTracking":{"urn":"urn:li:member:780222017","segmentIndex":5,"experimentId":4741617,"treatmentIndex":0,"$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigLixTrackingInfo"},"lixTreatment":"control","lixKey":"chameleon.OJ_GLOBAL.web-copy-definition.50312.child.50314","$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigLixTrackingInfoWrapper"},{"lixTracking":{"urn":"urn:li:member:780222017","segmentIndex":5,"experimentId":4741618,"treatmentIndex":0,"$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigLixTrackingInfo"},"lixTreatment":"control","lixKey":"chameleon.OJ_GLOBAL.web-copy-definition.50312.child.50314","$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigLixTrackingInfoWrapper"}],"$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigTrackingItem"},"data":{"namespace":"job-posting/templates/index","message":"","locale":"en_US","key":"i18n_form_draft_job_on_my_own","$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigDataI18n"},"parentLixKey":"chameleon.OJ_GLOBAL.web-copy-definition.50312.parent","lixTreatment":"control","parentLixTracking":{"urn":"urn:li:member:780222017","segmentIndex":5,"experimentId":4741618,"treatmentIndex":0,"$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigLixTrackingInfo"},"lixKey":"chameleon.OJ_GLOBAL.web-copy-definition.50312.child.50314","$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigItem"},{"lixTracking":{"urn":"urn:li:member:780222017","segmentIndex":6,"experimentId":4749852,"treatmentIndex":0,"$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigLixTrackingInfo"},"chameleonConfigTrackingItem":{"configLixTrackingInfoListV2":[{"lixTracking":{"urn":"urn:li:member:780222017","segmentIndex":6,"experimentId":4749852,"treatmentIndex":0,"$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigLixTrackingInfo"},"lixTreatment":"control","lixKey":"chameleon.GROWTH_global.web-copy-definition.51057","$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigLixTrackingInfoWrapper"}],"$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigTrackingItem"},"data":{"namespace":"onboarding/components/widgets/follow-recommendations-common","message":"","locale":"en_US","key":"i18n_onboarding_follow_recommendations_footer_no_follows","$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigDataI18n"},"lixTreatment":"control","lixKey":"chameleon.GROWTH_global.web-copy-definition.51057","$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigItem"},{"lixTracking":{"urn":"urn:li:member:780222017","segmentIndex":5,"experimentId":4741616,"treatmentIndex":0,"$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigLixTrackingInfo"},"chameleonConfigTrackingItem":{"configLixTrackingInfoListV2":[{"lixTracking":{"urn":"urn:li:member:780222017","segmentIndex":5,"experimentId":4741616,"treatmentIndex":0,"$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigLixTrackingInfo"},"lixTreatment":"control","lixKey":"chameleon.OJ_GLOBAL.web-copy-definition.50312.child.50313","$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigLixTrackingInfoWrapper"},{"lixTracking":{"urn":"urn:li:member:780222017","segmentIndex":5,"experimentId":4741618,"treatmentIndex":0,"$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigLixTrackingInfo"},"lixTreatment":"control","lixKey":"chameleon.OJ_GLOBAL.web-copy-definition.50312.child.50313","$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigLixTrackingInfoWrapper"}],"$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigTrackingItem"},"data":{"namespace":"job-posting/templates/index","message":"","locale":"en_US","key":"i18n_form_draft_job_with_ai_cta","$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigDataI18n"},"parentLixKey":"chameleon.OJ_GLOBAL.web-copy-definition.50312.parent","lixTreatment":"control","parentLixTracking":{"urn":"urn:li:member:780222017","segmentIndex":5,"experimentId":4741618,"treatmentIndex":0,"$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigLixTrackingInfo"},"lixKey":"chameleon.OJ_GLOBAL.web-copy-definition.50312.child.50313","$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigItem"},{"lixTracking":{"urn":"urn:li:member:780222017","segmentIndex":6,"experimentId":4741089,"treatmentIndex":1,"$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigLixTrackingInfo"},"chameleonConfigTrackingItem":{"configLixTrackingInfoListV2":[{"lixTracking":{"urn":"urn:li:member:780222017","segmentIndex":6,"experimentId":4741089,"treatmentIndex":1,"$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigLixTrackingInfo"},"lixTreatment":"VAR_t50328_PR_1","lixKey":"chameleon.ACQUIRE_Global.web-copy-definition.50327.child.50328","$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigLixTrackingInfoWrapper"},{"lixTracking":{"urn":"urn:li:member:780222017","segmentIndex":6,"experimentId":4741091,"treatmentIndex":1,"$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigLixTrackingInfo"},"lixTreatment":"VAR_t50327_PR_1","lixKey":"chameleon.ACQUIRE_Global.web-copy-definition.50327.child.50328","$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigLixTrackingInfoWrapper"}],"$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigTrackingItem"},"data":{"namespace":"support-chat/components/virtual-chat-assistant","message":"Ready to decide?","locale":"en_US","key":"i18n_tooltip_title","$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigDataI18n"},"parentLixKey":"chameleon.ACQUIRE_Global.web-copy-definition.50327.parent","lixTreatment":"VAR_t50328_PR_1","parentLixTracking":{"urn":"urn:li:member:780222017","segmentIndex":6,"experimentId":4741091,"treatmentIndex":1,"$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigLixTrackingInfo"},"lixKey":"chameleon.ACQUIRE_Global.web-copy-definition.50327.child.50328","$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigItem"},{"lixTracking":{"urn":"urn:li:member:780222017","segmentIndex":5,"experimentId":4510389,"treatmentIndex":0,"$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigLixTrackingInfo"},"chameleonConfigTrackingItem":{"configLixTrackingInfoListV2":[{"lixTracking":{"urn":"urn:li:member:780222017","segmentIndex":5,"experimentId":4510389,"treatmentIndex":0,"$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigLixTrackingInfo"},"lixTreatment":"control","lixKey":"chameleon.PAGES:Global.web-copy-definition.26520","$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigLixTrackingInfoWrapper"}],"$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigTrackingItem"},"data":{"namespace":"organization-admin/components/admin/admin-nav","message":"","locale":"en_US","key":"i18n_org_activity_tab","$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigDataI18n"},"lixTreatment":"control","lixKey":"chameleon.PAGES:Global.web-copy-definition.26520","$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigItem"},{"lixTracking":{"urn":"urn:li:member:780222017","segmentIndex":2,"experimentId":4656757,"treatmentIndex":1,"$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigLixTrackingInfo"},"chameleonConfigTrackingItem":{"configLixTrackingInfoListV2":[{"lixTracking":{"urn":"urn:li:member:780222017","segmentIndex":2,"experimentId":4656757,"treatmentIndex":1,"$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigLixTrackingInfo"},"lixTreatment":"VAR_t42423_PR_1","lixKey":"chameleon.PROJECTX:Global.web-copy-definition.29644.child.42423","$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigLixTrackingInfoWrapper"},{"lixTracking":{"urn":"urn:li:member:780222017","segmentIndex":2,"experimentId":4656763,"treatmentIndex":1,"$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigLixTrackingInfo"},"lixTreatment":"VAR_t42434_PR_1","lixKey":"chameleon.PROJECTX:Global.web-copy-definition.29644.child.42423","$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigLixTrackingInfoWrapper"}],"$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigTrackingItem"},"data":{"namespace":"social-details/components/x-article-insightful-button","message":"Upvote as insightful","locale":"en_US","key":"i18n_upvote_as_insightful","$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigDataI18n"},"parentLixKey":"chameleon.PROJECTX:Global.ssr-copy-definition.29654.parent","lixTreatment":"VAR_t42423_PR_1","parentLixTracking":{"urn":"urn:li:member:780222017","segmentIndex":2,"experimentId":4656763,"treatmentIndex":1,"$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigLixTrackingInfo"},"lixKey":"chameleon.PROJECTX:Global.web-copy-definition.29644.child.42423","$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigItem"},{"lixTracking":{"urn":"urn:li:member:780222017","segmentIndex":2,"experimentId":4656758,"treatmentIndex":1,"$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigLixTrackingInfo"},"chameleonConfigTrackingItem":{"configLixTrackingInfoListV2":[{"lixTracking":{"urn":"urn:li:member:780222017","segmentIndex":2,"experimentId":4656758,"treatmentIndex":1,"$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigLixTrackingInfo"},"lixTreatment":"VAR_t42424_PR_1","lixKey":"chameleon.PROJECTX:Global.web-copy-definition.29644.child.42424","$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigLixTrackingInfoWrapper"},{"lixTracking":{"urn":"urn:li:member:780222017","segmentIndex":2,"experimentId":4656763,"treatmentIndex":1,"$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigLixTrackingInfo"},"lixTreatment":"VAR_t42434_PR_1","lixKey":"chameleon.PROJECTX:Global.web-copy-definition.29644.child.42424","$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigLixTrackingInfoWrapper"}],"$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigTrackingItem"},"data":{"namespace":"social-details/components/x-article-insightful-button","message":"{:value,number,integer} {:value,choice,singular#insightful upvote|plural#insightful upvotes}","locale":"en_US","key":"i18n_reaction_count","$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigDataI18n"},"parentLixKey":"chameleon.PROJECTX:Global.ssr-copy-definition.29654.parent","lixTreatment":"VAR_t42424_PR_1","parentLixTracking":{"urn":"urn:li:member:780222017","segmentIndex":2,"experimentId":4656763,"treatmentIndex":1,"$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigLixTrackingInfo"},"lixKey":"chameleon.PROJECTX:Global.web-copy-definition.29644.child.42424","$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigItem"}],"paging":{"count":10,"start":0,"links":[]},"$type":"com.linkedin.restli.common.CollectionResponse"},"included":[]}
</code>
<code style="display: none" id="datalet-bpr-guid-15912206" aria-hidden="true">
  {"request":"/voyager/api/voyagerSegmentsDashChameleonConfig","status":200,"body":"bpr-guid-15912206","method":"GET","headers":{"x-li-uuid":"AAYLE+vtCEZ6Amp7P3S1tA\u003D\u003D"}}
</code>
<img src="data:image/gif;base64,R0lGODlhAQABAIAAAAAAAP///yH5BAEAAAAALAAAAAABAAEAAAIBRAA7" style="display: none" class="datalet-bpr-guid-15912206" aria-hidden="true"><code style="display: none" id="bpr-guid-15912207" aria-hidden="true">
  {"data":{"data":{"$recipeTypes":["com.linkedin.8e0c7a8828e4c16fcdc1196207bbd156"],"contactsDashMemberEmailAddressByFindEmailForVerification":{"$recipeTypes":["com.linkedin.ed8fec8807266362228e5d3cd36838b1"],"elements":[],"$type":"com.linkedin.restli.common.CollectionResponse"},"$type":"com.linkedin.8e0c7a8828e4c16fcdc1196207bbd156"},"extensions":{"webMetadata":{}}},"meta":{"microSchema":{"isGraphQL":true,"version":"2.1","types":{"com.linkedin.f5137496a80e6bbe4ca1511182d896f0":{"fields":{"emailAddress":{"type":"com.linkedin.common.EmailAddress"}},"baseType":"com.linkedin.voyager.dash.contacts.MemberEmailAddress"},"com.linkedin.ed8fec8807266362228e5d3cd36838b1":{"fields":{"elements":{"type":{"array":"com.linkedin.f5137496a80e6bbe4ca1511182d896f0"}}},"baseType":"com.linkedin.restli.common.CollectionResponse"},"com.linkedin.8e0c7a8828e4c16fcdc1196207bbd156":{"fields":{"contactsDashMemberEmailAddressByFindEmailForVerification":{"type":"com.linkedin.ed8fec8807266362228e5d3cd36838b1"}},"baseType":"com.linkedin.graphql.Query"}}}},"included":[]}
</code>
<code style="display: none" id="datalet-bpr-guid-15912207" aria-hidden="true">
  {"request":"/voyager/api/graphql?includeWebMetadata\u003Dtrue\u0026queryId\u003DvoyagerContactsDashMemberEmailAddress.dd06cc4aca95a6afd4b45e17a18a4007","status":200,"body":"bpr-guid-15912207","method":"GET","headers":{"x-li-uuid":"AAYLE+vtCEZ6Amp7P3S1tA\u003D\u003D"}}
</code>
<img src="data:image/gif;base64,R0lGODlhAQABAIAAAAAAAP///yH5BAEAAAAALAAAAAABAAEAAAIBRAA7" style="display: none" class="datalet-bpr-guid-15912207" aria-hidden="true"><code style="display: none" id="bpr-guid-15912208" aria-hidden="true">
  {"data":{"canBrowseProfiles":false,"reactivationFeaturesEligible":false,"canViewJobAnalytics":false,"canViewWVMP":false,"premiumFreeTrialEligible":true,"canAccessLearningVideos":true,"premiumLearningUpgradeEligible":false,"canViewCompanyInsights":false,"$type":"com.linkedin.voyager.premium.FeatureAccess"},"included":[]}
</code>
<code style="display: none" id="datalet-bpr-guid-15912208" aria-hidden="true">
  {"request":"/voyager/api/premium/featureAccess?name\u003DreactivationFeaturesEligible","status":200,"body":"bpr-guid-15912208","method":"GET","headers":{"x-li-uuid":"AAYLE+vtCEZ6Amp7P3S1tA\u003D\u003D"}}
</code>
<img src="data:image/gif;base64,R0lGODlhAQABAIAAAAAAAP///yH5BAEAAAAALAAAAAABAAEAAAIBRAA7" style="display: none" class="datalet-bpr-guid-15912208" aria-hidden="true"><code style="display: none" id="bpr-guid-15912209" aria-hidden="true">
  {"data":{"plainId":780222017,"publicContactInfo":{"$type":"com.linkedin.voyager.identity.shared.PublicContactInfo"},"premiumSubscriber":false,"*miniProfile":"urn:li:fs_miniProfile:ACoAAC6BPkEBwSD4NTQ7khSJbuAxrRMYhVSwmBo","$type":"com.linkedin.voyager.common.Me"},"included":[{"customPronoun":null,"memorialized":false,"lastName":"Zahari","dashEntityUrn":"urn:li:fsd_profile:ACoAAC6BPkEBwSD4NTQ7khSJbuAxrRMYhVSwmBo","standardizedPronoun":null,"occupation":"Performance Analyst | Aspiring Data Scientist | Data Enthusiast | Game Programmer","objectUrn":"urn:li:member:780222017","backgroundImage":null,"picture":{"artifacts":[{"width":99,"fileIdentifyingUrlPathSegment":"100_100/0/1605279012830?e=1706745600&amp;v=beta&amp;t=GoGDD2DE3nGXji1ktdhEWgWX0uNRNd93omIm9L4PTBM","expiresAt":1706745600000,"height":100,"$type":"com.linkedin.common.VectorArtifact"},{"width":199,"fileIdentifyingUrlPathSegment":"200_200/0/1605279012830?e=1706745600&amp;v=beta&amp;t=KxJr8NDL0QX2hH6Hgu-7kOudbuCrhYafGAmjrDIUX6g","expiresAt":1706745600000,"height":200,"$type":"com.linkedin.common.VectorArtifact"},{"width":399,"fileIdentifyingUrlPathSegment":"400_400/0/1605279012830?e=1706745600&amp;v=beta&amp;t=MEX6KTkEsBN1Iwy4RYn8IoyQikzi3wK-ws0Ij5Qr478","expiresAt":1706745600000,"height":400,"$type":"com.linkedin.common.VectorArtifact"},{"width":670,"fileIdentifyingUrlPathSegment":"800_800/0/1605279012830?e=1706745600&amp;v=beta&amp;t=72OsosG2vCVJHRjJ9BEqeLcfgJkzqOn-4Eq5tGu4964","expiresAt":1706745600000,"height":670,"$type":"com.linkedin.common.VectorArtifact"}],"rootUrl":"https://media.licdn.com/dms/image/C5603AQEkRciOp6B6dA/profile-displayphoto-shrink_","$type":"com.linkedin.common.VectorImage"},"$type":"com.linkedin.voyager.identity.shared.MiniProfile","firstName":"Naim Iskandar","entityUrn":"urn:li:fs_miniProfile:ACoAAC6BPkEBwSD4NTQ7khSJbuAxrRMYhVSwmBo","publicIdentifier":"naimiskandar22","trackingId":"a8kNoZxCT6CMzZRO1s6H+w=="}]}
</code>
<code style="display: none" id="datalet-bpr-guid-15912209" aria-hidden="true">
  {"request":"/voyager/api/me","status":200,"body":"bpr-guid-15912209","method":"GET","headers":{"x-li-uuid":"AAYLE+vtCEZ6Amp7P3S1tA\u003D\u003D"}}
</code>
<img src="data:image/gif;base64,R0lGODlhAQABAIAAAAAAAP///yH5BAEAAAAALAAAAAABAAEAAAIBRAA7" style="display: none" class="datalet-bpr-guid-15912209" aria-hidden="true"><code style="display: none" id="bpr-guid-15912210" aria-hidden="true">
  {"data":{"entityUrn":"urn:li:collectionResponse:71q+z9OBlTRLck+JrP7RA9AfJZYM19vZlhsXU1XxaK0=","elements":[],"paging":{"count":10,"start":0,"links":[]},"$type":"com.linkedin.restli.common.CollectionResponse"},"meta":{"microSchema":{"version":"2","types":{"com.linkedin.voyager.dash.deco.common.forms.DropdownFormComponent":{"fields":{"placeHolderText":{"type":"com.linkedin.voyager.dash.deco.common.text.TextViewModelV2"},"controlName":{"type":"string"},"textSelectableOptions":{"type":{"array":"com.linkedin.voyager.dash.deco.common.forms.TextSelectableOption"}}},"baseType":"com.linkedin.voyager.dash.common.forms.DropdownFormComponent"},"com.linkedin.deco.recipe.anonymous.Anon1080314395":{"fields":{"ringContentType":{"type":"string"},"actionTarget":{"type":"string"},"preDashEntityUrn":{"type":"string"},"entityUrn":{"type":"string"},"emphasized":{"type":"boolean"}},"baseType":"com.linkedin.voyager.dash.common.image.RingStatus"},"com.linkedin.deco.recipe.anonymous.Anon1117966179":{"fields":{"clusterType":{"type":"string"},"selectionCountRangeValidation":{"type":"com.linkedin.voyager.dash.deco.common.forms.SelectionCountRangeValidation"}},"baseType":"com.linkedin.voyager.dash.common.forms.ContextualRangeValidation"},"com.linkedin.deco.recipe.anonymous.Anon2065966944":{"fields":{"endpoint":{"type":"string"},"contextualSuggestionDependent":{"type":{"map":"com.linkedin.deco.recipe.anonymous.Anon109927483"},"resolvedFrom":"contextualSuggestionDependentUrns"},"contextualSuggestionQueryParameterUrns":{"type":{"array":"string"}},"contextualSuggestionDependentUrns":{"type":{"array":"string"}},"contextualSuggestionQueryParameter":{"type":{"map":"com.linkedin.deco.recipe.anonymous.Anon109927483"},"resolvedFrom":"contextualSuggestionQueryParameterUrns"},"contextualSuggestionType":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.common.forms.ContextualSuggestionsTriggers"},"com.linkedin.deco.recipe.anonymous.Anon489758292":{"fields":{"viewer":{"type":"com.linkedin.deco.recipe.anonymous.Anon430310092","resolvedFrom":"viewerUrn"},"contentSeriesUrn":{"type":"string"},"contentSeries":{"type":"com.linkedin.deco.recipe.anonymous.Anon1789236903","resolvedFrom":"contentSeriesUrn"},"viewerUrn":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.relationships.MemberToContentSeriesSubscribeMetadata"},"com.linkedin.voyager.dash.deco.common.forms.EntityInput":{"fields":{"inputEntityName":{"type":"string"},"inputEntityUrn":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.common.forms.EntityInputValue"},"com.linkedin.deco.recipe.anonymous.Anon377984030":{"fields":{"professionalEvent":{"type":"com.linkedin.deco.recipe.anonymous.Anon1213723597","resolvedFrom":"professionalEventUrn"},"vectorImage":{"type":"com.linkedin.voyager.dash.deco.common.VectorImageOnlyRootUrlAndAttribution"},"professionalEventUrn":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.common.image.NonEntityProfessionalEventLogo"},"com.linkedin.voyager.dash.deco.common.image.Company":{"fields":{"logo":{"type":{"union":{"url":"string","vectorImage":"com.linkedin.voyager.dash.deco.common.VectorImageOnlyRootUrlAndAttribution"}}},"entityUrn":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.organization.Company"},"com.linkedin.voyager.dash.deco.feed.publishing.SubscribeAction":{"fields":{"subscribed":{"type":"boolean"},"preDashEntityUrn":{"type":"string"},"entityUrn":{"type":"string"},"subscriberCount":{"type":"int"}},"baseType":"com.linkedin.voyager.dash.publishing.SubscribeAction"},"com.linkedin.voyager.dash.deco.launchpad.EdgeBuildingCohortReason":{"fields":{"sourceType":{"type":"string"},"reasonObjects":{"type":{"array":"string"}},"reasonContext":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.launchpad.attribute.EdgeBuildingCohortReason"},"com.linkedin.deco.recipe.anonymous.Anon1444411496":{"fields":{"countryUrn":{"type":"string"},"country":{"type":"com.linkedin.deco.recipe.anonymous.Anon1918694303","resolvedFrom":"countryUrn"},"defaultLocalizedNameWithoutCountryName":{"type":"string"},"entityUrn":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.common.Geo"},"com.linkedin.deco.recipe.anonymous.Anon1226238669":{"fields":{"name":{"type":"string"},"logo":{"type":{"union":{"url":"string","vectorImage":"com.linkedin.voyager.dash.deco.common.VectorImageOnlyRootUrlAndAttribution"}}},"universalName":{"type":"string"},"entityUrn":{"type":"string"},"url":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.organization.Company"},"com.linkedin.deco.recipe.anonymous.Anon798864187":{"fields":{"influencer":{"type":"boolean"},"entityUrn":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.identity.profile.Profile"},"com.linkedin.voyager.dash.deco.common.ux.StatefulButtonModel":{"fields":{"actionDataModel":{"type":{"union":{"follow":"com.linkedin.voyager.dash.deco.feed.FollowingState","connectionOrInvitation":"com.linkedin.voyager.dash.deco.relationships.MemberRelationshipV2","memberToEntityRelationship":"com.linkedin.voyager.dash.deco.relationships.DirectionalEntityRelationship"}},"resolvedFrom":"actionDataModelUnion"},"actionDataModelUnion":{"type":{"union":{"connectionOrInvitation":"string","relationshipActionData":"com.linkedin.voyager.dash.deco.common.ux.RelationshipActionData","memberToEntityRelationship":"string","follow":"string"}}},"emphasisStyle":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.common.ux.button.StatefulButtonModel"},"com.linkedin.deco.recipe.anonymous.Anon1636615691":{"fields":{"transcripts":{"type":{"array":"string"}},"pagesPerResolution":{"type":{"array":"com.linkedin.deco.recipe.anonymous.Anon1525160909"}}},"baseType":"com.linkedin.documentcontent.DocumentPages"},"com.linkedin.deco.recipe.anonymous.Anon934606613":{"fields":{"type":{"type":"string"},"name":{"type":"string"},"controlName":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.relationships.invitation.InvitationRelationshipOption"},"com.linkedin.voyager.dash.deco.common.TapTargetWithoutEntity":{"fields":{"stickerLinkViewUnion":{"type":{"union":{"mediumTemplate":"com.linkedin.deco.recipe.anonymous.Anon739258202","largeTemplate":"com.linkedin.deco.recipe.anonymous.Anon2003298771","smallTemplate":"com.linkedin.deco.recipe.anonymous.Anon548743587"}}},"stickerLinkTemplateSize":{"type":"string"},"firstCornerXOffsetPercentage":{"type":"float"},"type":{"type":"string"},"thirdCornerYOffsetPercentage":{"type":"float"},"url":{"type":"string"},"urn":{"type":"string"},"thirdCornerXOffsetPercentage":{"type":"float"},"secondCornerYOffsetPercentage":{"type":"float"},"fourthCornerXOffsetPercentage":{"type":"float"},"firstCornerYOffsetPercentage":{"type":"float"},"untaggable":{"type":"boolean"},"rank":{"type":"int"},"text":{"type":"string"},"fourthCornerYOffsetPercentage":{"type":"float"},"secondCornerXOffsetPercentage":{"type":"float"}},"baseType":"com.linkedin.voyager.dash.common.TapTarget"},"com.linkedin.deco.recipe.anonymous.Anon1956213100":{"fields":{"productCategories":{"type":{"array":"string"}},"signatureProduct":{"type":"boolean"},"localizedName":{"type":"string"},"organizationsUsingProduct":{"type":{"map":"com.linkedin.deco.recipe.anonymous.Anon570745354"},"resolvedFrom":"organizationsUsingProductUrns"},"entityUrn":{"type":"string"},"logo":{"type":"com.linkedin.voyager.dash.deco.common.image.ImageViewModel"},"connectionsUsingProductProfiles":{"type":"com.linkedin.deco.recipe.anonymous.Anon1122581856","isInjection":true},"company":{"type":"com.linkedin.deco.recipe.anonymous.Anon1927941263","resolvedFrom":"companyUrn"},"universalName":{"type":"string"},"organizationsUsingProductUrns":{"type":{"array":"string"}},"productCategoriesResolutionResults":{"type":{"map":"com.linkedin.voyager.dash.deco.organization.FullProductCategory"},"resolvedFrom":"productCategories"},"companyUrn":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.organization.OrganizationProduct"},"com.linkedin.voyager.dash.deco.premium.FullPremiumButton":{"fields":{"controlName":{"type":"string"},"openExternally":{"type":"boolean"},"appearance":{"type":"com.linkedin.voyager.dash.deco.common.ux.ButtonAppearance"},"accessibilityText":{"type":"string"},"url":{"type":"string"},"disable":{"type":"boolean"}},"baseType":"com.linkedin.voyager.dash.premium.PremiumButton"},"com.linkedin.deco.recipe.anonymous.Anon1312950162":{"fields":{"entityUrn":{"type":"string"},"memberRelationshipUnion":{"type":{"union":{"self":"com.linkedin.restli.common.EmptyRecord","connection":"com.linkedin.deco.recipe.anonymous.Anon2134644859","noConnection":"com.linkedin.deco.recipe.anonymous.Anon1792705474"}}}},"baseType":"com.linkedin.voyager.dash.relationships.MemberRelationship"},"com.linkedin.voyager.dash.deco.organization.OrganizationPermissions":{"fields":{"canCreateBroadcast":{"type":"boolean"},"canReadContentSuggestions":{"type":"boolean"},"canSeeProducts":{"type":"boolean"},"canUpdateOrganizationProfile":{"type":"boolean"},"canSponsorShare":{"type":"boolean"},"canEditPendingDirectSponsoredContentPosters":{"type":"boolean"},"canReadAdminFeedFollowingPill":{"type":"boolean"},"canUntagFromMention":{"type":"boolean"},"canReadOrganizationActivity":{"type":"boolean"},"canManageCareerPages":{"type":"boolean"},"canAssociateHashtag":{"type":"boolean"},"canDeleteShare":{"type":"boolean"},"canInviteMemberToFollow":{"type":"boolean"},"canEditPipelineBuilderAdministrators":{"type":"boolean"},"canReadBroadcastAnalytics":{"type":"boolean"},"canManageMessagingAccess":{"type":"boolean"},"canMembersInviteToFollow":{"type":"boolean"},"canReadPipelineBuilderAdminPage":{"type":"boolean"},"canDeactivateOrganization":{"type":"boolean"},"canCreateOrganicShare":{"type":"boolean"},"canNotifyEmployeesOfShare":{"type":"boolean"},"canEditDarkShare":{"type":"boolean"},"canSeePostJobButton":{"type":"boolean"},"canEnableCommentsShare":{"type":"boolean"},"canReadOrganizationLeadsAnalytics":{"type":"boolean"},"canDeleteDarkShare":{"type":"boolean"},"canCreateJobPosting":{"type":"boolean"},"canEditLeadGenerationFormManagers":{"type":"boolean"},"canReadMessages":{"type":"boolean"},"canReadOrganizationTalentBrandAnalytics":{"type":"boolean"},"canEditDirectSponsoredContentPosters":{"type":"boolean"},"canReadOrganizationNewsletterAnalytics":{"type":"boolean"},"canReadAdminDashboard":{"type":"boolean"},"canPinShare":{"type":"boolean"},"canReadPipelineBuilderAdministrators":{"type":"boolean"},"canCreateLinkedInPagesProductFeedBack":{"type":"boolean"},"canReadEvents":{"type":"boolean"},"organizationRoles":{"type":{"array":"com.linkedin.voyager.dash.deco.organization.OrganizationRoleType"}},"canEditEvents":{"type":"boolean"},"canManageOrganizationalPageFollow":{"type":"boolean"},"canReadOrganizationUpdateAnalytics":{"type":"boolean"},"canCreateComment":{"type":"boolean"},"canRequestAdminAccess":{"type":"boolean"},"canEditAdministrators":{"type":"boolean"},"canCreateDarkShare":{"type":"boolean"},"canEmployeesInviteToFollow":{"type":"boolean"},"canSeeEmployeeExperienceAsMember":{"type":"boolean"},"canReadAdministrators":{"type":"boolean"},"canExportLeads":{"type":"boolean"},"canReadDirectSponsoredContentPosters":{"type":"boolean"},"canCreateShowcase":{"type":"boolean"},"canManageEmployeeExperienceSettings":{"type":"boolean"},"canReadLeadGenerationFormManagers":{"type":"boolean"},"canReadOrganizationVisitorAnalytics":{"type":"boolean"},"canEditCurators":{"type":"boolean"},"canEditProducts":{"type":"boolean"},"canManageVerifiedEmailDomains":{"type":"boolean"},"canReadPendingAdministrators":{"type":"boolean"},"canReadPendingDirectSponsoredContentPosters":{"type":"boolean"},"canEditPipelineBuilderAdminPage":{"type":"boolean"},"canReadTermsAndAgreements":{"type":"boolean"},"canReadOrganizationFollowerAnalytics":{"type":"boolean"},"canManageOrganizationCompetitors":{"type":"boolean"},"canSendMessages":{"type":"boolean"},"canCreateReaction":{"type":"boolean"},"canDisableCommentsShare":{"type":"boolean"},"canEditPendingAdministrators":{"type":"boolean"},"canNotifyEmployees":{"type":"boolean"},"canReadOrganizationPipelineBuilderAnalytics":{"type":"boolean"},"canSeeOrganizationAdministrativePage":{"type":"boolean"}},"baseType":"com.linkedin.voyager.dash.organization.OrganizationPermissions"},"com.linkedin.voyager.dash.deco.common.video.Transcript":{"fields":{"captionFormat":{"type":"string"},"locale":{"type":"com.linkedin.voyager.dash.deco.common.Locale"},"lines":{"type":{"array":"com.linkedin.voyager.dash.deco.common.video.TranscriptLine"}},"isAutogenerated":{"type":"boolean"},"captionFile":{"type":"string"}},"baseType":"com.linkedin.videocontent.Transcript"},"com.linkedin.deco.recipe.anonymous.Anon1286825962":{"fields":{"sharedSecret":{"type":"string"},"entityUrn":{"type":"string"},"invitationId":{"type":"long"}},"baseType":"com.linkedin.voyager.dash.relationships.invitation.Invitation"},"com.linkedin.voyager.dash.deco.common.image.OrganizationalPage":{"fields":{"entityUrn":{"type":"string"},"pageType":{"type":"string"},"logoV2":{"type":"com.linkedin.voyager.dash.deco.common.VectorImageOnlyRootUrlAndAttribution"}},"baseType":"com.linkedin.voyager.dash.organization.OrganizationalPage"},"com.linkedin.voyager.dash.deco.premium.PremiumUpsellCard":{"fields":{"image":{"type":"com.linkedin.voyager.dash.deco.common.image.ImageViewModel"},"largeTitle":{"type":"com.linkedin.voyager.dash.deco.common.text.TextViewModelV2"},"socialProofInsight":{"type":"com.linkedin.voyager.dash.deco.common.ux.baseInsightViewModel"},"subtext":{"type":"com.linkedin.voyager.dash.deco.common.text.TextViewModelV2"},"upsellOrderOrigin":{"type":"string"},"ctaText":{"type":"com.linkedin.voyager.dash.deco.common.text.TextViewModelV2"},"controlName":{"type":"string"},"actionUrl":{"type":"string"},"legoTrackingToken":{"type":"string"},"title":{"type":"com.linkedin.voyager.dash.deco.common.text.TextViewModelV2"},"globalLegoTrackingToken":{"type":"string"},"cancelCta":{"type":"com.linkedin.voyager.dash.deco.premium.FullPremiumButton"},"highlightedValues":{"type":{"array":"com.linkedin.voyager.dash.deco.premium.plans.FullPremiumFeature"}},"promotionLegoTrackingToken":{"type":"string"},"funnelCommonHeader":{"type":"com.linkedin.voyager.dash.deco.premium.FullPremiumFunnelCommonHeader"},"fireDiscoveryImpressionEvent":{"type":"boolean"},"subtitle":{"type":"com.linkedin.voyager.dash.deco.common.text.TextViewModelV2"},"dismissible":{"type":"boolean"},"layoutStyle":{"type":"string"},"ctaButtonSecondaryEmphasizedTheme":{"type":"boolean"},"colorScheme":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.premium.PremiumUpsellCard"},"com.linkedin.deco.recipe.anonymous.Anon1358424483":{"fields":{"text":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.sponsoredcontent.leadgen.TextFieldComponent"},"com.linkedin.deco.recipe.anonymous.Anon632521550":{"fields":{"entityUrn":{"type":"string"},"joinedAt":{"type":"long"},"profile":{"type":"com.linkedin.deco.recipe.anonymous.Anon437558093","resolvedFrom":"profileUrn"},"groupUrn":{"type":"string"},"preDashEntityUrn":{"type":"string"},"profileUrn":{"type":"string"},"status":{"type":"string"},"group":{"type":"com.linkedin.deco.recipe.anonymous.Anon1869367056","resolvedFrom":"groupUrn"}},"baseType":"com.linkedin.voyager.dash.groups.GroupMembership"},"com.linkedin.deco.recipe.anonymous.Anon523036483":{"fields":{"title":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.organization.FollowersMetadata"},"com.linkedin.deco.recipe.anonymous.Anon1949606970":{"fields":{"followerResolutionResult":{"type":"com.linkedin.deco.recipe.anonymous.Anon1313960689","resolvedFrom":"follower"},"followedAt":{"type":"com.linkedin.voyager.dash.deco.common.text.TextViewModelV2"},"follower":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.organization.OrganizationFollower"},"com.linkedin.deco.recipe.anonymous.Anon407602795":{"fields":{"requiredFieldMissingErrorText":{"type":"string"},"formComponentUnion":{"type":{"union":{"textFieldComponent":"com.linkedin.deco.recipe.anonymous.Anon1358424483","dropdownSelectComponent":"com.linkedin.deco.recipe.anonymous.Anon8381687","textInputComponent":"com.linkedin.deco.recipe.anonymous.Anon1619791307","checkboxComponent":"com.linkedin.deco.recipe.anonymous.Anon554507693"}}},"question":{"type":"com.linkedin.voyager.dash.deco.common.text.TextViewModelV2"},"required":{"type":"boolean"},"editable":{"type":"boolean"},"leadGenFormQuestionUrn":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.sponsoredcontent.leadgen.LeadGenFormQuestion"},"com.linkedin.voyager.dash.deco.common.forms.DateFormComponent":{"fields":{"dateText":{"type":"com.linkedin.voyager.dash.deco.common.text.TextViewModelV2"},"controlName":{"type":"string"},"dateValidation":{"type":"com.linkedin.voyager.dash.deco.common.forms.DateValidation"},"dateInputType":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.common.forms.DateFormComponent"},"com.linkedin.voyager.dash.deco.common.image.Group":{"fields":{"logo":{"type":{"union":{"url":"string","vectorImage":"com.linkedin.voyager.dash.deco.common.VectorImageOnlyRootUrlAndAttribution"}}},"entityUrn":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.groups.Group"},"com.linkedin.deco.recipe.anonymous.Anon1918694303":{"fields":{"entityUrn":{"type":"string"},"defaultLocalizedName":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.common.Geo"},"com.linkedin.voyager.dash.deco.launchpad.OrganizationTopCardForEntityFollow":{"fields":{"socialProofInsight":{"type":{"union":{"standard":"com.linkedin.deco.recipe.anonymous.Anon1350307109","compact":"com.linkedin.deco.recipe.anonymous.Anon853977300"}},"resolvedFrom":"socialProofInsightUnion"},"viewerPermissions":{"type":"com.linkedin.voyager.dash.deco.launchpad.OrganizationPermissions"},"salesNavigatorCompanyUrl":{"type":"string"},"industryV2Taxonomy":{"type":{"map":"com.linkedin.voyager.dash.deco.common.IndustryV2"},"resolvedFrom":"industryV2TaxonomyUrns"},"industry":{"type":{"map":"com.linkedin.voyager.dash.deco.common.Industry"},"resolvedFrom":"industryUrns"},"industryUrns":{"type":{"array":"string"}},"industryV2Urns":{"type":{"array":"string"}},"socialProofInsightUnion":{"type":{"union":{"standard":"string","compact":"string"}}},"employeeCount":{"type":"long"},"associatedSignatureProductUrn":{"type":"string"},"url":{"type":"string"},"callToAction":{"type":"com.linkedin.voyager.dash.deco.launchpad.CallToAction"},"relevanceReason":{"type":"com.linkedin.voyager.dash.deco.common.ux.baseInsightViewModel"},"associatedSignatureProduct":{"type":"com.linkedin.voyager.dash.deco.organization.FullMemberProduct","resolvedFrom":"associatedSignatureProductUrn"},"industryV2":{"type":{"map":"com.linkedin.voyager.dash.deco.common.Industry"},"resolvedFrom":"industryV2Urns"},"industryV2TaxonomyUrns":{"type":{"array":"string"}},"followingStateUrn":{"type":"string"},"entityUrn":{"type":"string"},"followingState":{"type":"com.linkedin.voyager.dash.deco.feed.FollowingState","resolvedFrom":"followingStateUrn"},"name":{"type":"string"},"logo":{"type":{"union":{"url":"string","vectorImage":"com.linkedin.voyager.dash.deco.common.VectorImageOnlyRootUrlAndAttribution"}}},"universalName":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.organization.Company"},"com.linkedin.deco.recipe.anonymous.Anon1213723597":{"fields":{"entityUrn":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.events.ProfessionalEvent"},"com.linkedin.voyager.dash.deco.premium.FullPremiumFunnelCommonHeader":{"fields":{"utype":{"type":"string"},"referenceId":{"type":"bytes"}},"baseType":"com.linkedin.voyager.dash.premium.PremiumFunnelCommonHeader"},"com.linkedin.deco.recipe.anonymous.Anon184130218":{"fields":{"name":{"type":"string"},"logo":{"type":{"union":{"url":"string","vectorImage":"com.linkedin.voyager.dash.deco.common.VectorImageOnlyRootUrlAndAttribution"}}},"entityUrn":{"type":"string"},"url":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.organization.Company"},"com.linkedin.deco.recipe.anonymous.Anon487759616":{"fields":{"company":{"type":"com.linkedin.deco.recipe.anonymous.Anon1226238669","resolvedFrom":"companyUrn"},"schoolName":{"type":"string"},"school":{"type":"com.linkedin.deco.recipe.anonymous.Anon1621960601","resolvedFrom":"schoolUrn"},"entityUrn":{"type":"string"},"companyUrn":{"type":"string"},"schoolUrn":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.identity.profile.Education"},"com.linkedin.deco.recipe.anonymous.Anon1251747613":{"fields":{"urn":{"type":"string"},"modelName":{"type":"string"},"fieldName":{"type":"string"},"value":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.common.StringFieldReference"},"com.linkedin.voyager.dash.deco.organization.FullMemberProduct":{"fields":{"organizationsUsingProduct":{"type":{"map":"com.linkedin.deco.recipe.anonymous.Anon1432799903"},"resolvedFrom":"organizationsUsingProductUrns"},"productUserTitle":{"type":{"map":"com.linkedin.deco.recipe.anonymous.Anon1369245809"},"resolvedFrom":"productUserTitleUrns"},"companyUrn":{"type":"string"},"productUserTitleUrns":{"type":{"array":"string"}},"similarProducts":{"type":"com.linkedin.deco.recipe.anonymous.Anon975174582","isInjection":true},"entityUrn":{"type":"string"},"organizationalPage":{"type":"com.linkedin.deco.recipe.anonymous.Anon1196575529","resolvedFrom":"organizationalPageUrn"},"logo":{"type":"com.linkedin.voyager.dash.deco.common.image.ImageViewModel"},"similarProductsFromSameOrganization":{"type":"com.linkedin.deco.recipe.anonymous.Anon1914510571","isInjection":true},"company":{"type":"com.linkedin.deco.recipe.anonymous.Anon995204871","resolvedFrom":"companyUrn"},"productCardCallToActionUnion":{"type":{"union":{"organizationProductLeadGenFormCallToAction":"com.linkedin.deco.recipe.anonymous.Anon346909806","organizationProductCallToAction":"com.linkedin.deco.recipe.anonymous.Anon625674184","organizationProductAddSkillCallToAction":"com.linkedin.deco.recipe.anonymous.Anon2115336743"}}},"state":{"type":"string"},"universalName":{"type":"string"},"groupUrn":{"type":"string"},"localizedDescription":{"type":"string"},"localizedWebsite":{"type":"string"},"customizableCallToActionUnion":{"type":{"union":{"organizationProductLeadGenFormCallToAction":"com.linkedin.deco.recipe.anonymous.Anon346909806","organizationProductCallToAction":"com.linkedin.deco.recipe.anonymous.Anon625674184","organizationProductAddSkillCallToAction":"com.linkedin.deco.recipe.anonymous.Anon2115336743"}}},"group":{"type":"com.linkedin.deco.recipe.anonymous.Anon690575082","resolvedFrom":"groupUrn"},"hashtag":{"type":{"map":"com.linkedin.deco.recipe.anonymous.Anon2070591469"},"resolvedFrom":"hashtagUrns"},"localizedName":{"type":"string"},"connectionsUsingProductProfiles":{"type":"com.linkedin.deco.recipe.anonymous.Anon1517647575","isInjection":true},"mediaSections":{"type":{"array":"com.linkedin.deco.recipe.anonymous.Anon805140200"}},"productCategoriesResolutionResults":{"type":{"map":"com.linkedin.voyager.dash.deco.organization.FullProductCategory"},"resolvedFrom":"productCategories"},"hashtagUrns":{"type":{"array":"string"}},"productCategories":{"type":{"array":"string"}},"memberFacingCallToActionUnion":{"type":{"union":{"organizationProductLeadGenFormCallToAction":"com.linkedin.deco.recipe.anonymous.Anon346909806","organizationProductCallToAction":"com.linkedin.deco.recipe.anonymous.Anon625674184","organizationProductAddSkillCallToAction":"com.linkedin.deco.recipe.anonymous.Anon2115336743"}}},"standardizedSkillUrns":{"type":{"array":"string"}},"organizationalPageUrn":{"type":"string"},"autoPublished":{"type":"boolean"},"organizationsUsingProductUrns":{"type":{"array":"string"}},"standardizedSkill":{"type":{"map":"com.linkedin.deco.recipe.anonymous.Anon503562015"},"resolvedFrom":"standardizedSkillUrns"}},"baseType":"com.linkedin.voyager.dash.organization.OrganizationProduct"},"com.linkedin.voyager.dash.deco.common.forms.PageKey":{"fields":{"pageKey":{"type":"string"},"anchorPage":{"type":"boolean"}},"baseType":"com.linkedin.voyager.dash.common.tracking.PageKey"},"com.linkedin.voyager.dash.deco.launchpad.OrganizationPermissions":{"fields":{"canReadOrganizationVisitorAnalytics":{"type":"boolean"},"canCreateBroadcast":{"type":"boolean"},"canMembersInviteToFollow":{"type":"boolean"},"canSeeOrganizationAdministrativePage":{"type":"boolean"}},"baseType":"com.linkedin.voyager.dash.organization.OrganizationPermissions"},"com.linkedin.deco.recipe.anonymous.Anon1869367056":{"fields":{"entityUrn":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.groups.Group"},"com.linkedin.voyager.dash.deco.identity.profile.ProfileActionsInjection":{"fields":{"secondaryActionResolutionResult":{"type":{"union":{"personalizedConnect":"com.linkedin.deco.recipe.anonymous.Anon4539795","saveToPdf":"com.linkedin.deco.recipe.anonymous.Anon587401631","followingState":"com.linkedin.deco.recipe.anonymous.Anon1760283115","ignore":"com.linkedin.deco.recipe.anonymous.Anon1312950162","composeOption":"com.linkedin.deco.recipe.anonymous.Anon1575907873","withdraw":"com.linkedin.deco.recipe.anonymous.Anon1213132949","saveInSalesNavigatorState":"com.linkedin.deco.recipe.anonymous.Anon1975396536"}},"resolvedFrom":"secondaryAction"},"primaryActionResolutionResult":{"type":{"union":{"personalizedConnect":"com.linkedin.deco.recipe.anonymous.Anon4539795","saveToPdf":"com.linkedin.deco.recipe.anonymous.Anon587401631","followingState":"com.linkedin.deco.recipe.anonymous.Anon1760283115","ignore":"com.linkedin.deco.recipe.anonymous.Anon1312950162","composeOption":"com.linkedin.deco.recipe.anonymous.Anon1575907873","withdraw":"com.linkedin.deco.recipe.anonymous.Anon1213132949","saveInSalesNavigatorState":"com.linkedin.deco.recipe.anonymous.Anon1975396536"}},"resolvedFrom":"primaryAction"},"trackingIdentifier":{"type":"bytes"},"primaryAction":{"type":{"union":{"personalizedConnect":"string","saveToPdf":"string","shareProfileUrlViaMessage":"string","saveToPdfUrl":"string","composeOption":"string","saveInSalesNavigatorState":"string","statefulAction":"com.linkedin.voyager.dash.deco.common.ux.StatefulButtonModel","viewProfileInExternalContext":"com.linkedin.voyager.dash.deco.identity.profile.ViewProfileInExternalContextAction","statelessAction":"string","followingState":"string","report":"com.linkedin.deco.recipe.anonymous.Anon743120773","ignore":"string","shareProfileUrl":"string","connection":"com.linkedin.voyager.dash.deco.identity.profile.MemberRelationshipWrapper","withdraw":"string"}}},"secondaryAction":{"type":{"union":{"personalizedConnect":"string","saveToPdf":"string","shareProfileUrlViaMessage":"string","saveToPdfUrl":"string","composeOption":"string","saveInSalesNavigatorState":"string","statefulAction":"com.linkedin.voyager.dash.deco.common.ux.StatefulButtonModel","viewProfileInExternalContext":"com.linkedin.voyager.dash.deco.identity.profile.ViewProfileInExternalContextAction","statelessAction":"string","followingState":"string","report":"com.linkedin.deco.recipe.anonymous.Anon743120773","ignore":"string","shareProfileUrl":"string","connection":"com.linkedin.voyager.dash.deco.identity.profile.MemberRelationshipWrapper","withdraw":"string"}}},"overflowActionsV2":{"type":{"array":"com.linkedin.deco.recipe.anonymous.Anon956797500"}},"overflowActionsResolutionResults":{"type":{"map":{"union":{"personalizedConnect":"com.linkedin.deco.recipe.anonymous.Anon4539795","saveToPdf":"com.linkedin.deco.recipe.anonymous.Anon587401631","followingState":"com.linkedin.deco.recipe.anonymous.Anon1760283115","ignore":"com.linkedin.deco.recipe.anonymous.Anon1312950162","composeOption":"com.linkedin.deco.recipe.anonymous.Anon1575907873","withdraw":"com.linkedin.deco.recipe.anonymous.Anon1213132949","saveInSalesNavigatorState":"com.linkedin.deco.recipe.anonymous.Anon1975396536"}}},"resolvedFrom":"overflowActions"},"overflowActions":{"type":{"array":{"union":{"personalizedConnect":"string","saveToPdf":"string","shareProfileUrlViaMessage":"string","saveToPdfUrl":"string","composeOption":"string","saveInSalesNavigatorState":"string","statefulAction":"com.linkedin.voyager.dash.deco.common.ux.StatefulButtonModel","viewProfileInExternalContext":"com.linkedin.voyager.dash.deco.identity.profile.ViewProfileInExternalContextAction","statelessAction":"string","followingState":"string","report":"com.linkedin.deco.recipe.anonymous.Anon743120773","ignore":"string","shareProfileUrl":"string","connection":"com.linkedin.voyager.dash.deco.identity.profile.MemberRelationshipWrapper","withdraw":"string"}}}},"secondaryActionV2":{"type":"com.linkedin.deco.recipe.anonymous.Anon956797500"},"trackingId":{"type":"string"},"primaryActionV2":{"type":"com.linkedin.deco.recipe.anonymous.Anon956797500"}},"baseType":"com.linkedin.voyager.dash.identity.profile.actions.ProfileActions"},"com.linkedin.deco.recipe.anonymous.Anon2127563472":{"fields":{"followerCount":{"type":"long"},"entityUrn":{"type":"string"},"preDashFollowingInfoUrn":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.feed.FollowingState"},"com.linkedin.deco.recipe.anonymous.Anon514683669":{"fields":{"currentProgress":{"type":"int"},"threshold":{"type":"int"},"progressText":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.launchpad.LaunchpadProgressMeter"},"com.linkedin.voyager.dash.deco.premium.plans.FullPremiumFeature":{"fields":{"excluded":{"type":"boolean"},"topFeature":{"type":"boolean"},"tooltip":{"type":"com.linkedin.voyager.dash.deco.common.text.TextViewModelV2"},"icon":{"type":"com.linkedin.voyager.dash.deco.common.image.ImageViewModel"},"description":{"type":"string"},"illustration":{"type":"com.linkedin.voyager.dash.deco.common.image.ImageViewModel"},"value":{"type":"int"},"headlineV2":{"type":"com.linkedin.voyager.dash.deco.common.text.TextViewModelV2"},"headline":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.premium.plans.PremiumFeature"},"com.linkedin.voyager.dash.deco.common.sponsored.SponsoredUrlAttributes":{"fields":{"impressionId":{"type":"string"},"unwrappedUrlDomain":{"type":"string"},"creativeId":{"type":"string"},"campaignId":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.common.sponsored.SponsoredUrlAttributes"},"com.linkedin.deco.recipe.anonymous.Anon2086023182":{"fields":{"geo":{"type":"com.linkedin.deco.recipe.anonymous.Anon1444411496","resolvedFrom":"geoUrn"},"geoUrn":{"type":"string"},"postalCode":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.common.forms.LocationInputValue"},"com.linkedin.voyager.dash.deco.relationships.MemberRelationshipV2ForInjection":{"fields":{"memberRelationshipData":{"type":{"union":{"noInvitation":"com.linkedin.deco.recipe.anonymous.Anon1031101697","invitation":"com.linkedin.voyager.dash.deco.relationships.Invitation","connection":"com.linkedin.voyager.dash.deco.relationships.Connection"}}},"entityUrn":{"type":"string"},"memberRelationshipUnion":{"type":{"union":{"self":"com.linkedin.restli.common.EmptyRecord","connection":"com.linkedin.voyager.dash.deco.relationships.Connection","noConnection":"com.linkedin.voyager.dash.deco.relationships.NoConnection"}}}},"baseType":"com.linkedin.voyager.dash.relationships.MemberRelationship"},"com.linkedin.voyager.dash.deco.organization.FullProductCategory":{"fields":{"name":{"type":"com.linkedin.voyager.dash.deco.common.text.TextViewModelV2"},"entityUrn":{"type":"string"},"productCategoryPageUrl":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.organization.ProductCategory"},"com.linkedin.deco.recipe.anonymous.Anon1428941918":{"fields":{"paging":{"type":"com.linkedin.voyager.dash.deco.common.FullPaging"},"elements":{"type":{"array":"com.linkedin.deco.recipe.anonymous.Anon2116040434"}}},"baseType":"com.linkedin.restli.common.CollectionResponse"},"com.linkedin.deco.recipe.anonymous.Anon4539795":{"fields":{"entityUrn":{"type":"string"},"memberRelationshipUnion":{"type":{"union":{"self":"com.linkedin.restli.common.EmptyRecord","connection":"com.linkedin.deco.recipe.anonymous.Anon2134644859","noConnection":"com.linkedin.deco.recipe.anonymous.Anon2058972220"}}}},"baseType":"com.linkedin.voyager.dash.relationships.MemberRelationship"},"com.linkedin.voyager.dash.deco.common.text.ProfileForMention":{"fields":{"entityUrn":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.identity.profile.Profile"},"com.linkedin.deco.recipe.anonymous.Anon870521084":{"fields":{"formElementInputs":{"type":{"array":"com.linkedin.voyager.dash.deco.common.forms.FormElementInput"}}},"baseType":"com.linkedin.voyager.dash.common.forms.PrerequisiteFormInputs"},"com.linkedin.voyager.dash.deco.common.VectorImageOnlyRootUrlAndAttribution":{"fields":{"attribution":{"type":"string"},"rootUrl":{"type":"string"},"artifacts":{"type":{"array":"com.linkedin.voyager.dash.deco.common.VectorArtifact"}}},"baseType":"com.linkedin.common.VectorImage"},"com.linkedin.deco.recipe.anonymous.Anon831664802":{"fields":{"entityUrn":{"type":"string"},"invitationId":{"type":"long"}},"baseType":"com.linkedin.voyager.dash.relationships.invitation.Invitation"},"com.linkedin.deco.recipe.anonymous.Anon2070591469":{"fields":{"actionTarget":{"type":"string"},"entityUrn":{"type":"string"},"followingState":{"type":"com.linkedin.deco.recipe.anonymous.Anon2127563472","isInjection":true},"displayName":{"type":"string"},"trackingUrn":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.feed.Hashtag"},"com.linkedin.deco.recipe.anonymous.Anon2105959233":{"fields":{"maximumFileSizeSupported":{"type":"long"},"mimeTypes":{"type":{"array":"string"}},"uploadFileControlName":{"type":"string"},"uploadFileCtaText":{"type":"string"},"deleteControlName":{"type":"string"},"mediaUploadType":{"type":"string"},"previewText":{"type":"com.linkedin.voyager.dash.deco.common.text.TextViewModelV2"},"helperText":{"type":"com.linkedin.voyager.dash.deco.common.text.TextViewModelV2"}},"baseType":"com.linkedin.voyager.dash.common.forms.MediaUploadFormComponent"},"com.linkedin.voyager.dash.deco.common.Industry":{"fields":{"name":{"type":"string"},"entityUrn":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.common.Industry"},"com.linkedin.deco.recipe.anonymous.Anon1202743511":{"fields":{"entityUrn":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.jobs.StandardizedSkill"},"com.linkedin.deco.recipe.anonymous.Anon912817537":{"fields":{"memberDistance":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.relationships.NoConnection"},"com.linkedin.deco.recipe.anonymous.Anon8381687":{"fields":{"multipleChoiceQuestionOptions":{"type":{"array":"com.linkedin.deco.recipe.anonymous.Anon1507465610"}},"placeholderText":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.sponsoredcontent.leadgen.DropdownSelectComponent"},"com.linkedin.deco.recipe.anonymous.Anon1689150826":{"fields":{"typeaheadFilterQuery":{"type":"com.linkedin.deco.recipe.anonymous.Anon1435489553"},"showFullLastNameForConnections":{"type":"boolean"},"typeaheadUseCase":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.search.TypeaheadQuery"},"com.linkedin.voyager.dash.deco.relationships.InvitationRaw":{"fields":{"genericInvitationType":{"type":"string"},"invitationId":{"type":"long"},"sharedSecret":{"type":"string"},"entityUrn":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.relationships.invitation.Invitation"},"com.linkedin.deco.recipe.anonymous.Anon739258202":{"fields":{"name":{"type":"com.linkedin.voyager.dash.deco.common.text.TextViewModelV2"},"nameSupplementaryInfo":{"type":"com.linkedin.voyager.dash.deco.common.text.TextViewModelV2"},"imageUnion":{"type":{"union":{"url":"string","vectorImage":"com.linkedin.voyager.dash.deco.common.VectorImageOnlyRootUrlAndAttribution"}}},"headline":{"type":"com.linkedin.voyager.dash.deco.common.text.TextViewModelV2"},"subHeadline":{"type":"com.linkedin.voyager.dash.deco.common.text.TextViewModelV2"}},"baseType":"com.linkedin.voyager.dash.common.media.StickerLinkMediumTemplateView"},"com.linkedin.deco.recipe.anonymous.Anon902172343":{"fields":{"topLevelOption":{"type":"com.linkedin.voyager.dash.deco.common.forms.TextSelectableOption"},"subOptions":{"type":{"array":"com.linkedin.voyager.dash.deco.common.forms.TextSelectableOption"}}},"baseType":"com.linkedin.voyager.dash.common.forms.NestedTextSelectableOption"},"com.linkedin.voyager.dash.deco.common.forms.FormSection":{"fields":{"title":{"type":"com.linkedin.voyager.dash.deco.common.text.TextViewModelV2"},"footerText":{"type":"com.linkedin.voyager.dash.deco.common.text.TextViewModelV2"},"collapsedState":{"type":"string"},"formElementGroups":{"type":{"array":"com.linkedin.voyager.dash.deco.common.forms.FormElementGroup"}},"footerTextTitle":{"type":"com.linkedin.voyager.dash.deco.common.text.TextViewModelV2"},"subtitle":{"type":"com.linkedin.voyager.dash.deco.common.text.TextViewModelV2"}},"baseType":"com.linkedin.voyager.dash.common.forms.FormSection"},"com.linkedin.deco.recipe.anonymous.Anon1266426900":{"fields":{"controlName":{"type":"string"},"text":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.common.forms.FormButton"},"com.linkedin.voyager.dash.deco.common.video.AdaptiveStream":{"fields":{"initialBitRate":{"type":"int"},"protocol":{"type":"string"},"mediaType":{"type":"string"},"masterPlaylists":{"type":{"array":"com.linkedin.voyager.dash.deco.common.video.StreamingLocation"}}},"baseType":"com.linkedin.videocontent.AdaptiveStream"},"com.linkedin.deco.recipe.anonymous.Anon1529112441":{"fields":{"originalImageUrl":{"type":"string"},"originalHeight":{"type":"int"},"url":{"type":"string"},"originalWidth":{"type":"int"}},"baseType":"com.linkedin.voyager.dash.common.ImageUrl"},"com.linkedin.voyager.dash.deco.common.video.StreamingLocation":{"fields":{"url":{"type":"string"},"expiresAt":{"type":"long"}},"baseType":"com.linkedin.videocontent.StreamingLocation"},"com.linkedin.voyager.dash.deco.launchpad.LaunchpadView":{"fields":{"launchpadCards":{"type":{"array":"com.linkedin.deco.recipe.anonymous.Anon1353776944"}},"dismissDialog":{"type":"com.linkedin.deco.recipe.anonymous.Anon1670704461"},"headerImage":{"type":"com.linkedin.voyager.dash.deco.common.image.ImageViewModel"},"focusedCardIndex":{"type":"int"},"progressMeter":{"type":"com.linkedin.deco.recipe.anonymous.Anon514683669"},"branding":{"type":"com.linkedin.deco.recipe.anonymous.Anon1963883771"},"dismissible":{"type":"boolean"},"pageKey":{"type":"com.linkedin.voyager.dash.deco.launchpad.PageKey"},"theme":{"type":"string"},"legoTrackingToken":{"type":"string"},"title":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.launchpad.LaunchpadView"},"com.linkedin.voyager.dash.deco.common.video.VideoPlayMetadata":{"fields":{"thumbnail":{"type":"com.linkedin.voyager.dash.deco.common.VectorImageOnlyRootUrl"},"progressiveStreams":{"type":{"array":"com.linkedin.voyager.dash.deco.common.video.ProgressiveDownloadMetadata"}},"liveStreamCreatedAt":{"type":"long"},"prevMedia":{"type":"string"},"transcripts":{"type":{"array":"com.linkedin.voyager.dash.deco.common.video.Transcript"}},"aspectRatio":{"type":"float"},"media":{"type":"string"},"adaptiveStreams":{"type":{"array":"com.linkedin.voyager.dash.deco.common.video.AdaptiveStream"}},"liveStreamEndedAt":{"type":"long"},"duration":{"type":"long"},"provider":{"type":"string"},"entityUrn":{"type":"string"},"nextMedia":{"type":"string"},"thumbnails":{"type":{"array":"com.linkedin.voyager.dash.deco.common.video.Thumbnail"}},"trackingId":{"type":"string"}},"baseType":"com.linkedin.videocontent.VideoPlayMetadata"},"com.linkedin.voyager.dash.deco.identity.profile.ProfilePhotoDecoSpecReadOnlyBackground":{"fields":{"displayImageUrn":{"type":"string"},"displayImageReference":{"type":{"union":{"url":"string","vectorImage":"com.linkedin.voyager.dash.deco.common.VectorImageOnlyRootUrlAndAttribution"}}}},"baseType":"com.linkedin.voyager.dash.identity.profile.PhotoFilterPicture"},"com.linkedin.deco.recipe.anonymous.Anon2058972220":{"fields":{"invitationUnion":{"type":{"union":{"noInvitation":"com.linkedin.deco.recipe.anonymous.Anon412783727","invitation":"com.linkedin.deco.recipe.anonymous.Anon831664802"}}}},"baseType":"com.linkedin.voyager.dash.relationships.NoConnection"},"com.linkedin.deco.recipe.anonymous.Anon561948584":{"fields":{"ctaText":{"type":"string"},"landingPage":{"type":"com.linkedin.deco.recipe.anonymous.Anon1586568086"},"document":{"type":"com.linkedin.deco.recipe.anonymous.Anon856218111"},"confirmationTitle":{"type":"com.linkedin.voyager.dash.deco.common.text.TextViewModelV2"},"urlViewingBehavior":{"type":"string"},"sponsoredUrlAttributes":{"type":"com.linkedin.voyager.dash.deco.common.sponsored.SponsoredUrlAttributes"},"confirmationDescription":{"type":"com.linkedin.voyager.dash.deco.common.text.TextViewModelV2"}},"baseType":"com.linkedin.voyager.dash.sponsoredcontent.leadgen.LeadGenGatedContent"},"com.linkedin.voyager.dash.deco.premium.PremiumUpsellSlotContent":{"fields":{"entityUrn":{"type":"string"},"upsellCard":{"type":"com.linkedin.voyager.dash.deco.premium.PremiumUpsellCard"}},"baseType":"com.linkedin.voyager.dash.premium.PremiumUpsellSlotContent"},"com.linkedin.voyager.dash.deco.relationships.Invitation":{"fields":{"inviteeMemberResolutionResult":{"type":"com.linkedin.deco.recipe.anonymous.Anon336652209","resolvedFrom":"inviteeMember"},"invitationType":{"type":"string"},"inviteeMember":{"type":"string"},"entityUrn":{"type":"string"},"invitationState":{"type":"string"},"invitationId":{"type":"long"},"sharedSecret":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.relationships.invitation.Invitation"},"com.linkedin.voyager.dash.deco.common.IndustryV2":{"fields":{"name":{"type":"string"},"entityUrn":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.identity.profile.IndustryV2"},"com.linkedin.voyager.dash.deco.launchpad.ProfileTopCardForEntityFollow":{"fields":{"lastName":{"type":"string"},"profileTopEducation":{"type":"com.linkedin.deco.recipe.anonymous.Anon964860152","isInjection":true},"profilePositions":{"type":"com.linkedin.deco.recipe.anonymous.Anon1861880920","isInjection":true},"educationOnProfileTopCardShown":{"type":"boolean"},"profileTopPosition":{"type":"com.linkedin.deco.recipe.anonymous.Anon1449981882","isInjection":true},"associatedHashtagsCopy":{"type":"com.linkedin.voyager.dash.deco.common.text.TextViewModelV2"},"profilePicture":{"type":"com.linkedin.voyager.dash.deco.identity.profile.ProfilePhotoDecoSpecReadOnlyProfilePicture"},"firstName":{"type":"string"},"profileInsight":{"type":"com.linkedin.deco.recipe.anonymous.Anon1439603308","isInjection":true},"geoLocation":{"type":"com.linkedin.deco.recipe.anonymous.Anon708807917"},"entityUrn":{"type":"string"},"followingState":{"type":"com.linkedin.deco.recipe.anonymous.Anon89639392","isInjection":true},"memberRelationship":{"type":"com.linkedin.voyager.dash.deco.relationships.MemberRelationshipV2ForInjection","isInjection":true},"companyNameOnProfileTopCardShown":{"type":"boolean"},"headline":{"type":"string"},"publicIdentifier":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.identity.profile.Profile"},"com.linkedin.deco.recipe.anonymous.Anon1662119579":{"fields":{"singleQuestionSubForm":{"type":"com.linkedin.deco.recipe.anonymous.Anon795093289"},"icon":{"type":"com.linkedin.voyager.dash.deco.common.image.ImageViewModel"},"controlName":{"type":"string"},"iconAfterText":{"type":"boolean"},"placeHolderText":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.common.forms.VisibilitySettingButton"},"com.linkedin.deco.recipe.anonymous.Anon1069349607":{"fields":{"entityUrn":{"type":"string"},"memberRelationshipUnion":{"type":{"union":{"self":"com.linkedin.restli.common.EmptyRecord","connection":"com.linkedin.deco.recipe.anonymous.Anon2134644859","noConnection":"com.linkedin.deco.recipe.anonymous.Anon912817537"}}}},"baseType":"com.linkedin.voyager.dash.relationships.MemberRelationship"},"com.linkedin.deco.recipe.anonymous.Anon554507693":{"fields":{"checked":{"type":"boolean"},"text":{"type":"com.linkedin.voyager.dash.deco.common.text.TextViewModelV2"}},"baseType":"com.linkedin.voyager.dash.sponsoredcontent.leadgen.CheckboxComponent"},"com.linkedin.deco.recipe.anonymous.Anon1128782547":{"fields":{"entityUrn":{"type":"string"},"joinedAt":{"type":"long"},"profile":{"type":"com.linkedin.deco.recipe.anonymous.Anon437558093","resolvedFrom":"profileUrn"},"groupUrn":{"type":"string"},"preDashEntityUrn":{"type":"string"},"profileUrn":{"type":"string"},"status":{"type":"string"},"group":{"type":"com.linkedin.deco.recipe.anonymous.Anon1869367056","resolvedFrom":"groupUrn"}},"baseType":"com.linkedin.voyager.dash.groups.GroupMembership"},"com.linkedin.deco.recipe.anonymous.Anon1509374374":{"fields":{"displayText":{"type":"string"},"url":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.organization.GenericCallToAction"},"com.linkedin.deco.recipe.anonymous.Anon503562015":{"fields":{"name":{"type":"string"},"entityUrn":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.jobs.StandardizedSkill"},"com.linkedin.deco.recipe.anonymous.Anon109927483":{"fields":{"entityUrn":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.common.forms.FormElement"},"com.linkedin.deco.recipe.anonymous.Anon86528645":{"fields":{"infoText":{"type":"string"},"hintText":{"type":"string"},"typeaheadQuery":{"type":"com.linkedin.deco.recipe.anonymous.Anon1689150826"},"typeaheadQueryContext":{"type":{"array":"string"}},"freeFormTextNotAllowedErrorText":{"type":"string"},"validationMetadataUnion":{"type":{"union":{"pronounsV2":"com.linkedin.voyager.dash.deco.common.forms.StringValidationMetadata","string":"com.linkedin.voyager.dash.deco.common.forms.StringValidationMetadata","textFormInputType":"string","postalCode":"com.linkedin.voyager.dash.deco.common.forms.PostalCodeValidationMetadata","name":"com.linkedin.voyager.dash.deco.common.forms.StringValidationMetadata","pronouns":"string","integer":"com.linkedin.voyager.dash.deco.common.forms.NumericValidationMetadata","decimal":"com.linkedin.voyager.dash.deco.common.forms.NumericValidationMetadata","headline":"com.linkedin.voyager.dash.deco.common.forms.StringValidationMetadata","url":"com.linkedin.deco.recipe.anonymous.Anon906563573"}}},"typeaheadType":{"type":"string"},"freeFormTextAllowed":{"type":"boolean"},"selectionLimitExceededErrorText":{"type":"string"},"clusters":{"type":{"array":"string"}},"navTitle":{"type":"string"},"clustersAtMax":{"type":{"array":"string"}}},"baseType":"com.linkedin.voyager.dash.common.forms.TypeaheadMetadata"},"com.linkedin.deco.recipe.anonymous.Anon708807917":{"fields":{"geo":{"type":"com.linkedin.deco.recipe.anonymous.Anon1444411496","resolvedFrom":"geoUrn"},"geoUrn":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.identity.profile.ProfileGeoLocation"},"com.linkedin.voyager.dash.deco.identity.profile.ProfilePhotoDecoSpecReadOnlyProfilePicture":{"fields":{"displayImageWithFrameReferenceUnion":{"type":{"union":{"url":"string","vectorImage":"com.linkedin.voyager.dash.deco.common.VectorImageOnlyRootUrlAndAttribution"}}},"frameType":{"type":"string"},"a11yText":{"type":"string"},"displayImageUrn":{"type":"string"},"displayImageReference":{"type":{"union":{"url":"string","vectorImage":"com.linkedin.voyager.dash.deco.common.VectorImageOnlyRootUrlAndAttribution"}}}},"baseType":"com.linkedin.voyager.dash.identity.profile.PhotoFilterPicture"},"com.linkedin.deco.recipe.anonymous.Anon858976209":{"fields":{"type":{"type":"string"},"epochAt":{"type":"long"}},"baseType":"com.linkedin.voyager.dash.common.text.EpochTime"},"com.linkedin.deco.recipe.anonymous.Anon1435489553":{"fields":{"geoSearchTypes":{"type":{"array":"string"}},"excludedSkillUrns":{"type":{"array":"string"}},"companyUrns":{"type":{"array":"string"}},"segmentAttribute":{"type":"com.linkedin.deco.recipe.anonymous.Anon462114681","resolvedFrom":"segmentAttributeUrn"},"countryCodes":{"type":{"array":"string"}},"segmentAttributeUrn":{"type":"string"},"excludedSkill":{"type":{"map":"com.linkedin.deco.recipe.anonymous.Anon1202743511"},"resolvedFrom":"excludedSkillUrns"},"countryCodesResolutionResults":{"type":{"map":"com.linkedin.deco.recipe.anonymous.Anon1271139837"},"resolvedFrom":"countryCodes"},"company":{"type":{"map":"com.linkedin.deco.recipe.anonymous.Anon1927941263"},"resolvedFrom":"companyUrns"},"standardizationEntityType":{"type":"string"},"groupUrn":{"type":"string"},"segmentProductSurface":{"type":"string"},"group":{"type":"com.linkedin.deco.recipe.anonymous.Anon1869367056","resolvedFrom":"groupUrn"}},"baseType":"com.linkedin.voyager.dash.search.TypeaheadFilterQuery"},"com.linkedin.deco.recipe.anonymous.Anon1517647575":{"fields":{"paging":{"type":"com.linkedin.voyager.dash.deco.common.FullPaging"},"metadata":{"type":"com.linkedin.deco.recipe.anonymous.Anon1585981083"},"elements":{"type":{"array":"com.linkedin.deco.recipe.anonymous.Anon1703899306"}}},"baseType":"com.linkedin.restli.common.CollectionResponse"},"com.linkedin.deco.recipe.anonymous.Anon462114681":{"fields":{"entityUrn":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.hiring.SegmentAttribute"},"com.linkedin.voyager.dash.deco.identity.profile.ViewProfileInExternalContextAction":{"fields":{"deeplinkUrl":{"type":"string"},"recruiterContext":{"type":"boolean"}},"baseType":"com.linkedin.voyager.dash.identity.profile.actions.ViewProfileInExternalContextAction"},"com.linkedin.voyager.dash.deco.common.ux.ButtonAppearance":{"fields":{"premiumStyle":{"type":"boolean"},"size":{"type":"string"},"icon":{"type":"string"},"text":{"type":"string"},"iconAfterText":{"type":"boolean"},"emphasize":{"type":"boolean"},"category":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.common.ux.button.ButtonAppearance"},"com.linkedin.voyager.dash.deco.common.forms.ChoicePill":{"fields":{"textSelectableOption":{"type":"com.linkedin.voyager.dash.deco.common.forms.TextSelectableOption"}},"baseType":"com.linkedin.voyager.dash.common.forms.ChoicePill"},"com.linkedin.deco.recipe.anonymous.Anon1843097075":{"fields":{"invitationUnion":{"type":{"union":{"noInvitation":"com.linkedin.deco.recipe.anonymous.Anon881636240","invitation":"com.linkedin.deco.recipe.anonymous.Anon1286825962"}}}},"baseType":"com.linkedin.voyager.dash.relationships.NoConnection"},"com.linkedin.voyager.dash.deco.common.forms.RadioButtonFormComponent":{"fields":{"selectableOptions":{"type":{"array":{"union":{"textSelectableOption":"com.linkedin.voyager.dash.deco.common.forms.TextSelectableOption","entitySelectableOption":"com.linkedin.voyager.dash.deco.common.forms.EntitySelectableOption"}}}},"textSelectableOptions":{"type":{"array":"com.linkedin.voyager.dash.deco.common.forms.TextSelectableOption"}}},"baseType":"com.linkedin.voyager.dash.common.forms.RadioButtonFormComponent"},"com.linkedin.deco.recipe.anonymous.Anon89639392":{"fields":{"followerCount":{"type":"long"},"entityUrn":{"type":"string"},"preDashFollowingInfoUrn":{"type":"string"},"following":{"type":"boolean"}},"baseType":"com.linkedin.voyager.dash.feed.FollowingState"},"com.linkedin.voyager.dash.deco.common.forms.DateRangeFormComponent":{"fields":{"startDateText":{"type":"com.linkedin.voyager.dash.deco.common.text.TextViewModelV2"},"startDateControlName":{"type":"string"},"endDateControlName":{"type":"string"},"ongoingDateRangeFormElementUrn":{"type":"string"},"endDateValidation":{"type":"com.linkedin.voyager.dash.deco.common.forms.DateValidation"},"endDateText":{"type":"com.linkedin.voyager.dash.deco.common.text.TextViewModelV2"},"startDateValidation":{"type":"com.linkedin.voyager.dash.deco.common.forms.DateValidation"},"ongoingDateText":{"type":"com.linkedin.voyager.dash.deco.common.text.TextViewModelV2"},"dateInputType":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.common.forms.DateRangeFormComponent"},"com.linkedin.voyager.dash.deco.common.forms.SingleLineTextFormComponent":{"fields":{"controlName":{"type":"string"},"hintText":{"type":"com.linkedin.voyager.dash.deco.common.text.TextViewModelV2"},"helperText":{"type":"com.linkedin.voyager.dash.deco.common.text.TextViewModelV2"},"validationMetadata":{"type":{"union":{"pronounsV2":"com.linkedin.voyager.dash.deco.common.forms.StringValidationMetadata","string":"com.linkedin.voyager.dash.deco.common.forms.StringValidationMetadata","textFormInputType":"string","postalCode":"com.linkedin.voyager.dash.deco.common.forms.PostalCodeValidationMetadata","name":"com.linkedin.voyager.dash.deco.common.forms.StringValidationMetadata","pronouns":"string","integer":"com.linkedin.voyager.dash.deco.common.forms.NumericValidationMetadata","decimal":"com.linkedin.voyager.dash.deco.common.forms.NumericValidationMetadata","headline":"com.linkedin.voyager.dash.deco.common.forms.StringValidationMetadata","url":"com.linkedin.deco.recipe.anonymous.Anon906563573"}}}},"baseType":"com.linkedin.voyager.dash.common.forms.SingleLineTextFormComponent"},"com.linkedin.voyager.dash.deco.common.video.TranscriptLine":{"fields":{"lineStartAt":{"type":"long"},"caption":{"type":"string"},"lineEndAt":{"type":"long"}},"baseType":"com.linkedin.videocontent.TranscriptLine"},"com.linkedin.voyager.dash.deco.launchpad.CallToAction":{"fields":{"displayText":{"type":"string"},"visible":{"type":"boolean"},"type":{"type":"string"},"url":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.organization.CallToAction"},"com.linkedin.deco.recipe.anonymous.Anon625674184":{"fields":{"type":{"type":"string"},"callToAction":{"type":"com.linkedin.deco.recipe.anonymous.Anon1509374374"}},"baseType":"com.linkedin.voyager.dash.organization.OrganizationProductCallToAction"},"com.linkedin.voyager.dash.deco.common.forms.MultilineTextFormComponent":{"fields":{"controlName":{"type":"string"},"hintText":{"type":"com.linkedin.voyager.dash.deco.common.text.TextViewModelV2"},"numVisibleLines":{"type":"int"},"helperText":{"type":"com.linkedin.voyager.dash.deco.common.text.TextViewModelV2"},"validationMetadata":{"type":{"union":{"pronounsV2":"com.linkedin.voyager.dash.deco.common.forms.StringValidationMetadata","string":"com.linkedin.voyager.dash.deco.common.forms.StringValidationMetadata","textFormInputType":"string","postalCode":"com.linkedin.voyager.dash.deco.common.forms.PostalCodeValidationMetadata","name":"com.linkedin.voyager.dash.deco.common.forms.StringValidationMetadata","pronouns":"string","integer":"com.linkedin.voyager.dash.deco.common.forms.NumericValidationMetadata","decimal":"com.linkedin.voyager.dash.deco.common.forms.NumericValidationMetadata","headline":"com.linkedin.voyager.dash.deco.common.forms.StringValidationMetadata","url":"com.linkedin.deco.recipe.anonymous.Anon906563573"}}}},"baseType":"com.linkedin.voyager.dash.common.forms.MultilineTextFormComponent"},"com.linkedin.voyager.dash.deco.common.forms.FormElementGroup":{"fields":{"subtitle":{"type":"com.linkedin.voyager.dash.deco.common.text.TextViewModelV2"},"spacingType":{"type":"string"},"title":{"type":"string"},"navigationButton":{"type":"com.linkedin.deco.recipe.anonymous.Anon1957375702"},"horizontalOrientation":{"type":"boolean"},"visibilitySettingButton":{"type":"com.linkedin.deco.recipe.anonymous.Anon1662119579"},"formElements":{"type":{"array":"com.linkedin.voyager.dash.deco.common.forms.FormElement"}}},"baseType":"com.linkedin.voyager.dash.common.forms.FormElementGroup"},"com.linkedin.voyager.dash.deco.common.text.OrganizationalPage":{"fields":{"entityUrn":{"type":"string"},"localizedName":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.organization.OrganizationalPage"},"com.linkedin.deco.recipe.anonymous.Anon600254320":{"fields":{"errorText":{"type":"com.linkedin.voyager.dash.deco.common.text.TextViewModelV2"},"validRange":{"type":"com.linkedin.voyager.dash.deco.common.FloatRange"}},"baseType":"com.linkedin.voyager.dash.common.forms.validation.NumericValueRangeValidation"},"com.linkedin.deco.recipe.anonymous.Anon1432799903":{"fields":{"industryV2Taxonomy":{"type":{"map":"com.linkedin.voyager.dash.deco.common.IndustryV2"},"resolvedFrom":"industryV2TaxonomyUrns"},"industry":{"type":{"map":"com.linkedin.voyager.dash.deco.common.Industry"},"resolvedFrom":"industryUrns"},"industryV2Urns":{"type":{"array":"string"}},"industryUrns":{"type":{"array":"string"}},"url":{"type":"string"},"relevanceReason":{"type":"com.linkedin.voyager.dash.deco.common.ux.baseInsightViewModel"},"industryV2":{"type":{"map":"com.linkedin.voyager.dash.deco.common.Industry"},"resolvedFrom":"industryV2Urns"},"industryV2TaxonomyUrns":{"type":{"array":"string"}},"followingStateUrn":{"type":"string"},"entityUrn":{"type":"string"},"followingState":{"type":"com.linkedin.voyager.dash.deco.feed.FollowingState","resolvedFrom":"followingStateUrn"},"name":{"type":"string"},"logo":{"type":{"union":{"url":"string","vectorImage":"com.linkedin.voyager.dash.deco.common.VectorImage"}}},"universalName":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.organization.Company"},"com.linkedin.deco.recipe.anonymous.Anon1031101697":{"fields":{"targetInviteeResolutionResult":{"type":"com.linkedin.voyager.dash.deco.relationships.ProfileWithEmailRequired","resolvedFrom":"targetInvitee"},"inviter":{"type":"string"},"targetInvitee":{"type":"string"},"invitationRelationshipForm":{"type":"com.linkedin.deco.recipe.anonymous.Anon918376989"},"inviterResolutionResult":{"type":"com.linkedin.voyager.dash.deco.relationships.ProfileWithIweWarned","resolvedFrom":"inviter"}},"baseType":"com.linkedin.voyager.dash.relationships.invitation.NoInvitation"},"com.linkedin.deco.recipe.anonymous.Anon1271139837":{"fields":{"entityUrn":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.common.Geo"},"com.linkedin.deco.recipe.anonymous.Anon1313960689":{"fields":{"profilePicture":{"type":"com.linkedin.voyager.dash.deco.identity.profile.ProfilePhotoDecoSpecReadOnlyProfilePicture"},"lastName":{"type":"string"},"insight":{"type":"com.linkedin.deco.recipe.anonymous.Anon137428918","isInjection":true},"firstName":{"type":"string"},"profileProfileActions":{"type":"com.linkedin.voyager.dash.deco.identity.profile.ProfileActionsInjection","isInjection":true},"entityUrn":{"type":"string"},"memberRelationship":{"type":"com.linkedin.voyager.dash.deco.relationships.MemberRelationshipV2ForInjection","isInjection":true},"profileTopPosition":{"type":"com.linkedin.deco.recipe.anonymous.Anon1428941918","isInjection":true},"publicIdentifier":{"type":"string"},"headline":{"type":"string"},"backgroundPicture":{"type":"com.linkedin.voyager.dash.deco.identity.profile.ProfilePhotoDecoSpecReadOnlyBackground"}},"baseType":"com.linkedin.voyager.dash.identity.profile.Profile"},"com.linkedin.deco.recipe.anonymous.Anon163061530":{"fields":{"name":{"type":"string"},"entityUrn":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.groups.Group"},"com.linkedin.voyager.dash.deco.common.forms.CharacterCountRangeValidation":{"fields":{"showCharacterCount":{"type":"boolean"},"errorText":{"type":"com.linkedin.voyager.dash.deco.common.text.TextViewModelV2"},"validRange":{"type":"com.linkedin.voyager.dash.deco.common.IntegerRange"}},"baseType":"com.linkedin.voyager.dash.common.forms.validation.CharacterCountRangeValidation"},"com.linkedin.voyager.dash.deco.relationships.profile":{"fields":{"lastName":{"type":"string"},"firstName":{"type":"string"},"creator":{"type":"boolean"},"entityUrn":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.identity.profile.Profile"},"com.linkedin.voyager.dash.deco.common.text.ProfileForFullName":{"fields":{"lastName":{"type":"string"},"firstName":{"type":"string"},"entityUrn":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.identity.profile.Profile"},"com.linkedin.voyager.dash.deco.common.forms.DismissPill":{"fields":{"textSelectableOption":{"type":"com.linkedin.voyager.dash.deco.common.forms.TextSelectableOption"}},"baseType":"com.linkedin.voyager.dash.common.forms.DismissPill"},"com.linkedin.voyager.dash.deco.common.forms.MultiSelectTypeaheadEntityFormComponent":{"fields":{"typeaheadCta":{"type":"com.linkedin.voyager.dash.deco.common.forms.TypeaheadCta"},"description":{"type":"com.linkedin.voyager.dash.deco.common.text.TextViewModelV2"},"selectedEntities":{"type":{"array":"com.linkedin.voyager.dash.deco.common.forms.EntitySelectableOption"}},"selectionCountRangeValidation":{"type":"com.linkedin.voyager.dash.deco.common.forms.SelectionCountRangeValidation"}},"baseType":"com.linkedin.voyager.dash.common.forms.MultiSelectTypeaheadEntityFormComponent"},"com.linkedin.deco.recipe.anonymous.Anon378816379":{"fields":{"viewer":{"type":"com.linkedin.deco.recipe.anonymous.Anon430310092","resolvedFrom":"viewerUrn"},"company":{"type":"com.linkedin.deco.recipe.anonymous.Anon660720507","resolvedFrom":"companyUrn"},"companyUrn":{"type":"string"},"viewerUrn":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.relationships.MemberToCompanyFollowMetadata"},"com.linkedin.voyager.dash.deco.common.forms.StringValidationMetadata":{"fields":{"characterCountRangeValidation":{"type":"com.linkedin.voyager.dash.deco.common.forms.CharacterCountRangeValidation"},"errorText":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.common.forms.validation.StringValidationMetadata"},"com.linkedin.deco.recipe.anonymous.Anon743120773":{"fields":{"authorProfileId":{"type":"string"},"targetUrn":{"type":"string"},"contentSource":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.common.semaphore.SemaphoreContext"},"com.linkedin.deco.recipe.anonymous.Anon805140200":{"fields":{"mediaContentUnion":{"type":{"union":{"embeddedVideo":"com.linkedin.voyager.dash.deco.organization.EmbeddedVideoDecoSpec","url":"string","videoPlayMetadata":"com.linkedin.voyager.dash.deco.common.video.VideoPlayMetadata","vectorImage":"com.linkedin.voyager.dash.deco.common.VectorImage"}}},"description":{"type":"string"},"mediaEntityUnion":{"type":{"union":{"digitalMediaAssetUrn":"string","articleUrn":"string"}}},"title":{"type":"string"},"mediaSource":{"type":"string"},"primary":{"type":"boolean"}},"baseType":"com.linkedin.voyager.dash.organization.MediaSection"},"com.linkedin.deco.recipe.anonymous.Anon93322792":{"fields":{"entityUrn":{"type":"string"},"organizationalPageUrn":{"type":"string"},"organizationalPage":{"type":"com.linkedin.deco.recipe.anonymous.Anon1196575529","resolvedFrom":"organizationalPageUrn"}},"baseType":"com.linkedin.voyager.dash.organization.Company"},"com.linkedin.deco.recipe.anonymous.Anon964860152":{"fields":{"paging":{"type":"com.linkedin.voyager.dash.deco.common.FullPaging"},"elements":{"type":{"array":"com.linkedin.deco.recipe.anonymous.Anon487759616"}}},"baseType":"com.linkedin.restli.common.CollectionResponse"},"com.linkedin.voyager.dash.deco.relationships.NoConnection":{"fields":{"memberDistance":{"type":"string"},"invitationUnion":{"type":{"union":{"noInvitation":"com.linkedin.deco.recipe.anonymous.Anon1031101697","invitation":"com.linkedin.voyager.dash.deco.relationships.Invitation"}}}},"baseType":"com.linkedin.voyager.dash.relationships.NoConnection"},"com.linkedin.voyager.dash.deco.common.forms.TextSelectableOption":{"fields":{"optionUnion":{"type":{"union":{"optionUrn":"string","optionEnumString":"string"}}},"controlName":{"type":"string"},"optionUrn":{"type":"string"},"optionEnumString":{"type":"string"},"optionText":{"type":"com.linkedin.voyager.dash.deco.common.text.TextViewModelV2"}},"baseType":"com.linkedin.voyager.dash.common.forms.TextSelectableOption"},"com.linkedin.deco.recipe.anonymous.Anon1575907873":{"fields":{"icon":{"type":"com.linkedin.voyager.dash.deco.common.image.ImageViewModel"},"composeNavigationContext":{"type":"com.linkedin.deco.recipe.anonymous.Anon199392175"},"displayText":{"type":"com.linkedin.voyager.dash.deco.common.text.TextViewModelV2"},"textStartIcon":{"type":"com.linkedin.voyager.dash.deco.common.image.ImageViewModel"},"composeOptionType":{"type":"string"},"entityUrn":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.messaging.compose.ComposeOption"},"com.linkedin.deco.recipe.anonymous.Anon40833127":{"fields":{"metadataUnion":{"type":{"union":{"memberToContentSeriesSubscribe":"com.linkedin.deco.recipe.anonymous.Anon489758292"}}},"subscribeActionUrn":{"type":"string"},"subscribeAction":{"type":"com.linkedin.voyager.dash.deco.feed.publishing.SubscribeAction","resolvedFrom":"subscribeActionUrn"}},"baseType":"com.linkedin.voyager.dash.relationships.SubscribeRelationship"},"com.linkedin.deco.recipe.anonymous.Anon2115336743":{"fields":{"displayText":{"type":"string"},"toggledDisplayText":{"type":"string"},"enabled":{"type":"boolean"}},"baseType":"com.linkedin.voyager.dash.organization.OrganizationProductAddSkillCallToAction"},"com.linkedin.deco.recipe.anonymous.Anon336652209":{"fields":{"profilePicture":{"type":"com.linkedin.voyager.dash.deco.identity.profile.ProfilePhotoDecoSpecReadOnlyProfilePicture"},"memorialized":{"type":"boolean"},"lastName":{"type":"string"},"firstName":{"type":"string"},"tempStatus":{"type":"string"},"entityUrn":{"type":"string"},"tempStatusEmoji":{"type":"string"},"publicIdentifier":{"type":"string"},"headline":{"type":"string"},"tempStatusExpiredAtUnion":{"type":{"union":{"customizedExpiredAt":"long","standardizedExpiration":"string"}}}},"baseType":"com.linkedin.voyager.dash.identity.profile.Profile"},"com.linkedin.voyager.dash.deco.common.VectorArtifact":{"fields":{"width":{"type":"int"},"fileIdentifyingUrlPathSegment":{"type":"string"},"expiresAt":{"type":"long"},"height":{"type":"int"}},"baseType":"com.linkedin.common.VectorArtifact"},"com.linkedin.voyager.dash.deco.common.text.School":{"fields":{"name":{"type":"string"},"entityUrn":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.organization.School"},"com.linkedin.deco.recipe.anonymous.Anon2066892616":{"fields":{"invitationType":{"type":"string"},"invitationId":{"type":"long"},"sharedSecret":{"type":"string"},"entityUrn":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.relationships.invitation.Invitation"},"com.linkedin.deco.recipe.anonymous.Anon1350307109":{"fields":{"insightViewee":{"type":"com.linkedin.deco.recipe.anonymous.Anon798864187","resolvedFrom":"insightVieweeUrn"},"insightVieweeUrn":{"type":"string"},"entityUrn":{"type":"string"},"sharedConnectionDetailTargetResolutionResult":{"type":"com.linkedin.deco.recipe.anonymous.Anon798864187","resolvedFrom":"sharedConnectionDetailTarget"},"insightImage":{"type":"com.linkedin.voyager.dash.deco.common.image.ImageViewModel"},"sharedConnectionDetailTarget":{"type":"string"},"text":{"type":"com.linkedin.voyager.dash.deco.common.text.TextViewModelV2"},"navigationUrl":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.relationships.Insight"},"com.linkedin.voyager.dash.deco.common.forms.NumericValidationMetadata":{"fields":{"numericValueRangeValidation":{"type":"com.linkedin.deco.recipe.anonymous.Anon600254320"}},"baseType":"com.linkedin.voyager.dash.common.forms.validation.NumericValidationMetadata"},"com.linkedin.voyager.dash.deco.launchpad.PageKey":{"fields":{"pageKey":{"type":"string"},"anchorPage":{"type":"boolean"}},"baseType":"com.linkedin.voyager.dash.common.tracking.PageKey"},"com.linkedin.voyager.dash.deco.common.forms.ReorderableListFormComponent":{"fields":{"contextualSuggestionsTriggers":{"type":"com.linkedin.deco.recipe.anonymous.Anon2065966944"},"contextualRangeValidations":{"type":{"array":"com.linkedin.deco.recipe.anonymous.Anon1117966179"}},"suggestionUseCase":{"type":"string"},"contextualSuggestionViewModel":{"type":"com.linkedin.deco.recipe.anonymous.Anon1976515688"},"selectionCountRangeValidation":{"type":"com.linkedin.voyager.dash.deco.common.forms.SelectionCountRangeValidation"},"typeaheadCta":{"type":"com.linkedin.voyager.dash.deco.common.forms.TypeaheadCta"},"reorderableItemListUnion":{"type":{"union":{"textItems":{"array":"com.linkedin.voyager.dash.deco.common.forms.TextSelectableOption"}}}},"description":{"type":"com.linkedin.voyager.dash.deco.common.text.TextViewModelV2"},"numberOfInitialItems":{"type":"int"},"trackingId":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.common.forms.ReorderableListFormComponent"},"com.linkedin.deco.recipe.anonymous.Anon430310092":{"fields":{"entityUrn":{"type":"string"},"creator":{"type":"boolean"}},"baseType":"com.linkedin.voyager.dash.identity.profile.Profile"},"com.linkedin.deco.recipe.anonymous.Anon1586568086":{"fields":{"url":{"type":"string"},"text":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.common.Link"},"com.linkedin.deco.recipe.anonymous.Anon1976702492":{"fields":{"iweWarned":{"type":"boolean"},"entityUrn":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.identity.profile.Profile"},"com.linkedin.voyager.dash.deco.launchpad.ShareBoxAction":{"fields":{"shareMessage":{"type":"com.linkedin.voyager.dash.deco.common.text.TextViewModelV2"}},"baseType":"com.linkedin.voyager.dash.launchpad.actionrecommendation.ShareBoxAction"},"com.linkedin.deco.recipe.anonymous.Anon1439603308":{"fields":{"elements":{"type":{"array":"com.linkedin.deco.recipe.anonymous.Anon1122468175"}}},"baseType":"com.linkedin.restli.common.CollectionResponse"},"com.linkedin.deco.recipe.anonymous.Anon1898728769":{"fields":{"countryUrnRequired":{"type":"boolean"},"locationField":{"type":"com.linkedin.voyager.dash.deco.common.forms.SingleTypeaheadEntityFormComponent"},"requiredFieldMissingErrorText":{"type":"com.linkedin.voyager.dash.deco.common.text.TextViewModelV2"},"responseRequired":{"type":"boolean"},"labelText":{"type":"com.linkedin.voyager.dash.deco.common.text.TextViewModelV2"}},"baseType":"com.linkedin.voyager.dash.common.forms.LocationTypeaheadEntityField"},"com.linkedin.deco.recipe.anonymous.Anon1735278252":{"fields":{"controlName":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.common.forms.StarRatingFormComponent"},"com.linkedin.deco.recipe.anonymous.Anon437558093":{"fields":{"profilePicture":{"type":"com.linkedin.voyager.dash.deco.identity.profile.ProfilePhotoDecoSpecReadOnlyProfilePicture"},"memorialized":{"type":"boolean"},"lastName":{"type":"string"},"firstName":{"type":"string"},"tempStatus":{"type":"string"},"entityUrn":{"type":"string"},"memberRelationship":{"type":"com.linkedin.deco.recipe.anonymous.Anon1069349607","isInjection":true},"tempStatusEmoji":{"type":"string"},"publicIdentifier":{"type":"string"},"headline":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.identity.profile.Profile"},"com.linkedin.deco.recipe.anonymous.Anon1507465610":{"fields":{"type":{"type":"string"},"textOption":{"type":"string"},"selected":{"type":"boolean"}},"baseType":"com.linkedin.voyager.dash.sponsoredcontent.leadgen.MultipleChoiceQuestionOptions"},"com.linkedin.deco.recipe.anonymous.Anon409680083":{"fields":{"type":{"type":"string"},"index":{"type":"int"}},"baseType":"com.linkedin.voyager.dash.common.text.ListItemStyle"},"com.linkedin.deco.recipe.anonymous.Anon1792705474":{"fields":{"invitationUnion":{"type":{"union":{"noInvitation":"com.linkedin.deco.recipe.anonymous.Anon881636240","invitation":"com.linkedin.deco.recipe.anonymous.Anon2066892616"}}}},"baseType":"com.linkedin.voyager.dash.relationships.NoConnection"},"com.linkedin.deco.recipe.anonymous.Anon1369245809":{"fields":{"name":{"type":"string"},"entityUrn":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.identity.profile.StandardizedTitle"},"com.linkedin.deco.recipe.anonymous.Anon412783727":{"fields":{"targetInviteeResolutionResult":{"type":"com.linkedin.deco.recipe.anonymous.Anon352881287","resolvedFrom":"targetInvitee"},"inviter":{"type":"string"},"targetInvitee":{"type":"string"},"inviterResolutionResult":{"type":"com.linkedin.deco.recipe.anonymous.Anon1976702492","resolvedFrom":"inviter"}},"baseType":"com.linkedin.voyager.dash.relationships.invitation.NoInvitation"},"com.linkedin.deco.recipe.anonymous.Anon533924488":{"fields":{"insightImage":{"type":"com.linkedin.voyager.dash.deco.common.image.ImageViewModel"},"sharedConnectionDetailTarget":{"type":"string"},"text":{"type":"com.linkedin.voyager.dash.deco.common.text.TextViewModelV2"},"navigationUrl":{"type":"string"},"entityUrn":{"type":"string"},"sharedConnectionDetailTargetResolutionResult":{"type":"com.linkedin.deco.recipe.anonymous.Anon587401631","resolvedFrom":"sharedConnectionDetailTarget"}},"baseType":"com.linkedin.voyager.dash.relationships.Insight"},"com.linkedin.deco.recipe.anonymous.Anon918376989":{"fields":{"title":{"type":"string"},"invitationRelationshipOptions":{"type":{"array":"com.linkedin.deco.recipe.anonymous.Anon934606613"}},"subtitle":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.relationships.invitation.InvitationRelationshipForm"},"com.linkedin.deco.recipe.anonymous.Anon1308136603":{"fields":{"questions":{"type":{"array":"com.linkedin.deco.recipe.anonymous.Anon407602795"}}},"baseType":"com.linkedin.voyager.dash.sponsoredcontent.leadgen.LeadGenFormSection"},"com.linkedin.voyager.dash.deco.common.Locale":{"fields":{"variant":{"type":"string"},"country":{"type":"string"},"language":{"type":"string"}},"baseType":"com.linkedin.common.Locale"},"com.linkedin.voyager.dash.deco.common.forms.ActionableSuggestedEntity":{"fields":{"suggestedEntity":{"type":"com.linkedin.voyager.dash.deco.common.forms.TextSelectableOption"},"navigationUrl":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.common.forms.ActionableSuggestedEntity"},"com.linkedin.voyager.dash.deco.common.VectorImageOnlyRootUrl":{"fields":{"rootUrl":{"type":"string"},"artifacts":{"type":{"array":"com.linkedin.voyager.dash.deco.common.VectorArtifact"}}},"baseType":"com.linkedin.common.VectorImage"},"com.linkedin.voyager.dash.deco.common.video.Resolution":{"fields":{"width":{"type":"int"},"height":{"type":"int"}},"baseType":"com.linkedin.videocontent.Resolution"},"com.linkedin.deco.recipe.anonymous.Anon853977300":{"fields":{"entityUrn":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.relationships.Insight"},"com.linkedin.voyager.dash.deco.common.text.ProfileForFamiliarName":{"fields":{"lastName":{"type":"string"},"firstName":{"type":"string"},"entityUrn":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.identity.profile.Profile"},"com.linkedin.deco.recipe.anonymous.Anon1976515688":{"fields":{"suggestedEntities":{"type":{"array":"com.linkedin.voyager.dash.deco.common.forms.TextSelectableOption"}},"subtitle":{"type":"com.linkedin.voyager.dash.deco.common.text.TextViewModelV2"},"dismissSelectableOption":{"type":"com.linkedin.voyager.dash.deco.common.forms.TextSelectableOption"},"dismissControlName":{"type":"string"},"actionableSuggestedEntities":{"type":{"array":"com.linkedin.voyager.dash.deco.common.forms.ActionableSuggestedEntity"}},"title":{"type":"com.linkedin.voyager.dash.deco.common.text.TextViewModelV2"},"typeaheadFormSuggestionUseCase":{"type":"string"},"navigationButton":{"type":"com.linkedin.deco.recipe.anonymous.Anon1957375702"},"trackingId":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.common.forms.TypeaheadFormSuggestionViewModel"},"com.linkedin.voyager.dash.deco.organization.Profile":{"fields":{"profilePicture":{"type":"com.linkedin.voyager.dash.deco.identity.profile.ProfilePhotoDecoSpecReadOnlyProfilePicture"},"lastName":{"type":"string"},"firstName":{"type":"string"},"publicIdentifier":{"type":"string"},"headline":{"type":"string"},"entityUrn":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.identity.profile.Profile"},"com.linkedin.deco.recipe.anonymous.Anon1525160909":{"fields":{"width":{"type":"int"},"imageUrls":{"type":{"array":"string"}},"height":{"type":"int"}},"baseType":"com.linkedin.documentcontent.DocumentResolutionPages"},"com.linkedin.voyager.dash.deco.common.Link":{"fields":{"type":{"type":"string"},"rel":{"type":"string"},"href":{"type":"string"}},"baseType":"com.linkedin.restli.common.Link"},"com.linkedin.voyager.dash.deco.common.ux.baseInsightViewModel":{"fields":{"image":{"type":"com.linkedin.voyager.dash.deco.common.image.ImageViewModel"},"text":{"type":"com.linkedin.voyager.dash.deco.common.text.TextViewModelV2"}},"baseType":"com.linkedin.voyager.dash.common.ux.InsightViewModel"},"com.linkedin.voyager.dash.deco.common.image.School":{"fields":{"logo":{"type":{"union":{"url":"string","vectorImage":"com.linkedin.voyager.dash.deco.common.VectorImageOnlyRootUrlAndAttribution"}}},"entityUrn":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.organization.School"},"com.linkedin.voyager.dash.deco.common.VectorImage":{"fields":{"attribution":{"type":"string"},"digitalmediaAsset":{"type":"string"},"rootUrl":{"type":"string"},"artifacts":{"type":{"array":"com.linkedin.voyager.dash.deco.common.VectorArtifact"}}},"baseType":"com.linkedin.common.VectorImage"},"com.linkedin.deco.recipe.anonymous.Anon881636240":{"fields":{"inviter":{"type":"string"},"inviterResolutionResult":{"type":"com.linkedin.deco.recipe.anonymous.Anon587401631","resolvedFrom":"inviter"}},"baseType":"com.linkedin.voyager.dash.relationships.invitation.NoInvitation"},"com.linkedin.voyager.dash.deco.relationships.Connection":{"fields":{"createdAt":{"type":"long"},"connectedMemberResolutionResult":{"type":"com.linkedin.deco.recipe.anonymous.Anon336652209","resolvedFrom":"connectedMember"},"connectedMember":{"type":"string"},"entityUrn":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.relationships.Connection"},"com.linkedin.voyager.dash.deco.organization.MiniCompany":{"fields":{"name":{"type":"string"},"logo":{"type":{"union":{"url":"string","vectorImage":"com.linkedin.voyager.dash.deco.common.VectorImage"}}},"entityUrn":{"type":"string"},"url":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.organization.Company"},"com.linkedin.deco.recipe.anonymous.Anon198476569":{"fields":{"viewer":{"type":"com.linkedin.deco.recipe.anonymous.Anon430310092","resolvedFrom":"viewerUrn"},"vieweeFollowingViewer":{"type":"boolean"},"viewerUrn":{"type":"string"},"viewee":{"type":"com.linkedin.voyager.dash.deco.relationships.profile","resolvedFrom":"vieweeUrn"},"vieweeUrn":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.relationships.MemberToMemberFollowMetadata"},"com.linkedin.deco.recipe.anonymous.Anon1703899306":{"fields":{"profilePicture":{"type":"com.linkedin.voyager.dash.deco.identity.profile.ProfilePhotoDecoSpecReadOnlyProfilePicture"},"lastName":{"type":"string"},"firstName":{"type":"string"},"profileProfileActions":{"type":"com.linkedin.voyager.dash.deco.identity.profile.ProfileActionsInjection","isInjection":true},"entityUrn":{"type":"string"},"memberRelationship":{"type":"com.linkedin.voyager.dash.deco.relationships.MemberRelationshipV2ForInjection","isInjection":true},"publicIdentifier":{"type":"string"},"headline":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.identity.profile.Profile"},"com.linkedin.deco.recipe.anonymous.Anon2003298771":{"fields":{"nameSupplementaryInfo":{"type":"com.linkedin.voyager.dash.deco.common.text.TextViewModelV2"},"footerText":{"type":"com.linkedin.voyager.dash.deco.common.text.TextViewModelV2"},"imageUnion":{"type":{"union":{"url":"string","vectorImage":"com.linkedin.voyager.dash.deco.common.VectorImageOnlyRootUrlAndAttribution"}}},"subHeadline":{"type":"com.linkedin.voyager.dash.deco.common.text.TextViewModelV2"},"name":{"type":"com.linkedin.voyager.dash.deco.common.text.TextViewModelV2"},"insightText":{"type":"com.linkedin.voyager.dash.deco.common.text.TextViewModelV2"},"headline":{"type":"com.linkedin.voyager.dash.deco.common.text.TextViewModelV2"},"backgroundImageUnion":{"type":{"union":{"url":"string","vectorImage":"com.linkedin.voyager.dash.deco.common.VectorImageOnlyRootUrlAndAttribution"}}}},"baseType":"com.linkedin.voyager.dash.common.media.StickerLinkLargeTemplateView"},"com.linkedin.voyager.dash.deco.feed.FollowingState":{"fields":{"followeeCount":{"type":"long"},"entityUrn":{"type":"string"},"trackingUrn":{"type":"string"},"preDashFollowingInfoUrn":{"type":"string"},"following":{"type":"boolean"},"followingType":{"type":"string"},"followerCount":{"type":"long"}},"baseType":"com.linkedin.voyager.dash.feed.FollowingState"},"com.linkedin.deco.recipe.anonymous.Anon1914510571":{"fields":{"paging":{"type":"com.linkedin.voyager.dash.deco.common.FullPaging"},"elements":{"type":{"array":"com.linkedin.deco.recipe.anonymous.Anon1956213100"}}},"baseType":"com.linkedin.restli.common.CollectionResponse"},"com.linkedin.voyager.dash.deco.relationships.DirectionalEntityRelationship":{"fields":{"invitationUrn":{"type":"string"},"entityUrn":{"type":"string"},"invitation":{"type":"com.linkedin.voyager.dash.deco.relationships.InvitationRaw","resolvedFrom":"invitationUrn"},"relationshipDataUnion":{"type":{"union":{"follow":"com.linkedin.deco.recipe.anonymous.Anon191738356","subscribe":"com.linkedin.deco.recipe.anonymous.Anon40833127"}}}},"baseType":"com.linkedin.voyager.dash.relationships.DirectionalEntityRelationship"},"com.linkedin.voyager.dash.deco.common.forms.EntitySelectableOption":{"fields":{"optionUrn":{"type":"string"},"entityImage":{"type":"com.linkedin.voyager.dash.deco.common.image.ImageViewModel"},"dismissable":{"type":"boolean"},"shouldRenderTextBeforeImage":{"type":"boolean"},"subtitle":{"type":"com.linkedin.voyager.dash.deco.common.text.TextViewModelV2"},"optionUnion":{"type":{"union":{"optionUrn":"string","optionEnumString":"string"}}},"controlName":{"type":"string"},"title":{"type":"com.linkedin.voyager.dash.deco.common.text.TextViewModelV2"}},"baseType":"com.linkedin.voyager.dash.common.forms.EntitySelectableOption"},"com.linkedin.deco.recipe.anonymous.Anon1568806612":{"fields":{"school":{"type":"com.linkedin.deco.recipe.anonymous.Anon929195650","resolvedFrom":"schoolUrn"},"vectorImage":{"type":"com.linkedin.voyager.dash.deco.common.VectorImageOnlyRootUrlAndAttribution"},"schoolUrn":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.common.image.NonEntitySchoolLogo"},"com.linkedin.deco.recipe.anonymous.Anon1449981882":{"fields":{"paging":{"type":"com.linkedin.voyager.dash.deco.common.FullPaging"},"elements":{"type":{"array":"com.linkedin.deco.recipe.anonymous.Anon1566936928"}}},"baseType":"com.linkedin.restli.common.CollectionResponse"},"com.linkedin.deco.recipe.anonymous.Anon660720507":{"fields":{"name":{"type":"string"},"entityUrn":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.organization.Company"},"com.linkedin.voyager.dash.deco.common.forms.TextEntityListFormComponent":{"fields":{"controlName":{"type":"string"},"pageKey":{"type":"com.linkedin.voyager.dash.deco.common.forms.PageKey"},"placeHolderText":{"type":"com.linkedin.voyager.dash.deco.common.text.TextViewModelV2"},"textSelectableOptions":{"type":{"array":"com.linkedin.voyager.dash.deco.common.forms.TextSelectableOption"}},"helperText":{"type":"com.linkedin.voyager.dash.deco.common.text.TextViewModelV2"},"preferredRenderingStyle":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.common.forms.TextEntityListFormComponent"},"com.linkedin.voyager.dash.deco.relationships.ProfileWithIweWarned":{"fields":{"memorialized":{"type":"boolean"},"lastName":{"type":"string"},"firstName":{"type":"string"},"tempStatus":{"type":"string"},"entityUrn":{"type":"string"},"tempStatusEmoji":{"type":"string"},"iweWarned":{"type":"boolean"},"publicIdentifier":{"type":"string"},"headline":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.identity.profile.Profile"},"com.linkedin.voyager.dash.deco.common.image.ImageViewModel":{"fields":{"attributes":{"type":{"array":"com.linkedin.voyager.dash.deco.common.image.ImageAttribute"}},"actionTarget":{"type":"string"},"totalCount":{"type":"int"},"accessibilityTextAttributes":{"type":{"array":"com.linkedin.voyager.dash.deco.common.text.TextAttributeV2"}},"accessibilityText":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.common.image.ImageViewModel"},"com.linkedin.deco.recipe.anonymous.Anon1957375702":{"fields":{"icon":{"type":"string"},"controlName":{"type":"string"},"text":{"type":"string"},"openExternally":{"type":"boolean"},"navigationUrl":{"type":"string"},"appearance":{"type":"com.linkedin.voyager.dash.deco.common.ux.ButtonAppearance"}},"baseType":"com.linkedin.voyager.dash.common.forms.NavigationButton"},"com.linkedin.deco.recipe.anonymous.Anon648914460":{"fields":{"companyUrn":{"type":"string"},"company":{"type":"com.linkedin.deco.recipe.anonymous.Anon1927941263","resolvedFrom":"companyUrn"},"vectorImage":{"type":"com.linkedin.voyager.dash.deco.common.VectorImageOnlyRootUrlAndAttribution"}},"baseType":"com.linkedin.voyager.dash.common.image.NonEntityCompanyLogo"},"com.linkedin.deco.recipe.anonymous.Anon1042323916":{"fields":{"entityUrn":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.jobs.JobPosting"},"com.linkedin.voyager.dash.deco.common.text.Hashtag":{"fields":{"entityUrn":{"type":"string"},"trackingUrn":{"type":"string"},"actionTarget":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.feed.Hashtag"},"com.linkedin.voyager.dash.deco.common.FullPaging":{"fields":{"start":{"type":"int"},"count":{"type":"int"},"total":{"type":"int"},"links":{"type":{"array":"com.linkedin.voyager.dash.deco.common.Link"}}},"baseType":"com.linkedin.restli.common.CollectionMetadata"},"com.linkedin.deco.recipe.anonymous.Anon1122468175":{"fields":{"insightImage":{"type":"com.linkedin.voyager.dash.deco.common.image.ImageViewModel"},"text":{"type":"com.linkedin.voyager.dash.deco.common.text.TextViewModelV2"},"navigationUrl":{"type":"string"},"entityUrn":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.relationships.Insight"},"com.linkedin.voyager.dash.deco.launchpad.JobSearchQuery":{"fields":{"geo":{"type":"com.linkedin.deco.recipe.anonymous.Anon1271139837","resolvedFrom":"geoUrn"},"keywords":{"type":"string"},"geoUrn":{"type":"string"},"savedSearchId":{"type":"long"}},"baseType":"com.linkedin.voyager.dash.launchpad.attribute.JobSearchQuery"},"com.linkedin.voyager.dash.deco.common.image.ProfessionalEvent":{"fields":{"logoImage":{"type":{"union":{"url":"string","vectorImage":"com.linkedin.voyager.dash.deco.common.VectorImageOnlyRootUrlAndAttribution"}}},"entityUrn":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.events.ProfessionalEvent"},"com.linkedin.voyager.dash.deco.common.video.Thumbnail":{"fields":{"resolution":{"type":"com.linkedin.voyager.dash.deco.common.video.Resolution"},"url":{"type":"string"}},"baseType":"com.linkedin.videocontent.Thumbnail"},"com.linkedin.voyager.dash.deco.common.DateRange":{"fields":{"start":{"type":"com.linkedin.voyager.dash.deco.common.Date"},"end":{"type":"com.linkedin.voyager.dash.deco.common.Date"}},"baseType":"com.linkedin.common.DateRange"},"com.linkedin.deco.recipe.anonymous.Anon301818032":{"fields":{"entityImage":{"type":"com.linkedin.voyager.dash.deco.common.image.ImageViewModel"},"backgroundImage":{"type":"com.linkedin.voyager.dash.deco.common.image.ImageViewModel"},"subtitle":{"type":"com.linkedin.voyager.dash.deco.common.text.TextViewModelV2"},"subDescription":{"type":"com.linkedin.voyager.dash.deco.common.text.TextViewModelV2"},"description":{"type":"com.linkedin.voyager.dash.deco.common.text.TextViewModelV2"},"contentHorizontallyCentered":{"type":"boolean"},"title":{"type":"com.linkedin.voyager.dash.deco.common.text.TextViewModelV2"}},"baseType":"com.linkedin.voyager.dash.sponsoredcontent.leadgen.LeadGenBannerComponent"},"com.linkedin.voyager.dash.deco.organization.EmbeddedVideoDecoSpec":{"fields":{"url":{"type":"string"},"thumbnail":{"type":"com.linkedin.voyager.dash.deco.common.VectorImage"}},"baseType":"com.linkedin.voyager.dash.organization.EmbeddedVideo"},"com.linkedin.deco.recipe.anonymous.Anon1670704461":{"fields":{"title":{"type":"string"},"subTitle":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.launchpad.LaunchpadDismissDialog"},"com.linkedin.voyager.dash.deco.common.Date":{"fields":{"month":{"type":"int"},"year":{"type":"int"},"day":{"type":"int"}},"baseType":"com.linkedin.common.Date"},"com.linkedin.voyager.dash.deco.common.video.ProgressiveDownloadMetadata":{"fields":{"streamingLocations":{"type":{"array":"com.linkedin.voyager.dash.deco.common.video.StreamingLocation"}},"size":{"type":"long"},"bitRate":{"type":"int"},"width":{"type":"int"},"mediaType":{"type":"string"},"mimeType":{"type":"string"},"height":{"type":"int"}},"baseType":"com.linkedin.videocontent.ProgressiveDownloadMetadata"},"com.linkedin.deco.recipe.anonymous.Anon1196575529":{"fields":{"entityUrn":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.organization.OrganizationalPage"},"com.linkedin.voyager.dash.deco.organization.OrganizationRoleType":{"fields":{"localizedName":{"type":"string"},"role":{"type":"string"},"paidMediaRole":{"type":"boolean"},"localizedDescription":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.organization.OrganizationRoleType"},"com.linkedin.deco.recipe.anonymous.Anon906563573":{"fields":{"characterCountRangeValidation":{"type":"com.linkedin.voyager.dash.deco.common.forms.CharacterCountRangeValidation"},"invalidUrlErrorText":{"type":"com.linkedin.voyager.dash.deco.common.text.TextViewModelV2"}},"baseType":"com.linkedin.voyager.dash.common.forms.validation.UrlValidationMetadata"},"com.linkedin.voyager.dash.deco.common.IntegerRange":{"fields":{"start":{"type":"int"},"end":{"type":"int"}},"baseType":"com.linkedin.common.IntegerRange"},"com.linkedin.voyager.dash.deco.common.text.TextViewModelV2":{"fields":{"textDirection":{"type":"string"},"text":{"type":"string"},"attributesV2":{"type":{"array":"com.linkedin.voyager.dash.deco.common.text.TextAttributeV2"}},"accessibilityTextAttributesV2":{"type":{"array":"com.linkedin.voyager.dash.deco.common.text.TextAttributeV2"}},"accessibilityText":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.common.text.TextViewModel"},"com.linkedin.deco.recipe.anonymous.Anon1789236903":{"fields":{"title":{"type":"string"},"entityUrn":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.publishing.ContentSeries"},"com.linkedin.voyager.dash.deco.relationships.MemberRelationshipV2":{"fields":{"memberRelationshipData":{"type":{"union":{"noInvitation":"com.linkedin.deco.recipe.anonymous.Anon1031101697","invitation":"com.linkedin.voyager.dash.deco.relationships.Invitation","connection":"com.linkedin.voyager.dash.deco.relationships.Connection"}}},"entityUrn":{"type":"string"},"memberRelationshipUnion":{"type":{"union":{"self":"com.linkedin.restli.common.EmptyRecord","connection":"com.linkedin.voyager.dash.deco.relationships.Connection","noConnection":"com.linkedin.voyager.dash.deco.relationships.NoConnection"}}}},"baseType":"com.linkedin.voyager.dash.relationships.MemberRelationship"},"com.linkedin.voyager.dash.deco.sponsoredcontent.LeadGenForm":{"fields":{"formSections":{"type":{"array":"com.linkedin.deco.recipe.anonymous.Anon1308136603"}},"submitted":{"type":"boolean"},"entityUrn":{"type":"string"},"privacyPolicy":{"type":"com.linkedin.voyager.dash.deco.common.text.TextViewModelV2"},"postSubmissionContent":{"type":"com.linkedin.deco.recipe.anonymous.Anon561948584"},"lastSubmittedAt":{"type":"long"},"trackingMetadata":{"type":"string"},"header":{"type":"com.linkedin.voyager.dash.deco.common.text.TextViewModelV2"},"banner":{"type":"com.linkedin.deco.recipe.anonymous.Anon301818032"},"submitButtonText":{"type":"string"},"customPrivacyPolicy":{"type":"com.linkedin.voyager.dash.deco.common.text.TextViewModelV2"},"consentSection":{"type":"com.linkedin.deco.recipe.anonymous.Anon1308136603"}},"baseType":"com.linkedin.voyager.dash.sponsoredcontent.leadgen.LeadGenForm"},"com.linkedin.voyager.dash.deco.relationships.ProfileWithEmailRequired":{"fields":{"memorialized":{"type":"boolean"},"lastName":{"type":"string"},"firstName":{"type":"string"},"tempStatus":{"type":"string"},"entityUrn":{"type":"string"},"tempStatusEmoji":{"type":"string"},"publicIdentifier":{"type":"string"},"headline":{"type":"string"},"emailRequired":{"type":"boolean"}},"baseType":"com.linkedin.voyager.dash.identity.profile.Profile"},"com.linkedin.voyager.dash.deco.common.forms.FormElementInput":{"fields":{"evaluationType":{"type":"string"},"formElementInputValues":{"type":{"array":{"union":{"entityInputValue":"com.linkedin.voyager.dash.deco.common.forms.EntityInput","locationInputValue":"com.linkedin.deco.recipe.anonymous.Anon2086023182","integerInputValue":"int","urnInputValue":"string","dateRangeInputValue":"com.linkedin.voyager.dash.deco.common.DateRange","textInputValue":"string","floatInputValue":"float","booleanInputValue":"boolean"}}}},"formElementUrn":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.common.forms.FormElementInput"},"com.linkedin.deco.recipe.anonymous.Anon1975396536":{"fields":{"saved":{"type":"boolean"},"salesListNavigationUrl":{"type":"string"},"entityUrn":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.identity.profile.actions.SalesNavigatorSaveState"},"com.linkedin.deco.recipe.anonymous.Anon929195650":{"fields":{"entityUrn":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.organization.School"},"com.linkedin.voyager.dash.deco.common.forms.CheckboxFormComponent":{"fields":{"textSelectableOptions":{"type":{"array":"com.linkedin.voyager.dash.deco.common.forms.TextSelectableOption"}},"selectionCountRangeValidation":{"type":"com.linkedin.voyager.dash.deco.common.forms.SelectionCountRangeValidation"},"exclusiveSelectableOption":{"type":"com.linkedin.voyager.dash.deco.common.forms.TextSelectableOption"},"subtitle":{"type":"com.linkedin.voyager.dash.deco.common.text.TextViewModelV2"}},"baseType":"com.linkedin.voyager.dash.common.forms.CheckboxFormComponent"},"com.linkedin.voyager.dash.deco.common.forms.PillFormComponent":{"fields":{"cta":{"type":{"union":{"typeaheadCta":"com.linkedin.voyager.dash.deco.common.forms.TypeaheadCta","navigationCta":"com.linkedin.deco.recipe.anonymous.Anon1957375702"}}},"contextualSuggestionsTriggers":{"type":"com.linkedin.deco.recipe.anonymous.Anon2065966944"},"contextualRangeValidations":{"type":{"array":"com.linkedin.deco.recipe.anonymous.Anon1117966179"}},"pillsUnion":{"type":{"union":{"choicePills":{"array":"com.linkedin.voyager.dash.deco.common.forms.ChoicePill"},"dismissPills":{"array":"com.linkedin.voyager.dash.deco.common.forms.DismissPill"},"togglePills":{"array":"com.linkedin.voyager.dash.deco.common.forms.TogglePill"}}}},"contextualSuggestionViewModel":{"type":"com.linkedin.deco.recipe.anonymous.Anon1976515688"},"selectionCountRangeValidation":{"type":"com.linkedin.voyager.dash.deco.common.forms.SelectionCountRangeValidation"},"typeaheadCta":{"type":"com.linkedin.voyager.dash.deco.common.forms.TypeaheadCta"},"numberOfInitialPills":{"type":"int"},"description":{"type":"com.linkedin.voyager.dash.deco.common.text.TextViewModelV2"},"togglePills":{"type":{"array":"com.linkedin.voyager.dash.deco.common.forms.TogglePill"}},"pillSuggestionUseCase":{"type":"string"},"trackingId":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.common.forms.PillFormComponent"},"com.linkedin.deco.recipe.anonymous.Anon1299693827":{"fields":{"paging":{"type":"com.linkedin.voyager.dash.deco.common.FullPaging"},"elements":{"type":{"array":"com.linkedin.deco.recipe.anonymous.Anon1128782547"}}},"baseType":"com.linkedin.restli.common.CollectionResponse"},"com.linkedin.voyager.dash.deco.common.forms.SelectionCountRangeValidation":{"fields":{"lessThanMinErrorText":{"type":"string"},"errorText":{"type":"com.linkedin.voyager.dash.deco.common.text.TextViewModelV2"},"maxReachedInfoText":{"type":"string"},"maxReachedWithExclusionListInfoText":{"type":"string"},"validRange":{"type":"com.linkedin.voyager.dash.deco.common.IntegerRange"},"maxExceededErrorText":{"type":"string"},"exclusionListUnions":{"type":{"array":{"union":{"entityInputValue":"com.linkedin.voyager.dash.deco.common.forms.EntityInput","locationInputValue":"com.linkedin.deco.recipe.anonymous.Anon2086023182","integerInputValue":"int","urnInputValue":"string","dateRangeInputValue":"com.linkedin.voyager.dash.deco.common.DateRange","textInputValue":"string","floatInputValue":"float","booleanInputValue":"boolean"}}}}},"baseType":"com.linkedin.voyager.dash.common.forms.validation.SelectionCountRangeValidation"},"com.linkedin.deco.recipe.anonymous.Anon352881287":{"fields":{"entityUrn":{"type":"string"},"emailRequired":{"type":"boolean"}},"baseType":"com.linkedin.voyager.dash.identity.profile.Profile"},"com.linkedin.deco.recipe.anonymous.Anon1760283115":{"fields":{"entityUrn":{"type":"string"},"preDashFollowingInfoUrn":{"type":"string"},"following":{"type":"boolean"}},"baseType":"com.linkedin.voyager.dash.feed.FollowingState"},"com.linkedin.deco.recipe.anonymous.Anon690575082":{"fields":{"viewerGroupMembership":{"type":"com.linkedin.deco.recipe.anonymous.Anon632521550","resolvedFrom":"viewerGroupMembershipUrn"},"displayNotificationSubscriptionLevelOptions":{"type":"boolean"},"groupPostNotificationsEdgeSetting":{"type":"com.linkedin.deco.recipe.anonymous.Anon735489647","resolvedFrom":"groupPostNotificationsEdgeSettingUrn"},"groupPostNotificationsEdgeSettingUrn":{"type":"string"},"postApprovalEnabled":{"type":"boolean"},"backendGroupUrn":{"type":"string"},"memberCount":{"type":"int"},"owners":{"type":"com.linkedin.deco.recipe.anonymous.Anon1299693827","isInjection":true},"memberConnectionsFacePile":{"type":"com.linkedin.voyager.dash.deco.common.image.ImageViewModel"},"memberConnections":{"type":"com.linkedin.deco.recipe.anonymous.Anon721197842","isInjection":true},"membersFacePile":{"type":"com.linkedin.voyager.dash.deco.common.image.ImageViewModel"},"directJoinEnabled":{"type":"boolean"},"entityUrn":{"type":"string"},"logo":{"type":{"union":{"url":"string","vectorImage":"com.linkedin.voyager.dash.deco.common.VectorImage"}}},"globalNewPostNotificationSettingOn":{"type":"boolean"},"viewerGroupMembershipUrn":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.groups.Group"},"com.linkedin.voyager.dash.deco.common.text.LearningCourse":{"fields":{"title":{"type":"string"},"entityUrn":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.learning.LearningCourse"},"com.linkedin.deco.recipe.anonymous.Anon50101142":{"fields":{"ringStatus":{"type":"com.linkedin.deco.recipe.anonymous.Anon1320789737"},"profileUrn":{"type":"string"},"vectorImage":{"type":"com.linkedin.voyager.dash.deco.common.VectorImageOnlyRootUrlAndAttribution"},"profile":{"type":"com.linkedin.deco.recipe.anonymous.Anon587401631","resolvedFrom":"profileUrn"}},"baseType":"com.linkedin.voyager.dash.common.image.NonEntityProfilePicture"},"com.linkedin.deco.recipe.anonymous.Anon1566936928":{"fields":{"company":{"type":"com.linkedin.deco.recipe.anonymous.Anon1226238669","resolvedFrom":"companyUrn"},"multiLocaleCompanyName":{"type":{"map":"string"}},"entityUrn":{"type":"string"},"dateRange":{"type":"com.linkedin.voyager.dash.deco.common.DateRange"},"companyUrn":{"type":"string"},"companyName":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.identity.profile.Position"},"com.linkedin.deco.recipe.anonymous.Anon1834093919":{"fields":{"locationList":{"type":"com.linkedin.voyager.dash.deco.common.forms.TextEntityListFormComponent"},"requiredFieldMissingErrorText":{"type":"com.linkedin.voyager.dash.deco.common.text.TextViewModelV2"},"responseRequired":{"type":"boolean"},"labelText":{"type":"com.linkedin.voyager.dash.deco.common.text.TextViewModelV2"}},"baseType":"com.linkedin.voyager.dash.common.forms.LocationTextEntityListField"},"com.linkedin.deco.recipe.anonymous.Anon1122581856":{"fields":{"paging":{"type":"com.linkedin.voyager.dash.deco.common.FullPaging"},"metadata":{"type":"com.linkedin.deco.recipe.anonymous.Anon1585981083"},"elements":{"type":{"array":"com.linkedin.voyager.dash.deco.organization.Profile"}}},"baseType":"com.linkedin.restli.common.CollectionResponse"},"com.linkedin.voyager.dash.deco.common.forms.DateValidation":{"fields":{"errorText":{"type":"com.linkedin.voyager.dash.deco.common.text.TextViewModelV2"},"validDateRange":{"type":"com.linkedin.voyager.dash.deco.common.DateRange"}},"baseType":"com.linkedin.voyager.dash.common.forms.validation.DateValidation"},"com.linkedin.deco.recipe.anonymous.Anon2136767122":{"fields":{"requiredFieldMissingErrorText":{"type":"com.linkedin.voyager.dash.deco.common.text.TextViewModelV2"},"responseRequired":{"type":"boolean"},"textField":{"type":"com.linkedin.voyager.dash.deco.common.forms.SingleLineTextFormComponent"},"labelText":{"type":"com.linkedin.voyager.dash.deco.common.text.TextViewModelV2"}},"baseType":"com.linkedin.voyager.dash.common.forms.SingleLineTextField"},"com.linkedin.voyager.dash.deco.common.image.ImageAttribute":{"fields":{"detailData":{"type":{"union":{"profilePictureWithoutFrame":"com.linkedin.voyager.dash.deco.common.image.ProfileWithoutFrame","profilePicture":"com.linkedin.voyager.dash.deco.common.image.Profile","profilePictureWithRingStatus":"com.linkedin.voyager.dash.deco.common.image.ProfileWithRingStatus","companyLogo":"com.linkedin.voyager.dash.deco.common.image.Company","professionalEventLogo":"com.linkedin.voyager.dash.deco.common.image.ProfessionalEvent","organizationalPageLogo":"com.linkedin.voyager.dash.deco.common.image.OrganizationalPage","schoolLogo":"com.linkedin.voyager.dash.deco.common.image.School","groupLogo":"com.linkedin.voyager.dash.deco.common.image.Group"}},"resolvedFrom":"detailDataUnion"},"tintColor":{"type":"string"},"detailDataUnion":{"type":{"union":{"profilePictureWithoutFrame":"string","profilePictureWithRingStatus":"string","companyLogo":"string","icon":"string","systemImage":"string","nonEntityGroupLogo":"com.linkedin.deco.recipe.anonymous.Anon1436383648","organizationalPageLogo":"string","vectorImage":"com.linkedin.voyager.dash.deco.common.VectorImageOnlyRootUrlAndAttribution","nonEntityProfessionalEventLogo":"com.linkedin.deco.recipe.anonymous.Anon377984030","profilePicture":"string","imageUrl":"com.linkedin.deco.recipe.anonymous.Anon1529112441","professionalEventLogo":"string","nonEntitySchoolLogo":"com.linkedin.deco.recipe.anonymous.Anon1568806612","nonEntityCompanyLogo":"com.linkedin.deco.recipe.anonymous.Anon648914460","schoolLogo":"string","groupLogo":"string","ghostImage":"string","nonEntityProfilePicture":"com.linkedin.deco.recipe.anonymous.Anon50101142"}}},"tapTargets":{"type":{"array":"com.linkedin.voyager.dash.deco.common.TapTargetWithoutEntity"}},"scalingType":{"type":"string"},"displayAspectRatio":{"type":"double"}},"baseType":"com.linkedin.voyager.dash.common.image.ImageAttribute"},"com.linkedin.voyager.dash.deco.common.forms.TypeaheadCta":{"fields":{"controlName":{"type":"string"},"renderingStyle":{"type":"string"},"typeaheadType":{"type":"string"},"typeaheadQueryContext":{"type":{"array":"string"}},"typeaheadMetadata":{"type":"com.linkedin.deco.recipe.anonymous.Anon86528645"},"labelText":{"type":"com.linkedin.voyager.dash.deco.common.text.TextViewModelV2"}},"baseType":"com.linkedin.voyager.dash.common.forms.TypeaheadCta"},"com.linkedin.deco.recipe.anonymous.Anon2053503342":{"fields":{"paging":{"type":"com.linkedin.voyager.dash.deco.common.FullPaging"},"metadata":{"type":"com.linkedin.deco.recipe.anonymous.Anon523036483"},"elements":{"type":{"array":"com.linkedin.deco.recipe.anonymous.Anon1949606970"}}},"baseType":"com.linkedin.restli.common.CollectionResponse"},"com.linkedin.voyager.dash.deco.common.image.ProfileWithRingStatus":{"fields":{"ringStatus":{"type":"com.linkedin.deco.recipe.anonymous.Anon1080314395","isInjection":true},"profilePicture":{"type":"com.linkedin.voyager.dash.deco.common.image.PhotoFilterPicture"},"entityUrn":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.identity.profile.Profile"},"com.linkedin.deco.recipe.anonymous.Anon1149738921":{"fields":{"useCase":{"type":"string"},"premiumUpsellSlotUrn":{"type":"string"},"premiumUpsellSlot":{"type":"com.linkedin.voyager.dash.deco.premium.PremiumUpsellSlotContent","resolvedFrom":"premiumUpsellSlotUrn"}},"baseType":"com.linkedin.voyager.dash.common.forms.AiSuggestionBar"},"com.linkedin.voyager.dash.deco.common.image.PhotoFilterPicture":{"fields":{"displayImageWithFrameReferenceUnion":{"type":{"union":{"url":"string","vectorImage":"com.linkedin.voyager.dash.deco.common.VectorImageOnlyRootUrlAndAttribution"}}},"a11yText":{"type":"string"},"displayImageReference":{"type":{"union":{"url":"string","vectorImage":"com.linkedin.voyager.dash.deco.common.VectorImageOnlyRootUrlAndAttribution"}}}},"baseType":"com.linkedin.voyager.dash.identity.profile.PhotoFilterPicture"},"com.linkedin.voyager.dash.deco.common.text.Company":{"fields":{"name":{"type":"string"},"entityUrn":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.organization.Company"},"com.linkedin.deco.recipe.anonymous.Anon462345189":{"fields":{"controlName":{"type":"string"},"viewingBehavior":{"type":"string"},"url":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.common.text.TextLink"},"com.linkedin.voyager.dash.deco.common.forms.TogglePill":{"fields":{"textSelectableOption":{"type":"com.linkedin.voyager.dash.deco.common.forms.TextSelectableOption"},"positiveToggleIcon":{"type":"com.linkedin.voyager.dash.deco.common.image.ImageViewModel"},"negativeToggleIcon":{"type":"com.linkedin.voyager.dash.deco.common.image.ImageViewModel"}},"baseType":"com.linkedin.voyager.dash.common.forms.TogglePill"},"com.linkedin.deco.recipe.anonymous.Anon795093289":{"fields":{"dismissControlName":{"type":"string"},"title":{"type":"string"},"formElement":{"type":"com.linkedin.deco.recipe.anonymous.Anon517727117"},"subtitle":{"type":"com.linkedin.voyager.dash.deco.common.text.TextViewModelV2"}},"baseType":"com.linkedin.voyager.dash.common.forms.SingleQuestionSubForm"},"com.linkedin.deco.recipe.anonymous.Anon956797500":{"fields":{"profileActionUnion":{"type":{"union":{"personalizedConnect":"string","saveToPdf":"string","shareProfileUrlViaMessage":"string","saveToPdfUrl":"string","composeOption":"string","saveInSalesNavigatorState":"string","statefulAction":"com.linkedin.voyager.dash.deco.common.ux.StatefulButtonModel","viewProfileInExternalContext":"com.linkedin.voyager.dash.deco.identity.profile.ViewProfileInExternalContextAction","statelessAction":"string","followingState":"string","report":"com.linkedin.deco.recipe.anonymous.Anon743120773","ignore":"string","shareProfileUrl":"string","connection":"com.linkedin.voyager.dash.deco.identity.profile.MemberRelationshipWrapper","withdraw":"string"}}},"profileAction":{"type":{"union":{"personalizedConnect":"com.linkedin.deco.recipe.anonymous.Anon4539795","saveToPdf":"com.linkedin.deco.recipe.anonymous.Anon587401631","followingState":"com.linkedin.deco.recipe.anonymous.Anon1760283115","ignore":"com.linkedin.deco.recipe.anonymous.Anon1312950162","composeOption":"com.linkedin.deco.recipe.anonymous.Anon1575907873","withdraw":"com.linkedin.deco.recipe.anonymous.Anon1213132949","saveInSalesNavigatorState":"com.linkedin.deco.recipe.anonymous.Anon1975396536"}},"resolvedFrom":"profileActionUnion"},"trackingId":{"type":"bytes"}},"baseType":"com.linkedin.voyager.dash.identity.profile.actions.ProfileActionInfo"},"com.linkedin.deco.recipe.anonymous.Anon995204871":{"fields":{"viewerPermissions":{"type":"com.linkedin.voyager.dash.deco.organization.OrganizationPermissions"},"industryV2Taxonomy":{"type":{"map":"com.linkedin.voyager.dash.deco.common.IndustryV2"},"resolvedFrom":"industryV2TaxonomyUrns"},"productsAccessible":{"type":"boolean"},"croppedCoverImageUnion":{"type":{"union":{"url":"string","vectorImage":"com.linkedin.voyager.dash.deco.common.VectorImage"}}},"industry":{"type":{"map":"com.linkedin.voyager.dash.deco.common.Industry"},"resolvedFrom":"industryUrns"},"industryV2Urns":{"type":{"array":"string"}},"lcpStaffingOrganization":{"type":"boolean"},"industryUrns":{"type":{"array":"string"}},"url":{"type":"string"},"lcpTreatment":{"type":"boolean"},"industryV2":{"type":{"map":"com.linkedin.voyager.dash.deco.common.Industry"},"resolvedFrom":"industryV2Urns"},"relevantFollowersForViewerAndOrganization":{"type":"com.linkedin.deco.recipe.anonymous.Anon2053503342","isInjection":true},"industryV2TaxonomyUrns":{"type":{"array":"string"}},"followingStateUrn":{"type":"string"},"entityUrn":{"type":"string"},"followingState":{"type":"com.linkedin.voyager.dash.deco.feed.FollowingState","resolvedFrom":"followingStateUrn"},"originalCoverImageUnion":{"type":{"union":{"url":"string","vectorImage":"com.linkedin.voyager.dash.deco.common.VectorImage"}}},"name":{"type":"string"},"lcpCustomer":{"type":"boolean"},"logo":{"type":{"union":{"url":"string","vectorImage":"com.linkedin.voyager.dash.deco.common.VectorImage"}}},"universalName":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.organization.Company"},"com.linkedin.deco.recipe.anonymous.Anon2006189319":{"fields":{"postalCodeTextField":{"type":"com.linkedin.deco.recipe.anonymous.Anon2136767122"},"cityTypeaheadField":{"type":"com.linkedin.deco.recipe.anonymous.Anon1898728769"},"useCurrentLocation":{"type":"com.linkedin.deco.recipe.anonymous.Anon1266426900"},"countryField":{"type":"com.linkedin.deco.recipe.anonymous.Anon1898728769"},"cityTextEntityListField":{"type":"com.linkedin.deco.recipe.anonymous.Anon1834093919"}},"baseType":"com.linkedin.voyager.dash.common.forms.LocationFormComponent"},"com.linkedin.deco.recipe.anonymous.Anon735489647":{"fields":{"entityUrn":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.identity.notifications.edgesetting.EdgeSetting"},"com.linkedin.deco.recipe.anonymous.Anon1927941263":{"fields":{"entityUrn":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.organization.Company"},"com.linkedin.voyager.dash.deco.common.image.ProfileWithoutFrame":{"fields":{"profilePicture":{"type":"com.linkedin.voyager.dash.deco.common.image.PhotoFilterPicture"},"entityUrn":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.identity.profile.Profile"},"com.linkedin.deco.recipe.anonymous.Anon587401631":{"fields":{"entityUrn":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.identity.profile.Profile"},"com.linkedin.voyager.dash.deco.common.FloatRange":{"fields":{"start":{"type":"float"},"end":{"type":"float"}},"baseType":"com.linkedin.common.FloatRange"},"com.linkedin.voyager.dash.deco.common.forms.PostalCodeValidationMetadata":{"fields":{"errorText":{"type":"com.linkedin.voyager.dash.deco.common.text.TextViewModelV2"},"countryFormElementUrn":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.common.forms.validation.PostalCodeValidationMetadata"},"com.linkedin.deco.recipe.anonymous.Anon1619791307":{"fields":{"saveResponseAsWorkEmail":{"type":"boolean"},"maxResponseLength":{"type":"int"},"response":{"type":"string"},"validationType":{"type":"string"},"minResponseLength":{"type":"int"}},"baseType":"com.linkedin.voyager.dash.sponsoredcontent.leadgen.TextInputComponent"},"com.linkedin.deco.recipe.anonymous.Anon1621960601":{"fields":{"name":{"type":"string"},"logo":{"type":{"union":{"url":"string","vectorImage":"com.linkedin.voyager.dash.deco.common.VectorImageOnlyRootUrlAndAttribution"}}},"entityUrn":{"type":"string"},"url":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.organization.School"},"com.linkedin.deco.recipe.anonymous.Anon199392175":{"fields":{"paidInMail":{"type":"boolean"},"recipient":{"type":{"map":"com.linkedin.deco.recipe.anonymous.Anon587401631"},"resolvedFrom":"recipientUrns"},"messageRequestContextUrn":{"type":"string"},"recipientUrns":{"type":{"array":"string"}},"targetUrl":{"type":"string"},"existingConversation":{"type":"com.linkedin.deco.recipe.anonymous.Anon807183080","resolvedFrom":"existingConversationUrn"},"existingConversationUrn":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.messaging.compose.ComposeNavigationContext"},"com.linkedin.deco.recipe.anonymous.Anon1320789737":{"fields":{"ringContentType":{"type":"string"},"actionTarget":{"type":"string"},"preDashEntityUrn":{"type":"string"},"entityUrn":{"type":"string"},"emphasized":{"type":"boolean"}},"baseType":"com.linkedin.voyager.dash.common.image.RingStatus"},"com.linkedin.deco.recipe.anonymous.Anon721197842":{"fields":{"paging":{"type":"com.linkedin.voyager.dash.deco.common.FullPaging"},"elements":{"type":{"array":"com.linkedin.deco.recipe.anonymous.Anon1128782547"}}},"baseType":"com.linkedin.restli.common.CollectionResponse"},"com.linkedin.deco.recipe.anonymous.Anon517727117":{"fields":{"urn":{"type":"string"},"prerequisiteInputEvaluationStrategyUnion":{"type":{"union":{"answered":"string","matchAnyPrerequisiteInput":"com.linkedin.deco.recipe.anonymous.Anon870521084","notMatchAnyPrequisiteInput":"com.linkedin.deco.recipe.anonymous.Anon870521084"}}},"formComponent":{"type":{"union":{"gaiTextFormComponent":"com.linkedin.deco.recipe.anonymous.Anon146637213","multilineTextFormComponent":"com.linkedin.voyager.dash.deco.common.forms.MultilineTextFormComponent","textEntityListFormComponent":"com.linkedin.voyager.dash.deco.common.forms.TextEntityListFormComponent","toggleFormComponent":"com.linkedin.voyager.dash.deco.common.forms.ToggleFormComponent","starRatingFormComponent":"com.linkedin.deco.recipe.anonymous.Anon1735278252","nestedCheckboxFormComponent":"com.linkedin.deco.recipe.anonymous.Anon1048186976","mediaUploadFormComponent":"com.linkedin.deco.recipe.anonymous.Anon2105959233","checkboxFormComponent":"com.linkedin.voyager.dash.deco.common.forms.CheckboxFormComponent","dropdownFormComponent":"com.linkedin.voyager.dash.deco.common.forms.DropdownFormComponent","numberInputFormComponent":"com.linkedin.deco.recipe.anonymous.Anon2083470902","reorderableListFormComponent":"com.linkedin.voyager.dash.deco.common.forms.ReorderableListFormComponent","singleTypeaheadEntityFormComponent":"com.linkedin.voyager.dash.deco.common.forms.SingleTypeaheadEntityFormComponent","dateRangeFormComponent":"com.linkedin.voyager.dash.deco.common.forms.DateRangeFormComponent","dateFormComponent":"com.linkedin.voyager.dash.deco.common.forms.DateFormComponent","multiSelectTypeaheadEntityFormComponent":"com.linkedin.voyager.dash.deco.common.forms.MultiSelectTypeaheadEntityFormComponent","locationFormComponent":"com.linkedin.deco.recipe.anonymous.Anon2006189319","singleLineTextFormComponent":"com.linkedin.voyager.dash.deco.common.forms.SingleLineTextFormComponent","radioButtonFormComponent":"com.linkedin.voyager.dash.deco.common.forms.RadioButtonFormComponent","pillFormComponent":"com.linkedin.voyager.dash.deco.common.forms.PillFormComponent"}}},"input":{"type":"com.linkedin.voyager.dash.deco.common.forms.FormElementInput"},"requiredFieldMissingErrorText":{"type":"com.linkedin.voyager.dash.deco.common.text.TextViewModelV2"},"entityUrn":{"type":"string"},"helperLink":{"type":"com.linkedin.voyager.dash.deco.common.text.TextViewModelV2"},"required":{"type":"boolean"},"shouldAlwaysSendBackFormElementInput":{"type":"boolean"}},"baseType":"com.linkedin.voyager.dash.common.forms.FormElement"},"com.linkedin.deco.recipe.anonymous.Anon783667843":{"fields":{"ctaColor":{"type":"string"},"ctaType":{"type":"string"},"ctaStyle":{"type":"string"},"presentationStyle":{"type":"string"},"shareMessage":{"type":"com.linkedin.voyager.dash.deco.common.text.TextViewModelV2"},"ctaTitle":{"type":"com.linkedin.voyager.dash.deco.common.text.TextViewModelV2"},"deeplinkUrl":{"type":"string"},"ctaActionUnion":{"type":{"union":{"redirect":"com.linkedin.voyager.dash.deco.launchpad.RedirectAction","launchpadApiRequest":"com.linkedin.restli.common.EmptyRecord","launchpadApiRequestAndRedirect":"com.linkedin.voyager.dash.deco.launchpad.RedirectAction","shareBox":"com.linkedin.voyager.dash.deco.launchpad.ShareBoxAction","launchpadFormSubmission":"com.linkedin.restli.common.EmptyRecord"}}},"legoActionCategory":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.launchpad.LaunchpadCta"},"com.linkedin.deco.recipe.anonymous.Anon1085906135":{"fields":{"multiLocaleCompanyName":{"type":{"map":"string"}},"entityUrn":{"type":"string"},"dateRange":{"type":"com.linkedin.voyager.dash.deco.common.DateRange"},"companyName":{"type":"string"},"company":{"type":"com.linkedin.deco.recipe.anonymous.Anon184130218","resolvedFrom":"companyUrn"},"title":{"type":"string"},"companyUrn":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.identity.profile.Position"},"com.linkedin.deco.recipe.anonymous.Anon1963883771":{"fields":{"accentType":{"type":"string"},"image":{"type":"com.linkedin.voyager.dash.deco.common.image.ImageViewModel"}},"baseType":"com.linkedin.voyager.dash.launchpad.LaunchpadBranding"},"com.linkedin.deco.recipe.anonymous.Anon807183080":{"fields":{"entityUrn":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.messaging.Conversation"},"com.linkedin.deco.recipe.anonymous.Anon1353776944":{"fields":{"backgroundImage":{"type":"com.linkedin.voyager.dash.deco.common.image.ImageViewModel"},"cardType":{"type":"string"},"icon":{"type":"com.linkedin.voyager.dash.deco.common.image.ImageViewModel"},"pageKey":{"type":"com.linkedin.voyager.dash.deco.launchpad.PageKey"},"completed":{"type":"boolean"},"legoTrackingToken":{"type":"string"},"cardTitle":{"type":"com.linkedin.voyager.dash.deco.common.text.TextViewModelV2"},"ctas":{"type":{"array":"com.linkedin.deco.recipe.anonymous.Anon783667843"}},"animate":{"type":"boolean"},"videoMetadata":{"type":"com.linkedin.voyager.dash.deco.common.video.VideoPlayMetadata"},"cardLabel":{"type":"com.linkedin.voyager.dash.deco.common.text.TextViewModelV2"},"attributeUnion":{"type":{"union":{"followOrganization":"string","launchpad":"string","jobSearchQuery":"com.linkedin.voyager.dash.deco.launchpad.JobSearchQuery","companyJYMBII":"string","generalJYMBII":{"array":"string"},"viewExperienceEntity":"string","followPostCreator":"string","fifWidgetGroupId":"string","connectWithEntity":"string","entityActivityFeed":"string","edgeBuilding":{"array":"com.linkedin.voyager.dash.deco.launchpad.EdgeBuildingCohortReason"},"jobAlertJYMBII":{"array":"string"},"organizationActivityFeed":"string"}}},"insightImage":{"type":"com.linkedin.voyager.dash.deco.common.image.ImageViewModel"},"formSection":{"type":"com.linkedin.voyager.dash.deco.common.forms.FormSection"},"overlayImage":{"type":"com.linkedin.voyager.dash.deco.common.image.ImageViewModel"},"attribute":{"type":{"union":{"followOrganization":"com.linkedin.voyager.dash.deco.launchpad.OrganizationTopCardForEntityFollow","companyJYMBII":"com.linkedin.deco.recipe.anonymous.Anon184130218","generalJYMBII":{"map":"com.linkedin.deco.recipe.anonymous.Anon1042323916"},"connectWithEntity":"com.linkedin.voyager.dash.deco.launchpad.ProfileTopCardForEntityFollow","viewExperienceEntity":"com.linkedin.voyager.dash.deco.launchpad.ProfileTopCardForEntityFollow","entityActivityFeed":"com.linkedin.deco.recipe.anonymous.Anon587401631","followPostCreator":"com.linkedin.voyager.dash.deco.launchpad.ProfileTopCardForEntityFollow","jobAlertJYMBII":{"map":"com.linkedin.deco.recipe.anonymous.Anon1042323916"},"organizationActivityFeed":"com.linkedin.deco.recipe.anonymous.Anon93322792"}},"resolvedFrom":"attributeUnion"},"cardSubtitle":{"type":"com.linkedin.voyager.dash.deco.common.text.TextViewModelV2"},"cardStyle":{"type":"string"},"entitySize":{"type":"int"}},"baseType":"com.linkedin.voyager.dash.launchpad.LaunchpadCard"},"com.linkedin.deco.recipe.anonymous.Anon570745354":{"fields":{"name":{"type":"string"},"logo":{"type":{"union":{"url":"string","vectorImage":"com.linkedin.voyager.dash.deco.common.VectorImage"}}},"entityUrn":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.organization.Company"},"com.linkedin.deco.recipe.anonymous.Anon346909806":{"fields":{"dashLeadGenFormUrn":{"type":"string"},"privacyPolicyUrl":{"type":"string"},"dashLeadGenForm":{"type":"com.linkedin.voyager.dash.deco.sponsoredcontent.LeadGenForm","resolvedFrom":"dashLeadGenFormUrn"},"organizationProductCallToAction":{"type":"com.linkedin.deco.recipe.anonymous.Anon625674184"},"leadGenFormUrn":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.organization.OrganizationProductLeadGenFormCallToAction"},"com.linkedin.voyager.dash.deco.common.image.Profile":{"fields":{"profilePicture":{"type":"com.linkedin.voyager.dash.deco.common.image.PhotoFilterPicture"},"entityUrn":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.identity.profile.Profile"},"com.linkedin.voyager.dash.deco.common.text.JobPosting":{"fields":{"title":{"type":"string"},"entityUrn":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.jobs.JobPosting"},"com.linkedin.voyager.dash.deco.common.text.TextAttributeV2":{"fields":{"start":{"type":"int"},"length":{"type":"int"},"detailData":{"type":{"union":{"jobPostingName":"com.linkedin.voyager.dash.deco.common.text.JobPosting","profileFamiliarName":"com.linkedin.voyager.dash.deco.common.text.ProfileForFamiliarName","groupName":"com.linkedin.deco.recipe.anonymous.Anon163061530","profileFullName":"com.linkedin.voyager.dash.deco.common.text.ProfileForFullName","learningCourseName":"com.linkedin.voyager.dash.deco.common.text.LearningCourse","companyName":"com.linkedin.voyager.dash.deco.common.text.Company","profileMention":"com.linkedin.voyager.dash.deco.common.text.ProfileForMention","schoolName":"com.linkedin.voyager.dash.deco.common.text.School","organizationalPageName":"com.linkedin.voyager.dash.deco.common.text.OrganizationalPage","hashtag":"com.linkedin.voyager.dash.deco.common.text.Hashtag"}},"resolvedFrom":"detailDataUnion"},"detailDataUnion":{"type":{"union":{"jobPostingName":"string","profileFamiliarName":"string","hyperlink":"string","color":"string","companyName":"string","icon":"string","systemImage":"string","epoch":"com.linkedin.deco.recipe.anonymous.Anon858976209","organizationalPageName":"string","textLink":"com.linkedin.deco.recipe.anonymous.Anon462345189","listItemStyle":"com.linkedin.deco.recipe.anonymous.Anon409680083","hyperlinkOpenExternally":"string","listStyle":"string","groupName":"string","profileFullName":"string","stringFieldReference":"com.linkedin.deco.recipe.anonymous.Anon1251747613","learningCourseName":"string","profileMention":"string","style":"string","schoolName":"string","hashtag":"string"}}}},"baseType":"com.linkedin.voyager.dash.common.text.TextAttribute"},"com.linkedin.deco.recipe.anonymous.Anon146637213":{"fields":{"controlName":{"type":"string"},"hintText":{"type":"string"},"aiSuggestionBar":{"type":"com.linkedin.deco.recipe.anonymous.Anon1149738921"},"numVisibleLines":{"type":"int"},"helperText":{"type":"string"},"validationMetadata":{"type":{"union":{"pronounsV2":"com.linkedin.voyager.dash.deco.common.forms.StringValidationMetadata","string":"com.linkedin.voyager.dash.deco.common.forms.StringValidationMetadata","textFormInputType":"string","postalCode":"com.linkedin.voyager.dash.deco.common.forms.PostalCodeValidationMetadata","name":"com.linkedin.voyager.dash.deco.common.forms.StringValidationMetadata","pronouns":"string","integer":"com.linkedin.voyager.dash.deco.common.forms.NumericValidationMetadata","decimal":"com.linkedin.voyager.dash.deco.common.forms.NumericValidationMetadata","headline":"com.linkedin.voyager.dash.deco.common.forms.StringValidationMetadata","url":"com.linkedin.deco.recipe.anonymous.Anon906563573"}}}},"baseType":"com.linkedin.voyager.dash.common.forms.GaiTextFormComponent"},"com.linkedin.deco.recipe.anonymous.Anon975174582":{"fields":{"paging":{"type":"com.linkedin.voyager.dash.deco.common.FullPaging"},"elements":{"type":{"array":"com.linkedin.deco.recipe.anonymous.Anon1956213100"}}},"baseType":"com.linkedin.restli.common.CollectionResponse"},"com.linkedin.deco.recipe.anonymous.Anon1213132949":{"fields":{"entityUrn":{"type":"string"},"memberRelationshipUnion":{"type":{"union":{"self":"com.linkedin.restli.common.EmptyRecord","connection":"com.linkedin.deco.recipe.anonymous.Anon2134644859","noConnection":"com.linkedin.deco.recipe.anonymous.Anon1843097075"}}}},"baseType":"com.linkedin.voyager.dash.relationships.MemberRelationship"},"com.linkedin.voyager.dash.deco.common.ux.RelationshipActionData":{"fields":{"relationshipsTrackingId":{"type":"bytes"},"relationshipData":{"type":{"union":{"connectionOrInvitation":"com.linkedin.voyager.dash.deco.relationships.MemberRelationshipV2","memberToEntityRelationship":"com.linkedin.voyager.dash.deco.relationships.DirectionalEntityRelationship"}},"resolvedFrom":"relationshipDataUnion"},"relationshipDataUnion":{"type":{"union":{"connectionOrInvitation":"string","memberToEntityRelationship":"string"}}},"moduleKey":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.common.relationships.RelationshipActionData"},"com.linkedin.voyager.dash.deco.identity.profile.MemberRelationshipWrapper":{"fields":{"memberRelationshipUrn":{"type":"string"},"memberRelationship":{"type":"com.linkedin.voyager.dash.deco.relationships.MemberRelationshipV2","resolvedFrom":"memberRelationshipUrn"}},"baseType":"com.linkedin.voyager.dash.relationships.MemberRelationshipWrapper"},"com.linkedin.deco.recipe.anonymous.Anon137428918":{"fields":{"elements":{"type":{"array":"com.linkedin.deco.recipe.anonymous.Anon533924488"}}},"baseType":"com.linkedin.restli.common.CollectionResponse"},"com.linkedin.voyager.dash.deco.common.forms.SingleTypeaheadEntityFormComponent":{"fields":{"typeaheadFormSuggestionViewModel":{"type":"com.linkedin.deco.recipe.anonymous.Anon1976515688"},"controlName":{"type":"string"},"typeaheadType":{"type":"string"},"contextualSuggestionQueryParameterUrns":{"type":{"array":"string"}},"helperText":{"type":"com.linkedin.voyager.dash.deco.common.text.TextViewModelV2"},"hintText":{"type":"com.linkedin.voyager.dash.deco.common.text.TextViewModelV2"},"entityImage":{"type":"com.linkedin.voyager.dash.deco.common.image.ImageViewModel"},"typeaheadQueryContext":{"type":{"array":"string"}},"freeFormTextNotAllowedErrorText":{"type":"com.linkedin.voyager.dash.deco.common.text.TextViewModelV2"},"entityName":{"type":"com.linkedin.voyager.dash.deco.common.text.TextViewModelV2"},"validationMetadataUnion":{"type":{"union":{"pronounsV2":"com.linkedin.voyager.dash.deco.common.forms.StringValidationMetadata","string":"com.linkedin.voyager.dash.deco.common.forms.StringValidationMetadata","textFormInputType":"string","postalCode":"com.linkedin.voyager.dash.deco.common.forms.PostalCodeValidationMetadata","name":"com.linkedin.voyager.dash.deco.common.forms.StringValidationMetadata","pronouns":"string","integer":"com.linkedin.voyager.dash.deco.common.forms.NumericValidationMetadata","decimal":"com.linkedin.voyager.dash.deco.common.forms.NumericValidationMetadata","headline":"com.linkedin.voyager.dash.deco.common.forms.StringValidationMetadata","url":"com.linkedin.deco.recipe.anonymous.Anon906563573"}}},"freeFormTextAllowed":{"type":"boolean"},"typeaheadMetadata":{"type":"com.linkedin.deco.recipe.anonymous.Anon86528645"},"ghostImage":{"type":"com.linkedin.voyager.dash.deco.common.image.ImageViewModel"}},"baseType":"com.linkedin.voyager.dash.common.forms.SingleTypeaheadEntityFormComponent"},"com.linkedin.deco.recipe.anonymous.Anon191738356":{"fields":{"metadataUnion":{"type":{"union":{"memberToMemberFollow":"com.linkedin.deco.recipe.anonymous.Anon198476569","memberToCompanyFollow":"com.linkedin.deco.recipe.anonymous.Anon378816379"}}},"followingStateUrn":{"type":"string"},"followingState":{"type":"com.linkedin.voyager.dash.deco.feed.FollowingState","resolvedFrom":"followingStateUrn"}},"baseType":"com.linkedin.voyager.dash.relationships.FollowRelationship"},"com.linkedin.voyager.dash.deco.launchpad.RedirectAction":{"fields":{"presentationStyle":{"type":"string"},"url":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.launchpad.actionrecommendation.RedirectAction"},"com.linkedin.deco.recipe.anonymous.Anon2116040434":{"fields":{"company":{"type":"com.linkedin.voyager.dash.deco.organization.MiniCompany","resolvedFrom":"companyUrn"},"title":{"type":"string"},"multiLocaleTitle":{"type":{"map":"string"}},"entityUrn":{"type":"string"},"companyUrn":{"type":"string"},"companyName":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.identity.profile.Position"},"com.linkedin.deco.recipe.anonymous.Anon856218111":{"fields":{"transcribedDocumentUrl":{"type":"string"},"urn":{"type":"string"},"manifestUrl":{"type":"string"},"scanRequiredForDownload":{"type":"boolean"},"totalPageCount":{"type":"int"},"manifestUrlExpiresAt":{"type":"long"},"title":{"type":"string"},"coverPages":{"type":"com.linkedin.deco.recipe.anonymous.Anon1636615691"},"transcribedDocumentUrlExpiresAt":{"type":"long"}},"baseType":"com.linkedin.documentcontent.Document"},"com.linkedin.deco.recipe.anonymous.Anon1436383648":{"fields":{"groupUrn":{"type":"string"},"vectorImage":{"type":"com.linkedin.voyager.dash.deco.common.VectorImageOnlyRootUrlAndAttribution"},"group":{"type":"com.linkedin.deco.recipe.anonymous.Anon1869367056","resolvedFrom":"groupUrn"}},"baseType":"com.linkedin.voyager.dash.common.image.NonEntityGroupLogo"},"com.linkedin.deco.recipe.anonymous.Anon548743587":{"fields":{"name":{"type":"com.linkedin.voyager.dash.deco.common.text.TextViewModelV2"},"imageUnion":{"type":{"union":{"url":"string","vectorImage":"com.linkedin.voyager.dash.deco.common.VectorImageOnlyRootUrlAndAttribution"}}}},"baseType":"com.linkedin.voyager.dash.common.media.StickerLinkSmallTemplateView"},"com.linkedin.deco.recipe.anonymous.Anon2083470902":{"fields":{"controlName":{"type":"string"},"numFractionalDigits":{"type":"int"},"hintText":{"type":"string"},"numericRangeValidation":{"type":"com.linkedin.deco.recipe.anonymous.Anon600254320"}},"baseType":"com.linkedin.voyager.dash.common.forms.NumberInputFormComponent"},"com.linkedin.voyager.dash.deco.common.forms.FormElement":{"fields":{"urn":{"type":"string"},"prerequisiteInputEvaluationStrategyUnion":{"type":{"union":{"answered":"string","matchAnyPrerequisiteInput":"com.linkedin.deco.recipe.anonymous.Anon870521084","notMatchAnyPrequisiteInput":"com.linkedin.deco.recipe.anonymous.Anon870521084"}}},"formComponent":{"type":{"union":{"gaiTextFormComponent":"com.linkedin.deco.recipe.anonymous.Anon146637213","multilineTextFormComponent":"com.linkedin.voyager.dash.deco.common.forms.MultilineTextFormComponent","textEntityListFormComponent":"com.linkedin.voyager.dash.deco.common.forms.TextEntityListFormComponent","toggleFormComponent":"com.linkedin.voyager.dash.deco.common.forms.ToggleFormComponent","starRatingFormComponent":"com.linkedin.deco.recipe.anonymous.Anon1735278252","nestedCheckboxFormComponent":"com.linkedin.deco.recipe.anonymous.Anon1048186976","mediaUploadFormComponent":"com.linkedin.deco.recipe.anonymous.Anon2105959233","checkboxFormComponent":"com.linkedin.voyager.dash.deco.common.forms.CheckboxFormComponent","dropdownFormComponent":"com.linkedin.voyager.dash.deco.common.forms.DropdownFormComponent","numberInputFormComponent":"com.linkedin.deco.recipe.anonymous.Anon2083470902","reorderableListFormComponent":"com.linkedin.voyager.dash.deco.common.forms.ReorderableListFormComponent","singleTypeaheadEntityFormComponent":"com.linkedin.voyager.dash.deco.common.forms.SingleTypeaheadEntityFormComponent","dateRangeFormComponent":"com.linkedin.voyager.dash.deco.common.forms.DateRangeFormComponent","dateFormComponent":"com.linkedin.voyager.dash.deco.common.forms.DateFormComponent","multiSelectTypeaheadEntityFormComponent":"com.linkedin.voyager.dash.deco.common.forms.MultiSelectTypeaheadEntityFormComponent","locationFormComponent":"com.linkedin.deco.recipe.anonymous.Anon2006189319","singleLineTextFormComponent":"com.linkedin.voyager.dash.deco.common.forms.SingleLineTextFormComponent","radioButtonFormComponent":"com.linkedin.voyager.dash.deco.common.forms.RadioButtonFormComponent","pillFormComponent":"com.linkedin.voyager.dash.deco.common.forms.PillFormComponent"}}},"input":{"type":"com.linkedin.voyager.dash.deco.common.forms.FormElementInput"},"prerequisiteFormElementInputs":{"type":{"array":"com.linkedin.voyager.dash.deco.common.forms.FormElementInput"}},"requiredFieldMissingErrorText":{"type":"com.linkedin.voyager.dash.deco.common.text.TextViewModelV2"},"entityUrn":{"type":"string"},"weight":{"type":"float"},"helperLink":{"type":"com.linkedin.voyager.dash.deco.common.text.TextViewModelV2"},"title":{"type":"com.linkedin.voyager.dash.deco.common.text.TextViewModelV2"},"required":{"type":"boolean"},"shouldAlwaysSendBackFormElementInput":{"type":"boolean"}},"baseType":"com.linkedin.voyager.dash.common.forms.FormElement"},"com.linkedin.deco.recipe.anonymous.Anon1861880920":{"fields":{"paging":{"type":"com.linkedin.voyager.dash.deco.common.FullPaging"},"elements":{"type":{"array":"com.linkedin.deco.recipe.anonymous.Anon1085906135"}}},"baseType":"com.linkedin.restli.common.CollectionResponse"},"com.linkedin.voyager.dash.deco.common.forms.ToggleFormComponent":{"fields":{"icon":{"type":"string"},"controlName":{"type":"string"},"positiveToggleLabel":{"type":"com.linkedin.voyager.dash.deco.common.text.TextViewModelV2"},"negativeToggleLabel":{"type":"com.linkedin.voyager.dash.deco.common.text.TextViewModelV2"}},"baseType":"com.linkedin.voyager.dash.common.forms.ToggleFormComponent"},"com.linkedin.deco.recipe.anonymous.Anon1585981083":{"fields":{"title":{"type":"string"},"description":{"type":"string"},"connectionsUsingProductTypeUrn":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.identity.profile.ConnectionsUsingProductMetadata"},"com.linkedin.deco.recipe.anonymous.Anon2134644859":{"fields":{"entityUrn":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.relationships.Connection"},"com.linkedin.deco.recipe.anonymous.Anon1048186976":{"fields":{"nestedTextSelectableOptions":{"type":{"array":"com.linkedin.deco.recipe.anonymous.Anon902172343"}},"selectionCountRangeValidation":{"type":"com.linkedin.voyager.dash.deco.common.forms.SelectionCountRangeValidation"},"exclusiveSelectableOption":{"type":"com.linkedin.voyager.dash.deco.common.forms.TextSelectableOption"},"subtitle":{"type":"com.linkedin.voyager.dash.deco.common.text.TextViewModelV2"}},"baseType":"com.linkedin.voyager.dash.common.forms.NestedCheckboxFormComponent"}}}},"included":[]}
</code>
<code style="display: none" id="datalet-bpr-guid-15912210" aria-hidden="true">
  {"request":"/voyager/api/voyagerLaunchpadDashLaunchpadViews?decorationId\u003Dcom.linkedin.voyager.dash.deco.launchpad.LaunchpadView-76\u0026launchpadContext\u003DTAKEOVER\u0026q\u003Dcontext","status":200,"body":"bpr-guid-15912210","method":"GET","headers":{"x-li-uuid":"AAYLE+vtCEZ6Amp7P3S1tA\u003D\u003D"}}
</code>
<img src="data:image/gif;base64,R0lGODlhAQABAIAAAAAAAP///yH5BAEAAAAALAAAAAABAAEAAAIBRAA7" style="display: none" class="datalet-bpr-guid-15912210" aria-hidden="true"><code style="display: none" id="bpr-guid-15912213" aria-hidden="true">
  {"data":{"data":{"feedDashGlobalNavs":{"logo":{"controlName":"nav_inbug","logo":{"attributes":[{"scalingType":null,"detailData":{"profilePictureWithoutFrame":null,"profilePictureWithRingStatus":null,"companyLogo":null,"icon":"IC_APP_LINKEDIN_BUG_COLOR_24DP","systemImage":null,"nonEntityGroupLogo":null,"vectorImage":null,"nonEntityProfessionalEventLogo":null,"profilePicture":null,"imageUrl":null,"professionalEventLogo":null,"nonEntityCompanyLogo":null,"nonEntitySchoolLogo":null,"groupLogo":null,"schoolLogo":null,"ghostImage":null,"nonEntityProfilePicture":null},"tintColor":null,"$recipeTypes":["com.linkedin.e6506ba5f5cc1a9b5124781a3661a7db"],"tapTargets":[],"displayAspectRatio":null,"$type":"com.linkedin.voyager.dash.common.image.ImageAttribute"}],"editableAccessibilityText":false,"actionTarget":"https://www.linkedin.com/feed/?doFeedRefresh=true&amp;nis=true","accessibilityTextAttributes":[],"totalCount":null,"accessibilityText":"LinkedIn","$recipeTypes":["com.linkedin.f89edd0cdc11632a12f86e550f2f5e46"],"$type":"com.linkedin.voyager.dash.common.image.ImageViewModel"},"$recipeTypes":["com.linkedin.4973670c8e07d9bf8572c4c7f8658e2b"],"hoverText":null,"$type":"com.linkedin.voyager.dash.feed.nav.BrandingLogo"},"$recipeTypes":["com.linkedin.eae2ed6e463973c8e0d55baecd814363"],"spotlight":{"navContent":{"navElement":{"staticIcon":{"attributes":[{"scalingType":null,"detailData":{"profilePictureWithoutFrame":null,"profilePictureWithRingStatus":null,"companyLogo":null,"icon":"IC_NAV_SMALL_LEARNING_24DP","systemImage":null,"nonEntityGroupLogo":null,"vectorImage":null,"nonEntityProfessionalEventLogo":null,"profilePicture":null,"imageUrl":null,"professionalEventLogo":null,"nonEntityCompanyLogo":null,"nonEntitySchoolLogo":null,"groupLogo":null,"schoolLogo":null,"ghostImage":null,"nonEntityProfilePicture":null},"tintColor":null,"$recipeTypes":["com.linkedin.e6506ba5f5cc1a9b5124781a3661a7db"],"tapTargets":[],"displayAspectRatio":null,"$type":"com.linkedin.voyager.dash.common.image.ImageAttribute"}],"editableAccessibilityText":false,"actionTarget":null,"accessibilityTextAttributes":[],"totalCount":null,"accessibilityText":null,"$recipeTypes":["com.linkedin.f89edd0cdc11632a12f86e550f2f5e46"],"$type":"com.linkedin.voyager.dash.common.image.ImageViewModel"},"activeIcon":null,"hasCoachmark":false,"controlName":"nav_learning","showIconBorder":false,"actionTarget":"https://www.linkedin.com/learning/?trk=nav_neptune_learning","text":"Learning","$recipeTypes":["com.linkedin.b2c03744140a6cd6a2e693c32790988c"],"hasPaid":true,"$type":"com.linkedin.voyager.dash.feed.nav.NavElement"},"premiumUpsellSlotUrn":null},"type":"LEARNING","$recipeTypes":["com.linkedin.c7f876e8e9f8a631b2896f62a656ce64"],"$type":"com.linkedin.voyager.dash.feed.nav.NavItem"},"primaryItems":[{"appLauncher":null,"navItem":{"navContent":{"navElement":{"activeIcon":{"attributes":[{"scalingType":null,"detailData":{"profilePictureWithoutFrame":null,"profilePictureWithRingStatus":null,"companyLogo":null,"icon":"IC_NAV_SMALL_HOME_ACTIVE_24DP","systemImage":null,"nonEntityGroupLogo":null,"vectorImage":null,"nonEntityProfessionalEventLogo":null,"profilePicture":null,"imageUrl":null,"professionalEventLogo":null,"nonEntityCompanyLogo":null,"nonEntitySchoolLogo":null,"groupLogo":null,"schoolLogo":null,"ghostImage":null,"nonEntityProfilePicture":null},"tintColor":null,"$recipeTypes":["com.linkedin.e6506ba5f5cc1a9b5124781a3661a7db"],"tapTargets":[],"displayAspectRatio":null,"$type":"com.linkedin.voyager.dash.common.image.ImageAttribute"}],"editableAccessibilityText":false,"actionTarget":null,"accessibilityTextAttributes":[],"totalCount":null,"accessibilityText":null,"$recipeTypes":["com.linkedin.f89edd0cdc11632a12f86e550f2f5e46"],"$type":"com.linkedin.voyager.dash.common.image.ImageViewModel"},"staticIcon":{"attributes":[{"scalingType":null,"detailData":{"profilePictureWithoutFrame":null,"profilePictureWithRingStatus":null,"companyLogo":null,"icon":"IC_NAV_SMALL_HOME_24DP","systemImage":null,"nonEntityGroupLogo":null,"vectorImage":null,"nonEntityProfessionalEventLogo":null,"profilePicture":null,"imageUrl":null,"professionalEventLogo":null,"nonEntityCompanyLogo":null,"nonEntitySchoolLogo":null,"groupLogo":null,"schoolLogo":null,"ghostImage":null,"nonEntityProfilePicture":null},"tintColor":null,"$recipeTypes":["com.linkedin.e6506ba5f5cc1a9b5124781a3661a7db"],"tapTargets":[],"displayAspectRatio":null,"$type":"com.linkedin.voyager.dash.common.image.ImageAttribute"}],"editableAccessibilityText":false,"actionTarget":null,"accessibilityTextAttributes":[],"totalCount":null,"accessibilityText":null,"$recipeTypes":["com.linkedin.f89edd0cdc11632a12f86e550f2f5e46"],"$type":"com.linkedin.voyager.dash.common.image.ImageViewModel"},"hasCoachmark":false,"controlName":"nav_homepage","showIconBorder":false,"actionTarget":"https://www.linkedin.com/feed/?doFeedRefresh=true&amp;nis=true","text":"Home","$recipeTypes":["com.linkedin.b2c03744140a6cd6a2e693c32790988c"],"hasPaid":false,"$type":"com.linkedin.voyager.dash.feed.nav.NavElement"},"premiumUpsellSlotUrn":null},"type":"HOME","$recipeTypes":["com.linkedin.c7f876e8e9f8a631b2896f62a656ce64"],"$type":"com.linkedin.voyager.dash.feed.nav.NavItem"},"meMenu":null},{"appLauncher":null,"navItem":{"navContent":{"navElement":{"activeIcon":{"attributes":[{"scalingType":null,"detailData":{"profilePictureWithoutFrame":null,"profilePictureWithRingStatus":null,"companyLogo":null,"icon":"IC_NAV_SMALL_PEOPLE_ACTIVE_24DP","systemImage":null,"nonEntityGroupLogo":null,"vectorImage":null,"nonEntityProfessionalEventLogo":null,"profilePicture":null,"imageUrl":null,"professionalEventLogo":null,"nonEntityCompanyLogo":null,"nonEntitySchoolLogo":null,"groupLogo":null,"schoolLogo":null,"ghostImage":null,"nonEntityProfilePicture":null},"tintColor":null,"$recipeTypes":["com.linkedin.e6506ba5f5cc1a9b5124781a3661a7db"],"tapTargets":[],"displayAspectRatio":null,"$type":"com.linkedin.voyager.dash.common.image.ImageAttribute"}],"editableAccessibilityText":false,"actionTarget":null,"accessibilityTextAttributes":[],"totalCount":null,"accessibilityText":null,"$recipeTypes":["com.linkedin.f89edd0cdc11632a12f86e550f2f5e46"],"$type":"com.linkedin.voyager.dash.common.image.ImageViewModel"},"staticIcon":{"attributes":[{"scalingType":null,"detailData":{"profilePictureWithoutFrame":null,"profilePictureWithRingStatus":null,"companyLogo":null,"icon":"IC_NAV_SMALL_PEOPLE_24DP","systemImage":null,"nonEntityGroupLogo":null,"vectorImage":null,"nonEntityProfessionalEventLogo":null,"profilePicture":null,"imageUrl":null,"professionalEventLogo":null,"nonEntityCompanyLogo":null,"nonEntitySchoolLogo":null,"groupLogo":null,"schoolLogo":null,"ghostImage":null,"nonEntityProfilePicture":null},"tintColor":null,"$recipeTypes":["com.linkedin.e6506ba5f5cc1a9b5124781a3661a7db"],"tapTargets":[],"displayAspectRatio":null,"$type":"com.linkedin.voyager.dash.common.image.ImageAttribute"}],"editableAccessibilityText":false,"actionTarget":null,"accessibilityTextAttributes":[],"totalCount":null,"accessibilityText":null,"$recipeTypes":["com.linkedin.f89edd0cdc11632a12f86e550f2f5e46"],"$type":"com.linkedin.voyager.dash.common.image.ImageViewModel"},"hasCoachmark":false,"controlName":"nav_mynetwork","showIconBorder":false,"actionTarget":"https://www.linkedin.com/mynetwork/","text":"My Network","$recipeTypes":["com.linkedin.b2c03744140a6cd6a2e693c32790988c"],"hasPaid":false,"$type":"com.linkedin.voyager.dash.feed.nav.NavElement"},"premiumUpsellSlotUrn":null},"type":"MY_NETWORK","$recipeTypes":["com.linkedin.c7f876e8e9f8a631b2896f62a656ce64"],"$type":"com.linkedin.voyager.dash.feed.nav.NavItem"},"meMenu":null},{"appLauncher":null,"navItem":{"navContent":{"navElement":{"activeIcon":{"attributes":[{"scalingType":null,"detailData":{"profilePictureWithoutFrame":null,"profilePictureWithRingStatus":null,"companyLogo":null,"icon":"IC_NAV_SMALL_JOBS_ACTIVE_24DP","systemImage":null,"nonEntityGroupLogo":null,"vectorImage":null,"nonEntityProfessionalEventLogo":null,"profilePicture":null,"imageUrl":null,"professionalEventLogo":null,"nonEntityCompanyLogo":null,"nonEntitySchoolLogo":null,"groupLogo":null,"schoolLogo":null,"ghostImage":null,"nonEntityProfilePicture":null},"tintColor":null,"$recipeTypes":["com.linkedin.e6506ba5f5cc1a9b5124781a3661a7db"],"tapTargets":[],"displayAspectRatio":null,"$type":"com.linkedin.voyager.dash.common.image.ImageAttribute"}],"editableAccessibilityText":false,"actionTarget":null,"accessibilityTextAttributes":[],"totalCount":null,"accessibilityText":null,"$recipeTypes":["com.linkedin.f89edd0cdc11632a12f86e550f2f5e46"],"$type":"com.linkedin.voyager.dash.common.image.ImageViewModel"},"staticIcon":{"attributes":[{"scalingType":null,"detailData":{"profilePictureWithoutFrame":null,"profilePictureWithRingStatus":null,"companyLogo":null,"icon":"IC_NAV_SMALL_JOBS_24DP","systemImage":null,"nonEntityGroupLogo":null,"vectorImage":null,"nonEntityProfessionalEventLogo":null,"profilePicture":null,"imageUrl":null,"professionalEventLogo":null,"nonEntityCompanyLogo":null,"nonEntitySchoolLogo":null,"groupLogo":null,"schoolLogo":null,"ghostImage":null,"nonEntityProfilePicture":null},"tintColor":null,"$recipeTypes":["com.linkedin.e6506ba5f5cc1a9b5124781a3661a7db"],"tapTargets":[],"displayAspectRatio":null,"$type":"com.linkedin.voyager.dash.common.image.ImageAttribute"}],"editableAccessibilityText":false,"actionTarget":null,"accessibilityTextAttributes":[],"totalCount":null,"accessibilityText":null,"$recipeTypes":["com.linkedin.f89edd0cdc11632a12f86e550f2f5e46"],"$type":"com.linkedin.voyager.dash.common.image.ImageViewModel"},"hasCoachmark":false,"controlName":"nav_jobs","showIconBorder":false,"actionTarget":"https://www.linkedin.com/jobs/","text":"Jobs","$recipeTypes":["com.linkedin.b2c03744140a6cd6a2e693c32790988c"],"hasPaid":false,"$type":"com.linkedin.voyager.dash.feed.nav.NavElement"},"premiumUpsellSlotUrn":null},"type":"JOBS","$recipeTypes":["com.linkedin.c7f876e8e9f8a631b2896f62a656ce64"],"$type":"com.linkedin.voyager.dash.feed.nav.NavItem"},"meMenu":null},{"appLauncher":null,"navItem":{"navContent":{"navElement":{"activeIcon":{"attributes":[{"scalingType":null,"detailData":{"profilePictureWithoutFrame":null,"profilePictureWithRingStatus":null,"companyLogo":null,"icon":"IC_NAV_SMALL_MESSAGING_ACTIVE_24DP","systemImage":null,"nonEntityGroupLogo":null,"vectorImage":null,"nonEntityProfessionalEventLogo":null,"profilePicture":null,"imageUrl":null,"professionalEventLogo":null,"nonEntityCompanyLogo":null,"nonEntitySchoolLogo":null,"groupLogo":null,"schoolLogo":null,"ghostImage":null,"nonEntityProfilePicture":null},"tintColor":null,"$recipeTypes":["com.linkedin.e6506ba5f5cc1a9b5124781a3661a7db"],"tapTargets":[],"displayAspectRatio":null,"$type":"com.linkedin.voyager.dash.common.image.ImageAttribute"}],"editableAccessibilityText":false,"actionTarget":null,"accessibilityTextAttributes":[],"totalCount":null,"accessibilityText":null,"$recipeTypes":["com.linkedin.f89edd0cdc11632a12f86e550f2f5e46"],"$type":"com.linkedin.voyager.dash.common.image.ImageViewModel"},"staticIcon":{"attributes":[{"scalingType":null,"detailData":{"profilePictureWithoutFrame":null,"profilePictureWithRingStatus":null,"companyLogo":null,"icon":"IC_NAV_SMALL_MESSAGING_24DP","systemImage":null,"nonEntityGroupLogo":null,"vectorImage":null,"nonEntityProfessionalEventLogo":null,"profilePicture":null,"imageUrl":null,"professionalEventLogo":null,"nonEntityCompanyLogo":null,"nonEntitySchoolLogo":null,"groupLogo":null,"schoolLogo":null,"ghostImage":null,"nonEntityProfilePicture":null},"tintColor":null,"$recipeTypes":["com.linkedin.e6506ba5f5cc1a9b5124781a3661a7db"],"tapTargets":[],"displayAspectRatio":null,"$type":"com.linkedin.voyager.dash.common.image.ImageAttribute"}],"editableAccessibilityText":false,"actionTarget":null,"accessibilityTextAttributes":[],"totalCount":null,"accessibilityText":null,"$recipeTypes":["com.linkedin.f89edd0cdc11632a12f86e550f2f5e46"],"$type":"com.linkedin.voyager.dash.common.image.ImageViewModel"},"hasCoachmark":false,"controlName":"nav_messaging","showIconBorder":false,"actionTarget":"https://www.linkedin.com/messaging/","text":"Messaging","$recipeTypes":["com.linkedin.b2c03744140a6cd6a2e693c32790988c"],"hasPaid":false,"$type":"com.linkedin.voyager.dash.feed.nav.NavElement"},"premiumUpsellSlotUrn":null},"type":"MESSAGING","$recipeTypes":["com.linkedin.c7f876e8e9f8a631b2896f62a656ce64"],"$type":"com.linkedin.voyager.dash.feed.nav.NavItem"},"meMenu":null},{"appLauncher":null,"navItem":{"navContent":{"navElement":{"activeIcon":{"attributes":[{"scalingType":null,"detailData":{"profilePictureWithoutFrame":null,"profilePictureWithRingStatus":null,"companyLogo":null,"icon":"IC_NAV_SMALL_NOTIFICATIONS_ACTIVE_24DP","systemImage":null,"nonEntityGroupLogo":null,"vectorImage":null,"nonEntityProfessionalEventLogo":null,"profilePicture":null,"imageUrl":null,"professionalEventLogo":null,"nonEntityCompanyLogo":null,"nonEntitySchoolLogo":null,"groupLogo":null,"schoolLogo":null,"ghostImage":null,"nonEntityProfilePicture":null},"tintColor":null,"$recipeTypes":["com.linkedin.e6506ba5f5cc1a9b5124781a3661a7db"],"tapTargets":[],"displayAspectRatio":null,"$type":"com.linkedin.voyager.dash.common.image.ImageAttribute"}],"editableAccessibilityText":false,"actionTarget":null,"accessibilityTextAttributes":[],"totalCount":null,"accessibilityText":null,"$recipeTypes":["com.linkedin.f89edd0cdc11632a12f86e550f2f5e46"],"$type":"com.linkedin.voyager.dash.common.image.ImageViewModel"},"staticIcon":{"attributes":[{"scalingType":null,"detailData":{"profilePictureWithoutFrame":null,"profilePictureWithRingStatus":null,"companyLogo":null,"icon":"IC_NAV_SMALL_NOTIFICATIONS_24DP","systemImage":null,"nonEntityGroupLogo":null,"vectorImage":null,"nonEntityProfessionalEventLogo":null,"profilePicture":null,"imageUrl":null,"professionalEventLogo":null,"nonEntityCompanyLogo":null,"nonEntitySchoolLogo":null,"groupLogo":null,"schoolLogo":null,"ghostImage":null,"nonEntityProfilePicture":null},"tintColor":null,"$recipeTypes":["com.linkedin.e6506ba5f5cc1a9b5124781a3661a7db"],"tapTargets":[],"displayAspectRatio":null,"$type":"com.linkedin.voyager.dash.common.image.ImageAttribute"}],"editableAccessibilityText":false,"actionTarget":null,"accessibilityTextAttributes":[],"totalCount":null,"accessibilityText":null,"$recipeTypes":["com.linkedin.f89edd0cdc11632a12f86e550f2f5e46"],"$type":"com.linkedin.voyager.dash.common.image.ImageViewModel"},"hasCoachmark":false,"controlName":"nav_notifications","showIconBorder":false,"actionTarget":"https://www.linkedin.com/notifications/","text":"Notifications","$recipeTypes":["com.linkedin.b2c03744140a6cd6a2e693c32790988c"],"hasPaid":false,"$type":"com.linkedin.voyager.dash.feed.nav.NavElement"},"premiumUpsellSlotUrn":null},"type":"NOTIFICATIONS","$recipeTypes":["com.linkedin.c7f876e8e9f8a631b2896f62a656ce64"],"$type":"com.linkedin.voyager.dash.feed.nav.NavItem"},"meMenu":null},{"appLauncher":null,"meMenu":{"viewProfileText":"View Profile","*profile":"urn:li:fsd_profile:ACoAAC6BPkEBwSD4NTQ7khSJbuAxrRMYhVSwmBo","meGroups":[{"meItems":[{"meItem":{"controlName":"nav_settings_account_manage_account","actionTarget":"https://www.linkedin.com/mypreferences/d/","text":{"textDirection":"USER_LOCALE","text":"Settings &amp; Privacy","attributesV2":[],"accessibilityTextAttributesV2":[],"accessibilityText":null,"$recipeTypes":["com.linkedin.566376561ef86249fc89f5596eaf6fe1"],"$type":"com.linkedin.voyager.dash.common.text.TextViewModel"},"quickHelp":false,"$recipeTypes":["com.linkedin.fb4511fb6c2df4a805ef662367e15026"],"$type":"com.linkedin.voyager.dash.feed.nav.MeItem"},"premiumUpsellSlotUrn":null},{"meItem":{"controlName":"nav_settings_account_quick_help","actionTarget":"https://www.linkedin.com/help/linkedin/","text":{"textDirection":"USER_LOCALE","text":"Help","attributesV2":[],"accessibilityTextAttributesV2":[],"accessibilityText":null,"$recipeTypes":["com.linkedin.566376561ef86249fc89f5596eaf6fe1"],"$type":"com.linkedin.voyager.dash.common.text.TextViewModel"},"quickHelp":true,"$recipeTypes":["com.linkedin.fb4511fb6c2df4a805ef662367e15026"],"$type":"com.linkedin.voyager.dash.feed.nav.MeItem"},"premiumUpsellSlotUrn":null},{"meItem":{"controlName":"nav_settings_account_language","actionTarget":"https://www.linkedin.com/mypreferences/d/language/","text":{"textDirection":"USER_LOCALE","text":"Language","attributesV2":[],"accessibilityTextAttributesV2":[],"accessibilityText":null,"$recipeTypes":["com.linkedin.566376561ef86249fc89f5596eaf6fe1"],"$type":"com.linkedin.voyager.dash.common.text.TextViewModel"},"quickHelp":false,"$recipeTypes":["com.linkedin.fb4511fb6c2df4a805ef662367e15026"],"$type":"com.linkedin.voyager.dash.feed.nav.MeItem"},"premiumUpsellSlotUrn":null}],"title":"Account","$recipeTypes":["com.linkedin.bb35e914e880d9f9ad79d52bf644c44c"],"$type":"com.linkedin.voyager.dash.feed.nav.MeGroup"},{"meItems":[{"meItem":{"controlName":"recent_activity_nav_all","actionTarget":"https://www.linkedin.com/in/naimiskandar22/recent-activity/","text":{"textDirection":"USER_LOCALE","text":"Posts &amp; Activity","attributesV2":[],"accessibilityTextAttributesV2":[],"accessibilityText":null,"$recipeTypes":["com.linkedin.566376561ef86249fc89f5596eaf6fe1"],"$type":"com.linkedin.voyager.dash.common.text.TextViewModel"},"quickHelp":false,"$recipeTypes":["com.linkedin.fb4511fb6c2df4a805ef662367e15026"],"$type":"com.linkedin.voyager.dash.feed.nav.MeItem"},"premiumUpsellSlotUrn":null},{"meItem":{"controlName":"nav_settings_manage_job_posting_account","actionTarget":"https://www.linkedin.com/talent/job-management-redirect?trk=nav_app_launcher_manage_job_post_nept","text":{"textDirection":"USER_LOCALE","text":"Job Posting Account","attributesV2":[],"accessibilityTextAttributesV2":[],"accessibilityText":null,"$recipeTypes":["com.linkedin.566376561ef86249fc89f5596eaf6fe1"],"$type":"com.linkedin.voyager.dash.common.text.TextViewModel"},"quickHelp":false,"$recipeTypes":["com.linkedin.fb4511fb6c2df4a805ef662367e15026"],"$type":"com.linkedin.voyager.dash.feed.nav.MeItem"},"premiumUpsellSlotUrn":null}],"title":"Manage","$recipeTypes":["com.linkedin.bb35e914e880d9f9ad79d52bf644c44c"],"$type":"com.linkedin.voyager.dash.feed.nav.MeGroup"}],"controlName":"nav_settings","text":"Me","$recipeTypes":["com.linkedin.73113dd58d976ff22a5e4aa2ca35d21d"],"$type":"com.linkedin.voyager.dash.feed.nav.MeMenu"},"navItem":null},{"appLauncher":{"showDivider":true,"productItems":[{"navContent":{"navElement":{"staticIcon":{"attributes":[{"scalingType":null,"detailData":{"profilePictureWithoutFrame":null,"profilePictureWithRingStatus":null,"companyLogo":null,"icon":"IC_APP_TALENT_INSIGHTS_24DP","systemImage":null,"nonEntityGroupLogo":null,"vectorImage":null,"nonEntityProfessionalEventLogo":null,"profilePicture":null,"imageUrl":null,"professionalEventLogo":null,"nonEntityCompanyLogo":null,"nonEntitySchoolLogo":null,"groupLogo":null,"schoolLogo":null,"ghostImage":null,"nonEntityProfilePicture":null},"tintColor":null,"$recipeTypes":["com.linkedin.e6506ba5f5cc1a9b5124781a3661a7db"],"tapTargets":[],"displayAspectRatio":null,"$type":"com.linkedin.voyager.dash.common.image.ImageAttribute"}],"editableAccessibilityText":false,"actionTarget":null,"accessibilityTextAttributes":[],"totalCount":null,"accessibilityText":null,"$recipeTypes":["com.linkedin.f89edd0cdc11632a12f86e550f2f5e46"],"$type":"com.linkedin.voyager.dash.common.image.ImageViewModel"},"activeIcon":null,"hasCoachmark":false,"controlName":"nav_launcher_insights","showIconBorder":true,"actionTarget":"https://www.linkedin.com/insights?trk=nav_app_launcher_insights_nept&amp;src=li-nav","text":"Talent Insights","$recipeTypes":["com.linkedin.b2c03744140a6cd6a2e693c32790988c"],"hasPaid":false,"$type":"com.linkedin.voyager.dash.feed.nav.NavElement"},"premiumUpsellSlotUrn":null},"type":"INSIGHTS","$recipeTypes":["com.linkedin.c7f876e8e9f8a631b2896f62a656ce64"],"$type":"com.linkedin.voyager.dash.feed.nav.NavItem"},{"navContent":{"navElement":{"staticIcon":{"attributes":[{"scalingType":null,"detailData":{"profilePictureWithoutFrame":null,"profilePictureWithRingStatus":null,"companyLogo":null,"icon":"IC_APP_JOBS_POSTING_24DP","systemImage":null,"nonEntityGroupLogo":null,"vectorImage":null,"nonEntityProfessionalEventLogo":null,"profilePicture":null,"imageUrl":null,"professionalEventLogo":null,"nonEntityCompanyLogo":null,"nonEntitySchoolLogo":null,"groupLogo":null,"schoolLogo":null,"ghostImage":null,"nonEntityProfilePicture":null},"tintColor":null,"$recipeTypes":["com.linkedin.e6506ba5f5cc1a9b5124781a3661a7db"],"tapTargets":[],"displayAspectRatio":null,"$type":"com.linkedin.voyager.dash.common.image.ImageAttribute"}],"editableAccessibilityText":false,"actionTarget":null,"accessibilityTextAttributes":[],"totalCount":null,"accessibilityText":null,"$recipeTypes":["com.linkedin.f89edd0cdc11632a12f86e550f2f5e46"],"$type":"com.linkedin.voyager.dash.common.image.ImageViewModel"},"activeIcon":null,"hasCoachmark":false,"controlName":"nav_launcher_job_postings","showIconBorder":true,"actionTarget":"https://www.linkedin.com/talent/job-posting-redirect?trk=nav_app_launcher_job_post_nept","text":"Post a job","$recipeTypes":["com.linkedin.b2c03744140a6cd6a2e693c32790988c"],"hasPaid":true,"$type":"com.linkedin.voyager.dash.feed.nav.NavElement"},"premiumUpsellSlotUrn":null},"type":"JOB_POSTINGS","$recipeTypes":["com.linkedin.c7f876e8e9f8a631b2896f62a656ce64"],"$type":"com.linkedin.voyager.dash.feed.nav.NavItem"},{"navContent":{"navElement":{"staticIcon":{"attributes":[{"scalingType":null,"detailData":{"profilePictureWithoutFrame":null,"profilePictureWithRingStatus":null,"companyLogo":null,"icon":"IC_APP_ADS_24DP","systemImage":null,"nonEntityGroupLogo":null,"vectorImage":null,"nonEntityProfessionalEventLogo":null,"profilePicture":null,"imageUrl":null,"professionalEventLogo":null,"nonEntityCompanyLogo":null,"nonEntitySchoolLogo":null,"groupLogo":null,"schoolLogo":null,"ghostImage":null,"nonEntityProfilePicture":null},"tintColor":null,"$recipeTypes":["com.linkedin.e6506ba5f5cc1a9b5124781a3661a7db"],"tapTargets":[],"displayAspectRatio":null,"$type":"com.linkedin.voyager.dash.common.image.ImageAttribute"}],"editableAccessibilityText":false,"actionTarget":null,"accessibilityTextAttributes":[],"totalCount":null,"accessibilityText":null,"$recipeTypes":["com.linkedin.f89edd0cdc11632a12f86e550f2f5e46"],"$type":"com.linkedin.voyager.dash.common.image.ImageViewModel"},"activeIcon":null,"hasCoachmark":false,"controlName":"nav_launcher_ads","showIconBorder":true,"actionTarget":"https://www.linkedin.com/campaignmanager/accounts","text":"Advertise","$recipeTypes":["com.linkedin.b2c03744140a6cd6a2e693c32790988c"],"hasPaid":true,"$type":"com.linkedin.voyager.dash.feed.nav.NavElement"},"premiumUpsellSlotUrn":null},"type":"ADVERTISE","$recipeTypes":["com.linkedin.c7f876e8e9f8a631b2896f62a656ce64"],"$type":"com.linkedin.voyager.dash.feed.nav.NavItem"},{"navContent":{"navElement":{"staticIcon":{"attributes":[{"scalingType":null,"detailData":{"profilePictureWithoutFrame":null,"profilePictureWithRingStatus":null,"companyLogo":null,"icon":"IC_APP_SALES_NAVIGATOR_24DP","systemImage":null,"nonEntityGroupLogo":null,"vectorImage":null,"nonEntityProfessionalEventLogo":null,"profilePicture":null,"imageUrl":null,"professionalEventLogo":null,"nonEntityCompanyLogo":null,"nonEntitySchoolLogo":null,"groupLogo":null,"schoolLogo":null,"ghostImage":null,"nonEntityProfilePicture":null},"tintColor":null,"$recipeTypes":["com.linkedin.e6506ba5f5cc1a9b5124781a3661a7db"],"tapTargets":[],"displayAspectRatio":null,"$type":"com.linkedin.voyager.dash.common.image.ImageAttribute"}],"editableAccessibilityText":false,"actionTarget":null,"accessibilityTextAttributes":[],"totalCount":null,"accessibilityText":null,"$recipeTypes":["com.linkedin.f89edd0cdc11632a12f86e550f2f5e46"],"$type":"com.linkedin.voyager.dash.common.image.ImageViewModel"},"activeIcon":null,"hasCoachmark":false,"controlName":"nav_launcher_find_leads","showIconBorder":true,"actionTarget":"https://www.linkedin.com/premium/products/?intentType=FIND_LEADS&amp;upsellOrderOrigin=premium_nav_more_products_panel&amp;utype=sales","text":"Sell","$recipeTypes":["com.linkedin.b2c03744140a6cd6a2e693c32790988c"],"hasPaid":false,"$type":"com.linkedin.voyager.dash.feed.nav.NavElement"},"premiumUpsellSlotUrn":null},"type":"FIND_LEADS","$recipeTypes":["com.linkedin.c7f876e8e9f8a631b2896f62a656ce64"],"$type":"com.linkedin.voyager.dash.feed.nav.NavItem"},{"navContent":{"navElement":{"staticIcon":{"attributes":[{"scalingType":null,"detailData":{"profilePictureWithoutFrame":null,"profilePictureWithRingStatus":null,"companyLogo":null,"icon":"IC_APP_GROUPS_24DP","systemImage":null,"nonEntityGroupLogo":null,"vectorImage":null,"nonEntityProfessionalEventLogo":null,"profilePicture":null,"imageUrl":null,"professionalEventLogo":null,"nonEntityCompanyLogo":null,"nonEntitySchoolLogo":null,"groupLogo":null,"schoolLogo":null,"ghostImage":null,"nonEntityProfilePicture":null},"tintColor":null,"$recipeTypes":["com.linkedin.e6506ba5f5cc1a9b5124781a3661a7db"],"tapTargets":[],"displayAspectRatio":null,"$type":"com.linkedin.voyager.dash.common.image.ImageAttribute"}],"editableAccessibilityText":false,"actionTarget":null,"accessibilityTextAttributes":[],"totalCount":null,"accessibilityText":null,"$recipeTypes":["com.linkedin.f89edd0cdc11632a12f86e550f2f5e46"],"$type":"com.linkedin.voyager.dash.common.image.ImageViewModel"},"activeIcon":null,"hasCoachmark":false,"controlName":"nav_launcher_groups","showIconBorder":true,"actionTarget":"https://www.linkedin.com/groups","text":"Groups","$recipeTypes":["com.linkedin.b2c03744140a6cd6a2e693c32790988c"],"hasPaid":false,"$type":"com.linkedin.voyager.dash.feed.nav.NavElement"},"premiumUpsellSlotUrn":null},"type":"GROUPS","$recipeTypes":["com.linkedin.c7f876e8e9f8a631b2896f62a656ce64"],"$type":"com.linkedin.voyager.dash.feed.nav.NavItem"},{"navContent":{"navElement":{"staticIcon":{"attributes":[{"scalingType":null,"detailData":{"profilePictureWithoutFrame":null,"profilePictureWithRingStatus":null,"companyLogo":null,"icon":"IC_APP_PROFINDER_24DP","systemImage":null,"nonEntityGroupLogo":null,"vectorImage":null,"nonEntityProfessionalEventLogo":null,"profilePicture":null,"imageUrl":null,"professionalEventLogo":null,"nonEntityCompanyLogo":null,"nonEntitySchoolLogo":null,"groupLogo":null,"schoolLogo":null,"ghostImage":null,"nonEntityProfilePicture":null},"tintColor":null,"$recipeTypes":["com.linkedin.e6506ba5f5cc1a9b5124781a3661a7db"],"tapTargets":[],"displayAspectRatio":null,"$type":"com.linkedin.voyager.dash.common.image.ImageAttribute"}],"editableAccessibilityText":false,"actionTarget":null,"accessibilityTextAttributes":[],"totalCount":null,"accessibilityText":null,"$recipeTypes":["com.linkedin.f89edd0cdc11632a12f86e550f2f5e46"],"$type":"com.linkedin.voyager.dash.common.image.ImageViewModel"},"activeIcon":null,"hasCoachmark":false,"controlName":"nav_launcher_services_marketplace","showIconBorder":true,"actionTarget":"https://www.linkedin.com/services?trk=d_flagship3_nav","text":"Services Marketplace","$recipeTypes":["com.linkedin.b2c03744140a6cd6a2e693c32790988c"],"hasPaid":false,"$type":"com.linkedin.voyager.dash.feed.nav.NavElement"},"premiumUpsellSlotUrn":null},"type":"PROFINDER","$recipeTypes":["com.linkedin.c7f876e8e9f8a631b2896f62a656ce64"],"$type":"com.linkedin.voyager.dash.feed.nav.NavItem"}],"businessServices":[{"businessServiceAction":{"controlName":"nav_business_talent_solutions","actionTarget":"https://business.linkedin.com/talent-solutions?trk=flagship_nav&amp;veh=li-header-dropdown-lts-control&amp;src=li-nav","title":{"textDirection":"USER_LOCALE","text":"Hire on LinkedIn","attributesV2":[],"accessibilityTextAttributesV2":[],"accessibilityText":null,"$recipeTypes":["com.linkedin.566376561ef86249fc89f5596eaf6fe1"],"$type":"com.linkedin.voyager.dash.common.text.TextViewModel"},"$recipeTypes":["com.linkedin.188858304ae14097e59518d245c7cc85"],"$type":"com.linkedin.voyager.dash.feed.nav.BusinessServiceAction"},"description":"Find, attract and recruit talent","$recipeTypes":["com.linkedin.a94ff0919cddaa7056025e9491dc3794"],"$type":"com.linkedin.voyager.dash.feed.nav.BusinessService"},{"businessServiceAction":{"controlName":"nav_business_sales_solutions","actionTarget":"https://business.linkedin.com/sales-solutions?trk=flagship_nav&amp;veh=li-header-dropdown-lss-control&amp;src=li-nav","title":{"textDirection":"USER_LOCALE","text":"Sell with LinkedIn","attributesV2":[],"accessibilityTextAttributesV2":[],"accessibilityText":null,"$recipeTypes":["com.linkedin.566376561ef86249fc89f5596eaf6fe1"],"$type":"com.linkedin.voyager.dash.common.text.TextViewModel"},"$recipeTypes":["com.linkedin.188858304ae14097e59518d245c7cc85"],"$type":"com.linkedin.voyager.dash.feed.nav.BusinessServiceAction"},"description":"Build relationships with buyers","$recipeTypes":["com.linkedin.a94ff0919cddaa7056025e9491dc3794"],"$type":"com.linkedin.voyager.dash.feed.nav.BusinessService"},{"businessServiceAction":{"controlName":"nav_business_post_a_job","actionTarget":"https://www.linkedin.com/talent/job-posting-redirect?trk=nav_biz_serv_job_post_nept","title":{"textDirection":"USER_LOCALE","text":"Post a job for free","attributesV2":[],"accessibilityTextAttributesV2":[],"accessibilityText":null,"$recipeTypes":["com.linkedin.566376561ef86249fc89f5596eaf6fe1"],"$type":"com.linkedin.voyager.dash.common.text.TextViewModel"},"$recipeTypes":["com.linkedin.188858304ae14097e59518d245c7cc85"],"$type":"com.linkedin.voyager.dash.feed.nav.BusinessServiceAction"},"description":"Find quality candidates","$recipeTypes":["com.linkedin.a94ff0919cddaa7056025e9491dc3794"],"$type":"com.linkedin.voyager.dash.feed.nav.BusinessService"},{"businessServiceAction":{"controlName":"nav_business_advertise","actionTarget":"https://business.linkedin.com/marketing-solutions/ads?trk=n_nav_ads_rr_b&amp;src=li-nav","title":{"textDirection":"USER_LOCALE","text":"Advertise on LinkedIn","attributesV2":[],"accessibilityTextAttributesV2":[],"accessibilityText":null,"$recipeTypes":["com.linkedin.566376561ef86249fc89f5596eaf6fe1"],"$type":"com.linkedin.voyager.dash.common.text.TextViewModel"},"$recipeTypes":["com.linkedin.188858304ae14097e59518d245c7cc85"],"$type":"com.linkedin.voyager.dash.feed.nav.BusinessServiceAction"},"description":"Acquire customers and grow your business","$recipeTypes":["com.linkedin.a94ff0919cddaa7056025e9491dc3794"],"$type":"com.linkedin.voyager.dash.feed.nav.BusinessService"},{"businessServiceAction":{"controlName":"nav_business_learning_solutions","actionTarget":"https://learning.linkedin.com/?trk=d_flagship3_nav&amp;veh=learning_solutions&amp;src=li-nav","title":{"textDirection":"USER_LOCALE","text":"Learn with LinkedIn","attributesV2":[],"accessibilityTextAttributesV2":[],"accessibilityText":null,"$recipeTypes":["com.linkedin.566376561ef86249fc89f5596eaf6fe1"],"$type":"com.linkedin.voyager.dash.common.text.TextViewModel"},"$recipeTypes":["com.linkedin.188858304ae14097e59518d245c7cc85"],"$type":"com.linkedin.voyager.dash.feed.nav.BusinessServiceAction"},"description":"Courses to develop your employees","$recipeTypes":["com.linkedin.a94ff0919cddaa7056025e9491dc3794"],"$type":"com.linkedin.voyager.dash.feed.nav.BusinessService"},{"businessServiceAction":{"controlName":"nav_business_admin_center","actionTarget":"https://business.linkedin.com/talent-solutions/customer/admin-center","title":{"textDirection":"USER_LOCALE","text":"Admin Center","attributesV2":[],"accessibilityTextAttributesV2":[],"accessibilityText":null,"$recipeTypes":["com.linkedin.566376561ef86249fc89f5596eaf6fe1"],"$type":"com.linkedin.voyager.dash.common.text.TextViewModel"},"$recipeTypes":["com.linkedin.188858304ae14097e59518d245c7cc85"],"$type":"com.linkedin.voyager.dash.feed.nav.BusinessServiceAction"},"description":"Manage billing and account details","$recipeTypes":["com.linkedin.a94ff0919cddaa7056025e9491dc3794"],"$type":"com.linkedin.voyager.dash.feed.nav.BusinessService"}],"productItemsTitle":{"textDirection":"USER_LOCALE","text":"Visit More LinkedIn Products","attributesV2":[],"accessibilityTextAttributesV2":[],"accessibilityText":null,"$recipeTypes":["com.linkedin.566376561ef86249fc89f5596eaf6fe1"],"$type":"com.linkedin.voyager.dash.common.text.TextViewModel"},"businessServicesTitle":"Explore more for business","businessServiceActions":[{"controlName":"nav_business_create_company","actionTarget":"https://www.linkedin.com/company/setup/new/","title":{"textDirection":"USER_LOCALE","text":"Create a Company Page  ","attributesV2":[{"start":22,"length":1,"detailData":{"jobPostingName":null,"hyperlink":null,"profileFamiliarName":null,"color":null,"companyName":null,"icon":"IC_PLUS_16DP","epoch":null,"systemImage":null,"textLink":null,"listItemStyle":null,"groupName":null,"hyperlinkOpenExternally":null,"listStyle":null,"profileFullName":null,"stringFieldReference":null,"learningCourseName":null,"profileMention":null,"style":null,"schoolName":null,"hashtag":null},"$recipeTypes":["com.linkedin.9d1c9f8db73cc268b525043481708082"],"$type":"com.linkedin.voyager.dash.common.text.TextAttribute"}],"accessibilityTextAttributesV2":[],"accessibilityText":null,"$recipeTypes":["com.linkedin.566376561ef86249fc89f5596eaf6fe1"],"$type":"com.linkedin.voyager.dash.common.text.TextViewModel"},"$recipeTypes":["com.linkedin.188858304ae14097e59518d245c7cc85"],"$type":"com.linkedin.voyager.dash.feed.nav.BusinessServiceAction"}],"controlName":"nav_launcher","icon":{"attributes":[{"scalingType":null,"detailData":{"profilePictureWithoutFrame":null,"profilePictureWithRingStatus":null,"companyLogo":null,"icon":"IC_NAV_SMALL_APP_SWITCHER_24DP","systemImage":null,"nonEntityGroupLogo":null,"vectorImage":null,"nonEntityProfessionalEventLogo":null,"profilePicture":null,"imageUrl":null,"professionalEventLogo":null,"nonEntityCompanyLogo":null,"nonEntitySchoolLogo":null,"groupLogo":null,"schoolLogo":null,"ghostImage":null,"nonEntityProfilePicture":null},"tintColor":null,"$recipeTypes":["com.linkedin.e6506ba5f5cc1a9b5124781a3661a7db"],"tapTargets":[],"displayAspectRatio":null,"$type":"com.linkedin.voyager.dash.common.image.ImageAttribute"}],"editableAccessibilityText":false,"actionTarget":null,"accessibilityTextAttributes":[],"totalCount":null,"accessibilityText":null,"$recipeTypes":["com.linkedin.f89edd0cdc11632a12f86e550f2f5e46"],"$type":"com.linkedin.voyager.dash.common.image.ImageViewModel"},"text":"For Business","$recipeTypes":["com.linkedin.f94aca2770a997f8a58dfb7d6154954a"],"$type":"com.linkedin.voyager.dash.feed.nav.AppLauncher"},"meMenu":null,"navItem":null}],"$type":"com.linkedin.voyager.dash.feed.nav.GlobalNav"},"$recipeTypes":["com.linkedin.1f89c60d56117f36dc02eb3d95a6504d"],"$type":"com.linkedin.1f89c60d56117f36dc02eb3d95a6504d"},"extensions":{"webMetadata":{}}},"meta":{"microSchema":{"isGraphQL":true,"version":"2.1","types":{"com.linkedin.dcb85e1ca353361cb2b5f6fa24a1c9af":{"fields":{"groupUrn":{"type":"string"},"vectorImage":{"type":"com.linkedin.ca116024396bd29dd1345ad0998a57d5"},"group":{"resolvedFrom":"groupUrn","type":"com.linkedin.6c29a299f640b6e802d15141c355bd92"}},"baseType":"com.linkedin.voyager.dash.common.image.NonEntityGroupLogo"},"com.linkedin.1e797f68b521546886989869c5b44a2d":{"fields":{"name":{"type":"string"},"logo":{"type":{"union":{"url":"com.linkedin.common.Url","vectorImage":"com.linkedin.ca116024396bd29dd1345ad0998a57d5"}}},"logoResolutionResult":{"derivedFrom":"logo","type":{"union":{"url":"com.linkedin.common.Url","vectorImage":"com.linkedin.ca116024396bd29dd1345ad0998a57d5"}}},"entityUrn":{"type":"com.linkedin.voyager.dash.common.SchoolUrn"}},"baseType":"com.linkedin.voyager.dash.organization.School"},"com.linkedin.fb4511fb6c2df4a805ef662367e15026":{"fields":{"controlName":{"type":"com.linkedin.voyager.dash.common.tracking.ControlName"},"actionTarget":{"type":"com.linkedin.common.Url"},"text":{"type":"com.linkedin.566376561ef86249fc89f5596eaf6fe1"},"quickHelp":{"type":"boolean"}},"baseType":"com.linkedin.voyager.dash.feed.nav.MeItem"},"com.linkedin.f1332bd87cf4dccab8aa4b4583ad44be":{"fields":{"width":{"type":"int"},"fileIdentifyingUrlPathSegment":{"type":"string"},"expiresAt":{"type":"com.linkedin.common.Time"},"height":{"type":"int"}},"baseType":"com.linkedin.common.VectorArtifact"},"com.linkedin.61dcef34ee68ce7a34919062297c038b":{"fields":{"ringContentType":{"type":"com.linkedin.voyager.dash.common.image.RingContentType"},"actionTarget":{"type":"com.linkedin.common.Url"},"preDashEntityUrn":{"type":"com.linkedin.voyager.dash.common.PreDashUrn"},"entityUrn":{"type":"com.linkedin.voyager.dash.common.RingStatusUrn"},"emphasized":{"type":"boolean"}},"baseType":"com.linkedin.voyager.dash.common.image.RingStatus"},"com.linkedin.6c29a299f640b6e802d15141c355bd92":{"fields":{"name":{"type":"string"},"logo":{"type":{"union":{"url":"com.linkedin.common.Url","vectorImage":"com.linkedin.ca116024396bd29dd1345ad0998a57d5"}}},"logoResolutionResult":{"derivedFrom":"logo","type":{"union":{"url":"com.linkedin.common.Url","vectorImage":"com.linkedin.ca116024396bd29dd1345ad0998a57d5"}}},"entityUrn":{"type":"com.linkedin.voyager.dash.common.GroupUrn"}},"baseType":"com.linkedin.voyager.dash.groups.Group"},"com.linkedin.1f89c60d56117f36dc02eb3d95a6504d":{"fields":{"feedDashGlobalNavs":{"type":"com.linkedin.eae2ed6e463973c8e0d55baecd814363"}},"baseType":"com.linkedin.graphql.Query"},"com.linkedin.bb35e914e880d9f9ad79d52bf644c44c":{"fields":{"title":{"type":"string"},"meItemsUnions":{"type":{"array":{"union":{"meItem":"com.linkedin.fb4511fb6c2df4a805ef662367e15026","premiumUpsellSlotUrn":"string"}}}},"meItems":{"resolvedFrom":"meItemsUnions","type":{"array":{"union":{"meItem":"com.linkedin.fb4511fb6c2df4a805ef662367e15026","premiumUpsellSlotUrn":"com.linkedin.6cf7727c816c79567a0564982ee35ef4"}}}}},"baseType":"com.linkedin.voyager.dash.feed.nav.MeGroup"},"com.linkedin.1112232e110fbaf9f6866335ae420a21":{"fields":{"type":{"type":"com.linkedin.voyager.dash.common.text.ListStyleType"},"index":{"type":"int"}},"baseType":"com.linkedin.voyager.dash.common.text.ListItemStyle"},"com.linkedin.bda725ec0f4ddc7380fb44de84e29723":{"fields":{"displayImageWithFrameReferenceUnion":{"type":{"union":{"url":"com.linkedin.common.Url","vectorImage":"com.linkedin.ca116024396bd29dd1345ad0998a57d5"}}},"displayImageReferenceResolutionResult":{"derivedFrom":"displayImageReference","type":{"union":{"url":"com.linkedin.common.Url","vectorImage":"com.linkedin.ca116024396bd29dd1345ad0998a57d5"}}},"displayImageWithFrameReference":{"derivedFrom":"displayImageWithFrameReferenceUnion","type":{"union":{"url":"com.linkedin.common.Url","vectorImage":"com.linkedin.ca116024396bd29dd1345ad0998a57d5"}}},"displayImageReference":{"type":{"union":{"url":"com.linkedin.common.Url","vectorImage":"com.linkedin.ca116024396bd29dd1345ad0998a57d5"}}},"displayImageUrn":{"type":"com.linkedin.common.Urn"}},"baseType":"com.linkedin.voyager.dash.identity.profile.PhotoFilterPicture"},"com.linkedin.b2c03744140a6cd6a2e693c32790988c":{"fields":{"staticIcon":{"type":"com.linkedin.f89edd0cdc11632a12f86e550f2f5e46"},"activeIcon":{"type":"com.linkedin.f89edd0cdc11632a12f86e550f2f5e46"},"hasCoachmark":{"type":"boolean"},"controlName":{"type":"com.linkedin.voyager.dash.common.tracking.ControlName"},"showIconBorder":{"type":"boolean"},"text":{"type":"string"},"actionTarget":{"type":"com.linkedin.common.Url"},"hasPaid":{"type":"boolean"}},"baseType":"com.linkedin.voyager.dash.feed.nav.NavElement"},"com.linkedin.371c4584dc45b7919c4eb6ed6697e8cf":{"fields":{"professionalEvent":{"resolvedFrom":"professionalEventUrn","type":"com.linkedin.0d2a5c07095246fc3e756f3e922d8aa0"},"vectorImage":{"type":"com.linkedin.ca116024396bd29dd1345ad0998a57d5"},"professionalEventUrn":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.common.image.NonEntityProfessionalEventLogo"},"com.linkedin.1c710707068d7b33bcbee05036904a3b":{"fields":{"ringStatus":{"type":"com.linkedin.61dcef34ee68ce7a34919062297c038b"},"profileUrn":{"type":"string"},"vectorImage":{"type":"com.linkedin.ca116024396bd29dd1345ad0998a57d5"},"profile":{"resolvedFrom":"profileUrn","type":"com.linkedin.c8b3cc209fc06180b17a4ffb72120344"}},"baseType":"com.linkedin.voyager.dash.common.image.NonEntityProfilePicture"},"com.linkedin.8b2657e6d402c000382bd65eb94ba628":{"fields":{"image":{"type":"com.linkedin.f89edd0cdc11632a12f86e550f2f5e46"},"socialProofInsight":{"type":"com.linkedin.ae72a2e8f27d029e18d8e6d4350b0273"},"subtext":{"type":"com.linkedin.566376561ef86249fc89f5596eaf6fe1"},"ctaText":{"type":"com.linkedin.566376561ef86249fc89f5596eaf6fe1"},"upsellOrderOrigin":{"type":"string"},"controlName":{"type":"com.linkedin.voyager.dash.common.tracking.ControlName"},"actionUrl":{"type":"com.linkedin.common.Url"},"legoTrackingToken":{"type":"string"},"title":{"type":"com.linkedin.566376561ef86249fc89f5596eaf6fe1"},"cancelCta":{"type":"com.linkedin.513969f6493543c4ad7f04b46c3b67f6"},"globalLegoTrackingToken":{"type":"string"},"promotionLegoTrackingToken":{"type":"string"},"funnelCommonHeader":{"type":"com.linkedin.f58e91c23e63f52ceb12bd0fab3eb07b"},"fireDiscoveryImpressionEvent":{"type":"boolean"},"subtitle":{"type":"com.linkedin.566376561ef86249fc89f5596eaf6fe1"},"dismissible":{"type":"boolean"},"layoutStyle":{"type":"com.linkedin.voyager.dash.premium.PremiumUpsellLayoutStyle"},"ctaButtonSecondaryEmphasizedTheme":{"type":"boolean"}},"baseType":"com.linkedin.voyager.dash.premium.PremiumUpsellCard"},"com.linkedin.ae72a2e8f27d029e18d8e6d4350b0273":{"fields":{"image":{"type":"com.linkedin.f89edd0cdc11632a12f86e550f2f5e46"},"text":{"type":"com.linkedin.566376561ef86249fc89f5596eaf6fe1"}},"baseType":"com.linkedin.voyager.dash.common.ux.InsightViewModel"},"com.linkedin.d66f0cb7963e0345d3ff893927e60c88":{"fields":{"tapTargetEntityUnion":{"type":{"union":{"companyUrn":"string","profileUrn":"string"}}},"stickerLinkViewUnion":{"type":{"union":{"mediumTemplate":"com.linkedin.637f09655a21b1df5a03c68636fd2d18","largeTemplate":"com.linkedin.1bcf1aba50ed23a46d11fe327f319b18","smallTemplate":"com.linkedin.f223abd9a48757f8395ac497a91593f3"}}},"firstCornerXOffsetPercentage":{"type":"float"},"stickerLinkTemplateSize":{"type":"com.linkedin.voyager.dash.common.media.StickerLinkTemplateSize"},"type":{"type":"com.linkedin.voyager.dash.common.TapTargetAttributeType"},"thirdCornerYOffsetPercentage":{"type":"float"},"url":{"type":"com.linkedin.common.Url"},"stickerLinkView":{"derivedFrom":"stickerLinkViewUnion","type":{"union":{"mediumTemplate":"com.linkedin.637f09655a21b1df5a03c68636fd2d18","largeTemplate":"com.linkedin.1bcf1aba50ed23a46d11fe327f319b18","smallTemplate":"com.linkedin.f223abd9a48757f8395ac497a91593f3"}}},"urn":{"type":"com.linkedin.common.Urn"},"thirdCornerXOffsetPercentage":{"type":"float"},"secondCornerYOffsetPercentage":{"type":"float"},"fourthCornerXOffsetPercentage":{"type":"float"},"tapTargetEntity":{"resolvedFrom":"tapTargetEntityUnion","type":{"union":{"companyUrn":"com.linkedin.2b5530b88c12ba36048beee3aca27737","profileUrn":"com.linkedin.c8b3cc209fc06180b17a4ffb72120344"}}},"firstCornerYOffsetPercentage":{"type":"float"},"rank":{"type":"int"},"untaggable":{"type":"boolean"},"fourthCornerYOffsetPercentage":{"type":"float"},"text":{"type":"string"},"secondCornerXOffsetPercentage":{"type":"float"}},"baseType":"com.linkedin.voyager.dash.common.TapTarget"},"com.linkedin.1bcf1aba50ed23a46d11fe327f319b18":{"fields":{"image":{"derivedFrom":"imageUnion","type":{"union":{"url":"com.linkedin.common.Url","vectorImage":"com.linkedin.ca116024396bd29dd1345ad0998a57d5"}}},"nameSupplementaryInfo":{"type":"com.linkedin.566376561ef86249fc89f5596eaf6fe1"},"footerText":{"type":"com.linkedin.566376561ef86249fc89f5596eaf6fe1"},"imageUnion":{"type":{"union":{"url":"com.linkedin.common.Url","vectorImage":"com.linkedin.ca116024396bd29dd1345ad0998a57d5"}}},"backgroundImage":{"derivedFrom":"backgroundImageUnion","type":{"union":{"url":"com.linkedin.common.Url","vectorImage":"com.linkedin.ca116024396bd29dd1345ad0998a57d5"}}},"subHeadline":{"type":"com.linkedin.566376561ef86249fc89f5596eaf6fe1"},"name":{"type":"com.linkedin.566376561ef86249fc89f5596eaf6fe1"},"insightText":{"type":"com.linkedin.566376561ef86249fc89f5596eaf6fe1"},"headline":{"type":"com.linkedin.566376561ef86249fc89f5596eaf6fe1"},"backgroundImageUnion":{"type":{"union":{"url":"com.linkedin.common.Url","vectorImage":"com.linkedin.ca116024396bd29dd1345ad0998a57d5"}}}},"baseType":"com.linkedin.voyager.dash.common.media.StickerLinkLargeTemplateView"},"com.linkedin.eae2ed6e463973c8e0d55baecd814363":{"fields":{"logo":{"type":"com.linkedin.4973670c8e07d9bf8572c4c7f8658e2b"},"primaryItemsUnions":{"type":{"array":{"union":{"appLauncher":"com.linkedin.f94aca2770a997f8a58dfb7d6154954a","navItem":"com.linkedin.c7f876e8e9f8a631b2896f62a656ce64","meMenu":"com.linkedin.73113dd58d976ff22a5e4aa2ca35d21d"}}}},"spotlight":{"type":"com.linkedin.c7f876e8e9f8a631b2896f62a656ce64"},"primaryItems":{"derivedFrom":"primaryItemsUnions","type":{"array":{"union":{"appLauncher":"com.linkedin.f94aca2770a997f8a58dfb7d6154954a","navItem":"com.linkedin.c7f876e8e9f8a631b2896f62a656ce64","meMenu":"com.linkedin.73113dd58d976ff22a5e4aa2ca35d21d"}}}}},"baseType":"com.linkedin.voyager.dash.feed.nav.GlobalNav"},"com.linkedin.9d1c9f8db73cc268b525043481708082":{"fields":{"detailData":{"resolvedFrom":"detailDataUnion","type":{"union":{"jobPostingName":"com.linkedin.1febda95b00bfc8c540a7af0dc75a88a","profileFamiliarName":"com.linkedin.c8b3cc209fc06180b17a4ffb72120344","hyperlink":"com.linkedin.common.Url","color":"com.linkedin.voyager.dash.common.text.TextColor","companyName":"com.linkedin.2b5530b88c12ba36048beee3aca27737","icon":"com.linkedin.voyager.dash.common.ArtDecoIconName","epoch":"com.linkedin.feccefa74b57bf711c74d0b09805c9f6","systemImage":"com.linkedin.voyager.dash.common.SystemImageName","textLink":"com.linkedin.72c1c1091a60c5a77aaadac057a84b54","listItemStyle":"com.linkedin.1112232e110fbaf9f6866335ae420a21","listStyle":"com.linkedin.voyager.dash.common.text.ListStyleType","groupName":"com.linkedin.6c29a299f640b6e802d15141c355bd92","hyperlinkOpenExternally":"com.linkedin.common.Url","stringFieldReference":"com.linkedin.65deee184b56ce21c8d7d30fec6acfe4","profileFullName":"com.linkedin.c8b3cc209fc06180b17a4ffb72120344","learningCourseName":"com.linkedin.44f2985bd25b663c32c3c4d4ec070248","profileMention":"com.linkedin.c8b3cc209fc06180b17a4ffb72120344","style":"com.linkedin.voyager.dash.common.text.TextStyle","schoolName":"com.linkedin.1e797f68b521546886989869c5b44a2d","hashtag":"com.linkedin.10f415928dc4647be1fe1eebc72fe277"}}},"length":{"type":"int"},"start":{"type":"int"},"detailDataUnion":{"type":{"union":{"jobPostingName":"string","profileFamiliarName":"string","hyperlink":"com.linkedin.common.Url","color":"com.linkedin.voyager.dash.common.text.TextColor","companyName":"string","icon":"com.linkedin.voyager.dash.common.ArtDecoIconName","epoch":"com.linkedin.feccefa74b57bf711c74d0b09805c9f6","systemImage":"com.linkedin.voyager.dash.common.SystemImageName","textLink":"com.linkedin.72c1c1091a60c5a77aaadac057a84b54","listItemStyle":"com.linkedin.1112232e110fbaf9f6866335ae420a21","listStyle":"com.linkedin.voyager.dash.common.text.ListStyleType","groupName":"string","hyperlinkOpenExternally":"com.linkedin.common.Url","stringFieldReference":"com.linkedin.65deee184b56ce21c8d7d30fec6acfe4","profileFullName":"string","learningCourseName":"string","profileMention":"string","style":"com.linkedin.voyager.dash.common.text.TextStyle","schoolName":"string","hashtag":"string"}}}},"baseType":"com.linkedin.voyager.dash.common.text.TextAttribute"},"com.linkedin.513969f6493543c4ad7f04b46c3b67f6":{"fields":{"controlName":{"type":"com.linkedin.voyager.dash.common.tracking.ControlName"},"appearance":{"type":"com.linkedin.4ef51bc3dc729207a4f87aa1ac1b37cf"},"openExternally":{"type":"boolean"},"accessibilityText":{"type":"string"},"disable":{"type":"boolean"},"url":{"type":"com.linkedin.common.Url"}},"baseType":"com.linkedin.voyager.dash.premium.PremiumButton"},"com.linkedin.73113dd58d976ff22a5e4aa2ca35d21d":{"fields":{"viewProfileText":{"type":"string"},"controlName":{"type":"com.linkedin.voyager.dash.common.tracking.ControlName"},"text":{"type":"string"},"meGroups":{"type":{"array":"com.linkedin.bb35e914e880d9f9ad79d52bf644c44c"}},"profileUrn":{"type":"string"},"profile":{"resolvedFrom":"profileUrn","type":"com.linkedin.c8b3cc209fc06180b17a4ffb72120344"}},"baseType":"com.linkedin.voyager.dash.feed.nav.MeMenu"},"com.linkedin.d6038751fe26f2a90ed06dcfd5acb15e":{"fields":{"memberDistance":{"type":"com.linkedin.voyager.dash.relationships.NoConnectionMemberDistance"}},"baseType":"com.linkedin.voyager.dash.relationships.NoConnection"},"com.linkedin.68d330957951bfe0ba509507962b88b5":{"fields":{"entityUrn":{"type":"com.linkedin.voyager.dash.common.ConnectionUrn"}},"baseType":"com.linkedin.voyager.dash.relationships.Connection"},"com.linkedin.f223abd9a48757f8395ac497a91593f3":{"fields":{"name":{"type":"com.linkedin.566376561ef86249fc89f5596eaf6fe1"},"imageUnion":{"type":{"union":{"url":"com.linkedin.common.Url","vectorImage":"com.linkedin.ca116024396bd29dd1345ad0998a57d5"}}},"image":{"derivedFrom":"imageUnion","type":{"union":{"url":"com.linkedin.common.Url","vectorImage":"com.linkedin.ca116024396bd29dd1345ad0998a57d5"}}}},"baseType":"com.linkedin.voyager.dash.common.media.StickerLinkSmallTemplateView"},"com.linkedin.72c1c1091a60c5a77aaadac057a84b54":{"fields":{"controlName":{"type":"com.linkedin.voyager.dash.common.tracking.ControlName"},"viewingBehavior":{"type":"com.linkedin.voyager.dash.common.ux.UrlViewingBehavior"},"sponsoredUrlAttributes":{"type":"com.linkedin.8fbc11c3b77aca2f1a34ab7a34600da7"},"url":{"type":"com.linkedin.common.Url"}},"baseType":"com.linkedin.voyager.dash.common.text.TextLink"},"com.linkedin.72e4cd594b8572e0ec9ddb034eafc088":{"fields":{"companyUrn":{"type":"string"},"company":{"resolvedFrom":"companyUrn","type":"com.linkedin.2b5530b88c12ba36048beee3aca27737"},"vectorImage":{"type":"com.linkedin.ca116024396bd29dd1345ad0998a57d5"}},"baseType":"com.linkedin.voyager.dash.common.image.NonEntityCompanyLogo"},"com.linkedin.f58e91c23e63f52ceb12bd0fab3eb07b":{"fields":{"utype":{"type":"string"},"referenceId":{"type":"com.linkedin.common.TrackingId"}},"baseType":"com.linkedin.voyager.dash.premium.PremiumFunnelCommonHeader"},"com.linkedin.4973670c8e07d9bf8572c4c7f8658e2b":{"fields":{"controlName":{"type":"com.linkedin.voyager.dash.common.tracking.ControlName"},"logo":{"type":"com.linkedin.f89edd0cdc11632a12f86e550f2f5e46"},"hoverText":{"type":"com.linkedin.566376561ef86249fc89f5596eaf6fe1"}},"baseType":"com.linkedin.voyager.dash.feed.nav.BrandingLogo"},"com.linkedin.8fbc11c3b77aca2f1a34ab7a34600da7":{"fields":{"impressionId":{"type":"string"},"unwrappedUrlDomain":{"type":"string"},"creativeId":{"type":"string"},"campaignId":{"type":"string"},"memberId":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.common.sponsored.SponsoredUrlAttributes"},"com.linkedin.72ff698dfdea20e2538931532a528e81":{"fields":{"entityUrn":{"type":"com.linkedin.voyager.dash.common.MemberRelationshipUrn"},"memberRelationshipUnion":{"type":{"union":{"self":"com.linkedin.797f28906de48c13543c3ff29a412882","connection":"com.linkedin.68d330957951bfe0ba509507962b88b5","noConnection":"com.linkedin.d6038751fe26f2a90ed06dcfd5acb15e"}}},"memberRelationship":{"derivedFrom":"memberRelationshipUnion","type":{"union":{"self":"com.linkedin.797f28906de48c13543c3ff29a412882","connection":"com.linkedin.68d330957951bfe0ba509507962b88b5","noConnection":"com.linkedin.d6038751fe26f2a90ed06dcfd5acb15e"}}}},"baseType":"com.linkedin.voyager.dash.relationships.MemberRelationship"},"com.linkedin.6cf7727c816c79567a0564982ee35ef4":{"fields":{"entityUrn":{"type":"com.linkedin.voyager.dash.common.PremiumUpsellSlotUrn"},"upsellCard":{"type":"com.linkedin.8b2657e6d402c000382bd65eb94ba628"}},"baseType":"com.linkedin.voyager.dash.premium.PremiumUpsellSlotContent"},"com.linkedin.566376561ef86249fc89f5596eaf6fe1":{"fields":{"textDirection":{"type":"com.linkedin.voyager.dash.common.text.TextDirection"},"attributesV2":{"type":{"array":"com.linkedin.9d1c9f8db73cc268b525043481708082"}},"text":{"type":"string"},"accessibilityTextAttributesV2":{"type":{"array":"com.linkedin.9d1c9f8db73cc268b525043481708082"}},"accessibilityText":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.common.text.TextViewModel"},"com.linkedin.f89edd0cdc11632a12f86e550f2f5e46":{"fields":{"editableAccessibilityText":{"type":"boolean"},"attributes":{"type":{"array":"com.linkedin.e6506ba5f5cc1a9b5124781a3661a7db"}},"actionTarget":{"type":"com.linkedin.common.Url"},"accessibilityTextAttributes":{"type":{"array":"com.linkedin.9d1c9f8db73cc268b525043481708082"}},"totalCount":{"type":"int"},"accessibilityText":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.common.image.ImageViewModel"},"com.linkedin.797f28906de48c13543c3ff29a412882":{"fields":{},"baseType":"com.linkedin.restli.common.EmptyRecord"},"com.linkedin.0d2a5c07095246fc3e756f3e922d8aa0":{"fields":{"logoImage":{"type":{"union":{"url":"com.linkedin.common.Url","vectorImage":"com.linkedin.ca116024396bd29dd1345ad0998a57d5"}}},"logoImageResolutionResult":{"derivedFrom":"logoImage","type":{"union":{"url":"com.linkedin.common.Url","vectorImage":"com.linkedin.ca116024396bd29dd1345ad0998a57d5"}}},"entityUrn":{"type":"com.linkedin.voyager.dash.common.ProfessionalEventUrn"}},"baseType":"com.linkedin.voyager.dash.events.ProfessionalEvent"},"com.linkedin.2b5530b88c12ba36048beee3aca27737":{"fields":{"followingStateUrn":{"type":"string"},"entityUrn":{"type":"com.linkedin.voyager.dash.common.CompanyUrn"},"followingState":{"resolvedFrom":"followingStateUrn","type":"com.linkedin.98f5f0b76629959525795b08bdfbcc26"},"name":{"type":"string"},"logo":{"type":{"union":{"url":"com.linkedin.common.Url","vectorImage":"com.linkedin.ca116024396bd29dd1345ad0998a57d5"}}},"logoResolutionResult":{"derivedFrom":"logo","type":{"union":{"url":"com.linkedin.common.Url","vectorImage":"com.linkedin.ca116024396bd29dd1345ad0998a57d5"}}},"universalName":{"type":"string"},"url":{"type":"com.linkedin.common.Url"}},"baseType":"com.linkedin.voyager.dash.organization.Company"},"com.linkedin.feccefa74b57bf711c74d0b09805c9f6":{"fields":{"type":{"type":"com.linkedin.voyager.dash.common.text.EpochFormat"},"epochAt":{"type":"com.linkedin.common.Time"}},"baseType":"com.linkedin.voyager.dash.common.text.EpochTime"},"com.linkedin.1febda95b00bfc8c540a7af0dc75a88a":{"fields":{"title":{"type":"string"},"entityUrn":{"type":"com.linkedin.voyager.dash.common.JobPostingUrn"}},"baseType":"com.linkedin.voyager.dash.jobs.JobPosting"},"com.linkedin.65deee184b56ce21c8d7d30fec6acfe4":{"fields":{"urn":{"type":"com.linkedin.common.Urn"},"modelName":{"type":"string"},"fieldName":{"type":"string"},"value":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.common.StringFieldReference"},"com.linkedin.98f5f0b76629959525795b08bdfbcc26":{"fields":{"followerCount":{"type":"long"},"entityUrn":{"type":"com.linkedin.voyager.dash.common.FollowingStateUrn"}},"baseType":"com.linkedin.voyager.dash.feed.FollowingState"},"com.linkedin.188858304ae14097e59518d245c7cc85":{"fields":{"title":{"type":"com.linkedin.566376561ef86249fc89f5596eaf6fe1"},"controlName":{"type":"com.linkedin.voyager.dash.common.tracking.ControlName"},"actionTarget":{"type":"com.linkedin.common.Url"}},"baseType":"com.linkedin.voyager.dash.feed.nav.BusinessServiceAction"},"com.linkedin.550a9af095fffdd7fc0f6ceb8743a141":{"fields":{"originalImageUrl":{"type":"com.linkedin.common.Url"},"originalHeight":{"type":"int"},"originalWidth":{"type":"int"},"url":{"type":"com.linkedin.common.Url"}},"baseType":"com.linkedin.voyager.dash.common.ImageUrl"},"com.linkedin.44f2985bd25b663c32c3c4d4ec070248":{"fields":{"title":{"type":"string"},"entityUrn":{"type":"com.linkedin.voyager.dash.common.LearningCourseUrn"}},"baseType":"com.linkedin.voyager.dash.learning.LearningCourse"},"com.linkedin.10f415928dc4647be1fe1eebc72fe277":{"fields":{"entityUrn":{"type":"com.linkedin.voyager.dash.common.HashtagUrn"},"trackingUrn":{"type":"com.linkedin.common.Urn"},"actionTarget":{"type":"com.linkedin.common.Url"}},"baseType":"com.linkedin.voyager.dash.feed.Hashtag"},"com.linkedin.c7f876e8e9f8a631b2896f62a656ce64":{"fields":{"type":{"type":"com.linkedin.voyager.dash.feed.nav.NavType"},"navContent":{"resolvedFrom":"navContentUnion","type":{"union":{"premiumUpsellSlotUrn":"com.linkedin.6cf7727c816c79567a0564982ee35ef4","navElement":"com.linkedin.b2c03744140a6cd6a2e693c32790988c"}}},"navContentUnion":{"type":{"union":{"premiumUpsellSlotUrn":"string","navElement":"com.linkedin.b2c03744140a6cd6a2e693c32790988c"}}}},"baseType":"com.linkedin.voyager.dash.feed.nav.NavItem"},"com.linkedin.96db794fa6a744844b487cfa3e195632":{"fields":{"school":{"resolvedFrom":"schoolUrn","type":"com.linkedin.1e797f68b521546886989869c5b44a2d"},"vectorImage":{"type":"com.linkedin.ca116024396bd29dd1345ad0998a57d5"},"schoolUrn":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.common.image.NonEntitySchoolLogo"},"com.linkedin.f94aca2770a997f8a58dfb7d6154954a":{"fields":{"showDivider":{"type":"boolean"},"productItems":{"type":{"array":"com.linkedin.c7f876e8e9f8a631b2896f62a656ce64"}},"productItemsTitle":{"type":"com.linkedin.566376561ef86249fc89f5596eaf6fe1"},"businessServices":{"type":{"array":"com.linkedin.a94ff0919cddaa7056025e9491dc3794"}},"businessServicesTitle":{"type":"string"},"icon":{"type":"com.linkedin.f89edd0cdc11632a12f86e550f2f5e46"},"controlName":{"type":"com.linkedin.voyager.dash.common.tracking.ControlName"},"businessServiceActions":{"type":{"array":"com.linkedin.188858304ae14097e59518d245c7cc85"}},"text":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.feed.nav.AppLauncher"},"com.linkedin.a94ff0919cddaa7056025e9491dc3794":{"fields":{"businessServiceAction":{"type":"com.linkedin.188858304ae14097e59518d245c7cc85"},"description":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.feed.nav.BusinessService"},"com.linkedin.637f09655a21b1df5a03c68636fd2d18":{"fields":{"name":{"type":"com.linkedin.566376561ef86249fc89f5596eaf6fe1"},"image":{"derivedFrom":"imageUnion","type":{"union":{"url":"com.linkedin.common.Url","vectorImage":"com.linkedin.ca116024396bd29dd1345ad0998a57d5"}}},"nameSupplementaryInfo":{"type":"com.linkedin.566376561ef86249fc89f5596eaf6fe1"},"headline":{"type":"com.linkedin.566376561ef86249fc89f5596eaf6fe1"},"imageUnion":{"type":{"union":{"url":"com.linkedin.common.Url","vectorImage":"com.linkedin.ca116024396bd29dd1345ad0998a57d5"}}},"subHeadline":{"type":"com.linkedin.566376561ef86249fc89f5596eaf6fe1"}},"baseType":"com.linkedin.voyager.dash.common.media.StickerLinkMediumTemplateView"},"com.linkedin.c8b3cc209fc06180b17a4ffb72120344":{"fields":{"profilePicture":{"type":"com.linkedin.bda725ec0f4ddc7380fb44de84e29723"},"lastName":{"type":"string"},"firstName":{"type":"string"},"entityUrn":{"type":"com.linkedin.voyager.dash.common.ProfileUrn"},"memberRelationship":{"type":"com.linkedin.72ff698dfdea20e2538931532a528e81","isInjection":true},"ringStatus":{"type":"com.linkedin.61dcef34ee68ce7a34919062297c038b","isInjection":true},"headline":{"type":"string"},"publicIdentifier":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.identity.profile.Profile"},"com.linkedin.4ef51bc3dc729207a4f87aa1ac1b37cf":{"fields":{"premiumStyle":{"type":"boolean"},"size":{"type":"com.linkedin.voyager.dash.common.ux.button.ButtonSize"},"icon":{"type":"com.linkedin.voyager.dash.common.SystemImageName"},"text":{"type":"string"},"iconAfterText":{"type":"boolean"},"emphasize":{"type":"boolean"},"category":{"type":"com.linkedin.voyager.dash.common.ux.button.ButtonCategory"}},"baseType":"com.linkedin.voyager.dash.common.ux.button.ButtonAppearance"},"com.linkedin.e6506ba5f5cc1a9b5124781a3661a7db":{"fields":{"detailData":{"resolvedFrom":"detailDataUnion","type":{"union":{"profilePictureWithoutFrame":"com.linkedin.c8b3cc209fc06180b17a4ffb72120344","profilePictureWithRingStatus":"com.linkedin.c8b3cc209fc06180b17a4ffb72120344","companyLogo":"com.linkedin.2b5530b88c12ba36048beee3aca27737","icon":"com.linkedin.voyager.dash.common.ArtDecoIconName","systemImage":"com.linkedin.voyager.dash.common.SystemImageName","nonEntityGroupLogo":"com.linkedin.dcb85e1ca353361cb2b5f6fa24a1c9af","vectorImage":"com.linkedin.ca116024396bd29dd1345ad0998a57d5","nonEntityProfessionalEventLogo":"com.linkedin.371c4584dc45b7919c4eb6ed6697e8cf","profilePicture":"com.linkedin.c8b3cc209fc06180b17a4ffb72120344","imageUrl":"com.linkedin.550a9af095fffdd7fc0f6ceb8743a141","professionalEventLogo":"com.linkedin.0d2a5c07095246fc3e756f3e922d8aa0","nonEntitySchoolLogo":"com.linkedin.96db794fa6a744844b487cfa3e195632","nonEntityCompanyLogo":"com.linkedin.72e4cd594b8572e0ec9ddb034eafc088","schoolLogo":"com.linkedin.1e797f68b521546886989869c5b44a2d","groupLogo":"com.linkedin.6c29a299f640b6e802d15141c355bd92","ghostImage":"com.linkedin.voyager.dash.common.image.GhostImageType","nonEntityProfilePicture":"com.linkedin.1c710707068d7b33bcbee05036904a3b"}}},"tintColor":{"type":"com.linkedin.voyager.dash.common.SystemImageTintColor"},"detailDataUnion":{"type":{"union":{"profilePictureWithoutFrame":"string","profilePictureWithRingStatus":"string","companyLogo":"string","icon":"com.linkedin.voyager.dash.common.ArtDecoIconName","systemImage":"com.linkedin.voyager.dash.common.SystemImageName","nonEntityGroupLogo":"com.linkedin.dcb85e1ca353361cb2b5f6fa24a1c9af","vectorImage":"com.linkedin.ca116024396bd29dd1345ad0998a57d5","nonEntityProfessionalEventLogo":"com.linkedin.371c4584dc45b7919c4eb6ed6697e8cf","profilePicture":"string","imageUrl":"com.linkedin.550a9af095fffdd7fc0f6ceb8743a141","professionalEventLogo":"string","nonEntitySchoolLogo":"com.linkedin.96db794fa6a744844b487cfa3e195632","nonEntityCompanyLogo":"com.linkedin.72e4cd594b8572e0ec9ddb034eafc088","schoolLogo":"string","groupLogo":"string","ghostImage":"com.linkedin.voyager.dash.common.image.GhostImageType","nonEntityProfilePicture":"com.linkedin.1c710707068d7b33bcbee05036904a3b"}}},"tapTargets":{"type":{"array":"com.linkedin.d66f0cb7963e0345d3ff893927e60c88"}},"scalingType":{"type":"com.linkedin.voyager.dash.common.image.ScalingType"},"displayAspectRatio":{"type":"double"}},"baseType":"com.linkedin.voyager.dash.common.image.ImageAttribute"},"com.linkedin.ca116024396bd29dd1345ad0998a57d5":{"fields":{"attribution":{"type":"string"},"digitalmediaAsset":{"type":"com.linkedin.common.DigitalmediaAssetUrn"},"rootUrl":{"type":"string"},"artifacts":{"type":{"array":"com.linkedin.f1332bd87cf4dccab8aa4b4583ad44be"}}},"baseType":"com.linkedin.common.VectorImage"}}}},"included":[{"firstName":"Naim Iskandar","lastName":"Zahari","profilePicture":{"displayImageReferenceResolutionResult":{"url":null,"vectorImage":{"digitalmediaAsset":null,"attribution":null,"$recipeTypes":["com.linkedin.ca116024396bd29dd1345ad0998a57d5"],"artifacts":[{"width":99,"$recipeTypes":["com.linkedin.f1332bd87cf4dccab8aa4b4583ad44be"],"fileIdentifyingUrlPathSegment":"100_100/0/1605279012830?e=1706745600&amp;v=beta&amp;t=GoGDD2DE3nGXji1ktdhEWgWX0uNRNd93omIm9L4PTBM","expiresAt":1706745600000,"height":100,"$type":"com.linkedin.common.VectorArtifact"},{"width":199,"$recipeTypes":["com.linkedin.f1332bd87cf4dccab8aa4b4583ad44be"],"fileIdentifyingUrlPathSegment":"200_200/0/1605279012830?e=1706745600&amp;v=beta&amp;t=KxJr8NDL0QX2hH6Hgu-7kOudbuCrhYafGAmjrDIUX6g","expiresAt":1706745600000,"height":200,"$type":"com.linkedin.common.VectorArtifact"},{"width":399,"$recipeTypes":["com.linkedin.f1332bd87cf4dccab8aa4b4583ad44be"],"fileIdentifyingUrlPathSegment":"400_400/0/1605279012830?e=1706745600&amp;v=beta&amp;t=MEX6KTkEsBN1Iwy4RYn8IoyQikzi3wK-ws0Ij5Qr478","expiresAt":1706745600000,"height":400,"$type":"com.linkedin.common.VectorArtifact"},{"width":670,"$recipeTypes":["com.linkedin.f1332bd87cf4dccab8aa4b4583ad44be"],"fileIdentifyingUrlPathSegment":"800_800/0/1605279012830?e=1706745600&amp;v=beta&amp;t=72OsosG2vCVJHRjJ9BEqeLcfgJkzqOn-4Eq5tGu4964","expiresAt":1706745600000,"height":670,"$type":"com.linkedin.common.VectorArtifact"}],"rootUrl":"https://media.licdn.com/dms/image/C5603AQEkRciOp6B6dA/profile-displayphoto-shrink_","$type":"com.linkedin.common.VectorImage"}},"displayImageWithFrameReference":null,"$recipeTypes":["com.linkedin.bda725ec0f4ddc7380fb44de84e29723"],"$type":"com.linkedin.voyager.dash.identity.profile.PhotoFilterPicture"},"entityUrn":"urn:li:fsd_profile:ACoAAC6BPkEBwSD4NTQ7khSJbuAxrRMYhVSwmBo","headline":"Performance Analyst | Aspiring Data Scientist | Data Enthusiast | Game Programmer","publicIdentifier":"naimiskandar22","$recipeTypes":["com.linkedin.c8b3cc209fc06180b17a4ffb72120344"],"$type":"com.linkedin.voyager.dash.identity.profile.Profile"}]}
</code>
<code style="display: none" id="datalet-bpr-guid-15912213" aria-hidden="true">
  {"request":"/voyager/api/graphql?includeWebMetadata\u003Dtrue\u0026variables\u003D()\u0026queryId\u003DvoyagerFeedDashGlobalNavs.604cc295a50aa27ae69147afac5ff0e5","status":200,"body":"bpr-guid-15912213","method":"GET","headers":{"x-li-uuid":"AAYLE+vtCEZ6Amp7P3S1tA\u003D\u003D"}}
</code>
<img src="data:image/gif;base64,R0lGODlhAQABAIAAAAAAAP///yH5BAEAAAAALAAAAAABAAEAAAIBRAA7" style="display: none" class="datalet-bpr-guid-15912213" aria-hidden="true"><code style="display: none" id="bpr-guid-15912214" aria-hidden="true">
  {"data":{"data":{"premiumDashFeatureAccessByFeatureAccessTypes":{"*elements":["urn:li:fsd_featureAccess:CAN_ACCESS_SALES_NAV_BADGE","urn:li:fsd_featureAccess:CAN_ACCESS_ADVERTISE_BADGE"],"$recipeTypes":["com.linkedin.3fad2247165582aa95a2690797d7e0ab"],"$type":"com.linkedin.restli.common.CollectionResponse"},"$recipeTypes":["com.linkedin.6488012243094b4ef82d764d10e79d45"],"$type":"com.linkedin.6488012243094b4ef82d764d10e79d45"},"extensions":{"webMetadata":{}}},"meta":{"microSchema":{"isGraphQL":true,"version":"2.1","types":{"com.linkedin.3fad2247165582aa95a2690797d7e0ab":{"fields":{"elements":{"type":{"array":"com.linkedin.9e452672002943dc9a1521bcc7ad23cb"}}},"baseType":"com.linkedin.restli.common.CollectionResponse"},"com.linkedin.9e452672002943dc9a1521bcc7ad23cb":{"fields":{"entityUrn":{"type":"com.linkedin.voyager.dash.common.FeatureAccessUrn"},"hasAccess":{"type":"boolean"},"featureAccessType":{"type":"com.linkedin.voyager.dash.premium.FeatureAccessType"}},"baseType":"com.linkedin.voyager.dash.premium.FeatureAccess"},"com.linkedin.6488012243094b4ef82d764d10e79d45":{"fields":{"premiumDashFeatureAccessByFeatureAccessTypes":{"type":"com.linkedin.3fad2247165582aa95a2690797d7e0ab"}},"baseType":"com.linkedin.graphql.Query"}}}},"included":[{"featureAccessType":"CAN_ACCESS_SALES_NAV_BADGE","entityUrn":"urn:li:fsd_featureAccess:CAN_ACCESS_SALES_NAV_BADGE","hasAccess":false,"$recipeTypes":["com.linkedin.9e452672002943dc9a1521bcc7ad23cb"],"$type":"com.linkedin.voyager.dash.premium.FeatureAccess"},{"featureAccessType":"CAN_ACCESS_ADVERTISE_BADGE","entityUrn":"urn:li:fsd_featureAccess:CAN_ACCESS_ADVERTISE_BADGE","hasAccess":false,"$recipeTypes":["com.linkedin.9e452672002943dc9a1521bcc7ad23cb"],"$type":"com.linkedin.voyager.dash.premium.FeatureAccess"}]}
</code>
<code style="display: none" id="datalet-bpr-guid-15912214" aria-hidden="true">
  {"request":"/voyager/api/graphql?includeWebMetadata\u003Dtrue\u0026variables\u003D(featureAccessTypes:List(CAN_ACCESS_SALES_NAV_BADGE,CAN_ACCESS_ADVERTISE_BADGE))\u0026queryId\u003DvoyagerPremiumDashFeatureAccess.c87b20dac35795f9920f2a8072fd7af5","status":200,"body":"bpr-guid-15912214","method":"GET","headers":{"x-li-uuid":"AAYLE+vtCEZ6Amp7P3S1tA\u003D\u003D"}}
</code>
<img src="data:image/gif;base64,R0lGODlhAQABAIAAAAAAAP///yH5BAEAAAAALAAAAAABAAEAAAIBRAA7" style="display: none" class="datalet-bpr-guid-15912214" aria-hidden="true"><code style="display: none" id="bpr-guid-15912215" aria-hidden="true">
  {"data":{"entityUrn":"urn:li:collectionResponse:7ZvAwvPwYldk7Tvl3jXgC4ymrtxthgc+3EdE7ncMd98=","elements":[{"count":0,"tab":"MY_NETWORK","$type":"com.linkedin.voyager.common.communications.TabBadge"},{"count":0,"tab":"MESSAGING","$type":"com.linkedin.voyager.common.communications.TabBadge"},{"count":0,"tab":"NOTIFICATIONS","$type":"com.linkedin.voyager.common.communications.TabBadge"}],"paging":{"count":10,"start":0,"links":[]},"$type":"com.linkedin.restli.common.CollectionResponse"},"included":[]}
</code>
<code style="display: none" id="datalet-bpr-guid-15912215" aria-hidden="true">
  {"request":"/voyager/api/voyagerCommunicationsTabBadges?q\u003DtabBadges\u0026countFrom\u003D0","status":200,"body":"bpr-guid-15912215","method":"GET","headers":{"x-li-uuid":"AAYLE+vtCEZ6Amp7P3S1tA\u003D\u003D"}}
</code>
<img src="data:image/gif;base64,R0lGODlhAQABAIAAAAAAAP///yH5BAEAAAAALAAAAAABAAEAAAIBRAA7" style="display: none" class="datalet-bpr-guid-15912215" aria-hidden="true"><code style="display: none" id="bpr-guid-15912216" aria-hidden="true">
  {"data":{"dashEntityUrn":"urn:li:fsd_jobPosting:3768592532","companyDetails":{"company":"urn:li:fs_normalized_company:43338549","*companyResolutionResult":"urn:li:fs_normalized_company:43338549","$recipeTypes":["com.linkedin.voyager.deco.jobs.web.shared.WebCompactJobPostingCompany"],"$type":"com.linkedin.voyager.jobs.JobPostingCompany"},"jobState":"LISTED","description":{"attributes":[{"start":75,"length":1,"type":{"$type":"com.linkedin.pemberly.text.LineBreak"},"attributeKindUnion":{"lineBreak":{"$type":"com.linkedin.pemberly.text.LineBreak"}},"$type":"com.linkedin.pemberly.text.Attribute"},{"start":76,"length":1,"type":{"$type":"com.linkedin.pemberly.text.LineBreak"},"attributeKindUnion":{"lineBreak":{"$type":"com.linkedin.pemberly.text.LineBreak"}},"$type":"com.linkedin.pemberly.text.Attribute"},{"start":0,"length":77,"type":{"$type":"com.linkedin.pemberly.text.Bold"},"attributeKindUnion":{"bold":{"$type":"com.linkedin.pemberly.text.Bold"}},"$type":"com.linkedin.pemberly.text.Attribute"},{"start":365,"length":1,"type":{"$type":"com.linkedin.pemberly.text.LineBreak"},"attributeKindUnion":{"lineBreak":{"$type":"com.linkedin.pemberly.text.LineBreak"}},"$type":"com.linkedin.pemberly.text.Attribute"},{"start":366,"length":1,"type":{"$type":"com.linkedin.pemberly.text.LineBreak"},"attributeKindUnion":{"lineBreak":{"$type":"com.linkedin.pemberly.text.LineBreak"}},"$type":"com.linkedin.pemberly.text.Attribute"},{"start":577,"length":1,"type":{"$type":"com.linkedin.pemberly.text.LineBreak"},"attributeKindUnion":{"lineBreak":{"$type":"com.linkedin.pemberly.text.LineBreak"}},"$type":"com.linkedin.pemberly.text.Attribute"},{"start":578,"length":1,"type":{"$type":"com.linkedin.pemberly.text.LineBreak"},"attributeKindUnion":{"lineBreak":{"$type":"com.linkedin.pemberly.text.LineBreak"}},"$type":"com.linkedin.pemberly.text.Attribute"},{"start":791,"length":1,"type":{"$type":"com.linkedin.pemberly.text.LineBreak"},"attributeKindUnion":{"lineBreak":{"$type":"com.linkedin.pemberly.text.LineBreak"}},"$type":"com.linkedin.pemberly.text.Attribute"},{"start":792,"length":1,"type":{"$type":"com.linkedin.pemberly.text.LineBreak"},"attributeKindUnion":{"lineBreak":{"$type":"com.linkedin.pemberly.text.LineBreak"}},"$type":"com.linkedin.pemberly.text.Attribute"},{"start":883,"length":1,"type":{"$type":"com.linkedin.pemberly.text.LineBreak"},"attributeKindUnion":{"lineBreak":{"$type":"com.linkedin.pemberly.text.LineBreak"}},"$type":"com.linkedin.pemberly.text.Attribute"},{"start":884,"length":1,"type":{"$type":"com.linkedin.pemberly.text.LineBreak"},"attributeKindUnion":{"lineBreak":{"$type":"com.linkedin.pemberly.text.LineBreak"}},"$type":"com.linkedin.pemberly.text.Attribute"},{"start":890,"length":1,"type":{"$type":"com.linkedin.pemberly.text.LineBreak"},"attributeKindUnion":{"lineBreak":{"$type":"com.linkedin.pemberly.text.LineBreak"}},"$type":"com.linkedin.pemberly.text.Attribute"},{"start":891,"length":1,"type":{"$type":"com.linkedin.pemberly.text.LineBreak"},"attributeKindUnion":{"lineBreak":{"$type":"com.linkedin.pemberly.text.LineBreak"}},"$type":"com.linkedin.pemberly.text.Attribute"},{"start":885,"length":7,"type":{"$type":"com.linkedin.pemberly.text.Bold"},"attributeKindUnion":{"bold":{"$type":"com.linkedin.pemberly.text.Bold"}},"$type":"com.linkedin.pemberly.text.Attribute"},{"start":907,"length":1,"type":{"$type":"com.linkedin.pemberly.text.LineBreak"},"attributeKindUnion":{"lineBreak":{"$type":"com.linkedin.pemberly.text.LineBreak"}},"$type":"com.linkedin.pemberly.text.Attribute"},{"start":908,"length":1,"type":{"$type":"com.linkedin.pemberly.text.LineBreak"},"attributeKindUnion":{"lineBreak":{"$type":"com.linkedin.pemberly.text.LineBreak"}},"$type":"com.linkedin.pemberly.text.Attribute"},{"start":892,"length":17,"type":{"$type":"com.linkedin.pemberly.text.Bold"},"attributeKindUnion":{"bold":{"$type":"com.linkedin.pemberly.text.Bold"}},"$type":"com.linkedin.pemberly.text.Attribute"},{"start":909,"length":118,"type":{"$type":"com.linkedin.pemberly.text.ListItem"},"attributeKindUnion":{"listItem":{"$type":"com.linkedin.pemberly.text.ListItem"}},"$type":"com.linkedin.pemberly.text.Attribute"},{"start":1027,"length":156,"type":{"$type":"com.linkedin.pemberly.text.ListItem"},"attributeKindUnion":{"listItem":{"$type":"com.linkedin.pemberly.text.ListItem"}},"$type":"com.linkedin.pemberly.text.Attribute"},{"start":1183,"length":116,"type":{"$type":"com.linkedin.pemberly.text.ListItem"},"attributeKindUnion":{"listItem":{"$type":"com.linkedin.pemberly.text.ListItem"}},"$type":"com.linkedin.pemberly.text.Attribute"},{"start":1299,"length":144,"type":{"$type":"com.linkedin.pemberly.text.ListItem"},"attributeKindUnion":{"listItem":{"$type":"com.linkedin.pemberly.text.ListItem"}},"$type":"com.linkedin.pemberly.text.Attribute"},{"start":1443,"length":99,"type":{"$type":"com.linkedin.pemberly.text.ListItem"},"attributeKindUnion":{"listItem":{"$type":"com.linkedin.pemberly.text.ListItem"}},"$type":"com.linkedin.pemberly.text.Attribute"},{"start":1665,"length":1,"type":{"$type":"com.linkedin.pemberly.text.LineBreak"},"attributeKindUnion":{"lineBreak":{"$type":"com.linkedin.pemberly.text.LineBreak"}},"$type":"com.linkedin.pemberly.text.Attribute"},{"start":1666,"length":1,"type":{"$type":"com.linkedin.pemberly.text.LineBreak"},"attributeKindUnion":{"lineBreak":{"$type":"com.linkedin.pemberly.text.LineBreak"}},"$type":"com.linkedin.pemberly.text.Attribute"},{"start":1542,"length":125,"type":{"$type":"com.linkedin.pemberly.text.ListItem"},"attributeKindUnion":{"listItem":{"$type":"com.linkedin.pemberly.text.ListItem"}},"$type":"com.linkedin.pemberly.text.Attribute"},{"start":909,"length":758,"type":{"ordered":false,"$type":"com.linkedin.pemberly.text.List"},"attributeKindUnion":{"list":{"ordered":false,"$type":"com.linkedin.pemberly.text.List"}},"$type":"com.linkedin.pemberly.text.Attribute"},{"start":1679,"length":1,"type":{"$type":"com.linkedin.pemberly.text.LineBreak"},"attributeKindUnion":{"lineBreak":{"$type":"com.linkedin.pemberly.text.LineBreak"}},"$type":"com.linkedin.pemberly.text.Attribute"},{"start":1680,"length":1,"type":{"$type":"com.linkedin.pemberly.text.LineBreak"},"attributeKindUnion":{"lineBreak":{"$type":"com.linkedin.pemberly.text.LineBreak"}},"$type":"com.linkedin.pemberly.text.Attribute"},{"start":1667,"length":14,"type":{"$type":"com.linkedin.pemberly.text.Bold"},"attributeKindUnion":{"bold":{"$type":"com.linkedin.pemberly.text.Bold"}},"$type":"com.linkedin.pemberly.text.Attribute"},{"start":1693,"length":1,"type":{"$type":"com.linkedin.pemberly.text.LineBreak"},"attributeKindUnion":{"lineBreak":{"$type":"com.linkedin.pemberly.text.LineBreak"}},"$type":"com.linkedin.pemberly.text.Attribute"},{"start":1694,"length":1,"type":{"$type":"com.linkedin.pemberly.text.LineBreak"},"attributeKindUnion":{"lineBreak":{"$type":"com.linkedin.pemberly.text.LineBreak"}},"$type":"com.linkedin.pemberly.text.Attribute"},{"start":1681,"length":14,"type":{"$type":"com.linkedin.pemberly.text.Bold"},"attributeKindUnion":{"bold":{"$type":"com.linkedin.pemberly.text.Bold"}},"$type":"com.linkedin.pemberly.text.Attribute"},{"start":1695,"length":121,"type":{"$type":"com.linkedin.pemberly.text.ListItem"},"attributeKindUnion":{"listItem":{"$type":"com.linkedin.pemberly.text.ListItem"}},"$type":"com.linkedin.pemberly.text.Attribute"},{"start":1816,"length":159,"type":{"$type":"com.linkedin.pemberly.text.ListItem"},"attributeKindUnion":{"listItem":{"$type":"com.linkedin.pemberly.text.ListItem"}},"$type":"com.linkedin.pemberly.text.Attribute"},{"start":2126,"length":1,"type":{"$type":"com.linkedin.pemberly.text.LineBreak"},"attributeKindUnion":{"lineBreak":{"$type":"com.linkedin.pemberly.text.LineBreak"}},"$type":"com.linkedin.pemberly.text.Attribute"},{"start":2127,"length":1,"type":{"$type":"com.linkedin.pemberly.text.LineBreak"},"attributeKindUnion":{"lineBreak":{"$type":"com.linkedin.pemberly.text.LineBreak"}},"$type":"com.linkedin.pemberly.text.Attribute"},{"start":1975,"length":153,"type":{"$type":"com.linkedin.pemberly.text.ListItem"},"attributeKindUnion":{"listItem":{"$type":"com.linkedin.pemberly.text.ListItem"}},"$type":"com.linkedin.pemberly.text.Attribute"},{"start":1695,"length":433,"type":{"ordered":false,"$type":"com.linkedin.pemberly.text.List"},"attributeKindUnion":{"list":{"ordered":false,"$type":"com.linkedin.pemberly.text.List"}},"$type":"com.linkedin.pemberly.text.Attribute"},{"start":2151,"length":1,"type":{"$type":"com.linkedin.pemberly.text.LineBreak"},"attributeKindUnion":{"lineBreak":{"$type":"com.linkedin.pemberly.text.LineBreak"}},"$type":"com.linkedin.pemberly.text.Attribute"},{"start":2152,"length":1,"type":{"$type":"com.linkedin.pemberly.text.LineBreak"},"attributeKindUnion":{"lineBreak":{"$type":"com.linkedin.pemberly.text.LineBreak"}},"$type":"com.linkedin.pemberly.text.Attribute"},{"start":2128,"length":25,"type":{"$type":"com.linkedin.pemberly.text.Bold"},"attributeKindUnion":{"bold":{"$type":"com.linkedin.pemberly.text.Bold"}},"$type":"com.linkedin.pemberly.text.Attribute"},{"start":2153,"length":148,"type":{"$type":"com.linkedin.pemberly.text.ListItem"},"attributeKindUnion":{"listItem":{"$type":"com.linkedin.pemberly.text.ListItem"}},"$type":"com.linkedin.pemberly.text.Attribute"},{"start":2301,"length":152,"type":{"$type":"com.linkedin.pemberly.text.ListItem"},"attributeKindUnion":{"listItem":{"$type":"com.linkedin.pemberly.text.ListItem"}},"$type":"com.linkedin.pemberly.text.Attribute"},{"start":2517,"length":1,"type":{"$type":"com.linkedin.pemberly.text.LineBreak"},"attributeKindUnion":{"lineBreak":{"$type":"com.linkedin.pemberly.text.LineBreak"}},"$type":"com.linkedin.pemberly.text.Attribute"},{"start":2518,"length":1,"type":{"$type":"com.linkedin.pemberly.text.LineBreak"},"attributeKindUnion":{"lineBreak":{"$type":"com.linkedin.pemberly.text.LineBreak"}},"$type":"com.linkedin.pemberly.text.Attribute"},{"start":2453,"length":66,"type":{"$type":"com.linkedin.pemberly.text.ListItem"},"attributeKindUnion":{"listItem":{"$type":"com.linkedin.pemberly.text.ListItem"}},"$type":"com.linkedin.pemberly.text.Attribute"},{"start":2153,"length":366,"type":{"ordered":false,"$type":"com.linkedin.pemberly.text.List"},"attributeKindUnion":{"list":{"ordered":false,"$type":"com.linkedin.pemberly.text.List"}},"$type":"com.linkedin.pemberly.text.Attribute"},{"start":2569,"length":1,"type":{"$type":"com.linkedin.pemberly.text.LineBreak"},"attributeKindUnion":{"lineBreak":{"$type":"com.linkedin.pemberly.text.LineBreak"}},"$type":"com.linkedin.pemberly.text.Attribute"},{"start":2570,"length":1,"type":{"$type":"com.linkedin.pemberly.text.LineBreak"},"attributeKindUnion":{"lineBreak":{"$type":"com.linkedin.pemberly.text.LineBreak"}},"$type":"com.linkedin.pemberly.text.Attribute"},{"start":2519,"length":52,"type":{"$type":"com.linkedin.pemberly.text.Bold"},"attributeKindUnion":{"bold":{"$type":"com.linkedin.pemberly.text.Bold"}},"$type":"com.linkedin.pemberly.text.Attribute"},{"start":2571,"length":90,"type":{"$type":"com.linkedin.pemberly.text.ListItem"},"attributeKindUnion":{"listItem":{"$type":"com.linkedin.pemberly.text.ListItem"}},"$type":"com.linkedin.pemberly.text.Attribute"},{"start":2661,"length":101,"type":{"$type":"com.linkedin.pemberly.text.ListItem"},"attributeKindUnion":{"listItem":{"$type":"com.linkedin.pemberly.text.ListItem"}},"$type":"com.linkedin.pemberly.text.Attribute"},{"start":2823,"length":1,"type":{"$type":"com.linkedin.pemberly.text.LineBreak"},"attributeKindUnion":{"lineBreak":{"$type":"com.linkedin.pemberly.text.LineBreak"}},"$type":"com.linkedin.pemberly.text.Attribute"},{"start":2824,"length":1,"type":{"$type":"com.linkedin.pemberly.text.LineBreak"},"attributeKindUnion":{"lineBreak":{"$type":"com.linkedin.pemberly.text.LineBreak"}},"$type":"com.linkedin.pemberly.text.Attribute"},{"start":2762,"length":63,"type":{"$type":"com.linkedin.pemberly.text.ListItem"},"attributeKindUnion":{"listItem":{"$type":"com.linkedin.pemberly.text.ListItem"}},"$type":"com.linkedin.pemberly.text.Attribute"},{"start":2571,"length":254,"type":{"ordered":false,"$type":"com.linkedin.pemberly.text.List"},"attributeKindUnion":{"list":{"ordered":false,"$type":"com.linkedin.pemberly.text.List"}},"$type":"com.linkedin.pemberly.text.Attribute"},{"start":2833,"length":1,"type":{"$type":"com.linkedin.pemberly.text.LineBreak"},"attributeKindUnion":{"lineBreak":{"$type":"com.linkedin.pemberly.text.LineBreak"}},"$type":"com.linkedin.pemberly.text.Attribute"},{"start":2834,"length":1,"type":{"$type":"com.linkedin.pemberly.text.LineBreak"},"attributeKindUnion":{"lineBreak":{"$type":"com.linkedin.pemberly.text.LineBreak"}},"$type":"com.linkedin.pemberly.text.Attribute"},{"start":2825,"length":10,"type":{"$type":"com.linkedin.pemberly.text.Bold"},"attributeKindUnion":{"bold":{"$type":"com.linkedin.pemberly.text.Bold"}},"$type":"com.linkedin.pemberly.text.Attribute"},{"start":2849,"length":1,"type":{"$type":"com.linkedin.pemberly.text.LineBreak"},"attributeKindUnion":{"lineBreak":{"$type":"com.linkedin.pemberly.text.LineBreak"}},"$type":"com.linkedin.pemberly.text.Attribute"},{"start":2850,"length":1,"type":{"$type":"com.linkedin.pemberly.text.LineBreak"},"attributeKindUnion":{"lineBreak":{"$type":"com.linkedin.pemberly.text.LineBreak"}},"$type":"com.linkedin.pemberly.text.Attribute"},{"start":2835,"length":16,"type":{"$type":"com.linkedin.pemberly.text.Bold"},"attributeKindUnion":{"bold":{"$type":"com.linkedin.pemberly.text.Bold"}},"$type":"com.linkedin.pemberly.text.Attribute"},{"start":2851,"length":172,"type":{"$type":"com.linkedin.pemberly.text.ListItem"},"attributeKindUnion":{"listItem":{"$type":"com.linkedin.pemberly.text.ListItem"}},"$type":"com.linkedin.pemberly.text.Attribute"},{"start":3023,"length":40,"type":{"$type":"com.linkedin.pemberly.text.ListItem"},"attributeKindUnion":{"listItem":{"$type":"com.linkedin.pemberly.text.ListItem"}},"$type":"com.linkedin.pemberly.text.Attribute"},{"start":3063,"length":52,"type":{"$type":"com.linkedin.pemberly.text.ListItem"},"attributeKindUnion":{"listItem":{"$type":"com.linkedin.pemberly.text.ListItem"}},"$type":"com.linkedin.pemberly.text.Attribute"},{"start":3115,"length":40,"type":{"$type":"com.linkedin.pemberly.text.ListItem"},"attributeKindUnion":{"listItem":{"$type":"com.linkedin.pemberly.text.ListItem"}},"$type":"com.linkedin.pemberly.text.Attribute"},{"start":3155,"length":124,"type":{"$type":"com.linkedin.pemberly.text.ListItem"},"attributeKindUnion":{"listItem":{"$type":"com.linkedin.pemberly.text.ListItem"}},"$type":"com.linkedin.pemberly.text.Attribute"},{"start":3279,"length":85,"type":{"$type":"com.linkedin.pemberly.text.ListItem"},"attributeKindUnion":{"listItem":{"$type":"com.linkedin.pemberly.text.ListItem"}},"$type":"com.linkedin.pemberly.text.Attribute"},{"start":3432,"length":1,"type":{"$type":"com.linkedin.pemberly.text.LineBreak"},"attributeKindUnion":{"lineBreak":{"$type":"com.linkedin.pemberly.text.LineBreak"}},"$type":"com.linkedin.pemberly.text.Attribute"},{"start":3433,"length":1,"type":{"$type":"com.linkedin.pemberly.text.LineBreak"},"attributeKindUnion":{"lineBreak":{"$type":"com.linkedin.pemberly.text.LineBreak"}},"$type":"com.linkedin.pemberly.text.Attribute"},{"start":3364,"length":70,"type":{"$type":"com.linkedin.pemberly.text.ListItem"},"attributeKindUnion":{"listItem":{"$type":"com.linkedin.pemberly.text.ListItem"}},"$type":"com.linkedin.pemberly.text.Attribute"},{"start":2851,"length":583,"type":{"ordered":false,"$type":"com.linkedin.pemberly.text.List"},"attributeKindUnion":{"list":{"ordered":false,"$type":"com.linkedin.pemberly.text.List"}},"$type":"com.linkedin.pemberly.text.Attribute"},{"start":3704,"length":1,"type":{"$type":"com.linkedin.pemberly.text.LineBreak"},"attributeKindUnion":{"lineBreak":{"$type":"com.linkedin.pemberly.text.LineBreak"}},"$type":"com.linkedin.pemberly.text.Attribute"},{"start":3705,"length":1,"type":{"$type":"com.linkedin.pemberly.text.LineBreak"},"attributeKindUnion":{"lineBreak":{"$type":"com.linkedin.pemberly.text.LineBreak"}},"$type":"com.linkedin.pemberly.text.Attribute"}],"text":"Join Our Mission to Fight Climate Change as our Sustainability Data Analyst\n\nWant to be part of one of Europe's fastest growing climate tech startups? We are looking for a passionate Sustainability Data Analyst to help us transform every company into a sustainable business, with critical responsibility and influence on our product and overall venture development.\n\nAs a Sustainability Data Analyst your role will have a key responsibility of applying data-driven approaches to managing our carbon footprint calculations and ensuring our customers understand how we got there.\n\nIf you are excited about building an economy where every company creates an impact for a sustainable world and empowering companies to decarbonize and boost their business performance, we’d love to hear from you!\n\nIf you want to find out more about Cozero and our approach to hiring, check out this page.\n\nTasks\n\nWhat you’ll do:\n\nYou will be developing, maintaining and improving our methodologies and models for calculating companies GHG emissionsYou will develop and oversee processes to ensure our clients receive accurate CCF and PCF results (eg. Quality assurance, validating user carbon footprints)You will transform unstructured data sets so they are ready to be ingested into our calculation databases and modelsYou will work across Cozero’s climate, data science and engineering teams, to deliver on high quality data outputs for internal and external useYou will work closely with our Sales and Customer Success teams to define and prioritize data needsYou will proactively develop ideas on how we can create more value out of our data as well as external data for our clients\n\nRequirements\n\nWho you are:\n\nYou care deeply about our planet and staying informed on key environmental trends like CO2 emissions and climate change. Your meticulous eye for detail ensures thorough, systematic and accurate data analysis even when working with complex, unstructured and inconsistent datasets. A fast-evolving environment excites you, and you love contributing your skills to a diverse, collaborative team making a real difference in the world. \n\nWhat we’re looking for:\n\nExperience working in an analyst role or similar (ideally working with climate data like Corporate GHG emissions, Emission factor and LCA databases)You have programming skills in Python and SQL and a strong understanding of data gathering, inspecting, cleansing, transforming, and modeling techniquesYou have strong verbal and written English communication skills \n\nExtra bonus points if you also have the following:\n\nMaster's degree in Environmental Science, Engineering, Computer Science or a related fieldKnowledge of different carbon accounting standards and databases (e.g. GHG Protocol, GLEC, Ecoinvent)Experience in carbon accounting and/ or life cycle assessment\n\nBenefits\n\nWhat we offer:\n\nOpportunity to work remotely or from our great co-working space in Maschinenraum Berlin with access to innovation events of the German Mittelstand and the startup ecosystemVirtual meditation and wellness sessionsRegular team events and at least 2 offsites per yearIndividual learning &amp; development budgetThe option to participate in our Virtual Stock Option Plan (VSOP), to have direct involvement and impact on Cozero’s successA strong company culture focused on growth, collaboration and fighting climate changeParticipation opportunities in accelerator, startup &amp; event programs\n\nWe strive to build a diverse, inclusive and collaborative workplace and we follow a fair, objective, and non-discriminatory recruitment process. All applications will be considered for employment regardless of nationality, culture, or any other personal characteristics.\n\nThe glue that binds us together despite our differences is a common set of values, our purpose to transform every company into a low-emission business, and our culture that has a growth mindset at its heart. If you’re excited about this role we encourage you to apply, even if your past experience doesn’t perfectly match every requirement in the job description.","$type":"com.linkedin.pemberly.text.AttributedText"},"repostedJobPosting":null,"title":"Sustainability Data Analyst (f/m/x)","entityUrn":"urn:li:fs_normalized_jobPosting:3768592532","workRemoteAllowed":true,"applyMethod":{"companyApplyUrl":"https://join.com/companies/cozero/9872937-sustainability-data-analyst-f-m-x?pid=d7cb2e8e859c62cd7f75","applyStartersPreferenceVoid":false,"inPageOffsiteApply":false,"$type":"com.linkedin.voyager.jobs.OffsiteApply"},"talentHubJob":false,"formattedLocation":"Germany","workplaceTypes":["urn:li:fs_workplaceType:2"],"listedAt":1700948520000,"$recipeTypes":["com.linkedin.voyager.deco.jobs.web.shared.WebLightJobPosting"],"$type":"com.linkedin.voyager.jobs.JobPosting","standardizedAddresses":null,"jobPostingId":3768592532,"workplaceTypesResolutionResults":{"*urn:li:fs_workplaceType:2":"urn:li:fs_workplaceType:2"}},"meta":{"microSchema":{"version":"2","types":{"com.linkedin.voyager.deco.jobs.web.shared.WebLightJobPosting":{"fields":{"dashEntityUrn":{"type":"string"},"companyDetails":{"type":{"union":["com.linkedin.voyager.jobs.JobPostingCompanyName","com.linkedin.voyager.deco.jobs.web.shared.WebCompactJobPostingCompany"]}},"talentHubJob":{"type":"boolean"},"jobState":{"type":"string"},"formattedLocation":{"type":"string"},"description":{"type":"com.linkedin.pemberly.text.AttributedText"},"repostedJobPosting":{"type":"string"},"workplaceTypes":{"type":{"array":"string"}},"listedAt":{"type":"long"},"title":{"type":"string"},"standardizedAddresses":{"type":"com.linkedin.voyager.jobs.JobPostingAddresses"},"jobPostingId":{"type":"long"},"workRemoteAllowed":{"type":"boolean"},"entityUrn":{"type":"string"},"applyMethod":{"type":{"union":["com.linkedin.voyager.jobs.UnknownApply","com.linkedin.voyager.jobs.ComplexOnsiteApply","com.linkedin.voyager.jobs.SimpleOnsiteApply","com.linkedin.voyager.jobs.OffsiteApply"]}},"workplaceTypesResolutionResults":{"type":{"map":"com.linkedin.voyager.deco.jobs.shared.WebJobDetailsWorkplaceType"},"resolvedFrom":"workplaceTypes"}},"baseType":"com.linkedin.voyager.jobs.JobPosting"},"com.linkedin.voyager.deco.jobs.web.shared.WebCompactJobPostingCompany":{"fields":{"companyResolutionResult":{"type":"com.linkedin.voyager.deco.organization.shared.WebCompactCompany","resolvedFrom":"company"},"company":{"type":"string"}},"baseType":"com.linkedin.voyager.jobs.JobPostingCompany"},"com.linkedin.voyager.deco.jobs.shared.WebJobDetailsWorkplaceType":{"fields":{"entityUrn":{"type":"string"},"localizedName":{"type":"string"}},"baseType":"com.linkedin.voyager.jobs.shared.WorkplaceType"},"com.linkedin.voyager.deco.organization.shared.WebCompactCompany":{"fields":{"name":{"type":"string"},"logo":{"type":"com.linkedin.voyager.organization.CompanyLogoImage"},"universalName":{"type":"string"},"entityUrn":{"type":"string"},"url":{"type":"string"}},"baseType":"com.linkedin.voyager.organization.Company"}}}},"included":[{"localizedName":"Remote","entityUrn":"urn:li:fs_workplaceType:2","$recipeTypes":["com.linkedin.voyager.deco.jobs.shared.WebJobDetailsWorkplaceType"],"$type":"com.linkedin.voyager.jobs.shared.WorkplaceType"},{"$recipeTypes":["com.linkedin.voyager.deco.organization.shared.WebCompactCompany"],"url":"https://www.linkedin.com/company/cozero-io","$type":"com.linkedin.voyager.organization.Company","entityUrn":"urn:li:fs_normalized_company:43338549","name":"Cozero","logo":{"image":{"$type":"com.linkedin.common.VectorImage","rootUrl":"https://media.licdn.com/dms/image/C4D0BAQGkpvwIXWu1-g/company-logo_","artifacts":[{"width":200,"expiresAt":1709164800000,"fileIdentifyingUrlPathSegment":"200_200/0/1638439865071/cozero_io_logo?e=1709164800&amp;v=beta&amp;t=SkxNXYSV-cQiUJvKiBgKXlXVlM4i0GmvhuX8g6hGKZY","$type":"com.linkedin.common.VectorArtifact","height":200},{"width":100,"expiresAt":1709164800000,"fileIdentifyingUrlPathSegment":"100_100/0/1638439865071/cozero_io_logo?e=1709164800&amp;v=beta&amp;t=sVPH2ggrA8QHrpoTseVt0_N71x9Hg_xcONvZl48_Ogk","$type":"com.linkedin.common.VectorArtifact","height":100},{"width":400,"expiresAt":1709164800000,"fileIdentifyingUrlPathSegment":"400_400/0/1638439865071/cozero_io_logo?e=1709164800&amp;v=beta&amp;t=G2AlGC86FzMcCKbWIW1B4lPKCq9A1cBdlHhCGtKap9c","$type":"com.linkedin.common.VectorArtifact","height":400}]},"type":"SQUARE_LOGO","$type":"com.linkedin.voyager.organization.CompanyLogoImage"},"universalName":"cozero-io"}]}
</code>
<code style="display: none" id="datalet-bpr-guid-15912216" aria-hidden="true">
  {"request":"/voyager/api/jobs/jobPostings/3768592532?decorationId\u003Dcom.linkedin.voyager.deco.jobs.web.shared.WebLightJobPosting-23\u0026","status":200,"body":"bpr-guid-15912216","method":"GET","headers":{"x-li-uuid":"AAYLE+vtCEZ6Amp7P3S1tA\u003D\u003D"}}
</code>
<img src="data:image/gif;base64,R0lGODlhAQABAIAAAAAAAP///yH5BAEAAAAALAAAAAABAAEAAAIBRAA7" style="display: none" class="datalet-bpr-guid-15912216" aria-hidden="true"><code style="display: none" id="bpr-guid-15912217" aria-hidden="true">
  {"data":{"data":{"$recipeTypes":["com.linkedin.8fbc3627c8305a33e3e6840ec7688312"],"jobsDashJobPostingDetailSectionsByCardSectionTypes":{"$recipeTypes":["com.linkedin.f1d8ff8a5c9d0a2c73d8c4418e3a4526"],"elements":[{"$recipeTypes":["com.linkedin.180bad7e2024ccff1ee0157be65c0dd1"],"jobPostingDetailSection":[{"similarJobsCard":null,"interviewPrepCard":null,"salaryCard":null,"companyInsightsCard":null,"*topCard":"urn:li:fsd_jobPostingCard:(3768592532,JOB_DETAILS)","bannerCard":null,"jobSeekerActionCard":null,"learningRecommendationCard":null,"hiringTeamCard":null,"similarJobsAtCompanyCard":null,"jobSegmentAttributesCard":null,"howYouMatchCard":null,"jobApplicantInsightsUrn":null,"jobActivityCard":null,"postApplyPromo":null,"jobDescription":null,"benefitsCard":null,"jobAlertCard":null,"marketplacePromoCard":null,"companyCardV2":null,"recommendedActionsCard":null}],"$type":"com.linkedin.voyager.dash.jobs.JobPostingDetailSection"}],"$type":"com.linkedin.restli.common.CollectionResponse"},"$type":"com.linkedin.8fbc3627c8305a33e3e6840ec7688312"},"extensions":{"webMetadata":{}}},"meta":{"microSchema":{"isGraphQL":true,"version":"2.1","types":{"com.linkedin.b1148482c3f256b956d443d3fbab2904":{"fields":{"nestedTextSelectableOptions":{"type":{"array":"com.linkedin.7bd2de9a7b9fadfe651a8dd1c13db7ad"}},"exclusiveSelectableOption":{"type":"com.linkedin.98d89bae4aa4ef8f3ffc1018ca93419c"},"selectionCountRangeValidation":{"type":"com.linkedin.e18c41f5008d005150ad33f812748af9"},"subtitle":{"type":"com.linkedin.390ac1c1431d5b4e340a65af24ed68f2"}},"baseType":"com.linkedin.voyager.dash.common.forms.NestedCheckboxFormComponent"},"com.linkedin.c8842299f35a2aecf1baf1f52de866bf":{"fields":{"textSelectableOption":{"type":"com.linkedin.98d89bae4aa4ef8f3ffc1018ca93419c"},"positiveToggleIcon":{"type":"com.linkedin.30e1ec6a491035bd7a31cc9de8a12c18"},"negativeToggleIcon":{"type":"com.linkedin.30e1ec6a491035bd7a31cc9de8a12c18"}},"baseType":"com.linkedin.voyager.dash.common.forms.TogglePill"},"com.linkedin.45e3a7da0d85a4cd9d5aaa94b34dd17d":{"fields":{"suggestedEntities":{"type":{"array":"com.linkedin.98d89bae4aa4ef8f3ffc1018ca93419c"}},"subtitle":{"type":"com.linkedin.390ac1c1431d5b4e340a65af24ed68f2"},"dismissSelectableOption":{"type":"com.linkedin.98d89bae4aa4ef8f3ffc1018ca93419c"},"dismissControlName":{"type":"com.linkedin.voyager.dash.common.tracking.ControlName"},"actionableSuggestedEntities":{"type":{"array":"com.linkedin.65862eebdeb12a456370f32142eebd8c"}},"navigationButton":{"type":"com.linkedin.72471cc7e55a55b0fd549127127633f7"},"title":{"type":"com.linkedin.390ac1c1431d5b4e340a65af24ed68f2"},"typeaheadFormSuggestionUseCase":{"type":"com.linkedin.voyager.dash.common.forms.TypeaheadFormSuggestionUseCase"},"trackingId":{"type":"com.linkedin.voyager.dash.common.Base64String"}},"baseType":"com.linkedin.voyager.dash.common.forms.TypeaheadFormSuggestionViewModel"},"com.linkedin.d0f6c4718d68ff125ca6f7e0b91f510c":{"fields":{},"baseType":"com.linkedin.voyager.dash.jobs.JobSearchHistory"},"com.linkedin.deaeaf135b51841f133d0ceae7eca621":{"fields":{"premiumStyle":{"type":"boolean"},"size":{"type":"com.linkedin.voyager.dash.common.ux.button.ButtonSize"},"icon":{"type":"com.linkedin.voyager.dash.common.SystemImageName"},"text":{"type":"string"},"iconAfterText":{"type":"boolean"},"emphasize":{"type":"boolean"},"category":{"type":"com.linkedin.voyager.dash.common.ux.button.ButtonCategory"}},"baseType":"com.linkedin.voyager.dash.common.ux.button.ButtonAppearance"},"com.linkedin.427074b29d6e63261a7a2174b0df610b":{"fields":{"groups":{"type":{"array":"com.linkedin.30951a1ee13ae9dec8fa329260e12fc2"}},"tip":{"type":"com.linkedin.00ac75ef8dae327ab29a045438e7d186"},"viewImpressionTrackingKey":{"type":"string"},"buttonCTA":{"type":"com.linkedin.5de46613f3027e5a41b356871519a78b"}},"baseType":"com.linkedin.voyager.dash.jobs.ItemsMatchSection"},"com.linkedin.b429b28d37054e5232ecd5ecffd09315":{"fields":{"geo":{"resolvedFrom":"geoUrn","type":"com.linkedin.00c4f529ff2fa302d89eaea66344f268"},"geoUrn":{"type":"string"},"postalCode":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.common.forms.LocationInputValue"},"com.linkedin.79cb580711dcd3cf7d83e724b3d9e260":{"fields":{"localizedName":{"type":"string"},"localizedDescription":{"type":"string"},"workplaceTypeEnum":{"type":"com.linkedin.voyager.dash.jobs.WorkplaceTypeEnum"},"entityUrn":{"type":"com.linkedin.voyager.dash.common.WorkplaceTypeUrn"},"preDashWorkplaceTypeUrn":{"type":"com.linkedin.voyager.dash.common.PreDashUrn"}},"baseType":"com.linkedin.voyager.dash.jobs.WorkplaceType"},"com.linkedin.1b210594d26c621d240d5cf0136a8bdd":{"fields":{"count":{"type":"int"},"seniority":{"resolvedFrom":"seniorityUrn","type":"com.linkedin.0f63072b23eab5f48ddb389e6f328afc"},"seniorityUrn":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.premium.insights.JobApplicantInsightsSeniorityDetail"},"com.linkedin.9e0491106da7f36c0672ee51875a86fa":{"fields":{"displayImageWithFrameReferenceUnion":{"type":{"union":{"url":"com.linkedin.common.Url","vectorImage":"com.linkedin.b8949adeaaa915212d0136c4b9a1952b"}}},"displayImageReferenceResolutionResult":{"derivedFrom":"displayImageReference","type":{"union":{"url":"com.linkedin.common.Url","vectorImage":"com.linkedin.b8949adeaaa915212d0136c4b9a1952b"}}},"displayImageWithFrameReference":{"derivedFrom":"displayImageWithFrameReferenceUnion","type":{"union":{"url":"com.linkedin.common.Url","vectorImage":"com.linkedin.b8949adeaaa915212d0136c4b9a1952b"}}},"displayImageReference":{"type":{"union":{"url":"com.linkedin.common.Url","vectorImage":"com.linkedin.b8949adeaaa915212d0136c4b9a1952b"}}},"displayImageUrn":{"type":"com.linkedin.common.Urn"}},"baseType":"com.linkedin.voyager.dash.identity.profile.PhotoFilterPicture"},"com.linkedin.653bbb9565e4a43fc63d44ebc167fab1":{"fields":{},"baseType":"com.linkedin.voyager.dash.jobs.SeeAllCard"},"com.linkedin.3551030d48734b4fd9cb3238efc77ece":{"fields":{"image":{"type":"com.linkedin.30e1ec6a491035bd7a31cc9de8a12c18"},"text":{"type":"com.linkedin.390ac1c1431d5b4e340a65af24ed68f2"}},"baseType":"com.linkedin.voyager.dash.common.ux.InsightViewModel"},"com.linkedin.aa4a2fb807b219a33654acc9643ff224":{"fields":{},"baseType":"com.linkedin.voyager.dash.jobs.home.RefreshStateCard"},"com.linkedin.9774b5e7450ba6db34185afe39f9e516":{"fields":{"pills":{"derivedFrom":"pillsUnion","type":{"union":{"dismissPills":{"array":"com.linkedin.c8f6142f1647b4b53caab19e4fe964ef"},"togglePills":{"array":"com.linkedin.c8842299f35a2aecf1baf1f52de866bf"}}}},"contextualSuggestionsTriggers":{"type":"com.linkedin.5481c9cf9c53bc965a2e49fdf5e17760"},"pillsUnion":{"type":{"union":{"dismissPills":{"array":"com.linkedin.c8f6142f1647b4b53caab19e4fe964ef"},"togglePills":{"array":"com.linkedin.c8842299f35a2aecf1baf1f52de866bf"}}}},"contextualSuggestionViewModel":{"type":"com.linkedin.45e3a7da0d85a4cd9d5aaa94b34dd17d"},"selectionCountRangeValidation":{"type":"com.linkedin.e18c41f5008d005150ad33f812748af9"},"typeaheadCta":{"type":"com.linkedin.c754100718c581ece9a715a9165db201"},"description":{"type":"com.linkedin.390ac1c1431d5b4e340a65af24ed68f2"},"togglePills":{"type":{"array":"com.linkedin.c8842299f35a2aecf1baf1f52de866bf"}},"pillSuggestionUseCase":{"type":"com.linkedin.voyager.dash.common.forms.TypeaheadFormSuggestionUseCase"},"trackingId":{"type":"com.linkedin.voyager.dash.common.Base64String"}},"baseType":"com.linkedin.voyager.dash.common.forms.PillFormComponent"},"com.linkedin.2f3c656d762dff10246b267bedddfcfd":{"fields":{"requiredFieldMissingErrorText":{"type":"com.linkedin.390ac1c1431d5b4e340a65af24ed68f2"},"responseRequired":{"type":"boolean"},"textField":{"type":"com.linkedin.5f82b47e5488aec975bf096d809fb051"},"labelText":{"type":"com.linkedin.390ac1c1431d5b4e340a65af24ed68f2"}},"baseType":"com.linkedin.voyager.dash.common.forms.SingleLineTextField"},"com.linkedin.d77e6926ccb788017c5e7ffe0261e4d7":{"fields":{"icon":{"type":"com.linkedin.voyager.dash.common.SystemImageName"},"questionPageUrl":{"type":"com.linkedin.common.Url"},"title":{"type":"string"},"questionText":{"type":"string"},"subtitle":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.jobs.InterviewPrepQuestionRecommendation"},"com.linkedin.b2b7b2441fdd2b88fd82124f6d8c4a1c":{"fields":{"alumni":{"type":{"array":"com.linkedin.3df29ea52bb4c2126ad5c1daa0ab5b01"}},"totalNumberOfAlumni":{"type":"int"}},"baseType":"com.linkedin.voyager.dash.premium.companyinsights.AlumniInsights"},"com.linkedin.45d6d21a26aa62742acf906ac42d8306":{"fields":{"jobsOpeningsGrowthUnselectedFunctions":{"type":{"array":"com.linkedin.f6e5193777aa8860860ecb48d0212cce"}},"jobOpeningsGrowthByFunction":{"type":{"array":"com.linkedin.cca42244a99771eb4b1bfffed219f853"}},"jobOpeningsGrowthAllFunctions":{"type":{"array":"com.linkedin.f6e5193777aa8860860ecb48d0212cce"}},"jobOpeningsByFunction":{"type":{"array":"com.linkedin.acd5ebb13f6e0c9da292f5d03c57eac4"}}},"baseType":"com.linkedin.voyager.dash.premium.companyinsights.JobOpeningsInsights"},"com.linkedin.b97672681af5b861a5ec5c53b8fcbc96":{"fields":{"name":{"type":"string"},"jobCompanyUnion":{"type":{"union":{"company":"string","rawCompanyName":"string"}}},"jobCompany":{"resolvedFrom":"jobCompanyUnion","type":{"union":{"company":"com.linkedin.a2ef5c1c8d1bd7b83de7009b834a38c6","rawCompanyName":"string"}}}},"baseType":"com.linkedin.voyager.dash.jobs.JobPostingCompany"},"com.linkedin.9196412bf6c5c0443810d690321ab8f0":{"fields":{"title":{"type":"string"},"description":{"type":"com.linkedin.390ac1c1431d5b4e340a65af24ed68f2"},"image":{"type":"com.linkedin.30e1ec6a491035bd7a31cc9de8a12c18"}},"baseType":"com.linkedin.voyager.dash.jobs.SafetyTipViewModel"},"com.linkedin.57812d345916f0da7785a8887e8fe73d":{"fields":{},"baseType":"com.linkedin.voyager.dash.jobs.CarouselAdvantageCard"},"com.linkedin.e53bf26a30b7d5901401fa6088b455dc":{"fields":{"encryptedBiddingParameters":{"type":"string"},"trackingCode":{"type":"string"},"navigationAction":{"type":"com.linkedin.5de46613f3027e5a41b356871519a78b"},"referenceId":{"type":"com.linkedin.voyager.dash.common.Base64String"},"trackingId":{"type":"com.linkedin.voyager.dash.common.Base64String"}},"baseType":"com.linkedin.voyager.dash.jobs.JobTrackingData"},"com.linkedin.efd2ef2c46b7959f85a3d6461edc3e69":{"fields":{"image":{"type":"com.linkedin.30e1ec6a491035bd7a31cc9de8a12c18"},"descriptionUnions":{"type":{"array":{"union":{"text":"com.linkedin.390ac1c1431d5b4e340a65af24ed68f2","label":"com.linkedin.4a3588cb3f298939a6ef5b2e61cadc18"}}}},"description":{"derivedFrom":"descriptionUnions","type":{"array":{"union":{"text":"com.linkedin.390ac1c1431d5b4e340a65af24ed68f2","label":"com.linkedin.4a3588cb3f298939a6ef5b2e61cadc18"}}}}},"baseType":"com.linkedin.voyager.dash.jobs.JobInsightViewModel"},"com.linkedin.00ac75ef8dae327ab29a045438e7d186":{"fields":{"navigationAction":{"type":"com.linkedin.5de46613f3027e5a41b356871519a78b"},"image":{"type":"com.linkedin.voyager.dash.common.SystemImageName"},"content":{"type":"com.linkedin.390ac1c1431d5b4e340a65af24ed68f2"}},"baseType":"com.linkedin.voyager.dash.jobs.HowYouMatchTip"},"com.linkedin.91ef6d45f2b779a5fad235f8f9dccdb0":{"fields":{"composeNavigationContext":{"type":"com.linkedin.b69dade258a78e45f8205cb49b6a98b9"},"icon":{"type":"com.linkedin.30e1ec6a491035bd7a31cc9de8a12c18"},"displayText":{"type":"com.linkedin.390ac1c1431d5b4e340a65af24ed68f2"},"textStartIcon":{"type":"com.linkedin.30e1ec6a491035bd7a31cc9de8a12c18"},"composeOptionType":{"type":"com.linkedin.voyager.dash.messaging.compose.ComposeOptionType"},"entityUrn":{"type":"com.linkedin.voyager.dash.common.ComposeOptionUrn"}},"baseType":"com.linkedin.voyager.dash.messaging.compose.ComposeOption"},"com.linkedin.00c4f529ff2fa302d89eaea66344f268":{"fields":{"countryUrn":{"type":"string"},"country":{"resolvedFrom":"countryUrn","type":"com.linkedin.00c4f529ff2fa302d89eaea66344f268"},"localizedName":{"type":"string"},"defaultLocalizedNameWithoutCountryName":{"type":"string"},"entityUrn":{"type":"com.linkedin.voyager.dash.common.GeoUrn"},"latitude":{"type":"float"},"countryISOCode":{"type":"string"},"abbreviatedLocalizedName":{"type":"string"},"defaultLocalizedName":{"type":"string"},"longitude":{"type":"float"}},"baseType":"com.linkedin.voyager.dash.common.Geo"},"com.linkedin.72617837b227d77ce48feb965f1ecc1b":{"fields":{"activityType":{"type":"com.linkedin.voyager.dash.jobs.JobActivityType"},"description":{"type":"string"},"formattedTimeAgo":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.jobs.JobActivity"},"com.linkedin.6143491c72aa6f2fce542169c8baa1d6":{"fields":{"cta":{"type":"string"},"formType":{"type":"com.linkedin.voyager.dash.jobs.JobSeekerFormActionCardType"},"ctaControlName":{"type":"com.linkedin.voyager.dash.common.tracking.ControlName"},"subtitle":{"type":"com.linkedin.390ac1c1431d5b4e340a65af24ed68f2"},"formSection":{"type":"com.linkedin.55c2879f1a800fb9b17e54740f965515"},"legoTrackingToken":{"type":"string"},"title":{"type":"com.linkedin.390ac1c1431d5b4e340a65af24ed68f2"}},"baseType":"com.linkedin.voyager.dash.jobs.JobSeekerFormActionCard"},"com.linkedin.c2408eaefddaf66bea26cb69c95979d7":{"fields":{"utype":{"type":"string"},"referenceId":{"type":"com.linkedin.common.TrackingId"}},"baseType":"com.linkedin.voyager.dash.premium.PremiumFunnelCommonHeader"},"com.linkedin.90840606293e76f91e739cc9fbd0c304":{"fields":{"textSelectableOptions":{"type":{"array":"com.linkedin.98d89bae4aa4ef8f3ffc1018ca93419c"}},"exclusiveSelectableOption":{"type":"com.linkedin.98d89bae4aa4ef8f3ffc1018ca93419c"},"selectionCountRangeValidation":{"type":"com.linkedin.e18c41f5008d005150ad33f812748af9"},"subtitle":{"type":"com.linkedin.390ac1c1431d5b4e340a65af24ed68f2"}},"baseType":"com.linkedin.voyager.dash.common.forms.CheckboxFormComponent"},"com.linkedin.594a52f4e72625c921bc156c950ef9e3":{"fields":{"formElementInputs":{"type":{"array":"com.linkedin.81235c3224517a02609a065763927f01"}}},"baseType":"com.linkedin.voyager.dash.common.forms.PrerequisiteFormInputs"},"com.linkedin.181ad18bcf938cd6016ba9ffd2137150":{"fields":{"addButtonText":{"type":"string"},"removeButtonText":{"type":"string"},"selectionCountRangeValidation":{"type":"com.linkedin.e18c41f5008d005150ad33f812748af9"},"repeatableFormElementGroups":{"type":{"array":"com.linkedin.6a325d9bf5f44baf8fede1db5a7ce66f"}},"addButtonControlName":{"type":"com.linkedin.voyager.dash.common.tracking.ControlName"},"title":{"type":"string"},"removeButtonControlName":{"type":"com.linkedin.voyager.dash.common.tracking.ControlName"}},"baseType":"com.linkedin.voyager.dash.common.forms.RepeatableSectionData"},"com.linkedin.a2ef5c1c8d1bd7b83de7009b834a38c6":{"fields":{"industryV2Taxonomy":{"resolvedFrom":"industryV2TaxonomyUrns","type":{"array":"com.linkedin.7a7f0ab9f40a57ecd2a27e26059721fe"}},"description":{"type":"string"},"employeeCount":{"type":"long"},"url":{"type":"com.linkedin.common.Url"},"employeeCountRange":{"type":"com.linkedin.6148d310783eaffead8570485d05a156"},"industryV2TaxonomyUrns":{"type":{"array":"string"}},"followingStateUrn":{"type":"string"},"entityUrn":{"type":"com.linkedin.voyager.dash.common.CompanyUrn"},"followingState":{"resolvedFrom":"followingStateUrn","type":"com.linkedin.756c19defef3189024bfe8e3e74d83ab"},"name":{"type":"string"},"logo":{"type":{"union":{"url":"com.linkedin.common.Url","vectorImage":"com.linkedin.b8949adeaaa915212d0136c4b9a1952b"}}},"logoResolutionResult":{"derivedFrom":"logo","type":{"union":{"url":"com.linkedin.common.Url","vectorImage":"com.linkedin.b8949adeaaa915212d0136c4b9a1952b"}}},"universalName":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.organization.Company"},"com.linkedin.c4e907da1ff053ca48ebe32c32f78805":{"fields":{"entityUrn":{"type":"com.linkedin.voyager.dash.common.PromotionPlacementUrn"}},"baseType":"com.linkedin.voyager.dash.jobs.PromotionalCard"},"com.linkedin.7f23d6d0556f01f090158424e75ff2ab":{"fields":{"totalHires":{"type":"int"},"seniorHires":{"type":{"array":"com.linkedin.d09aeb85fdf4991b6b063ae75ef6ef54"}},"hireCounts":{"type":{"array":"com.linkedin.2815a36a9c66a3c907e5474fae6a1903"}},"totalNumberOfSeniorHires":{"type":"int"}},"baseType":"com.linkedin.voyager.dash.premium.companyinsights.HiresInsights"},"com.linkedin.94721090e597af7c97503cb8b1a7e50f":{"fields":{"type":{"type":"com.linkedin.voyager.dash.common.text.EpochFormat"},"epochAt":{"type":"com.linkedin.common.Time"}},"baseType":"com.linkedin.voyager.dash.common.text.EpochTime"},"com.linkedin.48407601ffa8bd83b8651487ea606df9":{"fields":{"errorText":{"type":"com.linkedin.390ac1c1431d5b4e340a65af24ed68f2"},"validDateRange":{"type":"com.linkedin.8fafa573919c2bc96c29a2be79ce3fb3"}},"baseType":"com.linkedin.voyager.dash.common.forms.validation.DateValidation"},"com.linkedin.0a38c5f17cb738fcfee67c772442d536":{"fields":{"screeningQuestions":{"type":{"array":"com.linkedin.55c2879f1a800fb9b17e54740f965515"}},"entityUrn":{"type":"com.linkedin.voyager.dash.common.ScreeningSurveyFormUrn"},"markedAsSeen":{"type":"boolean"}},"baseType":"com.linkedin.voyager.dash.jobs.assessments.ScreeningSurveyForm"},"com.linkedin.e5b59282f574d2493aeb20a65a9a1f94":{"fields":{"startedOn":{"type":"com.linkedin.fefbb3ce7d8629ad1cfd497514eb598e"},"employeeCount":{"type":"int"}},"baseType":"com.linkedin.voyager.dash.premium.companyinsights.CompanyInsightsEmployeeGrowthDetail"},"com.linkedin.b7cc21947d4a1ba974040324a832ba64":{"fields":{"employeeCount":{"type":"int"},"organizationUrn":{"type":"string"},"viewersOrganization":{"type":"boolean"},"organization":{"resolvedFrom":"organizationUrn","type":"com.linkedin.a2ef5c1c8d1bd7b83de7009b834a38c6"}},"baseType":"com.linkedin.voyager.dash.premium.companyinsights.CompanyInsightsOrganizationRankingDetail"},"com.linkedin.e94906839b912dcb4829422361b5028c":{"fields":{"ringContentType":{"type":"com.linkedin.voyager.dash.common.image.RingContentType"},"actionTarget":{"type":"com.linkedin.common.Url"},"preDashEntityUrn":{"type":"com.linkedin.voyager.dash.common.PreDashUrn"},"entityUrn":{"type":"com.linkedin.voyager.dash.common.RingStatusUrn"},"emphasized":{"type":"boolean"}},"baseType":"com.linkedin.voyager.dash.common.image.RingStatus"},"com.linkedin.2791a0d398e611fad8c820c60183e2bb":{"fields":{"recommendations":{"type":{"array":"com.linkedin.939138a9c6796d21af12eaf891a20e80"}},"branding":{"type":"com.linkedin.voyager.dash.learning.jobDetails.Branding"}},"baseType":"com.linkedin.voyager.dash.learning.LearningRecommendationCard"},"com.linkedin.ce28f52e436b3cdd8100675b5288a600":{"fields":{"controlName":{"type":"com.linkedin.voyager.dash.common.tracking.ControlName"},"viewingBehavior":{"type":"com.linkedin.voyager.dash.common.ux.UrlViewingBehavior"},"sponsoredUrlAttributes":{"type":"com.linkedin.7210b8f140dcf756fa9cea58d6413a4b"},"url":{"type":"com.linkedin.common.Url"}},"baseType":"com.linkedin.voyager.dash.common.text.TextLink"},"com.linkedin.0da643628377886b9fb1a0bb0cc0b73c":{"fields":{"hasConfirmedEmailAddress":{"type":"boolean"},"applyStartersPreferenceVoid":{"type":"boolean"},"applied":{"type":"boolean"},"jobDraftApplicationInfo":{"type":"com.linkedin.ff0a7ae4688d8d97ea2151c00e6e8392"},"formattedApplyDate":{"type":"string"},"onsiteApply":{"type":"boolean"},"applyCtaText":{"type":"com.linkedin.390ac1c1431d5b4e340a65af24ed68f2"},"entityUrn":{"type":"com.linkedin.voyager.dash.common.JobSeekerApplicationDetailUrn"},"inPageOffsiteApply":{"type":"boolean"},"applicantTrackingSystemName":{"type":"string"},"preApplySafetyTips":{"type":{"array":"com.linkedin.9196412bf6c5c0443810d690321ab8f0"}},"ambryResumeContainer":{"type":"string"},"companyApplyUrl":{"type":"com.linkedin.common.Url"},"appliedAt":{"type":"com.linkedin.common.Time"}},"baseType":"com.linkedin.voyager.dash.jobs.JobSeekerApplicationDetail"},"com.linkedin.c734e1c934a3cbf9562953df54d4e771":{"fields":{"startDateText":{"type":"com.linkedin.390ac1c1431d5b4e340a65af24ed68f2"},"startDateControlName":{"type":"com.linkedin.voyager.dash.common.tracking.ControlName"},"endDateControlName":{"type":"com.linkedin.voyager.dash.common.tracking.ControlName"},"ongoingDateRangeFormElementUrn":{"type":"com.linkedin.common.Urn"},"endDateValidation":{"type":"com.linkedin.48407601ffa8bd83b8651487ea606df9"},"endDateText":{"type":"com.linkedin.390ac1c1431d5b4e340a65af24ed68f2"},"startDateValidation":{"type":"com.linkedin.48407601ffa8bd83b8651487ea606df9"},"ongoingDateText":{"type":"com.linkedin.390ac1c1431d5b4e340a65af24ed68f2"},"dateInputType":{"type":"com.linkedin.voyager.dash.common.forms.DateInputType"}},"baseType":"com.linkedin.voyager.dash.common.forms.DateRangeFormComponent"},"com.linkedin.274132557f0c6d4e240f7372d489e1cc":{"fields":{"controlName":{"type":"string"},"numFractionalDigits":{"type":"int"},"hintText":{"type":"string"},"helperText":{"type":"string"},"numericRangeValidation":{"type":"com.linkedin.9ddc780810d0c93fa698a26faab52f10"}},"baseType":"com.linkedin.voyager.dash.common.forms.NumberInputFormComponent"},"com.linkedin.1b53ff96dc1e6cc3644919ffccaddf68":{"fields":{"applicantCount":{"type":"long"},"skillDetails":{"type":{"array":"com.linkedin.f3f2514a55c0f5153f3bd99b35ea8e2b"}},"degreeDetails":{"type":{"array":"com.linkedin.eef1cf048228a183463b81d9c6790e5f"}},"seniorityDetails":{"type":{"array":"com.linkedin.1b210594d26c621d240d5cf0136a8bdd"}},"applicantsInPastDay":{"type":"long"},"entityUrn":{"type":"com.linkedin.voyager.dash.common.JobApplicantInsightsUrn"},"applicantRankPercentile":{"type":"int"},"locationDetails":{"type":{"array":"com.linkedin.2c280c86f261412efd5c2e0d60bd88d4"}},"applicantThresholdMet":{"type":"boolean"}},"baseType":"com.linkedin.voyager.dash.premium.insights.JobApplicantInsights"},"com.linkedin.c4a89ad017a52048790e89ac5ba1ab35":{"fields":{"viewImpressionTrackingKey":{"type":"string"},"title":{"type":"string"},"sections":{"type":{"array":"com.linkedin.2d03db73fb8c8c66e4245b79a3d25e52"}},"subtitle":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.jobs.JobSegmentAttributesCard"},"com.linkedin.b49a045df66f28b2a65f3df75d119eae":{"fields":{"icon":{"type":"com.linkedin.voyager.dash.common.ArtDecoIconName"},"iconBeforeCtaText":{"type":"boolean"}},"baseType":"com.linkedin.voyager.dash.jobs.CtaIcon"},"com.linkedin.4fb8cecee08f0c588138694ff045dfd9":{"fields":{"latestHeadcountByFunction":{"type":"com.linkedin.acd5ebb13f6e0c9da292f5d03c57eac4"},"headcountGrowthByFunction":{"type":{"array":"com.linkedin.cca42244a99771eb4b1bfffed219f853"}}},"baseType":"com.linkedin.voyager.dash.premium.companyinsights.FunctionHeadcountInsights"},"com.linkedin.32bab5d7af29a8bdf4fe920d7d2155b1":{"fields":{"name":{"type":"string"},"logo":{"type":{"union":{"url":"com.linkedin.common.Url","vectorImage":"com.linkedin.b8949adeaaa915212d0136c4b9a1952b"}}},"logoResolutionResult":{"derivedFrom":"logo","type":{"union":{"url":"com.linkedin.common.Url","vectorImage":"com.linkedin.b8949adeaaa915212d0136c4b9a1952b"}}},"entityUrn":{"type":"com.linkedin.voyager.dash.common.GroupUrn"}},"baseType":"com.linkedin.voyager.dash.groups.Group"},"com.linkedin.c754100718c581ece9a715a9165db201":{"fields":{"controlName":{"type":"com.linkedin.voyager.dash.common.tracking.ControlName"},"renderingStyle":{"type":"com.linkedin.voyager.dash.common.forms.TypeaheadCtaRenderingStyle"},"typeaheadType":{"type":"com.linkedin.voyager.dash.search.TypeaheadType"},"typeaheadQueryContext":{"type":{"array":"string"}},"typeaheadMetadata":{"type":"com.linkedin.2dea04d4082c6960578ac30a3ace5ee0"},"labelText":{"type":"com.linkedin.390ac1c1431d5b4e340a65af24ed68f2"}},"baseType":"com.linkedin.voyager.dash.common.forms.TypeaheadCta"},"com.linkedin.8409c9296c5bda7be8aee44feff66dd3":{"fields":{"numericValueRangeValidation":{"type":"com.linkedin.9ddc780810d0c93fa698a26faab52f10"}},"baseType":"com.linkedin.voyager.dash.common.forms.validation.NumericValidationMetadata"},"com.linkedin.43d5fcbafc4556d0c474dabd56f017a2":{"fields":{"controlName":{"type":"com.linkedin.voyager.dash.common.tracking.ControlName"},"appearance":{"type":"com.linkedin.deaeaf135b51841f133d0ceae7eca621"},"openExternally":{"type":"boolean"},"accessibilityText":{"type":"string"},"disable":{"type":"boolean"},"url":{"type":"com.linkedin.common.Url"}},"baseType":"com.linkedin.voyager.dash.premium.PremiumButton"},"com.linkedin.8d64ead1eeea51a297dc70da47337de1":{"fields":{"width":{"type":"int"},"fileIdentifyingUrlPathSegment":{"type":"string"},"expiresAt":{"type":"com.linkedin.common.Time"},"height":{"type":"int"}},"baseType":"com.linkedin.common.VectorArtifact"},"com.linkedin.7210b8f140dcf756fa9cea58d6413a4b":{"fields":{"impressionId":{"type":"string"},"unwrappedUrlDomain":{"type":"string"},"creativeId":{"type":"string"},"campaignId":{"type":"string"},"memberId":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.common.sponsored.SponsoredUrlAttributes"},"com.linkedin.b0ac2d5e2d9d20cb81b800352f82dc8e":{"fields":{"icon":{"type":"com.linkedin.voyager.dash.common.ArtDecoIconName"},"controlName":{"type":"com.linkedin.voyager.dash.common.tracking.ControlName"},"positiveToggleLabel":{"type":"com.linkedin.390ac1c1431d5b4e340a65af24ed68f2"},"negativeToggleLabel":{"type":"com.linkedin.390ac1c1431d5b4e340a65af24ed68f2"}},"baseType":"com.linkedin.voyager.dash.common.forms.ToggleFormComponent"},"com.linkedin.64aff242d842d2f5c0d6ea1432ceb79a":{"fields":{"applyJobActionResolutionResult":{"resolvedFrom":"applyJobAction","type":"com.linkedin.0da643628377886b9fb1a0bb0cc0b73c"},"applyControlName":{"type":"com.linkedin.voyager.dash.common.tracking.ControlName"},"applyJobAction":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.jobs.ApplyJobAction"},"com.linkedin.f3f2514a55c0f5153f3bd99b35ea8e2b":{"fields":{"count":{"type":"int"},"skillUrn":{"type":"string"},"skill":{"resolvedFrom":"skillUrn","type":"com.linkedin.7a3dfe06b5ebab0041d8cea76d58565a"},"viewersHasSkill":{"type":"boolean"}},"baseType":"com.linkedin.voyager.dash.premium.insights.JobApplicantInsightsSkillDetail"},"com.linkedin.6e88a2e94acf8ea5aa8c9f7051705709":{"fields":{},"baseType":"com.linkedin.voyager.dash.jobs.search.NewCollectionHeaderCard"},"com.linkedin.c9156c9beebda6b437868e48c22a8f02":{"fields":{"ringStatus":{"type":"com.linkedin.e94906839b912dcb4829422361b5028c"},"profileUrn":{"type":"string"},"vectorImage":{"type":"com.linkedin.b8949adeaaa915212d0136c4b9a1952b"},"profile":{"resolvedFrom":"profileUrn","type":"com.linkedin.826b8c485ed1073773697502413b9597"}},"baseType":"com.linkedin.voyager.dash.common.image.NonEntityProfilePicture"},"com.linkedin.036f42f6cc87d5881cf2351020672e97":{"fields":{"insightViewModel":{"type":"com.linkedin.3551030d48734b4fd9cb3238efc77ece"},"responseGroup":{"type":"com.linkedin.voyager.dash.jobs.ResponsivenessActionGroup"},"learnMoreControlName":{"type":"com.linkedin.voyager.dash.common.tracking.ControlName"}},"baseType":"com.linkedin.voyager.dash.jobs.ResponsivenessInsightViewModel"},"com.linkedin.8fafa573919c2bc96c29a2be79ce3fb3":{"fields":{"start":{"type":"com.linkedin.fefbb3ce7d8629ad1cfd497514eb598e"},"end":{"type":"com.linkedin.fefbb3ce7d8629ad1cfd497514eb598e"}},"baseType":"com.linkedin.common.DateRange"},"com.linkedin.7a3dfe06b5ebab0041d8cea76d58565a":{"fields":{"name":{"type":"string"},"entityUrn":{"type":"com.linkedin.voyager.dash.common.StandardizedSkillUrn"}},"baseType":"com.linkedin.voyager.dash.jobs.StandardizedSkill"},"com.linkedin.194482b00bd3b5ae56a4d81712d0525c":{"fields":{"inputEntityName":{"type":"string"},"inputEntityUrn":{"type":"com.linkedin.common.Urn"}},"baseType":"com.linkedin.voyager.dash.common.forms.EntityInputValue"},"com.linkedin.3df29ea52bb4c2126ad5c1daa0ab5b01":{"fields":{"entityLockup":{"type":"com.linkedin.bd66bae736eabe6505d1b80c182e31af"},"exitedPosition":{"type":"com.linkedin.390ac1c1431d5b4e340a65af24ed68f2"},"exitYearMonthOn":{"type":"com.linkedin.fefbb3ce7d8629ad1cfd497514eb598e"}},"baseType":"com.linkedin.voyager.dash.premium.companyinsights.AlumniItem"},"com.linkedin.e1e02f8665f14f794053a412332f146a":{"fields":{"action":{"type":"com.linkedin.5de46613f3027e5a41b356871519a78b"},"description":{"type":"com.linkedin.390ac1c1431d5b4e340a65af24ed68f2"}},"baseType":"com.linkedin.voyager.dash.jobs.JobQualificationDescription"},"com.linkedin.ca1fb70631b137b6d459efdae8a18bc0":{"fields":{"image":{"derivedFrom":"imageUnion","type":{"union":{"url":"com.linkedin.common.Url","vectorImage":"com.linkedin.b8949adeaaa915212d0136c4b9a1952b"}}},"nameSupplementaryInfo":{"type":"com.linkedin.390ac1c1431d5b4e340a65af24ed68f2"},"footerText":{"type":"com.linkedin.390ac1c1431d5b4e340a65af24ed68f2"},"imageUnion":{"type":{"union":{"url":"com.linkedin.common.Url","vectorImage":"com.linkedin.b8949adeaaa915212d0136c4b9a1952b"}}},"backgroundImage":{"derivedFrom":"backgroundImageUnion","type":{"union":{"url":"com.linkedin.common.Url","vectorImage":"com.linkedin.b8949adeaaa915212d0136c4b9a1952b"}}},"subHeadline":{"type":"com.linkedin.390ac1c1431d5b4e340a65af24ed68f2"},"name":{"type":"com.linkedin.390ac1c1431d5b4e340a65af24ed68f2"},"insightText":{"type":"com.linkedin.390ac1c1431d5b4e340a65af24ed68f2"},"headline":{"type":"com.linkedin.390ac1c1431d5b4e340a65af24ed68f2"},"backgroundImageUnion":{"type":{"union":{"url":"com.linkedin.common.Url","vectorImage":"com.linkedin.b8949adeaaa915212d0136c4b9a1952b"}}}},"baseType":"com.linkedin.voyager.dash.common.media.StickerLinkLargeTemplateView"},"com.linkedin.7948e5031442b35b939b7672eac8bad0":{"fields":{"entityUrn":{"type":"com.linkedin.voyager.dash.common.HashtagUrn"},"trackingUrn":{"type":"com.linkedin.common.Urn"},"actionTarget":{"type":"com.linkedin.common.Url"}},"baseType":"com.linkedin.voyager.dash.feed.Hashtag"},"com.linkedin.cca42244a99771eb4b1bfffed219f853":{"fields":{"functionUrn":{"type":"string"},"growthPeriods":{"type":{"array":"com.linkedin.f6e5193777aa8860860ecb48d0212cce"}},"function":{"resolvedFrom":"functionUrn","type":"com.linkedin.39c50e73d5d652d9f5f7194f5023e967"}},"baseType":"com.linkedin.voyager.dash.premium.companyinsights.GrowthByFunction"},"com.linkedin.5481c9cf9c53bc965a2e49fdf5e17760":{"fields":{"contextualSuggestionDependent":{"resolvedFrom":"contextualSuggestionDependentUrns","type":{"array":"com.linkedin.258b7479f7c0999032ce22032fa02cdf"}},"contextualSuggestionQueryParameterUrns":{"type":{"array":"string"}},"contextualSuggestionDependentUrns":{"type":{"array":"string"}},"contextualSuggestionQueryParameter":{"resolvedFrom":"contextualSuggestionQueryParameterUrns","type":{"array":"com.linkedin.258b7479f7c0999032ce22032fa02cdf"}},"contextualSuggestionType":{"type":"com.linkedin.voyager.dash.search.TypeaheadType"}},"baseType":"com.linkedin.voyager.dash.common.forms.ContextualSuggestionsTriggers"},"com.linkedin.2c280c86f261412efd5c2e0d60bd88d4":{"fields":{"geo":{"resolvedFrom":"geoUrn","type":"com.linkedin.00c4f529ff2fa302d89eaea66344f268"},"numApplicants":{"type":"string"},"geoUrn":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.premium.insights.JobApplicantInsightsLocationDetail"},"com.linkedin.22e0be7fe535e4176ecde39607391326":{"fields":{"controlName":{"type":"com.linkedin.voyager.dash.common.tracking.ControlName"},"hintText":{"type":"com.linkedin.390ac1c1431d5b4e340a65af24ed68f2"},"numVisibleLines":{"type":"int"},"helperText":{"type":"com.linkedin.390ac1c1431d5b4e340a65af24ed68f2"},"validationMetadataResolutionResult":{"derivedFrom":"validationMetadata","type":{"union":{"pronounsV2":"com.linkedin.bfd20979d76e9c8ce942bc03238c6b02","string":"com.linkedin.bfd20979d76e9c8ce942bc03238c6b02","textFormInputType":"com.linkedin.voyager.dash.common.forms.validation.TextFormInputType","postalCode":"com.linkedin.2b6a63a1bb3f7b0bf5c50d8530db9f78","name":"com.linkedin.bfd20979d76e9c8ce942bc03238c6b02","pronouns":"string","integer":"com.linkedin.8409c9296c5bda7be8aee44feff66dd3","decimal":"com.linkedin.8409c9296c5bda7be8aee44feff66dd3","headline":"com.linkedin.bfd20979d76e9c8ce942bc03238c6b02","url":"com.linkedin.48be3e5c4d0dfb61ba0a4a296e719909"}}},"validationMetadata":{"type":{"union":{"pronounsV2":"com.linkedin.bfd20979d76e9c8ce942bc03238c6b02","string":"com.linkedin.bfd20979d76e9c8ce942bc03238c6b02","textFormInputType":"com.linkedin.voyager.dash.common.forms.validation.TextFormInputType","postalCode":"com.linkedin.2b6a63a1bb3f7b0bf5c50d8530db9f78","name":"com.linkedin.bfd20979d76e9c8ce942bc03238c6b02","pronouns":"string","integer":"com.linkedin.8409c9296c5bda7be8aee44feff66dd3","decimal":"com.linkedin.8409c9296c5bda7be8aee44feff66dd3","headline":"com.linkedin.bfd20979d76e9c8ce942bc03238c6b02","url":"com.linkedin.48be3e5c4d0dfb61ba0a4a296e719909"}}}},"baseType":"com.linkedin.voyager.dash.common.forms.MultilineTextFormComponent"},"com.linkedin.8b1684dd351b279ed40c6661257d0e63":{"fields":{},"baseType":"com.linkedin.restli.common.EmptyRecord"},"com.linkedin.23ea16078c6de6222c73b55c40e8e8ab":{"fields":{"linkUrl":{"type":"com.linkedin.common.Url"},"controlName":{"type":"com.linkedin.voyager.dash.common.tracking.ControlName"},"linkText":{"type":"string"},"text":{"type":"string"},"type":{"type":"com.linkedin.voyager.dash.common.ux.InlineFeedbackType"}},"baseType":"com.linkedin.voyager.dash.common.ux.InlineFeedbackViewModel"},"com.linkedin.dd03bc4de44734367e478c7c8eed60a0":{"fields":{"title":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.jobs.JobsFeedModuleHeader"},"com.linkedin.b7e88af0a8aa84677bc8effdf13ab5d9":{"fields":{"countryUrnRequired":{"type":"boolean"},"locationField":{"type":"com.linkedin.fe3639bb2accd8987c7e8ed7912c8eff"},"requiredFieldMissingErrorText":{"type":"com.linkedin.390ac1c1431d5b4e340a65af24ed68f2"},"responseRequired":{"type":"boolean"},"labelText":{"type":"com.linkedin.390ac1c1431d5b4e340a65af24ed68f2"}},"baseType":"com.linkedin.voyager.dash.common.forms.LocationTypeaheadEntityField"},"com.linkedin.bfb56f758ba39329fbb4081cfe291d5e":{"fields":{"controlName":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.common.forms.StarRatingFormComponent"},"com.linkedin.98d89bae4aa4ef8f3ffc1018ca93419c":{"fields":{"optionUnion":{"type":{"union":{"optionUrn":"com.linkedin.common.Urn","optionEnumString":"string"}}},"controlName":{"type":"com.linkedin.voyager.dash.common.tracking.ControlName"},"optionUrn":{"type":"com.linkedin.common.Urn"},"optionEnumString":{"type":"string"},"optionText":{"type":"com.linkedin.390ac1c1431d5b4e340a65af24ed68f2"},"option":{"derivedFrom":"optionUnion","type":{"union":{"optionUrn":"com.linkedin.common.Urn","optionEnumString":"string"}}}},"baseType":"com.linkedin.voyager.dash.common.forms.TextSelectableOption"},"com.linkedin.f6e5193777aa8860860ecb48d0212cce":{"fields":{"changePercentage":{"type":"int"},"monthDifference":{"type":"int"}},"baseType":"com.linkedin.voyager.dash.premium.companyinsights.GrowthPeriod"},"com.linkedin.d2acc160e7db812cd2fc11b5eaac7b0f":{"fields":{"qualification":{"derivedFrom":"qualificationUnion","type":{"union":{"qualificationDescription":"com.linkedin.e1e02f8665f14f794053a412332f146a","qualificationList":"com.linkedin.f26ce46d6c12df14239f36d844033955"}}},"header":{"type":"string"},"qualificationUnion":{"type":{"union":{"qualificationDescription":"com.linkedin.e1e02f8665f14f794053a412332f146a","qualificationList":"com.linkedin.f26ce46d6c12df14239f36d844033955"}}},"subheader":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.jobs.JobQualificationGroup"},"com.linkedin.7269fdb3a0f2f75ea98f76d3d3c9dfc3":{"fields":{"jobPostingRelevanceReasons":{"type":{"array":"com.linkedin.f4f99771435399029308495de4e742b6"}},"employeeGrowthForFunction":{"type":"com.linkedin.e276a8a343fc735c387871abc9190c7d"},"formattedJobFunction":{"type":"string"},"granularity":{"type":"com.linkedin.voyager.dash.premium.companyinsights.CompanyInsightsGranularity"},"employeeGrowth":{"type":"com.linkedin.e276a8a343fc735c387871abc9190c7d"},"company":{"resolvedFrom":"companyUrn","type":"com.linkedin.a2ef5c1c8d1bd7b83de7009b834a38c6"},"companyRanking":{"type":"com.linkedin.34436f930667efbac1e7e0a600c8c653"},"companyUrn":{"type":"string"},"schoolRanking":{"type":"com.linkedin.34436f930667efbac1e7e0a600c8c653"},"talentSourcesTitle":{"type":"com.linkedin.390ac1c1431d5b4e340a65af24ed68f2"}},"baseType":"com.linkedin.voyager.dash.premium.companyinsights.JobPostingCompanyInsights"},"com.linkedin.d85acf0e41970312e8912b9455cefa24":{"fields":{"dismissed":{"type":"boolean"},"entityUrn":{"type":"com.linkedin.voyager.dash.common.JobPostingRelevanceFeedbackUrn"},"dismissedEntityRelevanceFeedbackUrn":{"type":"string"},"dismissedEntityRelevanceFeedback":{"resolvedFrom":"dismissedEntityRelevanceFeedbackUrn","type":"com.linkedin.d8e085065c27e5a4b4a3b47e39d34eae"}},"baseType":"com.linkedin.voyager.dash.jobs.feedback.JobPostingRelevanceFeedback"},"com.linkedin.a7d235a42f3653f9539026976306f11b":{"fields":{"school":{"resolvedFrom":"schoolUrn","type":"com.linkedin.d4dcbabbf47d9675b0597099d9cd7d69"},"vectorImage":{"type":"com.linkedin.b8949adeaaa915212d0136c4b9a1952b"},"schoolUrn":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.common.image.NonEntitySchoolLogo"},"com.linkedin.bb47d92b56e85e0625cd6b7f5d3d0384":{"fields":{"entityUrn":{"type":"com.linkedin.voyager.dash.common.ConversationUrn"}},"baseType":"com.linkedin.voyager.dash.messaging.Conversation"},"com.linkedin.6c333b78b6aeac833ef88e8174932712":{"fields":{"type":{"type":"com.linkedin.voyager.dash.jobs.JobPostingCardFooterItemType"},"timeAt":{"type":"com.linkedin.common.Time"},"text":{"type":"com.linkedin.390ac1c1431d5b4e340a65af24ed68f2"}},"baseType":"com.linkedin.voyager.dash.jobs.JobPostingCardFooterItem"},"com.linkedin.f4f99771435399029308495de4e742b6":{"fields":{"relevanceReason":{"type":"com.linkedin.390ac1c1431d5b4e340a65af24ed68f2"},"currentCompanyUrn":{"type":"string"},"currentCompany":{"resolvedFrom":"currentCompanyUrn","type":"com.linkedin.a2ef5c1c8d1bd7b83de7009b834a38c6"},"relevanceReasonFlavor":{"type":"com.linkedin.voyager.dash.premium.companyinsights.JobPostingRelevanceReasonFlavor"}},"baseType":"com.linkedin.voyager.dash.premium.companyinsights.JobPostingRelevanceReason"},"com.linkedin.f0aa7f6258b1a3e754f7d6bbfc9af84b":{"fields":{"postApplyPromoType":{"resolvedFrom":"postApplyPromoTypeUnion","type":{"union":{"interviewPrep":{"array":"com.linkedin.85d03babfc1dc1ea8537cef9b2cc1bf5"},"eeocConsent":"com.linkedin.8b1684dd351b279ed40c6661257d0e63","premiumUpsell":"com.linkedin.dc4c6e3940e502e42eb82438140331c7","makeMeMove":"com.linkedin.8b1684dd351b279ed40c6661257d0e63","skillAssessments":{"array":"com.linkedin.4f63adbedb57590490ae8c38e3318a15"},"addSkill":"com.linkedin.8b1684dd351b279ed40c6661257d0e63","paidCompany":"com.linkedin.8b1684dd351b279ed40c6661257d0e63","similarJobs":"com.linkedin.8b1684dd351b279ed40c6661257d0e63","screeningSurvey":"com.linkedin.0a38c5f17cb738fcfee67c772442d536","verification":"com.linkedin.8b1684dd351b279ed40c6661257d0e63","demographics":"com.linkedin.8b1684dd351b279ed40c6661257d0e63"}}},"summary":{"type":"com.linkedin.390ac1c1431d5b4e340a65af24ed68f2"},"cta":{"type":"com.linkedin.390ac1c1431d5b4e340a65af24ed68f2"},"description":{"type":"com.linkedin.390ac1c1431d5b4e340a65af24ed68f2"},"postApplyPromoTypeUnion":{"type":{"union":{"interviewPrep":{"array":"com.linkedin.85d03babfc1dc1ea8537cef9b2cc1bf5"},"eeocConsent":"com.linkedin.8b1684dd351b279ed40c6661257d0e63","premiumUpsell":"string","makeMeMove":"com.linkedin.8b1684dd351b279ed40c6661257d0e63","skillAssessments":"string","addSkill":"com.linkedin.8b1684dd351b279ed40c6661257d0e63","paidCompany":"com.linkedin.8b1684dd351b279ed40c6661257d0e63","similarJobs":"com.linkedin.8b1684dd351b279ed40c6661257d0e63","screeningSurvey":"string","verification":"com.linkedin.8b1684dd351b279ed40c6661257d0e63","demographics":"com.linkedin.8b1684dd351b279ed40c6661257d0e63"}}},"legoTrackingToken":{"type":"string"},"renderType":{"type":"com.linkedin.voyager.dash.jobs.postapply.PostApplyPromoCardRenderType"}},"baseType":"com.linkedin.voyager.dash.jobs.postapply.PostApplyPromo"},"com.linkedin.7bd2de9a7b9fadfe651a8dd1c13db7ad":{"fields":{"subOptions":{"type":{"array":"com.linkedin.98d89bae4aa4ef8f3ffc1018ca93419c"}},"topLevelOption":{"type":"com.linkedin.98d89bae4aa4ef8f3ffc1018ca93419c"}},"baseType":"com.linkedin.voyager.dash.common.forms.NestedTextSelectableOption"},"com.linkedin.dbb12a0149e541da7067f13d22b542eb":{"fields":{"start":{"type":"float"},"end":{"type":"float"}},"baseType":"com.linkedin.common.FloatRange"},"com.linkedin.65862eebdeb12a456370f32142eebd8c":{"fields":{"suggestedEntity":{"type":"com.linkedin.98d89bae4aa4ef8f3ffc1018ca93419c"},"navigationUrl":{"type":"com.linkedin.common.Url"}},"baseType":"com.linkedin.voyager.dash.common.forms.ActionableSuggestedEntity"},"com.linkedin.81315c1c0ea01a8f7161ae78bbbd0ff1":{"fields":{"jobTrackingData":{"type":"com.linkedin.e53bf26a30b7d5901401fa6088b455dc"},"jobPostingCardUrn":{"type":"string"},"jobPostingCard":{"resolvedFrom":"jobPostingCardUrn","type":"com.linkedin.f42293b24e5d60e317d2d68ab7597f64"}},"baseType":"com.linkedin.voyager.dash.jobs.JobPostingCardWrapper"},"com.linkedin.5d8878fd7b4508fcbef4f2d67c765db3":{"fields":{"memberDistance":{"type":"com.linkedin.voyager.dash.relationships.NoConnectionMemberDistance"}},"baseType":"com.linkedin.voyager.dash.relationships.NoConnection"},"com.linkedin.8e19fba94618da7ceb9cca894420a5da":{"fields":{"entityUrn":{"type":"com.linkedin.voyager.dash.common.CoachJobsKickoffPromptsUrn"},"kickoffPrompts":{"type":{"array":"com.linkedin.70094a2b1194a3dfad82481287a40a81"}}},"baseType":"com.linkedin.voyager.dash.jobs.gai.CoachJobsKickoffPrompts"},"com.linkedin.847ffe38969d62d2505e06cf8cbfaa3d":{"fields":{"functionPercentage":{"type":"int"},"functionUrn":{"type":"string"},"functionCount":{"type":"int"},"function":{"resolvedFrom":"functionUrn","type":"com.linkedin.39c50e73d5d652d9f5f7194f5023e967"}},"baseType":"com.linkedin.voyager.dash.premium.companyinsights.FunctionCount"},"com.linkedin.59b66a5c22b53ee350982aba9da43296":{"fields":{"formattedBaseSalary":{"type":"string"},"geoLocationUrn":{"type":"string"},"geoLocation":{"resolvedFrom":"geoLocationUrn","type":"com.linkedin.00c4f529ff2fa302d89eaea66344f268"},"entityUrn":{"type":"com.linkedin.voyager.dash.common.SalaryInsightsUrn"},"formattedAdditionalCompensation":{"type":"string"},"collectAccuracyFeedback":{"type":"boolean"},"compensationSource":{"type":"com.linkedin.voyager.dash.salary.CompensationSource"},"additionalCompensationTypes":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.salary.SalaryInsights"},"com.linkedin.70094a2b1194a3dfad82481287a40a81":{"fields":{"queryText":{"type":"string"},"controlName":{"type":"com.linkedin.voyager.dash.common.tracking.ControlName"},"coachSparkleIcon":{"type":"com.linkedin.30e1ec6a491035bd7a31cc9de8a12c18"},"isPremium":{"type":"boolean"},"title":{"type":"com.linkedin.390ac1c1431d5b4e340a65af24ed68f2"},"toolTipText":{"type":"string"},"coachQueryContextV2":{"type":"com.linkedin.8864b68d440629f1899c8ff463b7f71d"},"coachUseCaseType":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.jobs.gai.CoachJobsKickoffPrompt"},"com.linkedin.be7717be8d269f7c62e51d9156652c9b":{"fields":{"navigationAction":{"type":"com.linkedin.5de46613f3027e5a41b356871519a78b"},"text":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.jobs.ItemsMatchDescription"},"com.linkedin.b4d79116ccce85e0d4ef36bf211da0a3":{"fields":{"singleQuestionSubForm":{"type":"com.linkedin.554b975e2cabf1c2139af63808fd028a"},"icon":{"type":"com.linkedin.30e1ec6a491035bd7a31cc9de8a12c18"},"controlName":{"type":"com.linkedin.voyager.dash.common.tracking.ControlName"},"iconAfterText":{"type":"boolean"},"placeHolderText":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.common.forms.VisibilitySettingButton"},"com.linkedin.bb168eba9e814ac2c87fa5288ea74e6f":{"fields":{"image":{"type":"com.linkedin.voyager.dash.common.SystemImageName"},"title":{"type":"string"},"imageColor":{"type":"com.linkedin.voyager.dash.common.SystemImageTintColor"},"navigationAction":{"type":"com.linkedin.5de46613f3027e5a41b356871519a78b"},"subtitle":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.jobs.ItemsMatchItem"},"com.linkedin.60257368060941d7900dbbf2748f711c":{"fields":{"postalCodeTextField":{"type":"com.linkedin.2f3c656d762dff10246b267bedddfcfd"},"cityTypeaheadField":{"type":"com.linkedin.b7e88af0a8aa84677bc8effdf13ab5d9"},"useCurrentLocation":{"type":"com.linkedin.0608c02e08bf95759764fc707e772cb9"},"countryField":{"type":"com.linkedin.b7e88af0a8aa84677bc8effdf13ab5d9"},"cityTextEntityListField":{"type":"com.linkedin.84ab2fe95ad74fa18ab07857b1c58889"}},"baseType":"com.linkedin.voyager.dash.common.forms.LocationFormComponent"},"com.linkedin.a1cf47049f3ce4950c977d8052ae04d3":{"fields":{"viewImpressionTrackingKey":{"type":"string"},"legoTrackingToken":{"type":"string"},"text":{"type":"com.linkedin.390ac1c1431d5b4e340a65af24ed68f2"},"type":{"type":"com.linkedin.voyager.dash.jobs.JobPostingTipType"}},"baseType":"com.linkedin.voyager.dash.jobs.JobPostingTip"},"com.linkedin.4c9ae0bb91144f3556c7bcc9bd1b33bd":{"fields":{"entityUrn":{"type":"com.linkedin.voyager.dash.common.SkillAssessmentAttemptReportUrn"}},"baseType":"com.linkedin.voyager.dash.assessments.SkillAssessmentAttemptReport"},"com.linkedin.2d03db73fb8c8c66e4245b79a3d25e52":{"fields":{"title":{"type":"string"},"subtitle":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.jobs.JobSegmentAttributesCardSection"},"com.linkedin.296fd4ebc3f7a46ccca548d534f656e4":{"fields":{"detailData":{"resolvedFrom":"detailDataUnion","type":{"union":{"profilePictureWithoutFrame":"com.linkedin.826b8c485ed1073773697502413b9597","profilePictureWithRingStatus":"com.linkedin.826b8c485ed1073773697502413b9597","companyLogo":"com.linkedin.a2ef5c1c8d1bd7b83de7009b834a38c6","icon":"com.linkedin.voyager.dash.common.ArtDecoIconName","systemImage":"com.linkedin.voyager.dash.common.SystemImageName","nonEntityGroupLogo":"com.linkedin.cdabf7a3337e639c42189d7fe44cdc92","vectorImage":"com.linkedin.b8949adeaaa915212d0136c4b9a1952b","nonEntityProfessionalEventLogo":"com.linkedin.6e3e751d09550f922978dedd7f52eea5","profilePicture":"com.linkedin.826b8c485ed1073773697502413b9597","imageUrl":"com.linkedin.2ef5b25d61e5db956d269b94a55f4763","professionalEventLogo":"com.linkedin.3900e4e094a1b889ef9285a430d6f2d5","nonEntitySchoolLogo":"com.linkedin.a7d235a42f3653f9539026976306f11b","nonEntityCompanyLogo":"com.linkedin.5fe8101cb3a914392f7301964e56573c","schoolLogo":"com.linkedin.d4dcbabbf47d9675b0597099d9cd7d69","groupLogo":"com.linkedin.32bab5d7af29a8bdf4fe920d7d2155b1","ghostImage":"com.linkedin.voyager.dash.common.image.GhostImageType","nonEntityProfilePicture":"com.linkedin.c9156c9beebda6b437868e48c22a8f02"}}},"tintColor":{"type":"com.linkedin.voyager.dash.common.SystemImageTintColor"},"detailDataUnion":{"type":{"union":{"profilePictureWithoutFrame":"string","profilePictureWithRingStatus":"string","companyLogo":"string","icon":"com.linkedin.voyager.dash.common.ArtDecoIconName","systemImage":"com.linkedin.voyager.dash.common.SystemImageName","nonEntityGroupLogo":"com.linkedin.cdabf7a3337e639c42189d7fe44cdc92","vectorImage":"com.linkedin.b8949adeaaa915212d0136c4b9a1952b","nonEntityProfessionalEventLogo":"com.linkedin.6e3e751d09550f922978dedd7f52eea5","profilePicture":"string","imageUrl":"com.linkedin.2ef5b25d61e5db956d269b94a55f4763","professionalEventLogo":"string","nonEntitySchoolLogo":"com.linkedin.a7d235a42f3653f9539026976306f11b","nonEntityCompanyLogo":"com.linkedin.5fe8101cb3a914392f7301964e56573c","schoolLogo":"string","groupLogo":"string","ghostImage":"com.linkedin.voyager.dash.common.image.GhostImageType","nonEntityProfilePicture":"com.linkedin.c9156c9beebda6b437868e48c22a8f02"}}},"tapTargets":{"type":{"array":"com.linkedin.409356f1c0f808083c5e0174516009dd"}},"scalingType":{"type":"com.linkedin.voyager.dash.common.image.ScalingType"},"displayAspectRatio":{"type":"double"}},"baseType":"com.linkedin.voyager.dash.common.image.ImageAttribute"},"com.linkedin.acd5ebb13f6e0c9da292f5d03c57eac4":{"fields":{"countByFunction":{"type":{"array":"com.linkedin.847ffe38969d62d2505e06cf8cbfaa3d"}},"totalCount":{"type":"long"},"yearMonthOn":{"type":"com.linkedin.fefbb3ce7d8629ad1cfd497514eb598e"}},"baseType":"com.linkedin.voyager.dash.premium.companyinsights.CountByFunction"},"com.linkedin.72471cc7e55a55b0fd549127127633f7":{"fields":{"appearance":{"type":"com.linkedin.deaeaf135b51841f133d0ceae7eca621"},"controlName":{"type":"com.linkedin.voyager.dash.common.tracking.ControlName"},"icon":{"type":"com.linkedin.voyager.dash.common.ArtDecoIconName"},"openExternally":{"type":"boolean"},"navigationUrl":{"type":"com.linkedin.common.Url"},"text":{"type":"string"},"accessibilityText":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.common.forms.NavigationButton"},"com.linkedin.985210b44c74343f912e35f90d721799":{"fields":{"typeaheadCta":{"type":"com.linkedin.c754100718c581ece9a715a9165db201"},"description":{"type":"com.linkedin.390ac1c1431d5b4e340a65af24ed68f2"},"selectedEntities":{"type":{"array":"com.linkedin.9013fa15f12f02c028d5b08f542f32d5"}},"selectionCountRangeValidation":{"type":"com.linkedin.e18c41f5008d005150ad33f812748af9"}},"baseType":"com.linkedin.voyager.dash.common.forms.MultiSelectTypeaheadEntityFormComponent"},"com.linkedin.be19a4b9083f30fb8796eb20ea30c1d0":{"fields":{"maximumFileSizeSupported":{"type":"long"},"mimeTypes":{"type":{"array":"com.linkedin.voyager.dash.common.MimeType"}},"uploadFileControlName":{"type":"com.linkedin.voyager.dash.common.tracking.ControlName"},"uploadFileCtaText":{"type":"string"},"deleteControlName":{"type":"com.linkedin.voyager.dash.common.tracking.ControlName"},"mediaUploadType":{"type":"com.linkedin.voyager.dash.video.MediaUploadType"},"previewText":{"type":"com.linkedin.390ac1c1431d5b4e340a65af24ed68f2"},"helperText":{"type":"com.linkedin.390ac1c1431d5b4e340a65af24ed68f2"}},"baseType":"com.linkedin.voyager.dash.common.forms.MediaUploadFormComponent"},"com.linkedin.39c50e73d5d652d9f5f7194f5023e967":{"fields":{"entityUrn":{"type":"com.linkedin.voyager.dash.common.FunctionUrn"},"localizedName":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.premium.companyinsights.Function"},"com.linkedin.03248b99e950a8be3201e56283063ed8":{"fields":{"primaryCtaText":{"type":"string"},"image":{"type":"com.linkedin.30e1ec6a491035bd7a31cc9de8a12c18"},"entityUrn":{"type":"com.linkedin.voyager.dash.common.MarketplacePromoCardUrn"},"subtitle":{"type":"com.linkedin.390ac1c1431d5b4e340a65af24ed68f2"},"primaryCtaNavigationUrl":{"type":"com.linkedin.common.Url"},"title":{"type":"com.linkedin.390ac1c1431d5b4e340a65af24ed68f2"},"trackingId":{"type":"com.linkedin.common.TrackingId"}},"baseType":"com.linkedin.voyager.dash.marketplaces.PromoCard"},"com.linkedin.cdabf7a3337e639c42189d7fe44cdc92":{"fields":{"groupUrn":{"type":"string"},"vectorImage":{"type":"com.linkedin.b8949adeaaa915212d0136c4b9a1952b"},"group":{"resolvedFrom":"groupUrn","type":"com.linkedin.32bab5d7af29a8bdf4fe920d7d2155b1"}},"baseType":"com.linkedin.voyager.dash.common.image.NonEntityGroupLogo"},"com.linkedin.b2a671b0acb9520f18deade4e955c616":{"fields":{"name":{"type":"com.linkedin.390ac1c1431d5b4e340a65af24ed68f2"},"image":{"derivedFrom":"imageUnion","type":{"union":{"url":"com.linkedin.common.Url","vectorImage":"com.linkedin.b8949adeaaa915212d0136c4b9a1952b"}}},"nameSupplementaryInfo":{"type":"com.linkedin.390ac1c1431d5b4e340a65af24ed68f2"},"headline":{"type":"com.linkedin.390ac1c1431d5b4e340a65af24ed68f2"},"imageUnion":{"type":{"union":{"url":"com.linkedin.common.Url","vectorImage":"com.linkedin.b8949adeaaa915212d0136c4b9a1952b"}}},"subHeadline":{"type":"com.linkedin.390ac1c1431d5b4e340a65af24ed68f2"}},"baseType":"com.linkedin.voyager.dash.common.media.StickerLinkMediumTemplateView"},"com.linkedin.55c2879f1a800fb9b17e54740f965515":{"fields":{"footerText":{"type":"com.linkedin.390ac1c1431d5b4e340a65af24ed68f2"},"collapsedState":{"type":"com.linkedin.voyager.dash.common.forms.FormSectionCollapsedState"},"formElementGroups":{"type":{"array":"com.linkedin.6a325d9bf5f44baf8fede1db5a7ce66f"}},"subtitle":{"type":"com.linkedin.390ac1c1431d5b4e340a65af24ed68f2"},"repeatableSectionData":{"type":"com.linkedin.181ad18bcf938cd6016ba9ffd2137150"},"title":{"type":"com.linkedin.390ac1c1431d5b4e340a65af24ed68f2"},"footerTextTitle":{"type":"com.linkedin.390ac1c1431d5b4e340a65af24ed68f2"}},"baseType":"com.linkedin.voyager.dash.common.forms.FormSection"},"com.linkedin.7a7f0ab9f40a57ecd2a27e26059721fe":{"fields":{"name":{"type":"string"},"entityUrn":{"type":"com.linkedin.voyager.dash.common.IndustryV2Urn"}},"baseType":"com.linkedin.voyager.dash.identity.profile.IndustryV2"},"com.linkedin.9ddc780810d0c93fa698a26faab52f10":{"fields":{"errorText":{"type":"com.linkedin.390ac1c1431d5b4e340a65af24ed68f2"},"validRange":{"type":"com.linkedin.dbb12a0149e541da7067f13d22b542eb"}},"baseType":"com.linkedin.voyager.dash.common.forms.validation.NumericValueRangeValidation"},"com.linkedin.6ba053f5dc7e14e40216570d97a71e52":{"fields":{"placeHolderText":{"type":"com.linkedin.390ac1c1431d5b4e340a65af24ed68f2"},"controlName":{"type":"com.linkedin.voyager.dash.common.tracking.ControlName"},"textSelectableOptions":{"type":{"array":"com.linkedin.98d89bae4aa4ef8f3ffc1018ca93419c"}}},"baseType":"com.linkedin.voyager.dash.common.forms.DropdownFormComponent"},"com.linkedin.1ed8d64e266be8d739bffff5d4200fb4":{"fields":{},"baseType":"com.linkedin.voyager.dash.jobs.NoResultsCard"},"com.linkedin.81235c3224517a02609a065763927f01":{"fields":{"formElementInputValues":{"type":{"array":{"union":{"entityInputValue":"com.linkedin.194482b00bd3b5ae56a4d81712d0525c","locationInputValue":"com.linkedin.b429b28d37054e5232ecd5ecffd09315","integerInputValue":"int","dateRangeInputValue":"com.linkedin.8fafa573919c2bc96c29a2be79ce3fb3","urnInputValue":"com.linkedin.common.Urn","floatInputValue":"float","textInputValue":"string","booleanInputValue":"boolean"}}}},"formElementUrn":{"type":"com.linkedin.common.Urn"},"formElementInputValuesResolutionResults":{"derivedFrom":"formElementInputValues","type":{"array":{"union":{"entityInputValue":"com.linkedin.194482b00bd3b5ae56a4d81712d0525c","locationInputValue":"com.linkedin.b429b28d37054e5232ecd5ecffd09315","integerInputValue":"int","dateRangeInputValue":"com.linkedin.8fafa573919c2bc96c29a2be79ce3fb3","urnInputValue":"com.linkedin.common.Urn","floatInputValue":"float","textInputValue":"string","booleanInputValue":"boolean"}}}}},"baseType":"com.linkedin.voyager.dash.common.forms.FormElementInput"},"com.linkedin.c138fd2c1944eec6c90de7592dd91018":{"fields":{"groups":{"type":{"array":"com.linkedin.d2acc160e7db812cd2fc11b5eaac7b0f"}},"tip":{"type":"com.linkedin.00ac75ef8dae327ab29a045438e7d186"},"viewImpressionTrackingKey":{"type":"string"},"buttonCTA":{"type":"com.linkedin.5de46613f3027e5a41b356871519a78b"}},"baseType":"com.linkedin.voyager.dash.jobs.JobQualificationSection"},"com.linkedin.29a8a179800ba03932dae2884cf40cff":{"fields":{"insightViewModel":{"type":"com.linkedin.3551030d48734b4fd9cb3238efc77ece"},"popoverTriggerIcon":{"type":"com.linkedin.voyager.dash.common.SystemImageName"},"popoverControlName":{"type":"com.linkedin.voyager.dash.common.tracking.ControlName"},"popoverContent":{"type":"com.linkedin.390ac1c1431d5b4e340a65af24ed68f2"}},"baseType":"com.linkedin.voyager.dash.jobs.TrustInsightViewModel"},"com.linkedin.c99e83ae287631a7e9fc1ff222ab73bb":{"fields":{},"baseType":"com.linkedin.voyager.dash.jobs.search.EndOfResultsCard"},"com.linkedin.fc957b3fdfa381bf007a3b1c29d6fe4c":{"fields":{"pageKey":{"type":"string"},"anchorPage":{"type":"boolean"}},"baseType":"com.linkedin.voyager.dash.common.tracking.PageKey"},"com.linkedin.21a4612bc8ff64d01e0062f74b272040":{"fields":{},"baseType":"com.linkedin.voyager.dash.jobs.CarouselCollectionCard"},"com.linkedin.85d03babfc1dc1ea8537cef9b2cc1bf5":{"fields":{"thumbnail":{"type":"com.linkedin.30e1ec6a491035bd7a31cc9de8a12c18"},"questionText":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.jobs.postapply.InterviewPrepQuestion"},"com.linkedin.84ab2fe95ad74fa18ab07857b1c58889":{"fields":{"locationList":{"type":"com.linkedin.310d48bff502026d1fdc80b24448401d"},"requiredFieldMissingErrorText":{"type":"com.linkedin.390ac1c1431d5b4e340a65af24ed68f2"},"responseRequired":{"type":"boolean"},"labelText":{"type":"com.linkedin.390ac1c1431d5b4e340a65af24ed68f2"}},"baseType":"com.linkedin.voyager.dash.common.forms.LocationTextEntityListField"},"com.linkedin.180bad7e2024ccff1ee0157be65c0dd1":{"fields":{"jobPostingDetailSectionUnions":{"type":{"array":{"union":{"similarJobsCard":"com.linkedin.be72f0bd614be87c519a3b06d3042062","interviewPrepCard":"com.linkedin.87b6c1610a369328d34d660f0b651ea4","salaryCard":"string","companyInsightsCard":"string","topCard":"string","bannerCard":"com.linkedin.19edabee847f40f1158364b31cb6309d","jobSeekerActionCard":"com.linkedin.44e816344c2289e8a62e33486601f239","learningRecommendationCard":"string","hiringTeamCard":"com.linkedin.392fa039af72d21e485244574f20ca3b","similarJobsAtCompanyCard":"com.linkedin.be72f0bd614be87c519a3b06d3042062","jobSegmentAttributesCard":"com.linkedin.c4a89ad017a52048790e89ac5ba1ab35","howYouMatchCard":"string","jobApplicantInsightsUrn":"string","jobActivityCard":"com.linkedin.863cd9abd8abab984b4347da5429d447","postApplyPromo":{"array":"com.linkedin.f0aa7f6258b1a3e754f7d6bbfc9af84b"},"jobDescription":"com.linkedin.f1007edfc8733ef870d8dc290d661d6a","benefitsCard":"com.linkedin.8e2783ee98d67c681658fdc2f0519697","jobAlertCard":"com.linkedin.2db94c9a357c51aebecf0054e59f9bd7","marketplacePromoCard":"string","companyCardV2":"string","recommendedActionsCard":"string"}}}},"jobPostingDetailSection":{"resolvedFrom":"jobPostingDetailSectionUnions","type":{"array":{"union":{"similarJobsCard":"com.linkedin.be72f0bd614be87c519a3b06d3042062","interviewPrepCard":"com.linkedin.87b6c1610a369328d34d660f0b651ea4","salaryCard":"com.linkedin.59b66a5c22b53ee350982aba9da43296","companyInsightsCard":"com.linkedin.f8ea4e1538764dc27e403a1d55047e51","topCard":"com.linkedin.f42293b24e5d60e317d2d68ab7597f64","bannerCard":"com.linkedin.19edabee847f40f1158364b31cb6309d","jobSeekerActionCard":"com.linkedin.44e816344c2289e8a62e33486601f239","learningRecommendationCard":"com.linkedin.2791a0d398e611fad8c820c60183e2bb","hiringTeamCard":"com.linkedin.392fa039af72d21e485244574f20ca3b","similarJobsAtCompanyCard":"com.linkedin.be72f0bd614be87c519a3b06d3042062","jobSegmentAttributesCard":"com.linkedin.c4a89ad017a52048790e89ac5ba1ab35","howYouMatchCard":"com.linkedin.6966477af81c9237d13230a0dced17c7","jobApplicantInsightsUrn":"com.linkedin.1b53ff96dc1e6cc3644919ffccaddf68","jobActivityCard":"com.linkedin.863cd9abd8abab984b4347da5429d447","postApplyPromo":{"array":"com.linkedin.f0aa7f6258b1a3e754f7d6bbfc9af84b"},"jobDescription":"com.linkedin.f1007edfc8733ef870d8dc290d661d6a","benefitsCard":"com.linkedin.8e2783ee98d67c681658fdc2f0519697","jobAlertCard":"com.linkedin.2db94c9a357c51aebecf0054e59f9bd7","marketplacePromoCard":"com.linkedin.03248b99e950a8be3201e56283063ed8","companyCardV2":"com.linkedin.ca8249c8427e4d60dbeb41d6b9cc8bbd","recommendedActionsCard":"com.linkedin.236a047ab00edc2a263bcd4777b91ac1"}}}}},"baseType":"com.linkedin.voyager.dash.jobs.JobPostingDetailSection"},"com.linkedin.3c0ef9a4c6857bf56baeb410170785ee":{"fields":{"duration":{"type":"string"},"thumbnail":{"type":"com.linkedin.30e1ec6a491035bd7a31cc9de8a12c18"},"navigationUrl":{"type":"com.linkedin.common.Url"},"chapterTitle":{"type":"string"},"entityUrn":{"type":"com.linkedin.voyager.dash.common.LearningVideoUrn"}},"baseType":"com.linkedin.voyager.dash.learning.LearningVideo"},"com.linkedin.30951a1ee13ae9dec8fa329260e12fc2":{"fields":{"header":{"type":"string"},"itemsDescription":{"type":"com.linkedin.be7717be8d269f7c62e51d9156652c9b"},"subheader":{"type":"string"},"items":{"type":{"array":"com.linkedin.bb168eba9e814ac2c87fa5288ea74e6f"}}},"baseType":"com.linkedin.voyager.dash.jobs.ItemsMatchGroup"},"com.linkedin.f8ea4e1538764dc27e403a1d55047e51":{"fields":{"jobWorkplaceTypes":{"resolvedFrom":"jobWorkplaceTypesUrns","type":{"array":"com.linkedin.79cb580711dcd3cf7d83e724b3d9e260"}},"companyInsightsCard":{"type":"com.linkedin.0165d1fc4938f280476b4c9f9ffd3f3f","isInjection":true},"companyDetails":{"type":"com.linkedin.b97672681af5b861a5ec5c53b8fcbc96"},"jobState":{"type":"com.linkedin.voyager.dash.jobs.JobState"},"trackingUrn":{"type":"com.linkedin.common.JobPostingUrn"},"description":{"type":"com.linkedin.390ac1c1431d5b4e340a65af24ed68f2"},"trustReviewDecision":{"type":"com.linkedin.voyager.dash.jobs.JobPostingTrustClassification"},"workplaceTypes":{"type":{"array":"string"}},"repostedJob":{"type":"boolean"},"title":{"type":"string"},"posterId":{"type":"string"},"createdAt":{"type":"com.linkedin.common.Time"},"contentSource":{"type":"com.linkedin.voyager.dash.common.semaphore.ContentSource"},"premiumUpsellCard":{"type":"com.linkedin.86ff9fd16c75bc659bfb6287b8ec1647","isInjection":true},"entityUrn":{"type":"com.linkedin.voyager.dash.common.JobPostingUrn"},"locationUrn":{"type":"string"},"location":{"resolvedFrom":"locationUrn","type":"com.linkedin.00c4f529ff2fa302d89eaea66344f268"},"workplaceTypesResolutionResults":{"resolvedFrom":"workplaceTypes","type":{"array":"com.linkedin.79cb580711dcd3cf7d83e724b3d9e260"}},"jobWorkplaceTypesUrns":{"type":{"array":"string"}},"jobDetailsSummaryCard":{"type":"com.linkedin.513522081428feabe67539f69de00fc6","isInjection":true}},"baseType":"com.linkedin.voyager.dash.jobs.JobPosting"},"com.linkedin.ff0a7ae4688d8d97ea2151c00e6e8392":{"fields":{"lastPageLeftOff":{"type":"int"},"entityUrn":{"type":"com.linkedin.voyager.dash.common.JobDraftApplicationInfoUrn"},"saveAsDraftAt":{"type":"com.linkedin.common.Time"}},"baseType":"com.linkedin.voyager.dash.jobs.JobDraftApplicationInfo"},"com.linkedin.e276a8a343fc735c387871abc9190c7d":{"fields":{"headcountGrowth":{"type":{"array":"com.linkedin.e5b59282f574d2493aeb20a65a9a1f94"}},"medianTenureYears":{"type":"com.linkedin.390ac1c1431d5b4e340a65af24ed68f2"}},"baseType":"com.linkedin.voyager.dash.premium.companyinsights.CompanyInsightsEmployeeGrowth"},"com.linkedin.f1007edfc8733ef870d8dc290d661d6a":{"fields":{"thirdPartyLabel":{"type":"com.linkedin.23ea16078c6de6222c73b55c40e8e8ab"},"jobPostingUrn":{"type":"string"},"entityUrn":{"type":"com.linkedin.voyager.dash.common.JobDescriptionUrn"},"postedOnText":{"type":"string"},"header":{"type":"com.linkedin.390ac1c1431d5b4e340a65af24ed68f2"},"jobPosting":{"resolvedFrom":"jobPostingUrn","type":"com.linkedin.f8ea4e1538764dc27e403a1d55047e51"},"descriptionText":{"type":"com.linkedin.390ac1c1431d5b4e340a65af24ed68f2"}},"baseType":"com.linkedin.voyager.dash.jobs.JobDescription"},"com.linkedin.b8949adeaaa915212d0136c4b9a1952b":{"fields":{"attribution":{"type":"string"},"digitalmediaAsset":{"type":"com.linkedin.common.DigitalmediaAssetUrn"},"rootUrl":{"type":"string"},"artifacts":{"type":{"array":"com.linkedin.8d64ead1eeea51a297dc70da47337de1"}}},"baseType":"com.linkedin.common.VectorImage"},"com.linkedin.236a047ab00edc2a263bcd4777b91ac1":{"fields":{"postApplyPromos":{"type":{"array":"com.linkedin.f0aa7f6258b1a3e754f7d6bbfc9af84b"}},"entityUrn":{"type":"com.linkedin.voyager.dash.common.RecommendedActionsCardUrn"}},"baseType":"com.linkedin.voyager.dash.jobs.postapply.RecommendedActionsCard"},"com.linkedin.cb6a34e10a21dc3fd25d32ef8d5002f5":{"fields":{"jobPostingRelevanceFeedback":{"resolvedFrom":"jobPostingRelevanceFeedbackUrn","type":"com.linkedin.d85acf0e41970312e8912b9455cefa24"},"dismissControlName":{"type":"com.linkedin.voyager.dash.common.tracking.ControlName"},"channel":{"type":"com.linkedin.voyager.dash.jobs.feedback.JobPostingRelevanceFeedbackChannel"},"followUpFeedbackReasons":{"type":{"array":"com.linkedin.d29a7d8226ce963ff5fc2dcd8f1376f6"}},"jobPostingRelevanceFeedbackUrn":{"type":"string"},"dismissUndoControlName":{"type":"com.linkedin.voyager.dash.common.tracking.ControlName"}},"baseType":"com.linkedin.voyager.dash.jobs.DismissJobAction"},"com.linkedin.2dea04d4082c6960578ac30a3ace5ee0":{"fields":{"typeaheadQueryContext":{"type":{"array":"string"}},"typeaheadQuery":{"type":"com.linkedin.d1ea2131164c2418a986748ae0e5000a"},"freeFormTextNotAllowedErrorText":{"type":"string"},"validationMetadata":{"derivedFrom":"validationMetadataUnion","type":{"union":{"pronounsV2":"com.linkedin.bfd20979d76e9c8ce942bc03238c6b02","string":"com.linkedin.bfd20979d76e9c8ce942bc03238c6b02","textFormInputType":"com.linkedin.voyager.dash.common.forms.validation.TextFormInputType","postalCode":"com.linkedin.2b6a63a1bb3f7b0bf5c50d8530db9f78","name":"com.linkedin.bfd20979d76e9c8ce942bc03238c6b02","pronouns":"string","integer":"com.linkedin.8409c9296c5bda7be8aee44feff66dd3","decimal":"com.linkedin.8409c9296c5bda7be8aee44feff66dd3","headline":"com.linkedin.bfd20979d76e9c8ce942bc03238c6b02","url":"com.linkedin.48be3e5c4d0dfb61ba0a4a296e719909"}}},"validationMetadataUnion":{"type":{"union":{"pronounsV2":"com.linkedin.bfd20979d76e9c8ce942bc03238c6b02","string":"com.linkedin.bfd20979d76e9c8ce942bc03238c6b02","textFormInputType":"com.linkedin.voyager.dash.common.forms.validation.TextFormInputType","postalCode":"com.linkedin.2b6a63a1bb3f7b0bf5c50d8530db9f78","name":"com.linkedin.bfd20979d76e9c8ce942bc03238c6b02","pronouns":"string","integer":"com.linkedin.8409c9296c5bda7be8aee44feff66dd3","decimal":"com.linkedin.8409c9296c5bda7be8aee44feff66dd3","headline":"com.linkedin.bfd20979d76e9c8ce942bc03238c6b02","url":"com.linkedin.48be3e5c4d0dfb61ba0a4a296e719909"}}},"typeaheadType":{"type":"com.linkedin.voyager.dash.search.TypeaheadType"},"freeFormTextAllowed":{"type":"boolean"}},"baseType":"com.linkedin.voyager.dash.common.forms.TypeaheadMetadata"},"com.linkedin.d9052d9e22b039f19b4d6484e2ebec27":{"fields":{"image":{"type":"com.linkedin.30e1ec6a491035bd7a31cc9de8a12c18"},"socialProofInsight":{"type":"com.linkedin.3551030d48734b4fd9cb3238efc77ece"},"subtext":{"type":"com.linkedin.390ac1c1431d5b4e340a65af24ed68f2"},"ctaText":{"type":"com.linkedin.390ac1c1431d5b4e340a65af24ed68f2"},"upsellOrderOrigin":{"type":"string"},"controlName":{"type":"com.linkedin.voyager.dash.common.tracking.ControlName"},"actionUrl":{"type":"com.linkedin.common.Url"},"legoTrackingToken":{"type":"string"},"title":{"type":"com.linkedin.390ac1c1431d5b4e340a65af24ed68f2"},"cancelCta":{"type":"com.linkedin.43d5fcbafc4556d0c474dabd56f017a2"},"globalLegoTrackingToken":{"type":"string"},"promotionLegoTrackingToken":{"type":"string"},"funnelCommonHeader":{"type":"com.linkedin.c2408eaefddaf66bea26cb69c95979d7"},"fireDiscoveryImpressionEvent":{"type":"boolean"},"subtitle":{"type":"com.linkedin.390ac1c1431d5b4e340a65af24ed68f2"},"dismissible":{"type":"boolean"},"layoutStyle":{"type":"com.linkedin.voyager.dash.premium.PremiumUpsellLayoutStyle"},"ctaButtonSecondaryEmphasizedTheme":{"type":"boolean"}},"baseType":"com.linkedin.voyager.dash.premium.PremiumUpsellCard"},"com.linkedin.310d48bff502026d1fdc80b24448401d":{"fields":{"controlName":{"type":"com.linkedin.voyager.dash.common.tracking.ControlName"},"pageKey":{"type":"com.linkedin.fc957b3fdfa381bf007a3b1c29d6fe4c"},"placeHolderText":{"type":"com.linkedin.390ac1c1431d5b4e340a65af24ed68f2"},"textSelectableOptions":{"type":{"array":"com.linkedin.98d89bae4aa4ef8f3ffc1018ca93419c"}},"helperText":{"type":"com.linkedin.390ac1c1431d5b4e340a65af24ed68f2"},"preferredRenderingStyle":{"type":"com.linkedin.voyager.dash.common.forms.TextEntityListPreferredRenderingStyle"}},"baseType":"com.linkedin.voyager.dash.common.forms.TextEntityListFormComponent"},"com.linkedin.c175e2014cb329dc190cefda1f8b1191":{"fields":{"saveControlName":{"type":"com.linkedin.voyager.dash.common.tracking.ControlName"},"saveState":{"type":"string"},"unsaveControlName":{"type":"com.linkedin.voyager.dash.common.tracking.ControlName"},"saveStateResolutionResult":{"resolvedFrom":"saveState","type":"com.linkedin.9731877e56bf195f5714e9f273590184"}},"baseType":"com.linkedin.voyager.dash.jobs.SaveJobAction"},"com.linkedin.2c2e713c6b2c9f6c424799fa2ababafd":{"fields":{"viewerCount":{"type":"int"},"thumbnail":{"type":"com.linkedin.30e1ec6a491035bd7a31cc9de8a12c18"},"title":{"type":"string"},"entityUrn":{"type":"com.linkedin.voyager.dash.common.LearningCourseUrn"},"learningNavigationUrl":{"type":"com.linkedin.common.Url"}},"baseType":"com.linkedin.voyager.dash.learning.LearningCourse"},"com.linkedin.c8f6142f1647b4b53caab19e4fe964ef":{"fields":{"textSelectableOption":{"type":"com.linkedin.98d89bae4aa4ef8f3ffc1018ca93419c"}},"baseType":"com.linkedin.voyager.dash.common.forms.DismissPill"},"com.linkedin.d29a7d8226ce963ff5fc2dcd8f1376f6":{"fields":{"type":{"type":"com.linkedin.voyager.dash.jobs.search.DismissedFeedbackReasonType"},"text":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.jobs.search.DismissedFeedbackReason"},"com.linkedin.0608c02e08bf95759764fc707e772cb9":{"fields":{"controlName":{"type":"com.linkedin.voyager.dash.common.tracking.ControlName"},"text":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.common.forms.FormButton"},"com.linkedin.48be3e5c4d0dfb61ba0a4a296e719909":{"fields":{"characterCountRangeValidation":{"type":"com.linkedin.0bec4a68c3bab987feaffd1b11fa4cf5"},"invalidUrlErrorText":{"type":"com.linkedin.390ac1c1431d5b4e340a65af24ed68f2"}},"baseType":"com.linkedin.voyager.dash.common.forms.validation.UrlValidationMetadata"},"com.linkedin.2b6a63a1bb3f7b0bf5c50d8530db9f78":{"fields":{"errorText":{"type":"com.linkedin.390ac1c1431d5b4e340a65af24ed68f2"},"countryFormElementUrn":{"type":"com.linkedin.common.Urn"}},"baseType":"com.linkedin.voyager.dash.common.forms.validation.PostalCodeValidationMetadata"},"com.linkedin.eef1cf048228a183463b81d9c6790e5f":{"fields":{"degree":{"resolvedFrom":"degreeUrn","type":"com.linkedin.b0ab2e740be6873e75f413db4e533bb9"},"viewersHighestDegree":{"type":"boolean"},"degreeUrn":{"type":"string"},"percentage":{"type":"int"}},"baseType":"com.linkedin.voyager.dash.premium.insights.JobApplicantInsightsDegreeDetail"},"com.linkedin.c88fbcf94b11ad7020d237307a79e8d5":{"fields":{"name":{"type":"com.linkedin.390ac1c1431d5b4e340a65af24ed68f2"},"imageUnion":{"type":{"union":{"url":"com.linkedin.common.Url","vectorImage":"com.linkedin.b8949adeaaa915212d0136c4b9a1952b"}}},"image":{"derivedFrom":"imageUnion","type":{"union":{"url":"com.linkedin.common.Url","vectorImage":"com.linkedin.b8949adeaaa915212d0136c4b9a1952b"}}}},"baseType":"com.linkedin.voyager.dash.common.media.StickerLinkSmallTemplateView"},"com.linkedin.9a2250b4fbfd26ffbc3d92812eb2880d":{"fields":{"entityUrn":{"type":"com.linkedin.voyager.dash.common.MemberRelationshipUrn"},"memberRelationshipUnion":{"type":{"union":{"self":"com.linkedin.8b1684dd351b279ed40c6661257d0e63","connection":"com.linkedin.0030135583a12944453eaad4d3729402","noConnection":"com.linkedin.5d8878fd7b4508fcbef4f2d67c765db3"}}},"memberRelationship":{"derivedFrom":"memberRelationshipUnion","type":{"union":{"self":"com.linkedin.8b1684dd351b279ed40c6661257d0e63","connection":"com.linkedin.0030135583a12944453eaad4d3729402","noConnection":"com.linkedin.5d8878fd7b4508fcbef4f2d67c765db3"}}}},"baseType":"com.linkedin.voyager.dash.relationships.MemberRelationship"},"com.linkedin.8864b68d440629f1899c8ff463b7f71d":{"fields":{"serverIntent":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.common.coach.CoachQueryContextV2"},"com.linkedin.0165d1fc4938f280476b4c9f9ffd3f3f":{"fields":{"elements":{"type":{"array":"com.linkedin.04a6d570213522d9270f88b0374505b6"}}},"baseType":"com.linkedin.restli.common.CollectionResponse"},"com.linkedin.a29bf6b9168c31d9f760e514428df1c5":{"fields":{"totalEmployees":{"type":"long"},"headcounts":{"type":"com.linkedin.e276a8a343fc735c387871abc9190c7d"},"growthPeriods":{"type":{"array":"com.linkedin.f6e5193777aa8860860ecb48d0212cce"}}},"baseType":"com.linkedin.voyager.dash.premium.companyinsights.HeadcountInsights"},"com.linkedin.8fbc3627c8305a33e3e6840ec7688312":{"fields":{"jobsDashJobPostingDetailSectionsByCardSectionTypes":{"type":"com.linkedin.f1d8ff8a5c9d0a2c73d8c4418e3a4526"}},"baseType":"com.linkedin.graphql.Query"},"com.linkedin.d8e085065c27e5a4b4a3b47e39d34eae":{"fields":{"entityUrn":{"type":"com.linkedin.voyager.dash.common.EntityRelevanceFeedbackUrn"}},"baseType":"com.linkedin.voyager.dash.jobs.feedback.EntityRelevanceFeedback"},"com.linkedin.2815a36a9c66a3c907e5474fae6a1903":{"fields":{"seniorHireCount":{"type":"long"},"allEmployeeHireCount":{"type":"long"},"yearMonthOn":{"type":"com.linkedin.fefbb3ce7d8629ad1cfd497514eb598e"}},"baseType":"com.linkedin.voyager.dash.premium.companyinsights.HireCountsItem"},"com.linkedin.272e46084795068d9243a5e3978fc2e2":{"fields":{"entityUrn":{"type":"com.linkedin.voyager.dash.common.JobAlertUrn"}},"baseType":"com.linkedin.voyager.dash.jobs.JobAlert"},"com.linkedin.e18c41f5008d005150ad33f812748af9":{"fields":{"maxReachedWithExclusionListInfoText":{"type":"string"},"lessThanMinErrorText":{"type":"string"},"errorText":{"type":"com.linkedin.390ac1c1431d5b4e340a65af24ed68f2"},"validRange":{"type":"com.linkedin.6148d310783eaffead8570485d05a156"},"maxExceededErrorText":{"type":"string"},"maxReachedInfoText":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.common.forms.validation.SelectionCountRangeValidation"},"com.linkedin.3900e4e094a1b889ef9285a430d6f2d5":{"fields":{"logoImage":{"type":{"union":{"url":"com.linkedin.common.Url","vectorImage":"com.linkedin.b8949adeaaa915212d0136c4b9a1952b"}}},"logoImageResolutionResult":{"derivedFrom":"logoImage","type":{"union":{"url":"com.linkedin.common.Url","vectorImage":"com.linkedin.b8949adeaaa915212d0136c4b9a1952b"}}},"entityUrn":{"type":"com.linkedin.voyager.dash.common.ProfessionalEventUrn"}},"baseType":"com.linkedin.voyager.dash.events.ProfessionalEvent"},"com.linkedin.5d5d123ba49a49b959a39ab7ab5ea1c4":{"fields":{"popoverTriggerIcon":{"type":"com.linkedin.voyager.dash.common.SystemImageName"},"title":{"type":"com.linkedin.390ac1c1431d5b4e340a65af24ed68f2"},"popoverContent":{"type":"com.linkedin.390ac1c1431d5b4e340a65af24ed68f2"},"subtitle":{"type":"com.linkedin.390ac1c1431d5b4e340a65af24ed68f2"}},"baseType":"com.linkedin.voyager.dash.premium.companyinsights.Header"},"com.linkedin.f26ce46d6c12df14239f36d844033955":{"fields":{"action":{"type":"com.linkedin.5de46613f3027e5a41b356871519a78b"},"qualifications":{"type":{"array":"string"}}},"baseType":"com.linkedin.voyager.dash.jobs.JobQualificationList"},"com.linkedin.939138a9c6796d21af12eaf891a20e80":{"fields":{"pivotUnion":{"type":{"union":{"skill":"string"}}},"pivot":{"resolvedFrom":"pivotUnion","type":{"union":{"skill":"com.linkedin.7a3dfe06b5ebab0041d8cea76d58565a"}}},"contentUnion":{"type":{"union":{"learningCourse":"com.linkedin.2c2e713c6b2c9f6c424799fa2ababafd"}}},"content":{"derivedFrom":"contentUnion","type":{"union":{"learningCourse":"com.linkedin.2c2e713c6b2c9f6c424799fa2ababafd"}}}},"baseType":"com.linkedin.voyager.dash.learning.recommendations.RecommendationContent"},"com.linkedin.aa89cadbee726b8d3c62475faddcf670":{"fields":{"dateText":{"type":"com.linkedin.390ac1c1431d5b4e340a65af24ed68f2"},"controlName":{"type":"com.linkedin.voyager.dash.common.tracking.ControlName"},"dateValidation":{"type":"com.linkedin.48407601ffa8bd83b8651487ea606df9"},"dateInputType":{"type":"com.linkedin.voyager.dash.common.forms.DateInputType"}},"baseType":"com.linkedin.voyager.dash.common.forms.DateFormComponent"},"com.linkedin.59e2a69f3c2d74e9d4b5b8530e816743":{"fields":{"learningVideoUrns":{"type":{"array":"string"}},"learningCourseUrn":{"type":"string"},"learningVideo":{"resolvedFrom":"learningVideoUrns","type":{"array":"com.linkedin.3c0ef9a4c6857bf56baeb410170785ee"}},"learningCourse":{"resolvedFrom":"learningCourseUrn","type":"com.linkedin.2c2e713c6b2c9f6c424799fa2ababafd"}},"baseType":"com.linkedin.voyager.dash.jobs.InterviewPrepLearningRecommendation"},"com.linkedin.30e1ec6a491035bd7a31cc9de8a12c18":{"fields":{"editableAccessibilityText":{"type":"boolean"},"attributes":{"type":{"array":"com.linkedin.296fd4ebc3f7a46ccca548d534f656e4"}},"actionTarget":{"type":"com.linkedin.common.Url"},"accessibilityTextAttributes":{"type":{"array":"com.linkedin.b0a683a611d43de435ed53c6232d8d44"}},"totalCount":{"type":"int"},"accessibilityText":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.common.image.ImageViewModel"},"com.linkedin.4ece7b9b6c81c2cdb35c6bf9c8e01cb7":{"fields":{},"baseType":"com.linkedin.voyager.dash.jobs.feedback.PageFeedbackWidget"},"com.linkedin.bfd20979d76e9c8ce942bc03238c6b02":{"fields":{"characterCountRangeValidation":{"type":"com.linkedin.0bec4a68c3bab987feaffd1b11fa4cf5"},"errorText":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.common.forms.validation.StringValidationMetadata"},"com.linkedin.65d9b685ce8d8355ba025fb1ae135598":{"fields":{"entityUrn":{"type":"com.linkedin.voyager.dash.common.JobSkillMatchInsightUrn"},"insightSummary":{"type":"com.linkedin.3551030d48734b4fd9cb3238efc77ece"}},"baseType":"com.linkedin.voyager.dash.assessments.JobSkillMatchInsight"},"com.linkedin.d09aeb85fdf4991b6b063ae75ef6ef54":{"fields":{"entityLockup":{"type":"com.linkedin.bd66bae736eabe6505d1b80c182e31af"},"hiredPosition":{"type":"com.linkedin.390ac1c1431d5b4e340a65af24ed68f2"},"hireYearMonthOn":{"type":"com.linkedin.fefbb3ce7d8629ad1cfd497514eb598e"}},"baseType":"com.linkedin.voyager.dash.premium.companyinsights.SeniorHiresItem"},"com.linkedin.554b975e2cabf1c2139af63808fd028a":{"fields":{"dismissControlName":{"type":"com.linkedin.voyager.dash.common.tracking.ControlName"},"formElement":{"type":"com.linkedin.258b7479f7c0999032ce22032fa02cdf"},"title":{"type":"string"},"subtitle":{"type":"com.linkedin.390ac1c1431d5b4e340a65af24ed68f2"}},"baseType":"com.linkedin.voyager.dash.common.forms.SingleQuestionSubForm"},"com.linkedin.f42293b24e5d60e317d2d68ab7597f64":{"fields":{"preDashNormalizedJobPostingUrn":{"type":"com.linkedin.voyager.dash.common.PreDashUrn"},"jobInsightsV2":{"type":{"array":{"union":{"premiumUpsellCard":"string","jobInsightViewModel":"com.linkedin.efd2ef2c46b7959f85a3d6461edc3e69","premiumUpsell":"string","insightViewModel":"com.linkedin.3551030d48734b4fd9cb3238efc77ece","verifiedHiringInsight":"string","skillMatchInsight":"string","responsivenessInsightViewModel":"com.linkedin.036f42f6cc87d5881cf2351020672e97","trustInsightViewModel":"com.linkedin.29a8a179800ba03932dae2884cf40cff"}}}},"footerItems":{"type":{"array":"com.linkedin.6c333b78b6aeac833ef88e8174932712"}},"jobPostingTitle":{"type":"string"},"jobPosting":{"resolvedFrom":"jobPostingUrn","type":"com.linkedin.f8ea4e1538764dc27e403a1d55047e51"},"primaryDescription":{"type":"com.linkedin.390ac1c1431d5b4e340a65af24ed68f2"},"navigationBarSubtitle":{"type":"string"},"jobInsightsV2ResolutionResults":{"resolvedFrom":"jobInsightsV2","type":{"array":{"union":{"premiumUpsellCard":"com.linkedin.f8ea4e1538764dc27e403a1d55047e51","jobInsightViewModel":"com.linkedin.efd2ef2c46b7959f85a3d6461edc3e69","premiumUpsell":"com.linkedin.f8ea4e1538764dc27e403a1d55047e51","insightViewModel":"com.linkedin.3551030d48734b4fd9cb3238efc77ece","verifiedHiringInsight":"com.linkedin.ac21e10e9207a4a6523aafa0c51066d4","skillMatchInsight":"com.linkedin.65d9b685ce8d8355ba025fb1ae135598","responsivenessInsightViewModel":"com.linkedin.036f42f6cc87d5881cf2351020672e97","trustInsightViewModel":"com.linkedin.29a8a179800ba03932dae2884cf40cff"}}}},"primaryActionV2Union":{"type":{"union":{"dismissJobAction":"com.linkedin.cb6a34e10a21dc3fd25d32ef8d5002f5","applyJobAction":"com.linkedin.64aff242d842d2f5c0d6ea1432ceb79a","saveJobAction":"com.linkedin.c175e2014cb329dc190cefda1f8b1191"}}},"secondaryActionsV2ResolutionResults":{"derivedFrom":"secondaryActionsV2","type":{"array":{"union":{"dismissJobAction":"com.linkedin.cb6a34e10a21dc3fd25d32ef8d5002f5","applyJobAction":"com.linkedin.64aff242d842d2f5c0d6ea1432ceb79a","saveJobAction":"com.linkedin.c175e2014cb329dc190cefda1f8b1191"}}}},"primaryActionV2":{"derivedFrom":"primaryActionV2Union","type":{"union":{"dismissJobAction":"com.linkedin.cb6a34e10a21dc3fd25d32ef8d5002f5","applyJobAction":"com.linkedin.64aff242d842d2f5c0d6ea1432ceb79a","saveJobAction":"com.linkedin.c175e2014cb329dc190cefda1f8b1191"}}},"coachJobsKickoffPrompts":{"resolvedFrom":"coachJobsKickoffPromptsUrn","type":"com.linkedin.8e19fba94618da7ceb9cca894420a5da"},"secondaryDescription":{"type":"com.linkedin.390ac1c1431d5b4e340a65af24ed68f2"},"secondaryActionsV2":{"type":{"array":{"union":{"dismissJobAction":"com.linkedin.cb6a34e10a21dc3fd25d32ef8d5002f5","applyJobAction":"com.linkedin.64aff242d842d2f5c0d6ea1432ceb79a","saveJobAction":"com.linkedin.c175e2014cb329dc190cefda1f8b1191"}}}},"jobPostingUrn":{"type":"string"},"entityUrn":{"type":"com.linkedin.voyager.dash.common.JobPostingCardUrn"},"logo":{"type":"com.linkedin.30e1ec6a491035bd7a31cc9de8a12c18"},"tip":{"type":"com.linkedin.a1cf47049f3ce4950c977d8052ae04d3"},"tertiaryDescription":{"type":"com.linkedin.390ac1c1431d5b4e340a65af24ed68f2"},"coachJobsKickoffPromptsUrn":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.jobs.JobPostingCard"},"com.linkedin.5de46613f3027e5a41b356871519a78b":{"fields":{"controlName":{"type":"com.linkedin.voyager.dash.common.tracking.ControlName"},"displayText":{"type":"string"},"systemImageNameIcon":{"type":"com.linkedin.voyager.dash.common.SystemImageName"},"actionTarget":{"type":"com.linkedin.common.Url"},"ctaIcon":{"type":"com.linkedin.b49a045df66f28b2a65f3df75d119eae"}},"baseType":"com.linkedin.voyager.dash.jobs.NavigationAction"},"com.linkedin.b69dade258a78e45f8205cb49b6a98b9":{"fields":{"paidInMail":{"type":"boolean"},"extensionContentContextUrn":{"type":"com.linkedin.common.Urn"},"recipient":{"resolvedFrom":"recipientUrns","type":{"array":"com.linkedin.826b8c485ed1073773697502413b9597"}},"messageRequestContextUrn":{"type":"com.linkedin.common.Urn"},"recipientUrns":{"type":{"array":"string"}},"targetUrl":{"type":"com.linkedin.common.Url"},"existingConversation":{"resolvedFrom":"existingConversationUrn","type":"com.linkedin.bb47d92b56e85e0625cd6b7f5d3d0384"},"existingConversationUrn":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.messaging.compose.ComposeNavigationContext"},"com.linkedin.3a31504449fad28c205c97d4985ea256":{"fields":{"upsellSlot":{"resolvedFrom":"upsellSlotUrn","type":"com.linkedin.dc4c6e3940e502e42eb82438140331c7"},"upsellSlotUrn":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.jobs.PremiumUpsellSection"},"com.linkedin.390ac1c1431d5b4e340a65af24ed68f2":{"fields":{"textDirection":{"type":"com.linkedin.voyager.dash.common.text.TextDirection"},"attributesV2":{"type":{"array":"com.linkedin.b0a683a611d43de435ed53c6232d8d44"}},"text":{"type":"string"},"accessibilityTextAttributesV2":{"type":{"array":"com.linkedin.b0a683a611d43de435ed53c6232d8d44"}},"accessibilityText":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.common.text.TextViewModel"},"com.linkedin.0bec4a68c3bab987feaffd1b11fa4cf5":{"fields":{"showCharacterCount":{"type":"boolean"},"errorText":{"type":"com.linkedin.390ac1c1431d5b4e340a65af24ed68f2"},"validRange":{"type":"com.linkedin.6148d310783eaffead8570485d05a156"}},"baseType":"com.linkedin.voyager.dash.common.forms.validation.CharacterCountRangeValidation"},"com.linkedin.1581fb7fcc9bc37fc013106a0e87c905":{"fields":{},"baseType":"com.linkedin.voyager.dash.jobs.search.JobSearchSuggestionComponent"},"com.linkedin.fefbb3ce7d8629ad1cfd497514eb598e":{"fields":{"month":{"type":"int"},"year":{"type":"int"},"day":{"type":"int"}},"baseType":"com.linkedin.common.Date"},"com.linkedin.0f63072b23eab5f48ddb389e6f328afc":{"fields":{"name":{"type":"string"},"entityUrn":{"type":"com.linkedin.voyager.dash.common.SeniorityUrn"}},"baseType":"com.linkedin.voyager.dash.premium.insights.StandardizedSeniority"},"com.linkedin.ac21e10e9207a4a6523aafa0c51066d4":{"fields":{"controlName":{"type":"com.linkedin.voyager.dash.common.tracking.ControlName"},"ctaText":{"type":"string"},"entityUrn":{"type":"com.linkedin.voyager.dash.common.JobVerifiedHiringInsightUrn"},"insightSummary":{"type":"com.linkedin.3551030d48734b4fd9cb3238efc77ece"}},"baseType":"com.linkedin.voyager.dash.jobs.JobVerifiedHiringInsight"},"com.linkedin.04a6d570213522d9270f88b0374505b6":{"fields":{"companyInsightsUnion":{"type":{"union":{"alumniInsights":"com.linkedin.b2b7b2441fdd2b88fd82124f6d8c4a1c","jobPostingCompanyInsights":"com.linkedin.7269fdb3a0f2f75ea98f76d3d3c9dfc3","jobOpeningsInsights":"com.linkedin.45d6d21a26aa62742acf906ac42d8306","functionHeadcountInsights":"com.linkedin.4fb8cecee08f0c588138694ff045dfd9","headcountInsights":"com.linkedin.a29bf6b9168c31d9f760e514428df1c5","hiresInsights":"com.linkedin.7f23d6d0556f01f090158424e75ff2ab"}}},"header":{"type":"com.linkedin.5d5d123ba49a49b959a39ab7ab5ea1c4"},"companyInsights":{"derivedFrom":"companyInsightsUnion","type":{"union":{"alumniInsights":"com.linkedin.b2b7b2441fdd2b88fd82124f6d8c4a1c","jobPostingCompanyInsights":"com.linkedin.7269fdb3a0f2f75ea98f76d3d3c9dfc3","jobOpeningsInsights":"com.linkedin.45d6d21a26aa62742acf906ac42d8306","functionHeadcountInsights":"com.linkedin.4fb8cecee08f0c588138694ff045dfd9","headcountInsights":"com.linkedin.a29bf6b9168c31d9f760e514428df1c5","hiresInsights":"com.linkedin.7f23d6d0556f01f090158424e75ff2ab"}}}},"baseType":"com.linkedin.voyager.dash.premium.companyinsights.CompanyInsightsCard"},"com.linkedin.b26a8e722557e805b373f37f9d0ef00a":{"fields":{},"baseType":"com.linkedin.voyager.dash.jobs.CarouselEntityHighlightCard"},"com.linkedin.b0a683a611d43de435ed53c6232d8d44":{"fields":{"detailData":{"resolvedFrom":"detailDataUnion","type":{"union":{"jobPostingName":"com.linkedin.f8ea4e1538764dc27e403a1d55047e51","profileFamiliarName":"com.linkedin.826b8c485ed1073773697502413b9597","hyperlink":"com.linkedin.common.Url","color":"com.linkedin.voyager.dash.common.text.TextColor","companyName":"com.linkedin.a2ef5c1c8d1bd7b83de7009b834a38c6","icon":"com.linkedin.voyager.dash.common.ArtDecoIconName","epoch":"com.linkedin.94721090e597af7c97503cb8b1a7e50f","systemImage":"com.linkedin.voyager.dash.common.SystemImageName","textLink":"com.linkedin.ce28f52e436b3cdd8100675b5288a600","listItemStyle":"com.linkedin.984a5b2fd75109bc59c5241e98cb46ac","listStyle":"com.linkedin.voyager.dash.common.text.ListStyleType","groupName":"com.linkedin.32bab5d7af29a8bdf4fe920d7d2155b1","hyperlinkOpenExternally":"com.linkedin.common.Url","stringFieldReference":"com.linkedin.146c92f10032ac18e61e96a3f143b9f1","profileFullName":"com.linkedin.826b8c485ed1073773697502413b9597","learningCourseName":"com.linkedin.2c2e713c6b2c9f6c424799fa2ababafd","profileMention":"com.linkedin.826b8c485ed1073773697502413b9597","style":"com.linkedin.voyager.dash.common.text.TextStyle","schoolName":"com.linkedin.d4dcbabbf47d9675b0597099d9cd7d69","hashtag":"com.linkedin.7948e5031442b35b939b7672eac8bad0"}}},"length":{"type":"int"},"start":{"type":"int"},"detailDataUnion":{"type":{"union":{"jobPostingName":"string","profileFamiliarName":"string","hyperlink":"com.linkedin.common.Url","color":"com.linkedin.voyager.dash.common.text.TextColor","companyName":"string","icon":"com.linkedin.voyager.dash.common.ArtDecoIconName","epoch":"com.linkedin.94721090e597af7c97503cb8b1a7e50f","systemImage":"com.linkedin.voyager.dash.common.SystemImageName","textLink":"com.linkedin.ce28f52e436b3cdd8100675b5288a600","listItemStyle":"com.linkedin.984a5b2fd75109bc59c5241e98cb46ac","listStyle":"com.linkedin.voyager.dash.common.text.ListStyleType","groupName":"string","hyperlinkOpenExternally":"com.linkedin.common.Url","stringFieldReference":"com.linkedin.146c92f10032ac18e61e96a3f143b9f1","profileFullName":"string","learningCourseName":"string","profileMention":"string","style":"com.linkedin.voyager.dash.common.text.TextStyle","schoolName":"string","hashtag":"string"}}}},"baseType":"com.linkedin.voyager.dash.common.text.TextAttribute"},"com.linkedin.6e3e751d09550f922978dedd7f52eea5":{"fields":{"professionalEvent":{"resolvedFrom":"professionalEventUrn","type":"com.linkedin.3900e4e094a1b889ef9285a430d6f2d5"},"vectorImage":{"type":"com.linkedin.b8949adeaaa915212d0136c4b9a1952b"},"professionalEventUrn":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.common.image.NonEntityProfessionalEventLogo"},"com.linkedin.409356f1c0f808083c5e0174516009dd":{"fields":{"tapTargetEntityUnion":{"type":{"union":{"companyUrn":"string","profileUrn":"string"}}},"stickerLinkViewUnion":{"type":{"union":{"mediumTemplate":"com.linkedin.b2a671b0acb9520f18deade4e955c616","largeTemplate":"com.linkedin.ca1fb70631b137b6d459efdae8a18bc0","smallTemplate":"com.linkedin.c88fbcf94b11ad7020d237307a79e8d5"}}},"firstCornerXOffsetPercentage":{"type":"float"},"stickerLinkTemplateSize":{"type":"com.linkedin.voyager.dash.common.media.StickerLinkTemplateSize"},"type":{"type":"com.linkedin.voyager.dash.common.TapTargetAttributeType"},"thirdCornerYOffsetPercentage":{"type":"float"},"url":{"type":"com.linkedin.common.Url"},"stickerLinkView":{"derivedFrom":"stickerLinkViewUnion","type":{"union":{"mediumTemplate":"com.linkedin.b2a671b0acb9520f18deade4e955c616","largeTemplate":"com.linkedin.ca1fb70631b137b6d459efdae8a18bc0","smallTemplate":"com.linkedin.c88fbcf94b11ad7020d237307a79e8d5"}}},"urn":{"type":"com.linkedin.common.Urn"},"thirdCornerXOffsetPercentage":{"type":"float"},"secondCornerYOffsetPercentage":{"type":"float"},"fourthCornerXOffsetPercentage":{"type":"float"},"tapTargetEntity":{"resolvedFrom":"tapTargetEntityUnion","type":{"union":{"companyUrn":"com.linkedin.a2ef5c1c8d1bd7b83de7009b834a38c6","profileUrn":"com.linkedin.826b8c485ed1073773697502413b9597"}}},"firstCornerYOffsetPercentage":{"type":"float"},"rank":{"type":"int"},"untaggable":{"type":"boolean"},"fourthCornerYOffsetPercentage":{"type":"float"},"text":{"type":"string"},"secondCornerXOffsetPercentage":{"type":"float"}},"baseType":"com.linkedin.voyager.dash.common.TapTarget"},"com.linkedin.826b8c485ed1073773697502413b9597":{"fields":{"profilePicture":{"type":"com.linkedin.9e0491106da7f36c0672ee51875a86fa"},"lastName":{"type":"string"},"firstName":{"type":"string"},"entityUrn":{"type":"com.linkedin.voyager.dash.common.ProfileUrn"},"memberRelationship":{"type":"com.linkedin.9a2250b4fbfd26ffbc3d92812eb2880d","isInjection":true},"ringStatus":{"type":"com.linkedin.e94906839b912dcb4829422361b5028c","isInjection":true},"headline":{"type":"string"},"publicIdentifier":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.identity.profile.Profile"},"com.linkedin.9731877e56bf195f5714e9f273590184":{"fields":{"preDashEntityUrn":{"type":"com.linkedin.voyager.dash.common.PreDashUrn"},"saved":{"type":"boolean"},"entityUrn":{"type":"com.linkedin.voyager.dash.common.SaveStateUrn"}},"baseType":"com.linkedin.voyager.dash.feed.SaveState"},"com.linkedin.19edabee847f40f1158364b31cb6309d":{"fields":{"jobPostingBanner":{"derivedFrom":"jobPostingBannerUnion","type":{"union":{"jobDetailsClaimJobBanner":"com.linkedin.b97672681af5b861a5ec5c53b8fcbc96","trustBanner":"com.linkedin.0054dfa46558041e61c38c995b4809c8","viewAsHirerBanner":"com.linkedin.common.Time","jobTrustBanner":"string"}}},"navigationActionUnion":{"type":{"union":{"navigateToReviewLearnMoreModal":"com.linkedin.8b1684dd351b279ed40c6661257d0e63","ctaAction":"com.linkedin.5de46613f3027e5a41b356871519a78b"}}},"navigationAction":{"derivedFrom":"navigationActionUnion","type":{"union":{"navigateToReviewLearnMoreModal":"com.linkedin.8b1684dd351b279ed40c6661257d0e63","ctaAction":"com.linkedin.5de46613f3027e5a41b356871519a78b"}}},"jobPostingBannerUnion":{"type":{"union":{"jobDetailsClaimJobBanner":"com.linkedin.b97672681af5b861a5ec5c53b8fcbc96","trustBanner":"com.linkedin.0054dfa46558041e61c38c995b4809c8","viewAsHirerBanner":"com.linkedin.common.Time","jobTrustBanner":"string"}}}},"baseType":"com.linkedin.voyager.dash.jobs.JobPostingBannerCard"},"com.linkedin.ca8249c8427e4d60dbeb41d6b9cc8bbd":{"fields":{"company":{"resolvedFrom":"companyUrn","type":"com.linkedin.a2ef5c1c8d1bd7b83de7009b834a38c6"},"commitments":{"type":{"array":"com.linkedin.b2744b01d95785f8c9ab5f466c1674d5"}},"published":{"type":"boolean"},"companyUrn":{"type":"string"},"entityUrn":{"type":"com.linkedin.voyager.dash.common.CompanyCardUrn"}},"baseType":"com.linkedin.voyager.dash.talentbrand.CompanyCard"},"com.linkedin.be72f0bd614be87c519a3b06d3042062":{"fields":{"header":{"type":"com.linkedin.dd03bc4de44734367e478c7c8eed60a0"},"footer":{"derivedFrom":"footerUnion","type":{"union":{"navigationFooter":"com.linkedin.5de46613f3027e5a41b356871519a78b"}}},"entities":{"type":{"array":{"union":{"promotionalCard":"com.linkedin.c4e907da1ff053ca48ebe32c32f78805","endOfResultsCard":"com.linkedin.c99e83ae287631a7e9fc1ff222ab73bb","jobPostingCardWrapper":"com.linkedin.81315c1c0ea01a8f7161ae78bbbd0ff1","jobSearchHistoryCard":"com.linkedin.d0f6c4718d68ff125ca6f7e0b91f510c","premiumUpsellSlot":"string","jobSearchSuggestion":"com.linkedin.1581fb7fcc9bc37fc013106a0e87c905","newCollectionHeaderCard":"com.linkedin.6e88a2e94acf8ea5aa8c9f7051705709","carouselEntityHighlightCard":"com.linkedin.b26a8e722557e805b373f37f9d0ef00a","feedbackCard":"com.linkedin.4ece7b9b6c81c2cdb35c6bf9c8e01cb7","carouselCollectionCard":"com.linkedin.21a4612bc8ff64d01e0062f74b272040","carouselAdvantageCard":"com.linkedin.57812d345916f0da7785a8887e8fe73d","careerEnrichmentCard":"com.linkedin.b2c46ce630f9fc1eb9ed163a1a7b2616","noResultsCard":"com.linkedin.1ed8d64e266be8d739bffff5d4200fb4","seeAllCard":"com.linkedin.653bbb9565e4a43fc63d44ebc167fab1","refreshStateCard":"com.linkedin.aa4a2fb807b219a33654acc9643ff224","jobPostingCard":"com.linkedin.f42293b24e5d60e317d2d68ab7597f64"}}}},"entityUrn":{"type":"com.linkedin.voyager.dash.common.JobsFeedCardModuleUrn"},"footerUnion":{"type":{"union":{"navigationFooter":"com.linkedin.5de46613f3027e5a41b356871519a78b"}}},"entitiesResolutionResults":{"resolvedFrom":"entities","type":{"array":{"union":{"promotionalCard":"com.linkedin.c4e907da1ff053ca48ebe32c32f78805","endOfResultsCard":"com.linkedin.c99e83ae287631a7e9fc1ff222ab73bb","jobPostingCardWrapper":"com.linkedin.81315c1c0ea01a8f7161ae78bbbd0ff1","jobSearchHistoryCard":"com.linkedin.d0f6c4718d68ff125ca6f7e0b91f510c","premiumUpsellSlot":"com.linkedin.dc4c6e3940e502e42eb82438140331c7","jobSearchSuggestion":"com.linkedin.1581fb7fcc9bc37fc013106a0e87c905","newCollectionHeaderCard":"com.linkedin.6e88a2e94acf8ea5aa8c9f7051705709","carouselEntityHighlightCard":"com.linkedin.b26a8e722557e805b373f37f9d0ef00a","feedbackCard":"com.linkedin.4ece7b9b6c81c2cdb35c6bf9c8e01cb7","carouselCollectionCard":"com.linkedin.21a4612bc8ff64d01e0062f74b272040","carouselAdvantageCard":"com.linkedin.57812d345916f0da7785a8887e8fe73d","careerEnrichmentCard":"com.linkedin.b2c46ce630f9fc1eb9ed163a1a7b2616","noResultsCard":"com.linkedin.1ed8d64e266be8d739bffff5d4200fb4","seeAllCard":"com.linkedin.653bbb9565e4a43fc63d44ebc167fab1","refreshStateCard":"com.linkedin.aa4a2fb807b219a33654acc9643ff224","jobPostingCard":"com.linkedin.f42293b24e5d60e317d2d68ab7597f64"}}}}},"baseType":"com.linkedin.voyager.dash.jobs.JobsFeedCardModule"},"com.linkedin.b2744b01d95785f8c9ab5f466c1674d5":{"fields":{"commitmentConfig":{"type":"com.linkedin.ef6520077f06821116709dada74ee910"},"commitmentType":{"type":"com.linkedin.voyager.dash.talentbrand.CommitmentType"},"description":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.talentbrand.Commitment"},"com.linkedin.87b6c1610a369328d34d660f0b651ea4":{"fields":{"contentUnions":{"type":{"array":{"union":{"questionRecommendation":"com.linkedin.d77e6926ccb788017c5e7ffe0261e4d7","learningRecommendation":"com.linkedin.59e2a69f3c2d74e9d4b5b8530e816743"}}}},"viewImpressionTrackingKey":{"type":"string"},"entityUrn":{"type":"com.linkedin.voyager.dash.common.InterviewPrepCardUrn"},"description":{"type":"string"},"title":{"type":"string"},"ctaNavigationAction":{"type":"com.linkedin.5de46613f3027e5a41b356871519a78b"},"content":{"derivedFrom":"contentUnions","type":{"array":{"union":{"questionRecommendation":"com.linkedin.d77e6926ccb788017c5e7ffe0261e4d7","learningRecommendation":"com.linkedin.59e2a69f3c2d74e9d4b5b8530e816743"}}}}},"baseType":"com.linkedin.voyager.dash.jobs.InterviewPrepCard"},"com.linkedin.8e2783ee98d67c681658fdc2f0519697":{"fields":{"benefits":{"type":{"array":"string"}},"header":{"type":"string"},"descriptionText":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.jobs.JobPostingBenefitsCard"},"com.linkedin.5f82b47e5488aec975bf096d809fb051":{"fields":{"controlName":{"type":"com.linkedin.voyager.dash.common.tracking.ControlName"},"hintText":{"type":"com.linkedin.390ac1c1431d5b4e340a65af24ed68f2"},"helperText":{"type":"com.linkedin.390ac1c1431d5b4e340a65af24ed68f2"},"validationMetadataResolutionResult":{"derivedFrom":"validationMetadata","type":{"union":{"pronounsV2":"com.linkedin.bfd20979d76e9c8ce942bc03238c6b02","string":"com.linkedin.bfd20979d76e9c8ce942bc03238c6b02","textFormInputType":"com.linkedin.voyager.dash.common.forms.validation.TextFormInputType","postalCode":"com.linkedin.2b6a63a1bb3f7b0bf5c50d8530db9f78","name":"com.linkedin.bfd20979d76e9c8ce942bc03238c6b02","pronouns":"string","integer":"com.linkedin.8409c9296c5bda7be8aee44feff66dd3","decimal":"com.linkedin.8409c9296c5bda7be8aee44feff66dd3","headline":"com.linkedin.bfd20979d76e9c8ce942bc03238c6b02","url":"com.linkedin.48be3e5c4d0dfb61ba0a4a296e719909"}}},"validationMetadata":{"type":{"union":{"pronounsV2":"com.linkedin.bfd20979d76e9c8ce942bc03238c6b02","string":"com.linkedin.bfd20979d76e9c8ce942bc03238c6b02","textFormInputType":"com.linkedin.voyager.dash.common.forms.validation.TextFormInputType","postalCode":"com.linkedin.2b6a63a1bb3f7b0bf5c50d8530db9f78","name":"com.linkedin.bfd20979d76e9c8ce942bc03238c6b02","pronouns":"string","integer":"com.linkedin.8409c9296c5bda7be8aee44feff66dd3","decimal":"com.linkedin.8409c9296c5bda7be8aee44feff66dd3","headline":"com.linkedin.bfd20979d76e9c8ce942bc03238c6b02","url":"com.linkedin.48be3e5c4d0dfb61ba0a4a296e719909"}}}},"baseType":"com.linkedin.voyager.dash.common.forms.SingleLineTextFormComponent"},"com.linkedin.5fe8101cb3a914392f7301964e56573c":{"fields":{"companyUrn":{"type":"string"},"company":{"resolvedFrom":"companyUrn","type":"com.linkedin.a2ef5c1c8d1bd7b83de7009b834a38c6"},"vectorImage":{"type":"com.linkedin.b8949adeaaa915212d0136c4b9a1952b"}},"baseType":"com.linkedin.voyager.dash.common.image.NonEntityCompanyLogo"},"com.linkedin.984a5b2fd75109bc59c5241e98cb46ac":{"fields":{"type":{"type":"com.linkedin.voyager.dash.common.text.ListStyleType"},"index":{"type":"int"}},"baseType":"com.linkedin.voyager.dash.common.text.ListItemStyle"},"com.linkedin.6966477af81c9237d13230a0dced17c7":{"fields":{"showPremiumBranding":{"type":"boolean"},"headerContent":{"type":"string"},"howYouMatchSectionUnions":{"type":{"array":{"union":{"qualificationSection":"com.linkedin.c138fd2c1944eec6c90de7592dd91018","premiumUpsellSection":"com.linkedin.3a31504449fad28c205c97d4985ea256","itemsMatchSection":"com.linkedin.427074b29d6e63261a7a2174b0df610b"}}}},"headerImage":{"type":"com.linkedin.voyager.dash.common.SystemImageName"},"entityUrn":{"type":"com.linkedin.voyager.dash.common.HowYouMatchCardUrn"},"headerTip":{"type":"com.linkedin.00ac75ef8dae327ab29a045438e7d186"},"howYouMatchSection":{"derivedFrom":"howYouMatchSectionUnions","type":{"array":{"union":{"qualificationSection":"com.linkedin.c138fd2c1944eec6c90de7592dd91018","premiumUpsellSection":"com.linkedin.3a31504449fad28c205c97d4985ea256","itemsMatchSection":"com.linkedin.427074b29d6e63261a7a2174b0df610b"}}}},"headerTitle":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.jobs.HowYouMatchCard"},"com.linkedin.4f63adbedb57590490ae8c38e3318a15":{"fields":{"memberStatusDescription":{"type":"com.linkedin.390ac1c1431d5b4e340a65af24ed68f2"},"standardizedSkillUrn":{"type":"string"},"entityUrn":{"type":"com.linkedin.voyager.dash.common.SkillAssessmentCardUrn"},"memberStatus":{"type":"com.linkedin.voyager.dash.jobs.SkillAssessmentMemberStatus"},"attemptReport":{"resolvedFrom":"attemptReportUrn","type":"com.linkedin.4c9ae0bb91144f3556c7bcc9bd1b33bd"},"attemptReportUrn":{"type":"string"},"standardizedSkill":{"resolvedFrom":"standardizedSkillUrn","type":"com.linkedin.7a3dfe06b5ebab0041d8cea76d58565a"}},"baseType":"com.linkedin.voyager.dash.jobs.SkillAssessmentCard"},"com.linkedin.82a950002c26f79d2a63a8f699ce13ca":{"fields":{"selectableOptions":{"type":{"array":{"union":{"textSelectableOption":"com.linkedin.98d89bae4aa4ef8f3ffc1018ca93419c","entitySelectableOption":"com.linkedin.9013fa15f12f02c028d5b08f542f32d5"}}}},"textSelectableOptions":{"type":{"array":"com.linkedin.98d89bae4aa4ef8f3ffc1018ca93419c"}},"selectableOptionsResolutionResults":{"derivedFrom":"selectableOptions","type":{"array":{"union":{"textSelectableOption":"com.linkedin.98d89bae4aa4ef8f3ffc1018ca93419c","entitySelectableOption":"com.linkedin.9013fa15f12f02c028d5b08f542f32d5"}}}}},"baseType":"com.linkedin.voyager.dash.common.forms.RadioButtonFormComponent"},"com.linkedin.d1ea2131164c2418a986748ae0e5000a":{"fields":{"typeaheadFilterQuery":{"type":"com.linkedin.21042ae01d5b4a25985f74505d1df54c"},"showFullLastNameForConnections":{"type":"boolean"},"typeaheadUseCase":{"type":"com.linkedin.voyager.dash.search.TypeaheadUseCase"}},"baseType":"com.linkedin.voyager.dash.search.TypeaheadQuery"},"com.linkedin.fe3639bb2accd8987c7e8ed7912c8eff":{"fields":{"typeaheadFormSuggestionViewModel":{"type":"com.linkedin.45e3a7da0d85a4cd9d5aaa94b34dd17d"},"controlName":{"type":"com.linkedin.voyager.dash.common.tracking.ControlName"},"typeaheadType":{"type":"com.linkedin.voyager.dash.search.TypeaheadType"},"contextualSuggestionQueryParameterUrns":{"type":{"array":"com.linkedin.common.Urn"}},"helperText":{"type":"com.linkedin.390ac1c1431d5b4e340a65af24ed68f2"},"entityImage":{"type":"com.linkedin.30e1ec6a491035bd7a31cc9de8a12c18"},"hintText":{"type":"com.linkedin.390ac1c1431d5b4e340a65af24ed68f2"},"typeaheadQueryContext":{"type":{"array":"string"}},"freeFormTextNotAllowedErrorText":{"type":"com.linkedin.390ac1c1431d5b4e340a65af24ed68f2"},"entityName":{"type":"com.linkedin.390ac1c1431d5b4e340a65af24ed68f2"},"validationMetadata":{"derivedFrom":"validationMetadataUnion","type":{"union":{"pronounsV2":"com.linkedin.bfd20979d76e9c8ce942bc03238c6b02","string":"com.linkedin.bfd20979d76e9c8ce942bc03238c6b02","textFormInputType":"com.linkedin.voyager.dash.common.forms.validation.TextFormInputType","postalCode":"com.linkedin.2b6a63a1bb3f7b0bf5c50d8530db9f78","name":"com.linkedin.bfd20979d76e9c8ce942bc03238c6b02","pronouns":"string","integer":"com.linkedin.8409c9296c5bda7be8aee44feff66dd3","decimal":"com.linkedin.8409c9296c5bda7be8aee44feff66dd3","headline":"com.linkedin.bfd20979d76e9c8ce942bc03238c6b02","url":"com.linkedin.48be3e5c4d0dfb61ba0a4a296e719909"}}},"validationMetadataUnion":{"type":{"union":{"pronounsV2":"com.linkedin.bfd20979d76e9c8ce942bc03238c6b02","string":"com.linkedin.bfd20979d76e9c8ce942bc03238c6b02","textFormInputType":"com.linkedin.voyager.dash.common.forms.validation.TextFormInputType","postalCode":"com.linkedin.2b6a63a1bb3f7b0bf5c50d8530db9f78","name":"com.linkedin.bfd20979d76e9c8ce942bc03238c6b02","pronouns":"string","integer":"com.linkedin.8409c9296c5bda7be8aee44feff66dd3","decimal":"com.linkedin.8409c9296c5bda7be8aee44feff66dd3","headline":"com.linkedin.bfd20979d76e9c8ce942bc03238c6b02","url":"com.linkedin.48be3e5c4d0dfb61ba0a4a296e719909"}}},"freeFormTextAllowed":{"type":"boolean"},"typeaheadMetadata":{"type":"com.linkedin.2dea04d4082c6960578ac30a3ace5ee0"},"ghostImage":{"type":"com.linkedin.30e1ec6a491035bd7a31cc9de8a12c18"}},"baseType":"com.linkedin.voyager.dash.common.forms.SingleTypeaheadEntityFormComponent"},"com.linkedin.756c19defef3189024bfe8e3e74d83ab":{"fields":{"followeeCount":{"type":"long"},"entityUrn":{"type":"com.linkedin.voyager.dash.common.FollowingStateUrn"},"preDashFollowingInfoUrn":{"type":"com.linkedin.voyager.dash.common.PreDashUrn"},"trackingUrn":{"type":"com.linkedin.common.Urn"},"following":{"type":"boolean"},"followingType":{"type":"com.linkedin.voyager.dash.feed.FollowingType"},"followerCount":{"type":"long"}},"baseType":"com.linkedin.voyager.dash.feed.FollowingState"},"com.linkedin.0030135583a12944453eaad4d3729402":{"fields":{"entityUrn":{"type":"com.linkedin.voyager.dash.common.ConnectionUrn"}},"baseType":"com.linkedin.voyager.dash.relationships.Connection"},"com.linkedin.2db94c9a357c51aebecf0054e59f9bd7":{"fields":{"recommendedWorkplaceType":{"resolvedFrom":"recommendedWorkplaceTypeUrns","type":{"array":"com.linkedin.79cb580711dcd3cf7d83e724b3d9e260"}},"existingJobAlert":{"resolvedFrom":"existingJobAlertUrn","type":"com.linkedin.272e46084795068d9243a5e3978fc2e2"},"recommendedGeoUrn":{"type":"string"},"existingJobAlertUrn":{"type":"string"},"recommendedGeo":{"resolvedFrom":"recommendedGeoUrn","type":"com.linkedin.00c4f529ff2fa302d89eaea66344f268"},"recommendedKeywords":{"type":"string"},"recommendedWorkplaceTypeUrns":{"type":{"array":"string"}},"newSearchResultsCount":{"type":"int"},"eligibleToCreate":{"type":"boolean"}},"baseType":"com.linkedin.voyager.dash.jobs.JobAlertCreateEligibility"},"com.linkedin.86ff9fd16c75bc659bfb6287b8ec1647":{"fields":{"elements":{"type":{"array":"com.linkedin.d9052d9e22b039f19b4d6484e2ebec27"}}},"baseType":"com.linkedin.restli.common.CollectionResponse"},"com.linkedin.0054dfa46558041e61c38c995b4809c8":{"fields":{"navigateToReviewLearnMoreModal":{"type":"boolean"},"bannerText":{"type":"string"},"reviewSla":{"type":"int"}},"baseType":"com.linkedin.voyager.dash.jobs.JobPostingTrustDetail"},"com.linkedin.6a325d9bf5f44baf8fede1db5a7ce66f":{"fields":{"subtitle":{"type":"com.linkedin.390ac1c1431d5b4e340a65af24ed68f2"},"spacingType":{"type":"com.linkedin.voyager.dash.common.ux.SpacingType"},"navigationButton":{"type":"com.linkedin.72471cc7e55a55b0fd549127127633f7"},"title":{"type":"string"},"horizontalOrientation":{"type":"boolean"},"formElements":{"type":{"array":"com.linkedin.258b7479f7c0999032ce22032fa02cdf"}},"visibilitySettingButton":{"type":"com.linkedin.b4d79116ccce85e0d4ef36bf211da0a3"}},"baseType":"com.linkedin.voyager.dash.common.forms.FormElementGroup"},"com.linkedin.2ef5b25d61e5db956d269b94a55f4763":{"fields":{"originalImageUrl":{"type":"com.linkedin.common.Url"},"originalHeight":{"type":"int"},"originalWidth":{"type":"int"},"url":{"type":"com.linkedin.common.Url"}},"baseType":"com.linkedin.voyager.dash.common.ImageUrl"},"com.linkedin.b0ab2e740be6873e75f413db4e533bb9":{"fields":{"name":{"type":"string"},"entityUrn":{"type":"com.linkedin.voyager.dash.common.DegreeUrn"}},"baseType":"com.linkedin.voyager.dash.identity.profile.StandardizedDegree"},"com.linkedin.513522081428feabe67539f69de00fc6":{"fields":{"elements":{"type":{"array":"com.linkedin.dc4c6e3940e502e42eb82438140331c7"}}},"baseType":"com.linkedin.restli.common.CollectionResponse"},"com.linkedin.4a3588cb3f298939a6ef5b2e61cadc18":{"fields":{"colorStyle":{"type":"com.linkedin.voyager.dash.common.ux.label.LabelColorStyle"},"text":{"type":"com.linkedin.390ac1c1431d5b4e340a65af24ed68f2"}},"baseType":"com.linkedin.voyager.dash.common.ux.label.LabelViewModel"},"com.linkedin.d4dcbabbf47d9675b0597099d9cd7d69":{"fields":{"name":{"type":"string"},"logo":{"type":{"union":{"url":"com.linkedin.common.Url","vectorImage":"com.linkedin.b8949adeaaa915212d0136c4b9a1952b"}}},"logoResolutionResult":{"derivedFrom":"logo","type":{"union":{"url":"com.linkedin.common.Url","vectorImage":"com.linkedin.b8949adeaaa915212d0136c4b9a1952b"}}},"entityUrn":{"type":"com.linkedin.voyager.dash.common.SchoolUrn"}},"baseType":"com.linkedin.voyager.dash.organization.School"},"com.linkedin.863cd9abd8abab984b4347da5429d447":{"fields":{"cardType":{"type":"com.linkedin.voyager.dash.jobs.JobActivityCardType"},"companyApplyUrl":{"type":"com.linkedin.common.Url"},"entityUrn":{"type":"com.linkedin.voyager.dash.common.JobActivityCardUrn"},"activities":{"type":{"array":"com.linkedin.72617837b227d77ce48feb965f1ecc1b"}}},"baseType":"com.linkedin.voyager.dash.jobs.JobActivityCard"},"com.linkedin.258b7479f7c0999032ce22032fa02cdf":{"fields":{"prerequisiteInputEvaluationStrategyUnion":{"type":{"union":{"answered":"com.linkedin.common.Urn","matchAnyPrerequisiteInput":"com.linkedin.594a52f4e72625c921bc156c950ef9e3","notMatchAnyPrequisiteInput":"com.linkedin.594a52f4e72625c921bc156c950ef9e3"}}},"formComponent":{"type":{"union":{"multilineTextFormComponent":"com.linkedin.22e0be7fe535e4176ecde39607391326","textEntityListFormComponent":"com.linkedin.310d48bff502026d1fdc80b24448401d","toggleFormComponent":"com.linkedin.b0ac2d5e2d9d20cb81b800352f82dc8e","starRatingFormComponent":"com.linkedin.bfb56f758ba39329fbb4081cfe291d5e","nestedCheckboxFormComponent":"com.linkedin.b1148482c3f256b956d443d3fbab2904","mediaUploadFormComponent":"com.linkedin.be19a4b9083f30fb8796eb20ea30c1d0","checkboxFormComponent":"com.linkedin.90840606293e76f91e739cc9fbd0c304","dropdownFormComponent":"com.linkedin.6ba053f5dc7e14e40216570d97a71e52","singleTypeaheadEntityFormComponent":"com.linkedin.fe3639bb2accd8987c7e8ed7912c8eff","numberInputFormComponent":"com.linkedin.274132557f0c6d4e240f7372d489e1cc","dateFormComponent":"com.linkedin.aa89cadbee726b8d3c62475faddcf670","dateRangeFormComponent":"com.linkedin.c734e1c934a3cbf9562953df54d4e771","multiSelectTypeaheadEntityFormComponent":"com.linkedin.985210b44c74343f912e35f90d721799","locationFormComponent":"com.linkedin.60257368060941d7900dbbf2748f711c","singleLineTextFormComponent":"com.linkedin.5f82b47e5488aec975bf096d809fb051","radioButtonFormComponent":"com.linkedin.82a950002c26f79d2a63a8f699ce13ca","pillFormComponent":"com.linkedin.9774b5e7450ba6db34185afe39f9e516"}}},"prerequisiteFormElementInputs":{"type":{"array":"com.linkedin.81235c3224517a02609a065763927f01"}},"requiredFieldMissingErrorText":{"type":"com.linkedin.390ac1c1431d5b4e340a65af24ed68f2"},"weight":{"type":"float"},"title":{"type":"com.linkedin.390ac1c1431d5b4e340a65af24ed68f2"},"prerequisiteInputEvaluationStrategy":{"derivedFrom":"prerequisiteInputEvaluationStrategyUnion","type":{"union":{"answered":"com.linkedin.common.Urn","matchAnyPrerequisiteInput":"com.linkedin.594a52f4e72625c921bc156c950ef9e3","notMatchAnyPrequisiteInput":"com.linkedin.594a52f4e72625c921bc156c950ef9e3"}}},"required":{"type":"boolean"},"shouldAlwaysSendBackFormElementInput":{"type":"boolean"},"urn":{"type":"com.linkedin.common.Urn"},"input":{"type":"com.linkedin.81235c3224517a02609a065763927f01"},"entityUrn":{"type":"com.linkedin.voyager.dash.common.FormElementUrn"},"formComponentResolutionResult":{"derivedFrom":"formComponent","type":{"union":{"multilineTextFormComponent":"com.linkedin.22e0be7fe535e4176ecde39607391326","textEntityListFormComponent":"com.linkedin.310d48bff502026d1fdc80b24448401d","toggleFormComponent":"com.linkedin.b0ac2d5e2d9d20cb81b800352f82dc8e","starRatingFormComponent":"com.linkedin.bfb56f758ba39329fbb4081cfe291d5e","nestedCheckboxFormComponent":"com.linkedin.b1148482c3f256b956d443d3fbab2904","mediaUploadFormComponent":"com.linkedin.be19a4b9083f30fb8796eb20ea30c1d0","checkboxFormComponent":"com.linkedin.90840606293e76f91e739cc9fbd0c304","dropdownFormComponent":"com.linkedin.6ba053f5dc7e14e40216570d97a71e52","singleTypeaheadEntityFormComponent":"com.linkedin.fe3639bb2accd8987c7e8ed7912c8eff","numberInputFormComponent":"com.linkedin.274132557f0c6d4e240f7372d489e1cc","dateFormComponent":"com.linkedin.aa89cadbee726b8d3c62475faddcf670","dateRangeFormComponent":"com.linkedin.c734e1c934a3cbf9562953df54d4e771","multiSelectTypeaheadEntityFormComponent":"com.linkedin.985210b44c74343f912e35f90d721799","locationFormComponent":"com.linkedin.60257368060941d7900dbbf2748f711c","singleLineTextFormComponent":"com.linkedin.5f82b47e5488aec975bf096d809fb051","radioButtonFormComponent":"com.linkedin.82a950002c26f79d2a63a8f699ce13ca","pillFormComponent":"com.linkedin.9774b5e7450ba6db34185afe39f9e516"}}},"helperLink":{"type":"com.linkedin.390ac1c1431d5b4e340a65af24ed68f2"}},"baseType":"com.linkedin.voyager.dash.common.forms.FormElement"},"com.linkedin.f1d8ff8a5c9d0a2c73d8c4418e3a4526":{"fields":{"elements":{"type":{"array":"com.linkedin.180bad7e2024ccff1ee0157be65c0dd1"}}},"baseType":"com.linkedin.restli.common.CollectionResponse"},"com.linkedin.ef6520077f06821116709dada74ee910":{"fields":{"commitmentDisplayText":{"type":"com.linkedin.390ac1c1431d5b4e340a65af24ed68f2"}},"baseType":"com.linkedin.voyager.dash.talentbrand.CommitmentConfig"},"com.linkedin.44e816344c2289e8a62e33486601f239":{"fields":{"dismissControlName":{"type":"com.linkedin.voyager.dash.common.tracking.ControlName"},"viewImpressionTrackingKey":{"type":"string"},"jobSeekerActionCardUnion":{"type":{"union":{"jobSeekerFormActionCard":"com.linkedin.6143491c72aa6f2fce542169c8baa1d6"}}},"jobSeekerActionCard":{"derivedFrom":"jobSeekerActionCardUnion","type":{"union":{"jobSeekerFormActionCard":"com.linkedin.6143491c72aa6f2fce542169c8baa1d6"}}}},"baseType":"com.linkedin.voyager.dash.jobs.JobSeekerActionCard"},"com.linkedin.dc4c6e3940e502e42eb82438140331c7":{"fields":{"entityUrn":{"type":"com.linkedin.voyager.dash.common.PremiumUpsellSlotUrn"},"upsellCard":{"type":"com.linkedin.d9052d9e22b039f19b4d6484e2ebec27"}},"baseType":"com.linkedin.voyager.dash.premium.PremiumUpsellSlotContent"},"com.linkedin.146c92f10032ac18e61e96a3f143b9f1":{"fields":{"urn":{"type":"com.linkedin.common.Urn"},"modelName":{"type":"string"},"fieldName":{"type":"string"},"value":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.common.StringFieldReference"},"com.linkedin.6148d310783eaffead8570485d05a156":{"fields":{"start":{"type":"int"},"end":{"type":"int"}},"baseType":"com.linkedin.common.IntegerRange"},"com.linkedin.34436f930667efbac1e7e0a600c8c653":{"fields":{"title":{"type":"com.linkedin.390ac1c1431d5b4e340a65af24ed68f2"},"organizationRankingDetails":{"type":{"array":"com.linkedin.b7cc21947d4a1ba974040324a832ba64"}}},"baseType":"com.linkedin.voyager.dash.premium.companyinsights.CompanyInsightsOrganizationRanking"},"com.linkedin.9013fa15f12f02c028d5b08f542f32d5":{"fields":{"optionUrn":{"type":"com.linkedin.common.Urn"},"entityImage":{"type":"com.linkedin.30e1ec6a491035bd7a31cc9de8a12c18"},"dismissable":{"type":"boolean"},"shouldRenderTextBeforeImage":{"type":"boolean"},"subtitle":{"type":"com.linkedin.390ac1c1431d5b4e340a65af24ed68f2"},"optionUnion":{"type":{"union":{"optionUrn":"com.linkedin.common.Urn","optionEnumString":"string"}}},"controlName":{"type":"com.linkedin.voyager.dash.common.tracking.ControlName"},"title":{"type":"com.linkedin.390ac1c1431d5b4e340a65af24ed68f2"},"option":{"derivedFrom":"optionUnion","type":{"union":{"optionUrn":"com.linkedin.common.Urn","optionEnumString":"string"}}}},"baseType":"com.linkedin.voyager.dash.common.forms.EntitySelectableOption"},"com.linkedin.b2c46ce630f9fc1eb9ed163a1a7b2616":{"fields":{},"baseType":"com.linkedin.voyager.dash.jobs.CareerEnrichmentCard"},"com.linkedin.392fa039af72d21e485244574f20ca3b":{"fields":{"image":{"type":"com.linkedin.30e1ec6a491035bd7a31cc9de8a12c18"},"titleInsight":{"type":"com.linkedin.390ac1c1431d5b4e340a65af24ed68f2"},"composeOptionControlName":{"type":"com.linkedin.voyager.dash.common.tracking.ControlName"},"navigationUrl":{"type":"com.linkedin.common.Url"},"title":{"type":"com.linkedin.390ac1c1431d5b4e340a65af24ed68f2"},"primaryCTA":{"type":"com.linkedin.5de46613f3027e5a41b356871519a78b"},"composeOption":{"resolvedFrom":"composeOptionUrn","type":"com.linkedin.91ef6d45f2b779a5fad235f8f9dccdb0"},"primaryInsight":{"type":"com.linkedin.390ac1c1431d5b4e340a65af24ed68f2"},"navigationAction":{"type":"com.linkedin.5de46613f3027e5a41b356871519a78b"},"accessibilityText":{"type":"string"},"localizedPrefilledMessageSubject":{"type":"string"},"secondaryInsight":{"type":"com.linkedin.390ac1c1431d5b4e340a65af24ed68f2"},"composeOptionUrn":{"type":"string"},"showPremiumBar":{"type":"boolean"},"subtitle":{"type":"com.linkedin.390ac1c1431d5b4e340a65af24ed68f2"},"localizedPrefilledMessageBody":{"type":"string"},"headline":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.jobs.HiringTeamCard"},"com.linkedin.bd66bae736eabe6505d1b80c182e31af":{"fields":{"image":{"type":"com.linkedin.30e1ec6a491035bd7a31cc9de8a12c18"},"trackingUrn":{"type":"com.linkedin.common.Urn"},"subtitle":{"type":"com.linkedin.390ac1c1431d5b4e340a65af24ed68f2"},"caption":{"type":"com.linkedin.390ac1c1431d5b4e340a65af24ed68f2"},"navigationUrl":{"type":"com.linkedin.common.Url"},"label":{"type":"com.linkedin.390ac1c1431d5b4e340a65af24ed68f2"},"title":{"type":"com.linkedin.390ac1c1431d5b4e340a65af24ed68f2"},"trackingId":{"type":"com.linkedin.voyager.dash.common.Base64String"}},"baseType":"com.linkedin.voyager.dash.common.ux.EntityLockupViewModel"},"com.linkedin.21042ae01d5b4a25985f74505d1df54c":{"fields":{"geoSearchTypes":{"type":{"array":"com.linkedin.voyager.dash.search.GeoSearchType"}},"countryCodes":{"type":{"array":"string"}},"excludedSkill":{"resolvedFrom":"excludedSkillUrns","type":{"array":"com.linkedin.7a3dfe06b5ebab0041d8cea76d58565a"}},"countryCodesResolutionResults":{"resolvedFrom":"countryCodes","type":{"array":"com.linkedin.00c4f529ff2fa302d89eaea66344f268"}},"company":{"resolvedFrom":"companyUrns","type":{"array":"com.linkedin.a2ef5c1c8d1bd7b83de7009b834a38c6"}},"excludedSkillUrns":{"type":{"array":"string"}},"standardizationEntityType":{"type":"string"},"groupUrn":{"type":"string"},"companyUrns":{"type":{"array":"string"}},"group":{"resolvedFrom":"groupUrn","type":"com.linkedin.32bab5d7af29a8bdf4fe920d7d2155b1"}},"baseType":"com.linkedin.voyager.dash.search.TypeaheadFilterQuery"}}}},"included":[{"entityUrn":"urn:li:fsd_jobSkillMatchInsight:3768592532","$recipeTypes":["com.linkedin.65d9b685ce8d8355ba025fb1ae135598"],"insightSummary":{"image":{"attributes":[{"scalingType":null,"detailData":{"profilePictureWithoutFrame":null,"profilePictureWithRingStatus":null,"companyLogo":null,"icon":null,"systemImage":"SYS_ICN_CHECKLIST_MEDIUM","nonEntityGroupLogo":null,"vectorImage":null,"nonEntityProfessionalEventLogo":null,"profilePicture":null,"imageUrl":null,"professionalEventLogo":null,"nonEntityCompanyLogo":null,"nonEntitySchoolLogo":null,"groupLogo":null,"schoolLogo":null,"ghostImage":null,"nonEntityProfilePicture":null},"tintColor":null,"$recipeTypes":["com.linkedin.296fd4ebc3f7a46ccca548d534f656e4"],"tapTargets":[],"displayAspectRatio":null,"$type":"com.linkedin.voyager.dash.common.image.ImageAttribute"}],"editableAccessibilityText":false,"actionTarget":null,"accessibilityTextAttributes":[],"totalCount":null,"accessibilityText":null,"$recipeTypes":["com.linkedin.30e1ec6a491035bd7a31cc9de8a12c18"],"$type":"com.linkedin.voyager.dash.common.image.ImageViewModel"},"text":{"textDirection":"USER_LOCALE","text":"Skills: Data Analytics, Data Analysis, +8 more","attributesV2":[{"start":0,"length":46,"detailData":{"jobPostingName":null,"hyperlink":"#HYM","profileFamiliarName":null,"color":null,"companyName":null,"icon":null,"epoch":null,"systemImage":null,"textLink":null,"listItemStyle":null,"groupName":null,"hyperlinkOpenExternally":null,"listStyle":null,"profileFullName":null,"stringFieldReference":null,"learningCourseName":null,"profileMention":null,"style":null,"schoolName":null,"hashtag":null},"$recipeTypes":["com.linkedin.b0a683a611d43de435ed53c6232d8d44"],"$type":"com.linkedin.voyager.dash.common.text.TextAttribute"}],"accessibilityTextAttributesV2":[],"accessibilityText":null,"$recipeTypes":["com.linkedin.390ac1c1431d5b4e340a65af24ed68f2"],"$type":"com.linkedin.voyager.dash.common.text.TextViewModel"},"$recipeTypes":["com.linkedin.3551030d48734b4fd9cb3238efc77ece"],"$type":"com.linkedin.voyager.dash.common.ux.InsightViewModel"},"$type":"com.linkedin.voyager.dash.assessments.JobSkillMatchInsight"},{"entityUrn":"urn:li:fsd_geo:101282230","countryISOCode":"DE","abbreviatedLocalizedName":"Germany","$recipeTypes":["com.linkedin.00c4f529ff2fa302d89eaea66344f268"],"defaultLocalizedName":"Germany","$type":"com.linkedin.voyager.dash.common.Geo"},{"saved":false,"entityUrn":"urn:li:fsd_saveState:(SAVE,urn:li:fsd_jobPosting:3768592532)","preDashEntityUrn":"urn:li:fs_saveAction:(SAVE,urn:li:fsd_jobPosting:3768592532)","$recipeTypes":["com.linkedin.9731877e56bf195f5714e9f273590184"],"$type":"com.linkedin.voyager.dash.feed.SaveState"},{"companyDetails":{"$recipeTypes":["com.linkedin.b97672681af5b861a5ec5c53b8fcbc96"],"jobCompany":{"rawCompanyName":null,"*company":"urn:li:fsd_company:43338549"},"$type":"com.linkedin.voyager.dash.jobs.JobPostingCompany"},"jobState":"LISTED","trackingUrn":"urn:li:jobPosting:3768592532","*jobWorkplaceTypes":["urn:li:fsd_workplaceType:2"],"repostedJob":false,"trustReviewDecision":null,"title":"Sustainability Data Analyst (f/m/x)","$recipeTypes":["com.linkedin.f8ea4e1538764dc27e403a1d55047e51"],"posterId":null,"$type":"com.linkedin.voyager.dash.jobs.JobPosting","createdAt":1700948520000,"contentSource":"JOBS_CREATE","entityUrn":"urn:li:fsd_jobPosting:3768592532","*location":"urn:li:fsd_geo:101282230","*workplaceTypesResolutionResults":["urn:li:fsd_workplaceType:2"],"jobDetailsSummaryCard":{"*elements":["urn:li:fsd_premiumUpsellSlot:JOB_DETAILS_SUMMARY_CARD"],"$recipeTypes":["com.linkedin.513522081428feabe67539f69de00fc6"],"$type":"com.linkedin.restli.common.CollectionResponse"}},{"preDashNormalizedJobPostingUrn":"urn:li:fs_normalized_jobPosting:3768592532","jobPostingTitle":"Sustainability Data Analyst (f/m/x)","*coachJobsKickoffPrompts":"urn:li:fsd_coachJobsKickoffPrompts:(3768592532,JOB_DETAILS)","navigationBarSubtitle":"Cozero · Germany (Remote)","primaryDescription":{"textDirection":"USER_LOCALE","text":"Cozero · Germany   22 hours ago · 71 applicants","attributesV2":[{"start":0,"length":6,"detailData":{"jobPostingName":null,"hyperlink":"https://www.linkedin.com/company/cozero-io/life","profileFamiliarName":null,"color":null,"companyName":null,"icon":null,"epoch":null,"systemImage":null,"textLink":null,"listItemStyle":null,"groupName":null,"hyperlinkOpenExternally":null,"listStyle":null,"profileFullName":null,"stringFieldReference":null,"learningCourseName":null,"profileMention":null,"style":null,"schoolName":null,"hashtag":null},"$recipeTypes":["com.linkedin.b0a683a611d43de435ed53c6232d8d44"],"$type":"com.linkedin.voyager.dash.common.text.TextAttribute"},{"start":19,"length":12,"detailData":{"jobPostingName":null,"hyperlink":null,"profileFamiliarName":null,"color":"POSITIVE","companyName":null,"icon":null,"epoch":null,"systemImage":null,"textLink":null,"listItemStyle":null,"groupName":null,"hyperlinkOpenExternally":null,"listStyle":null,"profileFullName":null,"stringFieldReference":null,"learningCourseName":null,"profileMention":null,"style":null,"schoolName":null,"hashtag":null},"$recipeTypes":["com.linkedin.b0a683a611d43de435ed53c6232d8d44"],"$type":"com.linkedin.voyager.dash.common.text.TextAttribute"},{"start":19,"length":12,"detailData":{"jobPostingName":null,"hyperlink":null,"profileFamiliarName":null,"color":null,"companyName":null,"icon":null,"epoch":null,"systemImage":null,"textLink":null,"listItemStyle":null,"groupName":null,"hyperlinkOpenExternally":null,"listStyle":null,"profileFullName":null,"stringFieldReference":null,"learningCourseName":null,"profileMention":null,"style":"BOLD","schoolName":null,"hashtag":null},"$recipeTypes":["com.linkedin.b0a683a611d43de435ed53c6232d8d44"],"$type":"com.linkedin.voyager.dash.common.text.TextAttribute"},{"start":19,"length":12,"detailData":{"jobPostingName":null,"hyperlink":null,"profileFamiliarName":null,"color":null,"companyName":null,"icon":null,"epoch":{"epochAt":1700948520000,"type":"TIME_AGO","$recipeTypes":["com.linkedin.94721090e597af7c97503cb8b1a7e50f"],"$type":"com.linkedin.voyager.dash.common.text.EpochTime"},"systemImage":null,"textLink":null,"listItemStyle":null,"groupName":null,"hyperlinkOpenExternally":null,"listStyle":null,"profileFullName":null,"stringFieldReference":null,"learningCourseName":null,"profileMention":null,"style":null,"schoolName":null,"hashtag":null},"$recipeTypes":["com.linkedin.b0a683a611d43de435ed53c6232d8d44"],"$type":"com.linkedin.voyager.dash.common.text.TextAttribute"},{"start":31,"length":3,"detailData":{"jobPostingName":null,"hyperlink":null,"profileFamiliarName":null,"color":"NEUTRAL","companyName":null,"icon":null,"epoch":null,"systemImage":null,"textLink":null,"listItemStyle":null,"groupName":null,"hyperlinkOpenExternally":null,"listStyle":null,"profileFullName":null,"stringFieldReference":null,"learningCourseName":null,"profileMention":null,"style":null,"schoolName":null,"hashtag":null},"$recipeTypes":["com.linkedin.b0a683a611d43de435ed53c6232d8d44"],"$type":"com.linkedin.voyager.dash.common.text.TextAttribute"},{"start":34,"length":13,"detailData":{"jobPostingName":null,"hyperlink":null,"profileFamiliarName":null,"color":"NEUTRAL","companyName":null,"icon":null,"epoch":null,"systemImage":null,"textLink":null,"listItemStyle":null,"groupName":null,"hyperlinkOpenExternally":null,"listStyle":null,"profileFullName":null,"stringFieldReference":null,"learningCourseName":null,"profileMention":null,"style":null,"schoolName":null,"hashtag":null},"$recipeTypes":["com.linkedin.b0a683a611d43de435ed53c6232d8d44"],"$type":"com.linkedin.voyager.dash.common.text.TextAttribute"}],"accessibilityTextAttributesV2":[],"accessibilityText":null,"$recipeTypes":["com.linkedin.390ac1c1431d5b4e340a65af24ed68f2"],"$type":"com.linkedin.voyager.dash.common.text.TextViewModel"},"jobInsightsV2ResolutionResults":[{"premiumUpsellCard":null,"jobInsightViewModel":{"image":{"attributes":[{"scalingType":null,"detailData":{"profilePictureWithoutFrame":null,"profilePictureWithRingStatus":null,"companyLogo":null,"icon":"IC_BRIEFCASE_24DP","systemImage":null,"nonEntityGroupLogo":null,"vectorImage":null,"nonEntityProfessionalEventLogo":null,"profilePicture":null,"imageUrl":null,"professionalEventLogo":null,"nonEntityCompanyLogo":null,"nonEntitySchoolLogo":null,"groupLogo":null,"schoolLogo":null,"ghostImage":null,"nonEntityProfilePicture":null},"tintColor":null,"$recipeTypes":["com.linkedin.296fd4ebc3f7a46ccca548d534f656e4"],"tapTargets":[],"displayAspectRatio":null,"$type":"com.linkedin.voyager.dash.common.image.ImageAttribute"}],"editableAccessibilityText":false,"actionTarget":null,"accessibilityTextAttributes":[],"totalCount":null,"accessibilityText":null,"$recipeTypes":["com.linkedin.30e1ec6a491035bd7a31cc9de8a12c18"],"$type":"com.linkedin.voyager.dash.common.image.ImageViewModel"},"description":[{"text":{"textDirection":"USER_LOCALE","text":"Remote","attributesV2":[],"accessibilityTextAttributesV2":[],"accessibilityText":null,"$recipeTypes":["com.linkedin.390ac1c1431d5b4e340a65af24ed68f2"],"$type":"com.linkedin.voyager.dash.common.text.TextViewModel"},"label":null},{"text":{"textDirection":"USER_LOCALE","text":"Full-time","attributesV2":[],"accessibilityTextAttributesV2":[],"accessibilityText":null,"$recipeTypes":["com.linkedin.390ac1c1431d5b4e340a65af24ed68f2"],"$type":"com.linkedin.voyager.dash.common.text.TextViewModel"},"label":null},{"text":{"textDirection":"USER_LOCALE","text":"Entry level","attributesV2":[],"accessibilityTextAttributesV2":[],"accessibilityText":null,"$recipeTypes":["com.linkedin.390ac1c1431d5b4e340a65af24ed68f2"],"$type":"com.linkedin.voyager.dash.common.text.TextViewModel"},"label":null}],"$recipeTypes":["com.linkedin.efd2ef2c46b7959f85a3d6461edc3e69"],"$type":"com.linkedin.voyager.dash.jobs.JobInsightViewModel"},"insightViewModel":null,"premiumUpsell":null,"skillMatchInsight":null,"verifiedHiringInsight":null,"responsivenessInsightViewModel":null,"trustInsightViewModel":null},{"premiumUpsellCard":null,"jobInsightViewModel":null,"insightViewModel":{"image":{"attributes":[{"scalingType":null,"detailData":{"profilePictureWithoutFrame":null,"profilePictureWithRingStatus":null,"companyLogo":null,"icon":"IC_COMPANY_24DP","systemImage":null,"nonEntityGroupLogo":null,"vectorImage":null,"nonEntityProfessionalEventLogo":null,"profilePicture":null,"imageUrl":null,"professionalEventLogo":null,"nonEntityCompanyLogo":null,"nonEntitySchoolLogo":null,"groupLogo":null,"schoolLogo":null,"ghostImage":null,"nonEntityProfilePicture":null},"tintColor":null,"$recipeTypes":["com.linkedin.296fd4ebc3f7a46ccca548d534f656e4"],"tapTargets":[],"displayAspectRatio":null,"$type":"com.linkedin.voyager.dash.common.image.ImageAttribute"}],"editableAccessibilityText":false,"actionTarget":null,"accessibilityTextAttributes":[],"totalCount":null,"accessibilityText":null,"$recipeTypes":["com.linkedin.30e1ec6a491035bd7a31cc9de8a12c18"],"$type":"com.linkedin.voyager.dash.common.image.ImageViewModel"},"text":{"textDirection":"USER_LOCALE","text":"11-50 employees · Software Development","attributesV2":[],"accessibilityTextAttributesV2":[],"accessibilityText":null,"$recipeTypes":["com.linkedin.390ac1c1431d5b4e340a65af24ed68f2"],"$type":"com.linkedin.voyager.dash.common.text.TextViewModel"},"$recipeTypes":["com.linkedin.3551030d48734b4fd9cb3238efc77ece"],"$type":"com.linkedin.voyager.dash.common.ux.InsightViewModel"},"premiumUpsell":null,"skillMatchInsight":null,"verifiedHiringInsight":null,"responsivenessInsightViewModel":null,"trustInsightViewModel":null},{"premiumUpsellCard":null,"jobInsightViewModel":null,"*premiumUpsell":"urn:li:fsd_jobPosting:3768592532","insightViewModel":null,"skillMatchInsight":null,"verifiedHiringInsight":null,"responsivenessInsightViewModel":null,"trustInsightViewModel":null},{"premiumUpsellCard":null,"jobInsightViewModel":null,"*skillMatchInsight":"urn:li:fsd_jobSkillMatchInsight:3768592532","insightViewModel":null,"premiumUpsell":null,"verifiedHiringInsight":null,"responsivenessInsightViewModel":null,"trustInsightViewModel":null},{"*verifiedHiringInsight":"urn:li:fsd_jobVerifiedHiringInsight:3768592532","premiumUpsellCard":null,"jobInsightViewModel":null,"insightViewModel":null,"premiumUpsell":null,"skillMatchInsight":null,"responsivenessInsightViewModel":null,"trustInsightViewModel":null}],"$recipeTypes":["com.linkedin.f42293b24e5d60e317d2d68ab7597f64"],"primaryActionV2":{"dismissJobAction":null,"applyJobAction":{"applyControlName":"job_details_job_action_apply","$recipeTypes":["com.linkedin.64aff242d842d2f5c0d6ea1432ceb79a"],"$type":"com.linkedin.voyager.dash.jobs.ApplyJobAction","*applyJobActionResolutionResult":"urn:li:fsd_jobSeekerApplicationDetail:3768592532"},"saveJobAction":null},"secondaryActionsV2ResolutionResults":[{"dismissJobAction":null,"applyJobAction":null,"saveJobAction":{"*saveStateResolutionResult":"urn:li:fsd_saveState:(SAVE,urn:li:fsd_jobPosting:3768592532)","saveControlName":"job_details_job_action_save","unsaveControlName":"job_details_job_action_unsave","$recipeTypes":["com.linkedin.c175e2014cb329dc190cefda1f8b1191"],"$type":"com.linkedin.voyager.dash.jobs.SaveJobAction"}}],"$type":"com.linkedin.voyager.dash.jobs.JobPostingCard","secondaryDescription":null,"entityUrn":"urn:li:fsd_jobPostingCard:(3768592532,JOB_DETAILS)","logo":{"attributes":[{"scalingType":null,"detailData":{"profilePictureWithoutFrame":null,"profilePictureWithRingStatus":null,"icon":null,"systemImage":null,"*companyLogo":"urn:li:fsd_company:43338549","nonEntityGroupLogo":null,"vectorImage":null,"nonEntityProfessionalEventLogo":null,"profilePicture":null,"imageUrl":null,"professionalEventLogo":null,"nonEntityCompanyLogo":null,"nonEntitySchoolLogo":null,"groupLogo":null,"schoolLogo":null,"ghostImage":null,"nonEntityProfilePicture":null},"tintColor":null,"$recipeTypes":["com.linkedin.296fd4ebc3f7a46ccca548d534f656e4"],"tapTargets":[],"displayAspectRatio":null,"$type":"com.linkedin.voyager.dash.common.image.ImageAttribute"}],"editableAccessibilityText":false,"actionTarget":"https://www.linkedin.com/company/cozero-io/life","accessibilityTextAttributes":[],"totalCount":null,"accessibilityText":"Cozero logo","$recipeTypes":["com.linkedin.30e1ec6a491035bd7a31cc9de8a12c18"],"$type":"com.linkedin.voyager.dash.common.image.ImageViewModel"},"tip":null,"*jobPosting":"urn:li:fsd_jobPosting:3768592532","tertiaryDescription":null},{"hasConfirmedEmailAddress":true,"applied":false,"applyStartersPreferenceVoid":false,"jobDraftApplicationInfo":null,"formattedApplyDate":null,"$recipeTypes":["com.linkedin.0da643628377886b9fb1a0bb0cc0b73c"],"onsiteApply":false,"$type":"com.linkedin.voyager.dash.jobs.JobSeekerApplicationDetail","applyCtaText":{"textDirection":"USER_LOCALE","text":"Apply","attributesV2":[],"accessibilityTextAttributesV2":[],"accessibilityText":"Apply on company website","$recipeTypes":["com.linkedin.390ac1c1431d5b4e340a65af24ed68f2"],"$type":"com.linkedin.voyager.dash.common.text.TextViewModel"},"entityUrn":"urn:li:fsd_jobSeekerApplicationDetail:3768592532","inPageOffsiteApply":false,"applicantTrackingSystemName":null,"ambryResumeContainer":null,"companyApplyUrl":"https://join.com/companies/cozero/9872937-sustainability-data-analyst-f-m-x?pid=d7cb2e8e859c62cd7f75","appliedAt":null},{"ctaText":null,"entityUrn":"urn:li:fsd_jobVerifiedHiringInsight:3768592532","controlName":"","$recipeTypes":["com.linkedin.ac21e10e9207a4a6523aafa0c51066d4"],"insightSummary":{"image":null,"text":{"textDirection":"USER_LOCALE","attributesV2":[],"text":null,"accessibilityTextAttributesV2":[],"accessibilityText":null,"$recipeTypes":["com.linkedin.390ac1c1431d5b4e340a65af24ed68f2"],"$type":"com.linkedin.voyager.dash.common.text.TextViewModel"},"$recipeTypes":["com.linkedin.3551030d48734b4fd9cb3238efc77ece"],"$type":"com.linkedin.voyager.dash.common.ux.InsightViewModel"},"$type":"com.linkedin.voyager.dash.jobs.JobVerifiedHiringInsight"},{"localizedName":"Remote","entityUrn":"urn:li:fsd_workplaceType:2","workplaceTypeEnum":"REMOTE","localizedDescription":"Employees work off-site.","$recipeTypes":["com.linkedin.79cb580711dcd3cf7d83e724b3d9e260"],"$type":"com.linkedin.voyager.dash.jobs.WorkplaceType"},{"entityUrn":"urn:li:fsd_coachJobsKickoffPrompts:(3768592532,JOB_DETAILS)","$recipeTypes":["com.linkedin.8e19fba94618da7ceb9cca894420a5da"],"kickoffPrompts":[{"queryText":"","controlName":"view_coach","coachSparkleIcon":{"attributes":[{"scalingType":null,"detailData":{"profilePictureWithoutFrame":null,"profilePictureWithRingStatus":null,"companyLogo":null,"icon":null,"systemImage":"SYS_ICN_SIGNAL_AI_SMALL","nonEntityGroupLogo":null,"vectorImage":null,"nonEntityProfessionalEventLogo":null,"profilePicture":null,"imageUrl":null,"professionalEventLogo":null,"nonEntityCompanyLogo":null,"nonEntitySchoolLogo":null,"groupLogo":null,"schoolLogo":null,"ghostImage":null,"nonEntityProfilePicture":null},"tintColor":"PREMIUM_INBUG","$recipeTypes":["com.linkedin.296fd4ebc3f7a46ccca548d534f656e4"],"tapTargets":[],"displayAspectRatio":null,"$type":"com.linkedin.voyager.dash.common.image.ImageAttribute"}],"editableAccessibilityText":false,"actionTarget":null,"accessibilityTextAttributes":[],"totalCount":null,"accessibilityText":null,"$recipeTypes":["com.linkedin.30e1ec6a491035bd7a31cc9de8a12c18"],"$type":"com.linkedin.voyager.dash.common.image.ImageViewModel"},"isPremium":true,"title":{"textDirection":"USER_LOCALE","text":"","attributesV2":[],"accessibilityTextAttributesV2":[],"accessibilityText":null,"$recipeTypes":["com.linkedin.390ac1c1431d5b4e340a65af24ed68f2"],"$type":"com.linkedin.voyager.dash.common.text.TextViewModel"},"$recipeTypes":["com.linkedin.70094a2b1194a3dfad82481287a40a81"],"toolTipText":"Get advice on the right jobs for you","coachQueryContextV2":{"serverIntent":"CoachClientIntent_JOB_ASSESSMENT","$recipeTypes":["com.linkedin.8864b68d440629f1899c8ff463b7f71d"],"$type":"com.linkedin.voyager.dash.common.coach.CoachQueryContextV2"},"coachUseCaseType":"JOB_SEEKER","$type":"com.linkedin.voyager.dash.jobs.gai.CoachJobsKickoffPrompt"},{"queryText":"Am I a good fit for this job?","controlName":"view_coach","coachSparkleIcon":{"attributes":[{"scalingType":null,"detailData":{"profilePictureWithoutFrame":null,"profilePictureWithRingStatus":null,"companyLogo":null,"icon":null,"systemImage":"SYS_ICN_SIGNAL_AI_SMALL","nonEntityGroupLogo":null,"vectorImage":null,"nonEntityProfessionalEventLogo":null,"profilePicture":null,"imageUrl":null,"professionalEventLogo":null,"nonEntityCompanyLogo":null,"nonEntitySchoolLogo":null,"groupLogo":null,"schoolLogo":null,"ghostImage":null,"nonEntityProfilePicture":null},"tintColor":"PREMIUM_INBUG","$recipeTypes":["com.linkedin.296fd4ebc3f7a46ccca548d534f656e4"],"tapTargets":[],"displayAspectRatio":null,"$type":"com.linkedin.voyager.dash.common.image.ImageAttribute"}],"editableAccessibilityText":false,"actionTarget":null,"accessibilityTextAttributes":[],"totalCount":null,"accessibilityText":null,"$recipeTypes":["com.linkedin.30e1ec6a491035bd7a31cc9de8a12c18"],"$type":"com.linkedin.voyager.dash.common.image.ImageViewModel"},"isPremium":true,"title":{"textDirection":"USER_LOCALE","text":"Am I a good fit for this job?","attributesV2":[],"accessibilityTextAttributesV2":[],"accessibilityText":null,"$recipeTypes":["com.linkedin.390ac1c1431d5b4e340a65af24ed68f2"],"$type":"com.linkedin.voyager.dash.common.text.TextViewModel"},"$recipeTypes":["com.linkedin.70094a2b1194a3dfad82481287a40a81"],"toolTipText":"Get advice on the right jobs for you","coachQueryContextV2":{"serverIntent":"CoachClientIntent_JOB_ASSESSMENT","$recipeTypes":["com.linkedin.8864b68d440629f1899c8ff463b7f71d"],"$type":"com.linkedin.voyager.dash.common.coach.CoachQueryContextV2"},"coachUseCaseType":"JOB_SEEKER","$type":"com.linkedin.voyager.dash.jobs.gai.CoachJobsKickoffPrompt"},{"queryText":"How can I best position myself for this job?","controlName":"view_coach","coachSparkleIcon":{"attributes":[{"scalingType":null,"detailData":{"profilePictureWithoutFrame":null,"profilePictureWithRingStatus":null,"companyLogo":null,"icon":null,"systemImage":"SYS_ICN_SIGNAL_AI_SMALL","nonEntityGroupLogo":null,"vectorImage":null,"nonEntityProfessionalEventLogo":null,"profilePicture":null,"imageUrl":null,"professionalEventLogo":null,"nonEntityCompanyLogo":null,"nonEntitySchoolLogo":null,"groupLogo":null,"schoolLogo":null,"ghostImage":null,"nonEntityProfilePicture":null},"tintColor":"PREMIUM_INBUG","$recipeTypes":["com.linkedin.296fd4ebc3f7a46ccca548d534f656e4"],"tapTargets":[],"displayAspectRatio":null,"$type":"com.linkedin.voyager.dash.common.image.ImageAttribute"}],"editableAccessibilityText":false,"actionTarget":null,"accessibilityTextAttributes":[],"totalCount":null,"accessibilityText":null,"$recipeTypes":["com.linkedin.30e1ec6a491035bd7a31cc9de8a12c18"],"$type":"com.linkedin.voyager.dash.common.image.ImageViewModel"},"isPremium":true,"title":{"textDirection":"USER_LOCALE","text":"How can I best position myself for this job?","attributesV2":[],"accessibilityTextAttributesV2":[],"accessibilityText":null,"$recipeTypes":["com.linkedin.390ac1c1431d5b4e340a65af24ed68f2"],"$type":"com.linkedin.voyager.dash.common.text.TextViewModel"},"$recipeTypes":["com.linkedin.70094a2b1194a3dfad82481287a40a81"],"toolTipText":"Get advice on the right jobs for you","coachQueryContextV2":{"serverIntent":"CoachClientIntent_JOB_RECOMMENDED_ACTIONS","$recipeTypes":["com.linkedin.8864b68d440629f1899c8ff463b7f71d"],"$type":"com.linkedin.voyager.dash.common.coach.CoachQueryContextV2"},"coachUseCaseType":"JOB_SEEKER","$type":"com.linkedin.voyager.dash.jobs.gai.CoachJobsKickoffPrompt"},{"queryText":"Tell me more about Cozero","controlName":"view_coach","coachSparkleIcon":{"attributes":[{"scalingType":null,"detailData":{"profilePictureWithoutFrame":null,"profilePictureWithRingStatus":null,"companyLogo":null,"icon":null,"systemImage":"SYS_ICN_SIGNAL_AI_SMALL","nonEntityGroupLogo":null,"vectorImage":null,"nonEntityProfessionalEventLogo":null,"profilePicture":null,"imageUrl":null,"professionalEventLogo":null,"nonEntityCompanyLogo":null,"nonEntitySchoolLogo":null,"groupLogo":null,"schoolLogo":null,"ghostImage":null,"nonEntityProfilePicture":null},"tintColor":"PREMIUM_INBUG","$recipeTypes":["com.linkedin.296fd4ebc3f7a46ccca548d534f656e4"],"tapTargets":[],"displayAspectRatio":null,"$type":"com.linkedin.voyager.dash.common.image.ImageAttribute"}],"editableAccessibilityText":false,"actionTarget":null,"accessibilityTextAttributes":[],"totalCount":null,"accessibilityText":null,"$recipeTypes":["com.linkedin.30e1ec6a491035bd7a31cc9de8a12c18"],"$type":"com.linkedin.voyager.dash.common.image.ImageViewModel"},"isPremium":true,"title":{"textDirection":"USER_LOCALE","text":"Tell me more about Cozero","attributesV2":[],"accessibilityTextAttributesV2":[],"accessibilityText":null,"$recipeTypes":["com.linkedin.390ac1c1431d5b4e340a65af24ed68f2"],"$type":"com.linkedin.voyager.dash.common.text.TextViewModel"},"$recipeTypes":["com.linkedin.70094a2b1194a3dfad82481287a40a81"],"toolTipText":"Get advice on the right jobs for you","coachQueryContextV2":{"serverIntent":"CoachClientIntent_COMPANY_INFO","$recipeTypes":["com.linkedin.8864b68d440629f1899c8ff463b7f71d"],"$type":"com.linkedin.voyager.dash.common.coach.CoachQueryContextV2"},"coachUseCaseType":"JOB_SEEKER","$type":"com.linkedin.voyager.dash.jobs.gai.CoachJobsKickoffPrompt"}],"$type":"com.linkedin.voyager.dash.jobs.gai.CoachJobsKickoffPrompts"},{"name":"Cozero","logoResolutionResult":{"url":null,"vectorImage":{"digitalmediaAsset":"urn:li:digitalmediaAsset:C4D0BAQGkpvwIXWu1-g","attribution":null,"$recipeTypes":["com.linkedin.b8949adeaaa915212d0136c4b9a1952b"],"artifacts":[{"width":200,"$recipeTypes":["com.linkedin.8d64ead1eeea51a297dc70da47337de1"],"fileIdentifyingUrlPathSegment":"200_200/0/1638439865071/cozero_io_logo?e=1709164800&amp;v=beta&amp;t=SkxNXYSV-cQiUJvKiBgKXlXVlM4i0GmvhuX8g6hGKZY","expiresAt":1709164800000,"height":200,"$type":"com.linkedin.common.VectorArtifact"},{"width":100,"$recipeTypes":["com.linkedin.8d64ead1eeea51a297dc70da47337de1"],"fileIdentifyingUrlPathSegment":"100_100/0/1638439865071/cozero_io_logo?e=1709164800&amp;v=beta&amp;t=sVPH2ggrA8QHrpoTseVt0_N71x9Hg_xcONvZl48_Ogk","expiresAt":1709164800000,"height":100,"$type":"com.linkedin.common.VectorArtifact"},{"width":400,"$recipeTypes":["com.linkedin.8d64ead1eeea51a297dc70da47337de1"],"fileIdentifyingUrlPathSegment":"400_400/0/1638439865071/cozero_io_logo?e=1709164800&amp;v=beta&amp;t=G2AlGC86FzMcCKbWIW1B4lPKCq9A1cBdlHhCGtKap9c","expiresAt":1709164800000,"height":400,"$type":"com.linkedin.common.VectorArtifact"}],"rootUrl":"https://media.licdn.com/dms/image/C4D0BAQGkpvwIXWu1-g/company-logo_","$type":"com.linkedin.common.VectorImage"}},"entityUrn":"urn:li:fsd_company:43338549","$recipeTypes":["com.linkedin.a2ef5c1c8d1bd7b83de7009b834a38c6"],"$type":"com.linkedin.voyager.dash.organization.Company"},{"entityUrn":"urn:li:fsd_premiumUpsellSlot:JOB_DETAILS_SUMMARY_CARD","$recipeTypes":["com.linkedin.dc4c6e3940e502e42eb82438140331c7"],"upsellCard":{"image":{"attributes":[{"scalingType":null,"detailData":{"profilePictureWithoutFrame":null,"profilePictureWithRingStatus":null,"companyLogo":null,"icon":null,"systemImage":"SYS_ICN_LIGHTBULB_MEDIUM","nonEntityGroupLogo":null,"vectorImage":null,"nonEntityProfessionalEventLogo":null,"profilePicture":null,"imageUrl":null,"professionalEventLogo":null,"nonEntityCompanyLogo":null,"nonEntitySchoolLogo":null,"groupLogo":null,"schoolLogo":null,"ghostImage":null,"nonEntityProfilePicture":null},"tintColor":null,"$recipeTypes":["com.linkedin.296fd4ebc3f7a46ccca548d534f656e4"],"tapTargets":[],"displayAspectRatio":null,"$type":"com.linkedin.voyager.dash.common.image.ImageAttribute"}],"editableAccessibilityText":false,"actionTarget":null,"accessibilityTextAttributes":[],"totalCount":null,"accessibilityText":null,"$recipeTypes":["com.linkedin.30e1ec6a491035bd7a31cc9de8a12c18"],"$type":"com.linkedin.voyager.dash.common.image.ImageViewModel"},"socialProofInsight":null,"subtext":null,"upsellOrderOrigin":"premium_job_details_summary_card","ctaText":{"textDirection":"USER_LOCALE","text":"See how you compare to 71 applicants. Try Premium for free","attributesV2":[{"start":38,"length":20,"detailData":{"jobPostingName":null,"hyperlink":"https://www.linkedin.com/premium/products/?family=JSS&amp;upsellOrderOrigin=premium_job_details_summary_card&amp;utype=job&amp;referenceId=35%2FiR8HFQmqrpPg04moZPA%3D%3D","profileFamiliarName":null,"color":null,"companyName":null,"icon":null,"epoch":null,"systemImage":null,"textLink":null,"listItemStyle":null,"groupName":null,"hyperlinkOpenExternally":null,"listStyle":null,"profileFullName":null,"stringFieldReference":null,"learningCourseName":null,"profileMention":null,"style":null,"schoolName":null,"hashtag":null},"$recipeTypes":["com.linkedin.b0a683a611d43de435ed53c6232d8d44"],"$type":"com.linkedin.voyager.dash.common.text.TextAttribute"}],"accessibilityTextAttributesV2":[],"accessibilityText":null,"$recipeTypes":["com.linkedin.390ac1c1431d5b4e340a65af24ed68f2"],"$type":"com.linkedin.voyager.dash.common.text.TextViewModel"},"actionUrl":"https://www.linkedin.com/premium/products/?family=JSS&amp;upsellOrderOrigin=premium_job_details_summary_card&amp;utype=job&amp;referenceId=35%2FiR8HFQmqrpPg04moZPA%3D%3D","controlName":"premium_job_details_summary_card","legoTrackingToken":null,"title":{"textDirection":"USER_LOCALE","text":"See how you compare to 71 applicants.","attributesV2":[],"accessibilityTextAttributesV2":[],"accessibilityText":null,"$recipeTypes":["com.linkedin.390ac1c1431d5b4e340a65af24ed68f2"],"$type":"com.linkedin.voyager.dash.common.text.TextViewModel"},"$recipeTypes":["com.linkedin.d9052d9e22b039f19b4d6484e2ebec27"],"$type":"com.linkedin.voyager.dash.premium.PremiumUpsellCard","cancelCta":null,"globalLegoTrackingToken":null,"promotionLegoTrackingToken":null,"funnelCommonHeader":{"utype":"job","$recipeTypes":["com.linkedin.c2408eaefddaf66bea26cb69c95979d7"],"referenceId":"ßâGÁÅBj«¤ø4âj\u0019&lt;","$type":"com.linkedin.voyager.dash.premium.PremiumFunnelCommonHeader"},"fireDiscoveryImpressionEvent":false,"subtitle":null,"dismissible":false,"layoutStyle":"CUSTOM","ctaButtonSecondaryEmphasizedTheme":false},"$type":"com.linkedin.voyager.dash.premium.PremiumUpsellSlotContent"}]}
</code>
<code style="display: none" id="datalet-bpr-guid-15912217" aria-hidden="true">
  {"request":"/voyager/api/graphql?includeWebMetadata\u003Dtrue\u0026variables\u003D(cardSectionTypes:List(TOP_CARD),jobPostingUrn:urn%3Ali%3Afsd_jobPosting%3A3768592532)\u0026queryId\u003DvoyagerJobsDashJobPostingDetailSections.05d61c74bf304974dfa84c4ec2016541","status":200,"body":"bpr-guid-15912217","method":"GET","headers":{"x-li-uuid":"AAYLE+vtCEZ6Amp7P3S1tA\u003D\u003D"}}
</code>
<img src="data:image/gif;base64,R0lGODlhAQABAIAAAAAAAP///yH5BAEAAAAALAAAAAABAAEAAAIBRAA7" style="display: none" class="datalet-bpr-guid-15912217" aria-hidden="true">



  <svg role="none" xmlns="http://www.w3.org/2000/svg" id="hue-web-icons-sprite" style="display: none;" aria-hidden="true">
    <svg xmlns="http://www.w3.org/2000/svg" viewBox="0 0 24 24" id="close-medium" aria-hidden="true" role="none" data-supported-dps="24x24" fill="currentColor">
  <path d="M13.42 12L20 18.58 18.58 20 12 13.42 5.42 20 4 18.58 10.58 12 4 5.42 5.42 4 12 10.58 18.58 4 20 5.42z"></path>
</svg>

    <svg xmlns="http://www.w3.org/2000/svg" viewBox="0 0 16 16" id="search-small" aria-hidden="true" role="none" data-supported-dps="16x16" fill="currentColor">
  <path d="M14.56 12.44L11.3 9.18a5.51 5.51 0 10-2.12 2.12l3.26 3.26a1.5 1.5 0 102.12-2.12zM3 6.5A3.5 3.5 0 116.5 10 3.5 3.5 0 013 6.5z"></path>
</svg>

    <svg xmlns="http://www.w3.org/2000/svg" viewBox="0 0 24 24" id="search-medium" aria-hidden="true" role="none" data-supported-dps="24x24" fill="currentColor">
  <path d="M21.41 18.59l-5.27-5.28A6.83 6.83 0 0017 10a7 7 0 10-7 7 6.83 6.83 0 003.31-.86l5.28 5.27a2 2 0 002.82-2.82zM5 10a5 5 0 115 5 5 5 0 01-5-5z"></path>
</svg>

    <svg xmlns="http://www.w3.org/2000/svg" viewBox="0 0 16 16" id="caret-small" aria-hidden="true" role="none" data-supported-dps="16x16" fill="currentColor">
  <path d="M8 11L3 6h10z" fill-rule="evenodd"></path>
</svg>

    <svg xmlns="http://www.w3.org/2000/svg" viewBox="0 0 24 24" id="overflow-web-ios-medium" aria-hidden="true" role="none" data-supported-dps="24x24" fill="currentColor">
  <path d="M14 12a2 2 0 11-2-2 2 2 0 012 2zM4 10a2 2 0 102 2 2 2 0 00-2-2zm16 0a2 2 0 102 2 2 2 0 00-2-2z"></path>
</svg>

    <svg xmlns="http://www.w3.org/2000/svg" viewBox="0 0 16 16" id="link-external-small" aria-hidden="true" role="none" data-supported-dps="16x16" fill="currentColor">
  <path d="M15 1v6h-2V4.41L7.41 10 6 8.59 11.59 3H9V1zm-4 10a1 1 0 01-1 1H5a1 1 0 01-1-1V6a1 1 0 011-1h2V3H5a3 3 0 00-3 3v5a3 3 0 003 3h5a3 3 0 003-3V9h-2z"></path>
</svg>

    <svg xmlns="http://www.w3.org/2000/svg" viewBox="0 0 24 24" class="rtl-flip" id="share-linkedin-medium" aria-hidden="true" role="none" data-supported-dps="24x24" fill="currentColor">
  <path d="M23 12l-4.61 7H16l4-6H8a3.92 3.92 0 00-4 3.84V17a4 4 0 00.19 1.24L5.12 21H3l-.73-2.22A6.4 6.4 0 012 16.94 6 6 0 018 11h12l-4-6h2.39z"></path>
</svg>

    <svg xmlns="http://www.w3.org/2000/svg" viewBox="0 0 16 16" id="chevron-down-small" aria-hidden="true" role="none" data-supported-dps="16x16" fill="currentColor">
  <path d="M1 5l7 4.61L15 5v2.39L8 12 1 7.39z"></path>
</svg>

    <svg xmlns="http://www.w3.org/2000/svg" viewBox="0 0 16 16" id="bell-outline-small" aria-hidden="true" role="none" data-supported-dps="16x16" fill="currentColor">
  <path d="M15 11.84a2.56 2.56 0 00-.5-1.51L12.9 8.2l-.52-3.41a4.43 4.43 0 00-8.76 0L3.1 8.2l-1.6 2.13a2.56 2.56 0 00-.5 1.51V13h5.59a1.64 1.64 0 00-.09.5 1.5 1.5 0 003 0 1.64 1.64 0 00-.09-.5H15zM8 3a2.43 2.43 0 012.41 2.1l.45 2.9H5.14l.45-2.91A2.43 2.43 0 018 3zm-4.52 8l1.21-1.61L5 9h6l.29.39L12.52 11z"></path>
</svg>

    <svg xmlns="http://www.w3.org/2000/svg" viewBox="0 0 16 16" id="overflow-web-ios-small" aria-hidden="true" role="none" data-supported-dps="16x16" fill="currentColor">
  <path d="M3 9.5A1.5 1.5 0 114.5 8 1.5 1.5 0 013 9.5zM11.5 8A1.5 1.5 0 1013 6.5 1.5 1.5 0 0011.5 8zm-5 0A1.5 1.5 0 108 6.5 1.5 1.5 0 006.5 8z"></path>
</svg>

    <svg xmlns="http://www.w3.org/2000/svg" viewBox="0 0 16 16" id="compose-small" aria-hidden="true" role="none" data-supported-dps="16x16" fill="currentColor">
  <path d="M15 2.53a1.51 1.51 0 01-.44 1L9.15 9 6 10l1-3.12 5.44-5.44A1.5 1.5 0 0115 2.53zM12 11a1 1 0 01-1 1H5a1 1 0 01-1-1V5a1 1 0 011-1h3V2H5a3 3 0 00-3 3v6a3 3 0 003 3h6a3 3 0 003-3V8h-2z"></path>
</svg>

    <svg xmlns="http://www.w3.org/2000/svg" viewBox="0 0 16 16" id="chevron-up-small" aria-hidden="true" role="none" data-supported-dps="16x16" fill="currentColor">
  <path d="M15 11L8 6.39 1 11V8.61L8 4l7 4.61z"></path>
</svg>

    <svg xmlns="http://www.w3.org/2000/svg" viewBox="0 0 16 16" id="check-small" aria-hidden="true" role="none" data-supported-dps="16x16" fill="currentColor">
  <path d="M12.57 2H15L6 15l-5-5 1.41-1.41 3.31 3.3z"></path>
</svg>

    <svg xmlns="http://www.w3.org/2000/svg" viewBox="0 0 16 16" id="add-small" aria-hidden="true" role="none" data-supported-dps="16x16" fill="currentColor">
  <path d="M14 9H9v5H7V9H2V7h5V2h2v5h5z"></path>
</svg>

    <svg xmlns="http://www.w3.org/2000/svg" viewBox="0 0 16 16" id="signal-success-small" aria-hidden="true" role="none" data-supported-dps="16x16" fill="currentColor">
  <path d="M8 1a7 7 0 107 7 7 7 0 00-7-7zm-.6 11L4.25 8.85 5.6 7.51 7.1 9l2.63-4H12z"></path>
</svg>

    <svg xmlns="http://www.w3.org/2000/svg" viewBox="0 0 16 16" id="signal-caution-small" aria-hidden="true" role="none" data-supported-dps="16x16" fill="currentColor">
  <path d="M14.42 6.59l-5-5a2 2 0 00-2.82 0l-5 5a2 2 0 000 2.82l5 5a2 2 0 002.82 0l5-5a2 2 0 000-2.82zM7 4h2v5H7zm1 8.25A1.25 1.25 0 119.25 11 1.25 1.25 0 018 12.25z"></path>
</svg>

    <svg xmlns="http://www.w3.org/2000/svg" viewBox="0 0 24 24" id="lightbulb-medium" aria-hidden="true" role="none" data-supported-dps="24x24" fill="currentColor">
  <path d="M10 20h4v.1a1.9 1.9 0 01-1.9 1.9h-.2a1.9 1.9 0 01-1.9-1.9zm4.68-14.47A1 1 0 0016 5l.76-1.85a1 1 0 00-.54-1.3 1 1 0 00-1.31.54l-.76 1.85a1 1 0 00.53 1.29zm5.09 4.33l1.85-.76a1 1 0 00.54-1.31 1 1 0 00-1.3-.54L19 8a1 1 0 00-.54 1.31 1 1 0 001.3.55zM4.23 14.14l-1.85.76a1 1 0 00-.54 1.31 1 1 0 001.3.54L5 16a1 1 0 00.54-1.31 1 1 0 00-1.3-.55zm17.39.76l-1.85-.76A1 1 0 0019 16l1.85.76a1 1 0 00.78-1.86zM5 8l-1.86-.75a1 1 0 00-1.3.54 1 1 0 00.54 1.31l1.85.76A1 1 0 005 8zm3-3a1 1 0 001.31.54 1 1 0 00.54-1.3L9.1 2.38a1 1 0 00-1.31-.54 1 1 0 00-.54 1.3zm9 6.91V12a5.48 5.48 0 01-1.61 3.88l-.07.08a3.94 3.94 0 00-1.08 1.94L14 19h-4l-.24-1.08A4 4 0 008.68 16l-.07-.07A5.51 5.51 0 017 12v-.1A4.9 4.9 0 0111.9 7h.2a4.9 4.9 0 014.9 4.9zm-2 0A2.9 2.9 0 0012.11 9h-.21A2.9 2.9 0 009 11.9v.1a3.47 3.47 0 001 2.47l.08.08a5.92 5.92 0 011.5 2.45h.84a6 6 0 011.48-2.44l.08-.07A3.48 3.48 0 0015 12z"></path>
</svg>

    <svg xmlns="http://www.w3.org/2000/svg" viewBox="0 0 16 16" class="rtl-flip" id="arrow-right-small" aria-hidden="true" role="none" data-supported-dps="16x16" fill="currentColor">
  <path d="M11.45 3L15 8l-3.55 5H9l2.84-4H2V7h9.84L9 3z"></path>
</svg>

    <svg xmlns="http://www.w3.org/2000/svg" viewBox="0 0 24 24" id="signal-success-medium" aria-hidden="true" role="none" data-supported-dps="24x24" fill="currentColor">
  <path d="M12 2a10 10 0 1010 10A10 10 0 0012 2zm-1.25 15L7 13.25l1.41-1.41L10.59 14l4.84-6H18z"></path>
</svg>

    <svg xmlns="http://www.w3.org/2000/svg" viewBox="0 0 16 16" id="thumbs-up-outline-small" aria-hidden="true" role="none" data-supported-dps="16x16" fill="currentColor">
  <path d="M12.91 7l-2.25-2.57a8.21 8.21 0 01-1.5-2.55L9 1.37A2.08 2.08 0 007 0a2.08 2.08 0 00-2.06 2.08v1.17a5.81 5.81 0 00.31 1.89l.28.86H2.38A1.47 1.47 0 001 7.47a1.45 1.45 0 00.64 1.21 1.48 1.48 0 00-.37 2.06 1.54 1.54 0 00.62.51h.05a1.6 1.6 0 00-.19.71A1.47 1.47 0 003 13.42v.1A1.46 1.46 0 004.4 15h4.83a5.61 5.61 0 002.48-.58l1-.42H14V7zM12 12.11l-1.19.52a3.59 3.59 0 01-1.58.37H5.1a.55.55 0 01-.53-.4l-.14-.48-.49-.21a.56.56 0 01-.34-.6l.09-.56-.42-.42a.56.56 0 01-.09-.68L3.55 9l-.4-.61A.28.28 0 013.3 8h5L7.14 4.51a4.15 4.15 0 01-.2-1.26V2.08A.09.09 0 017 2a.11.11 0 01.08 0l.18.51a10 10 0 001.9 3.24l2.84 3z"></path>
</svg>

    <svg xmlns="http://www.w3.org/2000/svg" viewBox="0 0 16 16" id="thumbs-down-outline-small" aria-hidden="true" role="none" data-supported-dps="16x16" fill="currentColor">
  <path d="M14 9V2h-1.33l-1-.42A5.72 5.72 0 009.23 1H4.4A1.45 1.45 0 003 2.47v.1A1.47 1.47 0 001.75 4a1.6 1.6 0 00.19.71h-.05a1.48 1.48 0 00-.76 2 1.54 1.54 0 00.51.62A1.43 1.43 0 001 8.53 1.47 1.47 0 002.38 10h3.15l-.28.86a5.81 5.81 0 00-.31 1.89v1.17A2.08 2.08 0 007 16a2.08 2.08 0 002-1.37l.18-.51a8.21 8.21 0 011.5-2.55L12.91 9zm-2-1.78l-2.84 3a10 10 0 00-1.88 3.19L7.1 14a.09.09 0 01-.1 0 .09.09 0 01-.08-.08v-1.17a4.15 4.15 0 01.2-1.26L8.3 8H3.38a.27.27 0 01-.27-.29.23.23 0 010-.14L3.55 7l-.37-.61a.56.56 0 01.09-.68l.42-.42-.09-.6a.56.56 0 01.34-.6l.49-.21.14-.48A.55.55 0 015.1 3h4.13a3.59 3.59 0 011.58.37l1.19.52z"></path>
</svg>

</svg>

  <!---->

  
<!---->  

<!---->
<div class="application-outlet" aria-hidden="true">
<!---->
        
    <div class="global-nav__a11y-menu
        ">
      <div class="global-nav__a11y-menu-container">
          <button id="ember7" class="mr4 artdeco-button artdeco-button--muted artdeco-button--2 artdeco-button--secondary ember-view"><!---->
<span class="artdeco-button__text">
    Skip to search
</span></button>

        <button id="ember8" class="artdeco-button artdeco-button--muted artdeco-button--2 artdeco-button--secondary ember-view"><!---->
<span class="artdeco-button__text">
    Skip to main content
</span></button>

<!---->
        <button id="ember9" class="global-nav__a11y-menu-close artdeco-button artdeco-button--circle artdeco-button--muted artdeco-button--2 artdeco-button--tertiary ember-view">        <svg role="none" aria-hidden="true" class="artdeco-button__icon " xmlns="http://www.w3.org/2000/svg" width="24" height="24" viewBox="0 0 24 24" data-supported-dps="24x24" data-test-icon="close-medium">
<!---->    

    <use href="#close-medium" width="24" height="24"></use>
</svg>


<span class="artdeco-button__text">
    Close jump menu
</span></button>
      </div>
    </div>
  
      
    <header id="global-nav" class="global-nav global-alert-offset-top
        
        global-nav--visible">
      <div class="global-nav__content">
          
    <a class="app-aware-link " href="https://www.linkedin.com/feed/?nis=true" data-test-app-aware-link="">
      
    <div class="ivm-image-view-model    global-nav__branding-logo">
        
    <div class="ivm-view-attr__img-wrapper display-flex">
        <li-icon type="app-linkedin-bug-color-icon" class=" " size="large" role="img" aria-label="LinkedIn"><svg xmlns="http://www.w3.org/2000/svg" viewBox="0 0 24 24" data-supported-dps="24x24" fill="currentColor" class="mercado-match" width="24" height="24" focusable="false">
      <path d="M20.5 2h-17A1.5 1.5 0 002 3.5v17A1.5 1.5 0 003.5 22h17a1.5 1.5 0 001.5-1.5v-17A1.5 1.5 0 0020.5 2zM8 19H5v-9h3zM6.5 8.25A1.75 1.75 0 118.3 6.5a1.78 1.78 0 01-1.8 1.75zM19 19h-3v-4.74c0-1.42-.6-1.93-1.38-1.93A1.74 1.74 0 0013 14.19a.66.66 0 000 .14V19h-3v-9h2.9v1.3a3.11 3.11 0 012.7-1.4c1.55 0 3.36.86 3.36 3.66z"></path>
    </svg></li-icon>
    </div>
  
          </div>
  
    </a>
  

        <div id="global-nav-search" class="global-nav__search
            ">
          
          
    <div class="search-global-typeahead
        
         global-nav__search-typeahead">
        
    <div id="global-nav-typeahead" class="search-global-typeahead__typeahead">
<!---->        
          
    <input class="search-global-typeahead__input" placeholder="Search" role="combobox" aria-autocomplete="list" aria-label="Search" aria-activedescendant="" aria-expanded="false" type="text">
  
          <div aria-hidden="true" class="search-global-typeahead__search-icon-container">
            <svg role="none" aria-hidden="true" class="search-global-typeahead__search-icon" xmlns="http://www.w3.org/2000/svg" width="16" height="16" viewBox="0 0 16 16" data-supported-dps="16x16" data-test-icon="search-small">
<!---->    

    <use href="#search-small" width="16" height="16"></use>
</svg>

          </div>
<!---->          <div class="search-global-typeahead__overlay global-alert-offset-top
              "></div>
<!---->        
    </div>
  
      <button class="search-global-typeahead__collapsed-search-button" aria-label="Click to start a search" type="button">
        <span class="search-global-typeahead__collapsed-search-button-icon t-black--light">
          <svg role="none" aria-hidden="true" xmlns="http://www.w3.org/2000/svg" width="24" height="24" viewBox="0 0 24 24" data-supported-dps="24x24" data-test-icon="search-medium">
<!---->    

    <use href="#search-medium" width="24" height="24"></use>
</svg>

        </span>
        <div class="search-global-typeahead__collapsed-search-button-text t-black--light t-12 t-normal">
          Search
        </div>
      </button>
      
  <div id="ember11" class="ember-view"><!----></div>

    </div>
  
      
        </div>

        <nav class="global-nav__nav" aria-label="Primary Navigation">
          <ul class="global-nav__primary-items
              ">
              <li class="global-nav__primary-item
                  ">
                  
      
    <a class="app-aware-link  global-nav__primary-link" target="_self" href="https://www.linkedin.com/feed/?nis=true&amp;" data-test-app-aware-link="">
      

      <div id="ember12" class="global-nav__primary-link-notif artdeco-notification-badge ember-view">  <span class="notification-badge notification-badge--show ">
        <span aria-hidden="true" class="notification-badge__no-count"></span>
        <span class="a11y-text" data-test-notification-a11y="">new feed updates notifications</span>
  </span>
  
        
    <div class="ivm-image-view-model    global-nav__icon-ivm
            ">
        
    <div class="ivm-view-attr__img-wrapper display-flex">
        <li-icon aria-hidden="true" type="home" class=" " size="large"><svg xmlns="http://www.w3.org/2000/svg" viewBox="0 0 24 24" data-supported-dps="24x24" fill="currentColor" class="mercado-match" width="24" height="24" focusable="false">
      <path d="M23 9v2h-2v7a3 3 0 01-3 3h-4v-6h-4v6H6a3 3 0 01-3-3v-7H1V9l11-7z"></path>
    </svg></li-icon>
    </div>
  
          </div>
  
      
</div>
  

      <span class="t-12 break-words block t-black--light t-normal
          global-nav__primary-link-text" title="Home">
        Home
      </span>
    </a>
  

<!---->  
<!----><!----><!---->              </li>
              <li class="global-nav__primary-item
                  ">
                  
      
    <a class="app-aware-link  global-nav__primary-link" target="_self" href="https://www.linkedin.com/mynetwork/?" data-test-app-aware-link="">
      

        
    <div class="ivm-image-view-model    global-nav__icon-ivm
            ">
        
    <div class="ivm-view-attr__img-wrapper display-flex">
        <li-icon aria-hidden="true" type="people" class=" " size="large"><svg xmlns="http://www.w3.org/2000/svg" viewBox="0 0 24 24" data-supported-dps="24x24" fill="currentColor" class="mercado-match" width="24" height="24" focusable="false">
      <path d="M12 16v6H3v-6a3 3 0 013-3h3a3 3 0 013 3zm5.5-3A3.5 3.5 0 1014 9.5a3.5 3.5 0 003.5 3.5zm1 2h-2a2.5 2.5 0 00-2.5 2.5V22h7v-4.5a2.5 2.5 0 00-2.5-2.5zM7.5 2A4.5 4.5 0 1012 6.5 4.49 4.49 0 007.5 2z"></path>
    </svg></li-icon>
    </div>
  
          </div>
  
  

      <span class="t-12 break-words block t-black--light t-normal
          global-nav__primary-link-text" title="My Network">
        My Network
      </span>
    </a>
  

<!---->  
<!----><!----><!---->              </li>
              <li class="global-nav__primary-item
                  ">
                  
      
    <a class="app-aware-link  global-nav__primary-link" target="_self" href="https://www.linkedin.com/jobs/?" data-test-app-aware-link="">
      

        
    <div class="ivm-image-view-model    global-nav__icon-ivm
            ">
        
    <div class="ivm-view-attr__img-wrapper display-flex">
        <li-icon aria-hidden="true" type="job" class=" " size="large"><svg xmlns="http://www.w3.org/2000/svg" viewBox="0 0 24 24" data-supported-dps="24x24" fill="currentColor" class="mercado-match" width="24" height="24" focusable="false">
      <path d="M17 6V5a3 3 0 00-3-3h-4a3 3 0 00-3 3v1H2v4a3 3 0 003 3h14a3 3 0 003-3V6zM9 5a1 1 0 011-1h4a1 1 0 011 1v1H9zm10 9a4 4 0 003-1.38V17a3 3 0 01-3 3H5a3 3 0 01-3-3v-4.38A4 4 0 005 14z"></path>
    </svg></li-icon>
    </div>
  
          </div>
  
  

      <span class="t-12 break-words block t-black--light t-normal
          global-nav__primary-link-text" title="Jobs">
        Jobs
      </span>
    </a>
  

<!---->  
<!----><!----><!---->              </li>
              <li class="global-nav__primary-item
                  ">
                  
      
    <a class="app-aware-link  global-nav__primary-link" target="_self" href="https://www.linkedin.com/messaging/?" data-test-app-aware-link="">
      

        
    <div class="ivm-image-view-model    global-nav__icon-ivm
            ">
        
    <div class="ivm-view-attr__img-wrapper display-flex">
        <li-icon aria-hidden="true" type="nav-small-messaging-icon" class=" " size="large"><svg xmlns="http://www.w3.org/2000/svg" viewBox="0 0 24 24" data-supported-dps="24x24" fill="currentColor" class="mercado-match" width="24" height="24" focusable="false">
      <path d="M16 4H8a7 7 0 000 14h4v4l8.16-5.39A6.78 6.78 0 0023 11a7 7 0 00-7-7zm-8 8.25A1.25 1.25 0 119.25 11 1.25 1.25 0 018 12.25zm4 0A1.25 1.25 0 1113.25 11 1.25 1.25 0 0112 12.25zm4 0A1.25 1.25 0 1117.25 11 1.25 1.25 0 0116 12.25z"></path>
    </svg></li-icon>
    </div>
  
          </div>
  
  

      <span class="t-12 break-words block t-black--light t-normal
          global-nav__primary-link-text" title="Messaging">
        Messaging
      </span>
    </a>
  

<!---->  
<!----><!----><!---->              </li>
              <li class="global-nav__primary-item
                  ">
                  
      
    <a class="app-aware-link  global-nav__primary-link" target="_self" href="https://www.linkedin.com/notifications/?" data-test-app-aware-link="">
      

        
    <div class="ivm-image-view-model    global-nav__icon-ivm
            ">
        
    <div class="ivm-view-attr__img-wrapper display-flex">
        <li-icon aria-hidden="true" type="bell-fill" class=" " size="large"><svg xmlns="http://www.w3.org/2000/svg" viewBox="0 0 24 24" data-supported-dps="24x24" fill="currentColor" class="mercado-match" width="24" height="24" focusable="false">
      <path d="M22 19h-8.28a2 2 0 11-3.44 0H2v-1a4.52 4.52 0 011.17-2.83l1-1.17h15.7l1 1.17A4.42 4.42 0 0122 18zM18.21 7.44A6.27 6.27 0 0012 2a6.27 6.27 0 00-6.21 5.44L5 13h14z"></path>
    </svg></li-icon>
    </div>
  
          </div>
  
  

      <span class="t-12 break-words block t-black--light t-normal
          global-nav__primary-link-text" title="Notifications">
        Notifications
      </span>
    </a>
  

<!---->  
<!----><!----><!---->              </li>
              <li class="global-nav__primary-item
                  ">
<!---->                  
    <div id="ember13" class="global-nav__me artdeco-dropdown artdeco-dropdown--placement-bottom artdeco-dropdown--justification-left ember-view">
      <button aria-expanded="false" id="ember14" class="global-nav__primary-link global-nav__primary-link-me-menu-trigger artdeco-dropdown__trigger artdeco-dropdown__trigger--placement-bottom ember-view" type="button" tabindex="0">
        <img width="24" src="https://media.licdn.com/dms/image/C5603AQEkRciOp6B6dA/profile-displayphoto-shrink_100_100/0/1605279012830?e=1706745600&amp;v=beta&amp;t=GoGDD2DE3nGXji1ktdhEWgWX0uNRNd93omIm9L4PTBM" height="24" alt="Naim Iskandar Zahari" id="ember15" class="global-nav__me-photo evi-image ember-view">

        <span class="global-nav__primary-link-text">
          Me
          <svg role="none" aria-hidden="true" class="global-nav__icon global-nav__icon--small" xmlns="http://www.w3.org/2000/svg" width="16" height="16" viewBox="0 0 16 16" data-supported-dps="16x16" data-test-icon="caret-small">
<!---->    

    <use href="#caret-small" width="16" height="16"></use>
</svg>

        </span>
      
<!----></button>

      <div tabindex="-1" aria-hidden="true" id="ember16" class="global-nav__me-content artdeco-dropdown__content artdeco-dropdown--is-dropdown-element artdeco-dropdown__content--justification-right artdeco-dropdown__content--placement-bottom ember-view"><!----></div>
    </div>

<!---->  
<!----><!---->              </li>
              <li class="global-nav__primary-item
                  ">
<!----><!---->                  
    
    
      
  <button aria-expanded="false" class="global-nav__primary-link
          global-nav__primary-item--divider pl3
          global-nav__app-launcher-trigger pr3" type="button">
    
          
    <div class="ivm-image-view-model    global-nav__icon-ivm">
        
    <div class="ivm-view-attr__img-wrapper display-flex">
        <li-icon aria-hidden="true" type="grid" class=" " size="large"><svg xmlns="http://www.w3.org/2000/svg" viewBox="0 0 24 24" data-supported-dps="24x24" fill="currentColor" class="mercado-match" width="24" height="24" focusable="false">
      <path d="M3 3h4v4H3zm7 4h4V3h-4zm7-4v4h4V3zM3 14h4v-4H3zm7 0h4v-4h-4zm7 0h4v-4h-4zM3 21h4v-4H3zm7 0h4v-4h-4zm7 0h4v-4h-4z"></path>
    </svg></li-icon>
    </div>
  
          </div>
  

        <span class="global-nav__primary-link-text" title="For Business">
          For Business
          <svg role="none" aria-hidden="true" xmlns="http://www.w3.org/2000/svg" width="16" height="16" viewBox="0 0 16 16" data-supported-dps="16x16" data-test-icon="caret-small">
<!---->    

    <use href="#caret-small" width="16" height="16"></use>
</svg>

        </span>
      
  </button>


      <!---->
    

    <div id="ember17" class="ember-view"><!----></div>
  
  
<!---->              </li>
              <li class="global-nav__primary-item
                  ">
<!----><!----><!---->                  
      
    <a class="app-aware-link  global-nav__primary-link" target="_blank" rel="noopener noreferrer" href="https://www.linkedin.com/learning/?trk=nav_neptune_learning&amp;" data-test-app-aware-link="">
      

        
    <div class="ivm-image-view-model    global-nav__icon-ivm
            ">
        
    <div class="ivm-view-attr__img-wrapper display-flex">
        <li-icon aria-hidden="true" type="nav-small-learning-icon" active="true" class=" " size="large"><svg xmlns="http://www.w3.org/2000/svg" viewBox="0 0 24 24" data-supported-dps="24x24" fill="currentColor" width="24" height="24" focusable="false">
      <path d="M22 5v14H2V5h20m1-2H1a1 1 0 00-1 1v16a1 1 0 001 1h22a1 1 0 001-1V4a1 1 0 00-1-1z"></path>
      <path d="M2 5v14h10V5H2zm8 12H4v-2h6v2zm0-4H4v-2h6v2zm0-4H4V7h6v2z" style="isolation:isolate" opacity=".25"></path>
      <path style="isolation:isolate" opacity=".6" d="M14 7h6v2h-6zM14 11h6v2h-6zM14 15h6v2h-6z"></path>
      <path d="M10 7.53v8.93a.28.28 0 00.44.23l6.43-4.44a.33.33 0 00.06-.46l-.06-.06-6.43-4.43a.28.28 0 00-.44.23z"></path>
    </svg></li-icon>
    </div>
  
          </div>
  
  

      <span class="t-12 break-words block t-black--light t-normal
          global-nav__primary-link-text" title="Learning">
        Learning
      </span>
    </a>
  

<!---->  
              </li>

<!---->          </ul>
        </nav>
      </div>
    </header>
  

    
    <div class="videoinappalert-inapp-alerts-manager
        hidden">
      
<!---->    
    </div>
  

<!---->
<div class="authentication-outlet">
<!---->  <!---->



  
        <div class="scaffold-layout__tracking-element"></div>

<div class="scaffold-layout
    scaffold-layout--breakpoint-none
    scaffold-layout--main-aside
    scaffold-layout--single-column
    scaffold-layout--reflow
    
     job-view-layout jobs-details">
    <section class="scaffold-layout-toolbar
    scaffold-layout-toolbar--is-fixed
    scaffold-layout-toolbar--is-fixed-visible
    ">
  <div class="scaffold-layout-toolbar__content scaffold-layout-container
      scaffold-layout-container--reflow
      ">
    
      
          
    <div class="display-flex flex-row justify-space-between">
      <div>
        <h2 class="t-16 t-black t-bold truncate">
          Sustainability Data Analyst (f/m/x)
        </h2>
        <span>Cozero</span>
          <span class="jobs-unified-top-card__bullet">Germany</span>
          <span class="jobs-unified-top-card__workplace-type">Remote</span>
      </div>
      <div class="display-flex align-items-center">
        
    <div id="ember23" class="artdeco-dropdown jobs-options artdeco-dropdown--placement-bottom artdeco-dropdown--justification-right ember-view inline-flex
            ">
      
          <button aria-expanded="false" id="ember24" class="artdeco-button artdeco-button--2 artdeco-button--tertiary artdeco-button--muted artdeco-button--circle artdeco-dropdown__trigger artdeco-dropdown__trigger--placement-bottom ember-view" type="button" tabindex="0">
            <svg role="none" aria-hidden="true" class="artdeco-button__icon" xmlns="http://www.w3.org/2000/svg" width="24" height="24" viewBox="0 0 24 24" data-supported-dps="24x24" data-test-icon="overflow-web-ios-medium">
<!---->    

    <use href="#overflow-web-ios-medium" width="24" height="24"></use>
</svg>

            <span class="artdeco-button__text">
              Show more options
            </span>
          
<!----></button>
        
      <div tabindex="-1" aria-hidden="true" id="ember25" class="artdeco-dropdown__content artdeco-dropdown--is-dropdown-element artdeco-dropdown__content--has-arrow artdeco-dropdown__content--arrow-right artdeco-dropdown__content--justification-right artdeco-dropdown__content--placement-bottom ember-view"><!----></div>
    </div>
  
            
    <button class="jobs-save-button artdeco-button artdeco-button--3 artdeco-button--secondary ml2" type="button">
<!---->          <span aria-hidden="true">
            Save
          </span>
        <span class="a11y-text">
          Save Sustainability Data Analyst (f/m/x) at Cozero
        </span>
    </button>
  
        
    <div class="jobs-s-apply jobs-s-apply--fadein inline-flex ml2">
        
    <div>
      <button role="link" aria-label="Apply to Sustainability Data Analyst (f/m/x) on company website" id="ember26" class="jobs-apply-button artdeco-button artdeco-button--icon-right artdeco-button--3 artdeco-button--primary ember-view">        <svg role="none" aria-hidden="true" class="artdeco-button__icon " xmlns="http://www.w3.org/2000/svg" width="16" height="16" viewBox="0 0 16 16" data-supported-dps="16x16" data-test-icon="link-external-small">
<!---->    

    <use href="#link-external-small" width="16" height="16"></use>
</svg>


<span class="artdeco-button__text">
    Apply
</span></button>
    </div>
  
    </div>
  
      </div>
    </div>
  
        
    
  </div>
</section>

  <div class="scaffold-layout__inner scaffold-layout-container
      scaffold-layout-container--reflow
      ">
<!---->
      <div class="scaffold-layout__row scaffold-layout__content
          scaffold-layout__content--main-aside
          
          
          scaffold-layout__content--has-aside
          
          ">
<!---->
          <main class="scaffold-layout__main">
  
            
          
    <div class="job-view-layout jobs-details">
      <div>
        <div class="
            mt4
            " role="main">
            

<!---->  
          
          <div>
            
    <div class="jobs-unified-top-card t-14 artdeco-card mb4">
<!---->      <div class="relative
          ">

        <div class="p5">
              <a class="app-aware-link " aria-label="Cozero logo" href="https://www.linkedin.com/company/cozero-io/life" data-test-app-aware-link="">
                
    <div class="ivm-image-view-model    mb3">
        
    <div class="ivm-view-attr__img-wrapper display-flex">
<!---->          <img width="48" src="https://media.licdn.com/dms/image/C4D0BAQGkpvwIXWu1-g/company-logo_100_100/0/1638439865071/cozero_io_logo?e=1709164800&amp;v=beta&amp;t=sVPH2ggrA8QHrpoTseVt0_N71x9Hg_xcONvZl48_Ogk" loading="lazy" height="48" alt="Cozero logo" id="ember46" class="ivm-view-attr__img--centered EntityPhoto-square-3   evi-image lazy-image ember-view">
    </div>
  
          </div>
  
              </a>

          <div class="display-flex justify-space-between flex-wrap">
              <h1 class="t-24 t-bold job-details-jobs-unified-top-card__job-title">
                  Sustainability Data Analyst (f/m/x)
              </h1>

<!---->          </div>

          <div class="job-details-jobs-unified-top-card__primary-description">
              <div>
                <a class="app-aware-link " target="_self" href="https://www.linkedin.com/company/cozero-io/life" data-test-app-aware-link=""><!---->Cozero<!----></a><span class="white-space-pre"> </span>· Germany<span class="white-space-pre"> </span><span class="tvm__text tvm__text--positive"><strong>      <span>
        <!---->22 hours ago<!---->
      </span>
</strong></span><span class="tvm__text tvm__text--neutral"><span class="white-space-pre"> </span>·<span class="white-space-pre"> </span></span><span class="tvm__text tvm__text--neutral"><!---->71 applicants<!----></span>
              </div>
          </div>

<!---->
          <div class="mt5 mb2">
              <ul>
                    <li class="job-details-jobs-unified-top-card__job-insight">
                      <div class="flex-shrink-zero mr2 t-black--light">
                        
    <div class="ivm-image-view-model   ">
        
    <div class="ivm-view-attr__img-wrapper display-flex">
        <li-icon aria-hidden="true" type="job" class=" " size="large"><svg xmlns="http://www.w3.org/2000/svg" viewBox="0 0 24 24" data-supported-dps="24x24" fill="currentColor" class="mercado-match" width="24" height="24" focusable="false">
      <path d="M17 6V5a3 3 0 00-3-3h-4a3 3 0 00-3 3v1H2v4a3 3 0 003 3h14a3 3 0 003-3V6zM9 5a1 1 0 011-1h4a1 1 0 011 1v1H9zm10 9a4 4 0 003-1.38V17a3 3 0 01-3 3H5a3 3 0 01-3-3v-4.38A4 4 0 005 14z"></path>
    </svg></li-icon>
    </div>
  
          </div>
  
                      </div>
                      <span>
                            <span>
                              <!---->Remote<!---->
                            </span>

<!---->                            <span class="job-details-jobs-unified-top-card__job-insight-view-model-secondary">
                              <!---->Full-time<!---->
                            </span>

<!---->                            <span class="job-details-jobs-unified-top-card__job-insight-view-model-secondary">
                              <!---->Entry level<!---->
                            </span>

<!---->                      </span>
                    </li>
                                      <li class="job-details-jobs-unified-top-card__job-insight">
                      <div class="flex-shrink-zero mr2 t-black--light">
                        
    <div class="ivm-image-view-model   ">
        
    <div class="ivm-view-attr__img-wrapper display-flex">
        <li-icon aria-hidden="true" type="company" class=" " size="large"><svg xmlns="http://www.w3.org/2000/svg" viewBox="0 0 24 24" data-supported-dps="24x24" fill="currentColor" class="mercado-match" width="24" height="24" focusable="false">
      <path d="M4 2v20h16V2zm14 18h-4v-2h-4v2H6V4h12zm-7-8H8v-2h3zm0 4H8v-2h3zm5-4h-3v-2h3zm-5-4H8V6h3zm5 0h-3V6h3zm0 8h-3v-2h3z"></path>
    </svg></li-icon>
    </div>
  
          </div>
  
                      </div>
                      <span>
                        <!---->11-50 employees · Software Development<!---->
                      </span>
                    </li>
                        <li class="job-details-jobs-unified-top-card__job-insight job-details-jobs-unified-top-card__job-insight--highlight">
                          <div class="flex-shrink-zero mr2 t-black--light">
                            
    <div class="ivm-image-view-model   ">
        
    <div class="ivm-view-attr__img-wrapper display-flex">
        <li-icon aria-hidden="true" type="lightbulb" class=" " size="medium"><svg xmlns="http://www.w3.org/2000/svg" viewBox="0 0 24 24" data-supported-dps="24x24" fill="currentColor" class="mercado-match" width="24" height="24" focusable="false">
      <path d="M10 20h4v.1a1.9 1.9 0 01-1.9 1.9h-.2a1.9 1.9 0 01-1.9-1.9zm4.68-14.47A1 1 0 0016 5l.76-1.85a1 1 0 00-.54-1.3 1 1 0 00-1.31.54l-.76 1.85a1 1 0 00.53 1.29zm5.09 4.33l1.85-.76a1 1 0 00.54-1.31 1 1 0 00-1.3-.54L19 8a1 1 0 00-.54 1.31 1 1 0 001.3.55zM4.23 14.14l-1.85.76a1 1 0 00-.54 1.31 1 1 0 001.3.54L5 16a1 1 0 00.54-1.31 1 1 0 00-1.3-.55zm17.39.76l-1.85-.76A1 1 0 0019 16l1.85.76a1 1 0 00.78-1.86zM5 8l-1.86-.75a1 1 0 00-1.3.54 1 1 0 00.54 1.31l1.85.76A1 1 0 005 8zm3-3a1 1 0 001.31.54 1 1 0 00.54-1.3L9.1 2.38a1 1 0 00-1.31-.54 1 1 0 00-.54 1.3zm9 6.91V12a5.48 5.48 0 01-1.61 3.88l-.07.08a3.94 3.94 0 00-1.08 1.94L14 19h-4l-.24-1.08A4 4 0 008.68 16l-.07-.07A5.51 5.51 0 017 12v-.1A4.9 4.9 0 0111.9 7h.2a4.9 4.9 0 014.9 4.9zm-2 0A2.9 2.9 0 0012.11 9h-.21A2.9 2.9 0 009 11.9v.1a3.47 3.47 0 001 2.47l.08.08a5.92 5.92 0 011.5 2.45h.84a6 6 0 011.48-2.44l.08-.07A3.48 3.48 0 0015 12z"></path>
    </svg></li-icon>
    </div>
  
          </div>
  
                          </div>
                          <span>
                            <!---->See how you compare to 71 applicants.<span class="white-space-pre"> </span><a class="app-aware-link " target="_self" href="https://www.linkedin.com/premium/products/?family=JSS&amp;upsellOrderOrigin=premium_job_details_summary_card&amp;utype=job&amp;referenceId=35%2FiR8HFQmqrpPg04moZPA%3D%3D" data-test-app-aware-link=""><!---->Try Premium for free<!----></a>
                          </span>
                        </li>
                        <li class="job-details-jobs-unified-top-card__job-insight">
                          <div class="flex-shrink-zero mr2 t-black--light">
                            
    <div class="ivm-image-view-model   ">
        
    <div class="ivm-view-attr__img-wrapper display-flex">
        <li-icon aria-hidden="true" type="checklist" class=" " size="medium"><svg xmlns="http://www.w3.org/2000/svg" viewBox="0 0 24 24" data-supported-dps="24x24" fill="currentColor" class="mercado-match" width="24" height="24" focusable="false">
      <path d="M22 3v2H11V3zM11 13h11v-2H11zm0 8h11v-2H11zM4.85 4L3.34 2.51 2 3.85 5.15 7l4.6-7H7.48zm0 8l-1.51-1.49L2 11.85 5.15 15l4.6-7H7.48zm0 8l-1.51-1.49L2 19.85 5.15 23l4.6-7H7.48z"></path>
    </svg></li-icon>
    </div>
  
          </div>
  
                          </div>
                          <button class="job-details-jobs-unified-top-card__job-insight-text-button" aria-label="View strong skill match modal" tabindex="-1" type="button">
                            <a class="app-aware-link " target="_self" href="#HYM" data-test-app-aware-link=""><!---->Skills: Data Analytics, Data Analysis, +8 more<!----></a>
                          </button>
                        </li>
<!---->              </ul>
          </div>

<!---->
          <div class="mt5">
              <div class="display-flex">
                  
    <div class="jobs-s-apply jobs-s-apply--fadein inline-flex mr2">
        
    <div class="jobs-apply-button--top-card">
      <button role="link" aria-label="Apply to Sustainability Data Analyst (f/m/x) on company website" id="ember47" class="jobs-apply-button artdeco-button artdeco-button--icon-right artdeco-button--3 artdeco-button--primary ember-view">        <svg role="none" aria-hidden="true" class="artdeco-button__icon " xmlns="http://www.w3.org/2000/svg" width="16" height="16" viewBox="0 0 16 16" data-supported-dps="16x16" data-test-icon="link-external-small">
<!---->    

    <use href="#link-external-small" width="16" height="16"></use>
</svg>


<span class="artdeco-button__text">
    Apply
</span></button>
    </div>
  
    </div>
  

                  <span class="visibility-hidden"></span>

                      
    <button class="jobs-save-button artdeco-button artdeco-button--3 artdeco-button--secondary" type="button">
<!---->          <span aria-hidden="true">
            Save
          </span>
        <span class="a11y-text">
          Save Sustainability Data Analyst (f/m/x) at Cozero
        </span>
    </button>
  
              </div>

<!---->
<!---->
<!----><!---->          </div>

<!---->
<!---->
<!----><!---->
<!---->        </div>

<!---->
<!---->
        <div class="job-details-jobs-unified-top-card__buttons-container absolute">
          <div class="display-flex flex-column">
            <div class="display-flex justify-flex-end">
                
    <div id="ember30" class="artdeco-dropdown artdeco-dropdown--placement-bottom artdeco-dropdown--justification-right ember-view">
        <button aria-expanded="false" aria-label="Share" id="ember31" class="social-share__dropdown-trigger artdeco-button artdeco-button--3 artdeco-button--tertiary artdeco-button--circle artdeco-button--muted artdeco-dropdown__trigger artdeco-dropdown__trigger--placement-bottom ember-view" type="button" tabindex="0">
          <svg role="none" aria-hidden="true" class="artdeco-button__icon rtl-flip" xmlns="http://www.w3.org/2000/svg" width="24" height="24" viewBox="0 0 24 24" data-supported-dps="24x24" data-test-icon="share-linkedin-medium">
<!---->    

    <use href="#share-linkedin-medium" width="24" height="24"></use>
</svg>

          <span class="artdeco-button__text">Share</span>
        
<!----></button>

      <div tabindex="-1" aria-hidden="true" id="ember32" class="social-share__content text-align-left artdeco-dropdown__content artdeco-dropdown--is-dropdown-element artdeco-dropdown__content--has-arrow artdeco-dropdown__content--arrow-right artdeco-dropdown__content--justification-right artdeco-dropdown__content--placement-bottom ember-view"><!----></div>
    </div>

    <div>
  
      
<!---->  
  
</div>

<!---->

<!---->  
                
    <div id="ember33" class="artdeco-dropdown jobs-options artdeco-dropdown--placement-bottom artdeco-dropdown--justification-right ember-view">
      
                  <button aria-expanded="false" id="ember34" class="artdeco-button artdeco-button--3 artdeco-button--tertiary artdeco-button--muted artdeco-button--circle artdeco-dropdown__trigger artdeco-dropdown__trigger--placement-bottom ember-view" type="button" tabindex="0">
                    <svg role="none" aria-hidden="true" class="artdeco-button__icon" xmlns="http://www.w3.org/2000/svg" width="24" height="24" viewBox="0 0 24 24" data-supported-dps="24x24" data-test-icon="overflow-web-ios-medium">
<!---->    

    <use href="#overflow-web-ios-medium" width="24" height="24"></use>
</svg>

                    <span class="artdeco-button__text">
                      Show more options
                    </span>
                  
<!----></button>
                
      <div tabindex="-1" aria-hidden="true" id="ember35" class="artdeco-dropdown__content artdeco-dropdown--is-dropdown-element artdeco-dropdown__content--has-arrow artdeco-dropdown__content--arrow-right artdeco-dropdown__content--justification-right artdeco-dropdown__content--placement-bottom ember-view"><!----></div>
    </div>
  
            </div>
          </div>
        </div>
      </div>
<!---->    </div>
  
<!---->
<!---->
<!---->
<!---->          </div>

<!---->
            
    <div class="jobs-box--fadein jobs-box--full-width jobs-box--with-cta-large jobs-description
         jobs-description--is-truncated jobs-description--is-truncated-poster
        
        
         artdeco-card mb4
        
        ">

<!---->
      <article class="jobs-description__container
          jobs-description__container--condensed">
        <div class="jobs-description__content jobs-description-content
            jobs-description__content--condensed">
          <div class="jobs-box__html-content jobs-description-content__text t-14 t-normal
              jobs-description-content__text--stretch" id="job-details" tabindex="-1">
            <h2 class="text-heading-large
                mb4">
              About the job
            </h2>

<!---->            <span>
                <strong><!---->Join Our Mission to Fight Climate Change as our Sustainability Data Analyst<!----><br><br></strong><!---->Want to be part of one of Europe's fastest growing climate tech startups? We are looking for a passionate Sustainability Data Analyst to help us transform every company into a sustainable business, with critical responsibility and influence on our product and overall venture development.<!----><br><br><!---->As a Sustainability Data Analyst your role will have a key responsibility of applying data-driven approaches to managing our carbon footprint calculations and ensuring our customers understand how we got there.<!----><br><br><!---->If you are excited about building an economy where every company creates an impact for a sustainable world and empowering companies to decarbonize and boost their business performance, we’d love to hear from you!<!----><br><br><!---->If you want to find out more about Cozero and our approach to hiring, check out this page.<!----><br><br><strong><!---->Tasks<!----><br><br></strong><strong><!---->What you’ll do:<!----><br><br></strong>      <ul><li><!---->You will be developing, maintaining and improving our methodologies and models for calculating companies GHG emissions<!----></li><li><!---->You will develop and oversee processes to ensure our clients receive accurate CCF and PCF results (eg. Quality assurance, validating user carbon footprints)<!----></li><li><!---->You will transform unstructured data sets so they are ready to be ingested into our calculation databases and models<!----></li><li><!---->You will work across Cozero’s climate, data science and engineering teams, to deliver on high quality data outputs for internal and external use<!----></li><li><!---->You will work closely with our Sales and Customer Success teams to define and prioritize data needs<!----></li><li><!---->You will proactively develop ideas on how we can create more value out of our data as well as external data for our clients<!----><br><br></li></ul>
<strong><!---->Requirements<!----><br><br></strong><strong><!---->Who you are:<!----><br><br></strong>      <ul><li><!---->You care deeply about our planet and staying informed on key environmental trends like CO2 emissions and climate change.<span class="white-space-pre"> </span></li><li><!---->Your meticulous eye for detail ensures thorough, systematic and accurate data analysis even when working with complex, unstructured and inconsistent datasets.<span class="white-space-pre"> </span></li><li><!---->A fast-evolving environment excites you, and you love contributing your skills to a diverse, collaborative team making a real difference in the world.<span class="white-space-pre"> </span><br><br></li></ul>
<strong><!---->What we’re looking for:<!----><br><br></strong>      <ul><li><!---->Experience working in an analyst role or similar (ideally working with climate data like Corporate GHG emissions, Emission factor and LCA databases)<!----></li><li><!---->You have programming skills in Python and SQL and a strong understanding of data gathering, inspecting, cleansing, transforming, and modeling techniques<!----></li><li><!---->You have strong verbal and written English communication skills<span class="white-space-pre"> </span><br><br></li></ul>
<strong><!---->Extra bonus points if you also have the following:<!----><br><br></strong>      <ul><li><!---->Master's degree in Environmental Science, Engineering, Computer Science or a related field<!----></li><li><!---->Knowledge of different carbon accounting standards and databases (e.g. GHG Protocol, GLEC, Ecoinvent)<!----></li><li><!---->Experience in carbon accounting and/ or life cycle assessment<!----><br><br></li></ul>
<strong><!---->Benefits<!----><br><br></strong><strong><!---->What we offer:<!----><br><br></strong>      <ul><li><!---->Opportunity to work remotely or from our great co-working space in Maschinenraum Berlin with access to innovation events of the German Mittelstand and the startup ecosystem<!----></li><li><!---->Virtual meditation and wellness sessions<!----></li><li><!---->Regular team events and at least 2 offsites per year<!----></li><li><!---->Individual learning &amp; development budget<!----></li><li><!---->The option to participate in our Virtual Stock Option Plan (VSOP), to have direct involvement and impact on Cozero’s success<!----></li><li><!---->A strong company culture focused on growth, collaboration and fighting climate change<!----></li><li><!---->Participation opportunities in accelerator, startup &amp; event programs<!----><br><br></li></ul>
<!---->We strive to build a diverse, inclusive and collaborative workplace and we follow a fair, objective, and non-discriminatory recruitment process. All applications will be considered for employment regardless of nationality, culture, or any other personal characteristics.<!----><br><br><!---->The glue that binds us together despite our differences is a common set of values, our purpose to transform every company into a low-emission business, and our culture that has a growth mindset at its heart. If you’re excited about this role we encourage you to apply, even if your past experience doesn’t perfectly match every requirement in the job description.<!---->
<!---->            </span>
          </div>
<!---->          <div class="jobs-description__details">
<!---->          </div>
        </div>
      </article>
        <footer>
          <button aria-label="Click to see more description" aria-expanded="false" id="ember36" class="jobs-description__footer-button t-14 t-black--light t-bold artdeco-card__action artdeco-button artdeco-button--icon-right artdeco-button--3 artdeco-button--fluid artdeco-button--tertiary ember-view">        <svg role="none" aria-hidden="true" class="artdeco-button__icon " xmlns="http://www.w3.org/2000/svg" width="16" height="16" viewBox="0 0 16 16" data-supported-dps="16x16" data-test-icon="chevron-down-small">
<!---->    

    <use href="#chevron-down-small" width="16" height="16"></use>
</svg>


<span class="artdeco-button__text">
    See more
</span></button>
        </footer>
    </div>
  
            
    <div class="artdeco-card pv5 mb4 full-width ph5">
      <div class="display-flex justify-space-between align-items-center">
        <div>
          <h2 class="t-sans t-black job-details-job-alert__header text-heading-large">
            Set alert for similar jobs
          </h2>
            <div class="job-details-job-alert__description text-body-small">
              <span>
                Data Analyst, Germany
              </span>
            </div>
        </div>
        <div>
          <button disabled="" id="ember37" class="artdeco-button artdeco-button--3 artdeco-button--secondary artdeco-button--disabled ember-view">        <svg role="none" aria-hidden="true" class="artdeco-button__icon " xmlns="http://www.w3.org/2000/svg" width="16" height="16" viewBox="0 0 16 16" data-supported-dps="16x16" data-test-icon="check-small">
<!---->    

    <use href="#check-small" width="16" height="16"></use>
</svg>


<span class="artdeco-button__text">
    Alert on
</span></button>
        </div>
      </div>
    </div>
  

<!---->
              <div id="SALARY" class="jobs-details__salary-main-rail-card">
<!----><!---->              </div>

                  
    <div class="job-details-how-you-match-card__container mb4 artdeco-card
        " id="how-you-match-card-container">
      <section class="job-details-how-you-match-card__header pt5 ph5 pb4">
<!---->        <h2 id="how-you-match-card-title" tabindex="-1" class="t-20 t-bold">
          How you match
        </h2>
          <div class="display-flex flex-row align-items-center mt4">
              <figure class="ivm-view-attr__system-image ivm-view-attr__system-image--small ivm-view-attr__system-image--illustration-microspot ivm-view-attr__system-image--pencil-ruler-small"></figure>
            <h2 class="job-details-how-you-match-card__header-content fit-content-width text-body-medium">
              3 skills match your profile. Stand out by adding other skills you have.
            </h2>
          </div>
<!---->      </section>

<!---->
          <section class="ph5">
            
    <div>
        <div class="pt5">
          <h3 class="t-16 t-bold">Skills associated with the job post</h3>
          <p class="job-details-how-you-match__skills-section-subheader t-12">Identified by LinkedIn</p>
            
    <div class="job-details-how-you-match__skills-item-wrapper display-flex flex-row pt4">
        <span class="ivm-view-attr__icon--signal-positive job-details-how-you-match__skills-item-icon">
          <svg role="none" aria-hidden="true" xmlns="http://www.w3.org/2000/svg" width="16" height="16" viewBox="0 0 16 16" data-supported-dps="16x16" data-test-icon="signal-success-small">
<!---->    

    <use href="#signal-success-small" width="16" height="16"></use>
</svg>

        </span>
      <div class="display-flex flex-column overflow-hidden">
          <h3 class="t-14 t-bold">3 skills on your profile</h3>
          <a class="app-aware-link  job-details-how-you-match__skills-item-subtitle t-14 overflow-hidden" href="#" data-test-app-aware-link="">Data Analysis, Data Analytics, and Unstructured Data</a>
      </div>
    </div>
  
            
    <div class="job-details-how-you-match__skills-item-wrapper display-flex flex-row pt4">
        <span class="ivm-view-attr__icon--signal-neutral job-details-how-you-match__skills-item-icon">
          <svg role="none" aria-hidden="true" xmlns="http://www.w3.org/2000/svg" width="16" height="16" viewBox="0 0 16 16" data-supported-dps="16x16" data-test-icon="signal-caution-small">
<!---->    

    <use href="#signal-caution-small" width="16" height="16"></use>
</svg>

        </span>
      <div class="display-flex flex-column overflow-hidden">
          <h3 class="t-14 t-bold">7 skills missing on your profile</h3>
          <a class="app-aware-link  job-details-how-you-match__skills-item-subtitle t-14 overflow-hidden" href="#" data-test-app-aware-link="">Carbon, Carbon Accounting, Data Collection, Databases, Datasets, Life Cycle Assessment, and Offsites</a>
      </div>
    </div>
  
<!---->        </div>
        
    <div class="ph4 pv3 mt4 display-flex align-items-center job-details-how-you-match__tip-wrapper--freemium">
<!---->        <div class="display-flex">
          <svg role="none" aria-hidden="true" class="mr2 job-details-how-you-match__tip-icon--freemium" xmlns="http://www.w3.org/2000/svg" width="24" height="24" viewBox="0 0 24 24" data-supported-dps="24x24" data-test-icon="lightbulb-medium">
<!---->    

    <use href="#lightbulb-medium" width="24" height="24"></use>
</svg>

        </div>
        <div class="text-body-small">
          <!---->Add skills you have to your profile to stand out to the employer.<span class="white-space-pre"> </span><a class="app-aware-link " target="_self" href="https://www.linkedin.com/jobs/skill-match?jobId=3768592532" data-test-app-aware-link=""><span><strong><!---->Add skills<!----></strong></span></a>
        </div>
<!---->    </div>
  
        <button id="ember86" class="mv5 t-16 pt1 pb1 artdeco-button artdeco-button--muted artdeco-button--icon-right artdeco-button--1 artdeco-button--secondary ember-view">        <svg role="none" aria-hidden="true" class="artdeco-button__icon  rtl-flip" xmlns="http://www.w3.org/2000/svg" width="16" height="16" viewBox="0 0 16 16" data-supported-dps="16x16" data-test-icon="arrow-right-small">
<!---->    

    <use href="#arrow-right-small" width="16" height="16"></use>
</svg>


<span class="artdeco-button__text">
    Show all skills
</span></button>
    </div>
  
          </section>

<!----><!---->
<!---->
          <section class="job-details-how-you-match-card__upsell pv4 ph5">
              
          

    <div class="align-items-flex-start pb3
              display-flex align-items-center
              flex-column
              ">
      <div class="display-flex flex-column">
<!---->          
<!---->          
        <div>
            <h3 class="t-16 t-bold">
              Achieve your goals faster with Premium
            </h3>

            <p class="t-14 t-black--light mb2">
              Get exclusive access to applicant insights, see jobs where you’d be a top applicant and more
            </p>

            <div class="display-flex align-items-flex-start mb3">
              
    <div class="ivm-image-view-model   ">
        <ul class="ivm-image-view-model__img-list--stacked list-style-none display-flex justify-center">
            <li class="ivm-image-view-model__img-list-item--stacked">
              
    <div class="ivm-view-attr__img-wrapper display-flex">
<!---->          <img width="32" src="https://media.licdn.com/dms/image/C5603AQE4h5-0UwsrOw/profile-displayphoto-shrink_100_100/0/1517793443567?e=1706745600&amp;v=beta&amp;t=vL5XEiLfXpHDRePgYWgkkstxh1ljlMpZs0vCtEGDNPw" loading="lazy" height="32" alt="" id="ember87" class="ivm-view-attr__img--centered ivm-view-attr__img--stacked ivm-view-attr__img--stacked-circle-size-1 EntityPhoto-circle-1-stackedFacepile   evi-image lazy-image ember-view">
    </div>
  
            </li>
            <li class="ivm-image-view-model__img-list-item--stacked">
              
    <div class="ivm-view-attr__img-wrapper display-flex">
<!---->          <img width="32" src="https://media.licdn.com/dms/image/D5603AQGUsLQAhDzKNg/profile-displayphoto-shrink_100_100/0/1690879601692?e=1706745600&amp;v=beta&amp;t=LfZ9rwFQnFkz_2Y3w6sR7UBQU76hSKYvIglEVkxXaYM" loading="lazy" height="32" alt="" id="ember88" class="ivm-view-attr__img--centered ivm-view-attr__img--stacked ivm-view-attr__img--stacked-circle-size-1 EntityPhoto-circle-1-stackedFacepile   evi-image lazy-image ember-view">
    </div>
  
            </li>
            <li class="ivm-image-view-model__img-list-item--stacked">
              
    <div class="ivm-view-attr__img-wrapper display-flex">
<!---->          <img width="32" src="https://media.licdn.com/dms/image/C5603AQH8wcMQgoBQ1A/profile-displayphoto-shrink_100_100/0/1588736600325?e=1706745600&amp;v=beta&amp;t=Abw1ANS7DCIw25UEoBzIGt7elFmtBFsodDI8e0c7vW0" loading="lazy" height="32" alt="" id="ember89" class="ivm-view-attr__img--centered ivm-view-attr__img--stacked ivm-view-attr__img--stacked-circle-size-1 EntityPhoto-circle-1-stackedFacepile   evi-image lazy-image ember-view">
    </div>
  
            </li>
        </ul>

<!---->    </div>
  
              <p class="t-12 t-black--light ml2 align-self-center">
                <!---->Crystine PL and millions of other members use Premium<!---->
              </p>
            </div>
        </div>
      </div>
<!---->        <a class="app-aware-link  artdeco-button artdeco-button--premium artdeco-button--secondary " href="https://www.linkedin.com/premium/products/?upsellOrderOrigin=premium_job_details_how_you_match_upsell&amp;utype=job&amp;referenceId=rmtcapsfTIW2Dx4aX44fvA%3D%3D&amp;destRedirectURL=https%3A%2F%2Fwww.linkedin.com%2Fjobs%2Fview%2F3768592532%2F%3FeBP%3DJOB_SEARCH_ORGANIC%26refId%3DzJUYZKcKNK0KckLEpIRG6g%253D%253D%26trackingId%3DLwVF686xVu9u%252FR6wEjnRaQ%253D%253D%26trk%3Dflagship3_search_srp_jobs" data-test-app-aware-link="">
          Try Premium for free
        </a>
        <p class="t-12 card-upsell__subtext">
          1-month free trial. Cancel anytime.
        </p>
    </div>
  
  
          </section>
    </div>
  


                    
    <section id="ember71" class="artdeco-card ember-view jobs-box jobs-details__smp-promo artdeco-card"><!---->

        <div class="display-flex pt2">
          <div class="smp-promos-card__icon"></div>
          <div>
            <h2 class="smp-promos-card__title text-heading-large">
              
                        <span dir="ltr"><span aria-hidden="true"><!---->Put your best foot forward with your application<!----></span><span class="visually-hidden"><!---->Put your best foot forward with your application<!----></span></span>
                      
            </h2>
            <p class="smp-promos-card__subtitle text-body-small">
              
                        <span dir="ltr"><span aria-hidden="true"><!---->Hire a resume writer<!----></span><span class="visually-hidden"><!---->Hire a resume writer<!----></span></span>
                      
            </p>
          </div>
        </div>
      <div class="smp-promos-card__button--jdp">
          <div data-view-name="marketplace-jobs-distribution-platform-promo-cta">
            <a class="app-aware-link  full-width artdeco-button artdeco-button--2 artdeco-button--secondary" target="_self" href="https://www.linkedin.com/service-marketplace/create-project/1005?trk=profinder_neptune_jobdetails_rotation_with_premium" data-test-app-aware-link="">
              <span class="artdeco-button__text">
                
                        Get a resume review
                      
              </span>
            </a>
          </div>
      </div>
    </section>
  
<!---->
                <section id="ember72" class="artdeco-card ember-view"><!---->

                  
    <section class="jobs-company jobs-box--fadein mb4">
      <div class="jobs-company__box">
        <h2 class="text-heading-large">
          About the company
        </h2>

        <div class="display-flex align-items-center mt5">
          <div id="ember73" class="artdeco-entity-lockup artdeco-entity-lockup--size-5 ember-view flex-grow-1">
            <div id="ember74" class="artdeco-entity-lockup__image artdeco-entity-lockup__image--type-square ember-view" type="square">
              <a href="/company/cozero-io/life/" id="ember75" class="ember-view link-without-hover-state inline-block">
                <img title="Cozero" src="https://media.licdn.com/dms/image/C4D0BAQGkpvwIXWu1-g/company-logo_100_100/0/1638439865071/cozero_io_logo?e=1709164800&amp;v=beta&amp;t=sVPH2ggrA8QHrpoTseVt0_N71x9Hg_xcONvZl48_Ogk" alt="Cozero company logo" id="ember76" class="evi-image ember-view">
              </a>
            
</div>
            <div id="ember77" class="artdeco-entity-lockup__content ember-view flex-grow-1">
              <div id="ember78" class="artdeco-entity-lockup__title ember-view t-20">
                <a href="/company/cozero-io/life/" id="ember79" class="ember-view link-without-visited-state inline-block t-black">
                  Cozero
                </a>
              
</div>
              <div id="ember80" class="artdeco-entity-lockup__subtitle ember-view t-16">
                5,298 followers
              </div>
            </div>
          
</div>
          <button class="follow   artdeco-button artdeco-button--secondary
              
              ml5" aria-label="Follow" aria-live="polite" type="button">
      <svg role="none" aria-hidden="true" class="artdeco-button__icon" xmlns="http://www.w3.org/2000/svg" width="16" height="16" viewBox="0 0 16 16" data-supported-dps="16x16" data-test-icon="add-small">
<!---->    

    <use href="#add-small" width="16" height="16"></use>
</svg>

    <span aria-hidden="true">Follow</span>
</button>
        </div>

        <div class="t-14 mt5">
          Software Development
            <span class="jobs-company__inline-information">
              11-50 employees
            </span>
            <span class="jobs-company__inline-information">
              35 on LinkedIn
            </span>
        </div>

        <p class="jobs-company__company-description text-body-small-open">
          
    <div class="inline-show-more-text
        inline-show-more-text--is-collapsed
        inline-show-more-text--is-collapsed-with-line-clamp
        
        
        " style="-webkit-line-clamp:3;" tabindex="-1">

        Take control of your corporate emission data with Cozero's digital Climate Action Platform (CAP). Cozero tracks, optimizes, and engages on your carbon performance. Forget endless excel files &amp; project sounding boards. Cozero digitally unites all carbon management data in one end-to-end platform, so that you can easily keep track - and act. Customize your climate action platform for your industry and company. Get smart on your carbon emissions and take the right decisions. <br><br>Cozero was founded in 2020 with the purpose to enable the corporate sector in a transformational shift to a lower emission economy. We strongly believe that the private sector has the potential to make this transformation possible and to fight climate change effectively. <br><br>Whoever you are - be it a business leader, a climate action enthusiast, a potential low carbon service or technology partner or a motivated individual who wants to join our mission - please reach out and get on board: info@cozero.io
      
          <span class="inline-show-more-text__link-container-collapsed">
              <span>…</span>
            <button class="inline-show-more-text__button
                inline-show-more-text__button--light
                link" aria-expanded="false" role="button" type="button">
              show more
            </button>
          </span>

<!---->    </div>
  
        </p>

<!---->
<!---->
<!---->      </div>

<!---->
        <div class="jobs-company__footer text-align-center">
          <a href="/company/cozero-io/life/" id="ember81" class="ember-view link-without-hover-state block pv3" aria-label="Show more about the company">
            Show more
          </a>
        </div>
    </section>
  
                </section>

<!---->
<!---->
<!---->                    </div>

<!---->      </div>

<!---->
      
    <div id="ember39" class="ember-view"><!----></div>

    <div id="ember40" class="ember-view"><div id="ember41" class="ember-view"><!----></div></div>
  

      
  <div id="ember42" class="ember-view"><!----></div>


<!---->
<!---->
<!---->
      
    <div>
        
    <div id="ember43" class="ember-view"><!----></div>
  
    </div>
  

<!---->
<!---->    </div>
  
        
          
</main>

          <aside class="scaffold-layout__aside
    
    ">
  
            
          <div class="pt4">
              <div class="mb4">
                  
          

    <div class="artdeco-card justify-center p4
              align-items-left
              display-flex flex-column
              
              ">
      <div class="display-flex flex-column">
<!---->          
<!---->          
        <div class="mb3">
            <h3 class="text-heading-medium">
              Get ahead with Premium Career
            </h3>

            <p class="mt2 t-14">
              Contact recruiters directly, see who's viewed your profile, stand out as a top applicant, and more.
            </p>

            <div class="display-flex align-items-center mt2">
              
    <div class="ivm-image-view-model   ">
        <ul class="ivm-image-view-model__img-list--stacked list-style-none display-flex justify-center">
            <li class="ivm-image-view-model__img-list-item--stacked">
              
    <div class="ivm-view-attr__img-wrapper display-flex">
<!---->          <img width="32" src="https://media.licdn.com/dms/image/C5603AQE4h5-0UwsrOw/profile-displayphoto-shrink_100_100/0/1517793443567?e=1706745600&amp;v=beta&amp;t=vL5XEiLfXpHDRePgYWgkkstxh1ljlMpZs0vCtEGDNPw" loading="lazy" height="32" alt="" id="ember83" class="ivm-view-attr__img--centered ivm-view-attr__img--stacked ivm-view-attr__img--stacked-circle-size-1 EntityPhoto-circle-1-stackedFacepile   evi-image lazy-image ember-view">
    </div>
  
            </li>
            <li class="ivm-image-view-model__img-list-item--stacked">
              
    <div class="ivm-view-attr__img-wrapper display-flex">
<!---->          <img width="32" src="https://media.licdn.com/dms/image/D5603AQGUsLQAhDzKNg/profile-displayphoto-shrink_100_100/0/1690879601692?e=1706745600&amp;v=beta&amp;t=LfZ9rwFQnFkz_2Y3w6sR7UBQU76hSKYvIglEVkxXaYM" loading="lazy" height="32" alt="" id="ember84" class="ivm-view-attr__img--centered ivm-view-attr__img--stacked ivm-view-attr__img--stacked-circle-size-1 EntityPhoto-circle-1-stackedFacepile   evi-image lazy-image ember-view">
    </div>
  
            </li>
            <li class="ivm-image-view-model__img-list-item--stacked">
              
    <div class="ivm-view-attr__img-wrapper display-flex">
<!---->          <img width="32" src="https://media.licdn.com/dms/image/C5603AQH8wcMQgoBQ1A/profile-displayphoto-shrink_100_100/0/1588736600325?e=1706745600&amp;v=beta&amp;t=Abw1ANS7DCIw25UEoBzIGt7elFmtBFsodDI8e0c7vW0" loading="lazy" height="32" alt="" id="ember85" class="ivm-view-attr__img--centered ivm-view-attr__img--stacked ivm-view-attr__img--stacked-circle-size-1 EntityPhoto-circle-1-stackedFacepile   evi-image lazy-image ember-view">
    </div>
  
            </li>
        </ul>

<!---->    </div>
  
              <p class="t-12 t-black--light ml2">
                <!---->Crystine PL and millions of other members use Premium<!---->
              </p>
            </div>
        </div>
      </div>
<!---->        <a class="app-aware-link  artdeco-button artdeco-button--premium artdeco-button--secondary fit-content-width" href="https://www.linkedin.com/premium/products/?upsellOrderOrigin=job_right_rail_upsell&amp;utype=job&amp;referenceId=UZX%2BvE5RREmdRVUCFTO1Qw%3D%3D&amp;destRedirectURL=https%3A%2F%2Fwww.linkedin.com%2Fjobs%2Fview%2F3768592532%2F%3FeBP%3DJOB_SEARCH_ORGANIC%26refId%3DzJUYZKcKNK0KckLEpIRG6g%253D%253D%26trackingId%3DLwVF686xVu9u%252FR6wEjnRaQ%253D%253D%26trk%3Dflagship3_search_srp_jobs" data-test-app-aware-link="">
          Try Premium for free
        </a>
<!---->    </div>
  
  
              </div>

              
  <div class="jobs-post-job artdeco-card jobs-box--inset">
    <p class="jobs-box__body jobs-post-job__title">
      Looking for talent?
    </p>

    <a tabindex="0" rel="noopener noreferrer" target="_blank" href="https://www.linkedin.com/talent/job-posting-redirect?trk=flagship3_job_detail" id="ember57" class="jobs-box__link jobs-post-job__link artdeco-button artdeco-button--secondary ember-view">
      Post a job
    </a>
  </div>

          </div>
        
          
</aside>
      </div>

<!---->  </div>
</div>
  


        <div id="ember44" class="ember-view jobs-box--generic-occludable-area-small  ">
<!---->        </div>
  
</div>

<div class="application-outlet__overlay-container">
    <!----><aside id="msg-overlay" class="msg-overlay-container msg-overlay-container-reflow
    ">
  
    <div tabindex="-1" class="msg-overlay-list-bubble
        msg-overlay-list-bubble--is-minimized
        
        ml4">
      <header class="msg-overlay-bubble-header">
        <div class="msg-overlay-bubble-header__badge-container
            "></div>
        
<!---->  
        <div class="msg-overlay-bubble-header__details flex-row align-items-center ml1">
          
    <div class="presence-entity presence-entity--size-1">
      <img src="https://media.licdn.com/dms/image/C5603AQEkRciOp6B6dA/profile-displayphoto-shrink_100_100/0/1605279012830?e=1706745600&amp;v=beta&amp;t=GoGDD2DE3nGXji1ktdhEWgWX0uNRNd93omIm9L4PTBM" loading="lazy" alt="Naim Iskandar Zahari" id="ember51" class="presence-entity__image EntityPhoto-circle-1  evi-image lazy-image ember-view">

        

    <div class="presence-entity__indicator presence-entity__indicator--size-1
             presence-indicator
        presence-indicator--is-online
        presence-indicator--size-1">
      <span class="visually-hidden">
          Status is online
      </span>
    </div>
  
    </div>
  
          <button class="msg-overlay-bubble-header__button truncate ml2" type="button">
            <span class="truncate t-14 t-bold
                t-black">
              <span aria-hidden="true">
                Messaging
              </span>
              <span class="visually-hidden">
                You are on the messaging overlay. Press enter to open the list of conversations.
              </span>
            </span>
          </button>

<!----><!---->        </div>
        <div class="msg-overlay-bubble-header__controls display-flex">
          <div id="ember52" class="artdeco-dropdown artdeco-dropdown--placement-top artdeco-dropdown--justification-right ember-view">
            <button aria-expanded="false" id="ember53" class="msg-overlay-bubble-header__dropdown-trigger artdeco-button artdeco-button--1 artdeco-button--circle artdeco-button--muted artdeco-button--tertiary artdeco-dropdown__trigger artdeco-dropdown__trigger--placement-top ember-view" type="button" tabindex="0">
              <svg role="img" aria-hidden="false" aria-label="Open messenger dropdown menu" xmlns="http://www.w3.org/2000/svg" width="16" height="16" viewBox="0 0 16 16" data-supported-dps="16x16" data-test-icon="overflow-web-ios-small">
<!---->    

    <use href="#overflow-web-ios-small" width="16" height="16"></use>
</svg>

            
<!----></button>
            <div class="msg-overlay-bubble-header__dropdown-container">
              <div tabindex="-1" aria-hidden="true" id="ember54" class="artdeco-dropdown__content artdeco-dropdown--is-dropdown-element artdeco-dropdown__content--has-arrow artdeco-dropdown__content--arrow-right artdeco-dropdown__content--justification-right artdeco-dropdown__content--placement-top ember-view"><!----></div>
            </div>
          </div>

          <button id="ember55" class="msg-overlay-bubble-header__control msg-overlay-bubble-header__control--new-convo-btn artdeco-button artdeco-button--circle artdeco-button--muted artdeco-button--1 artdeco-button--tertiary ember-view">        <svg role="none" aria-hidden="true" class="artdeco-button__icon " xmlns="http://www.w3.org/2000/svg" width="16" height="16" viewBox="0 0 16 16" data-supported-dps="16x16" data-test-icon="compose-small">
<!---->    

    <use href="#compose-small" width="16" height="16"></use>
</svg>


<span class="artdeco-button__text">
    Compose message
</span></button>

          <button id="ember56" class="msg-overlay-bubble-header__control msg-overlay-bubble-header__control--new-convo-btn artdeco-button artdeco-button--circle artdeco-button--muted artdeco-button--1 artdeco-button--tertiary ember-view">        <svg role="none" aria-hidden="true" class="artdeco-button__icon " xmlns="http://www.w3.org/2000/svg" width="16" height="16" viewBox="0 0 16 16" data-supported-dps="16x16" data-test-icon="chevron-up-small">
<!---->    

    <use href="#chevron-up-small" width="16" height="16"></use>
</svg>


<span class="artdeco-button__text">
    You are on the messaging overlay. Press enter to open the list of conversations.
</span></button>
        </div>
      </header>

      
<!---->  

<!---->
<!----><!---->    </div>
  

<!---->
<!---->
  <div id="msg-overlay__emoji-hoverable-outlet"></div>

  <div id="msg-overlay__reactor-list-outlet"></div>
</aside>

      
    <aside class="coach-container
        coach-container--overlay
        
        coach-container--standalone-page" id="coach-container" tabindex="-1">
<!---->    </aside>
  
<!----></div>

<!---->
  
      
        
    <div id="ember48" class="ember-view"><!----></div>
  
    
  

  
      
    <img src="https://px.ads.linkedin.com/collect/?pid=6883&amp;fmt=gif&amp;_t=1701030053004" alt="" role="none" class="third-party-tracking-pixel hidden" data-test-third-party-tracking-pixel="">
  
  

</div>

<div id="artdeco-hoverable-outlet"></div>

<!---->
<div id="type-ahead-wormhole" class="type-ahead type-ahead-wormhole" aria-hidden="true">
</div>

<div id="toast-wormhole" class="toast-wormhole" aria-hidden="true">
</div>

<div id="profile-inline-modal-outlet" aria-hidden="true">
</div>

  <iframe title="cedex beacon iframe" sandbox="allow-scripts" style="display:none;" src="//radar.cedexis.com/1/11326/radar.html" aria-hidden="true">
  </iframe>


  <div aria-hidden="true"><!----></div>
<code style="display: none" id="clientPageInstance" aria-hidden="true">
  urn:li:page:d_flagship3_job_details;4deffb7b-0190-47ed-9b38-6a728e1de036
</code>

<img src="data:image/gif;base64,R0lGODlhAQABAIAAAAAAAP///yH5BAEAAAAALAAAAAABAAEAAAIBRAA7" style="display: none" class="terminatorlet" aria-hidden="true">



    <div id="a11y-notification" class="visually-hidden" role="region" aria-live="polite"></div>
  

</body>
"""

In [24]:
# <ul class="job-details-skill-match-status-list">

soup = bs(html_skills_matched_html)
# id: bpr-guid-15912216
# id_key = 'bpr-guid-15912216'
# long_text = soup.find(id = id_key).text

# json_dict = json.loads(long_text, strict = False)
# return json_dict['data']['description']['text']
[li.find('div').get_text().strip() for li in soup.find('ul', {'class': 'job-details-skill-match-status-list'}).find_all('li')]
    # print(li.find('div').get_text().strip())

['Data Analysis',
 'Data Analytics',
 'Unstructured Data',
 'Carbon',
 'Carbon Accounting',
 'Data Collection',
 'Databases',
 'Datasets',
 'Life Cycle Assessment',
 'Offsites']

In [19]:
soup = bs(html_skills_matched_html)
# # id: bpr-guid-15912216
id_key = 'bpr-guid-15912216'
long_text = soup.find(id = id_key).text

json_dict = json.loads(long_text, strict = False)
json_dict['data']['jobPostingId']

# GetJobPostingID(html_skills_matched_html)

3768592532

In [ ]:
jobPostingId